In [70]:
learning_rate_x = 0.0001
learning_rate_y = 0.0001
training_epochs = 10000
batch_size = 100
test_batch_size = 136

In [71]:
train_dataset_x = tf.data.Dataset.from_tensor_slices((x_train, y_train_x)).shuffle(buffer_size=100000).batch(batch_size)
train_dataset_y = tf.data.Dataset.from_tensor_slices((x_train, y_train_y)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset_x = tf.data.Dataset.from_tensor_slices((x_test, y_test_x)).batch(test_batch_size)
test_dataset_y = tf.data.Dataset.from_tensor_slices((x_test, y_test_y)).batch(test_batch_size)

In [72]:
def create_model_x():
    model=keras.Sequential() 
    model.add(keras.layers.Conv2D(filters=1024, kernel_size=(1,4),strides=[1,4],kernel_regularizer=tf.keras.regularizers.l1(0.01), input_shape=(4,4,1)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    return model

def create_model_y():
    model=keras.Sequential() 
    model.add(keras.layers.Conv2D(filters=1024, kernel_size=(1,4),strides=[1,4],kernel_regularizer=tf.keras.regularizers.l1(0.01), input_shape=(4,4,1)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    return model

In [73]:
model_x=create_model_x()
model_y=create_model_y()
model_y.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 4, 1, 1024)        5120      
_________________________________________________________________
flatten_5 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              4195328   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 1025      
Total params: 4,201,473
Trainable params: 4,201,473
Non-trainable params: 0
_________________________________________________________________


In [74]:
def loss_fn_x(model_x, images_x, labels_x):
#    w1x=sum(sum(sum(sum(tf.square(model_x.get_weights()[0])))))
#    w2x=sum(tf.square(model_x.get_weights()[1]))
#    w3x=sum(sum(tf.square(model_x.get_weights()[2])))
#    w4x=sum(tf.square(model_x.get_weights()[3]))
#    w5x=sum(sum(tf.square(model_x.get_weights()[4])))
#    w6x=sum(tf.square(model_x.get_weights()[5]))
#     w1x=sum(sum(sum(sum(tf.square(model_x.get_weights()[0])))))
#     w2x=sum(tf.square(model_x.get_weights()[1]))
#     w3x=sum(sum(tf.square(model_x.get_weights()[2])))
#     w4x=sum(tf.square(model_x.get_weights()[3]))
#     w5x=sum(sum(tf.square(model_x.get_weights()[4])))
#     w6x=sum(tf.square(model_x.get_weights()[5]))
#+0.001*(w1x+w2x+w3x+w4x+w5x+w6x)
    logits_x = model_x(images_x, training=True)
    loss_x = tf.reduce_mean((tf.square(logits_x-labels_x)))
    return loss_x
def loss_fn_y(model_y, images_y, labels_y):
#    w1y=sum(sum(sum(sum(tf.square(model_y.get_weights()[0])))))
#    w2y=sum(tf.square(model_y.get_weights()[1]))
#    w3y=sum(sum(tf.square(model_y.get_weights()[2])))
#    w4y=sum(tf.square(model_y.get_weights()[3]))
#    w5y=sum(sum(tf.square(model_y.get_weights()[4])))
#    w6y=sum(tf.square(model_y.get_weights()[5]))
#     w1y=sum(sum(sum(sum(tf.square(model_y.get_weights()[0])))))
#     w2y=sum(tf.square(model_y.get_weights()[1]))
#     w3y=sum(sum(tf.square(model_y.get_weights()[2])))
#     w4y=sum(tf.square(model_y.get_weights()[3]))
#     w5y=sum(sum(tf.square(model_y.get_weights()[4])))
#     w6y=sum(tf.square(model_y.get_weights()[5]))
#+0.001*(w1y+w2y+w3y+w4y+w5y+w6y)
    logits_y = model_y(images_y, training=True)
    loss_y = tf.reduce_mean((tf.square(logits_y-labels_y)))
    return loss_y

def grad_x(model_x, images_x, labels_x):
    with tf.GradientTape() as tape:
        loss_x = loss_fn_x(model_x, images_x, labels_x)
    return tape.gradient(loss_x, model_x.variables)
def grad_y(model_y, images_y, labels_y):
    with tf.GradientTape() as tape:
        loss_y = loss_fn_y(model_y, images_y, labels_y)
    return tape.gradient(loss_y, model_y.variables)

In [75]:
optimizer_x = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate_x)
optimizer_y = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate_y)
def evaluate_x(model_x, images_x, labels_x):
    logits_x=model_x(images_x, training=False)
    correct_prediction_x=tf.reduce_sum(abs(labels_x-logits_x))*28.9/len(labels_x)
    accuracy_x=abs(correct_prediction_x)
    return accuracy_x
def evaluate_y(model_y, images_y, labels_y):
    logits_y=model_y(images_y, training=False)
    correct_prediction_y=tf.reduce_sum(abs(labels_y-logits_y))*16.2/len(labels_y)
    accuracy_y=abs(correct_prediction_y)
    return accuracy_y

In [76]:
avg_test_acc_x_ref=1.
avg_test_acc_y_ref=1.
avg_train_acc_x_ref=1.
avg_train_acc_y_ref=1.
stop_rate_x=0.15
stop_rate_y=0.2
for epoch in range(training_epochs):
    avg_loss_x=0.
    avg_loss_y=0.
    avg_train_acc_x=0.
    avg_train_acc_y=0.
    avg_test_acc_x=0.
    avg_test_acc_y=0.
    train_step=0
    test_step=0
    if ((avg_test_acc_x_ref>stop_rate_x)or(avg_train_acc_x_ref>stop_rate_x)):
        for images_x, labels_x in train_dataset_x:
            grads_x = grad_x(model_x, images_x, labels_x)
            optimizer_x.apply_gradients(zip(grads_x, model_x.variables))
    #        loss_x = loss_fn_x(model_x, images_x, labels_x)
    #        avg_loss_x = avg_loss_x+loss_x
            train_step+=1
#    avg_loss_x = avg_loss_x / train_step
    train_step=0
    if ((avg_test_acc_y_ref>stop_rate_y)or(avg_train_acc_y_ref>stop_rate_y)):
        for images_y, labels_y in train_dataset_y:
            grads_y = grad_y(model_y, images_y, labels_y)
            optimizer_y.apply_gradients(zip(grads_y, model_y.variables))
    #        loss_y = loss_fn_y(model_y, images_y, labels_y)
    #        avg_loss_y = avg_loss_y+loss_y
            train_step+=1
#    avg_loss_y = avg_loss_y / train_step
    train_step=0
    
    
    for images_x, labels_x in test_dataset_x:
        acc_x=evaluate_x(model_x, images_x, labels_x)
        avg_test_acc_x=avg_test_acc_x+acc_x
        test_step+=1
    avg_test_acc_x=avg_test_acc_x/test_step
    test_step=0
    for images_x, labels_x in train_dataset_x:
        acc_x=evaluate_x(model_x, images_x, labels_x)
        avg_train_acc_x=avg_train_acc_x+acc_x
        test_step+=1
    avg_train_acc_x=avg_train_acc_x/test_step
    test_step=0
    
    for images_y, labels_y in test_dataset_y:
        acc_y=evaluate_y(model_y, images_y, labels_y)
        avg_test_acc_y=avg_test_acc_y+acc_y
        test_step+=1    
    avg_test_acc_y=avg_test_acc_y/test_step
    test_step=0
    for images_y, labels_y in train_dataset_y:
        acc_y=evaluate_y(model_y, images_y, labels_y)
        avg_train_acc_y=avg_train_acc_y+acc_y
        test_step+=1    
    avg_train_acc_y=avg_train_acc_y/test_step
    

    avg_test_acc_x_ref=avg_test_acc_x
    avg_train_acc_x_ref=avg_train_acc_x
    avg_test_acc_y_ref=avg_test_acc_y
    avg_train_acc_y_ref=avg_train_acc_y
    print('Epoch:', '{}'.format(epoch+1), '|| (avg delta x, avg delta y) ','|| avg train delta = ','(', '{:.4f}'.format(avg_train_acc_x), 'cm, ', '{:.4f}'.format(avg_train_acc_y), 'cm)', '|| avg test delta = ','(', '{:.4f}'.format(avg_test_acc_x), 'cm, ', '{:.4f}'.format(avg_test_acc_y), 'cm)')
    if (avg_test_acc_x_ref<stop_rate_x) and (avg_test_acc_y_ref<stop_rate_y) and (avg_train_acc_x_ref<stop_rate_x) and (avg_train_acc_y_ref<stop_rate_y):
        break
#'|| x_loss =', '{:.12f}'.format(avg_loss_x), '|| y_loss =', '{:.12f}'.format(avg_loss_y),

Epoch: 1 || (avg delta x, avg delta y)  || avg train delta =  ( 6.0546 cm,  1.5851 cm) || avg test delta =  ( 5.8564 cm,  1.5354 cm)
Epoch: 2 || (avg delta x, avg delta y)  || avg train delta =  ( 2.9680 cm,  1.0743 cm) || avg test delta =  ( 2.8211 cm,  0.8433 cm)
Epoch: 3 || (avg delta x, avg delta y)  || avg train delta =  ( 1.5603 cm,  0.9056 cm) || avg test delta =  ( 1.4922 cm,  0.7542 cm)
Epoch: 4 || (avg delta x, avg delta y)  || avg train delta =  ( 0.5872 cm,  0.8583 cm) || avg test delta =  ( 0.5315 cm,  0.6904 cm)
Epoch: 5 || (avg delta x, avg delta y)  || avg train delta =  ( 0.1260 cm,  0.7740 cm) || avg test delta =  ( 0.0954 cm,  0.6228 cm)
Epoch: 6 || (avg delta x, avg delta y)  || avg train delta =  ( 0.1260 cm,  0.6711 cm) || avg test delta =  ( 0.0954 cm,  0.5483 cm)
Epoch: 7 || (avg delta x, avg delta y)  || avg train delta =  ( 0.1260 cm,  0.6102 cm) || avg test delta =  ( 0.0954 cm,  0.4773 cm)
Epoch: 8 || (avg delta x, avg delta y)  || avg train delta =  ( 0.126

In [ ]:
# 17행 31열 (학습이 이루어지지 않은 영역)

In [77]:
test_one=np.array([[[	0.091086428	,	0.360874226	,	0.49478791	,	0.331322881	,],
[	0.268757442	,	0.096542407	,	0.398028818	,	0.371364791	,],
[	0.275943246	,	0.261929296	,	0.101199483	,	0.261007253	,],
[	0.276761246	,	0.407192993	,	0.432672556	,	0.101857965	]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([30/33],dtype=np.float32)
test_label_y=np.array([16/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.8610 cm || delta y = 2.5956 cm


In [78]:
# 5행 7열

In [79]:
test_one=np.array([[[	0.097713769	,	0.256358871	,	0.28632803	,	0.276126352	,],
[	0.370912956	,	0.089508408	,	0.263574981	,	0.391965488	,],
[	0.450002353	,	0.319168028	,	0.08937607	,	0.365187627	,],
[	0.348198827	,	0.351123577	,	0.269076932	,	0.09903081	]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([6/33],dtype=np.float32)
test_label_y=np.array([4/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.1394 cm || delta y = 0.0089 cm


In [80]:
# 6행 8열

In [81]:
test_one=np.array([[[	0.100408922	,	0.26100062	,	0.297618806	,	0.284301018	,],
[	0.361398809	,	0.092653412	,	0.268753774	,	0.393201309	,],
[	0.434453358	,	0.310601542	,	0.083866596	,	0.360374302	,],
[	0.333619969	,	0.343804922	,	0.268984386	,	0.098257107	]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([7/33],dtype=np.float32)
test_label_y=np.array([5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.1357 cm || delta y = 0.0081 cm


In [82]:
# 6.5행 3.5열

In [83]:
test_one=np.array([[[	0.09466807	,	0.231550653	,	0.271861704	,	0.280718676	,],
[	0.366122086	,	0.081478087	,	0.258236617	,	0.39898103	,],
[	0.433695657	,	0.292366753	,	0.075579302	,	0.363684319	,],
[0.329897025	,	0.317065743	,	0.245254944	,	0.100792946	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([2.5/33],dtype=np.float32)
test_label_y=np.array([5.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.0909 cm || delta y = 0.1066 cm


In [84]:
# 3.5행 11.5열

In [85]:
test_one=np.array([[[	0.097637581	,	0.276745107	,	0.290970979	,	0.256396771	,],
[	0.333666823	,	0.084733348	,	0.242173971	,	0.34480141	,],
[	0.413081217	,	0.312479193	,	0.073646506	,	0.321776891	,],
[	0.33293432	,	0.363077597	,	0.271692953	,	0.091212038	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([10.5/33],dtype=np.float32)
test_label_y=np.array([2.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.0192 cm || delta y = 0.1489 cm


In [86]:
# 4.5행 21.5열

In [87]:
test_one=np.array([[[	0.088080767	,	0.316322341	,	0.329592912	,	0.252405077	,],
[	0.275418248	,	0.084438994	,	0.246008016	,	0.303934564	,],
[	0.360480699	,	0.318339428	,	0.074965047	,	0.283362866	,],
[	0.30484062	,	0.392951395	,	0.303958395	,	0.087506458	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([20.5/33],dtype=np.float32)
test_label_y=np.array([3.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.1554 cm || delta y = 0.1274 cm


In [88]:
# 7.5행 32.5열

In [89]:
test_one=np.array([[[	0.086169553	,	0.375018307	,	0.412511154	,	0.274506509	,],
[	0.231997682	,	0.093242947	,	0.291770717	,	0.28705421	,],
[	0.31283136	,	0.330191604	,	0.088837348	,	0.25159492	,],
[	0.29226257	,	0.439605341	,	0.371849016	,	0.089095354	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([31.5/33],dtype=np.float32)
test_label_y=np.array([6.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.9022 cm || delta y = 0.2740 cm


In [90]:
# 6.5행 3.5열 (면적이 1/3로 줄어든 경우)

In [91]:
test_one=np.array([[[	0.059325045	,	0.143705082	,	0.166999287	,	0.177885254	,],
[	0.237884857	,	0.051584033	,	0.164642627	,	0.262596442	,],
[	0.285398722	,	0.191246464	,	0.044898621	,	0.240981837	,],
[	0.207429683	,	0.19946683	,	0.151020675	,	0.066049975	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([2.5/33],dtype=np.float32)
test_label_y=np.array([5.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.1863 cm || delta y = 0.0684 cm


In [92]:
# 3.5행 11.5열 (면적이 1/3로 줄어든 경우)

In [93]:
test_one=np.array([[[	0.062022645	,	0.179257345	,	0.187888374	,	0.168764282	,],
[	0.22805242	,	0.063640955	,	0.164666705	,	0.238109121	,],
[	0.286948039	,	0.216060578	,	0.047005556	,	0.22474898	,],
[	0.229754974	,	0.249657508	,	0.185197833	,	0.070145707	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([10.5/33],dtype=np.float32)
test_label_y=np.array([2.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.2253 cm || delta y = 0.0562 cm


In [94]:
# 4.5행 21.5열 (면적이 1/3로 줄어든 경우)

In [95]:
test_one=np.array([[[	0.057285406	,	0.208765163	,	0.216296308	,	0.169361881	,],
[	0.179434061	,	0.052913	,	0.155934171	,	0.200318242	,],
[	0.238937045	,	0.20907716	,	0.044803552	,	0.189227691	,],
[	0.202735671	,	0.260065864	,	0.198237074	,	0.059149701	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([20.5/33],dtype=np.float32)
test_label_y=np.array([3.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.2618 cm || delta y = 0.0333 cm


In [96]:
# 7.5행 32.5열 (면적이 1/3로 줄어든 경우)

In [97]:
test_one=np.array([[[	0.058624565	,	0.254104939	,	0.27661388	,	0.188231565	,],
[	0.148995755	,	0.058024551	,	0.182144665	,	0.186310561	,],
[	0.203605123	,	0.21268779	,	0.052404284	,	0.164372392	,],
[	0.197241651	,	0.296248784	,	0.246232405	,	0.059918039	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([31.5/33],dtype=np.float32)
test_label_y=np.array([6.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.9388 cm || delta y = 0.3224 cm


In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
model_x.variables

In [ ]:
remove.model_x.variables[1]

In [ ]:
model_x.get_weights()[6].shape

In [ ]:
w1x=sum(sum(sum(sum(tf.square(model_x.get_weights()[0])))))
w2x=sum(tf.square(model_x.get_weights()[1]))
w3x=sum(sum(tf.square(model_x.get_weights()[2])))
w4x=sum(tf.square(model_x.get_weights()[3]))
w5x=sum(sum(tf.square(model_x.get_weights()[4])))
w6x=sum(tf.square(model_x.get_weights()[5]))

In [ ]:
w6x

In [ ]:
avg_test_acc_x=0
acc_x=0
test_step=0
for images_x, labels_x in test_dataset_x:
        acc_x=evaluate_x(model_x, images_x, labels_x)
        avg_test_acc_x=avg_test_acc_x+acc_x
        test_step+=1
avg_test_acc_x=avg_test_acc_x/test_step
test_step=0

In [ ]:
avg_test_acc_x

In [ ]:
avg_test_acc_ref_x

In [ ]:
not 0

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from random import shuffle
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import os

In [3]:
x_test=np.array([[[	0.103593038	,	0.229979747	,	0.246631688	,	0.250122949	,],
[	0.396084817	,	0.085869223	,	0.246406102	,	0.386731828	,],
[	0.472931629	,	0.315588987	,	0.074078901	,	0.36109223	,],
[	0.374013309	,	0.345955676	,	0.250190001	,	0.099851088	,]],
[[	0.102494739	,	0.232398687	,	0.248725446	,	0.251863541	,],
[	0.394785514	,	0.086211917	,	0.245063838	,	0.385345949	,],
[	0.476889202	,	0.319868224	,	0.080802224	,	0.365017424	,],
[	0.372086149	,	0.344562969	,	0.248944989	,	0.095302488	,]],
[[	0.100823929	,	0.228977363	,	0.245514547	,	0.248775864	,],
[	0.399132659	,	0.093658225	,	0.249892467	,	0.389704897	,],
[	0.473248542	,	0.316017475	,	0.07371374	,	0.361595043	,],
[	0.371673732	,	0.343810677	,	0.247893949	,	0.09595473	,]],
[[	0.106994043	,	0.237176789	,	0.253416464	,	0.256574209	,],
[	0.394877363	,	0.085179301	,	0.24505394	,	0.385343018	,],
[	0.474028223	,	0.316697167	,	0.076164996	,	0.362103225	,],
[	0.375349866	,	0.348020445	,	0.252430954	,	0.098381269	,]],
[[	0.100880268	,	0.230172281	,	0.246548252	,	0.24971307	,],
[	0.396041715	,	0.088590778	,	0.246437548	,	0.38651042	,],
[	0.475602116	,	0.318613016	,	0.075933645	,	0.364185229	,],
[	0.371131826	,	0.34344664	,	0.24767752	,	0.094732672	,]],
[[	0.102694872	,	0.22963166	,	0.246173744	,	0.249550565	,],
[	0.396953603	,	0.086788504	,	0.247258264	,	0.387502414	,],
[	0.472899653	,	0.315596634	,	0.073799818	,	0.361001423	,],
[	0.373310652	,	0.345265321	,	0.249423802	,	0.09859376	,]],
[[	0.103601312	,	0.233489964	,	0.249925365	,	0.253061489	,],
[	0.394620418	,	0.085622855	,	0.244803098	,	0.385134211	,],
[	0.476027346	,	0.318837192	,	0.079423816	,	0.36406452	,],
[	0.372927603	,	0.34534324	,	0.249743436	,	0.095862558	,]],
[[	0.100593476	,	0.229071285	,	0.245568318	,	0.248772349	,],
[	0.398235456	,	0.092099499	,	0.248835107	,	0.388644523	,],
[	0.473827164	,	0.316615956	,	0.074132882	,	0.36218462	,],
[	0.371290347	,	0.343427746	,	0.247544923	,	0.095303231	,]],
[[	0.105651856	,	0.231131069	,	0.247687036	,	0.251235559	,],
[	0.395210536	,	0.085268544	,	0.245495278	,	0.385741587	,],
[	0.473577419	,	0.316195205	,	0.075193194	,	0.36159885	,],
[	0.375819946	,	0.347562836	,	0.251744612	,	0.102179447	,]],
[[	0.101517085	,	0.230987051	,	0.247421924	,	0.250538109	,],
[	0.395517615	,	0.087480206	,	0.245782667	,	0.385919756	,],
[	0.476717449	,	0.31974521	,	0.077056624	,	0.365303165	,],
[	0.371395121	,	0.343800343	,	0.248022991	,	0.094632246	,]],
[[	0.101797064	,	0.229286975	,	0.245821256	,	0.249124928	,],
[	0.397936239	,	0.087719214	,	0.248325554	,	0.388504611	,],
[	0.472910541	,	0.315524895	,	0.073522193	,	0.361013251	,],
[	0.372561993	,	0.344608368	,	0.24861656	,	0.097362221	,]],
[[	0.10496382	,	0.234975689	,	0.251290979	,	0.254330969	,],
[	0.394681274	,	0.085226814	,	0.244768966	,	0.385107107	,],
[	0.475130206	,	0.317960233	,	0.077930519	,	0.363165405	,],
[	0.373858019	,	0.346315558	,	0.250598857	,	0.096818861	,]],
[[	0.101208551	,	0.232233059	,	0.248640629	,	0.247617542	,],
[	0.388812737	,	0.092386716	,	0.24633396	,	0.381477507	,],
[	0.463816764	,	0.314637581	,	0.074444038	,	0.355699371	,],
[	0.366980244	,	0.344780811	,	0.2502526	,	0.098853293	,]],
[[	0.101106899	,	0.235304113	,	0.251435349	,	0.250131963	,],
[	0.385182441	,	0.084827592	,	0.242362257	,	0.377907365	,],
[	0.463278022	,	0.313949894	,	0.075160539	,	0.354876082	,],
[	0.365115077	,	0.34345268	,	0.248910726	,	0.094265595	,]],
[[	0.098754338	,	0.231426439	,	0.247638276	,	0.24652253	,],
[	0.385963244	,	0.087667493	,	0.243180954	,	0.378674885	,],
[	0.466800119	,	0.317869762	,	0.077459555	,	0.358988314	,],
[	0.364496299	,	0.342486609	,	0.247876731	,	0.094931486	,]],
[[	0.104556154	,	0.234204293	,	0.250460244	,	0.249569475	,],
[	0.38764252	,	0.086968424	,	0.24495969	,	0.38045475	,],
[	0.462970795	,	0.313638917	,	0.073732365	,	0.354789501	,],
[	0.368371296	,	0.346745642	,	0.252413173	,	0.097334225	,]],
[[	0.099178715	,	0.232908704	,	0.249052832	,	0.247774126	,],
[	0.384738752	,	0.085099655	,	0.241885818	,	0.377451882	,],
[	0.464781502	,	0.315518647	,	0.077690377	,	0.356216078	,],
[	0.36410608	,	0.342329935	,	0.247746932	,	0.093581035	,]],
[[	0.100332818	,	0.231858692	,	0.248103711	,	0.247018574	,],
[	0.387926346	,	0.090897238	,	0.2453405	,	0.380450727	,],
[	0.464528789	,	0.315361911	,	0.075067061	,	0.356481391	,],
[	0.366169502	,	0.344029061	,	0.249442495	,	0.097394571	,]],
[[	0.102276126	,	0.236630332	,	0.252675892	,	0.251388042	,],
[	0.385738355	,	0.085230208	,	0.242921905	,	0.37838576	,],
[	0.463111673	,	0.31366935	,	0.074430814	,	0.354614088	,],
[	0.365905643	,	0.344184845	,	0.249696526	,	0.094803135	,]],
[[	0.098644289	,	0.231682951	,	0.247794681	,	0.246645046	,],
[	0.38539958	,	0.08677704	,	0.242610707	,	0.37808056	,],
[	0.467004245	,	0.31796567	,	0.081212653	,	0.358447915	,],
[	0.364302677	,	0.342308898	,	0.247714021	,	0.094285975	,]],
[[	0.103195677	,	0.233322577	,	0.249662615	,	0.248659153	,],
[	0.388711169	,	0.088013661	,	0.246032817	,	0.381376142	,],
[	0.463320811	,	0.314024467	,	0.073910764	,	0.354950811	,],
[	0.368613489	,	0.346304046	,	0.251772056	,	0.101113917	,]],
[[	0.099802959	,	0.233800728	,	0.249819028	,	0.24861762	,],
[	0.384893466	,	0.084910683	,	0.241976279	,	0.377581433	,],
[	0.464194931	,	0.314858917	,	0.076473895	,	0.355592013	,],
[	0.36445649	,	0.342598657	,	0.248034563	,	0.093659928	,]],
[[	0.09967195	,	0.231540618	,	0.247687339	,	0.246563776	,],
[	0.387166238	,	0.089635057	,	0.244483728	,	0.379618922	,],
[	0.46543699	,	0.316307557	,	0.075941929	,	0.357354981	,],
[	0.365505436	,	0.343445209	,	0.248758326	,	0.096326937	,]],
[[	0.103686458	,	0.238048083	,	0.254051359	,	0.252786025	,],
[	0.38639101	,	0.08581429	,	0.243701818	,	0.379044789	,],
[	0.463038217	,	0.313532834	,	0.07407416	,	0.354491396	,],
[	0.366823899	,	0.345135541	,	0.2507123	,	0.095713042	,]],
[[	0.10267953	,	0.238106904	,	0.253476189	,	0.248369337	,],
[	0.380213771	,	0.091934131	,	0.244764327	,	0.375438393	,],
[	0.456180196	,	0.313808881	,	0.073719791	,	0.350138939	,],
[	0.35847181	,	0.344124537	,	0.249282741	,	0.093111868	,]],
[[	0.097604259	,	0.236903693	,	0.252183483	,	0.246681088	,],
[	0.376376661	,	0.08402048	,	0.240478138	,	0.371650561	,],
[	0.455203563	,	0.312522875	,	0.07382837	,	0.348772094	,],
[	0.362323709	,	0.347632464	,	0.252767583	,	0.099237442	,]],
[[	0.098687955	,	0.235857489	,	0.251213811	,	0.245818823	,],
[	0.377285609	,	0.087092743	,	0.241442205	,	0.372565923	,],
[	0.459501861	,	0.31733123	,	0.07725124	,	0.353727665	,],
[	0.359052748	,	0.344982747	,	0.250106436	,	0.093160736	,]],
[[	0.100861981	,	0.240780884	,	0.255853681	,	0.250318684	,],
[	0.378681951	,	0.085995439	,	0.242947147	,	0.373995651	,],
[	0.454971123	,	0.312475744	,	0.072762751	,	0.348797925	,],
[	0.359516611	,	0.345054873	,	0.250132555	,	0.094947267	,]],
[[	0.096983275	,	0.23575903	,	0.250930541	,	0.245516638	,],
[	0.37596537	,	0.084353061	,	0.240153241	,	0.371285099	,],
[	0.456541004	,	0.313981263	,	0.076214218	,	0.350044135	,],
[	0.361784164	,	0.347689923	,	0.25297906	,	0.095546271	,]],
[[	0.101377544	,	0.237382341	,	0.252690472	,	0.247487328	,],
[	0.37933537	,	0.090418773	,	0.243732593	,	0.374586897	,],
[	0.457014981	,	0.314661316	,	0.074540005	,	0.351051081	,],
[	0.358512776	,	0.344240819	,	0.249329177	,	0.092871693	,]],
[[	0.09839797	,	0.237954532	,	0.253099842	,	0.24757694	,],
[	0.376924885	,	0.084324917	,	0.240926669	,	0.372170145	,],
[	0.454932205	,	0.312408955	,	0.073216557	,	0.348513018	,],
[	0.361361186	,	0.346818176	,	0.251871953	,	0.097859175	,]],
[[	0.097898777	,	0.23556749	,	0.250834994	,	0.245466878	,],
[	0.376741004	,	0.085989904	,	0.240810655	,	0.37190939	,],
[	0.458643983	,	0.316249064	,	0.07958348	,	0.352190746	,],
[	0.359681272	,	0.345460496	,	0.250745639	,	0.09346192	,]],
[[	0.102239455	,	0.242190809	,	0.257163637	,	0.251657976	,],
[	0.37987669	,	0.087017248	,	0.244116777	,	0.375120247	,],
[	0.455478254	,	0.312890931	,	0.073086692	,	0.349287251	,],
[	0.358997759	,	0.344546646	,	0.249606104	,	0.094059507	,]],
[[	0.097190503	,	0.23622709	,	0.251330829	,	0.245881347	,],
[	0.376019866	,	0.084015816	,	0.240102561	,	0.371291717	,],
[	0.455838032	,	0.313277938	,	0.075087196	,	0.349418024	,],
[	0.363083099	,	0.348991534	,	0.254315831	,	0.09674995	,]],
[[	0.100262086	,	0.236737746	,	0.252003416	,	0.246685004	,],
[	0.378382886	,	0.088925723	,	0.242731214	,	0.373535833	,],
[	0.457961256	,	0.315608839	,	0.075472036	,	0.351944531	,],
[	0.358751207	,	0.344418277	,	0.249578677	,	0.092806194	,]],
[[	0.099317998	,	0.239084078	,	0.254145089	,	0.248619808	,],
[	0.377560592	,	0.084832078	,	0.241667128	,	0.372789778	,],
[	0.454909668	,	0.312285808	,	0.07293414	,	0.348525626	,],
[	0.360615987	,	0.345993051	,	0.251017211	,	0.096576703	,]],
[[	0.101173904	,	0.243345417	,	0.255897302	,	0.246764142	,],
[	0.3720467	,	0.088997263	,	0.240739161	,	0.36817208	,],
[	0.451602274	,	0.315605632	,	0.074219819	,	0.347073558	,],
[	0.354889248	,	0.347441969	,	0.251548665	,	0.093045237	,]],
[[	0.097378333	,	0.243172032	,	0.255792263	,	0.246261698	,],
[	0.371442298	,	0.085315719	,	0.239976925	,	0.367667031	,],
[	0.448987015	,	0.312699874	,	0.072181801	,	0.344113025	,],
[	0.359665046	,	0.351855353	,	0.255946645	,	0.100189212	,]],
[[	0.097466873	,	0.241323098	,	0.253914729	,	0.244570768	,],
[	0.369954521	,	0.085339877	,	0.238591537	,	0.366139823	,],
[	0.452378587	,	0.316335973	,	0.078107279	,	0.34729911	,],
[	0.355033535	,	0.347561101	,	0.251768665	,	0.092294641	,]],
[[	0.101194008	,	0.247320254	,	0.259787043	,	0.250243476	,],
[	0.374385129	,	0.092873172	,	0.243407218	,	0.370478706	,],
[	0.449797521	,	0.313570439	,	0.072374796	,	0.345077124	,],
[	0.356333363	,	0.348630191	,	0.252760265	,	0.095261743	,]],
[[	0.096553861	,	0.241630551	,	0.254248978	,	0.244746106	,],
[	0.370061968	,	0.084296081	,	0.23859327	,	0.366168857	,],
[	0.449787745	,	0.313448183	,	0.073728656	,	0.344723961	,],
[	0.357236767	,	0.349971316	,	0.254280136	,	0.094276711	,]],
[[	0.099964847	,	0.242582041	,	0.25516997	,	0.245937876	,],
[	0.371209606	,	0.087809584	,	0.239960934	,	0.367347267	,],
[	0.452772007	,	0.316680374	,	0.075337413	,	0.348174595	,],
[	0.354805481	,	0.347275973	,	0.251323415	,	0.092620891	,]],
[[	0.098351787	,	0.244259958	,	0.256670062	,	0.247199955	,],
[	0.372392577	,	0.08609688	,	0.240919657	,	0.368473258	,],
[	0.449154991	,	0.31278653	,	0.072010171	,	0.34416753	,],
[	0.358717168	,	0.350815232	,	0.254912075	,	0.098633891	,]],
[[	0.096938205	,	0.241262808	,	0.253764414	,	0.244315809	,],
[	0.36986494	,	0.084850802	,	0.238414187	,	0.365966119	,],
[	0.451596085	,	0.315366367	,	0.076586463	,	0.346409502	,],
[	0.355377404	,	0.348058227	,	0.252282777	,	0.092538834	,]],
[[	0.102715922	,	0.248823952	,	0.26119981	,	0.251760873	,],
[	0.373391253	,	0.09124402	,	0.24224932	,	0.369454096	,],
[	0.450547352	,	0.314373951	,	0.072985607	,	0.345745526	,],
[	0.355740267	,	0.348070478	,	0.252152573	,	0.094223191	,]],
[[	0.096772571	,	0.242183366	,	0.254674998	,	0.245212989	,],
[	0.370465414	,	0.084570009	,	0.239106125	,	0.366601044	,],
[	0.449507866	,	0.313079162	,	0.073005408	,	0.344387628	,],
[	0.358294366	,	0.351116595	,	0.255427221	,	0.095240506	,]],
[[	0.098914051	,	0.241924655	,	0.254543887	,	0.245192091	,],
[	0.37070654	,	0.086783454	,	0.239263251	,	0.366677495	,],
[	0.453963553	,	0.318010278	,	0.076605357	,	0.349428213	,],
[	0.354755176	,	0.347302638	,	0.251375818	,	0.092317185	,]],
[[	0.099418296	,	0.245466427	,	0.257816498	,	0.248320433	,],
[	0.373360249	,	0.087198643	,	0.241986671	,	0.369402257	,],
[	0.449341662	,	0.313009387	,	0.072027034	,	0.344406135	,],
[	0.35764631	,	0.34988965	,	0.254036467	,	0.097228867	,]],
[[	0.100981495	,	0.247426029	,	0.259401614	,	0.245703374	,],
[	0.363581971	,	0.08524116	,	0.237366339	,	0.360093472	,],
[	0.44745724	,	0.317506039	,	0.079487383	,	0.343206821	,],
[	0.351306236	,	0.349419117	,	0.253134485	,	0.091787492	,]],
[[	0.096594005	,	0.246813925	,	0.258814155	,	0.244764513	,],
[	0.367835012	,	0.087774478	,	0.241768879	,	0.364350272	,],
[	0.443834141	,	0.313401492	,	0.072102178	,	0.339509735	,],
[	0.355579914	,	0.353423907	,	0.257110782	,	0.098412879	,]],
[[	0.097102425	,	0.245327048	,	0.257388697	,	0.243515549	,],
[	0.363390157	,	0.083992615	,	0.237213278	,	0.359957181	,],
[	0.444590119	,	0.314377484	,	0.074765514	,	0.340194009	,],
[	0.35169843	,	0.350008049	,	0.253787027	,	0.091439484	,]],
[[	0.10002503	,	0.250577702	,	0.262527464	,	0.24854648	,],
[	0.364994226	,	0.087759844	,	0.238972988	,	0.36139462	,],
[	0.446060736	,	0.315975204	,	0.074310041	,	0.342106786	,],
[	0.352360992	,	0.350568574	,	0.254171261	,	0.093776226	,]],
[[	0.095720101	,	0.245355099	,	0.257305338	,	0.243359739	,],
[	0.365367245	,	0.085208712	,	0.239103178	,	0.361831913	,],
[	0.443473598	,	0.313044733	,	0.072401208	,	0.339174629	,],
[	0.354308443	,	0.352795362	,	0.256658349	,	0.093829358	,]],
[[	0.099635923	,	0.246641585	,	0.258653038	,	0.244865089	,],
[	0.363295695	,	0.084587398	,	0.237109654	,	0.359728262	,],
[	0.446576099	,	0.316466173	,	0.077925677	,	0.342079525	,],
[	0.351199366	,	0.349452526	,	0.253057299	,	0.091366462	,]],
[[	0.097389718	,	0.247840161	,	0.25972819	,	0.245692686	,],
[	0.36725937	,	0.091414493	,	0.241330388	,	0.363603903	,],
[	0.444342436	,	0.313971538	,	0.07249293	,	0.340055164	,],
[	0.354475775	,	0.352450701	,	0.256063391	,	0.09677203	,]],
[[	0.096440432	,	0.245094694	,	0.257048122	,	0.24307115	,],
[	0.36381403	,	0.083916178	,	0.237403838	,	0.360136552	,],
[	0.44410376	,	0.313825204	,	0.07380363	,	0.339660705	,],
[	0.352284555	,	0.350664387	,	0.254417982	,	0.091836411	,]],
[[	0.101475807	,	0.252058488	,	0.263951486	,	0.249964203	,],
[	0.364409127	,	0.086748714	,	0.238234297	,	0.360733001	,],
[	0.447133333	,	0.317028448	,	0.075340861	,	0.343222606	,],
[	0.351881657	,	0.349961834	,	0.253637898	,	0.092772276	,]],
[[	0.096025566	,	0.245859636	,	0.257826509	,	0.2438197	,],
[	0.366373339	,	0.086068293	,	0.240166217	,	0.362697954	,],
[	0.443509754	,	0.313152061	,	0.072247077	,	0.339253515	,],
[	0.355475684	,	0.353952326	,	0.257904727	,	0.095011508	,]],
[[	0.098600117	,	0.246029649	,	0.258015391	,	0.244166688	,],
[	0.363320414	,	0.084196854	,	0.237064838	,	0.359756089	,],
[	0.445849305	,	0.315564537	,	0.076568902	,	0.341350809	,],
[	0.351364543	,	0.349614945	,	0.253409867	,	0.091265806	,]],
[[	0.098407198	,	0.248812906	,	0.26073404	,	0.246684312	,],
[	0.366245376	,	0.089885598	,	0.240349999	,	0.362562196	,],
[	0.444951079	,	0.314645314	,	0.073116853	,	0.34090854	,],
[	0.35366797	,	0.351599558	,	0.255243376	,	0.095440024	,]],
[[	0.100136914	,	0.252578861	,	0.263437309	,	0.245704267	,],
[	0.357299607	,	0.084463141	,	0.236080258	,	0.354997216	,],
[	0.440294897	,	0.316233499	,	0.076312136	,	0.337124725	,],
[	0.34833784	,	0.353354917	,	0.256537624	,	0.091189526	,]],
[[	0.096055277	,	0.252060357	,	0.262860164	,	0.244825075	,],
[	0.361131389	,	0.091174199	,	0.240220613	,	0.358615058	,],
[	0.439075068	,	0.314856358	,	0.072424682	,	0.336117385	,],
[	0.350092339	,	0.35482894	,	0.257902919	,	0.094558117	,]],
[[	0.096262306	,	0.250391256	,	0.261263625	,	0.243378696	,],
[	0.357876488	,	0.084034456	,	0.236547147	,	0.355481319	,],
[	0.438040755	,	0.313824964	,	0.072591715	,	0.334779818	,],
[	0.350552352	,	0.355610075	,	0.258894121	,	0.092838376	,]],
[[	0.099735828	,	0.25605362	,	0.266826197	,	0.248845858	,],
[	0.358334761	,	0.086457114	,	0.237028744	,	0.355783364	,],
[	0.442497218	,	0.318555979	,	0.076034306	,	0.33976402	,],
[	0.348266874	,	0.353184597	,	0.256294216	,	0.091567737	,]],
[[	0.095215205	,	0.250519201	,	0.261385494	,	0.243447292	,],
[	0.360688398	,	0.086435269	,	0.239390319	,	0.35820386	,],
[	0.438049079	,	0.31371056	,	0.071609847	,	0.334814605	,],
[	0.352527475	,	0.357029381	,	0.260201954	,	0.098036671	,]],
[[	0.098932529	,	0.251761376	,	0.262670967	,	0.244859796	,],
[	0.357301952	,	0.084134798	,	0.23594259	,	0.354815505	,],
[	0.439473174	,	0.315320595	,	0.075000841	,	0.336238153	,],
[	0.348826673	,	0.353803804	,	0.257034153	,	0.091340746	,]],
[[	0.096911981	,	0.253058409	,	0.263812094	,	0.245704226	,],
[	0.360106621	,	0.089624626	,	0.239146635	,	0.357643333	,],
[	0.440019892	,	0.31575095	,	0.073302303	,	0.337125857	,],
[	0.349441208	,	0.354157566	,	0.257169192	,	0.093421055	,]],
[[	0.095703641	,	0.250116757	,	0.261055751	,	0.243031732	,],
[	0.358554489	,	0.084512646	,	0.23707344	,	0.356020735	,],
[	0.437990239	,	0.313591198	,	0.072053803	,	0.334739846	,],
[	0.35158455	,	0.35656791	,	0.259949862	,	0.093843204	,]],
[[	0.10113835	,	0.25760542	,	0.268199553	,	0.250168531	,],
[	0.357960985	,	0.085530731	,	0.236634706	,	0.355420283	,],
[	0.441785135	,	0.31777173	,	0.078536425	,	0.338385114	,],
[	0.348257745	,	0.353152224	,	0.256349901	,	0.091236748	,]],
[[	0.095433153	,	0.251114485	,	0.261835859	,	0.24380338	,],
[	0.361854366	,	0.087565045	,	0.240613143	,	0.359346781	,],
[	0.438310997	,	0.314090569	,	0.071729579	,	0.33520959	,],
[	0.351600837	,	0.356117115	,	0.259219862	,	0.096371437	,]],
[[	0.0978729	,	0.251181031	,	0.262062593	,	0.244221602	,],
[	0.357612249	,	0.084038131	,	0.236096762	,	0.355046695	,],
[	0.438995033	,	0.314723338	,	0.073953945	,	0.335527563	,],
[	0.349385886	,	0.354397364	,	0.257612121	,	0.091754675	,]],
[[	0.09801839	,	0.254268178	,	0.264996355	,	0.246928031	,],
[	0.359417047	,	0.088244505	,	0.238221368	,	0.356861275	,],
[	0.440999837	,	0.316885992	,	0.074280837	,	0.338045553	,],
[	0.348902921	,	0.353690994	,	0.256845967	,	0.092467858	,]],
[[	0.098332841	,	0.256388534	,	0.266589285	,	0.24455951	,],
[	0.35118215	,	0.083340241	,	0.235830671	,	0.350883972	,],
[	0.433897195	,	0.315196595	,	0.074268288	,	0.331834261	,],
[	0.346124595	,	0.357097613	,	0.260129159	,	0.090742511	,]],
[[	0.095783644	,	0.257113127	,	0.267152048	,	0.24503531	,],
[	0.353152504	,	0.087711066	,	0.237999426	,	0.352818785	,],
[	0.43602735	,	0.317417232	,	0.074729237	,	0.334463636	,],
[	0.346105243	,	0.356822729	,	0.259784698	,	0.092159533	,]],
[[	0.095035086	,	0.254714928	,	0.264851874	,	0.242652172	,],
[	0.353061718	,	0.084512873	,	0.237705448	,	0.352737238	,],
[	0.432916461	,	0.314093505	,	0.072022746	,	0.330836474	,],
[	0.349154894	,	0.359990896	,	0.263297781	,	0.093597387	,]],
[[	0.099886165	,	0.261509149	,	0.271412269	,	0.249305776	,],
[	0.351220615	,	0.084213491	,	0.236019749	,	0.350953996	,],
[	0.435779635	,	0.317299859	,	0.077483881	,	0.333746409	,],
[	0.345350275	,	0.356139893	,	0.259170751	,	0.090377437	,]],
[[	0.094502677	,	0.255308467	,	0.265420054	,	0.243199566	,],
[	0.355489337	,	0.091378418	,	0.240565854	,	0.355029019	,],
[	0.433995025	,	0.315312105	,	0.072775352	,	0.332271714	,],
[	0.348044735	,	0.358603914	,	0.261430904	,	0.095007425	,]],
[[	0.09721537	,	0.255686178	,	0.265808798	,	0.243670111	,],
[	0.351548734	,	0.083428847	,	0.236127981	,	0.351217007	,],
[	0.433485264	,	0.314684833	,	0.073483663	,	0.331349296	,],
[	0.346806167	,	0.357671961	,	0.260760052	,	0.091294612	,]],
[[	0.096767415	,	0.258192122	,	0.268178494	,	0.2459374	,],
[	0.352416986	,	0.086477291	,	0.237213105	,	0.352025559	,],
[	0.4372077	,	0.318590081	,	0.076007336	,	0.33561889	,],
[	0.3457047	,	0.356461626	,	0.259338895	,	0.091338822	,]],
[[	0.094539267	,	0.254549435	,	0.264685787	,	0.242384404	,],
[	0.353886331	,	0.085390797	,	0.238661596	,	0.353622083	,],
[	0.433003896	,	0.314268691	,	0.071940843	,	0.331061728	,],
[	0.350437699	,	0.361312691	,	0.264448749	,	0.094924213	,]],
[[	0.100763119	,	0.257882884	,	0.267962985	,	0.245877221	,],
[	0.351189062	,	0.08373432	,	0.235817164	,	0.350888974	,],
[	0.435045725	,	0.316459927	,	0.076164637	,	0.332932511	,],
[	0.345557672	,	0.356330229	,	0.259315889	,	0.090292248	,]],
[[	0.094972513	,	0.255849503	,	0.265990146	,	0.243688832	,],
[	0.354438741	,	0.089892433	,	0.239406894	,	0.353934358	,],
[	0.4348121	,	0.316033296	,	0.073378043	,	0.332945751	,],
[	0.347216406	,	0.357788649	,	0.260634998	,	0.093709742	,]],
[[	0.096280168	,	0.255186043	,	0.265300173	,	0.243147488	,],
[	0.352135813	,	0.083750618	,	0.236687053	,	0.351635084	,],
[	0.433234699	,	0.314406929	,	0.072801562	,	0.331073311	,],
[	0.347603622	,	0.358497136	,	0.261573297	,	0.09203623	,]],
[[	0.097968104	,	0.2593544	,	0.269412245	,	0.247144017	,],
[	0.3518066	,	0.085412057	,	0.236489844	,	0.351365646	,],
[	0.437334463	,	0.318977587	,	0.079868265	,	0.335255094	,],
[	0.345535468	,	0.356309847	,	0.259153178	,	0.090844581	,]],
[[	0.098080806	,	0.262035584	,	0.270857503	,	0.24496282	,],
[	0.344770528	,	0.083056245	,	0.234813159	,	0.345963347	,],
[	0.428369914	,	0.315504171	,	0.07405446	,	0.327630824	,],
[	0.3427332	,	0.359289811	,	0.262684905	,	0.090577728	,]],
[[	0.094171387	,	0.261760375	,	0.270454084	,	0.244505327	,],
[	0.346747228	,	0.087469868	,	0.236928523	,	0.347720164	,],
[	0.430999538	,	0.318156245	,	0.074955519	,	0.330814362	,],
[	0.343074329	,	0.359414409	,	0.262862872	,	0.092373049	,]],
[[	0.094222772	,	0.259967614	,	0.268708413	,	0.242639498	,],
[	0.346771433	,	0.084329772	,	0.236737981	,	0.347930613	,],
[	0.42754848	,	0.314596387	,	0.072110262	,	0.327011074	,],
[	0.345560422	,	0.362103097	,	0.26569617	,	0.093132556	,]],
[[	0.097795186	,	0.265780043	,	0.274390514	,	0.248416004	,],
[	0.344816143	,	0.08396293	,	0.234925841	,	0.346054984	,],
[	0.430168705	,	0.317554292	,	0.077166503	,	0.329557199	,],
[	0.342048398	,	0.358571015	,	0.261949409	,	0.090299004	,]],
[[	0.093117806	,	0.26014858	,	0.268879771	,	0.242772981	,],
[	0.348993507	,	0.091082342	,	0.23942718	,	0.35001471	,],
[	0.428847086	,	0.315922414	,	0.072841851	,	0.328487911	,],
[	0.345111874	,	0.361293541	,	0.264638154	,	0.095353906	,]],
[[	0.09670772	,	0.261320025	,	0.270095829	,	0.244152014	,],
[	0.345309693	,	0.083194197	,	0.235182236	,	0.346338438	,],
[	0.428047314	,	0.315063727	,	0.073250317	,	0.327299199	,],
[	0.343449134	,	0.359995894	,	0.26344587	,	0.091044391	,]],
[[	0.095040861	,	0.262737498	,	0.27143986	,	0.2453249	,],
[	0.346051104	,	0.086286117	,	0.236163655	,	0.347059279	,],
[	0.43205901	,	0.319336134	,	0.076172491	,	0.331958523	,],
[	0.342566547	,	0.359001054	,	0.262300936	,	0.091541729	,]],
[[	0.09366306	,	0.259861878	,	0.268549325	,	0.242482272	,],
[	0.347747806	,	0.085334148	,	0.237714197	,	0.348865486	,],
[	0.427741308	,	0.314785227	,	0.072012508	,	0.327197719	,],
[	0.346806921	,	0.363319449	,	0.267023093	,	0.094423118	,]],
[[	0.099512827	,	0.267286459	,	0.275935762	,	0.249917135	,],
[	0.344857292	,	0.083532081	,	0.23475952	,	0.345872466	,],
[	0.429550365	,	0.316763671	,	0.075796216	,	0.32889834	,],
[	0.342279355	,	0.358785293	,	0.262142095	,	0.090216586	,]],
[[	0.093461166	,	0.260743193	,	0.269454243	,	0.243364351	,],
[	0.348086962	,	0.089630455	,	0.238374092	,	0.349092848	,],
[	0.429683759	,	0.31673674	,	0.073553027	,	0.329140142	,],
[	0.344326523	,	0.360586795	,	0.263760165	,	0.094062509	,]],
[[	0.095645985	,	0.260730009	,	0.269429871	,	0.243378858	,],
[	0.345762066	,	0.08354335	,	0.235656657	,	0.346812029	,],
[	0.427870987	,	0.314799992	,	0.072665374	,	0.327089056	,],
[	0.344222198	,	0.360738318	,	0.264316181	,	0.091693706	,]],
[[	0.096117727	,	0.263877654	,	0.272588575	,	0.246434585	,],
[	0.345421231	,	0.085220736	,	0.235544132	,	0.346459413	,],
[	0.431714926	,	0.319278261	,	0.07947629	,	0.331105428	,],
[	0.342376891	,	0.35879733	,	0.26208119	,	0.090981409	,]],
[[	0.096638697	,	0.265380763	,	0.273900954	,	0.243185171	,],
[	0.340087051	,	0.083641745	,	0.235235086	,	0.341912863	,],
[	0.421950337	,	0.315054538	,	0.072303081	,	0.322077143	,],
[	0.341010539	,	0.363801291	,	0.266560489	,	0.091074969	,]],
[[	0.095128358	,	0.266879516	,	0.275198644	,	0.244447737	,],
[	0.338596921	,	0.083848832	,	0.233798231	,	0.340357288	,],
[	0.425318699	,	0.318805956	,	0.07833133	,	0.325318275	,],
[	0.339085412	,	0.361693481	,	0.264406414	,	0.090112184	,]],
[[	0.093786933	,	0.264052075	,	0.272408328	,	0.241621889	,],
[	0.343095613	,	0.091461337	,	0.238741132	,	0.344790153	,],
[	0.422816767	,	0.315838772	,	0.072473116	,	0.322919954	,],
[	0.343508975	,	0.365793459	,	0.268487368	,	0.096739625	,]],
[[	0.099536116	,	0.271431795	,	0.279725577	,	0.248918982	,],
[	0.338612382	,	0.082763761	,	0.233693265	,	0.340485225	,],
[	0.422874019	,	0.315948089	,	0.074120673	,	0.32278976	,],
[	0.339395859	,	0.362115801	,	0.264794763	,	0.089584264	,]],
[[	0.093484352	,	0.264828217	,	0.273187655	,	0.242425401	,],
[	0.339911825	,	0.086308575	,	0.235244697	,	0.341590665	,],
[	0.425568451	,	0.318797645	,	0.075240291	,	0.326047663	,],
[	0.340240859	,	0.362877304	,	0.265387761	,	0.092112935	,]],
[[	0.095654064	,	0.264824152	,	0.273258155	,	0.24246006	,],
[	0.340965261	,	0.08446347	,	0.235955274	,	0.342716707	,],
[	0.422049989	,	0.31512693	,	0.072132519	,	0.322094191	,],
[	0.342037675	,	0.364752383	,	0.267648506	,	0.091930751	,]],
[[	0.096227393	,	0.268025307	,	0.27633836	,	0.245529201	,],
[	0.338396426	,	0.083315372	,	0.233485741	,	0.340168897	,],
[	0.42456992	,	0.317895217	,	0.076933811	,	0.324444741	,],
[	0.339025598	,	0.361705046	,	0.264255439	,	0.089582613	,]],
[[	0.093460436	,	0.264104326	,	0.272452825	,	0.241529944	,],
[	0.342167869	,	0.089850761	,	0.237656963	,	0.343826082	,],
[	0.423462227	,	0.316572417	,	0.073017121	,	0.323610248	,],
[	0.342512615	,	0.364824129	,	0.267405449	,	0.095290781	,]],
[[	0.098807999	,	0.266676854	,	0.275029746	,	0.244379297	,],
[	0.339110161	,	0.082942671	,	0.234152154	,	0.340838705	,],
[	0.422404406	,	0.315570924	,	0.073166409	,	0.322228277	,],
[	0.340056933	,	0.362743647	,	0.26545479	,	0.090029055	,]],
[[	0.094084117	,	0.265598735	,	0.273964919	,	0.243071261	,],
[	0.339298565	,	0.08524616	,	0.234589221	,	0.340982904	,],
[	0.426792438	,	0.320084939	,	0.076455605	,	0.327251246	,],
[	0.339785473	,	0.362306352	,	0.264827985	,	0.091148772	,]],
[[	0.094846695	,	0.26448921	,	0.272750298	,	0.241988358	,],
[	0.342008642	,	0.085381321	,	0.237049683	,	0.343801187	,],
[	0.422303251	,	0.315339749	,	0.072062263	,	0.32231589	,],
[	0.343176099	,	0.365834889	,	0.268825857	,	0.093057335	,]],
[[	0.097492803	,	0.269338197	,	0.277671398	,	0.246789984	,],
[	0.338412332	,	0.082948125	,	0.233517734	,	0.340088936	,],
[	0.423836477	,	0.317081222	,	0.075653255	,	0.323603544	,],
[	0.339015617	,	0.361773142	,	0.264311942	,	0.089568345	,]],
[[	0.096276862	,	0.270765439	,	0.27687668	,	0.241731585	,],
[	0.335820321	,	0.085825329	,	0.235812956	,	0.337058072	,],
[	0.418659921	,	0.317697495	,	0.073069383	,	0.318766696	,],
[	0.341622464	,	0.370280842	,	0.272306989	,	0.09265162	,]],
[[	0.093814851	,	0.271443353	,	0.277419722	,	0.242105989	,],
[	0.332545614	,	0.083576726	,	0.232627674	,	0.333743235	,],
[	0.418995115	,	0.318068936	,	0.074944957	,	0.318927526	,],
[	0.337285812	,	0.366093629	,	0.26768666	,	0.08884541	,]],
[[	0.09301894	,	0.269011854	,	0.275092214	,	0.239818015	,],
[	0.335608908	,	0.089453169	,	0.235939982	,	0.336884462	,],
[	0.421335239	,	0.32050146	,	0.075630588	,	0.321747942	,],
[	0.339654753	,	0.368241774	,	0.269817646	,	0.093149966	,]],
[[	0.097961213	,	0.275713757	,	0.281621013	,	0.246343278	,],
[	0.333830794	,	0.083994894	,	0.233722901	,	0.33491919	,],
[	0.418129773	,	0.317073729	,	0.072804663	,	0.318068467	,],
[	0.339169349	,	0.367873941	,	0.269723805	,	0.090127938	,]],
[[	0.092415797	,	0.269578277	,	0.275544484	,	0.240191517	,],
[	0.333219686	,	0.085260524	,	0.23328217	,	0.334490523	,],
[	0.420794953	,	0.32014305	,	0.077931258	,	0.320795547	,],
[	0.337547117	,	0.366295748	,	0.267957729	,	0.089657321	,]],
[[	0.0951498	,	0.270149386	,	0.276186649	,	0.240948712	,],
[	0.337046925	,	0.08699016	,	0.237056598	,	0.338207447	,],
[	0.419404563	,	0.318412106	,	0.073581703	,	0.319464406	,],
[	0.342222348	,	0.370768002	,	0.272389591	,	0.096753751	,]],
[[	0.094895669	,	0.272555604	,	0.278469499	,	0.243151742	,],
[	0.332701537	,	0.083430683	,	0.232579258	,	0.333885319	,],
[	0.418702197	,	0.317670986	,	0.074044879	,	0.318567805	,],
[	0.337631256	,	0.366360288	,	0.268014995	,	0.088957606	,]],
[[	0.092586819	,	0.268945021	,	0.275014717	,	0.239611397	,],
[	0.334844739	,	0.088039455	,	0.235041041	,	0.336065338	,],
[	0.422454441	,	0.321654721	,	0.076881032	,	0.322802882	,],
[	0.338927267	,	0.36757594	,	0.269117965	,	0.091863705	,]],
[[	0.098522019	,	0.272296474	,	0.27828318	,	0.243174344	,],
[	0.334539734	,	0.08460266	,	0.234433824	,	0.335630387	,],
[	0.418302906	,	0.31724161	,	0.072843	,	0.318265461	,],
[	0.340029763	,	0.368727953	,	0.270614389	,	0.091016253	,]],
[[	0.092995524	,	0.270246837	,	0.276168971	,	0.24080806	,],
[	0.332856478	,	0.084435085	,	0.232767523	,	0.333967925	,],
[	0.420077642	,	0.319262437	,	0.076590197	,	0.319940239	,],
[	0.337385685	,	0.36610776	,	0.267630416	,	0.089139533	,]],
[[	0.09415294	,	0.269614724	,	0.275669968	,	0.240354249	,],
[	0.337143562	,	0.091794946	,	0.237642604	,	0.338475523	,],
[	0.420129689	,	0.319112114	,	0.074318451	,	0.320262983	,],
[	0.341275207	,	0.369715732	,	0.271236468	,	0.09522609	,]],
[[	0.096080159	,	0.273796575	,	0.279597112	,	0.244295212	,],
[	0.333111224	,	0.083538763	,	0.232955448	,	0.334204647	,],
[	0.418430571	,	0.317421877	,	0.073550043	,	0.31825717	,],
[	0.338093679	,	0.366809379	,	0.268511941	,	0.089194505	,]],
[[	0.095259751	,	0.27614216	,	0.282271781	,	0.242362498	,],
[	0.330902583	,	0.091966594	,	0.238040279	,	0.335228258	,],
[	0.413480145	,	0.319019082	,	0.073406876	,	0.315939383	,],
[	0.339282196	,	0.374156253	,	0.27616669	,	0.093136124	,]],
[[	0.093513029	,	0.277241215	,	0.283138524	,	0.243263171	,],
[	0.326722797	,	0.083543146	,	0.233141745	,	0.330749492	,],
[	0.412614832	,	0.31812793	,	0.073679233	,	0.314801843	,],
[	0.334489985	,	0.369446802	,	0.271165573	,	0.088643303	,]],
[[	0.092171871	,	0.274528679	,	0.280461117	,	0.240554834	,],
[	0.3277252	,	0.086804401	,	0.234489219	,	0.33192974	,],
[	0.416842981	,	0.322483984	,	0.07687116	,	0.319454	,],
[	0.336235308	,	0.371078309	,	0.272579026	,	0.092113858	,]],
[[	0.097894635	,	0.28157353	,	0.28745194	,	0.24773093	,],
[	0.329069423	,	0.085357852	,	0.235487988	,	0.333137243	,],
[	0.412374523	,	0.317892163	,	0.072484746	,	0.314645151	,],
[	0.336687539	,	0.371570808	,	0.273434677	,	0.090372579	,]],
[[	0.091937868	,	0.275207839	,	0.281150012	,	0.241276139	,],
[	0.326432083	,	0.084041954	,	0.232931121	,	0.330513619	,],
[	0.414084803	,	0.319661605	,	0.076201204	,	0.316168624	,],
[	0.334391396	,	0.369503902	,	0.270961691	,	0.089088358	,]],
[[	0.094149915	,	0.275573359	,	0.28153034	,	0.241650233	,],
[	0.329837862	,	0.090337687	,	0.236962976	,	0.334128142	,],
[	0.414420338	,	0.319827385	,	0.074213129	,	0.31672138	,],
[	0.338642823	,	0.373415526	,	0.274947487	,	0.09564163	,]],
[[	0.094549772	,	0.278345136	,	0.284157892	,	0.244279585	,],
[	0.327232776	,	0.083881988	,	0.23370898	,	0.331261612	,],
[	0.412344684	,	0.31793449	,	0.073117745	,	0.314467259	,],
[	0.33482895	,	0.369822686	,	0.27154396	,	0.088943789	,]],
[[	0.091932328	,	0.27456423	,	0.280466466	,	0.240531665	,],
[	0.327217824	,	0.085772708	,	0.233901918	,	0.33128418	,],
[	0.41632467	,	0.322249682	,	0.079819137	,	0.31852995	,],
[	0.335542009	,	0.370421231	,	0.271965751	,	0.090968295	,]],
[[	0.097497276	,	0.27755979	,	0.283496021	,	0.243784305	,],
[	0.330214988	,	0.086513424	,	0.236727596	,	0.33418793	,],
[	0.412853035	,	0.318243623	,	0.072792063	,	0.315099893	,],
[	0.337686647	,	0.372598568	,	0.274491963	,	0.091339261	,]],
[[	0.092542777	,	0.276052875	,	0.281839138	,	0.24192188	,],
[	0.326524448	,	0.083697087	,	0.232899072	,	0.330500112	,],
[	0.413473892	,	0.318980798	,	0.075106659	,	0.315527033	,],
[	0.334485552	,	0.369339474	,	0.270916303	,	0.088775408	,]],
[[	0.093412408	,	0.274971177	,	0.280838225	,	0.241020506	,],
[	0.328907832	,	0.088908252	,	0.23592495	,	0.333097424	,],
[	0.415318942	,	0.32079679	,	0.075063255	,	0.317664518	,],
[	0.33763415	,	0.372450102	,	0.273940688	,	0.094129676	,]],
[[	0.09585896	,	0.279572069	,	0.285435547	,	0.245550069	,],
[	0.327890303	,	0.084345403	,	0.234353686	,	0.331938896	,],
[	0.412308997	,	0.317612078	,	0.072699521	,	0.314376841	,],
[	0.335562269	,	0.370509434	,	0.272193279	,	0.089396939	,]],
[[	0.093857751	,	0.279193423	,	0.283464409	,	0.240913335	,],
[	0.323803933	,	0.091896601	,	0.235622284	,	0.328765857	,],
[	0.406994358	,	0.318252286	,	0.071912495	,	0.310462014	,],
[	0.334650685	,	0.375334478	,	0.276462391	,	0.092114078	,]],
[[	0.092581932	,	0.280666214	,	0.284871939	,	0.24231131	,],
[	0.319586359	,	0.083249822	,	0.230690525	,	0.324388263	,],
[	0.406484497	,	0.317858073	,	0.072858485	,	0.30973616	,],
[	0.330544517	,	0.371373564	,	0.272172759	,	0.088484782	,]],
[[	0.090937322	,	0.277643403	,	0.281905812	,	0.239334567	,],
[	0.320694726	,	0.086778321	,	0.232165229	,	0.325688466	,],
[	0.410126336	,	0.321555232	,	0.075100112	,	0.313832612	,],
[	0.333125506	,	0.373758197	,	0.274529665	,	0.092951737	,]],
[[	0.097114153	,	0.285156133	,	0.289315815	,	0.246797854	,],
[	0.32202549	,	0.085085314	,	0.232973763	,	0.326779214	,],
[	0.406052934	,	0.317281357	,	0.071271924	,	0.309302469	,],
[	0.3321838	,	0.372880683	,	0.27391282	,	0.089617755	,]],
[[	0.090875325	,	0.278579521	,	0.282780916	,	0.240171121	,],
[	0.319271216	,	0.08372858	,	0.230389219	,	0.324028903	,],
[	0.408118823	,	0.319744685	,	0.07563073	,	0.311357481	,],
[	0.330889805	,	0.371638885	,	0.272341747	,	0.089453311	,]],
[[	0.092812921	,	0.278557741	,	0.282889705	,	0.24023037	,],
[	0.322793699	,	0.090265966	,	0.234487952	,	0.327704176	,],
[	0.407760928	,	0.319028706	,	0.072548153	,	0.311177368	,],
[	0.335746645	,	0.376335527	,	0.277180539	,	0.096720122	,]],
[[	0.093742699	,	0.281949897	,	0.286022645	,	0.243420318	,],
[	0.320056541	,	0.083488249	,	0.230975056	,	0.324753886	,],
[	0.40615531	,	0.317531411	,	0.072058288	,	0.309420083	,],
[	0.330828632	,	0.371602057	,	0.272424751	,	0.088467055	,]],
[[	0.090599888	,	0.277691012	,	0.281900788	,	0.239229578	,],
[	0.32010475	,	0.085712804	,	0.231484523	,	0.325013349	,],
[	0.410524074	,	0.322448208	,	0.079447727	,	0.31386949	,],
[	0.332168643	,	0.372834781	,	0.273612657	,	0.091653255	,]],
[[	0.096009788	,	0.280543681	,	0.284803401	,	0.242220716	,],
[	0.323073146	,	0.086040631	,	0.234101614	,	0.327719287	,],
[	0.406351897	,	0.317637715	,	0.071417856	,	0.309581144	,],
[	0.333205118	,	0.37390935	,	0.274931713	,	0.090461147	,]],
[[	0.091446816	,	0.279274809	,	0.283473523	,	0.240770746	,],
[	0.319271289	,	0.083429571	,	0.230373652	,	0.324089783	,],
[	0.407489419	,	0.319024236	,	0.074372962	,	0.310581874	,],
[	0.330721801	,	0.371357806	,	0.272122967	,	0.088825967	,]],
[[	0.091905674	,	0.278071657	,	0.28228654	,	0.239651091	,],
[	0.321952205	,	0.088862672	,	0.233500033	,	0.326816385	,],
[	0.408593076	,	0.319945213	,	0.073345543	,	0.312013081	,],
[	0.334585085	,	0.375227119	,	0.275965191	,	0.095099394	,]],
[[	0.095014614	,	0.283041269	,	0.287305256	,	0.244678907	,],
[	0.320760849	,	0.084010889	,	0.23170189	,	0.325430505	,],
[	0.406175365	,	0.317376897	,	0.071642769	,	0.309212612	,],
[	0.331318807	,	0.372103265	,	0.272977426	,	0.088867883	,]],
[[	0.094981268	,	0.288659585	,	0.291598315	,	0.243629316	,],
[	0.321215681	,	0.091495848	,	0.236694944	,	0.326814554	,],
[	0.407951836	,	0.3251907	,	0.074194758	,	0.311003794	,],
[	0.337713739	,	0.385520225	,	0.284285052	,	0.093251721	,]],
[[	0.094191517	,	0.290421295	,	0.293355862	,	0.245404105	,],
[	0.319127936	,	0.085462012	,	0.233913845	,	0.3244688	,],
[	0.407022198	,	0.32441198	,	0.07464916	,	0.309852329	,],
[	0.333119733	,	0.381112198	,	0.279489319	,	0.089032489	,]],
[[	0.092290709	,	0.287329243	,	0.290245246	,	0.242186495	,],
[	0.318632471	,	0.08713304	,	0.233897977	,	0.324266498	,],
[	0.41122792	,	0.328658599	,	0.077582889	,	0.314543114	,],
[	0.334980516	,	0.38291738	,	0.281268407	,	0.092901239	,]],
[[	0.098567234	,	0.290874474	,	0.293784469	,	0.2458897	,],
[	0.322299989	,	0.088301867	,	0.237147484	,	0.327559659	,],
[	0.406814473	,	0.324077725	,	0.073364734	,	0.309668723	,],
[	0.335107216	,	0.38291419	,	0.281475237	,	0.090564282	,]],
[[	0.092363232	,	0.288383114	,	0.291141405	,	0.243111845	,],
[	0.318187087	,	0.085042037	,	0.232995337	,	0.323511064	,],
[	0.408544139	,	0.326042617	,	0.077122525	,	0.311336172	,],
[	0.333193101	,	0.381119201	,	0.279452868	,	0.089720002	,]],
[[	0.094099576	,	0.288120242	,	0.290963443	,	0.24297967	,],
[	0.320453105	,	0.090057947	,	0.235829233	,	0.325923208	,],
[	0.408705678	,	0.326027966	,	0.074935245	,	0.311870519	,],
[	0.337568067	,	0.385315656	,	0.283742933	,	0.096430526	,]],
[[	0.095424156	,	0.291599227	,	0.294454359	,	0.246513506	,],
[	0.319826948	,	0.086077416	,	0.234654982	,	0.325185027	,],
[	0.40685202	,	0.324122028	,	0.074008943	,	0.309617021	,],
[	0.333546226	,	0.38151293	,	0.279868794	,	0.089226882	,]],
[[	0.09206468	,	0.287366294	,	0.290181573	,	0.242146716	,],
[	0.318334192	,	0.086396507	,	0.233440657	,	0.323843664	,],
[	0.410790264	,	0.328576448	,	0.08076489	,	0.313672671	,],
[	0.334383532	,	0.382334124	,	0.280551571	,	0.091729492	,]],
[[	0.097173485	,	0.289999939	,	0.292915649	,	0.244908738	,],
[	0.322905384	,	0.094052104	,	0.238524641	,	0.328460902	,],
[	0.407250674	,	0.324423681	,	0.073570319	,	0.310178435	,],
[	0.336097084	,	0.383877719	,	0.28251761	,	0.091581991	,]],
[[	0.093060071	,	0.289195716	,	0.29196415	,	0.243828885	,],
[	0.318393238	,	0.085154987	,	0.233216481	,	0.323773543	,],
[	0.407903658	,	0.325391021	,	0.076037019	,	0.310660734	,],
[	0.333165506	,	0.381068322	,	0.279350107	,	0.089238893	,]],
[[	0.093270277	,	0.287755456	,	0.290663982	,	0.242463368	,],
[	0.319709939	,	0.088908528	,	0.235002424	,	0.32515955	,],
[	0.40968375	,	0.326990302	,	0.075816073	,	0.312819765	,],
[	0.336571988	,	0.384375175	,	0.282716164	,	0.094933776	,]],
[[	0.096677686	,	0.292836581	,	0.295654519	,	0.247714739	,],
[	0.320756972	,	0.086835643	,	0.23552067	,	0.325994006	,],
[	0.406759342	,	0.323952907	,	0.0736356	,	0.309511471	,],
[	0.334085462	,	0.381905493	,	0.280370838	,	0.089627381	,]],
[[	0.094319733	,	0.294471092	,	0.296517939	,	0.243590534	,],
[	0.314900047	,	0.09024473	,	0.235636531	,	0.321653584	,],
[	0.407146713	,	0.330208942	,	0.077251972	,	0.311215577	,],
[	0.334786888	,	0.388933572	,	0.286396832	,	0.094697444	,]],
[[	0.093715217	,	0.296468764	,	0.298188118	,	0.245481544	,],
[	0.314867747	,	0.086945994	,	0.235038059	,	0.321336636	,],
[	0.403144578	,	0.326058873	,	0.073622101	,	0.306755479	,],
[	0.331990358	,	0.386122158	,	0.283737659	,	0.089065273	,]],
[[	0.091629241	,	0.293274896	,	0.295084453	,	0.242246574	,],
[	0.313134462	,	0.08673642	,	0.233558556	,	0.319673685	,],
[	0.40502198	,	0.328355354	,	0.077680408	,	0.308621425	,],
[	0.331899277	,	0.386219776	,	0.283595377	,	0.090438361	,]],
[[	0.09783486	,	0.296679873	,	0.298595406	,	0.245917835	,],
[	0.317362149	,	0.094136248	,	0.238318017	,	0.323913486	,],
[	0.404782882	,	0.32777806	,	0.07482516	,	0.308547484	,],
[	0.335104404	,	0.389052727	,	0.286932981	,	0.091931243	,]],
[[	0.091920127	,	0.29430266	,	0.296174886	,	0.243317403	,],
[	0.313310998	,	0.085802502	,	0.233557974	,	0.319782395	,],
[	0.403231993	,	0.326326874	,	0.074411278	,	0.306746478	,],
[	0.331146084	,	0.385409507	,	0.282786354	,	0.088610645	,]],
[[	0.093374772	,	0.293996503	,	0.295791505	,	0.242996595	,],
[	0.314269853	,	0.089106511	,	0.23483387	,	0.320853117	,],
[	0.407285001	,	0.330858818	,	0.081203896	,	0.310955548	,],
[	0.333924433	,	0.38799196	,	0.285369363	,	0.093269308	,]],
[[	0.094948791	,	0.297642631	,	0.299500193	,	0.246719007	,],
[	0.315838526	,	0.087732064	,	0.235987102	,	0.322276295	,],
[	0.403562132	,	0.326321327	,	0.073711162	,	0.307144832	,],
[	0.332688179	,	0.386766461	,	0.284406499	,	0.089592172	,]],
[[	0.091560636	,	0.293291119	,	0.295106507	,	0.242249413	,],
[	0.312915135	,	0.086177916	,	0.233186854	,	0.319381586	,],
[	0.404329963	,	0.327505482	,	0.076546059	,	0.307830678	,],
[	0.331603805	,	0.385773235	,	0.283160222	,	0.089637872	,]],
[[	0.09645232	,	0.295785377	,	0.297644196	,	0.244932031	,],
[	0.316363628	,	0.09256736	,	0.237235926	,	0.322921481	,],
[	0.405737436	,	0.328646031	,	0.075751973	,	0.309605909	,],
[	0.336273436	,	0.39036616	,	0.288212185	,	0.093149287	,]],
[[	0.092535204	,	0.295020978	,	0.297006877	,	0.244058451	,],
[	0.314002006	,	0.086127869	,	0.234091025	,	0.320329316	,],
[	0.403105164	,	0.326104548	,	0.073869239	,	0.306704216	,],
[	0.331431262	,	0.385650205	,	0.283143851	,	0.088638407	,]],
[[	0.092613413	,	0.293669291	,	0.295459361	,	0.242517926	,],
[	0.313721786	,	0.088006985	,	0.234166045	,	0.320200324	,],
[	0.406447604	,	0.329764659	,	0.079745554	,	0.309968161	,],
[	0.33307902	,	0.387216228	,	0.284654206	,	0.092010016	,]],
[[	0.096295959	,	0.298926778	,	0.300748019	,	0.247970867	,],
[	0.316957155	,	0.088822725	,	0.237023184	,	0.323328912	,],
[	0.403878504	,	0.326801627	,	0.07401964	,	0.307519609	,],
[	0.333713231	,	0.387714859	,	0.285350894	,	0.090424781	,]],
[[	0.094045149	,	0.300436192	,	0.301699251	,	0.244931249	,],
[	0.307968438	,	0.088204075	,	0.23432046	,	0.316515866	,],
[	0.402498021	,	0.332891922	,	0.081557244	,	0.307231702	,],
[	0.331997144	,	0.392320343	,	0.28894402	,	0.093556322	,]],
[[	0.093360859	,	0.302293156	,	0.303445194	,	0.246741865	,],
[	0.311582401	,	0.089439841	,	0.237648199	,	0.320008667	,],
[	0.398610034	,	0.328359345	,	0.073910986	,	0.303413431	,],
[	0.330410461	,	0.390661199	,	0.287458517	,	0.089272803	,]],
[[	0.091397703	,	0.299194919	,	0.30030298	,	0.243538906	,],
[	0.307401306	,	0.086173109	,	0.233455649	,	0.315832444	,],
[	0.399503541	,	0.329480978	,	0.076791998	,	0.304019404	,],
[	0.329570542	,	0.389980435	,	0.286564727	,	0.089715876	,]],
[[	0.097499121	,	0.302679381	,	0.303881728	,	0.247196169	,],
[	0.309540504	,	0.09101611	,	0.236142054	,	0.318038222	,],
[	0.400920105	,	0.330688807	,	0.076050509	,	0.305847264	,],
[	0.333884235	,	0.394013524	,	0.291015655	,	0.092707369	,]],
[[	0.091572223	,	0.300242381	,	0.301378865	,	0.244554771	,],
[	0.309130086	,	0.087162935	,	0.235122441	,	0.317546166	,],
[	0.398309866	,	0.327958749	,	0.074120896	,	0.302788985	,],
[	0.329254836	,	0.389637395	,	0.286321437	,	0.088457807	,]],
[[	0.09308976	,	0.299898452	,	0.301080376	,	0.244263174	,],
[	0.30757111	,	0.087307498	,	0.233904898	,	0.3160742	,],
[	0.401553608	,	0.331815857	,	0.080032073	,	0.306116247	,],
[	0.331224437	,	0.39144383	,	0.287997134	,	0.09219003	,]],
[[	0.094562084	,	0.303517562	,	0.304612237	,	0.247891344	,],
[	0.31200533	,	0.094811485	,	0.238807437	,	0.320455992	,],
[	0.399109645	,	0.328729804	,	0.074314765	,	0.303714185	,],
[	0.331252801	,	0.391372172	,	0.288273263	,	0.08998193	,]],
[[	0.091262895	,	0.299245132	,	0.300421529	,	0.243607671	,],
[	0.30772799	,	0.086249353	,	0.233700201	,	0.316065985	,],
[	0.398978189	,	0.328861808	,	0.075754981	,	0.30344791	,],
[	0.3293059	,	0.389634184	,	0.286255919	,	0.089057114	,]],
[[	0.096074441	,	0.301717535	,	0.302916425	,	0.246108718	,],
[	0.308876652	,	0.089760834	,	0.23531045	,	0.317315977	,],
[	0.401918569	,	0.331676532	,	0.07711164	,	0.306849665	,],
[	0.333717568	,	0.393918035	,	0.290501781	,	0.095698319	,]],
[[	0.092252197	,	0.300871516	,	0.302001348	,	0.24529194	,],
[	0.310084568	,	0.087945766	,	0.236009399	,	0.31838014	,],
[	0.398335232	,	0.327930753	,	0.073757224	,	0.302787941	,],
[	0.329722827	,	0.390033346	,	0.286706372	,	0.088708538	,]],
[[	0.092248766	,	0.299500963	,	0.300615722	,	0.243835157	,],
[	0.307449062	,	0.086756497	,	0.233563526	,	0.315792196	,],
[	0.400681919	,	0.330777019	,	0.078621632	,	0.305259698	,],
[	0.330552456	,	0.390955754	,	0.287402293	,	0.091080541	,]],
[[	0.095931576	,	0.304819038	,	0.305889148	,	0.2492506	,],
[	0.310921192	,	0.093324549	,	0.237677139	,	0.319467308	,],
[	0.39976168	,	0.329376419	,	0.074876648	,	0.304469179	,],
[	0.33229431	,	0.392390197	,	0.289339669	,	0.09090652	,]],
[[	0.093284888	,	0.305171617	,	0.304207559	,	0.243845108	,],
[	0.302070906	,	0.087014655	,	0.233088837	,	0.310954798	,],
[	0.39728641	,	0.332794071	,	0.079236676	,	0.302384998	,],
[	0.329865738	,	0.396209318	,	0.291974313	,	0.093028636	,]],
[[	0.094374533	,	0.30842439	,	0.307434676	,	0.247122976	,],
[	0.306296804	,	0.094425428	,	0.23797495	,	0.31538014	,],
[	0.395542334	,	0.330498195	,	0.074311696	,	0.300773571	,],
[	0.328456627	,	0.394593316	,	0.290698733	,	0.089034072	,]],
[[	0.091236974	,	0.304448441	,	0.303352268	,	0.242964743	,],
[	0.302221059	,	0.085945764	,	0.233013852	,	0.311155841	,],
[	0.39492286	,	0.330054744	,	0.07519067	,	0.299926538	,],
[	0.327421421	,	0.393822833	,	0.28966651	,	0.089236527	,]],
[[	0.096450091	,	0.307153299	,	0.306213527	,	0.245789711	,],
[	0.303213311	,	0.089375666	,	0.234384026	,	0.312172361	,],
[	0.398668741	,	0.333662169	,	0.077443196	,	0.304159039	,],
[	0.331894443	,	0.397919451	,	0.294262544	,	0.092475132	,]],
[[	0.09216578	,	0.305993874	,	0.304940224	,	0.244531082	,],
[	0.304751041	,	0.087950947	,	0.235560711	,	0.313544162	,],
[	0.394568291	,	0.329493108	,	0.073790566	,	0.299622244	,],
[	0.327144678	,	0.393506398	,	0.28949859	,	0.088117136	,]],
[[	0.09244716	,	0.304720152	,	0.303710244	,	0.243251398	,],
[	0.301854882	,	0.086459388	,	0.232724067	,	0.31075297	,],
[	0.396456003	,	0.331852992	,	0.077915661	,	0.30153623	,],
[	0.329043949	,	0.395397015	,	0.291161243	,	0.091759846	,]],
[[	0.095711613	,	0.30970504	,	0.308636804	,	0.248341247	,],
[	0.305304351	,	0.092798257	,	0.236805267	,	0.314288046	,],
[	0.396231483	,	0.331163868	,	0.075013459	,	0.301519123	,],
[	0.329272071	,	0.395521906	,	0.29163811	,	0.089803129	,]],
[[	0.091291835	,	0.304653643	,	0.303685908	,	0.243171622	,],
[	0.302850394	,	0.086310846	,	0.23348362	,	0.311543956	,],
[	0.394654967	,	0.329688889	,	0.074496428	,	0.299661162	,],
[	0.327210646	,	0.393456726	,	0.289395383	,	0.088668745	,]],
[[	0.095138445	,	0.306434495	,	0.305390502	,	0.244889752	,],
[	0.302720499	,	0.088259616	,	0.233697362	,	0.311531526	,],
[	0.398828854	,	0.3344375	,	0.081513335	,	0.303824578	,],
[	0.33153402	,	0.397771873	,	0.293646888	,	0.095310389	,]],
[[	0.09291005	,	0.306828302	,	0.305766012	,	0.24535452	,],
[	0.305933293	,	0.088974377	,	0.236592755	,	0.314621986	,],
[	0.394896121	,	0.329830843	,	0.07377021	,	0.299927656	,],
[	0.327586966	,	0.393916057	,	0.289755824	,	0.088320276	,]],
[[	0.091938501	,	0.304598057	,	0.303581773	,	0.243017395	,],
[	0.301937179	,	0.085994914	,	0.232749054	,	0.310718846	,],
[	0.395723826	,	0.330971644	,	0.076677107	,	0.30064142	,],
[	0.328367206	,	0.394755065	,	0.29046309	,	0.090650642	,]],
[[	0.097138431	,	0.311189215	,	0.310081617	,	0.249768307	,],
[	0.30451015	,	0.091412207	,	0.235807336	,	0.313389907	,],
[	0.397228966	,	0.332075852	,	0.075839938	,	0.302476851	,],
[	0.330289447	,	0.396358649	,	0.292596546	,	0.090763338	,]],
[[	0.09335238	,	0.312174673	,	0.309929408	,	0.244714529	,],
[	0.297508007	,	0.087880805	,	0.233199071	,	0.306794511	,],
[	0.394971137	,	0.336849216	,	0.080836611	,	0.300046112	,],
[	0.329059496	,	0.401979393	,	0.29595487	,	0.092621892	,]],
[[	0.09350012	,	0.314642079	,	0.312188521	,	0.247205812	,],
[	0.300404777	,	0.093756012	,	0.23673061	,	0.309823207	,],
[	0.391976774	,	0.33319133	,	0.074204511	,	0.297101205	,],
[	0.328160089	,	0.400943427	,	0.295155905	,	0.089187688	,]],
[[	0.090976115	,	0.311116803	,	0.308703461	,	0.243546632	,],
[	0.29869194	,	0.088120806	,	0.234298072	,	0.307882133	,],
[	0.392209759	,	0.333718147	,	0.07621079	,	0.297211814	,],
[	0.326857814	,	0.399856055	,	0.293789994	,	0.089140571	,]],
[[	0.096738007	,	0.314331376	,	0.311896096	,	0.246941454	,],
[	0.298044851	,	0.089530069	,	0.233953803	,	0.307347511	,],
[	0.39473712	,	0.336033051	,	0.076862336	,	0.300147951	,],
[	0.331854458	,	0.404652055	,	0.298682331	,	0.096436918	,]],
[[	0.091512549	,	0.312320729	,	0.30995807	,	0.24484668	,],
[	0.302184731	,	0.091207951	,	0.237634085	,	0.311282936	,],
[	0.391358044	,	0.332540689	,	0.074003257	,	0.296247229	,],
[	0.326831119	,	0.399705264	,	0.293756201	,	0.08815779	,]],
[[	0.092407497	,	0.311601196	,	0.309318948	,	0.244089276	,],
[	0.297702671	,	0.0877134	,	0.233313806	,	0.306934962	,],
[	0.394100198	,	0.335831099	,	0.079235636	,	0.299100087	,],
[	0.32834841	,	0.401253062	,	0.295176	,	0.091339737	,]],
[[	0.094833489	,	0.315789337	,	0.313367193	,	0.248470159	,],
[	0.299644256	,	0.092377973	,	0.23580207	,	0.30894773	,],
[	0.392640695	,	0.333814212	,	0.074681256	,	0.297775608	,],
[	0.329130439	,	0.40171065	,	0.296016401	,	0.089940774	,]],
[[	0.090844405	,	0.311234821	,	0.308882986	,	0.243699338	,],
[	0.29958295	,	0.088803621	,	0.235084699	,	0.308752966	,],
[	0.391823573	,	0.333144649	,	0.075340466	,	0.296675436	,],
[	0.326708438	,	0.399607164	,	0.293500403	,	0.08855146	,]],
[[	0.095305023	,	0.313412644	,	0.311037585	,	0.245953808	,],
[	0.297853662	,	0.088762522	,	0.233580883	,	0.307059432	,],
[	0.395934864	,	0.337225738	,	0.07804891	,	0.301258095	,],
[	0.330610964	,	0.403529344	,	0.297615851	,	0.094890046	,]],
[[	0.092160715	,	0.313142281	,	0.310726777	,	0.245575076	,],
[	0.302047275	,	0.096184717	,	0.238412382	,	0.31141018	,],
[	0.391522457	,	0.332710431	,	0.074041191	,	0.296533599	,],
[	0.327288111	,	0.40008366	,	0.294189344	,	0.088395379	,]],
[[	0.091709449	,	0.311324707	,	0.308839657	,	0.243721162	,],
[	0.298083674	,	0.087768632	,	0.23352881	,	0.307155253	,],
[	0.393309953	,	0.334919905	,	0.077848743	,	0.298185329	,],
[	0.327682351	,	0.400491585	,	0.294528284	,	0.090348855	,]],
[[	0.09619154	,	0.317214933	,	0.314774819	,	0.249841241	,],
[	0.29894825	,	0.091126596	,	0.234935396	,	0.30826244	,],
[	0.393517651	,	0.334641912	,	0.075468082	,	0.298471633	,],
[	0.3301983	,	0.402724044	,	0.297026668	,	0.091017618	,]],
[[	0.092743863	,	0.31700192	,	0.313549392	,	0.244579052	,],
[	0.290888001	,	0.087589342	,	0.232528915	,	0.302151954	,],
[	0.388126759	,	0.336312708	,	0.077590309	,	0.294396389	,],
[	0.325628854	,	0.404777306	,	0.297752041	,	0.090908217	,]],
[[	0.092728886	,	0.319287119	,	0.315617595	,	0.24677441	,],
[	0.293208328	,	0.092645671	,	0.235430129	,	0.304686374	,],
[	0.389112629	,	0.336809284	,	0.075978755	,	0.295638596	,],
[	0.32636984	,	0.405211467	,	0.298566263	,	0.089481785	,]],
[[	0.090360763	,	0.315844672	,	0.312234667	,	0.24325096	,],
[	0.292660722	,	0.088285164	,	0.234045264	,	0.303840065	,],
[	0.386645837	,	0.334481003	,	0.074626757	,	0.292868658	,],
[	0.323926766	,	0.403049991	,	0.29607566	,	0.08801967	,]],
[[	0.096173655	,	0.319239968	,	0.315839638	,	0.246818088	,],
[	0.291209402	,	0.088723887	,	0.232981187	,	0.30250536	,],
[	0.390271179	,	0.33867787	,	0.081007394	,	0.296611294	,],
[	0.328005781	,	0.40709717	,	0.300100792	,	0.094314949	,]],
[[	0.090764153	,	0.317181687	,	0.313450314	,	0.244478222	,],
[	0.295726846	,	0.096647559	,	0.238222476	,	0.307174086	,],
[	0.387305188	,	0.334970763	,	0.074468128	,	0.293519349	,],
[	0.324475292	,	0.403490079	,	0.296611553	,	0.087827716	,]],
[[	0.091857213	,	0.316498098	,	0.312932116	,	0.243917695	,],
[	0.291243571	,	0.087504928	,	0.232709437	,	0.302410464	,],
[	0.387692647	,	0.335631288	,	0.076502013	,	0.293738722	,],
[	0.325025179	,	0.40405756	,	0.297083181	,	0.089858492	,]],
[[	0.093981895	,	0.320459356	,	0.316777653	,	0.248038305	,],
[	0.292545437	,	0.091296789	,	0.234470701	,	0.303843967	,],
[	0.390054489	,	0.337638964	,	0.076950141	,	0.296462692	,],
[	0.327431508	,	0.406190734	,	0.299649482	,	0.090459834	,]],
[[	0.090228841	,	0.316048166	,	0.312478579	,	0.243398888	,],
[	0.293604738	,	0.089137787	,	0.234960963	,	0.304719722	,],
[	0.386691827	,	0.334551426	,	0.074339765	,	0.292883011	,],
[	0.323955282	,	0.403015856	,	0.29601869	,	0.087619664	,]],
[[	0.094751	,	0.318284249	,	0.314710277	,	0.245850736	,],
[	0.290992345	,	0.088202828	,	0.232660379	,	0.302190188	,],
[	0.389517936	,	0.337783281	,	0.079600165	,	0.295708862	,],
[	0.327078435	,	0.406019273	,	0.299132539	,	0.092841474	,]],
[[	0.091507551	,	0.317769041	,	0.314140551	,	0.245279469	,],
[	0.294721106	,	0.094819859	,	0.236933124	,	0.306011684	,],
[	0.387933498	,	0.335609267	,	0.074920567	,	0.294226091	,],
[	0.325249172	,	0.404101152	,	0.297388591	,	0.08836809	,]],
[[	0.091209556	,	0.316154501	,	0.312610903	,	0.243533317	,],
[	0.291768264	,	0.087674228	,	0.233169689	,	0.302861288	,],
[	0.387181005	,	0.33502318	,	0.075625184	,	0.293286936	,],
[	0.324492307	,	0.40347505	,	0.296552053	,	0.088897101	,]],
[[	0.095356703	,	0.321711983	,	0.318042885	,	0.249317125	,],
[	0.291934849	,	0.090312255	,	0.233835347	,	0.303142787	,],
[	0.391281661	,	0.338843087	,	0.078191801	,	0.297666677	,],
[	0.328558435	,	0.40719689	,	0.300780859	,	0.091660304	,]],
[[	0.092338657	,	0.321524974	,	0.317047023	,	0.244151709	,],
[	0.28477929	,	0.087516686	,	0.231575463	,	0.297389301	,],
[	0.380410871	,	0.334731276	,	0.075450704	,	0.287708196	,],
[	0.321684239	,	0.405178523	,	0.298214556	,	0.089278123	,]],
[[	0.092901357	,	0.324103616	,	0.319554423	,	0.246979089	,],
[	0.286069554	,	0.09125846	,	0.233347284	,	0.298711358	,],
[	0.384149148	,	0.338153407	,	0.077502731	,	0.2918686	,],
[	0.323955268	,	0.407113664	,	0.300594382	,	0.089647349	,]],
[[	0.090065429	,	0.320463184	,	0.315854658	,	0.243181936	,],
[	0.28719583	,	0.089242773	,	0.233956032	,	0.299741402	,],
[	0.379926115	,	0.334076538	,	0.073810488	,	0.287327993	,],
[	0.320716563	,	0.404138852	,	0.29717106	,	0.087079843	,]],
[[	0.095442063	,	0.323507672	,	0.318955834	,	0.246230022	,],
[	0.28451952	,	0.088016369	,	0.231433321	,	0.297040251	,],
[	0.382019535	,	0.336639618	,	0.07819975	,	0.289345739	,],
[	0.323765966	,	0.407344095	,	0.300351803	,	0.092351224	,]],
[[	0.090802489	,	0.321863594	,	0.317222016	,	0.244589337	,],
[	0.288223554	,	0.094714122	,	0.235780956	,	0.300824431	,],
[	0.381717661	,	0.33568099	,	0.075040495	,	0.289116045	,],
[	0.321868019	,	0.405057729	,	0.298322329	,	0.087564495	,]],
[[	0.0914756	,	0.320926448	,	0.316358463	,	0.243558001	,],
[	0.285315078	,	0.087752817	,	0.232007452	,	0.297844475	,],
[	0.380140855	,	0.334421706	,	0.074714398	,	0.287415766	,],
[	0.321210416	,	0.404833634	,	0.29779052	,	0.08841745	,]],
[[	0.094237568	,	0.325438565	,	0.320855442	,	0.248215799	,],
[	0.285404187	,	0.090177224	,	0.232543254	,	0.297982604	,],
[	0.384572323	,	0.339417998	,	0.082038638	,	0.291837745	,],
[	0.325092271	,	0.408171302	,	0.301596814	,	0.090694422	,]],
[[	0.089936499	,	0.320673881	,	0.316059806	,	0.243165166	,],
[	0.288308715	,	0.090161936	,	0.234930055	,	0.30075806	,],
[	0.380264901	,	0.334320304	,	0.073844445	,	0.287574147	,],
[	0.320766742	,	0.404170539	,	0.297241615	,	0.086841216	,]],
[[	0.094222781	,	0.322596728	,	0.318020292	,	0.245319034	,],
[	0.284643426	,	0.087720111	,	0.231425514	,	0.297052701	,],
[	0.381382063	,	0.33583872	,	0.076982368	,	0.288642006	,],
[	0.322967914	,	0.406526772	,	0.299429564	,	0.090846736	,]],
[[	0.091529514	,	0.322591434	,	0.318078926	,	0.245312035	,],
[	0.287249555	,	0.093298225	,	0.234663425	,	0.299824209	,],
[	0.382681882	,	0.336608505	,	0.075921526	,	0.290092215	,],
[	0.322568026	,	0.405752633	,	0.299062626	,	0.088220566	,]],
[[	0.090803817	,	0.320589886	,	0.316075524	,	0.243181553	,],
[	0.286071023	,	0.088218682	,	0.23270093	,	0.298480695	,],
[	0.379909807	,	0.334131167	,	0.074197314	,	0.287127391	,],
[	0.320964148	,	0.404451441	,	0.297398903	,	0.087651425	,]],
[[	0.09567194	,	0.326722133	,	0.322033477	,	0.249510222	,],
[	0.28501576	,	0.08923148	,	0.232038812	,	0.297538911	,],
[	0.383530168	,	0.338186986	,	0.080492507	,	0.290819554	,],
[	0.325594574	,	0.408927557	,	0.302033282	,	0.094688177	,]],
[[	0.090912362	,	0.323287877	,	0.317262506	,	0.241168946	,],
[	0.277584805	,	0.087755109	,	0.23002115	,	0.290578976	,],
[	0.374272964	,	0.335021304	,	0.075482694	,	0.282974774	,],
[	0.318218407	,	0.408071722	,	0.300586375	,	0.08857746	,]],
[[	0.092400339	,	0.326796107	,	0.320657122	,	0.244760603	,],
[	0.276833507	,	0.089076544	,	0.229685523	,	0.289900591	,],
[	0.378541698	,	0.338785128	,	0.077970448	,	0.287617214	,],
[	0.32090225	,	0.410309553	,	0.303266165	,	0.089450883	,]],
[[	0.089009541	,	0.322601381	,	0.316474984	,	0.240407331	,],
[	0.281039562	,	0.090552414	,	0.233363469	,	0.293919908	,],
[	0.374006518	,	0.334544143	,	0.074097125	,	0.28274573	,],
[	0.317285842	,	0.407059518	,	0.299561648	,	0.086462802	,]],
[[	0.093937298	,	0.325209986	,	0.319096742	,	0.242984166	,],
[	0.276471753	,	0.087229685	,	0.229062017	,	0.289546719	,],
[	0.37578943	,	0.33679428	,	0.078085184	,	0.284502272	,],
[	0.320289048	,	0.410069629	,	0.302601305	,	0.091612451	,]],
[[	0.090077559	,	0.324220025	,	0.318059807	,	0.242078702	,],
[	0.278372507	,	0.091955782	,	0.231504195	,	0.291475319	,],
[	0.376012969	,	0.336374775	,	0.075482698	,	0.284891506	,],
[	0.318690585	,	0.408264734	,	0.300951358	,	0.087155736	,]],
[[	0.090140491	,	0.32296265	,	0.316830187	,	0.240636281	,],
[	0.278370405	,	0.088309414	,	0.230720734	,	0.291264057	,],
[	0.374061966	,	0.334675815	,	0.074873705	,	0.282850595	,],
[	0.31781089	,	0.407652037	,	0.300133377	,	0.08763785	,]],
[[	0.093888781	,	0.328165508	,	0.322002645	,	0.246146369	,],
[	0.27662066	,	0.088371106	,	0.229323904	,	0.289542785	,],
[	0.378141862	,	0.339427886	,	0.081904117	,	0.28692193	,],
[	0.322164309	,	0.411495942	,	0.304448908	,	0.090597232	,]],
[[	0.089113464	,	0.322909531	,	0.316752689	,	0.240649157	,],
[	0.280748867	,	0.095845492	,	0.234245834	,	0.293905307	,],
[	0.374467744	,	0.334888818	,	0.074248117	,	0.283151847	,],
[	0.317455647	,	0.407104641	,	0.299633309	,	0.086364444	,]],
[[	0.092611984	,	0.324353302	,	0.318274714	,	0.242068943	,],
[	0.276816269	,	0.087306128	,	0.229336728	,	0.289794785	,],
[	0.375189551	,	0.336065867	,	0.076998508	,	0.283793343	,],
[	0.319489364	,	0.40924057	,	0.301669137	,	0.09023507	,]],
[[	0.090965381	,	0.325248127	,	0.319018133	,	0.243034814	,],
[	0.277726851	,	0.090716906	,	0.230654878	,	0.290731327	,],
[	0.376999819	,	0.337308842	,	0.076424217	,	0.285862399	,],
[	0.319549723	,	0.408991979	,	0.301756421	,	0.087910216	,]],
[[	0.089580476	,	0.32267971	,	0.316521752	,	0.240388006	,],
[	0.279440914	,	0.089139834	,	0.231651526	,	0.292232619	,],
[	0.374129575	,	0.334575181	,	0.07434475	,	0.282659332	,],
[	0.317641965	,	0.407301126	,	0.299754865	,	0.087014381	,]],
[[	0.095394832	,	0.329589894	,	0.32344535	,	0.247577742	,],
[	0.276482646	,	0.087863668	,	0.229042667	,	0.289401394	,],
[	0.377166584	,	0.338284907	,	0.080220639	,	0.285819525	,],
[	0.321978042	,	0.411688985	,	0.304197046	,	0.093840156	,]],
[[	0.091906035	,	0.3314806	,	0.324504325	,	0.243673484	,],
[	0.273713926	,	0.088217736	,	0.231133781	,	0.288156085	,],
[	0.372130837	,	0.338905228	,	0.076374548	,	0.281440892	,],
[	0.318282356	,	0.414323315	,	0.306396477	,	0.089468292	,]],
[[	0.092153178	,	0.333870252	,	0.326937651	,	0.246262352	,],
[	0.272818308	,	0.089387784	,	0.230634651	,	0.287411373	,],
[	0.375557224	,	0.341846518	,	0.077855205	,	0.285075252	,],
[	0.320452125	,	0.415896372	,	0.308400362	,	0.089574356	,]],
[[	0.089445588	,	0.330301701	,	0.323316128	,	0.242449454	,],
[	0.277325384	,	0.091259891	,	0.234677482	,	0.291614098	,],
[	0.371520201	,	0.337950398	,	0.074589386	,	0.280836711	,],
[	0.317095428	,	0.413051253	,	0.305086663	,	0.087006567	,]],
[[	0.095134421	,	0.333637946	,	0.326579982	,	0.245830781	,],
[	0.272648212	,	0.087648686	,	0.230144049	,	0.286941194	,],
[	0.373847338	,	0.340911197	,	0.07933934	,	0.283165368	,],
[	0.320374616	,	0.416479744	,	0.308471587	,	0.092588539	,]],
[[	0.090122735	,	0.331629792	,	0.324610502	,	0.24387744	,],
[	0.274372148	,	0.09237001	,	0.232462536	,	0.288876682	,],
[	0.373154179	,	0.339470426	,	0.075457559	,	0.282571523	,],
[	0.318310522	,	0.414014891	,	0.306275297	,	0.087610616	,]],
[[	0.091020109	,	0.331050937	,	0.324066662	,	0.243103935	,],
[	0.274623532	,	0.088876123	,	0.231966092	,	0.288964393	,],
[	0.371847224	,	0.338506435	,	0.075633146	,	0.280994048	,],
[	0.317797626	,	0.413832877	,	0.305827975	,	0.088430682	,]],
[[	0.093529922	,	0.33521988	,	0.328105768	,	0.247504535	,],
[	0.272661445	,	0.08873425	,	0.230369974	,	0.287011006	,],
[	0.37637564	,	0.343771696	,	0.08328881	,	0.28568201	,],
[	0.321688883	,	0.417178894	,	0.309645504	,	0.090664955	,]],
[[	0.089417572	,	0.330503346	,	0.32344412	,	0.242570492	,],
[	0.276763021	,	0.09618965	,	0.235208379	,	0.291224801	,],
[	0.371911292	,	0.338267608	,	0.074486943	,	0.281182746	,],
[	0.317404921	,	0.413227094	,	0.305287027	,	0.086910903	,]],
[[	0.093800344	,	0.332721435	,	0.325726113	,	0.244815344	,],
[	0.272945025	,	0.087803623	,	0.230373	,	0.287387044	,],
[	0.373036298	,	0.339994155	,	0.078028131	,	0.282272227	,],
[	0.319466677	,	0.41546293	,	0.307562856	,	0.09116704	,]],
[[	0.09087982	,	0.332540138	,	0.325478803	,	0.244679847	,],
[	0.273773729	,	0.091096794	,	0.231681777	,	0.288285168	,],
[	0.374072513	,	0.340349349	,	0.076296804	,	0.283409949	,],
[	0.31914679	,	0.414842093	,	0.307157673	,	0.088314585	,]],
[[	0.090278373	,	0.330623602	,	0.323657718	,	0.242630788	,],
[	0.275647895	,	0.08970887	,	0.232977529	,	0.289879341	,],
[	0.371699485	,	0.338196747	,	0.075075048	,	0.280863341	,],
[	0.31741851	,	0.413479974	,	0.305465992	,	0.087784634	,]],
[[	0.094915146	,	0.336537623	,	0.329525845	,	0.24891113	,],
[	0.272553395	,	0.088175971	,	0.230112213	,	0.286823989	,],
[	0.375380839	,	0.342496055	,	0.081608797	,	0.284552443	,],
[	0.322102478	,	0.41809206	,	0.310183255	,	0.094749446	,]],
[[	0.090869901	,	0.337086973	,	0.329055226	,	0.244554425	,],
[	0.268394759	,	0.089198348	,	0.231916726	,	0.285092962	,],
[	0.36735121	,	0.341660528	,	0.076384843	,	0.27865614	,],
[	0.316083197	,	0.419098464	,	0.309637945	,	0.089438651	,]],
[[	0.091001937	,	0.339334835	,	0.331169281	,	0.24700742	,],
[	0.266759901	,	0.089683029	,	0.230713091	,	0.283608776	,],
[	0.371549919	,	0.34525011	,	0.078744093	,	0.283175417	,],
[	0.317779787	,	0.420237911	,	0.311280543	,	0.088975493	,]],
[[	0.08839247	,	0.336000868	,	0.327786082	,	0.243375818	,],
[	0.271066322	,	0.097414543	,	0.235791072	,	0.287983594	,],
[	0.367102121	,	0.341064568	,	0.074935098	,	0.278397083	,],
[	0.314704319	,	0.417572489	,	0.308250647	,	0.086886891	,]],
[[	0.094172799	,	0.339293213	,	0.331216813	,	0.246674671	,],
[	0.266929896	,	0.088423706	,	0.230587518	,	0.283678877	,],
[	0.368862062	,	0.343358436	,	0.079226827	,	0.280218745	,],
[	0.31832599	,	0.42129503	,	0.311805504	,	0.092745675	,]],
[[	0.088993144	,	0.337237378	,	0.329143531	,	0.244710579	,],
[	0.267996068	,	0.09210957	,	0.232141056	,	0.284760439	,],
[	0.368975717	,	0.342756477	,	0.076217546	,	0.280365215	,],
[	0.315790712	,	0.418457346	,	0.309145341	,	0.087070446	,]],
[[	0.089950948	,	0.336586481	,	0.328488359	,	0.24393147	,],
[	0.269452319	,	0.089968148	,	0.232840538	,	0.285995978	,],
[	0.367216061	,	0.34130978	,	0.075752645	,	0.278400588	,],
[	0.315542499	,	0.418503016	,	0.308996999	,	0.088374326	,]],
[[	0.092285448	,	0.34064638	,	0.332474792	,	0.248260041	,],
[	0.266644352	,	0.089076169	,	0.230381464	,	0.283365812	,],
[	0.371144101	,	0.346109571	,	0.082966049	,	0.282542591	,],
[	0.318983231	,	0.42138321	,	0.312317551	,	0.090088146	,]],
[[	0.088343526	,	0.336110865	,	0.32802157	,	0.243518169	,],
[	0.270115515	,	0.095771944	,	0.234620068	,	0.286978463	,],
[	0.367470175	,	0.341311521	,	0.075007729	,	0.278695167	,],
[	0.314811782	,	0.417703612	,	0.308173448	,	0.086574703	,]],
[[	0.092765144	,	0.338290596	,	0.33025006	,	0.245646032	,],
[	0.267427699	,	0.08852925	,	0.230943932	,	0.284107923	,],
[	0.368230851	,	0.342616219	,	0.078041838	,	0.27948364	,],
[	0.317390558	,	0.420352545	,	0.310884105	,	0.091262009	,]],
[[	0.089747008	,	0.338020468	,	0.329894819	,	0.245514077	,],
[	0.267450359	,	0.091055904	,	0.231487833	,	0.284243885	,],
[	0.369959642	,	0.343788241	,	0.077133427	,	0.281370917	,],
[	0.316520558	,	0.41899903	,	0.309835597	,	0.087641902	,]],
[[	0.089261731	,	0.336219077	,	0.328157207	,	0.243554809	,],
[	0.270579934	,	0.09096601	,	0.23388361	,	0.287115247	,],
[	0.367058957	,	0.341163453	,	0.075213711	,	0.278224313	,],
[	0.315235902	,	0.418024572	,	0.308588022	,	0.087613786	,]],
[[	0.093675329	,	0.34191593	,	0.333760203	,	0.249550381	,],
[	0.266703342	,	0.088573324	,	0.230318831	,	0.283349446	,],
[	0.370245532	,	0.344897666	,	0.081296012	,	0.281602032	,],
[	0.320102896	,	0.422981165	,	0.313646567	,	0.095219479	,]],
[[	0.089974496	,	0.33956962	,	0.330825959	,	0.243232625	,],
[	0.261947173	,	0.089428216	,	0.230991212	,	0.279621912	,],
[	0.360437243	,	0.339954017	,	0.07519399	,	0.273231298	,],
[	0.312358333	,	0.419751413	,	0.310583913	,	0.088349474	,]],
[[	0.090359871	,	0.341954907	,	0.333303082	,	0.245932382	,],
[	0.258984854	,	0.088298507	,	0.228554933	,	0.276881841	,],
[	0.364444168	,	0.344651222	,	0.082450196	,	0.277377248	,],
[	0.31442336	,	0.421233684	,	0.31252341	,	0.088263548	,]],
[[	0.087656926	,	0.338434678	,	0.32970241	,	0.24219235	,],
[	0.262429869	,	0.095014402	,	0.23267752	,	0.280390209	,],
[	0.360774142	,	0.33998489	,	0.074540648	,	0.273544644	,],
[	0.31117544	,	0.418432045	,	0.309301121	,	0.085913491	,]],
[[	0.09321094	,	0.341579745	,	0.332880273	,	0.245343596	,],
[	0.260037489	,	0.088017986	,	0.229136204	,	0.277686725	,],
[	0.36147925	,	0.341319908	,	0.07744657	,	0.274299141	,],
[	0.314585507	,	0.421885267	,	0.312719348	,	0.091463206	,]],
[[	0.088320321	,	0.339717745	,	0.331003137	,	0.243554422	,],
[	0.259872472	,	0.090250154	,	0.229615708	,	0.277702693	,],
[	0.363205296	,	0.342284318	,	0.076542565	,	0.276140142	,],
[	0.312334417	,	0.419341634	,	0.310308977	,	0.086362311	,]],
[[	0.089105759	,	0.339044346	,	0.330282432	,	0.242729977	,],
[	0.263035014	,	0.090400327	,	0.232090011	,	0.280733593	,],
[	0.360422478	,	0.339776783	,	0.074774207	,	0.273129978	,],
[	0.311863144	,	0.419199571	,	0.310019387	,	0.087318865	,]],
[[	0.091592557	,	0.34329955	,	0.334502707	,	0.247163324	,],
[	0.259194548	,	0.088039779	,	0.228540745	,	0.276888175	,],
[	0.363536606	,	0.343580392	,	0.080861466	,	0.27635353	,],
[	0.315659901	,	0.422267536	,	0.313626684	,	0.089347132	,]],
[[	0.087586737	,	0.338628288	,	0.329877593	,	0.242303168	,],
[	0.261523271	,	0.093485634	,	0.231673921	,	0.279395701	,],
[	0.361422799	,	0.340504833	,	0.07482084	,	0.274113522	,],
[	0.311348127	,	0.418446438	,	0.309389176	,	0.08568789	,]],
[[	0.091913179	,	0.34078918	,	0.331988838	,	0.244413883	,],
[	0.260699629	,	0.088399807	,	0.229693721	,	0.278324821	,],
[	0.361071293	,	0.34065187	,	0.076486633	,	0.27374545	,],
[	0.313727694	,	0.420989115	,	0.311836127	,	0.090187875	,]],
[[	0.089104312	,	0.340632878	,	0.331853892	,	0.244399372	,],
[	0.259508118	,	0.089430421	,	0.229128943	,	0.277282412	,],
[	0.36444425	,	0.343431456	,	0.077650534	,	0.277370255	,],
[	0.31318321	,	0.420101395	,	0.311185201	,	0.086984316	,]],
[[	0.088404588	,	0.338728133	,	0.329948784	,	0.242308049	,],
[	0.26409218	,	0.097613208	,	0.234531575	,	0.281899147	,],
[	0.360500867	,	0.339733986	,	0.074522879	,	0.273163145	,],
[	0.311487752	,	0.418802223	,	0.30966969	,	0.086581919	,]],
[[	0.093062581	,	0.344514049	,	0.335765109	,	0.248578382	,],
[	0.259398732	,	0.087827304	,	0.228590467	,	0.277068522	,],
[	0.362754339	,	0.342631912	,	0.079486859	,	0.27547531	,],
[	0.316194499	,	0.423472404	,	0.314341553	,	0.093874939	,]],
[[	0.088584651	,	0.34450009	,	0.335018865	,	0.243285567	,],
[	0.258746245	,	0.091842277	,	0.233315179	,	0.277451302	,],
[	0.356488546	,	0.342085347	,	0.074784051	,	0.269634084	,],
[	0.310258356	,	0.423576967	,	0.313487689	,	0.087287277	,]],
[[	0.090642909	,	0.348193688	,	0.338706557	,	0.247361942	,],
[	0.254023911	,	0.088441528	,	0.228869735	,	0.272767109	,],
[	0.358932748	,	0.345163072	,	0.079956683	,	0.272080334	,],
[	0.314428559	,	0.427128092	,	0.317434462	,	0.089745292	,]],
[[	0.086931722	,	0.343897155	,	0.33447822	,	0.242709576	,],
[	0.255706128	,	0.093013715	,	0.231258126	,	0.274619377	,],
[	0.357900242	,	0.343289287	,	0.075447886	,	0.271069897	,],
[	0.309981911	,	0.423148924	,	0.313020305	,	0.0857589	,]],
[[	0.091517387	,	0.346335115	,	0.336792281	,	0.244998043	,],
[	0.256042245	,	0.089475569	,	0.230676093	,	0.274669646	,],
[	0.35675246	,	0.342606538	,	0.076072019	,	0.269885027	,],
[	0.311851758	,	0.42535685	,	0.315130759	,	0.089852367	,]],
[[	0.088059291	,	0.345623236	,	0.336107892	,	0.244539956	,],
[	0.254120395	,	0.0894222	,	0.229173705	,	0.272838513	,],
[	0.361378531	,	0.346535395	,	0.078693923	,	0.27469649	,],
[	0.311892384	,	0.424718179	,	0.314864609	,	0.087097582	,]],
[[	0.087943847	,	0.344217556	,	0.334695064	,	0.242846574	,],
[	0.258032602	,	0.096881764	,	0.233855804	,	0.276812867	,],
[	0.356727964	,	0.342239749	,	0.074727523	,	0.269808499	,],
[	0.310015931	,	0.423306045	,	0.313207859	,	0.086534409	,]],
[[	0.091919926	,	0.349463093	,	0.339875943	,	0.248539773	,],
[	0.25449927	,	0.088571567	,	0.229183958	,	0.273130822	,],
[	0.358289124	,	0.344358052	,	0.078683218	,	0.271368645	,],
[	0.314543299	,	0.42785011	,	0.31770141	,	0.093497922	,]],
[[	0.087071776	,	0.344254949	,	0.33477535	,	0.242979325	,],
[	0.255085534	,	0.091827392	,	0.230405488	,	0.273866304	,],
[	0.358776383	,	0.344041202	,	0.076124435	,	0.271915723	,],
[	0.310158912	,	0.423253224	,	0.313209181	,	0.085769695	,]],
[[	0.090256435	,	0.345520896	,	0.335935615	,	0.244136849	,],
[	0.257131003	,	0.090330802	,	0.231670831	,	0.275657397	,],
[	0.35668282	,	0.342344676	,	0.075380094	,	0.269644293	,],
[	0.311304742	,	0.424660734	,	0.314365279	,	0.088634628	,]],
[[	0.089064902	,	0.346542736	,	0.337014361	,	0.245482477	,],
[	0.254065651	,	0.088891752	,	0.228952836	,	0.272651666	,],
[	0.360430263	,	0.346886985	,	0.082331438	,	0.27353216	,],
[	0.312770783	,	0.425607278	,	0.315759564	,	0.088030665	,]],
[[	0.087404796	,	0.34393268	,	0.334473768	,	0.242685206	,],
[	0.257182596	,	0.095297672	,	0.232761967	,	0.275822224	,],
[	0.357120979	,	0.342544706	,	0.074908836	,	0.270165968	,],
[	0.309992943	,	0.423201086	,	0.31292442	,	0.086040668	,]],
[[	0.093493644	,	0.35098249	,	0.341349243	,	0.250034632	,],
[	0.255090675	,	0.088737047	,	0.229674474	,	0.273615867	,],
[	0.357663846	,	0.343604641	,	0.077516321	,	0.270528473	,],
[	0.313497515	,	0.426871469	,	0.316655162	,	0.091969965	,]],
[[	0.088863304	,	0.349215397	,	0.338537409	,	0.242784773	,],
[	0.251686099	,	0.097923048	,	0.233469835	,	0.272161304	,],
[	0.351398911	,	0.34372839	,	0.074900001	,	0.265695024	,],
[	0.308169555	,	0.427037899	,	0.316151591	,	0.086802823	,]],
[[	0.089381596	,	0.351790451	,	0.340809039	,	0.24551525	,],
[	0.247208029	,	0.088319961	,	0.227923519	,	0.267491284	,],
[	0.35356637	,	0.346484352	,	0.079620521	,	0.267795234	,],
[	0.312126887	,	0.430364596	,	0.319969547	,	0.089018747	,]],
[[	0.086644912	,	0.348212635	,	0.337411771	,	0.241724363	,],
[	0.248535027	,	0.092574362	,	0.229858403	,	0.268958676	,],
[	0.353212559	,	0.345138368	,	0.07588773	,	0.267458074	,],
[	0.307638783	,	0.426324607	,	0.315585549	,	0.085174684	,]],
[[	0.092190811	,	0.351304374	,	0.340504145	,	0.244834544	,],
[	0.249400657	,	0.089679363	,	0.229927468	,	0.269664058	,],
[	0.351628307	,	0.344124279	,	0.076007226	,	0.265806241	,],
[	0.309922802	,	0.428826893	,	0.317979929	,	0.089488123	,]],
[[	0.087323309	,	0.349479209	,	0.338670165	,	0.243071074	,],
[	0.246968251	,	0.089196479	,	0.227885688	,	0.267339219	,],
[	0.355961694	,	0.349129279	,	0.083545787	,	0.270305655	,],
[	0.309532427	,	0.427966595	,	0.317336072	,	0.086494741	,]],
[[	0.08812122	,	0.348814522	,	0.338037165	,	0.242251347	,],
[	0.250706308	,	0.096251782	,	0.232411701	,	0.271163262	,],
[	0.351928497	,	0.343975322	,	0.074965809	,	0.266002937	,],
[	0.307868118	,	0.426715585	,	0.315759091	,	0.086002065	,]],
[[	0.09067639	,	0.352933913	,	0.342173427	,	0.246789144	,],
[	0.247750701	,	0.088538283	,	0.228196144	,	0.267953676	,],
[	0.352778771	,	0.345582209	,	0.078338424	,	0.266982189	,],
[	0.312482683	,	0.431265177	,	0.320465804	,	0.093163177	,]],
[[	0.086585671	,	0.348311327	,	0.337467316	,	0.241898506	,],
[	0.247953142	,	0.091375278	,	0.229103436	,	0.268231101	,],
[	0.354164195	,	0.346021397	,	0.076656115	,	0.268441724	,],
[	0.308011333	,	0.426674663	,	0.315938051	,	0.085278328	,]],
[[	0.090804338	,	0.350492625	,	0.339634909	,	0.243929797	,],
[	0.25065676	,	0.090672798	,	0.230992277	,	0.270723939	,],
[	0.351587793	,	0.343909286	,	0.075386971	,	0.265649369	,],
[	0.309180194	,	0.428032843	,	0.317146542	,	0.088248015	,]],
[[	0.088070152	,	0.350329916	,	0.339376379	,	0.243905684	,],
[	0.247009902	,	0.088785229	,	0.227763304	,	0.267309439	,],
[	0.354963672	,	0.348077685	,	0.081836747	,	0.269197139	,],
[	0.310575495	,	0.428826085	,	0.318279493	,	0.087344158	,]],
[[	0.087397877	,	0.348419507	,	0.33758308	,	0.241842245	,],
[	0.249713635	,	0.094725305	,	0.231308669	,	0.270162163	,],
[	0.352250767	,	0.344350783	,	0.075113245	,	0.266431556	,],
[	0.307657299	,	0.4264244	,	0.315515519	,	0.085476019	,]],
[[	0.092151558	,	0.354302056	,	0.343348063	,	0.248177917	,],
[	0.248348503	,	0.088942306	,	0.228810129	,	0.268458399	,],
[	0.352411324	,	0.344905036	,	0.077305404	,	0.26646179	,],
[	0.311347947	,	0.43024443	,	0.319380945	,	0.091636256	,]],
[[	0.100411701	,	0.225096674	,	0.24666613	,	0.25337198	,],
[	0.375869208	,	0.080565194	,	0.23931221	,	0.376402654	,],
[	0.447965544	,	0.297421984	,	0.070264243	,	0.348572548	,],
[	0.348255707	,	0.322049222	,	0.235486168	,	0.093489491	,]],
[[	0.094663744	,	0.218262013	,	0.240104322	,	0.24699776	,],
[	0.37475157	,	0.0810888	,	0.238175441	,	0.375289039	,],
[	0.450698737	,	0.300355732	,	0.075316932	,	0.351265824	,],
[	0.348628578	,	0.322951124	,	0.236460999	,	0.091663979	,]],
[[	0.097056082	,	0.218348795	,	0.240165849	,	0.247239233	,],
[	0.379342139	,	0.083854766	,	0.242898675	,	0.37981723	,],
[	0.449263285	,	0.298753117	,	0.070859408	,	0.350014744	,],
[	0.346446182	,	0.320530534	,	0.23384797	,	0.090546029	,]],
[[	0.097263137	,	0.22167114	,	0.243284561	,	0.250028893	,],
[	0.374710393	,	0.079764045	,	0.238016613	,	0.375145608	,],
[	0.448467706	,	0.297962557	,	0.071454099	,	0.348989322	,],
[	0.350599281	,	0.324228049	,	0.237751996	,	0.097124394	,]],
[[	0.094645198	,	0.217428373	,	0.239319889	,	0.246187796	,],
[	0.37622716	,	0.083717025	,	0.239828622	,	0.376553387	,],
[	0.452367306	,	0.302219174	,	0.074097934	,	0.353348875	,],
[	0.346963817	,	0.321212672	,	0.234566808	,	0.090167262	,]],
[[	0.100297516	,	0.22017928	,	0.242086416	,	0.24923918	,],
[	0.376799347	,	0.081371435	,	0.240218169	,	0.377175818	,],
[	0.448173936	,	0.297597653	,	0.070219607	,	0.348777176	,],
[	0.347592503	,	0.321462368	,	0.234799855	,	0.092416666	,]],
[[	0.095140069	,	0.219059387	,	0.240714219	,	0.247459977	,],
[	0.374610447	,	0.080555588	,	0.237891828	,	0.375044613	,],
[	0.449909444	,	0.29954132	,	0.074045884	,	0.350449307	,],
[	0.349806975	,	0.324113507	,	0.237536798	,	0.092615706	,]],
[[	0.096148379	,	0.217848908	,	0.239730545	,	0.246680394	,],
[	0.378408626	,	0.087335687	,	0.242184015	,	0.378707836	,],
[	0.44996406	,	0.299559049	,	0.071572416	,	0.350752191	,],
[	0.34646604	,	0.320499716	,	0.233767839	,	0.090173737	,]],
[[	0.09839864	,	0.222996019	,	0.244542487	,	0.25125874	,],
[	0.375185445	,	0.080041512	,	0.238476298	,	0.375561906	,],
[	0.448277584	,	0.297601357	,	0.070892103	,	0.348747231	,],
[	0.349648712	,	0.323335389	,	0.23673945	,	0.095549277	,]],
[[	0.094464077	,	0.217721466	,	0.239417267	,	0.246313897	,],
[	0.375610183	,	0.082613294	,	0.23910592	,	0.375992863	,],
[	0.452153848	,	0.301985811	,	0.077618142	,	0.352782327	,],
[	0.347681212	,	0.321860239	,	0.235228664	,	0.090610205	,]],
[[	0.098918033	,	0.219309742	,	0.241234585	,	0.248304648	,],
[	0.377714542	,	0.082237583	,	0.241182808	,	0.378082545	,],
[	0.448525223	,	0.298003361	,	0.07034571	,	0.349123656	,],
[	0.347054838	,	0.320963269	,	0.234341588	,	0.09154618	,]],
[[	0.095816554	,	0.220020259	,	0.241628771	,	0.248408948	,],
[	0.374617276	,	0.080115637	,	0.237926815	,	0.374975873	,],
[	0.449279534	,	0.298782224	,	0.07296017	,	0.349777181	,],
[	0.35092704	,	0.325259979	,	0.238761099	,	0.093678847	,]],
[[	0.099422799	,	0.225543068	,	0.246186954	,	0.249020725	,],
[	0.369686141	,	0.085015771	,	0.239359962	,	0.372945468	,],
[	0.446008428	,	0.302965815	,	0.078008549	,	0.350104599	,],
[	0.342857213	,	0.324820634	,	0.23773925	,	0.090678446	,]],
[[	0.094983384	,	0.224866193	,	0.245324552	,	0.247753035	,],
[	0.369909616	,	0.082417647	,	0.23944918	,	0.373136641	,],
[	0.442149538	,	0.298950621	,	0.070731726	,	0.34633962	,],
[	0.344379216	,	0.326020718	,	0.238944855	,	0.093889892	,]],
[[	0.095514481	,	0.223262003	,	0.243961202	,	0.246520662	,],
[	0.368059731	,	0.081728276	,	0.237453711	,	0.371221786	,],
[	0.443094076	,	0.29981404	,	0.073332021	,	0.347038108	,],
[	0.345004201	,	0.327009345	,	0.240004824	,	0.092456215	,]],
[[	0.09843528	,	0.228772664	,	0.249088393	,	0.251434973	,],
[	0.37205137	,	0.08863062	,	0.241906952	,	0.375201649	,],
[	0.444645979	,	0.301467468	,	0.072909422	,	0.348945839	,],
[	0.342843498	,	0.32454274	,	0.237494429	,	0.091134163	,]],
[[	0.094210604	,	0.223377644	,	0.243978756	,	0.246333951	,],
[	0.368348022	,	0.081139341	,	0.237826163	,	0.371528027	,],
[	0.441829632	,	0.298503105	,	0.070911021	,	0.345887057	,],
[	0.346692473	,	0.328153475	,	0.241085912	,	0.09727405	,]],
[[	0.098215402	,	0.224776595	,	0.245406016	,	0.248156973	,],
[	0.369086617	,	0.083853806	,	0.238628869	,	0.372282799	,],
[	0.445039641	,	0.302038227	,	0.076454795	,	0.349066479	,],
[	0.343421783	,	0.325341689	,	0.238309613	,	0.090907131	,]],
[[	0.095799216	,	0.225824384	,	0.246230483	,	0.248566905	,],
[	0.370869713	,	0.083332765	,	0.240377175	,	0.374046478	,],
[	0.442696334	,	0.299492684	,	0.071124814	,	0.346870139	,],
[	0.343811308	,	0.32546336	,	0.238264815	,	0.092959227	,]],
[[	0.094833202	,	0.223180126	,	0.243679463	,	0.246192082	,],
[	0.367873543	,	0.081413027	,	0.237310105	,	0.371088826	,],
[	0.44259544	,	0.299233965	,	0.072391665	,	0.346443102	,],
[	0.346141454	,	0.328096885	,	0.241175793	,	0.093409774	,]],
[[	0.099864584	,	0.23030105	,	0.250588391	,	0.252931749	,],
[	0.37113496	,	0.0871657	,	0.240826208	,	0.374347808	,],
[	0.445609731	,	0.302622256	,	0.074006432	,	0.350013316	,],
[	0.342801568	,	0.324614298	,	0.237437189	,	0.090797022	,]],
[[	0.094352624	,	0.223797096	,	0.2443587	,	0.246699279	,],
[	0.368925948	,	0.081484453	,	0.238299463	,	0.372094611	,],
[	0.441948948	,	0.298587583	,	0.07066819	,	0.34592965	,],
[	0.345808392	,	0.327231708	,	0.240186013	,	0.095834844	,]],
[[	0.097038444	,	0.22412497	,	0.244760502	,	0.2473784	,],
[	0.368613372	,	0.082928036	,	0.238035982	,	0.371704882	,],
[	0.444267462	,	0.301104143	,	0.075092562	,	0.348133523	,],
[	0.343956478	,	0.325916738	,	0.238927238	,	0.09136932	,]],
[[	0.096722736	,	0.226982444	,	0.247345845	,	0.249689006	,],
[	0.371975409	,	0.08438184	,	0.241510205	,	0.375173663	,],
[	0.44343952	,	0.300183846	,	0.071667318	,	0.347673227	,],
[	0.343374437	,	0.324962165	,	0.237790285	,	0.091993439	,]],
[[	0.097478546	,	0.227920571	,	0.247969242	,	0.245809585	,],
[	0.359820743	,	0.080893694	,	0.235164203	,	0.362931421	,],
[	0.435592571	,	0.298504986	,	0.074228569	,	0.340332191	,],
[	0.3383888	,	0.325736098	,	0.239754668	,	0.08969587	,]],
[[	0.09397138	,	0.227944403	,	0.247872225	,	0.245390654	,],
[	0.364316105	,	0.088263965	,	0.240145263	,	0.367347102	,],
[	0.435591089	,	0.298418263	,	0.071839571	,	0.340566394	,],
[	0.33837173	,	0.325406122	,	0.239394983	,	0.091029108	,]],
[[	0.093804197	,	0.225916489	,	0.245891574	,	0.243678691	,],
[	0.359864819	,	0.079763147	,	0.23520041	,	0.36293734	,],
[	0.433896921	,	0.296535288	,	0.07093081	,	0.338530832	,],
[	0.341561771	,	0.329106946	,	0.243176444	,	0.092614865	,]],
[[	0.097685487	,	0.232217248	,	0.251927568	,	0.249439778	,],
[	0.361126567	,	0.083192515	,	0.236677023	,	0.364199035	,],
[	0.437786405	,	0.300809435	,	0.077726455	,	0.342562861	,],
[	0.337618861	,	0.324874344	,	0.238808494	,	0.089275428	,]],
[[	0.09290885	,	0.226328322	,	0.246180144	,	0.243772005	,],
[	0.362119096	,	0.081552282	,	0.237389913	,	0.365161371	,],
[	0.434197911	,	0.296891805	,	0.070508338	,	0.338944207	,],
[	0.340224202	,	0.327171056	,	0.241109166	,	0.093876371	,]],
[[	0.096183609	,	0.227243624	,	0.247191084	,	0.244959598	,],
[	0.359695805	,	0.08030433	,	0.235009375	,	0.362774523	,],
[	0.434962123	,	0.297713174	,	0.073040458	,	0.339594965	,],
[	0.339177669	,	0.326550324	,	0.240593723	,	0.090357705	,]],
[[	0.094885196	,	0.229058867	,	0.248785732	,	0.246297155	,],
[	0.363319944	,	0.086680493	,	0.239028363	,	0.366313966	,],
[	0.436487255	,	0.299428015	,	0.072729975	,	0.341491706	,],
[	0.337991561	,	0.325103674	,	0.238967621	,	0.090270211	,]],
[[	0.093284172	,	0.225840264	,	0.24578847	,	0.2433741	,],
[	0.360411464	,	0.080028971	,	0.235562467	,	0.363391549	,],
[	0.43374568	,	0.296412977	,	0.070632728	,	0.338323605	,],
[	0.342881881	,	0.330298689	,	0.244438535	,	0.093921614	,]],
[[	0.099321535	,	0.23376383	,	0.253439975	,	0.250928537	,],
[	0.360637091	,	0.082174324	,	0.235966291	,	0.36357927	,],
[	0.436924225	,	0.29981478	,	0.076183884	,	0.34163381	,],
[	0.337957845	,	0.325231374	,	0.239172263	,	0.089317594	,]],
[[	0.09314604	,	0.226812556	,	0.246646463	,	0.244193581	,],
[	0.363150674	,	0.082555016	,	0.238502668	,	0.366126476	,],
[	0.43471429	,	0.297446427	,	0.07087992	,	0.339402726	,],
[	0.339380722	,	0.326353408	,	0.240258453	,	0.092550665	,]],
[[	0.095217225	,	0.226653683	,	0.246579251	,	0.244265105	,],
[	0.359732383	,	0.079983409	,	0.234944198	,	0.362738755	,],
[	0.434598967	,	0.297208636	,	0.072117838	,	0.339002295	,],
[	0.340114902	,	0.32746867	,	0.241487443	,	0.091120344	,]],
[[	0.095959959	,	0.230224756	,	0.24999451	,	0.247515801	,],
[	0.362418558	,	0.085295187	,	0.237997933	,	0.36543928	,],
[	0.437707874	,	0.300565937	,	0.073902793	,	0.342665306	,],
[	0.337713504	,	0.324878613	,	0.238770263	,	0.089680214	,]],
[[	0.095382784	,	0.231728442	,	0.250483503	,	0.244427773	,],
[	0.352091974	,	0.080953597	,	0.233864716	,	0.357450029	,],
[	0.428028544	,	0.297935345	,	0.07307735	,	0.335162443	,],
[	0.334147702	,	0.328075278	,	0.241596373	,	0.088926703	,]],
[[	0.092759154	,	0.232571941	,	0.251077227	,	0.244732144	,],
[	0.355887084	,	0.08761752	,	0.238046161	,	0.36123487	,],
[	0.428763312	,	0.298708628	,	0.071620331	,	0.336169364	,],
[	0.333659847	,	0.327330135	,	0.240773406	,	0.089816994	,]],
[[	0.0920236	,	0.229987444	,	0.248643912	,	0.242520616	,],
[	0.352667716	,	0.080522588	,	0.234347526	,	0.35797136	,],
[	0.426533224	,	0.29627555	,	0.070133886	,	0.333611751	,],
[	0.337439849	,	0.331308485	,	0.245067535	,	0.092025893	,]],
[[	0.096844438	,	0.237013594	,	0.255418516	,	0.24908269	,],
[	0.353050539	,	0.082964658	,	0.234853591	,	0.358280194	,],
[	0.430247543	,	0.300297643	,	0.076471458	,	0.337359087	,],
[	0.333235654	,	0.327003869	,	0.240426674	,	0.088260792	,]],
[[	0.091415402	,	0.230608165	,	0.249179012	,	0.242877828	,],
[	0.35547201	,	0.082924101	,	0.237109158	,	0.360601187	,],
[	0.427092662	,	0.296967606	,	0.069975888	,	0.334329823	,],
[	0.335370283	,	0.328974357	,	0.242367656	,	0.092437085	,]],
[[	0.09421546	,	0.230974074	,	0.249721039	,	0.243596123	,],
[	0.352135431	,	0.080584265	,	0.233670238	,	0.35729065	,],
[	0.427420062	,	0.297247706	,	0.071915968	,	0.334440067	,],
[	0.334945882	,	0.328840601	,	0.242344326	,	0.089547609	,]],
[[	0.093711698	,	0.233692169	,	0.252142186	,	0.245736159	,],
[	0.355071445	,	0.086104275	,	0.236959729	,	0.360276292	,],
[	0.429904847	,	0.299774081	,	0.072668391	,	0.337198377	,],
[	0.333381428	,	0.326968253	,	0.240436255	,	0.089064681	,]],
[[	0.091660837	,	0.229977498	,	0.248670289	,	0.242395844	,],
[	0.353378665	,	0.081020836	,	0.234904562	,	0.358453805	,],
[	0.426601669	,	0.296318618	,	0.069936353	,	0.333682992	,],
[	0.338064421	,	0.3314463	,	0.244910743	,	0.096200904	,]],
[[	0.097688762	,	0.233138397	,	0.251762051	,	0.245874064	,],
[	0.352662011	,	0.082074057	,	0.234234624	,	0.357831129	,],
[	0.429397485	,	0.299301879	,	0.075029193	,	0.336319871	,],
[	0.333503555	,	0.32725038	,	0.240830913	,	0.088361601	,]],
[[	0.091884464	,	0.231422818	,	0.249837609	,	0.243526957	,],
[	0.356611661	,	0.08406322	,	0.238271577	,	0.361679454	,],
[	0.42771939	,	0.297579458	,	0.070491677	,	0.334932668	,],
[	0.334740615	,	0.328253367	,	0.241667728	,	0.09127747	,]],
[[	0.093308278	,	0.23060632	,	0.249318461	,	0.243067365	,],
[	0.352226533	,	0.080393219	,	0.233772213	,	0.357397479	,],
[	0.427044481	,	0.296719468	,	0.071177092	,	0.33401601	,],
[	0.335870212	,	0.329732586	,	0.243387354	,	0.090406472	,]],
[[	0.094916857	,	0.234938365	,	0.253315117	,	0.247009181	,],
[	0.354214394	,	0.084770509	,	0.236033638	,	0.359365235	,],
[	0.430980741	,	0.300979285	,	0.073804836	,	0.338477738	,],
[	0.333253246	,	0.326962152	,	0.240353718	,	0.088636457	,]],
[[	0.094678061	,	0.236054773	,	0.25359836	,	0.243038041	,],
[	0.345210963	,	0.079675739	,	0.231672599	,	0.351230281	,],
[	0.421101124	,	0.296482599	,	0.071063	,	0.328821514	,],
[	0.331424393	,	0.331637065	,	0.24492839	,	0.089637362	,]],
[[	0.091920021	,	0.236699719	,	0.25408545	,	0.243174585	,],
[	0.34783156	,	0.084889458	,	0.234730848	,	0.353953968	,],
[	0.423994257	,	0.299633577	,	0.072393333	,	0.332190623	,],
[	0.329597505	,	0.329593444	,	0.242728482	,	0.088808553	,]],
[[	0.091245679	,	0.234359058	,	0.251802481	,	0.241155342	,],
[	0.346591547	,	0.080339619	,	0.233107632	,	0.352598368	,],
[	0.420332333	,	0.295770688	,	0.069142392	,	0.328171093	,],
[	0.333930306	,	0.333749742	,	0.246819854	,	0.095627305	,]],
[[	0.095983448	,	0.241047837	,	0.258334718	,	0.247548166	,],
[	0.345701116	,	0.081088155	,	0.232157034	,	0.351675079	,],
[	0.422823563	,	0.298416877	,	0.073907312	,	0.330500737	,],
[	0.329856324	,	0.330031929	,	0.243144705	,	0.088326364	,]],
[[	0.090529745	,	0.234798427	,	0.252217334	,	0.241476995	,],
[	0.350164068	,	0.083643944	,	0.236646926	,	0.356048542	,],
[	0.421797489	,	0.297230285	,	0.07007362	,	0.329701724	,],
[	0.330687117	,	0.330750967	,	0.243706791	,	0.09084131	,]],
[[	0.093537164	,	0.235434372	,	0.252939327	,	0.242265936	,],
[	0.34542086	,	0.079597141	,	0.231837486	,	0.351368753	,],
[	0.420678732	,	0.296086387	,	0.07020953	,	0.328379455	,],
[	0.332424168	,	0.332677583	,	0.245931703	,	0.090515525	,]],
[[	0.092890372	,	0.237819945	,	0.2551399	,	0.24421443	,],
[	0.347131415	,	0.083646256	,	0.23370539	,	0.352984634	,],
[	0.425268978	,	0.300900708	,	0.0737457	,	0.333392269	,],
[	0.329466472	,	0.329642164	,	0.242666153	,	0.088447004	,]],
[[	0.090834122	,	0.234261364	,	0.251737541	,	0.240914355	,],
[	0.347581266	,	0.081063701	,	0.233865855	,	0.353380902	,],
[	0.420615294	,	0.295962887	,	0.069201836	,	0.328345708	,],
[	0.332904586	,	0.332731409	,	0.245876924	,	0.094105945	,]],
[[	0.09695265	,	0.237466515	,	0.255021506	,	0.244598525	,],
[	0.345318474	,	0.080406523	,	0.231799392	,	0.351308858	,],
[	0.422204935	,	0.29763811	,	0.072675109	,	0.329728256	,],
[	0.330370074	,	0.330621685	,	0.243758209	,	0.0887515	,]],
[[	0.0910944	,	0.235560117	,	0.252967081	,	0.24203849	,],
[	0.349503159	,	0.087359434	,	0.23639433	,	0.355333684	,],
[	0.422629055	,	0.298111195	,	0.070890818	,	0.330500655	,],
[	0.330213489	,	0.330194295	,	0.243195721	,	0.089881681	,]],
[[	0.092508331	,	0.234821042	,	0.252370878	,	0.241667863	,],
[	0.345912069	,	0.079710855	,	0.232188971	,	0.351819505	,],
[	0.42054487	,	0.295801051	,	0.069634004	,	0.328165647	,],
[	0.333608272	,	0.333865053	,	0.247088765	,	0.091616894	,]],
[[	0.094035386	,	0.239006157	,	0.256303059	,	0.245431933	,],
[	0.346462468	,	0.082470494	,	0.233003503	,	0.352351775	,],
[	0.424387838	,	0.300088802	,	0.076290019	,	0.332021824	,],
[	0.329511924	,	0.329636427	,	0.242669695	,	0.088217128	,]],
[[	0.093274429	,	0.240088921	,	0.257147409	,	0.242544598	,],
[	0.340297864	,	0.080437948	,	0.232798287	,	0.348641998	,],
[	0.41511009	,	0.296408877	,	0.069454095	,	0.324729876	,],
[	0.330721529	,	0.33654234	,	0.250115647	,	0.095310643	,]],
[[	0.091922001	,	0.24185372	,	0.258614293	,	0.243842631	,],
[	0.339479033	,	0.081304698	,	0.23207386	,	0.347748233	,],
[	0.416364722	,	0.297777922	,	0.072699338	,	0.325926003	,],
[	0.326072474	,	0.332278688	,	0.245836335	,	0.087332264	,]],
[[	0.090258627	,	0.238641905	,	0.255621434	,	0.240993618	,],
[	0.343892102	,	0.083562564	,	0.236426076	,	0.351912052	,],
[	0.417157978	,	0.298529321	,	0.071203175	,	0.327017542	,],
[	0.327367267	,	0.3332987	,	0.246804897	,	0.090429985	,]],
[[	0.0965505	,	0.246669216	,	0.263319306	,	0.248547564	,],
[	0.339277416	,	0.079720512	,	0.231631763	,	0.347437272	,],
[	0.414951716	,	0.296107445	,	0.069870332	,	0.324432279	,],
[	0.328522199	,	0.33467313	,	0.248382954	,	0.089365952	,]],
[[	0.090313036	,	0.239745775	,	0.256599525	,	0.241821041	,],
[	0.341066046	,	0.084000194	,	0.233786271	,	0.349346168	,],
[	0.418651187	,	0.300252618	,	0.076192135	,	0.328146383	,],
[	0.325865168	,	0.331924035	,	0.245416021	,	0.087611296	,]],
[[	0.092287601	,	0.239508067	,	0.256486209	,	0.241857655	,],
[	0.341245729	,	0.081073668	,	0.233565032	,	0.349261556	,],
[	0.415635412	,	0.296825876	,	0.069741684	,	0.325122361	,],
[	0.329654809	,	0.335450204	,	0.249051633	,	0.093679035	,]],
[[	0.093063678	,	0.243007592	,	0.259803551	,	0.244983314	,],
[	0.339314629	,	0.08060479	,	0.231816032	,	0.347461564	,],
[	0.415873655	,	0.297283845	,	0.071684141	,	0.325330769	,],
[	0.326606593	,	0.332805386	,	0.246395802	,	0.087710239	,]],
[[	0.090080594	,	0.2388119	,	0.255669637	,	0.24091761	,],
[	0.343392211	,	0.087655722	,	0.236362973	,	0.351508192	,],
[	0.418200608	,	0.299570942	,	0.072292406	,	0.328007768	,],
[	0.326697793	,	0.332703866	,	0.246212599	,	0.089376467	,]],
[[	0.095439468	,	0.241376846	,	0.258340302	,	0.243833358	,],
[	0.339575333	,	0.079816596	,	0.231907561	,	0.347624139	,],
[	0.414986996	,	0.296195247	,	0.069535561	,	0.32436207	,],
[	0.329674709	,	0.335807033	,	0.249517224	,	0.090391618	,]],
[[	0.090936031	,	0.240536903	,	0.257293547	,	0.242539627	,],
[	0.340303002	,	0.082746292	,	0.232950847	,	0.348507971	,],
[	0.417638613	,	0.299199322	,	0.074673281	,	0.327131444	,],
[	0.325905558	,	0.331927114	,	0.245385924	,	0.087367189	,]],
[[	0.091425398	,	0.239149365	,	0.255990528	,	0.241319295	,],
[	0.342052789	,	0.081937727	,	0.234567751	,	0.350182149	,],
[	0.416167303	,	0.297442463	,	0.070216684	,	0.325843879	,],
[	0.32860988	,	0.334646933	,	0.248038094	,	0.092238027	,]],
[[	0.094373518	,	0.244480898	,	0.261118445	,	0.246304635	,],
[	0.339109277	,	0.080179679	,	0.231530845	,	0.34727872	,],
[	0.415444036	,	0.296649705	,	0.070802077	,	0.324832181	,],
[	0.327223891	,	0.333423127	,	0.24695429	,	0.088082006	,]],
[[	0.09233213	,	0.245184823	,	0.260988945	,	0.242030482	,],
[	0.337048219	,	0.082396344	,	0.234580888	,	0.345506641	,],
[	0.411665614	,	0.298675575	,	0.070512303	,	0.322140325	,],
[	0.328625461	,	0.340278251	,	0.253856056	,	0.095039094	,]],
[[	0.09163407	,	0.247432172	,	0.263041075	,	0.243867439	,],
[	0.333705304	,	0.08020779	,	0.231224912	,	0.342172081	,],
[	0.410762179	,	0.297742738	,	0.070881024	,	0.320907367	,],
[	0.323619642	,	0.335761573	,	0.249156061	,	0.086827469	,]],
[[	0.089671266	,	0.244069712	,	0.259815865	,	0.240639007	,],
[	0.336504301	,	0.085644062	,	0.2344642	,	0.345105627	,],
[	0.415044783	,	0.302205532	,	0.073905828	,	0.325789307	,],
[	0.325120952	,	0.336985643	,	0.250424901	,	0.089942719	,]],
[[	0.095893425	,	0.247423213	,	0.263131051	,	0.24425918	,],
[	0.334839588	,	0.080473021	,	0.232300252	,	0.343287947	,],
[	0.410617958	,	0.297588571	,	0.069653384	,	0.320978785	,],
[	0.326092992	,	0.338151375	,	0.251761374	,	0.088837961	,]],
[[	0.089838237	,	0.245127756	,	0.260794905	,	0.241616809	,],
[	0.334150373	,	0.081686564	,	0.231824325	,	0.34269245	,],
[	0.412207953	,	0.299344192	,	0.073131797	,	0.322273585	,],
[	0.323427434	,	0.335398546	,	0.248862904	,	0.087118868	,]],
[[	0.0914327	,	0.244817799	,	0.260489098	,	0.241429038	,],
[	0.338288596	,	0.083558096	,	0.235834184	,	0.346681434	,],
[	0.412596772	,	0.299595055	,	0.071225139	,	0.323063075	,],
[	0.327435447	,	0.339159417	,	0.252595054	,	0.093385829	,]],
[[	0.092804563	,	0.248720468	,	0.264195092	,	0.245030536	,],
[	0.333758737	,	0.079973876	,	0.231190953	,	0.342280555	,],
[	0.410533669	,	0.297409928	,	0.070100231	,	0.320729099	,],
[	0.324256417	,	0.336290857	,	0.249726521	,	0.087167269	,]],
[[	0.089453754	,	0.244151797	,	0.259820302	,	0.240722315	,],
[	0.33572121	,	0.084429887	,	0.233576534	,	0.344218885	,],
[	0.414409393	,	0.301619795	,	0.076615799	,	0.324536471	,],
[	0.324494681	,	0.336375696	,	0.249767376	,	0.088885496	,]],
[[	0.0944738	,	0.246490221	,	0.262162409	,	0.243243282	,],
[	0.33568924	,	0.081139453	,	0.233099348	,	0.344069676	,],
[	0.41107664	,	0.297984896	,	0.069937195	,	0.321358377	,],
[	0.327236151	,	0.339277639	,	0.252809963	,	0.089764045	,]],
[[	0.090520258	,	0.24607473	,	0.261586347	,	0.242398424	,],
[	0.333805393	,	0.080882067	,	0.231368483	,	0.342290241	,],
[	0.411563962	,	0.298562476	,	0.072090852	,	0.321622423	,],
[	0.323465238	,	0.335456303	,	0.248988781	,	0.086778389	,]],
[[	0.090679742	,	0.244388933	,	0.260025491	,	0.240987354	,],
[	0.338088961	,	0.088159955	,	0.236177346	,	0.346563989	,],
[	0.413557121	,	0.300505381	,	0.072157477	,	0.323909444	,],
[	0.326506007	,	0.338303754	,	0.251633574	,	0.091914583	,]],
[[	0.094122918	,	0.250070095	,	0.265528407	,	0.246437195	,],
[	0.334057711	,	0.080058812	,	0.23156085	,	0.342489184	,],
[	0.410527033	,	0.297411103	,	0.069857292	,	0.320771443	,],
[	0.324790735	,	0.336881034	,	0.250340412	,	0.0875548	,]],
[[	0.091256124	,	0.248796102	,	0.263556884	,	0.240336995	,],
[	0.329830943	,	0.081399869	,	0.232114469	,	0.339466084	,],
[	0.405114749	,	0.297985862	,	0.068735137	,	0.316204118	,],
[	0.325123783	,	0.343001022	,	0.255652109	,	0.090581472	,]],
[[	0.090594004	,	0.251041336	,	0.265642696	,	0.242164058	,],
[	0.327361055	,	0.080242816	,	0.229656851	,	0.337076957	,],
[	0.406078914	,	0.29916854	,	0.071789928	,	0.317056058	,],
[	0.320337501	,	0.338235857	,	0.250613074	,	0.08626467	,]],
[[	0.088547671	,	0.247579937	,	0.26226778	,	0.238813193	,],
[	0.331145066	,	0.087073112	,	0.233965376	,	0.340835629	,],
[	0.407338173	,	0.300399391	,	0.070945173	,	0.318681044	,],
[	0.322265818	,	0.340043938	,	0.252433156	,	0.090195391	,]],
[[	0.094790271	,	0.250927814	,	0.265700867	,	0.242460188	,],
[	0.328046389	,	0.079949457	,	0.230289462	,	0.337644403	,],
[	0.404865731	,	0.297821281	,	0.06912704	,	0.315780329	,],
[	0.322358679	,	0.340255041	,	0.252743889	,	0.087754048	,]],
[[	0.088801052	,	0.248819246	,	0.263425042	,	0.240010596	,],
[	0.328231206	,	0.08230128	,	0.230737913	,	0.338026521	,],
[	0.40821326	,	0.301486266	,	0.075031339	,	0.319214836	,],
[	0.320304429	,	0.338174629	,	0.250559388	,	0.086968431	,]],
[[	0.090305979	,	0.248300632	,	0.262985329	,	0.239614743	,],
[	0.330961396	,	0.082389087	,	0.233180628	,	0.340464576	,],
[	0.405759915	,	0.298496142	,	0.069189016	,	0.31679241	,],
[	0.324961666	,	0.342539016	,	0.254841038	,	0.093919252	,]],
[[	0.091714818	,	0.252342294	,	0.266825907	,	0.243402335	,],
[	0.327410248	,	0.079937473	,	0.229660501	,	0.336955037	,],
[	0.405731298	,	0.298700683	,	0.0707683	,	0.316627563	,],
[	0.320784236	,	0.338602624	,	0.250956132	,	0.086409116	,]],
[[	0.088367025	,	0.247766651	,	0.262351984	,	0.238999252	,],
[	0.330215095	,	0.085487764	,	0.232809149	,	0.339890175	,],
[	0.408513499	,	0.301505432	,	0.071989279	,	0.319799407	,],
[	0.321637501	,	0.339224485	,	0.251602619	,	0.088999711	,]],
[[	0.093351339	,	0.250075726	,	0.264702037	,	0.241474161	,],
[	0.328620426	,	0.080428319	,	0.230775747	,	0.338225361	,],
[	0.404847487	,	0.297675281	,	0.068923114	,	0.315805532	,],
[	0.323385783	,	0.341296544	,	0.253785413	,	0.088696531	,]],
[[	0.089409025	,	0.249543688	,	0.264143154	,	0.240670057	,],
[	0.327793794	,	0.081369925	,	0.230145328	,	0.337472502	,],
[	0.40745586	,	0.300579506	,	0.073781924	,	0.318251751	,],
[	0.32021359	,	0.33806269	,	0.250381362	,	0.086465117	,]],
[[	0.089483465	,	0.247901393	,	0.262582238	,	0.239274481	,],
[	0.33212923	,	0.083576418	,	0.234476586	,	0.341596904	,],
[	0.406316178	,	0.299116822	,	0.069747145	,	0.317357013	,],
[	0.323826435	,	0.341511247	,	0.253837159	,	0.092262656	,]],
[[	0.093087135	,	0.25366819	,	0.268116903	,	0.244709594	,],
[	0.327517373	,	0.079753823	,	0.229738222	,	0.337170876	,],
[	0.405344236	,	0.298165244	,	0.069946637	,	0.316167418	,],
[	0.321288356	,	0.339168719	,	0.251580675	,	0.086805864	,]],
[[	0.09054616	,	0.25316772	,	0.267050523	,	0.239132197	,],
[	0.32511261	,	0.088299135	,	0.23390091	,	0.336007348	,],
[	0.399752683	,	0.298679485	,	0.070160714	,	0.312288378	,],
[	0.321227301	,	0.344782582	,	0.257380558	,	0.092066912	,]],
[[	0.090216116	,	0.255539042	,	0.269344082	,	0.241303303	,],
[	0.321089422	,	0.07987394	,	0.229075158	,	0.331805699	,],
[	0.398533537	,	0.297459225	,	0.070171884	,	0.310747223	,],
[	0.317885748	,	0.341768479	,	0.254363281	,	0.085766466	,]],
[[	0.087898117	,	0.252091792	,	0.265807476	,	0.237877245	,],
[	0.322041895	,	0.083177405	,	0.230414479	,	0.332807946	,],
[	0.403150666	,	0.302502951	,	0.077771917	,	0.315400838	,],
[	0.318133918	,	0.341817807	,	0.254477051	,	0.087598697	,]],
[[	0.094012727	,	0.255333188	,	0.269303283	,	0.241310398	,],
[	0.323513638	,	0.081683175	,	0.231495063	,	0.334108262	,],
[	0.398468762	,	0.297321028	,	0.069213152	,	0.310894858	,],
[	0.32082285	,	0.344551388	,	0.257500993	,	0.088377524	,]],
[[	0.088302074	,	0.253277284	,	0.267152191	,	0.239063836	,],
[	0.32063998	,	0.080243159	,	0.228727846	,	0.331473429	,],
[	0.399919948	,	0.298877078	,	0.072617652	,	0.312075642	,],
[	0.317213916	,	0.340941083	,	0.253501078	,	0.085526082	,]],
[[	0.089624258	,	0.252760928	,	0.266583446	,	0.23855733	,],
[	0.324209969	,	0.086684904	,	0.232719886	,	0.334863708	,],
[	0.400722389	,	0.299611898	,	0.07110522	,	0.313152043	,],
[	0.32021922	,	0.343759077	,	0.256343853	,	0.090626802	,]],
[[	0.091321138	,	0.256807467	,	0.27050951	,	0.242486963	,],
[	0.3216007	,	0.080047328	,	0.229491094	,	0.332261274	,],
[	0.398354034	,	0.297189744	,	0.069671763	,	0.310594492	,],
[	0.31849657	,	0.342243029	,	0.255007214	,	0.086261432	,]],
[[	0.087783334	,	0.252186331	,	0.266036821	,	0.237885483	,],
[	0.321471191	,	0.082113169	,	0.229718409	,	0.332224558	,],
[	0.402023472	,	0.301288245	,	0.076059622	,	0.31420447	,],
[	0.3178162	,	0.341479358	,	0.253994574	,	0.086757776	,]],
[[	0.092692313	,	0.254447467	,	0.268250906	,	0.240221816	,],
[	0.324521456	,	0.082693788	,	0.232491559	,	0.335151818	,],
[	0.399009985	,	0.297825424	,	0.069534005	,	0.311318755	,],
[	0.322039905	,	0.345815951	,	0.258628016	,	0.08957135	,]],
[[	0.088909119	,	0.254075854	,	0.267764912	,	0.239651886	,],
[	0.320703016	,	0.079944912	,	0.228702547	,	0.331429502	,],
[	0.399375587	,	0.298250091	,	0.071563207	,	0.311451513	,],
[	0.317390007	,	0.341091234	,	0.253690342	,	0.085379233	,]],
[[	0.088826601	,	0.252206671	,	0.266063665	,	0.238130675	,],
[	0.323351539	,	0.085153345	,	0.231700848	,	0.334006287	,],
[	0.401752962	,	0.30068329	,	0.07208321	,	0.314235271	,],
[	0.319330697	,	0.343072	,	0.255589411	,	0.089281278	,]],
[[	0.092798977	,	0.258200352	,	0.271822987	,	0.243838581	,],
[	0.322245083	,	0.080549858	,	0.230047674	,	0.33286638	,],
[	0.39841665	,	0.297170027	,	0.069426895	,	0.310559937	,],
[	0.319378496	,	0.343251762	,	0.255869914	,	0.086897597	,]],
[[	0.089634382	,	0.257255582	,	0.269685281	,	0.238425826	,],
[	0.318352082	,	0.086540075	,	0.231714656	,	0.329883212	,],
[	0.39710066	,	0.302187914	,	0.072203813	,	0.310958418	,],
[	0.316855677	,	0.346314284	,	0.25786159	,	0.089952366	,]],
[[	0.089352999	,	0.259712657	,	0.272025663	,	0.240678138	,],
[	0.315835032	,	0.080185806	,	0.22864436	,	0.327273978	,],
[	0.393448995	,	0.298481229	,	0.069119837	,	0.307024478	,],
[	0.316420508	,	0.346000867	,	0.257704991	,	0.086912979	,]],
[[	0.087008347	,	0.256129269	,	0.268563872	,	0.23718888	,],
[	0.315615179	,	0.082089265	,	0.228790234	,	0.327220795	,],
[	0.396004986	,	0.301322104	,	0.074065556	,	0.309469272	,],
[	0.314718533	,	0.344301451	,	0.255850852	,	0.086494437	,]],
[[	0.092958145	,	0.259264233	,	0.271722751	,	0.240531439	,],
[	0.318941818	,	0.082898586	,	0.231785876	,	0.330399936	,],
[	0.395007414	,	0.299932106	,	0.069977725	,	0.308505195	,],
[	0.319452839	,	0.348814275	,	0.260349222	,	0.093637197	,]],
[[	0.087387551	,	0.257426576	,	0.269837636	,	0.238380705	,],
[	0.315032091	,	0.080000584	,	0.227830208	,	0.326438349	,],
[	0.393832121	,	0.298936142	,	0.070271187	,	0.307228968	,],
[	0.314942359	,	0.34449264	,	0.256124906	,	0.085842327	,]],
[[	0.08860205	,	0.256700541	,	0.269142784	,	0.23782107	,],
[	0.317554838	,	0.085157067	,	0.230725836	,	0.328999298	,],
[	0.398220316	,	0.303349562	,	0.073312301	,	0.312032336	,],
[	0.316170069	,	0.345680849	,	0.257149129	,	0.088764373	,]],
[[	0.09051237	,	0.260896655	,	0.273255395	,	0.24186284	,],
[	0.316550691	,	0.080743825	,	0.2293585	,	0.328002175	,],
[	0.393774795	,	0.298754541	,	0.069027892	,	0.307217531	,],
[	0.317421067	,	0.346893325	,	0.258660431	,	0.087765963	,]],
[[	0.086831911	,	0.256204485	,	0.268702104	,	0.237184581	,],
[	0.315385482	,	0.081217651	,	0.228185635	,	0.326772208	,],
[	0.395300728	,	0.300573004	,	0.072773564	,	0.308631651	,],
[	0.314659519	,	0.344194467	,	0.255711972	,	0.085956469	,]],
[[	0.091552119	,	0.258493158	,	0.270855714	,	0.239558055	,],
[	0.320122074	,	0.084080458	,	0.232959797	,	0.331553823	,],
[	0.395660457	,	0.300672149	,	0.070572076	,	0.309327192	,],
[	0.318504074	,	0.347787311	,	0.259295583	,	0.092061939	,]],
[[	0.087946176	,	0.258090208	,	0.270543853	,	0.239079993	,],
[	0.315330975	,	0.080013016	,	0.228039041	,	0.326661505	,],
[	0.393652166	,	0.298651648	,	0.069616515	,	0.3069297	,],
[	0.315450146	,	0.345058201	,	0.256627671	,	0.086139535	,]],
[[	0.087873202	,	0.256312522	,	0.268812569	,	0.237375916	,],
[	0.31683587	,	0.083888152	,	0.22995026	,	0.328287843	,],
[	0.397434465	,	0.302953903	,	0.076307235	,	0.310923652	,],
[	0.315605582	,	0.345057738	,	0.256507225	,	0.087777806	,]],
[[	0.091878891	,	0.262344546	,	0.274651902	,	0.243289718	,],
[	0.317523709	,	0.081500198	,	0.230179276	,	0.328864617	,],
[	0.39408114	,	0.299084997	,	0.069227741	,	0.307529961	,],
[	0.318458915	,	0.348046036	,	0.259708981	,	0.08884119	,]],
[[	0.088640439	,	0.261958252	,	0.273553822	,	0.237677993	,],
[	0.31104146	,	0.084932471	,	0.230055997	,	0.323749393	,],
[	0.392513039	,	0.303308895	,	0.07322617	,	0.307304373	,],
[	0.313848416	,	0.348823701	,	0.260232307	,	0.088318442	,]],
[[	0.088365243	,	0.264466218	,	0.27574208	,	0.239936404	,],
[	0.309954203	,	0.080232307	,	0.228442878	,	0.322632992	,],
[	0.388165727	,	0.298652499	,	0.06893748	,	0.302486439	,],
[	0.314117377	,	0.349170524	,	0.260643248	,	0.086172425	,]],
[[	0.085909876	,	0.26088568	,	0.272212676	,	0.236389527	,],
[	0.308896875	,	0.08089555	,	0.227459509	,	0.321523967	,],
[	0.389757636	,	0.300671237	,	0.072882331	,	0.30403376	,],
[	0.311965723	,	0.347117024	,	0.258484929	,	0.085247717	,]],
[[	0.091961241	,	0.264098796	,	0.275576054	,	0.23981947	,],
[	0.313596606	,	0.083563715	,	0.231997827	,	0.326189863	,],
[	0.389993144	,	0.300701071	,	0.070557644	,	0.30461864	,],
[	0.316391505	,	0.351211714	,	0.262562209	,	0.091893971	,]],
[[	0.086330231	,	0.262168505	,	0.273474778	,	0.237569084	,],
[	0.30869344	,	0.079440509	,	0.227112619	,	0.321395508	,],
[	0.38808621	,	0.298664324	,	0.06966538	,	0.302275005	,],
[	0.312379345	,	0.34745065	,	0.258905046	,	0.084820624	,]],
[[	0.087655683	,	0.261471843	,	0.273010755	,	0.237043676	,],
[	0.310257043	,	0.083606573	,	0.2290523	,	0.322901122	,],
[	0.392060806	,	0.303233269	,	0.076557976	,	0.306386718	,],
[	0.313238019	,	0.34822987	,	0.259444639	,	0.087112583	,]],
[[	0.089549655	,	0.265704411	,	0.27708161	,	0.241107429	,],
[	0.31096638	,	0.081001687	,	0.229293579	,	0.32340411	,],
[	0.388411205	,	0.298983471	,	0.069191744	,	0.302783395	,],
[	0.315153088	,	0.350149248	,	0.261648353	,	0.087128398	,]],
[[	0.085805788	,	0.260993886	,	0.272456904	,	0.236379776	,],
[	0.308605339	,	0.080227898	,	0.227228939	,	0.321307764	,],
[	0.38911026	,	0.299878184	,	0.07170142	,	0.303278312	,],
[	0.31196323	,	0.346950358	,	0.258293974	,	0.084761616	,]],
[[	0.090587559	,	0.263177662	,	0.274606362	,	0.238765761	,],
[	0.312695295	,	0.08730999	,	0.231633408	,	0.325154648	,],
[	0.391102426	,	0.301701256	,	0.071535646	,	0.305534316	,],
[	0.315373501	,	0.350192507	,	0.261523651	,	0.090239283	,]],
[[	0.087000672	,	0.262912956	,	0.274307661	,	0.238431687	,],
[	0.3092024	,	0.079587345	,	0.227542993	,	0.321715616	,],
[	0.388063993	,	0.29855546	,	0.069294183	,	0.302286382	,],
[	0.312996083	,	0.348117697	,	0.259533819	,	0.085176541	,]],
[[	0.086887356	,	0.261132053	,	0.272486683	,	0.236673961	,],
[	0.309751958	,	0.082459881	,	0.228303225	,	0.322226556	,],
[	0.391050444	,	0.30208382	,	0.074998123	,	0.305330421	,],
[	0.312641854	,	0.347609579	,	0.258866867	,	0.086173459	,]],
[[	0.09091546	,	0.267006117	,	0.278334897	,	0.242504164	,],
[	0.3119168	,	0.08187159	,	0.23021449	,	0.324407816	,],
[	0.389088142	,	0.299588612	,	0.069654922	,	0.303467956	,],
[	0.316273117	,	0.351285528	,	0.262797764	,	0.088140496	,]],
[[	0.088318789	,	0.266323092	,	0.276884737	,	0.237030762	,],
[	0.304175549	,	0.083433647	,	0.227931047	,	0.317204943	,],
[	0.387161233	,	0.303661504	,	0.076669768	,	0.302225314	,],
[	0.310887733	,	0.351383355	,	0.262601248	,	0.088217788	,]],
[[	0.087700608	,	0.268441125	,	0.27886272	,	0.238997772	,],
[	0.305274074	,	0.081396813	,	0.228622355	,	0.318196743	,],
[	0.383119554	,	0.29905628	,	0.068830573	,	0.298276416	,],
[	0.311064517	,	0.351595565	,	0.26291005	,	0.085956074	,]],
[[	0.085774709	,	0.265127318	,	0.275725559	,	0.235766445	,],
[	0.302721635	,	0.080347999	,	0.226212424	,	0.315701782	,],
[	0.3841366	,	0.300281983	,	0.071784239	,	0.299071212	,],
[	0.309034122	,	0.349567872	,	0.260682047	,	0.085083233	,]],
[[	0.091970546	,	0.268630273	,	0.27918945	,	0.239395625	,],
[	0.306376026	,	0.086998256	,	0.230376554	,	0.319400792	,],
[	0.385572812	,	0.301611058	,	0.070992737	,	0.300966556	,],
[	0.313411741	,	0.353795614	,	0.265070784	,	0.091998377	,]],
[[	0.085800392	,	0.266142087	,	0.276663487	,	0.236695958	,],
[	0.303435267	,	0.079845678	,	0.226721024	,	0.316319226	,],
[	0.382733865	,	0.298793966	,	0.069041396	,	0.297804258	,],
[	0.309306893	,	0.349925547	,	0.261115929	,	0.08455263	,]],
[[	0.087471022	,	0.265707509	,	0.276300881	,	0.236429919	,],
[	0.303615997	,	0.08231152	,	0.227211565	,	0.316568324	,],
[	0.386200104	,	0.302613872	,	0.075052671	,	0.301249565	,],
[	0.310253981	,	0.350729662	,	0.261865433	,	0.087111392	,]],
[[	0.088786087	,	0.269510493	,	0.279968998	,	0.240178978	,],
[	0.306304133	,	0.082303113	,	0.229629798	,	0.319274054	,],
[	0.383702177	,	0.299778009	,	0.069189487	,	0.298943326	,],
[	0.312086438	,	0.352523555	,	0.263879926	,	0.086786679	,]],
[[	0.085506859	,	0.26510577	,	0.275732965	,	0.235733876	,],
[	0.302696226	,	0.079900208	,	0.226044832	,	0.315637357	,],
[	0.383662792	,	0.299675519	,	0.070704776	,	0.298583657	,],
[	0.308890756	,	0.349493226	,	0.26059673	,	0.084500674	,]],
[[	0.090537401	,	0.267624123	,	0.278104772	,	0.238257151	,],
[	0.305505923	,	0.08552054	,	0.229408147	,	0.318422093	,],
[	0.386786725	,	0.302763569	,	0.071966399	,	0.302029126	,],
[	0.312354187	,	0.352732685	,	0.263884827	,	0.090232509	,]],
[[	0.086563679	,	0.266963204	,	0.277433589	,	0.237450449	,],
[	0.30409718	,	0.08036072	,	0.227422297	,	0.316995959	,],
[	0.382801703	,	0.298842228	,	0.068818958	,	0.297885967	,],
[	0.310066343	,	0.350541628	,	0.261784953	,	0.084981828	,]],
[[	0.086616932	,	0.265388683	,	0.27594132	,	0.236013934	,],
[	0.303186302	,	0.081390333	,	0.226737058	,	0.316137136	,],
[	0.385215652	,	0.301580848	,	0.073617279	,	0.300246183	,],
[	0.309645475	,	0.350159253	,	0.261248634	,	0.086105213	,]],
[[	0.09007984	,	0.270864076	,	0.281372712	,	0.241433263	,],
[	0.307563879	,	0.083508906	,	0.230890311	,	0.320374006	,],
[	0.384518714	,	0.300449118	,	0.06972054	,	0.299598583	,],
[	0.313206669	,	0.353670756	,	0.264995669	,	0.087949041	,]],
[[	0.090131562	,	0.276503664	,	0.287060194	,	0.241493593	,],
[	0.305136598	,	0.084254661	,	0.23234911	,	0.319325625	,],
[	0.389365243	,	0.309776734	,	0.076595261	,	0.303368667	,],
[	0.315224491	,	0.362268545	,	0.271444462	,	0.088907994	,]],
[[	0.090899695	,	0.279737403	,	0.29018782	,	0.244692985	,],
[	0.308181907	,	0.084615119	,	0.234968042	,	0.322219611	,],
[	0.387407098	,	0.307303918	,	0.071380548	,	0.30163787	,],
[	0.316513456	,	0.363445337	,	0.27294074	,	0.087956065	,]],
[[	0.087956056	,	0.275623724	,	0.286176302	,	0.240564668	,],
[	0.304327719	,	0.08204218	,	0.231295382	,	0.318484419	,],
[	0.38700048	,	0.307063653	,	0.072531437	,	0.300955565	,],
[	0.313767273	,	0.360861185	,	0.269979597	,	0.086225357	,]],
[[	0.093177095	,	0.278424704	,	0.288775523	,	0.243380345	,],
[	0.307135826	,	0.08745946	,	0.234391038	,	0.321119864	,],
[	0.390582527	,	0.310485484	,	0.074395431	,	0.304882037	,],
[	0.317592156	,	0.364398093	,	0.273728154	,	0.092268691	,]],
[[	0.088709594	,	0.277228038	,	0.28765294	,	0.242140736	,],
[	0.305895896	,	0.082685038	,	0.232703926	,	0.319954149	,],
[	0.386523762	,	0.30638045	,	0.070764513	,	0.300421136	,],
[	0.314507023	,	0.361562138	,	0.270872019	,	0.086312958	,]],
[[	0.089242031	,	0.276055206	,	0.286652423	,	0.240952226	,],
[	0.304776808	,	0.08341072	,	0.231816966	,	0.318885478	,],
[	0.38862795	,	0.308756008	,	0.075253196	,	0.302488982	,],
[	0.314646422	,	0.361585979	,	0.27079014	,	0.087799024	,]],
[[	0.092173319	,	0.28098787	,	0.291358534	,	0.245833814	,],
[	0.309381012	,	0.085882097	,	0.236085607	,	0.323371646	,],
[	0.388216402	,	0.308039583	,	0.07196138	,	0.302349957	,],
[	0.317657127	,	0.364490391	,	0.274066883	,	0.089046949	,]],
[[	0.087962884	,	0.275887848	,	0.286311628	,	0.240756684	,],
[	0.304634895	,	0.081981056	,	0.231406088	,	0.31867086	,],
[	0.386695292	,	0.306670199	,	0.071819789	,	0.300579758	,],
[	0.313817437	,	0.360677911	,	0.270025897	,	0.085861819	,]],
[[	0.091931296	,	0.277593166	,	0.288059013	,	0.242554144	,],
[	0.306227928	,	0.086133063	,	0.233414693	,	0.320300239	,],
[	0.390936038	,	0.311453114	,	0.079048599	,	0.304903407	,],
[	0.316710052	,	0.36357257	,	0.272722867	,	0.090735758	,]],
[[	0.089497238	,	0.278047062	,	0.28857074	,	0.243051579	,],
[	0.306732056	,	0.083329431	,	0.233532213	,	0.32074936	,],
[	0.386738553	,	0.306679042	,	0.070825192	,	0.300747982	,],
[	0.315291025	,	0.362161108	,	0.271593422	,	0.086713269	,]],
[[	0.088693142	,	0.275779084	,	0.286306455	,	0.240662512	,],
[	0.304547471	,	0.082688166	,	0.231447967	,	0.318553524	,],
[	0.387856336	,	0.308029538	,	0.074001672	,	0.301664065	,],
[	0.314203693	,	0.361203138	,	0.270419297	,	0.086936071	,]],
[[	0.093652286	,	0.282387497	,	0.292713839	,	0.247329926	,],
[	0.308602819	,	0.089803583	,	0.23608154	,	0.322523588	,],
[	0.389157648	,	0.308984145	,	0.072891051	,	0.303293397	,],
[	0.318855141	,	0.365770041	,	0.275261055	,	0.090309007	,]],
[[	0.0898443	,	0.281505649	,	0.29099972	,	0.24131256	,],
[	0.299106739	,	0.083610239	,	0.231523866	,	0.314388106	,],
[	0.384087038	,	0.310575368	,	0.075814588	,	0.299017581	,],
[	0.312681365	,	0.365954519	,	0.274371843	,	0.08781376	,]],
[[	0.090394764	,	0.284585836	,	0.293948634	,	0.244403573	,],
[	0.3035324	,	0.091209545	,	0.236487019	,	0.318752672	,],
[	0.383726224	,	0.309753217	,	0.072564895	,	0.298899217	,],
[	0.314861323	,	0.36808185	,	0.276783164	,	0.087999606	,]],
[[	0.087590013	,	0.280612698	,	0.290161304	,	0.240355188	,],
[	0.299275104	,	0.082589546	,	0.231530904	,	0.314579101	,],
[	0.382163808	,	0.308321951	,	0.072254298	,	0.297105797	,],
[	0.311528984	,	0.364834627	,	0.273339173	,	0.085504677	,]],
[[	0.092961029	,	0.283506124	,	0.293056433	,	0.243346664	,],
[	0.300351897	,	0.086106741	,	0.232944771	,	0.315562294	,],
[	0.386412161	,	0.313060512	,	0.07945936	,	0.301388125	,],
[	0.314874398	,	0.367990166	,	0.276519944	,	0.09097006	,]],
[[	0.088220128	,	0.282065803	,	0.291540863	,	0.241842406	,],
[	0.301622969	,	0.084239967	,	0.233849984	,	0.316814065	,],
[	0.382365006	,	0.30827139	,	0.071480431	,	0.297373058	,],
[	0.312788844	,	0.365981524	,	0.274570725	,	0.086055068	,]],
[[	0.088906806	,	0.281104426	,	0.290512261	,	0.240775723	,],
[	0.299010058	,	0.083061034	,	0.231326278	,	0.314161756	,],
[	0.383397986	,	0.309724489	,	0.074515882	,	0.298196585	,],
[	0.312251557	,	0.365428649	,	0.273870481	,	0.086816429	,]],
[[	0.091751195	,	0.285845988	,	0.295221412	,	0.245516996	,],
[	0.302639826	,	0.089702754	,	0.235383789	,	0.317748583	,],
[	0.384646629	,	0.310632546	,	0.073530469	,	0.299821199	,],
[	0.316093303	,	0.369245587	,	0.277976094	,	0.08918318	,]],
[[	0.087434006	,	0.280800973	,	0.290256476	,	0.240445639	,],
[	0.299840506	,	0.082874967	,	0.232034311	,	0.314978517	,],
[	0.38211463	,	0.308184963	,	0.071808682	,	0.296938549	,],
[	0.311678871	,	0.364871041	,	0.273327076	,	0.085238825	,]],
[[	0.091734442	,	0.282679913	,	0.292261201	,	0.242404057	,],
[	0.299796584	,	0.085024983	,	0.232284471	,	0.3149953	,],
[	0.385446175	,	0.312021345	,	0.077835918	,	0.300343846	,],
[	0.313983812	,	0.36715365	,	0.275547001	,	0.089462397	,]],
[[	0.089013825	,	0.28299259	,	0.292395178	,	0.242670508	,],
[	0.30274857	,	0.085308462	,	0.234882882	,	0.317795435	,],
[	0.382860505	,	0.308860692	,	0.071794813	,	0.297896431	,],
[	0.313413042	,	0.366661867	,	0.275307516	,	0.086520364	,]],
[[	0.088194762	,	0.280840419	,	0.290268903	,	0.240405404	,],
[	0.299014333	,	0.082697859	,	0.231176688	,	0.314033577	,],
[	0.382807768	,	0.309083335	,	0.073494907	,	0.297621719	,],
[	0.311865825	,	0.365141104	,	0.273545707	,	0.086157509	,]],
[[	0.093113866	,	0.287172094	,	0.296561202	,	0.246935489	,],
[	0.30172653	,	0.088147992	,	0.234428446	,	0.31682658	,],
[	0.385709106	,	0.311692702	,	0.074523426	,	0.300922016	,],
[	0.316691412	,	0.369741112	,	0.278098974	,	0.093206729	,]],
[[	0.088707506	,	0.286780646	,	0.294882607	,	0.241445566	,],
[	0.293653401	,	0.082666235	,	0.230631293	,	0.31011106	,],
[	0.378639142	,	0.311270568	,	0.073718779	,	0.29435411	,],
[	0.311262611	,	0.370373641	,	0.277285793	,	0.087382266	,]],
[[	0.089638444	,	0.290097798	,	0.298130912	,	0.244806154	,],
[	0.296726844	,	0.088556195	,	0.234173441	,	0.31306657	,],
[	0.381202717	,	0.313588306	,	0.074271757	,	0.297326499	,],
[	0.314405973	,	0.37337506	,	0.280709306	,	0.088847224	,]],
[[	0.086600183	,	0.285970401	,	0.294108096	,	0.240567945	,],
[	0.294896299	,	0.082864344	,	0.231686772	,	0.311330051	,],
[	0.377810015	,	0.310154776	,	0.071508108	,	0.293557537	,],
[	0.310469507	,	0.369541442	,	0.276680708	,	0.08553762	,]],
[[	0.091761357	,	0.288692233	,	0.296774272	,	0.243366721	,],
[	0.294383122	,	0.084251583	,	0.231440734	,	0.310719693	,],
[	0.380426848	,	0.313245396	,	0.076736732	,	0.296118545	,],
[	0.31316628	,	0.372293356	,	0.279239439	,	0.090143934	,]],
[[	0.087423929	,	0.287562015	,	0.295555719	,	0.242253121	,],
[	0.298379543	,	0.085883729	,	0.235149668	,	0.314727103	,],
[	0.379113519	,	0.311334023	,	0.072207042	,	0.295000421	,],
[	0.311973186	,	0.371016613	,	0.27823949	,	0.086461721	,]],
[[	0.087915079	,	0.286393343	,	0.294455087	,	0.240921532	,],
[	0.293822991	,	0.082400725	,	0.230677526	,	0.310198802	,],
[	0.378120558	,	0.310672558	,	0.072813674	,	0.293879383	,],
[	0.310888598	,	0.370000182	,	0.277003442	,	0.086507538	,]],
[[	0.091047612	,	0.291360174	,	0.299331479	,	0.246149673	,],
[	0.295903094	,	0.087156727	,	0.233140314	,	0.312257778	,],
[	0.382390953	,	0.31466674	,	0.075449719	,	0.298481255	,],
[	0.315711212	,	0.374595329	,	0.282010722	,	0.090080129	,]],
[[	0.086526338	,	0.28619303	,	0.294283878	,	0.240840755	,],
[	0.295762217	,	0.083531368	,	0.232555157	,	0.312062935	,],
[	0.377958048	,	0.310242398	,	0.07142035	,	0.293761435	,],
[	0.310775717	,	0.369854543	,	0.276803149	,	0.085579082	,]],
[[	0.090434434	,	0.287857858	,	0.295904553	,	0.242483773	,],
[	0.293974419	,	0.083491293	,	0.230979382	,	0.310358854	,],
[	0.379641644	,	0.312249905	,	0.075377612	,	0.295247288	,],
[	0.312466267	,	0.371387939	,	0.27835974	,	0.088857734	,]],
[[	0.088372604	,	0.288535597	,	0.296516211	,	0.243159359	,],
[	0.298370226	,	0.091000187	,	0.235890736	,	0.314643911	,],
[	0.379771702	,	0.312076952	,	0.072811196	,	0.295713341	,],
[	0.312978641	,	0.371916191	,	0.279180413	,	0.087297782	,]],
[[	0.087255512	,	0.286054615	,	0.294107586	,	0.240694107	,],
[	0.294239213	,	0.082491002	,	0.23095565	,	0.310468304	,],
[	0.378090648	,	0.310479333	,	0.072229484	,	0.29373304	,],
[	0.310722465	,	0.369813451	,	0.276784533	,	0.086032809	,]],
[[	0.092475333	,	0.292782568	,	0.300763011	,	0.247500218	,],
[	0.295215268	,	0.085931371	,	0.232359726	,	0.311416185	,],
[	0.381925196	,	0.314889378	,	0.078998747	,	0.297678661	,],
[	0.314816992	,	0.373792211	,	0.280730438	,	0.092328151	,]],
[[	0.08332082	,	0.286176121	,	0.294485621	,	0.23457768	,],
[	0.283108424	,	0.076668787	,	0.226030861	,	0.299023597	,],
[	0.369779875	,	0.306349632	,	0.069348643	,	0.284501962	,],
[	0.304672008	,	0.368273804	,	0.276679949	,	0.080741394	,]],
[[	0.084254468	,	0.289465897	,	0.297482537	,	0.23777277	,],
[	0.285994522	,	0.082575765	,	0.229646939	,	0.30205693	,],
[	0.372621125	,	0.308668776	,	0.069912294	,	0.287437094	,],
[	0.308548401	,	0.371846625	,	0.280622643	,	0.082859362	,]],
[[	0.081273946	,	0.285363355	,	0.293584869	,	0.233647719	,],
[	0.284284267	,	0.076956591	,	0.227096178	,	0.300116807	,],
[	0.369060772	,	0.305337954	,	0.067216618	,	0.283692716	,],
[	0.304085208	,	0.367609369	,	0.276082123	,	0.079113103	,]],
[[	0.086557573	,	0.288216085	,	0.296360041	,	0.236593422	,],
[	0.283481759	,	0.078262356	,	0.226663772	,	0.299464425	,],
[	0.371476748	,	0.308397325	,	0.07233807	,	0.286328041	,],
[	0.306313534	,	0.36981942	,	0.278317567	,	0.083389347	,]],
[[	0.081948685	,	0.286955109	,	0.294999288	,	0.235157567	,],
[	0.287769035	,	0.079913565	,	0.2304057	,	0.303437872	,],
[	0.370366768	,	0.306568932	,	0.067821036	,	0.285147211	,],
[	0.305896068	,	0.369350441	,	0.277904322	,	0.080397813	,]],
[[	0.082630462	,	0.285813846	,	0.29403424	,	0.234112403	,],
[	0.283255146	,	0.076476621	,	0.226092997	,	0.299145606	,],
[	0.369396624	,	0.305790419	,	0.068434979	,	0.284014471	,],
[	0.304332667	,	0.3679096	,	0.276258696	,	0.079962628	,]],
[[	0.085582451	,	0.290670985	,	0.298696514	,	0.238968918	,],
[	0.285072728	,	0.081241988	,	0.228524826	,	0.301109249	,],
[	0.373910112	,	0.309888527	,	0.071131122	,	0.288567714	,],
[	0.308851049	,	0.372439209	,	0.280954258	,	0.087262461	,]],
[[	0.081236417	,	0.285655707	,	0.293715206	,	0.23394259	,],
[	0.285068218	,	0.077517127	,	0.227839252	,	0.300968525	,],
[	0.369242602	,	0.305467985	,	0.067124882	,	0.283878774	,],
[	0.304429437	,	0.367960215	,	0.276434076	,	0.079065982	,]],
[[	0.085137555	,	0.287365123	,	0.295582659	,	0.235735193	,],
[	0.283316079	,	0.077563573	,	0.226327896	,	0.29923162	,],
[	0.370787749	,	0.307499175	,	0.071117012	,	0.285456064	,],
[	0.305624881	,	0.369164404	,	0.277570513	,	0.082232847	,]],
[[	0.082757364	,	0.287807441	,	0.295877061	,	0.236071825	,],
[	0.287551375	,	0.08502022	,	0.231238222	,	0.303545568	,],
[	0.371209388	,	0.307333065	,	0.068496048	,	0.285966523	,],
[	0.306900035	,	0.370314495	,	0.278882454	,	0.081177465	,]],
[[	0.082000598	,	0.28561325	,	0.293749697	,	0.233833598	,],
[	0.283537381	,	0.07650141	,	0.226289661	,	0.299373883	,],
[	0.369174364	,	0.305567842	,	0.067710804	,	0.283695098	,],
[	0.304231201	,	0.367673061	,	0.276110332	,	0.079444494	,]],
[[	0.087018948	,	0.29207451	,	0.300071866	,	0.240439756	,],
[	0.2845059	,	0.079968592	,	0.22775989	,	0.300447526	,],
[	0.373043223	,	0.310004846	,	0.074692541	,	0.287715737	,],
[	0.307931919	,	0.371472902	,	0.279935577	,	0.085647694	,]],
[[	0.083010024	,	0.291882352	,	0.299483332	,	0.235436012	,],
[	0.278576046	,	0.077552694	,	0.22668496	,	0.29589113	,],
[	0.365237117	,	0.308346406	,	0.068474179	,	0.281101666	,],
[	0.303427466	,	0.373067075	,	0.280015655	,	0.080935484	,]],
[[	0.084153967	,	0.295178948	,	0.302629393	,	0.238677282	,],
[	0.279626642	,	0.081201424	,	0.228194724	,	0.296979771	,],
[	0.36957729	,	0.313393106	,	0.076053707	,	0.285637332	,],
[	0.308044518	,	0.377724754	,	0.284937326	,	0.088217324	,]],
[[	0.080964972	,	0.29109556	,	0.298635877	,	0.234518852	,],
[	0.281000171	,	0.079295155	,	0.228887588	,	0.298170449	,],
[	0.365080249	,	0.308043965	,	0.067186829	,	0.281058921	,],
[	0.30348163	,	0.373127602	,	0.28021463	,	0.080091735	,]],
[[	0.086200367	,	0.293894662	,	0.301429612	,	0.23740201	,],
[	0.278272701	,	0.078170325	,	0.226449108	,	0.295627989	,],
[	0.366496579	,	0.309800478	,	0.070895866	,	0.282342051	,],
[	0.304735284	,	0.374407705	,	0.281463102	,	0.083232439	,]],
[[	0.081862974	,	0.292709993	,	0.300160601	,	0.236040742	,],
[	0.28166701	,	0.084835113	,	0.230579368	,	0.299118038	,],
[	0.367360429	,	0.3100827	,	0.068820631	,	0.28330155	,],
[	0.305823395	,	0.37526132	,	0.282547045	,	0.081937343	,]],
[[	0.082271491	,	0.291506394	,	0.299039708	,	0.234861986	,],
[	0.279163809	,	0.077899503	,	0.227074628	,	0.296346933	,],
[	0.364958002	,	0.308031271	,	0.067893964	,	0.280755546	,],
[	0.303282162	,	0.372968524	,	0.280032047	,	0.080402082	,]],
[[	0.085383989	,	0.29645311	,	0.303799288	,	0.239902462	,],
[	0.279163839	,	0.080140898	,	0.227430742	,	0.29633652	,],
[	0.368646519	,	0.312289329	,	0.074473395	,	0.284544314	,],
[	0.307010371	,	0.376703755	,	0.283796104	,	0.086610775	,]],
[[	0.081012148	,	0.291383248	,	0.298811886	,	0.234718284	,],
[	0.282229394	,	0.080357292	,	0.230063389	,	0.29926094	,],
[	0.365667629	,	0.308436885	,	0.067487186	,	0.281445679	,],
[	0.303995898	,	0.373450558	,	0.280682951	,	0.080280159	,]],
[[	0.084849469	,	0.293105995	,	0.300549668	,	0.236532939	,],
[	0.278397038	,	0.077808319	,	0.226395257	,	0.29563771	,],
[	0.365934236	,	0.309138	,	0.06972861	,	0.281796361	,],
[	0.304120076	,	0.373795474	,	0.280892903	,	0.082163278	,]],
[[	0.082647688	,	0.293575527	,	0.301019357	,	0.236904058	,],
[	0.280836017	,	0.083245775	,	0.229570105	,	0.298131152	,],
[	0.36834013	,	0.311044171	,	0.069748493	,	0.284248715	,],
[	0.306897428	,	0.376221371	,	0.283537211	,	0.082963241	,]],
[[	0.081652975	,	0.291245184	,	0.298712271	,	0.234594984	,],
[	0.279826994	,	0.078388097	,	0.227690793	,	0.2968478	,],
[	0.364999486	,	0.307927995	,	0.067429715	,	0.280671557	,],
[	0.303277971	,	0.372801269	,	0.2799454	,	0.080038061	,]],
[[	0.086925736	,	0.297829316	,	0.305208746	,	0.241360208	,],
[	0.27877222	,	0.079339359	,	0.226947943	,	0.295852503	,],
[	0.367814776	,	0.311251622	,	0.072991387	,	0.28367916	,],
[	0.306010409	,	0.37570904	,	0.282737447	,	0.085161897	,]],
[[	0.082970197	,	0.29666326	,	0.302664749	,	0.235342946	,],
[	0.275009967	,	0.078226078	,	0.226992312	,	0.292573279	,],
[	0.361292799	,	0.309912706	,	0.0674859	,	0.277522943	,],
[	0.301471151	,	0.376729081	,	0.282212535	,	0.079378888	,]],
[[	0.084994775	,	0.30060889	,	0.306413357	,	0.239411702	,],
[	0.274434139	,	0.079940776	,	0.226836074	,	0.29215755	,],
[	0.36423944	,	0.313383528	,	0.073083973	,	0.280455418	,],
[	0.305311576	,	0.380753544	,	0.286215054	,	0.085823787	,]],
[[	0.081297421	,	0.296126705	,	0.302029187	,	0.23479634	,],
[	0.278434609	,	0.081162356	,	0.230187751	,	0.295757186	,],
[	0.362565286	,	0.310892957	,	0.067730759	,	0.278738154	,],
[	0.30223792	,	0.377315093	,	0.282963981	,	0.079313391	,]],
[[	0.085973594	,	0.298399069	,	0.304374761	,	0.237175022	,],
[	0.274000228	,	0.078016186	,	0.226143192	,	0.291592344	,],
[	0.36188702	,	0.310690914	,	0.068866896	,	0.27803719	,],
[	0.302411496	,	0.377728953	,	0.283215593	,	0.081266269	,]],
[[	0.082444499	,	0.297927995	,	0.303867984	,	0.236614114	,],
[	0.276057424	,	0.082931639	,	0.228771856	,	0.293747111	,],
[	0.365726235	,	0.314075831	,	0.070790293	,	0.28211499	,],
[	0.305250311	,	0.380130315	,	0.285936127	,	0.081923595	,]],
[[	0.082304196	,	0.296358195	,	0.302290795	,	0.234996263	,],
[	0.275864849	,	0.078888469	,	0.227691911	,	0.293324629	,],
[	0.361549625	,	0.310021714	,	0.067195602	,	0.277645577	,],
[	0.30159908	,	0.376787726	,	0.282292428	,	0.079103026	,]],
[[	0.086394785	,	0.301892463	,	0.307790134	,	0.240558394	,],
[	0.274154888	,	0.079170084	,	0.22638098	,	0.291759904	,],
[	0.363446221	,	0.312540754	,	0.07167869	,	0.279666159	,],
[	0.304382437	,	0.379716958	,	0.285176212	,	0.08419399	,]],
[[	0.081409895	,	0.296465822	,	0.302382488	,	0.235054027	,],
[	0.278672852	,	0.086932425	,	0.231580386	,	0.296312904	,],
[	0.363198405	,	0.311565162	,	0.068416742	,	0.279389725	,],
[	0.302901407	,	0.377932156	,	0.283606816	,	0.079768148	,]],
[[	0.084742105	,	0.297667303	,	0.303689499	,	0.236336086	,],
[	0.27439076	,	0.077937618	,	0.226314079	,	0.291897075	,],
[	0.361638034	,	0.310350535	,	0.068178996	,	0.277723009	,],
[	0.302012922	,	0.3772513	,	0.2827496	,	0.08025527	,]],
[[	0.083406751	,	0.298859178	,	0.304750441	,	0.237637986	,],
[	0.275419722	,	0.081691745	,	0.227941471	,	0.292997963	,],
[	0.365834263	,	0.31517074	,	0.075456604	,	0.282121057	,],
[	0.306420925	,	0.381339221	,	0.287137514	,	0.083178101	,]],
[[	0.081818397	,	0.296169462	,	0.302119237	,	0.23473135	,],
[	0.276824628	,	0.079700308	,	0.228577517	,	0.294154538	,],
[	0.361758793	,	0.310212161	,	0.067307966	,	0.277932893	,],
[	0.301677561	,	0.376848651	,	0.28244694	,	0.078920716	,]],
[[	0.087944175	,	0.303364128	,	0.309245979	,	0.242215712	,],
[	0.27400478	,	0.078575412	,	0.22611957	,	0.291437054	,],
[	0.362735989	,	0.311676581	,	0.070469466	,	0.278984986	,],
[	0.303613463	,	0.378822554	,	0.284382992	,	0.082846268	,]],
[[	0.082098665	,	0.300946806	,	0.305988253	,	0.234485376	,],
[	0.270682373	,	0.079383652	,	0.227756802	,	0.288895475	,],
[	0.356825772	,	0.31074291	,	0.06710034	,	0.273585942	,],
[	0.29917081	,	0.380145594	,	0.286041151	,	0.079234574	,]],
[[	0.084674438	,	0.305051826	,	0.309999565	,	0.23873488	,],
[	0.267976304	,	0.078320292	,	0.225355844	,	0.286324878	,],
[	0.357810285	,	0.31216672	,	0.070258702	,	0.274420956	,],
[	0.301830739	,	0.383020089	,	0.288810845	,	0.084098966	,]],
[[	0.080575509	,	0.300426868	,	0.305390341	,	0.23394577	,],
[	0.271563403	,	0.085142882	,	0.229714173	,	0.290036293	,],
[	0.35935333	,	0.313120356	,	0.069166891	,	0.276137421	,],
[	0.300591941	,	0.381529228	,	0.287362623	,	0.080016343	,]],
[[	0.084938071	,	0.302545072	,	0.307633107	,	0.236138086	,],
[	0.268748389	,	0.077896407	,	0.225900569	,	0.286996959	,],
[	0.356585072	,	0.310544286	,	0.067503846	,	0.273160042	,],
[	0.299618443	,	0.380696431	,	0.286528619	,	0.080363335	,]],
[[	0.081937746	,	0.302342496	,	0.307331255	,	0.235982241	,],
[	0.268782301	,	0.080403774	,	0.226426243	,	0.287224222	,],
[	0.359781638	,	0.314389131	,	0.07368726	,	0.276546458	,],
[	0.304240169	,	0.38491627	,	0.291000328	,	0.083460489	,]],
[[	0.081402624	,	0.300510341	,	0.305534148	,	0.233967266	,],
[	0.271824752	,	0.080352311	,	0.228859107	,	0.289969651	,],
[	0.357519158	,	0.311349007	,	0.067510529	,	0.274155543	,],
[	0.299435848	,	0.380342137	,	0.286152289	,	0.079124107	,]],
[[	0.086019919	,	0.306510158	,	0.311387389	,	0.240156487	,],
[	0.268021338	,	0.077871809	,	0.2252047	,	0.286266064	,],
[	0.357282354	,	0.311530998	,	0.069220171	,	0.273872783	,],
[	0.3011145	,	0.382252778	,	0.288011981	,	0.082924121	,]],
[[	0.080700361	,	0.300747188	,	0.305729786	,	0.234282385	,],
[	0.270634719	,	0.083573996	,	0.228656365	,	0.289095584	,],
[	0.360525941	,	0.314163492	,	0.070196587	,	0.277198747	,],
[	0.301557539	,	0.382391289	,	0.288341382	,	0.080767835	,]],
[[	0.083805748	,	0.301844023	,	0.306790329	,	0.235307293	,],
[	0.269533147	,	0.078322703	,	0.2265682	,	0.28773306	,],
[	0.356626897	,	0.31056578	,	0.067163702	,	0.273136036	,],
[	0.299366877	,	0.380389373	,	0.286202563	,	0.079720472	,]],
[[	0.082926992	,	0.303410651	,	0.308272251	,	0.23705132	,],
[	0.268447314	,	0.079457744	,	0.225916122	,	0.286669794	,],
[	0.358888879	,	0.313425392	,	0.072161506	,	0.275574962	,],
[	0.303503514	,	0.384588767	,	0.290337048	,	0.086406996	,]],
[[	0.080983276	,	0.300495721	,	0.305326389	,	0.233744976	,],
[	0.273092428	,	0.08153232	,	0.23002045	,	0.291205687	,],
[	0.358151392	,	0.311967884	,	0.068037554	,	0.274853942	,],
[	0.299844186	,	0.380839994	,	0.286556593	,	0.079330622	,]],
[[	0.087290277	,	0.304025257	,	0.309204093	,	0.237656732	,],
[	0.268334298	,	0.077759559	,	0.225459274	,	0.286470126	,],
[	0.356980577	,	0.311007527	,	0.068348699	,	0.273444707	,],
[	0.300398965	,	0.38152513	,	0.287227766	,	0.081725296	,]],
[[	0.081902793	,	0.306221798	,	0.310769357	,	0.234916843	,],
[	0.26555809	,	0.079577237	,	0.228176692	,	0.285170434	,],
[	0.352875058	,	0.312754347	,	0.06766184	,	0.270543769	,],
[	0.298368313	,	0.384767589	,	0.289829636	,	0.079211656	,]],
[[	0.084455349	,	0.310501545	,	0.314934494	,	0.239372408	,],
[	0.262796984	,	0.078569866	,	0.225905556	,	0.282532447	,],
[	0.354286887	,	0.314801328	,	0.071624169	,	0.272111419	,],
[	0.301536953	,	0.388065488	,	0.293148906	,	0.08468766	,]],
[[	0.080418435	,	0.305830939	,	0.310330625	,	0.234455841	,],
[	0.26624731	,	0.085322935	,	0.230056074	,	0.286159493	,],
[	0.354817348	,	0.314552957	,	0.069146469	,	0.272586782	,],
[	0.299518272	,	0.38579588	,	0.290980728	,	0.079597618	,]],
[[	0.084806787	,	0.307896092	,	0.312400504	,	0.23662767	,],
[	0.263634611	,	0.078122924	,	0.226359706	,	0.28323059	,],
[	0.352682089	,	0.312766932	,	0.068316921	,	0.270388932	,],
[	0.298934289	,	0.385503558	,	0.290473165	,	0.080569575	,]],
[[	0.081869907	,	0.307900756	,	0.312310977	,	0.236609359	,],
[	0.263624468	,	0.080569821	,	0.22686196	,	0.283304058	,],
[	0.35652939	,	0.31725656	,	0.075218454	,	0.274381906	,],
[	0.302834138	,	0.388929352	,	0.294338483	,	0.082776661	,]],
[[	0.081286231	,	0.30591096	,	0.31032169	,	0.234471314	,],
[	0.266594269	,	0.080505724	,	0.229283273	,	0.28611309	,],
[	0.353235513	,	0.313058149	,	0.067855542	,	0.270881295	,],
[	0.298516945	,	0.384970875	,	0.289972947	,	0.079049564	,]],
[[	0.085904797	,	0.311919132	,	0.316259496	,	0.240700949	,],
[	0.262950731	,	0.078218663	,	0.225751498	,	0.282545469	,],
[	0.353709723	,	0.314011351	,	0.070440708	,	0.271378097	,],
[	0.300670715	,	0.387290217	,	0.292225361	,	0.083281277	,]],
[[	0.080591538	,	0.306296263	,	0.310685974	,	0.234794113	,],
[	0.265412017	,	0.0838055	,	0.228976184	,	0.285206127	,],
[	0.355942849	,	0.315724161	,	0.070134149	,	0.27378933	,],
[	0.300387103	,	0.386578709	,	0.291797347	,	0.080185748	,]],
[[	0.08355817	,	0.307097801	,	0.311626326	,	0.2357119	,],
[	0.264363549	,	0.078598907	,	0.227022127	,	0.283837753	,],
[	0.352737853	,	0.312708593	,	0.06792292	,	0.270297488	,],
[	0.298713998	,	0.385230463	,	0.290221672	,	0.079911236	,]],
[[	0.082833447	,	0.308866824	,	0.313250002	,	0.237492715	,],
[	0.263212061	,	0.079643406	,	0.226354882	,	0.28292331	,],
[	0.355654043	,	0.316249902	,	0.073781501	,	0.273392826	,],
[	0.303147737	,	0.389726088	,	0.294762483	,	0.086972491	,]],
[[	0.080848525	,	0.305852552	,	0.310240916	,	0.234292	,],
[	0.267956702	,	0.081634037	,	0.230442237	,	0.287392107	,],
[	0.353866135	,	0.313608594	,	0.068165444	,	0.271530515	,],
[	0.298868943	,	0.385219498	,	0.290365571	,	0.079083734	,]],
[[	0.087094181	,	0.309499543	,	0.313970042	,	0.238167675	,],
[	0.263086747	,	0.078018677	,	0.225881768	,	0.282701334	,],
[	0.353260097	,	0.313482652	,	0.069530689	,	0.270874013	,],
[	0.300010752	,	0.38654292	,	0.2915481	,	0.082095514	,]],
[[	0.082485334	,	0.313392004	,	0.316343262	,	0.236089177	,],
[	0.260311444	,	0.080009809	,	0.228472139	,	0.280989196	,],
[	0.34969829	,	0.315877052	,	0.068195492	,	0.267284505	,],
[	0.297856915	,	0.390878351	,	0.293834186	,	0.079583764	,]],
[[	0.084646523	,	0.317331748	,	0.320295464	,	0.240287912	,],
[	0.258164534	,	0.079892187	,	0.226658176	,	0.27891106	,],
[	0.35189376	,	0.318825927	,	0.07308925	,	0.269716194	,],
[	0.302369214	,	0.395304168	,	0.298404882	,	0.086851033	,]],
[[	0.080821405	,	0.312838778	,	0.31584175	,	0.235603864	,],
[	0.262073571	,	0.087283771	,	0.23150215	,	0.282991844	,],
[	0.351195693	,	0.317253897	,	0.068973691	,	0.268953546	,],
[	0.297971728	,	0.390853568	,	0.293953746	,	0.078868823	,]],
[[	0.085350114	,	0.315119009	,	0.31814415	,	0.23790448	,],
[	0.258672467	,	0.079033327	,	0.226846899	,	0.279235179	,],
[	0.349972388	,	0.316427742	,	0.069236538	,	0.267633424	,],
[	0.298943904	,	0.392084378	,	0.295024259	,	0.08183767	,]],
[[	0.082151146	,	0.314805483	,	0.317696826	,	0.237558384	,],
[	0.259227784	,	0.08221106	,	0.227945434	,	0.279971355	,],
[	0.35432675	,	0.321442927	,	0.076992983	,	0.272235977	,],
[	0.300450189	,	0.393189948	,	0.296350982	,	0.080887062	,]],
[[	0.0818305	,	0.313006654	,	0.316050305	,	0.235716853	,],
[	0.261429788	,	0.080957642	,	0.229349451	,	0.281898835	,],
[	0.350053064	,	0.31616705	,	0.068157689	,	0.267633103	,],
[	0.297732876	,	0.390698993	,	0.293668659	,	0.079137777	,]],
[[	0.086137406	,	0.318763745	,	0.321571541	,	0.241655151	,],
[	0.258133357	,	0.079282048	,	0.226460449	,	0.278707765	,],
[	0.351230049	,	0.317942557	,	0.071825347	,	0.268883373	,],
[	0.30137542	,	0.394345279	,	0.297308045	,	0.08526213	,]],
[[	0.081029071	,	0.313202325	,	0.316067733	,	0.235878206	,],
[	0.261175286	,	0.085691461	,	0.230400255	,	0.281982017	,],
[	0.352314399	,	0.318187234	,	0.069788229	,	0.269901216	,],
[	0.298490812	,	0.391334593	,	0.29432271	,	0.079144834	,]],
[[	0.084193444	,	0.314306588	,	0.317318306	,	0.23703957	,],
[	0.259339632	,	0.079270049	,	0.227371831	,	0.279806097	,],
[	0.349789571	,	0.316044574	,	0.068731453	,	0.267288056	,],
[	0.298588971	,	0.391537747	,	0.294474433	,	0.080867698	,]],
[[	0.083049738	,	0.315745481	,	0.3185685	,	0.238456921	,],
[	0.25875351	,	0.081157527	,	0.22742813	,	0.279391323	,],
[	0.353416257	,	0.32040765	,	0.075447695	,	0.27112264	,],
[	0.301566996	,	0.394119884	,	0.297358546	,	0.081904018	,]],
[[	0.081304028	,	0.312875465	,	0.315838361	,	0.235529499	,],
[	0.262611944	,	0.081972252	,	0.230521716	,	0.282961009	,],
[	0.350493394	,	0.316509805	,	0.068359124	,	0.268016814	,],
[	0.297719153	,	0.390514147	,	0.293613738	,	0.078831407	,]],
[[	0.087605967	,	0.320094843	,	0.322944721	,	0.243100083	,],
[	0.258182399	,	0.079058329	,	0.226534666	,	0.278671598	,],
[	0.350749049	,	0.317225074	,	0.070729055	,	0.268227621	,],
[	0.300352229	,	0.393407135	,	0.296405256	,	0.083788525	,]],
[[	0.081627161	,	0.317683842	,	0.319991004	,	0.235706769	,],
[	0.257574407	,	0.082874826	,	0.230363871	,	0.279207496	,],
[	0.345440016	,	0.317350644	,	0.068007611	,	0.26394473	,],
[	0.295680918	,	0.393686312	,	0.296852658	,	0.078538878	,]],
[[	0.084371937	,	0.322130639	,	0.324223309	,	0.240258274	,],
[	0.253250022	,	0.080043938	,	0.226479894	,	0.275095308	,],
[	0.346651756	,	0.319164758	,	0.071700975	,	0.265152562	,],
[	0.299250631	,	0.397312657	,	0.30046063	,	0.084545048	,]],
[[	0.080204684	,	0.317320286	,	0.31946242	,	0.235248041	,],
[	0.255399639	,	0.085135176	,	0.229217152	,	0.27742173	,],
[	0.347655138	,	0.319349168	,	0.069620145	,	0.266246531	,],
[	0.29664589	,	0.394532983	,	0.297831418	,	0.078661362	,]],
[[	0.084499287	,	0.319357613	,	0.321585113	,	0.237348163	,],
[	0.254997922	,	0.080596603	,	0.227865679	,	0.276626758	,],
[	0.345318134	,	0.317398332	,	0.068583359	,	0.263652036	,],
[	0.296361397	,	0.394464511	,	0.297584523	,	0.080094921	,]],
[[	0.081640858	,	0.319334822	,	0.321398559	,	0.237342378	,],
[	0.253451986	,	0.081207146	,	0.22683269	,	0.275206619	,],
[	0.348819719	,	0.321577345	,	0.075299841	,	0.267381372	,],
[	0.299818617	,	0.397352481	,	0.300825041	,	0.081678476	,]],
[[	0.081025972	,	0.31740622	,	0.319544265	,	0.235314548	,],
[	0.25788026	,	0.089130285	,	0.232119459	,	0.279787463	,],
[	0.345886151	,	0.317633417	,	0.068142456	,	0.26427719	,],
[	0.295671116	,	0.393677668	,	0.296918024	,	0.078289213	,]],
[[	0.085773726	,	0.323429485	,	0.325504763	,	0.241555512	,],
[	0.253566883	,	0.079941344	,	0.22657409	,	0.275227424	,],
[	0.346073934	,	0.318425074	,	0.070527398	,	0.264583117	,],
[	0.298279684	,	0.396358327	,	0.299478638	,	0.083007561	,]],
[[	0.08045578	,	0.317763347	,	0.319989681	,	0.235624299	,],
[	0.254624377	,	0.083866956	,	0.228421811	,	0.276575565	,],
[	0.348782838	,	0.320495817	,	0.070577968	,	0.26727587	,],
[	0.297419006	,	0.395063581	,	0.298501634	,	0.079266457	,]],
[[	0.083333577	,	0.318659991	,	0.320753546	,	0.236539773	,],
[	0.256024968	,	0.081436372	,	0.228837282	,	0.277566436	,],
[	0.345285178	,	0.317254523	,	0.068191275	,	0.263588434	,],
[	0.296105194	,	0.394192343	,	0.29730491	,	0.079350314	,]],
[[	0.082718282	,	0.320433743	,	0.322527038	,	0.238426131	,],
[	0.253320791	,	0.080613594	,	0.226606486	,	0.274977956	,],
[	0.347946208	,	0.320568809	,	0.073789577	,	0.266479381	,],
[	0.301020213	,	0.39899776	,	0.3022488	,	0.086916406	,]],
[[	0.080604974	,	0.317312192	,	0.319436463	,	0.235073038	,],
[	0.256839047	,	0.087537197	,	0.230946227	,	0.278695014	,],
[	0.346562875	,	0.318354104	,	0.068703306	,	0.264903381	,],
[	0.296048181	,	0.393946816	,	0.297058001	,	0.07829601	,]],
[[	0.086847944	,	0.320920971	,	0.323103452	,	0.238843746	,],
[	0.254048824	,	0.080132601	,	0.226911968	,	0.275620958	,],
[	0.34574641	,	0.317903237	,	0.069651606	,	0.264100836	,],
[	0.297502072	,	0.395522248	,	0.298657137	,	0.081812044	,]],
[[	0.081483537	,	0.323370959	,	0.324890355	,	0.235869078	,],
[	0.253057555	,	0.090146238	,	0.233135697	,	0.275709298	,],
[	0.342830671	,	0.320425069	,	0.069006469	,	0.261646807	,],
[	0.295336913	,	0.399027099	,	0.301187617	,	0.078649088	,]],
[[	0.083794681	,	0.327521335	,	0.328940355	,	0.240271958	,],
[	0.248614033	,	0.080496267	,	0.227305308	,	0.270761238	,],
[	0.342220262	,	0.32040514	,	0.070287059	,	0.260983897	,],
[	0.298525548	,	0.402356933	,	0.30449263	,	0.084143345	,]],
[[	0.0797503	,	0.322866565	,	0.324375259	,	0.235458175	,],
[	0.250010581	,	0.084756598	,	0.229413509	,	0.272340171	,],
[	0.346137849	,	0.323689812	,	0.071916041	,	0.265052136	,],
[	0.296685614	,	0.400084129	,	0.30229145	,	0.079060738	,]],
[[	0.084220502	,	0.325187286	,	0.326653708	,	0.237691056	,],
[	0.250876262	,	0.081813273	,	0.229265786	,	0.272942679	,],
[	0.341843321	,	0.319713254	,	0.06848152	,	0.260547402	,],
[	0.296023597	,	0.399781235	,	0.30177799	,	0.079987275	,]],
[[	0.081174714	,	0.324811808	,	0.326248115	,	0.237396263	,],
[	0.248437802	,	0.081353259	,	0.227436107	,	0.270745639	,],
[	0.343919024	,	0.322322404	,	0.073136251	,	0.262627473	,],
[	0.299998017	,	0.403114226	,	0.305549914	,	0.082236317	,]],
[[	0.080764772	,	0.323065603	,	0.32457889	,	0.235553265	,],
[	0.252102048	,	0.088438289	,	0.23183575	,	0.274576545	,],
[	0.343727135	,	0.321264863	,	0.069689674	,	0.262419749	,],
[	0.295432223	,	0.399034641	,	0.301104149	,	0.078383169	,]],
[[	0.085201272	,	0.328902012	,	0.330269261	,	0.241576517	,],
[	0.249051705	,	0.08067077	,	0.227743927	,	0.271207078	,],
[	0.342001462	,	0.32005499	,	0.069542286	,	0.26071718	,],
[	0.297693864	,	0.40135231	,	0.303446526	,	0.082730257	,]],
[[	0.079932809	,	0.32328201	,	0.324647828	,	0.23576249	,],
[	0.249377006	,	0.083560268	,	0.22862588	,	0.271669232	,],
[	0.346212372	,	0.324934163	,	0.077068738	,	0.265146477	,],
[	0.297429214	,	0.400710787	,	0.303061479	,	0.079712909	,]],
[[	0.083170483	,	0.324459325	,	0.325871486	,	0.236811419	,],
[	0.251878717	,	0.082716101	,	0.230337965	,	0.273953097	,],
[	0.342269173	,	0.3199384	,	0.068584399	,	0.260924466	,],
[	0.295612212	,	0.399394908	,	0.301491349	,	0.079274075	,]],
[[	0.082066256	,	0.325775275	,	0.327211935	,	0.238389752	,],
[	0.248475108	,	0.080868328	,	0.227210416	,	0.270623342	,],
[	0.34321945	,	0.321522767	,	0.071917803	,	0.261908584	,],
[	0.300259347	,	0.403897324	,	0.306018222	,	0.086462928	,]],
[[	0.080285379	,	0.322896096	,	0.324421732	,	0.235292921	,],
[	0.251248771	,	0.086896722	,	0.230774614	,	0.273530979	,],
[	0.34465507	,	0.322152283	,	0.070354306	,	0.263377325	,],
[	0.295869928	,	0.399427176	,	0.301439167	,	0.078550486	,]],
[[	0.086671974	,	0.326764345	,	0.32824329	,	0.239210723	,],
[	0.249782535	,	0.08103006	,	0.228352721	,	0.271785849	,],
[	0.341799865	,	0.319731153	,	0.06897885	,	0.260475968	,],
[	0.296902272	,	0.400650452	,	0.302734751	,	0.081561137	,]],
[[	0.08154611	,	0.328691579	,	0.329490832	,	0.236735944	,],
[	0.246582891	,	0.088065789	,	0.23178276	,	0.270383995	,],
[	0.340153421	,	0.322832584	,	0.069569311	,	0.258782074	,],
[	0.293873569	,	0.403279164	,	0.304458858	,	0.078244579	,]],
[[	0.083629292	,	0.332529911	,	0.333143215	,	0.24083176	,],
[	0.243965292	,	0.080767557	,	0.227867494	,	0.267358724	,],
[	0.338973896	,	0.322221105	,	0.070156986	,	0.257582644	,],
[	0.296985121	,	0.406551072	,	0.307731851	,	0.083759533	,]],
[[	0.079887058	,	0.328184097	,	0.328933225	,	0.236315229	,],
[	0.243934636	,	0.08335913	,	0.228590221	,	0.267639593	,],
[	0.343506899	,	0.327520106	,	0.078154339	,	0.262399901	,],
[	0.295225016	,	0.404442923	,	0.305630134	,	0.078859043	,]],
[[	0.084470328	,	0.330488412	,	0.331188993	,	0.238563036	,],
[	0.247078699	,	0.083043383	,	0.230737679	,	0.27031176	,],
[	0.338890126	,	0.321703431	,	0.068859355	,	0.257528267	,],
[	0.294469374	,	0.404030789	,	0.305036553	,	0.079673645	,]],
[[	0.081126183	,	0.329998429	,	0.330728704	,	0.238098898	,],
[	0.243155857	,	0.080825275	,	0.227326169	,	0.2666019	,],
[	0.34034251	,	0.323868704	,	0.072853046	,	0.259033615	,],
[	0.298546844	,	0.407553629	,	0.308969546	,	0.08204634	,]],
[[	0.080941118	,	0.328376375	,	0.329147182	,	0.236421578	,],
[	0.2457641	,	0.086668186	,	0.230602156	,	0.269406074	,],
[	0.340940284	,	0.323557665	,	0.07020239	,	0.259577091	,],
[	0.294055249	,	0.403402376	,	0.304550451	,	0.078111027	,]],
[[	0.085048331	,	0.333969106	,	0.334615047	,	0.242269207	,],
[	0.244810094	,	0.081223666	,	0.228609322	,	0.268020008	,],
[	0.33884816	,	0.321815644	,	0.069565565	,	0.257351211	,],
[	0.296091825	,	0.405680285	,	0.306801948	,	0.08236238	,]],
[[	0.080005518	,	0.328541815	,	0.329274951	,	0.236581175	,],
[	0.243624578	,	0.08238015	,	0.228089105	,	0.267036177	,],
[	0.342570061	,	0.326356269	,	0.076493109	,	0.261229603	,],
[	0.296002362	,	0.405083333	,	0.306355097	,	0.079405824	,]],
[[	0.083253473	,	0.329773922	,	0.330498828	,	0.237705307	,],
[	0.248292239	,	0.090767186	,	0.233564616	,	0.271947573	,],
[	0.339337319	,	0.32218151	,	0.068928643	,	0.257933923	,],
[	0.294139707	,	0.403635586	,	0.304697321	,	0.078960665	,]],
[[	0.082093597	,	0.331029301	,	0.331598933	,	0.239108701	,],
[	0.243436736	,	0.080673343	,	0.227468632	,	0.266884736	,],
[	0.339819719	,	0.323121188	,	0.071646751	,	0.25841277	,],
[	0.298638907	,	0.408191392	,	0.309280087	,	0.086039854	,]],
[[	0.080368222	,	0.328134131	,	0.328909684	,	0.23620496	,],
[	0.245015287	,	0.085244433	,	0.229705238	,	0.268532805	,],
[	0.341919018	,	0.324556767	,	0.07108106	,	0.260591509	,],
[	0.294465223	,	0.40370442	,	0.304883861	,	0.078258048	,]],
[[	0.086536802	,	0.335366116	,	0.335923007	,	0.243648529	,],
[	0.245501147	,	0.081773171	,	0.22933033	,	0.268804862	,],
[	0.338729809	,	0.321709648	,	0.069061568	,	0.257297189	,],
[	0.295388516	,	0.40494818	,	0.305986943	,	0.081165582	,]],
[[	0.081119312	,	0.332951245	,	0.332474636	,	0.236508097	,],
[	0.239936665	,	0.08715039	,	0.230991192	,	0.265182814	,],
[	0.33471148	,	0.323781318	,	0.069660821	,	0.254852303	,],
[	0.291920984	,	0.406555238	,	0.307963213	,	0.078752615	,]],
[[	0.083532043	,	0.337092984	,	0.336420984	,	0.240846748	,],
[	0.238195604	,	0.080848686	,	0.228080026	,	0.263005167	,],
[	0.333850164	,	0.323475287	,	0.070830158	,	0.254028311	,],
[	0.295058913	,	0.409708024	,	0.311192697	,	0.08402952	,]],
[[	0.079498761	,	0.332409413	,	0.33188086	,	0.235971196	,],
[	0.237659508	,	0.082741738	,	0.228171755	,	0.262667024	,],
[	0.338032403	,	0.326877716	,	0.072707203	,	0.258354033	,],
[	0.293271665	,	0.407555441	,	0.309114169	,	0.079174415	,]],
[[	0.084053114	,	0.334718545	,	0.334232356	,	0.238207382	,],
[	0.24169694	,	0.083557333	,	0.231323338	,	0.266393221	,],
[	0.333502767	,	0.322803529	,	0.069103061	,	0.253690131	,],
[	0.29264377	,	0.407159365	,	0.30856145	,	0.080048598	,]],
[[	0.080960787	,	0.334374859	,	0.33381169	,	0.238054374	,],
[	0.237207486	,	0.080550005	,	0.227299268	,	0.262155905	,],
[	0.335309556	,	0.325324351	,	0.073687285	,	0.255501247	,],
[	0.296713808	,	0.410730703	,	0.312426667	,	0.082358911	,]],
[[	0.080570808	,	0.332634246	,	0.332115892	,	0.236180706	,],
[	0.239220335	,	0.085731275	,	0.230085072	,	0.264270573	,],
[	0.335412263	,	0.324415291	,	0.070293659	,	0.255570787	,],
[	0.292214758	,	0.406598957	,	0.308006173	,	0.078520773	,]],
[[	0.084932994	,	0.338245744	,	0.337741136	,	0.24221665	,],
[	0.23908615	,	0.081421732	,	0.228803728	,	0.263788445	,],
[	0.333617576	,	0.323114045	,	0.069945786	,	0.253638579	,],
[	0.294188957	,	0.408801177	,	0.310190852	,	0.082786743	,]],
[[	0.079758605	,	0.332826843	,	0.332249133	,	0.23637823	,],
[	0.237327208	,	0.081834301	,	0.227685981	,	0.262273176	,],
[	0.337780956	,	0.328081156	,	0.077650945	,	0.258076484	,],
[	0.294090115	,	0.408243406	,	0.309920299	,	0.079880241	,]],
[[	0.08286358	,	0.33391291	,	0.333447066	,	0.237486845	,],
[	0.241626408	,	0.089661211	,	0.232821318	,	0.266742703	,],
[	0.333946662	,	0.323033716	,	0.069219948	,	0.254001123	,],
[	0.292336647	,	0.406858978	,	0.308209247	,	0.079344778	,]],
[[	0.081844633	,	0.335312206	,	0.334695854	,	0.23891949	,],
[	0.237573633	,	0.080556529	,	0.227437906	,	0.262332769	,],
[	0.33469132	,	0.324477819	,	0.072388558	,	0.254845641	,],
[	0.296674002	,	0.411283498	,	0.3128358	,	0.086421345	,]],
[[	0.080058787	,	0.332443802	,	0.331913945	,	0.235986029	,],
[	0.23855646	,	0.084453262	,	0.229178079	,	0.26352177	,],
[	0.336427899	,	0.325349802	,	0.071081103	,	0.256558484	,],
[	0.292626561	,	0.4069698	,	0.308350031	,	0.078601945	,]],
[[	0.086513684	,	0.339830049	,	0.339170229	,	0.243687875	,],
[	0.240043704	,	0.082146149	,	0.229678575	,	0.264745879	,],
[	0.333494673	,	0.32289003	,	0.069503721	,	0.253553058	,],
[	0.293483236	,	0.408129937	,	0.309454309	,	0.081540099	,]],
[[	0.096302603	,	0.217622085	,	0.247443848	,	0.256109372	,],
[	0.371228565	,	0.081574548	,	0.243444005	,	0.382208892	,],
[	0.44325405	,	0.294498673	,	0.07139781	,	0.354225641	,],
[	0.335296552	,	0.311705566	,	0.231215343	,	0.086694546	,]],
[[	0.090915004	,	0.216264572	,	0.245716172	,	0.253991979	,],
[	0.368849199	,	0.075825792	,	0.240792815	,	0.379655391	,],
[	0.43938945	,	0.290205085	,	0.067641734	,	0.350090807	,],
[	0.337228364	,	0.313305093	,	0.232824998	,	0.090517912	,]],
[[	0.091955185	,	0.215155933	,	0.244861274	,	0.253349525	,],
[	0.368632339	,	0.07724043	,	0.240527979	,	0.379453838	,],
[	0.441473241	,	0.292415818	,	0.07175262	,	0.352150352	,],
[	0.337234511	,	0.313797608	,	0.233286308	,	0.088167336	,]],
[[	0.094197587	,	0.220027697	,	0.249414563	,	0.257574337	,],
[	0.372072393	,	0.078790633	,	0.244079159	,	0.382787074	,],
[	0.440959316	,	0.292075929	,	0.068974713	,	0.351716187	,],
[	0.335339039	,	0.311573612	,	0.231094003	,	0.087360751	,]],
[[	0.090282627	,	0.214905708	,	0.244525	,	0.252817781	,],
[	0.367980711	,	0.075382937	,	0.239833955	,	0.378732255	,],
[	0.439616511	,	0.290378633	,	0.068410839	,	0.350120717	,],
[	0.339746218	,	0.315531805	,	0.235187574	,	0.094003297	,]],
[[	0.094896439	,	0.216772794	,	0.246460595	,	0.255149385	,],
[	0.370400346	,	0.080098478	,	0.24243408	,	0.381301633	,],
[	0.443886661	,	0.29501036	,	0.075371674	,	0.354560072	,],
[	0.335675564	,	0.311946876	,	0.231554653	,	0.086816654	,]],
[[	0.091664706	,	0.217212133	,	0.246714004	,	0.254885842	,],
[	0.369627067	,	0.076386615	,	0.241514304	,	0.380304493	,],
[	0.439677871	,	0.290537553	,	0.067774534	,	0.350323386	,],
[	0.336644951	,	0.312741647	,	0.232238897	,	0.089348393	,]],
[[	0.091309369	,	0.214924717	,	0.24451845	,	0.252913494	,],
[	0.368326653	,	0.076432054	,	0.240179725	,	0.379066053	,],
[	0.440695624	,	0.291636343	,	0.07061259	,	0.35128705	,],
[	0.338202443	,	0.31471331	,	0.23438219	,	0.089039134	,]],
[[	0.095610738	,	0.221528932	,	0.25087147	,	0.258870963	,],
[	0.372967105	,	0.083969746	,	0.245182558	,	0.383737521	,],
[	0.441906296	,	0.292984035	,	0.069848811	,	0.352669673	,],
[	0.335283929	,	0.311528726	,	0.230994435	,	0.086872445	,]],
[[	0.090474423	,	0.215391968	,	0.244860496	,	0.253198737	,],
[	0.368296319	,	0.075502808	,	0.240141837	,	0.379047462	,],
[	0.439474508	,	0.290230824	,	0.067987552	,	0.349984444	,],
[	0.338712086	,	0.314573084	,	0.234129273	,	0.092497855	,]],
[[	0.093624193	,	0.216015499	,	0.24575061	,	0.254252834	,],
[	0.369645729	,	0.078843288	,	0.241608331	,	0.380461149	,],
[	0.442917261	,	0.293890928	,	0.07389497	,	0.353509856	,],
[	0.33616723	,	0.312681026	,	0.232178043	,	0.087142903	,]],
[[	0.092668592	,	0.218227665	,	0.247621705	,	0.255749113	,],
[	0.370619166	,	0.077284783	,	0.242433968	,	0.38119941	,],
[	0.440147605	,	0.291052189	,	0.06804323	,	0.350776065	,],
[	0.335968602	,	0.312173374	,	0.231543364	,	0.088388766	,]],
[[	0.092430552	,	0.21815745	,	0.246268363	,	0.250077987	,],
[	0.356766021	,	0.075639818	,	0.235830818	,	0.368182295	,],
[	0.429321046	,	0.288443224	,	0.070166874	,	0.341686213	,],
[	0.328475186	,	0.312045466	,	0.231767581	,	0.085485789	,]],
[[	0.089006229	,	0.218298174	,	0.246216235	,	0.249544938	,],
[	0.361432498	,	0.078780447	,	0.24056678	,	0.372644942	,],
[	0.429567776	,	0.288792883	,	0.068438879	,	0.342292152	,],
[	0.327341337	,	0.310762461	,	0.230268965	,	0.085610548	,]],
[[	0.088857261	,	0.216095524	,	0.244265405	,	0.247776153	,],
[	0.356762797	,	0.074440473	,	0.235656539	,	0.368032118	,],
[	0.427732898	,	0.286696993	,	0.067280378	,	0.34005505	,],
[	0.331984877	,	0.315797512	,	0.235606115	,	0.089007062	,]],
[[	0.092909414	,	0.222589207	,	0.250275353	,	0.253569245	,],
[	0.358319904	,	0.078223409	,	0.237542276	,	0.369635989	,],
[	0.431571583	,	0.290869911	,	0.073665586	,	0.34407867	,],
[	0.327156815	,	0.310698363	,	0.230242301	,	0.084565298	,]],
[[	0.087938697	,	0.21649624	,	0.244535117	,	0.247914991	,],
[	0.358777864	,	0.076079211	,	0.237800307	,	0.370000172	,],
[	0.428222961	,	0.287166196	,	0.067012609	,	0.340632068	,],
[	0.328951786	,	0.312109153	,	0.23169333	,	0.088115548	,]],
[[	0.091180688	,	0.217339863	,	0.245459373	,	0.249115272	,],
[	0.356647327	,	0.075127246	,	0.235554195	,	0.367918672	,],
[	0.428788152	,	0.287851447	,	0.069124242	,	0.341037735	,],
[	0.329332006	,	0.313062181	,	0.232644514	,	0.086325015	,]],
[[	0.089930187	,	0.219239604	,	0.247088112	,	0.250447851	,],
[	0.360562384	,	0.081834055	,	0.2399439	,	0.371960187	,],
[	0.430716211	,	0.289873538	,	0.069385143	,	0.343218978	,],
[	0.326983166	,	0.310616051	,	0.230041353	,	0.085051426	,]],
[[	0.088372682	,	0.216099317	,	0.244082955	,	0.24749915	,],
[	0.35710022	,	0.074727987	,	0.236021151	,	0.368330119	,],
[	0.427662424	,	0.286596551	,	0.066862163	,	0.33993971	,],
[	0.331215982	,	0.314314489	,	0.233965505	,	0.091520574	,]],
[[	0.094523538	,	0.224122024	,	0.25174264	,	0.255099465	,],
[	0.357725719	,	0.077126066	,	0.236784443	,	0.36900814	,],
[	0.43061596	,	0.289849894	,	0.072220395	,	0.343013412	,],
[	0.327660226	,	0.311252441	,	0.230759748	,	0.084728122	,]],
[[	0.08827599	,	0.21723534	,	0.245082686	,	0.248400662	,],
[	0.359911292	,	0.077020747	,	0.238801805	,	0.371022107	,],
[	0.428685648	,	0.287784827	,	0.067366565	,	0.3410096	,],
[	0.328174771	,	0.311460468	,	0.230965726	,	0.086902106	,]],
[[	0.09014516	,	0.216879311	,	0.24493048	,	0.248471349	,],
[	0.356597339	,	0.074746611	,	0.23557575	,	0.36781995	,],
[	0.428309673	,	0.287229404	,	0.06832617	,	0.340605355	,],
[	0.330328881	,	0.314012862	,	0.233741699	,	0.087276036	,]],
[[	0.091049067	,	0.220587964	,	0.248304121	,	0.251592922	,],
[	0.359637664	,	0.080331416	,	0.238883862	,	0.370944545	,],
[	0.43177512	,	0.291021081	,	0.070594958	,	0.344294472	,],
[	0.327028847	,	0.310573895	,	0.230049108	,	0.084683132	,]],
[[	0.089991534	,	0.220974596	,	0.247949183	,	0.246485001	,],
[	0.348749375	,	0.074545049	,	0.233958079	,	0.361324588	,],
[	0.420355733	,	0.285763319	,	0.067243631	,	0.334199233	,],
[	0.325458991	,	0.315087881	,	0.235680786	,	0.086225142	,]],
[[	0.088785584	,	0.223042641	,	0.249651635	,	0.247791714	,],
[	0.351361179	,	0.079536339	,	0.23694279	,	0.364074085	,],
[	0.424658988	,	0.290513206	,	0.070607153	,	0.338965389	,],
[	0.32216663	,	0.311729427	,	0.231964484	,	0.083687163	,]],
[[	0.087035898	,	0.219758022	,	0.246687635	,	0.244962632	,],
[	0.350112211	,	0.075340157	,	0.235314375	,	0.362683224	,],
[	0.42018199	,	0.285576198	,	0.066193087	,	0.334063672	,],
[	0.325492324	,	0.314775549	,	0.235062181	,	0.089245107	,]],
[[	0.093354867	,	0.227789165	,	0.254246591	,	0.252328523	,],
[	0.349143986	,	0.075761461	,	0.234544072	,	0.361794179	,],
[	0.42183363	,	0.287300973	,	0.069699816	,	0.335683264	,],
[	0.323232268	,	0.312870267	,	0.233321623	,	0.084111025	,]],
[[	0.087081096	,	0.220819488	,	0.247594404	,	0.245757069	,],
[	0.353684098	,	0.078762287	,	0.239047446	,	0.366132091	,],
[	0.422276971	,	0.287766308	,	0.067801774	,	0.336230139	,],
[	0.322835144	,	0.31217981	,	0.23250868	,	0.085139447	,]],
[[	0.088992775	,	0.220546904	,	0.247464046	,	0.245865038	,],
[	0.348876392	,	0.074552708	,	0.234066353	,	0.361525045	,],
[	0.420207306	,	0.285552226	,	0.066696143	,	0.33393907	,],
[	0.326633805	,	0.316326433	,	0.236874472	,	0.087302021	,]],
[[	0.089985438	,	0.224241362	,	0.250777174	,	0.248969377	,],
[	0.350639182	,	0.078330718	,	0.23598704	,	0.363233111	,],
[	0.424058779	,	0.289816382	,	0.073208035	,	0.337971942	,],
[	0.32228891	,	0.3119146	,	0.232197376	,	0.083642315	,]],
[[	0.086876343	,	0.219780562	,	0.246609943	,	0.244798461	,],
[	0.351026137	,	0.076012182	,	0.236256572	,	0.363462914	,],
[	0.420742815	,	0.285958466	,	0.066382474	,	0.334494825	,],
[	0.324618703	,	0.313923393	,	0.234208809	,	0.087851538	,]],
[[	0.092097558	,	0.222398115	,	0.249315974	,	0.247912116	,],
[	0.34901355	,	0.075196106	,	0.234105208	,	0.361499779	,],
[	0.421233247	,	0.286624657	,	0.068585416	,	0.334960311	,],
[	0.324095142	,	0.313698233	,	0.234091855	,	0.08479698	,]],
[[	0.087662206	,	0.221656493	,	0.248267116	,	0.246503888	,],
[	0.353015233	,	0.08200188	,	0.238543134	,	0.365574604	,],
[	0.423204214	,	0.288779634	,	0.068816605	,	0.337160804	,],
[	0.32254668	,	0.312014244	,	0.232214177	,	0.084418069	,]],
[[	0.08814786	,	0.220040679	,	0.246978255	,	0.245378784	,],
[	0.349362537	,	0.074676351	,	0.234437696	,	0.361894574	,],
[	0.420146773	,	0.285453713	,	0.066296923	,	0.333916126	,],
[	0.32732624	,	0.316324454	,	0.236784719	,	0.091566653	,]],
[[	0.091153717	,	0.225528259	,	0.252058175	,	0.250163799	,],
[	0.350023711	,	0.077285237	,	0.235306028	,	0.362629165	,],
[	0.423170966	,	0.288697115	,	0.07164476	,	0.336887653	,],
[	0.322625372	,	0.31207812	,	0.232403035	,	0.083555047	,]],
[[	0.088530781	,	0.224442477	,	0.249774129	,	0.243837058	,],
[	0.340758594	,	0.073920269	,	0.231710428	,	0.354125261	,],
[	0.411758752	,	0.283044817	,	0.065117122	,	0.326895426	,],
[	0.322212499	,	0.317792298	,	0.238561863	,	0.086813799	,]],
[[	0.086659536	,	0.225814845	,	0.250946654	,	0.244614893	,],
[	0.341201275	,	0.076387836	,	0.232327746	,	0.354739489	,],
[	0.414806185	,	0.286409587	,	0.07058236	,	0.329984123	,],
[	0.317427321	,	0.312950363	,	0.233468655	,	0.082559366	,]],
[[	0.085439473	,	0.222873746	,	0.248195119	,	0.242057531	,],
[	0.343388037	,	0.076207962	,	0.234392807	,	0.35670952	,],
[	0.412838048	,	0.284192787	,	0.065541024	,	0.328028514	,],
[	0.319226179	,	0.314559922	,	0.235070793	,	0.086390541	,]],
[[	0.091094411	,	0.230493897	,	0.255458094	,	0.249210673	,],
[	0.340121939	,	0.07399497	,	0.231041555	,	0.353593518	,],
[	0.412463925	,	0.283726609	,	0.066504061	,	0.327458424	,],
[	0.319523124	,	0.315053722	,	0.235773095	,	0.084268656	,]],
[[	0.085098662	,	0.223697867	,	0.248844004	,	0.242624818	,],
[	0.343233448	,	0.079617793	,	0.234522512	,	0.356752408	,],
[	0.415995763	,	0.287633448	,	0.068732995	,	0.331366249	,],
[	0.31744358	,	0.312892248	,	0.233340489	,	0.083305167	,]],
[[	0.087327943	,	0.223757238	,	0.249100127	,	0.243079677	,],
[	0.341307614	,	0.074362129	,	0.232280019	,	0.354703944	,],
[	0.411977055	,	0.283214264	,	0.064950799	,	0.327042503	,],
[	0.321803835	,	0.316887203	,	0.237418645	,	0.089942633	,]],
[[	0.08776653	,	0.226968032	,	0.251953939	,	0.245642635	,],
[	0.340800629	,	0.075557743	,	0.231778517	,	0.354211183	,],
[	0.414093394	,	0.285567896	,	0.069206916	,	0.329017258	,],
[	0.317866385	,	0.313372099	,	0.23386983	,	0.082826448	,]],
[[	0.084977937	,	0.222915538	,	0.248241134	,	0.241934455	,],
[	0.34459219	,	0.077398565	,	0.235551419	,	0.357833959	,],
[	0.413691446	,	0.285017937	,	0.066205848	,	0.32878989	,],
[	0.318623568	,	0.313855832	,	0.234345029	,	0.085297032	,]],
[[	0.090610001	,	0.225723411	,	0.251072085	,	0.245158501	,],
[	0.340397705	,	0.073839996	,	0.23114472	,	0.353691077	,],
[	0.412163821	,	0.283432344	,	0.065777312	,	0.327083557	,],
[	0.320553545	,	0.316086321	,	0.236778827	,	0.085076862	,]],
[[	0.085592704	,	0.224438563	,	0.249639351	,	0.243247204	,],
[	0.342437767	,	0.078351072	,	0.233539582	,	0.355738578	,],
[	0.41639903	,	0.288184243	,	0.072967809	,	0.331626048	,],
[	0.31741329	,	0.312750641	,	0.233263278	,	0.082824974	,]],
[[	0.086492159	,	0.223281806	,	0.248661518	,	0.24248356	,],
[	0.342075494	,	0.074963363	,	0.232915218	,	0.355378946	,],
[	0.412245907	,	0.283539557	,	0.06502861	,	0.327354963	,],
[	0.320791227	,	0.31591268	,	0.236529395	,	0.088467359	,]],
[[	0.088971558	,	0.228329112	,	0.253250625	,	0.246941318	,],
[	0.340505358	,	0.074830074	,	0.231410713	,	0.353835609	,],
[	0.413383313	,	0.284803349	,	0.068015769	,	0.32837546	,],
[	0.318406184	,	0.313992857	,	0.23446427	,	0.083205962	,]],
[[	0.087752343	,	0.228429306	,	0.253008904	,	0.243014343	,],
[	0.334410488	,	0.073324745	,	0.230625839	,	0.34888108	,],
[	0.405961612	,	0.282656158	,	0.065386999	,	0.322351481	,],
[	0.318836602	,	0.319058884	,	0.240566861	,	0.090373803	,]],
[[	0.085922748	,	0.229761029	,	0.254119599	,	0.243770928	,],
[	0.334887734	,	0.07580161	,	0.231259358	,	0.349433849	,],
[	0.40825555	,	0.285243437	,	0.069942054	,	0.32458163	,],
[	0.31388612	,	0.314525925	,	0.235890284	,	0.081963431	,]],
[[	0.084609953	,	0.226824618	,	0.251364944	,	0.241218721	,],
[	0.337294939	,	0.075713074	,	0.233578572	,	0.351752003	,],
[	0.407461562	,	0.284202424	,	0.066464374	,	0.323787447	,],
[	0.315347185	,	0.31576378	,	0.23711369	,	0.085243565	,]],
[[	0.090206251	,	0.234305262	,	0.258541364	,	0.248260702	,],
[	0.333900387	,	0.073334439	,	0.230122644	,	0.348256305	,],
[	0.406315958	,	0.283068802	,	0.066436417	,	0.322515115	,],
[	0.316334374	,	0.317003991	,	0.238504854	,	0.083880951	,]],
[[	0.084347506	,	0.227696262	,	0.252075032	,	0.241887028	,],
[	0.336942107	,	0.079108539	,	0.233537235	,	0.351524903	,],
[	0.410810422	,	0.287970121	,	0.073913558	,	0.327142868	,],
[	0.31369134	,	0.314281201	,	0.235577776	,	0.082358749	,]],
[[	0.086739505	,	0.227788073	,	0.252366475	,	0.242316463	,],
[	0.335197474	,	0.073756179	,	0.23125253	,	0.349499815	,],
[	0.40630025	,	0.283000643	,	0.065463688	,	0.322566379	,],
[	0.31770863	,	0.317929009	,	0.239397648	,	0.088619487	,]],
[[	0.087043277	,	0.230943391	,	0.255182934	,	0.244909791	,],
[	0.334485108	,	0.074960212	,	0.230815126	,	0.34909234	,],
[	0.407625035	,	0.284539225	,	0.068774229	,	0.323870806	,],
[	0.314380546	,	0.314973966	,	0.236393371	,	0.082159876	,]],
[[	0.084322995	,	0.226840696	,	0.25142208	,	0.24119195	,],
[	0.338523983	,	0.076988204	,	0.234691672	,	0.352764716	,],
[	0.408402576	,	0.285212893	,	0.067182858	,	0.324758062	,],
[	0.314745014	,	0.315130963	,	0.236452355	,	0.084115171	,]],
[[	0.089907897	,	0.229693238	,	0.254227531	,	0.244412492	,],
[	0.333986795	,	0.073209437	,	0.230100643	,	0.348447811	,],
[	0.406073423	,	0.282806949	,	0.065897311	,	0.32224154	,],
[	0.317297987	,	0.31793169	,	0.239506088	,	0.084760918	,]],
[[	0.08493511	,	0.22845496	,	0.252800493	,	0.242462987	,],
[	0.336119947	,	0.077676735	,	0.232608062	,	0.350614489	,],
[	0.409723774	,	0.286766475	,	0.072202792	,	0.326066825	,],
[	0.313753207	,	0.314280808	,	0.235590977	,	0.081947604	,]],
[[	0.085770144	,	0.227277163	,	0.251790157	,	0.24159383	,],
[	0.336070393	,	0.074519794	,	0.232057296	,	0.350288686	,],
[	0.406719693	,	0.283416233	,	0.065699256	,	0.322921344	,],
[	0.316756461	,	0.316980248	,	0.238389493	,	0.087157433	,]],
[[	0.088261	,	0.232262631	,	0.256437674	,	0.246162216	,],
[	0.334157434	,	0.074110969	,	0.230365852	,	0.348641942	,],
[	0.406965977	,	0.283809877	,	0.067731437	,	0.323133961	,],
[	0.315039144	,	0.315671961	,	0.237072346	,	0.082684858	,]],
[[	0.086101771	,	0.231418426	,	0.255484917	,	0.240973494	,],
[	0.330059529	,	0.07441501	,	0.231032881	,	0.344802156	,],
[	0.401004781	,	0.284054982	,	0.065016407	,	0.318123183	,],
[	0.315200567	,	0.321638331	,	0.242258949	,	0.089373683	,]],
[[	0.08696964	,	0.234990329	,	0.258703252	,	0.244033729	,],
[	0.328096848	,	0.074125582	,	0.229251509	,	0.34306948	,],
[	0.401552845	,	0.284746455	,	0.067479747	,	0.318752009	,],
[	0.311241915	,	0.317967681	,	0.23842589	,	0.08182983	,]],
[[	0.083898893	,	0.230590704	,	0.25459854	,	0.240002754	,],
[	0.332456543	,	0.081273697	,	0.234073779	,	0.347446921	,],
[	0.403383587	,	0.286745006	,	0.067268244	,	0.320704106	,],
[	0.312000226	,	0.3186606	,	0.239103771	,	0.084540657	,]],
[[	0.089274995	,	0.233354269	,	0.257465809	,	0.243024795	,],
[	0.328419142	,	0.07317755	,	0.229447369	,	0.343294326	,],
[	0.400471068	,	0.283616822	,	0.065093332	,	0.317622468	,],
[	0.313748199	,	0.320465561	,	0.24112171	,	0.084080222	,]],
[[	0.084665671	,	0.232347363	,	0.256231573	,	0.24147313	,],
[	0.32938926	,	0.07630145	,	0.230593544	,	0.344199105	,],
[	0.403581373	,	0.287096134	,	0.07071617	,	0.320765673	,],
[	0.310779913	,	0.317442059	,	0.237877556	,	0.082057674	,]],
[[	0.085262323	,	0.230996852	,	0.255027395	,	0.240406116	,],
[	0.331041988	,	0.075344133	,	0.232035883	,	0.345720627	,],
[	0.401541612	,	0.28466187	,	0.065433177	,	0.318606552	,],
[	0.3144419	,	0.320849066	,	0.241378257	,	0.087920503	,]],
[[	0.08826099	,	0.236281043	,	0.259984776	,	0.2452209	,],
[	0.328044012	,	0.073480969	,	0.229042066	,	0.342813679	,],
[	0.401191979	,	0.284286855	,	0.066599722	,	0.318174307	,],
[	0.311744039	,	0.318624303	,	0.238950913	,	0.08227504	,]],
[[	0.083923652	,	0.230964886	,	0.254837886	,	0.240145095	,],
[	0.331387214	,	0.079644933	,	0.232886276	,	0.346322474	,],
[	0.404626402	,	0.288012681	,	0.068465478	,	0.321895186	,],
[	0.311526072	,	0.318152328	,	0.238496779	,	0.083617086	,]],
[[	0.088013859	,	0.232452034	,	0.256563092	,	0.242053966	,],
[	0.328983579	,	0.073428538	,	0.229882133	,	0.343602897	,],
[	0.400656456	,	0.283692354	,	0.064943742	,	0.317620272	,],
[	0.314996402	,	0.321795087	,	0.242276309	,	0.0852312	,]],
[[	0.085514923	,	0.233338102	,	0.25705818	,	0.242296208	,],
[	0.328783273	,	0.075360266	,	0.229851598	,	0.343590501	,],
[	0.402780215	,	0.286157025	,	0.069417652	,	0.3198532	,],
[	0.310866107	,	0.317594701	,	0.237921679	,	0.081815096	,]],
[[	0.084665745	,	0.23076409	,	0.254719559	,	0.240028955	,],
[	0.332159636	,	0.076426297	,	0.233117723	,	0.34680955	,],
[	0.402207529	,	0.285386708	,	0.065950342	,	0.319446365	,],
[	0.313307223	,	0.319881816	,	0.240324162	,	0.086375453	,]],
[[	0.089771404	,	0.237852959	,	0.26139386	,	0.246699143	,],
[	0.328108262	,	0.073277668	,	0.229038712	,	0.342812913	,],
[	0.400800325	,	0.284010226	,	0.06587206	,	0.317768889	,],
[	0.31247436	,	0.319220621	,	0.239687613	,	0.082815953	,]],
[[	0.085757176	,	0.23613319	,	0.258950647	,	0.240449775	,],
[	0.326386874	,	0.076768282	,	0.232707482	,	0.342476897	,],
[	0.396227842	,	0.284862668	,	0.064978203	,	0.31495611	,],
[	0.310710842	,	0.322709595	,	0.243254652	,	0.086525631	,]],
[[	0.084774321	,	0.238187619	,	0.260769913	,	0.242048407	,],
[	0.322351532	,	0.073606681	,	0.228656238	,	0.338450001	,],
[	0.395186099	,	0.283900715	,	0.065194323	,	0.313898839	,],
[	0.309368206	,	0.321669395	,	0.242098072	,	0.082293066	,]],
[[	0.082990053	,	0.23482626	,	0.257606076	,	0.238923633	,],
[	0.324510005	,	0.07823229	,	0.231246071	,	0.340747975	,],
[	0.399626267	,	0.288504265	,	0.068443451	,	0.318709007	,],
[	0.308400346	,	0.320604961	,	0.240885872	,	0.082728428	,]],
[[	0.089296408	,	0.238452107	,	0.261296354	,	0.242953668	,],
[	0.32408425	,	0.074527391	,	0.230237319	,	0.340115474	,],
[	0.395019662	,	0.283615466	,	0.064133759	,	0.313674462	,],
[	0.312970102	,	0.325256631	,	0.245684211	,	0.085720447	,]],
[[	0.083064006	,	0.236046852	,	0.25865295	,	0.239885281	,],
[	0.32249208	,	0.07468509	,	0.228929429	,	0.338615011	,],
[	0.396590049	,	0.285435357	,	0.067724414	,	0.31516712	,],
[	0.308218613	,	0.320393893	,	0.240787196	,	0.081550519	,]],
[[	0.084822668	,	0.235692803	,	0.258443523	,	0.239877731	,],
[	0.327014539	,	0.082164075	,	0.233942998	,	0.343247629	,],
[	0.397068966	,	0.28577514	,	0.065833914	,	0.315901777	,],
[	0.309973186	,	0.321992641	,	0.242385919	,	0.085195462	,]],
[[	0.085968458	,	0.239471191	,	0.261966735	,	0.243208065	,],
[	0.322559308	,	0.073615032	,	0.228828677	,	0.338554477	,],
[	0.39495776	,	0.283584403	,	0.064569322	,	0.31350007	,],
[	0.310296472	,	0.322587658	,	0.242957136	,	0.083025388	,]],
[[	0.082748196	,	0.2349534	,	0.257676785	,	0.238927785	,],
[	0.323798297	,	0.07715562	,	0.230383334	,	0.339971134	,],
[	0.398759896	,	0.287879247	,	0.071226798	,	0.317505616	,],
[	0.308104998	,	0.320280666	,	0.240598675	,	0.082083247	,]],
[[	0.087846909	,	0.237447189	,	0.260250816	,	0.241829181	,],
[	0.325030555	,	0.075446829	,	0.231253029	,	0.340991756	,],
[	0.395452678	,	0.284028113	,	0.064280783	,	0.314036916	,],
[	0.312375561	,	0.324338885	,	0.244702778	,	0.088709912	,]],
[[	0.083690324	,	0.236789978	,	0.259424321	,	0.240608095	,],
[	0.322402852	,	0.07415043	,	0.228677162	,	0.338527977	,],
[	0.395940252	,	0.28478428	,	0.066593566	,	0.314547965	,],
[	0.308596147	,	0.320869719	,	0.241204904	,	0.081674282	,]],
[[	0.083992992	,	0.235167412	,	0.257909895	,	0.239307553	,],
[	0.325853886	,	0.080475837	,	0.232716232	,	0.342090005	,],
[	0.39812582	,	0.286865509	,	0.066743566	,	0.316893768	,],
[	0.309313729	,	0.321476477	,	0.241707567	,	0.084094962	,]],
[[	0.087292569	,	0.240834171	,	0.263283059	,	0.244475124	,],
[	0.323071294	,	0.073836484	,	0.22928253	,	0.339063163	,],
[	0.394926921	,	0.283534002	,	0.064285023	,	0.313411848	,],
[	0.311307718	,	0.323673541	,	0.243995601	,	0.083955277	,]],
[[	0.085401048	,	0.240382443	,	0.262585626	,	0.240491664	,],
[	0.321342002	,	0.081704863	,	0.233791609	,	0.339226654	,],
[	0.392051271	,	0.285955804	,	0.066141709	,	0.311967921	,],
[	0.30788406	,	0.325483017	,	0.246328906	,	0.086231979	,]],
[[	0.08557069	,	0.243400038	,	0.265263304	,	0.243040751	,],
[	0.317047678	,	0.073191254	,	0.22866867	,	0.334651781	,],
[	0.389986335	,	0.283923092	,	0.065188119	,	0.309676316	,],
[	0.306705866	,	0.324614958	,	0.245359419	,	0.082056623	,]],
[[	0.082971925	,	0.239388404	,	0.261475404	,	0.239207347	,],
[	0.318229737	,	0.076722906	,	0.230281847	,	0.336050648	,],
[	0.393754892	,	0.288150601	,	0.071909303	,	0.313617474	,],
[	0.305600076	,	0.323395499	,	0.244146898	,	0.082444636	,]],
[[	0.088635808	,	0.242409177	,	0.264539602	,	0.242558502	,],
[	0.319445826	,	0.074910634	,	0.230981571	,	0.336965345	,],
[	0.390321125	,	0.284269913	,	0.064749799	,	0.310083182	,],
[	0.310160264	,	0.328076003	,	0.248948236	,	0.085378377	,]],
[[	0.083572706	,	0.240929164	,	0.26291852	,	0.240614944	,],
[	0.316786073	,	0.073688287	,	0.228530014	,	0.334412245	,],
[	0.390905319	,	0.284944121	,	0.067169074	,	0.310598257	,],
[	0.305386181	,	0.323208576	,	0.243972108	,	0.081140427	,]],
[[	0.084423771	,	0.23994387	,	0.262059977	,	0.239876098	,],
[	0.320365718	,	0.080193163	,	0.232668446	,	0.338135138	,],
[	0.393004667	,	0.286937411	,	0.067033656	,	0.312851079	,],
[	0.30712523	,	0.32491124	,	0.245628679	,	0.085000674	,]],
[[	0.086896901	,	0.244738449	,	0.266599868	,	0.244232984	,],
[	0.317646761	,	0.073455491	,	0.229195244	,	0.335140022	,],
[	0.389897416	,	0.283821128	,	0.064758321	,	0.309502564	,],
[	0.30745787	,	0.325299376	,	0.246155706	,	0.082712131	,]],
[[	0.082920235	,	0.239609358	,	0.261658484	,	0.239416167	,],
[	0.317617494	,	0.075607753	,	0.229479448	,	0.335290854	,],
[	0.392886692	,	0.287150619	,	0.07038896	,	0.31257723	,],
[	0.305419376	,	0.323124334	,	0.243897852	,	0.081775967	,]],
[[	0.08729785	,	0.241518085	,	0.263621602	,	0.24160791	,],
[	0.320478281	,	0.075802018	,	0.232025263	,	0.337898678	,],
[	0.391070204	,	0.284830142	,	0.065114947	,	0.310617966	,],
[	0.309646609	,	0.327206446	,	0.248009599	,	0.08849041	,]],
[[	0.084252152	,	0.241756783	,	0.263723021	,	0.24134321	,],
[	0.316814093	,	0.073338866	,	0.22849404	,	0.33440067	,],
[	0.390517295	,	0.284436501	,	0.066171896	,	0.310075085	,],
[	0.305806672	,	0.323599913	,	0.244344973	,	0.081343384	,]],
[[	0.083817976	,	0.239719423	,	0.261718888	,	0.239477998	,],
[	0.319519956	,	0.078713613	,	0.231565918	,	0.337189484	,],
[	0.394159788	,	0.288064164	,	0.068221706	,	0.313988338	,],
[	0.306423417	,	0.324151871	,	0.244944637	,	0.083833349	,]],
[[	0.088376213	,	0.246111831	,	0.267935098	,	0.245626635	,],
[	0.318268546	,	0.073936438	,	0.229810901	,	0.335687506	,],
[	0.389972628	,	0.283843263	,	0.064582469	,	0.309555664	,],
[	0.308518855	,	0.326354918	,	0.247187774	,	0.083702263	,]],
[[	0.085254185	,	0.245294479	,	0.267163192	,	0.239231465	,],
[	0.315839483	,	0.08083415	,	0.232650421	,	0.333160396	,],
[	0.389095773	,	0.288113567	,	0.067613348	,	0.308806964	,],
[	0.305977176	,	0.329402417	,	0.249359128	,	0.084673251	,]],
[[	0.085352608	,	0.248035645	,	0.269522677	,	0.241663173	,],
[	0.312565148	,	0.073431409	,	0.228497576	,	0.329559586	,],
[	0.385508946	,	0.284487351	,	0.064791683	,	0.304980138	,],
[	0.305133188	,	0.328713498	,	0.248632295	,	0.081041499	,]],
[[	0.082775943	,	0.24436941	,	0.265916542	,	0.238094236	,],
[	0.31304681	,	0.076069182	,	0.229410222	,	0.330246074	,],
[	0.388274353	,	0.287621626	,	0.070018351	,	0.30786155	,],
[	0.303645186	,	0.327196066	,	0.247013598	,	0.08088807	,]],
[[	0.088544862	,	0.247439953	,	0.269162749	,	0.241538087	,],
[	0.315416999	,	0.075728693	,	0.231330841	,	0.332227961	,],
[	0.386916168	,	0.285928771	,	0.065540079	,	0.306558376	,],
[	0.308696717	,	0.331950889	,	0.251978158	,	0.088503059	,]],
[[	0.083288698	,	0.245732038	,	0.267346289	,	0.239295914	,],
[	0.311957982	,	0.073587969	,	0.228096665	,	0.329066389	,],
[	0.38597315	,	0.285108262	,	0.066074148	,	0.305468837	,],
[	0.303668413	,	0.327290832	,	0.247124131	,	0.079964008	,]],
[[	0.08447703	,	0.244964708	,	0.266561538	,	0.238719348	,],
[	0.31511989	,	0.079414175	,	0.231602987	,	0.332249214	,],
[	0.390377563	,	0.28935971	,	0.0688884	,	0.310069823	,],
[	0.305157976	,	0.328608367	,	0.248495274	,	0.083281833	,]],
[[	0.086443007	,	0.249329945	,	0.270789728	,	0.242765991	,],
[	0.313277534	,	0.073978462	,	0.22913893	,	0.33018757	,],
[	0.385765207	,	0.284684895	,	0.064711931	,	0.305229619	,],
[	0.306116746	,	0.329704626	,	0.249591885	,	0.081829145	,]],
[[	0.082672532	,	0.244515649	,	0.266114582	,	0.238038688	,],
[	0.312641073	,	0.075149861	,	0.228829218	,	0.329634807	,],
[	0.387345035	,	0.286636939	,	0.068602028	,	0.306855654	,],
[	0.303540878	,	0.327046579	,	0.246875402	,	0.08033998	,]],
[[	0.087182067	,	0.246524165	,	0.268224852	,	0.240554416	,],
[	0.316529749	,	0.07687602	,	0.232388691	,	0.333373947	,],
[	0.387737684	,	0.286640503	,	0.066218522	,	0.307244745	,],
[	0.307567523	,	0.330954396	,	0.250844799	,	0.086810804	,]],
[[	0.083942264	,	0.246595261	,	0.268073936	,	0.240083943	,],
[	0.312156845	,	0.073420701	,	0.228123441	,	0.329114513	,],
[	0.38578814	,	0.284726992	,	0.065368441	,	0.305130761	,],
[	0.30421053	,	0.327730786	,	0.247605521	,	0.080198419	,]],
[[	0.083631312	,	0.244449973	,	0.266200645	,	0.238288533	,],
[	0.314271018	,	0.078106557	,	0.230643839	,	0.331313929	,],
[	0.389809765	,	0.289372693	,	0.072415013	,	0.309372874	,],
[	0.30461798	,	0.328002368	,	0.247907137	,	0.082298307	,]],
[[	0.08786846	,	0.250794073	,	0.272199039	,	0.24421585	,],
[	0.314062726	,	0.074485522	,	0.229913605	,	0.330851839	,],
[	0.386085641	,	0.285009656	,	0.064854685	,	0.30558227	,],
[	0.307325454	,	0.330880418	,	0.250766536	,	0.082933696	,]],
[[	0.084072384	,	0.24922696	,	0.269131944	,	0.237924148	,],
[	0.307159238	,	0.077636604	,	0.228592947	,	0.325353518	,],
[	0.382607013	,	0.288608091	,	0.070554592	,	0.303497429	,],
[	0.301473382	,	0.330091344	,	0.249923835	,	0.081626069	,]],
[[	0.084936703	,	0.252719344	,	0.272330609	,	0.240981216	,],
[	0.30785351	,	0.07521935	,	0.228721889	,	0.325783162	,],
[	0.380573097	,	0.285964816	,	0.065021615	,	0.30134267	,],
[	0.304403141	,	0.333051017	,	0.252873308	,	0.082548324	,]],
[[	0.081959089	,	0.248506641	,	0.26826574	,	0.236994036	,],
[	0.305549922	,	0.074440337	,	0.226612175	,	0.323655277	,],
[	0.380183542	,	0.285749761	,	0.066319631	,	0.300842524	,],
[	0.300607564	,	0.329248232	,	0.249032926	,	0.079563311	,]],
[[	0.087179905	,	0.25120506	,	0.270953971	,	0.239880259	,],
[	0.309401063	,	0.081340104	,	0.231081366	,	0.327722076	,],
[	0.383602572	,	0.289086922	,	0.067889794	,	0.304477121	,],
[	0.303376794	,	0.331951055	,	0.251770323	,	0.084585693	,]],
[[	0.082749019	,	0.250138553	,	0.269780728	,	0.238460924	,],
[	0.30607136	,	0.073792944	,	0.226897723	,	0.32406639	,],
[	0.379764491	,	0.285114927	,	0.064638203	,	0.300378033	,],
[	0.302027736	,	0.330725851	,	0.250520442	,	0.080339673	,]],
[[	0.083261825	,	0.248861132	,	0.268678601	,	0.237313921	,],
[	0.306523165	,	0.076543035	,	0.227798099	,	0.324662027	,],
[	0.381909989	,	0.287656283	,	0.069179701	,	0.30257283	,],
[	0.300998924	,	0.329601489	,	0.249362123	,	0.080691827	,]],
[[	0.086317374	,	0.254116296	,	0.273562791	,	0.242247241	,],
[	0.30882192	,	0.076124841	,	0.22968882	,	0.326703053	,],
[	0.381233025	,	0.28667789	,	0.065596711	,	0.301996862	,],
[	0.305652025	,	0.334363886	,	0.25425394	,	0.083797102	,]],
[[	0.081951918	,	0.248788575	,	0.268414258	,	0.237145116	,],
[	0.305505412	,	0.074013575	,	0.226465918	,	0.323497903	,],
[	0.379753821	,	0.285326136	,	0.065603992	,	0.300452311	,],
[	0.300715764	,	0.329457903	,	0.249201768	,	0.079357519	,]],
[[	0.08586696	,	0.250424864	,	0.270221973	,	0.239048935	,],
[	0.308537223	,	0.079831558	,	0.230057253	,	0.326744735	,],
[	0.384246978	,	0.290315731	,	0.072951835	,	0.305003374	,],
[	0.302528917	,	0.331075765	,	0.250851577	,	0.083301857	,]],
[[	0.083578378	,	0.251076815	,	0.27071541	,	0.239288143	,],
[	0.3066731	,	0.074259042	,	0.227519104	,	0.324590956	,],
[	0.379901102	,	0.285358181	,	0.064587895	,	0.300530858	,],
[	0.3028998	,	0.331528385	,	0.25138654	,	0.081053786	,]],
[[	0.082581432	,	0.248594924	,	0.268369257	,	0.237054392	,],
[	0.30599411	,	0.075617436	,	0.227182614	,	0.324075576	,],
[	0.381173287	,	0.286860064	,	0.068004719	,	0.301741777	,],
[	0.300694514	,	0.329466723	,	0.249232674	,	0.080122346	,]],
[[	0.087733684	,	0.255441749	,	0.274983089	,	0.243653018	,],
[	0.310024119	,	0.077252435	,	0.230946109	,	0.327819044	,],
[	0.382221276	,	0.287573496	,	0.066448706	,	0.302954185	,],
[	0.305043231	,	0.333511251	,	0.253317809	,	0.086840448	,]],
[[	0.082870635	,	0.252657277	,	0.271926799	,	0.236542251	,],
[	0.300476717	,	0.076483218	,	0.22752955	,	0.319498752	,],
[	0.376343392	,	0.287545199	,	0.069999879	,	0.297970305	,],
[	0.297523966	,	0.331287045	,	0.251422182	,	0.080515266	,]],
[[	0.08359111	,	0.255894144	,	0.27491942	,	0.239412466	,],
[	0.302225284	,	0.075285676	,	0.228782641	,	0.3209073	,],
[	0.374913612	,	0.285561691	,	0.065202624	,	0.296484014	,],
[	0.300745147	,	0.334493588	,	0.254789997	,	0.08185396	,]],
[[	0.080579044	,	0.251775745	,	0.270968171	,	0.23535004	,],
[	0.299288202	,	0.073705247	,	0.225876026	,	0.318208908	,],
[	0.374077218	,	0.284975104	,	0.066047462	,	0.295518028	,],
[	0.29667886	,	0.330512774	,	0.250572505	,	0.078591152	,]],
[[	0.086019301	,	0.254568486	,	0.27382618	,	0.238537309	,],
[	0.302478296	,	0.079930974	,	0.229788585	,	0.321578946	,],
[	0.378218689	,	0.288958524	,	0.068501863	,	0.299830375	,],
[	0.299340476	,	0.33293856	,	0.253150993	,	0.083415932	,]],
[[	0.081414192	,	0.25339264	,	0.272513845	,	0.236913705	,],
[	0.30018778	,	0.073640085	,	0.226738883	,	0.318971635	,],
[	0.373823214	,	0.284468687	,	0.064578756	,	0.295250485	,],
[	0.298404375	,	0.332191859	,	0.25234768	,	0.079684967	,]],
[[	0.081959446	,	0.25218418	,	0.271347999	,	0.235921434	,],
[	0.299971045	,	0.075451038	,	0.226869941	,	0.318847568	,],
[	0.375509934	,	0.286561887	,	0.068649742	,	0.296997884	,],
[	0.297053583	,	0.330842832	,	0.250955332	,	0.079693789	,]],
[[	0.084775751	,	0.257126888	,	0.276122762	,	0.240680209	,],
[	0.303450021	,	0.076370251	,	0.229897526	,	0.32204695	,],
[	0.375738933	,	0.286446838	,	0.066021677	,	0.29723556	,],
[	0.30203782	,	0.33561399	,	0.255875721	,	0.087323768	,]],
[[	0.080616783	,	0.25205719	,	0.271150972	,	0.235580634	,],
[	0.299384447	,	0.073420599	,	0.225810252	,	0.318169613	,],
[	0.373860592	,	0.284651775	,	0.065340007	,	0.295319235	,],
[	0.296983771	,	0.330737941	,	0.250886821	,	0.078488035	,]],
[[	0.084742587	,	0.253819504	,	0.273010801	,	0.237602011	,],
[	0.301571263	,	0.078481071	,	0.228814714	,	0.320648884	,],
[	0.377905617	,	0.289223654	,	0.072386466	,	0.299504747	,],
[	0.298524838	,	0.33211792	,	0.252312529	,	0.082046555	,]],
[[	0.082133725	,	0.254218691	,	0.273317415	,	0.237693757	,],
[	0.300986848	,	0.074135282	,	0.227490782	,	0.31969759	,],
[	0.374237901	,	0.284855485	,	0.064653324	,	0.295536041	,],
[	0.299296374	,	0.33296807	,	0.253193373	,	0.080250515	,]],
[[	0.081309085	,	0.251836901	,	0.271195726	,	0.235542451	,],
[	0.299612272	,	0.074596612	,	0.226309961	,	0.318481222	,],
[	0.374983109	,	0.285902049	,	0.067485095	,	0.29630159	,],
[	0.296884584	,	0.330575297	,	0.250715998	,	0.079052168	,]],
[[	0.086318156	,	0.258621954	,	0.277557975	,	0.24206406	,],
[	0.30425068	,	0.08243074	,	0.231707736	,	0.323203552	,],
[	0.376710575	,	0.287369023	,	0.066801105	,	0.298128296	,],
[	0.300966682	,	0.334489437	,	0.254666014	,	0.085581819	,]],
[[	0.081486601	,	0.256167198	,	0.274400942	,	0.233829249	,],
[	0.292674755	,	0.07515358	,	0.224980588	,	0.312212159	,],
[	0.368318132	,	0.286228798	,	0.06767566	,	0.291036853	,],
[	0.293133551	,	0.333203278	,	0.253050716	,	0.07936153	,]],
[[	0.08341839	,	0.260304995	,	0.278328372	,	0.23781235	,],
[	0.296754539	,	0.076870744	,	0.228726533	,	0.315933552	,],
[	0.369215606	,	0.286723781	,	0.065718949	,	0.291956214	,],
[	0.297786673	,	0.337624645	,	0.257668762	,	0.086487876	,]],
[[	0.079701692	,	0.25557247	,	0.27386034	,	0.233231498	,],
[	0.292439858	,	0.073439378	,	0.22436152	,	0.311770645	,],
[	0.366833554	,	0.284409433	,	0.064574907	,	0.289478863	,],
[	0.293262762	,	0.33328422	,	0.253184858	,	0.078444963	,]],
[[	0.084470937	,	0.257840881	,	0.27618286	,	0.235694872	,],
[	0.294269841	,	0.078017756	,	0.226802698	,	0.313737039	,],
[	0.370604438	,	0.288775038	,	0.071220572	,	0.293356661	,],
[	0.294448783	,	0.334408811	,	0.254315278	,	0.081568351	,]],
[[	0.081026645	,	0.257553662	,	0.27575553	,	0.23512366	,],
[	0.294315163	,	0.074630914	,	0.226234867	,	0.313538626	,],
[	0.367577401	,	0.285039625	,	0.064308526	,	0.290208165	,],
[	0.295764185	,	0.335678083	,	0.255594266	,	0.080350217	,]],
[[	0.0807616	,	0.255744379	,	0.273976933	,	0.233337051	,],
[	0.292412827	,	0.074452325	,	0.224575387	,	0.31175814	,],
[	0.367870134	,	0.285553318	,	0.066566952	,	0.290402554	,],
[	0.292955692	,	0.33300163	,	0.252824609	,	0.078707081	,]],
[[	0.084851577	,	0.261755453	,	0.279777136	,	0.239218656	,],
[	0.296798965	,	0.081841456	,	0.229600871	,	0.316117654	,],
[	0.370267488	,	0.287716645	,	0.066786815	,	0.293005171	,],
[	0.296740803	,	0.33660523	,	0.256537606	,	0.084824068	,]],
[[	0.079956328	,	0.255949832	,	0.274191387	,	0.233546941	,],
[	0.292755845	,	0.07351453	,	0.224628146	,	0.311977947	,],
[	0.366872069	,	0.284370834	,	0.064210663	,	0.289364434	,],
[	0.293837104	,	0.33384114	,	0.25366693	,	0.078656824	,]],
[[	0.083224785	,	0.257109505	,	0.275365984	,	0.234801266	,],
[	0.293580655	,	0.076826444	,	0.22598686	,	0.312980395	,],
[	0.369632034	,	0.287542042	,	0.069615042	,	0.292231358	,],
[	0.293945657	,	0.333877603	,	0.253664687	,	0.080489218	,]],
[[	0.081886381	,	0.258545905	,	0.276618938	,	0.236050004	,],
[	0.295216881	,	0.075393101	,	0.227052577	,	0.314310734	,],
[	0.36818588	,	0.28556481	,	0.064762956	,	0.290751559	,],
[	0.29685441	,	0.336715681	,	0.256690891	,	0.081330006	,]],
[[	0.080278683	,	0.25555345	,	0.273826406	,	0.233130164	,],
[	0.292333265	,	0.07389564	,	0.224354094	,	0.311606904	,],
[	0.367435975	,	0.285110243	,	0.065681783	,	0.289950914	,],
[	0.293014296	,	0.332995786	,	0.25282268	,	0.078420192	,]],
[[	0.086357845	,	0.26321646	,	0.281273181	,	0.240666905	,],
[	0.295727157	,	0.080303239	,	0.228419516	,	0.315164774	,],
[	0.371497476	,	0.288853958	,	0.067907675	,	0.294193323	,],
[	0.295858792	,	0.335669167	,	0.255599455	,	0.083465627	,]],
[[	0.083566975	,	0.26728896	,	0.286214788	,	0.239697577	,],
[	0.295236632	,	0.076376473	,	0.231448859	,	0.315477643	,],
[	0.372967294	,	0.293877351	,	0.067978668	,	0.293536303	,],
[	0.300506878	,	0.347569879	,	0.2649263	,	0.080152216	,]],
[[	0.085936765	,	0.271780114	,	0.290321599	,	0.243946487	,],
[	0.297684871	,	0.081643212	,	0.234435241	,	0.317976342	,],
[	0.375138441	,	0.296328906	,	0.072572944	,	0.295715882	,],
[	0.303401341	,	0.350427607	,	0.267861062	,	0.085468001	,]],
[[	0.081968352	,	0.266944878	,	0.285656048	,	0.23920761	,],
[	0.29688965	,	0.077131391	,	0.232818399	,	0.316928686	,],
[	0.374556915	,	0.295314769	,	0.06899471	,	0.295245518	,],
[	0.3019117	,	0.348876054	,	0.266397761	,	0.080868115	,]],
[[	0.086426086	,	0.269092622	,	0.287780257	,	0.241512802	,],
[	0.295634071	,	0.0776824	,	0.231882222	,	0.315837628	,],
[	0.373295779	,	0.294238843	,	0.069025751	,	0.293824375	,],
[	0.300988787	,	0.348096274	,	0.265474258	,	0.081497136	,]],
[[	0.08344129	,	0.26906283	,	0.287663568	,	0.241245352	,],
[	0.300257329	,	0.085490398	,	0.237147053	,	0.320576007	,],
[	0.377471525	,	0.298994881	,	0.076303167	,	0.29825311	,],
[	0.305284909	,	0.352232232	,	0.269798284	,	0.084200106	,]],
[[	0.0829596	,	0.267059217	,	0.285813567	,	0.239317588	,],
[	0.295539349	,	0.076352918	,	0.231649933	,	0.315728785	,],
[	0.373271091	,	0.29407519	,	0.068021607	,	0.293730981	,],
[	0.300642519	,	0.347665161	,	0.265013719	,	0.080059091	,]],
[[	0.087365469	,	0.273205325	,	0.291642542	,	0.245405965	,],
[	0.296994026	,	0.080269133	,	0.233551308	,	0.317213013	,],
[	0.374457056	,	0.295637262	,	0.071332632	,	0.294994236	,],
[	0.302581619	,	0.349633757	,	0.267024969	,	0.084074112	,]],
[[	0.082151745	,	0.267332799	,	0.285984272	,	0.239557989	,],
[	0.297844904	,	0.077811171	,	0.233752352	,	0.317724409	,],
[	0.375391664	,	0.296215666	,	0.069696634	,	0.295994743	,],
[	0.302720828	,	0.349738448	,	0.267236291	,	0.081571809	,]],
[[	0.085280652	,	0.268336037	,	0.287091289	,	0.240665088	,],
[	0.295473997	,	0.077018029	,	0.231555275	,	0.315571107	,],
[	0.37308305	,	0.293998084	,	0.068482228	,	0.29358353	,],
[	0.300668865	,	0.347700758	,	0.265122203	,	0.080816549	,]],
[[	0.084266759	,	0.270089941	,	0.288508775	,	0.24210507	,],
[	0.299231469	,	0.083922448	,	0.236058402	,	0.319490751	,],
[	0.37652507	,	0.298006235	,	0.074736055	,	0.29710732	,],
[	0.30515934	,	0.3520433	,	0.269514311	,	0.087718525	,]],
[[	0.082399006	,	0.266794891	,	0.285591527	,	0.239104477	,],
[	0.296031641	,	0.076536518	,	0.23202642	,	0.315981448	,],
[	0.373719429	,	0.294500035	,	0.068307677	,	0.294176192	,],
[	0.30097359	,	0.34805869	,	0.265444005	,	0.080261973	,]],
[[	0.088689368	,	0.270504088	,	0.289282075	,	0.242980378	,],
[	0.296397698	,	0.079131164	,	0.232787943	,	0.316449245	,],
[	0.373852107	,	0.294957771	,	0.070234446	,	0.294412232	,],
[	0.301950688	,	0.348917913	,	0.266299657	,	0.082922556	,]],
[[	0.082441429	,	0.271117171	,	0.288372833	,	0.238586057	,],
[	0.289281974	,	0.075870778	,	0.230082848	,	0.310318855	,],
[	0.367275347	,	0.29350596	,	0.066996321	,	0.288867769	,],
[	0.297521146	,	0.34961564	,	0.266796026	,	0.079902956	,]],
[[	0.085065403	,	0.275740074	,	0.292787326	,	0.243099413	,],
[	0.290296981	,	0.079423825	,	0.2315117	,	0.311440455	,],
[	0.36884026	,	0.295338574	,	0.070639059	,	0.290375532	,],
[	0.300067159	,	0.352036824	,	0.269347213	,	0.084507691	,]],
[[	0.080927141	,	0.27082978	,	0.287977665	,	0.238190298	,],
[	0.291755208	,	0.077768875	,	0.232501843	,	0.312725497	,],
[	0.36937646	,	0.295525547	,	0.068444205	,	0.290898101	,],
[	0.2991586	,	0.351143219	,	0.268494035	,	0.080957653	,]],
[[	0.085269551	,	0.272793155	,	0.290097405	,	0.240330687	,],
[	0.289010318	,	0.076422849	,	0.229897053	,	0.310053463	,],
[	0.367161513	,	0.293476457	,	0.067573254	,	0.288679283	,],
[	0.297907554	,	0.349779135	,	0.267065557	,	0.080830049	,]],
[[	0.082451215	,	0.272969355	,	0.290093685	,	0.240305208	,],
[	0.292393517	,	0.082871765	,	0.234003969	,	0.313670887	,],
[	0.370920968	,	0.297677595	,	0.074160561	,	0.292566234	,],
[	0.302709882	,	0.354484779	,	0.271906394	,	0.088297463	,]],
[[	0.081761823	,	0.270873097	,	0.288037237	,	0.238205557	,],
[	0.289838608	,	0.076209833	,	0.230558451	,	0.310713902	,],
[	0.367790054	,	0.29391108	,	0.067183242	,	0.289226227	,],
[	0.297875453	,	0.349873903	,	0.267156171	,	0.079895863	,]],
[[	0.086520924	,	0.277234029	,	0.294248684	,	0.244458727	,],
[	0.289631651	,	0.078308465	,	0.230790159	,	0.31075343	,],
[	0.368297631	,	0.294649881	,	0.069568791	,	0.289739021	,],
[	0.299326851	,	0.351175265	,	0.268409999	,	0.083180782	,]],
[[	0.08110411	,	0.271189897	,	0.288382614	,	0.238440259	,],
[	0.29289565	,	0.078795508	,	0.233486969	,	0.313662277	,],
[	0.370305866	,	0.296424224	,	0.0693735	,	0.29179897	,],
[	0.300143064	,	0.351975093	,	0.269325993	,	0.081720852	,]],
[[	0.084082881	,	0.272032463	,	0.289363388	,	0.239481621	,],
[	0.289052725	,	0.07602025	,	0.229839983	,	0.309941646	,],
[	0.367216874	,	0.293370451	,	0.067206807	,	0.288587138	,],
[	0.297728002	,	0.349661557	,	0.266884013	,	0.080354345	,]],
[[	0.083336212	,	0.273966451	,	0.290976268	,	0.241202743	,],
[	0.291598826	,	0.081507908	,	0.232951184	,	0.312644653	,],
[	0.370081514	,	0.296844278	,	0.072704848	,	0.291562163	,],
[	0.301675309	,	0.353546056	,	0.270824258	,	0.086655789	,]],
[[	0.081312267	,	0.270738629	,	0.287898119	,	0.237993923	,],
[	0.290540115	,	0.076707623	,	0.231148718	,	0.31140872	,],
[	0.368287222	,	0.294388549	,	0.067512298	,	0.289749609	,],
[	0.298363608	,	0.350277735	,	0.267524047	,	0.080185945	,]],
[[	0.087536966	,	0.27429106	,	0.291537351	,	0.241887705	,],
[	0.289380606	,	0.077500128	,	0.230349028	,	0.310387101	,],
[	0.367716657	,	0.294111412	,	0.068621168	,	0.289075779	,],
[	0.298690413	,	0.350600081	,	0.267806168	,	0.082129916	,]],
[[	0.082093562	,	0.274967932	,	0.291679383	,	0.23728723	,],
[	0.283022303	,	0.076139391	,	0.229872424	,	0.305520257	,],
[	0.361173088	,	0.293733931	,	0.066743905	,	0.284234233	,],
[	0.294850884	,	0.352382819	,	0.270081734	,	0.079476284	,]],
[[	0.085246411	,	0.279925691	,	0.296373852	,	0.242147687	,],
[	0.283093885	,	0.078401912	,	0.230355688	,	0.305777661	,],
[	0.361725453	,	0.294516276	,	0.069286444	,	0.284691989	,],
[	0.296430815	,	0.354030463	,	0.271646917	,	0.082939718	,]],
[[	0.080928674	,	0.274895953	,	0.29143861	,	0.237055943	,],
[	0.286150215	,	0.078618577	,	0.232880195	,	0.30842949	,],
[	0.3639078	,	0.296490561	,	0.069195233	,	0.286990355	,],
[	0.296991736	,	0.354466577	,	0.272187646	,	0.081090981	,]],
[[	0.08471796	,	0.27639862	,	0.293084363	,	0.238829223	,],
[	0.282268891	,	0.076018513	,	0.229314608	,	0.304863971	,],
[	0.360626527	,	0.293324789	,	0.066749181	,	0.283603133	,],
[	0.294738764	,	0.352278199	,	0.269847907	,	0.079962167	,]],
[[	0.082559723	,	0.277041696	,	0.293579673	,	0.239174894	,],
[	0.284921497	,	0.081561813	,	0.232469383	,	0.307670087	,],
[	0.363546021	,	0.296745877	,	0.072452169	,	0.286602461	,],
[	0.298873794	,	0.35634943	,	0.2740796	,	0.086505871	,]],
[[	0.081537551	,	0.274736609	,	0.29143034	,	0.236973712	,],
[	0.28388317	,	0.076672233	,	0.230524114	,	0.306228467	,],
[	0.361866062	,	0.294355932	,	0.06723021	,	0.284696113	,],
[	0.295326695	,	0.352860503	,	0.270544295	,	0.079647188	,]],
[[	0.086695751	,	0.281406765	,	0.297811581	,	0.243565229	,],
[	0.282660513	,	0.077371199	,	0.229700818	,	0.30516892	,],
[	0.361327666	,	0.294087673	,	0.068332959	,	0.284116408	,],
[	0.295919362	,	0.353360012	,	0.271010436	,	0.081837747	,]],
[[	0.081122154	,	0.275238656	,	0.291738793	,	0.237365357	,],
[	0.28737925	,	0.079771042	,	0.233999255	,	0.309595532	,],
[	0.365034857	,	0.297617764	,	0.070315276	,	0.288094773	,],
[	0.298156092	,	0.355567839	,	0.273258858	,	0.082129358	,]],
[[	0.083679602	,	0.275928186	,	0.292509766	,	0.23820651	,],
[	0.28253151	,	0.075970405	,	0.229357554	,	0.304914928	,],
[	0.36082974	,	0.293367919	,	0.066653992	,	0.283689814	,],
[	0.29475113	,	0.352364882	,	0.269915927	,	0.079602799	,]],
[[	0.083573555	,	0.278262522	,	0.294670736	,	0.24026549	,],
[	0.284153145	,	0.080165708	,	0.231542432	,	0.306801199	,],
[	0.362741849	,	0.295765128	,	0.070974474	,	0.285710393	,],
[	0.297867023	,	0.355393378	,	0.273041565	,	0.084999244	,]],
[[	0.081150732	,	0.274600498	,	0.291263611	,	0.236784984	,],
[	0.284659167	,	0.077325277	,	0.231249034	,	0.307013918	,],
[	0.362667901	,	0.295094845	,	0.067818354	,	0.285637507	,],
[	0.29592266	,	0.353408325	,	0.271033848	,	0.080171729	,]],
[[	0.086996842	,	0.277927606	,	0.294483701	,	0.240338291	,],
[	0.28247429	,	0.076763596	,	0.229425025	,	0.304933785	,],
[	0.36091789	,	0.293585049	,	0.067511904	,	0.283796553	,],
[	0.295345368	,	0.352893653	,	0.270493091	,	0.080899625	,]],
[[	0.081641313	,	0.280193445	,	0.296437746	,	0.237715209	,],
[	0.280006893	,	0.077069718	,	0.231496712	,	0.30294877	,],
[	0.358241769	,	0.295955385	,	0.067096526	,	0.281860549	,],
[	0.294007904	,	0.357047445	,	0.273730148	,	0.079392009	,]],
[[	0.086359001	,	0.286426787	,	0.302458484	,	0.243989627	,],
[	0.278627646	,	0.077584265	,	0.230415347	,	0.30168105	,],
[	0.358736747	,	0.296744115	,	0.069468054	,	0.282271352	,],
[	0.295903745	,	0.358909751	,	0.275551838	,	0.083100791	,]],
[[	0.081099293	,	0.280675826	,	0.296814383	,	0.238134553	,],
[	0.283184994	,	0.085500371	,	0.235739158	,	0.306367864	,],
[	0.360861964	,	0.298531901	,	0.069398378	,	0.284550768	,],
[	0.296249418	,	0.35919364	,	0.275967592	,	0.08108588	,]],
[[	0.083966176	,	0.281386574	,	0.297654705	,	0.239036977	,],
[	0.278574441	,	0.076269055	,	0.230120991	,	0.301622611	,],
[	0.357618754	,	0.295464552	,	0.067060737	,	0.281195187	,],
[	0.294052206	,	0.357125421	,	0.273640791	,	0.079958398	,]],
[[	0.083422828	,	0.283287452	,	0.299369083	,	0.240840115	,],
[	0.279962353	,	0.080169254	,	0.232060266	,	0.303088782	,],
[	0.360635732	,	0.298955827	,	0.072759646	,	0.28426172	,],
[	0.298428271	,	0.361316708	,	0.278010933	,	0.086756667	,]],
[[	0.081280615	,	0.280057955	,	0.296256938	,	0.237617251	,],
[	0.280945421	,	0.0778462	,	0.232212686	,	0.303734529	,],
[	0.35873327	,	0.296411985	,	0.067419236	,	0.282330049	,],
[	0.294537088	,	0.357540953	,	0.274109113	,	0.079516567	,]],
[[	0.087486167	,	0.283608751	,	0.299850472	,	0.241381785	,],
[	0.278407655	,	0.076968047	,	0.230136404	,	0.301388616	,],
[	0.358162983	,	0.29621098	,	0.068580865	,	0.281652588	,],
[	0.2952519	,	0.358243166	,	0.27472264	,	0.081926782	,]],
[[	0.081458863	,	0.281102985	,	0.297224733	,	0.238640422	,],
[	0.282270288	,	0.0838064	,	0.234512183	,	0.305362479	,],
[	0.362031725	,	0.299743712	,	0.070555523	,	0.28564845	,],
[	0.297381763	,	0.360210664	,	0.276954256	,	0.0820554	,]],
[[	0.082923867	,	0.280851304	,	0.297012903	,	0.238413567	,],
[	0.279120103	,	0.076502686	,	0.230572407	,	0.302037117	,],
[	0.35780368	,	0.295563099	,	0.06692007	,	0.281292454	,],
[	0.294027397	,	0.357083226	,	0.273566495	,	0.079518337	,]],
[[	0.084550576	,	0.284475661	,	0.300515811	,	0.241982126	,],
[	0.279430628	,	0.079102695	,	0.231301421	,	0.302403448	,],
[	0.359752744	,	0.297936953	,	0.071372197	,	0.283264288	,],
[	0.297366494	,	0.360327073	,	0.276890777	,	0.085283989	,]],
[[	0.081104803	,	0.280185383	,	0.296362551	,	0.237738149	,],
[	0.281989542	,	0.078797218	,	0.233252031	,	0.304716648	,],
[	0.359554556	,	0.29717628	,	0.068175093	,	0.283030883	,],
[	0.295093698	,	0.358133096	,	0.27459462	,	0.07993411	,]],
[[	0.085977461	,	0.282596848	,	0.298870327	,	0.24023732	,],
[	0.278446608	,	0.076620046	,	0.230041499	,	0.301307206	,],
[	0.358005553	,	0.295797845	,	0.067852485	,	0.281385786	,],
[	0.294700232	,	0.357685827	,	0.274181547	,	0.080981087	,]],
[[	0.08158582	,	0.285639536	,	0.300841488	,	0.237691225	,],
[	0.275904365	,	0.078011699	,	0.231982602	,	0.299633912	,],
[	0.354824783	,	0.297983609	,	0.067295022	,	0.278288028	,],
[	0.2935654	,	0.361797519	,	0.277241871	,	0.079075716	,]],
[[	0.084803493	,	0.290545314	,	0.305596366	,	0.242664159	,],
[	0.273822077	,	0.077811491	,	0.230239468	,	0.297673057	,],
[	0.35528552	,	0.298839386	,	0.069780812	,	0.278804415	,],
[	0.295270634	,	0.363538257	,	0.278930225	,	0.082873264	,]],
[[	0.080326669	,	0.285416679	,	0.300537726	,	0.237444371	,],
[	0.277967995	,	0.085160137	,	0.235137467	,	0.30195793	,],
[	0.357587016	,	0.300683617	,	0.069610166	,	0.28109599	,],
[	0.295760338	,	0.363757348	,	0.2793071	,	0.080769385	,]],
[[	0.084250617	,	0.287108329	,	0.30233197	,	0.239288635	,],
[	0.274238629	,	0.076909965	,	0.230432126	,	0.298142777	,],
[	0.354285218	,	0.297521906	,	0.067310798	,	0.27769649	,],
[	0.293558019	,	0.361711683	,	0.277094707	,	0.079672317	,]],
[[	0.082078812	,	0.287777372	,	0.302856227	,	0.23991427	,],
[	0.27490801	,	0.080165409	,	0.231529858	,	0.298911167	,],
[	0.357124397	,	0.300959337	,	0.07289984	,	0.280679268	,],
[	0.297785792	,	0.365906507	,	0.281381532	,	0.08644787	,]],
[[	0.080974308	,	0.285253667	,	0.300459518	,	0.237366895	,],
[	0.277012733	,	0.078942948	,	0.232911322	,	0.300577848	,],
[	0.355478583	,	0.29857852	,	0.067733291	,	0.278896188	,],
[	0.293959089	,	0.362064278	,	0.2775884	,	0.079326804	,]],
[[	0.086302135	,	0.292050856	,	0.307005352	,	0.244176609	,],
[	0.273794389	,	0.07729592	,	0.230025019	,	0.297569379	,],
[	0.354858698	,	0.298235528	,	0.068849787	,	0.27821005	,],
[	0.294708471	,	0.362924975	,	0.278190613	,	0.081707131	,]],
[[	0.080583467	,	0.285733349	,	0.3008781	,	0.237882543	,],
[	0.277091642	,	0.083586898	,	0.233892784	,	0.300860819	,],
[	0.358683524	,	0.301857784	,	0.070704269	,	0.282224314	,],
[	0.296733376	,	0.364711967	,	0.280303098	,	0.081650421	,]],
[[	0.083133191	,	0.286335153	,	0.30168625	,	0.238546712	,],
[	0.274882964	,	0.077289986	,	0.230921154	,	0.298492123	,],
[	0.354507738	,	0.297643599	,	0.067147939	,	0.277833822	,],
[	0.293506233	,	0.36165566	,	0.277021125	,	0.079236521	,]],
[[	0.083104748	,	0.288781902	,	0.303828137	,	0.24087192	,],
[	0.274507629	,	0.07908289	,	0.231015617	,	0.298331006	,],
[	0.356413943	,	0.300052813	,	0.071553754	,	0.279889476	,],
[	0.296795471	,	0.364886177	,	0.280352046	,	0.08495932	,]],
[[	0.080584862	,	0.285194753	,	0.300453146	,	0.237265603	,],
[	0.278104479	,	0.079985115	,	0.234036183	,	0.301651731	,],
[	0.356202413	,	0.299357929	,	0.068226712	,	0.279606401	,],
[	0.294540728	,	0.362574954	,	0.27806803	,	0.079713569	,]],
[[	0.086449809	,	0.288523503	,	0.303741788	,	0.240756307	,],
[	0.273831691	,	0.076993581	,	0.229980931	,	0.29760937	,],
[	0.35456797	,	0.297944684	,	0.068064372	,	0.277919633	,],
[	0.294205616	,	0.362322744	,	0.27773211	,	0.080812328	,]],
[[	0.081064011	,	0.289683215	,	0.304072286	,	0.2376662	,],
[	0.272209022	,	0.079243513	,	0.233371153	,	0.296930839	,],
[	0.351290795	,	0.299348208	,	0.068363753	,	0.275505577	,],
[	0.292126955	,	0.365283898	,	0.28133846	,	0.079451681	,]],
[[	0.085849593	,	0.29584071	,	0.310029778	,	0.243853368	,],
[	0.268550963	,	0.076880207	,	0.229880846	,	0.293413267	,],
[	0.349896843	,	0.298246505	,	0.068532885	,	0.274052111	,],
[	0.292544785	,	0.365895152	,	0.281886671	,	0.081568138	,]],
[[	0.080475977	,	0.290105727	,	0.304390463	,	0.237971357	,],
[	0.271286235	,	0.082503985	,	0.233347974	,	0.296317067	,],
[	0.354464924	,	0.303299122	,	0.076251564	,	0.278781799	,],
[	0.294975055	,	0.36795563	,	0.284116848	,	0.081823778	,]],
[[	0.083176907	,	0.290790137	,	0.305253184	,	0.238797044	,],
[	0.269957906	,	0.07733033	,	0.231102217	,	0.294737285	,],
[	0.349948506	,	0.298068166	,	0.067498744	,	0.273986669	,],
[	0.291608771	,	0.36483742	,	0.280699751	,	0.079298944	,]],
[[	0.082850248	,	0.292868289	,	0.307132975	,	0.240638152	,],
[	0.268939503	,	0.078385429	,	0.230587638	,	0.293849909	,],
[	0.351130477	,	0.299687573	,	0.071029398	,	0.275326542	,],
[	0.294674941	,	0.367811754	,	0.283849325	,	0.084658822	,]],
[[	0.080585119	,	0.28956938	,	0.303861086	,	0.237355534	,],
[	0.273590835	,	0.080418231	,	0.234581141	,	0.298102659	,],
[	0.352231316	,	0.30016708	,	0.06913082	,	0.276362603	,],
[	0.292928832	,	0.365940746	,	0.281935279	,	0.079894639	,]],
[[	0.086744654	,	0.292983425	,	0.307409669	,	0.241095192	,],
[	0.268794373	,	0.076754499	,	0.229996821	,	0.293470268	,],
[	0.349720169	,	0.297927381	,	0.067960857	,	0.273775705	,],
[	0.292121624	,	0.36543788	,	0.28126121	,	0.080614167	,]],
[[	0.080868528	,	0.290605223	,	0.304947308	,	0.238394736	,],
[	0.270510817	,	0.081156279	,	0.232387006	,	0.295453281	,],
[	0.353410821	,	0.302125567	,	0.07458259	,	0.277674346	,],
[	0.296213896	,	0.369105249	,	0.285304799	,	0.083023447	,]],
[[	0.082280316	,	0.290255202	,	0.30460471	,	0.23808457	,],
[	0.270887153	,	0.077965229	,	0.232030617	,	0.295493152	,],
[	0.350501211	,	0.298388631	,	0.067683558	,	0.274448272	,],
[	0.291769799	,	0.364967146	,	0.280843198	,	0.079113062	,]],
[[	0.083989694	,	0.293952147	,	0.308173473	,	0.241830028	,],
[	0.268664193	,	0.077734921	,	0.230222758	,	0.293522484	,],
[	0.350770638	,	0.299032305	,	0.069876803	,	0.274717429	,],
[	0.293750613	,	0.367107579	,	0.283008695	,	0.083377532	,]],
[[	0.080434189	,	0.289606026	,	0.303960199	,	0.237402983	,],
[	0.272811578	,	0.084945571	,	0.234922577	,	0.297703988	,],
[	0.353165761	,	0.301127409	,	0.070054975	,	0.277387945	,],
[	0.293660185	,	0.366761538	,	0.282664008	,	0.080518704	,]],
[[	0.085311175	,	0.292029648	,	0.306548659	,	0.239972374	,],
[	0.269193234	,	0.076885031	,	0.230341373	,	0.293876749	,],
[	0.349740692	,	0.297853596	,	0.067531895	,	0.273709341	,],
[	0.291919101	,	0.365236127	,	0.281056277	,	0.080002989	,]],
[[	0.079914349	,	0.293372873	,	0.307444132	,	0.235653488	,],
[	0.26567968	,	0.084529663	,	0.233497895	,	0.291199385	,],
[	0.348019737	,	0.302441201	,	0.070741198	,	0.273204855	,],
[	0.290255095	,	0.369185538	,	0.284520891	,	0.079767772	,]],
[[	0.08558005	,	0.300269836	,	0.314099748	,	0.242694668	,],
[	0.262693573	,	0.077028126	,	0.229317574	,	0.287805294	,],
[	0.343897138	,	0.298509031	,	0.067408119	,	0.268947769	,],
[	0.289104617	,	0.368212676	,	0.283390081	,	0.079871178	,]],
[[	0.079610675	,	0.294106584	,	0.308104494	,	0.236308305	,],
[	0.262920279	,	0.079728188	,	0.230216322	,	0.2881755	,],
[	0.346218274	,	0.301297109	,	0.072126345	,	0.27127405	,],
[	0.29400131	,	0.372984086	,	0.288425754	,	0.087508556	,]],
[[	0.081843827	,	0.294389516	,	0.308414352	,	0.236773605	,],
[	0.265604257	,	0.079417075	,	0.23221943	,	0.290588236	,],
[	0.345625707	,	0.300063854	,	0.068286187	,	0.270645889	,],
[	0.288935933	,	0.367908781	,	0.283148344	,	0.078676668	,]],
[[	0.082201598	,	0.296956772	,	0.310864082	,	0.2392849	,],
[	0.26210183	,	0.077180814	,	0.228918417	,	0.2872445	,],
[	0.344253268	,	0.299022248	,	0.068532877	,	0.269170327	,],
[	0.29055492	,	0.369573555	,	0.28487189	,	0.082299272	,]],
[[	0.079460199	,	0.293281381	,	0.307328557	,	0.235537624	,],
[	0.264835329	,	0.08302624	,	0.232395262	,	0.290166576	,],
[	0.348545549	,	0.303934671	,	0.075991539	,	0.273808928	,],
[	0.291194247	,	0.370048362	,	0.285385046	,	0.080526016	,]],
[[	0.085064576	,	0.296463474	,	0.310425796	,	0.238703098	,],
[	0.26332099	,	0.077512864	,	0.230084063	,	0.288406996	,],
[	0.344377599	,	0.298810697	,	0.067409731	,	0.269274051	,],
[	0.288862071	,	0.367931526	,	0.283065306	,	0.079191522	,]],
[[	0.080143503	,	0.294690219	,	0.308618099	,	0.236917205	,],
[	0.262477356	,	0.078776333	,	0.22962384	,	0.28773172	,],
[	0.345519266	,	0.300523145	,	0.070965522	,	0.270518484	,],
[	0.2928734	,	0.371950527	,	0.287164102	,	0.085822471	,]],
[[	0.080920127	,	0.293881977	,	0.307964708	,	0.236040722	,],
[	0.266822882	,	0.080611702	,	0.233398726	,	0.291834386	,],
[	0.346554907	,	0.301035927	,	0.069170193	,	0.271554322	,],
[	0.289466196	,	0.368384106	,	0.283632619	,	0.078990839	,]],
[[	0.083478921	,	0.29822371	,	0.312130588	,	0.240492645	,],
[	0.262134422	,	0.076991472	,	0.229053757	,	0.287302358	,],
[	0.344081822	,	0.298752128	,	0.067969464	,	0.268914824	,],
[	0.289928572	,	0.369009234	,	0.284139803	,	0.081199926	,]],
[[	0.079441055	,	0.293560473	,	0.307490734	,	0.235671399	,],
[	0.263928225	,	0.081626823	,	0.231415555	,	0.289231043	,],
[	0.347644186	,	0.302880941	,	0.074435113	,	0.272633908	,],
[	0.292277995	,	0.37103519	,	0.286328067	,	0.081468299	,]],
[[	0.08369544	,	0.295637942	,	0.309772153	,	0.237901995	,],
[	0.264231179	,	0.078236523	,	0.230770713	,	0.289213158	,],
[	0.344765355	,	0.299269058	,	0.067678973	,	0.269721803	,],
[	0.288896126	,	0.36793424	,	0.283025617	,	0.078913214	,]],
[[	0.080233243	,	0.298976575	,	0.312579381	,	0.236727227	,],
[	0.262640679	,	0.085277523	,	0.235400354	,	0.288141649	,],
[	0.345034702	,	0.304692293	,	0.070670523	,	0.269363815	,],
[	0.289859784	,	0.374582076	,	0.288971082	,	0.078162508	,]],
[[	0.084732158	,	0.304801586	,	0.318191295	,	0.242795946	,],
[	0.259587827	,	0.077639522	,	0.231119515	,	0.284738804	,],
[	0.341896051	,	0.301875684	,	0.068674386	,	0.26606825	,],
[	0.289499677	,	0.374412581	,	0.288628215	,	0.079391357	,]],
[[	0.07963637	,	0.299304714	,	0.312747213	,	0.237035267	,],
[	0.259863771	,	0.080543843	,	0.232086614	,	0.285158869	,],
[	0.344881042	,	0.305461169	,	0.074539009	,	0.269177232	,],
[	0.293108105	,	0.377554471	,	0.29205651	,	0.081114495	,]],
[[	0.082606484	,	0.300226264	,	0.313857842	,	0.238147674	,],
[	0.26241164	,	0.079808736	,	0.23381167	,	0.287411685	,],
[	0.343046923	,	0.302783022	,	0.068831242	,	0.267157728	,],
[	0.288874106	,	0.37370768	,	0.287920208	,	0.077597078	,]],
[[	0.081681313	,	0.301868069	,	0.315286614	,	0.239688606	,],
[	0.25891439	,	0.077821364	,	0.230693398	,	0.284101667	,],
[	0.342456768	,	0.302608928	,	0.070297957	,	0.266649686	,],
[	0.291288722	,	0.376141205	,	0.290503366	,	0.08225406	,]],
[[	0.079878298	,	0.298789934	,	0.312423169	,	0.236570642	,],
[	0.261821998	,	0.083801032	,	0.234415734	,	0.287212464	,],
[	0.346239347	,	0.305793381	,	0.071733137	,	0.270444129	,],
[	0.290582233	,	0.375211539	,	0.289633381	,	0.078699065	,]],
[[	0.086165345	,	0.302530377	,	0.31626524	,	0.240449914	,],
[	0.260246329	,	0.077964451	,	0.231756385	,	0.285237875	,],
[	0.342084113	,	0.301856506	,	0.068386078	,	0.266094248	,],
[	0.289199451	,	0.373982664	,	0.288197148	,	0.078567152	,]],
[[	0.079941776	,	0.299792353	,	0.313323725	,	0.237581289	,],
[	0.259371991	,	0.079421781	,	0.231323629	,	0.284514595	,],
[	0.344143086	,	0.30448799	,	0.07311095	,	0.268312836	,],
[	0.294095839	,	0.378853436	,	0.293199526	,	0.086098134	,]],
[[	0.081626857	,	0.299707392	,	0.31333244	,	0.237407506	,],
[	0.263565268	,	0.080840336	,	0.235055834	,	0.288518158	,],
[	0.343837028	,	0.303393335	,	0.069415975	,	0.267829992	,],
[	0.289108608	,	0.373922579	,	0.288183792	,	0.07759126	,]],
[[	0.082801045	,	0.302905309	,	0.316415935	,	0.240642541	,],
[	0.259080376	,	0.077604278	,	0.230745929	,	0.284117537	,],
[	0.342249823	,	0.302313886	,	0.069537891	,	0.266358188	,],
[	0.290489166	,	0.375461346	,	0.289645141	,	0.081040838	,]],
[[	0.079673646	,	0.298935969	,	0.31241369	,	0.236590781	,],
[	0.261080378	,	0.082405937	,	0.233415712	,	0.286270765	,],
[	0.34644866	,	0.30721189	,	0.076968761	,	0.270741936	,],
[	0.291555065	,	0.376173811	,	0.290585832	,	0.079600045	,]],
[[	0.084657527	,	0.301558712	,	0.315249582	,	0.239497313	,],
[	0.261075043	,	0.078690083	,	0.232460269	,	0.28600197	,],
[	0.342356012	,	0.302203098	,	0.068446045	,	0.266430796	,],
[	0.288955768	,	0.373785602	,	0.288012455	,	0.078051406	,]],
[[	0.080056531	,	0.304776188	,	0.317422355	,	0.237000993	,],
[	0.257149332	,	0.085334857	,	0.235802766	,	0.284276536	,],
[	0.341970586	,	0.307473154	,	0.071816963	,	0.26771876	,],
[	0.288946709	,	0.379251344	,	0.293814679	,	0.079024475	,]],
[[	0.085177097	,	0.311219224	,	0.323651222	,	0.243638901	,],
[	0.254819126	,	0.078597107	,	0.232393194	,	0.281578025	,],
[	0.33789866	,	0.303484294	,	0.068434101	,	0.263269271	,],
[	0.288361793	,	0.378859567	,	0.293139386	,	0.079796112	,]],
[[	0.079572889	,	0.305238735	,	0.317831467	,	0.23751116	,],
[	0.254575586	,	0.080707991	,	0.232720654	,	0.281513582	,],
[	0.339887285	,	0.306033496	,	0.073032123	,	0.265386972	,],
[	0.292522999	,	0.382619435	,	0.297155009	,	0.08219386	,]],
[[	0.082187481	,	0.305955925	,	0.318588327	,	0.238147832	,],
[	0.257894774	,	0.080947626	,	0.235291157	,	0.284441706	,],
[	0.339611146	,	0.305051249	,	0.069495042	,	0.265153651	,],
[	0.28802418	,	0.378440122	,	0.292848339	,	0.078363546	,]],
[[	0.082054257	,	0.307966801	,	0.320435836	,	0.24039986	,],
[	0.253916489	,	0.078523233	,	0.23167168	,	0.280766814	,],
[	0.33797808	,	0.30369124	,	0.069387051	,	0.263376346	,],
[	0.290125008	,	0.380591361	,	0.294996521	,	0.082530496	,]],
[[	0.079726119	,	0.304649717	,	0.317232796	,	0.23688186	,],
[	0.256369758	,	0.083855581	,	0.234785625	,	0.283301151	,],
[	0.342224904	,	0.308623683	,	0.076819325	,	0.267904071	,],
[	0.28991172	,	0.380140501	,	0.294588788	,	0.079622997	,]],
[[	0.08561828	,	0.308099013	,	0.320730261	,	0.240358552	,],
[	0.255515364	,	0.079088329	,	0.23300766	,	0.282187899	,],
[	0.338289175	,	0.303787995	,	0.068439259	,	0.263621092	,],
[	0.288128615	,	0.378570793	,	0.292862912	,	0.079166706	,]],
[[	0.079975008	,	0.305859147	,	0.318316877	,	0.238032342	,],
[	0.254198256	,	0.079850854	,	0.232211375	,	0.281077338	,],
[	0.339176307	,	0.305169933	,	0.071789201	,	0.264680698	,],
[	0.292522874	,	0.382926585	,	0.297393026	,	0.086138643	,]],
[[	0.081240592	,	0.305384852	,	0.317930606	,	0.237525506	,],
[	0.258859627	,	0.087928809	,	0.237640101	,	0.285755107	,],
[	0.34060382	,	0.305981985	,	0.070275423	,	0.26607911	,],
[	0.288358429	,	0.378712972	,	0.29313253	,	0.078356762	,]],
[[	0.083258161	,	0.309199167	,	0.321668977	,	0.241526919	,],
[	0.254191069	,	0.078323499	,	0.23177758	,	0.28088736	,],
[	0.337869427	,	0.303505103	,	0.068882933	,	0.263269989	,],
[	0.289318979	,	0.37975626	,	0.294079187	,	0.081250733	,]],
[[	0.079540671	,	0.304870704	,	0.317400131	,	0.237000086	,],
[	0.255541368	,	0.082474806	,	0.233838537	,	0.282459674	,],
[	0.3412242	,	0.307556295	,	0.075234715	,	0.2666889	,],
[	0.29082108	,	0.380938144	,	0.295420386	,	0.080513183	,]],
[[	0.084235477	,	0.307106951	,	0.319683455	,	0.239396106	,],
[	0.256442236	,	0.079702997	,	0.23384838	,	0.283001685	,],
[	0.338660469	,	0.304073353	,	0.068691707	,	0.26400392	,],
[	0.287988944	,	0.378334986	,	0.29265955	,	0.078622965	,]],
[[	0.079763438	,	0.310965349	,	0.322526802	,	0.237945204	,],
[	0.252678132	,	0.084590571	,	0.235458883	,	0.280394161	,],
[	0.339593405	,	0.311009964	,	0.072258209	,	0.265288989	,],
[	0.288913302	,	0.385277224	,	0.298163029	,	0.078976411	,]],
[[	0.085066705	,	0.317514475	,	0.328996671	,	0.244785423	,],
[	0.252193697	,	0.07997256	,	0.233986726	,	0.279505543	,],
[	0.335342488	,	0.307218969	,	0.069050039	,	0.261016248	,],
[	0.288332972	,	0.38494944	,	0.297638475	,	0.079871284	,]],
[[	0.079407262	,	0.31150848	,	0.323038492	,	0.238571453	,],
[	0.250654765	,	0.080574955	,	0.233002075	,	0.278289796	,],
[	0.337411911	,	0.309680121	,	0.073710135	,	0.263082911	,],
[	0.292251062	,	0.388409104	,	0.301406632	,	0.082084554	,]],
[[	0.081827098	,	0.312098742	,	0.323568916	,	0.239006083	,],
[	0.255213461	,	0.088662876	,	0.238364358	,	0.282885009	,],
[	0.337192413	,	0.308678912	,	0.069990755	,	0.262825427	,],
[	0.28789392	,	0.384378075	,	0.297054997	,	0.078313438	,]],
[[	0.081950211	,	0.314441947	,	0.325874185	,	0.241558459	,],
[	0.250781672	,	0.07930647	,	0.2327855	,	0.278125086	,],
[	0.335564552	,	0.307571	,	0.070079397	,	0.26117533	,],
[	0.290130328	,	0.386706492	,	0.299384631	,	0.082601103	,]],
[[	0.079490466	,	0.311043168	,	0.322549948	,	0.237987617	,],
[	0.252030048	,	0.083376046	,	0.234627298	,	0.279571128	,],
[	0.33980502	,	0.312414152	,	0.077726063	,	0.265596879	,],
[	0.289698536	,	0.385913217	,	0.298911629	,	0.079585774	,]],
[[	0.085187625	,	0.314277489	,	0.325782894	,	0.241284284	,],
[	0.253078172	,	0.080803736	,	0.234682987	,	0.280350216	,],
[	0.335658565	,	0.307345259	,	0.069124188	,	0.26126702	,],
[	0.288031148	,	0.384566732	,	0.297244315	,	0.079204118	,]],
[[	0.079907519	,	0.312207086	,	0.323588941	,	0.239225473	,],
[	0.250578557	,	0.080073623	,	0.232712929	,	0.278083239	,],
[	0.336794987	,	0.308935188	,	0.072425168	,	0.262293383	,],
[	0.292696972	,	0.389189357	,	0.301986398	,	0.086404566	,]],
[[	0.080929528	,	0.31163639	,	0.3231587	,	0.238484541	,],
[	0.254283073	,	0.087076634	,	0.237185003	,	0.28182805	,],
[	0.338094514	,	0.309491285	,	0.070692499	,	0.263614581	,],
[	0.288305156	,	0.384682291	,	0.297411017	,	0.078368458	,]],
[[	0.083222262	,	0.315615137	,	0.327052728	,	0.242741344	,],
[	0.251294956	,	0.079434737	,	0.233156666	,	0.278550126	,],
[	0.335465183	,	0.307245794	,	0.069580587	,	0.260985026	,],
[	0.289340562	,	0.385789155	,	0.298641353	,	0.081359834	,]],
[[	0.079311745	,	0.311149777	,	0.322643224	,	0.238061393	,],
[	0.251479725	,	0.082206671	,	0.233980115	,	0.279059913	,],
[	0.338860861	,	0.311346454	,	0.076070177	,	0.264519465	,],
[	0.290767315	,	0.386899374	,	0.299767176	,	0.080406126	,]],
[[	0.083719091	,	0.313382895	,	0.32494157	,	0.24024714	,],
[	0.254208168	,	0.081672356	,	0.235836017	,	0.281397055	,],
[	0.336357525	,	0.307764153	,	0.069351187	,	0.261731945	,],
[	0.287926564	,	0.384348344	,	0.297136942	,	0.07865107	,]],
[[	0.080346412	,	0.318292296	,	0.329644778	,	0.240129656	,],
[	0.247981663	,	0.083829015	,	0.2368906	,	0.276915917	,],
[	0.337169577	,	0.314881893	,	0.077587255	,	0.262712642	,],
[	0.290082369	,	0.392309359	,	0.304430961	,	0.079763666	,]],
[[	0.08532549	,	0.324453517	,	0.335552731	,	0.246598032	,],
[	0.248968837	,	0.081177117	,	0.236829044	,	0.277664816	,],
[	0.333788237	,	0.310558177	,	0.069844093	,	0.25913285	,],
[	0.288444284	,	0.390956454	,	0.30284931	,	0.079263459	,]],
[[	0.079756475	,	0.318620501	,	0.329883429	,	0.24053653	,],
[	0.246510278	,	0.080513685	,	0.234919421	,	0.275316553	,],
[	0.334366816	,	0.311603966	,	0.072644176	,	0.259662376	,],
[	0.292876249	,	0.395434118	,	0.30750999	,	0.086262639	,]],
[[	0.082527765	,	0.319307616	,	0.330750478	,	0.241316072	,],
[	0.250108699	,	0.087517371	,	0.239381822	,	0.279187831	,],
[	0.336366547	,	0.312941182	,	0.07210287	,	0.26183224	,],
[	0.288677584	,	0.391026293	,	0.302889184	,	0.078565442	,]],
[[	0.082180457	,	0.321388984	,	0.332473723	,	0.243401714	,],
[	0.247182644	,	0.079954254	,	0.235194624	,	0.275872265	,],
[	0.333209806	,	0.310064917	,	0.06999038	,	0.258606454	,],
[	0.28962048	,	0.39214746	,	0.304117042	,	0.081408766	,]],
[[	0.079942802	,	0.318112116	,	0.329422054	,	0.239877307	,],
[	0.247324401	,	0.082713933	,	0.236100818	,	0.276287598	,],
[	0.336148651	,	0.313799444	,	0.076006202	,	0.261603128	,],
[	0.291319034	,	0.393347878	,	0.305411147	,	0.080664518	,]],
[[	0.085868182	,	0.321708352	,	0.332989524	,	0.243545287	,],
[	0.250125366	,	0.082217295	,	0.237927637	,	0.278653821	,],
[	0.334387416	,	0.31101564	,	0.070236517	,	0.259728262	,],
[	0.288362663	,	0.390891051	,	0.302630857	,	0.07875949	,]],
[[	0.08031407	,	0.319264479	,	0.330486984	,	0.241107564	,],
[	0.246511528	,	0.080126035	,	0.23473763	,	0.275210777	,],
[	0.333772189	,	0.310994394	,	0.071633402	,	0.259122262	,],
[	0.291812825	,	0.394415401	,	0.306312123	,	0.084732385	,]],
[[	0.081456554	,	0.318844884	,	0.330121129	,	0.240653383	,],
[	0.249180058	,	0.085997952	,	0.238281493	,	0.27819477	,],
[	0.337695022	,	0.314174567	,	0.07318427	,	0.263112269	,],
[	0.289327378	,	0.391521166	,	0.30351719	,	0.078978709	,]],
[[	0.083326337	,	0.322450462	,	0.333592259	,	0.24450144	,],
[	0.247919868	,	0.080277811	,	0.235839306	,	0.27649743	,],
[	0.333503363	,	0.310317561	,	0.069844437	,	0.258723874	,],
[	0.289133371	,	0.391678447	,	0.303479861	,	0.08035892	,]],
[[	0.079576718	,	0.318115633	,	0.329342427	,	0.239985058	,],
[	0.24687268	,	0.08172268	,	0.23555226	,	0.275771873	,],
[	0.335520355	,	0.312923497	,	0.074563925	,	0.260835758	,],
[	0.292316186	,	0.394314776	,	0.306494982	,	0.081635981	,]],
[[	0.084491332	,	0.320646442	,	0.332070423	,	0.242558381	,],
[	0.251412718	,	0.083303273	,	0.239155436	,	0.279946539	,],
[	0.335239237	,	0.311749018	,	0.0708911	,	0.260513332	,],
[	0.288435799	,	0.39086451	,	0.302641573	,	0.07852983	,]],
[[	0.080343104	,	0.322401891	,	0.332739683	,	0.239279589	,],
[	0.242620655	,	0.083364599	,	0.236069014	,	0.272140751	,],
[	0.332141248	,	0.315163925	,	0.07650041	,	0.258446065	,],
[	0.288592312	,	0.395092876	,	0.307747288	,	0.080161209	,]],
[[	0.085867866	,	0.32894531	,	0.339161973	,	0.246136136	,],
[	0.245079489	,	0.082371712	,	0.237538111	,	0.274157576	,],
[	0.32977021	,	0.311836182	,	0.070057023	,	0.255874053	,],
[	0.286683259	,	0.39357589	,	0.305974413	,	0.079473869	,]],
[[	0.080097815	,	0.322994884	,	0.333176236	,	0.239856714	,],
[	0.241593506	,	0.080727383	,	0.234527517	,	0.271011949	,],
[	0.329645737	,	0.3121666	,	0.071981855	,	0.255759689	,],
[	0.291106433	,	0.397974588	,	0.31044647	,	0.086377668	,]],
[[	0.082487363	,	0.323499828	,	0.333818631	,	0.24038467	,],
[	0.244602889	,	0.086877686	,	0.238340214	,	0.2741043	,],
[	0.332792908	,	0.314583119	,	0.072705065	,	0.258989579	,],
[	0.287109579	,	0.393759971	,	0.306187595	,	0.078934494	,]],
[[	0.082566533	,	0.325778077	,	0.33592721	,	0.242738085	,],
[	0.242893248	,	0.080659014	,	0.235439787	,	0.271977596	,],
[	0.32906822	,	0.311203517	,	0.069814966	,	0.255071423	,],
[	0.287973702	,	0.394805894	,	0.307257761	,	0.081610832	,]],
[[	0.080110406	,	0.322379444	,	0.332659174	,	0.239222209	,],
[	0.242324266	,	0.082412818	,	0.235488692	,	0.271625294	,],
[	0.331393187	,	0.314179441	,	0.07507366	,	0.25756172	,],
[	0.28962221	,	0.395995426	,	0.308621998	,	0.080997908	,]],
[[	0.085928707	,	0.325711584	,	0.336053598	,	0.242627869	,],
[	0.24618767	,	0.083391683	,	0.238556453	,	0.275187094	,],
[	0.330483986	,	0.312470778	,	0.070622777	,	0.256605527	,],
[	0.286666564	,	0.393420687	,	0.305868092	,	0.079007225	,]],
[[	0.080636015	,	0.323601622	,	0.333821074	,	0.240543935	,],
[	0.241835412	,	0.080405417	,	0.234600391	,	0.270986657	,],
[	0.329257153	,	0.311679409	,	0.07115267	,	0.255323047	,],
[	0.290168828	,	0.397036645	,	0.309434341	,	0.084926525	,]],
[[	0.081624683	,	0.322966707	,	0.333318737	,	0.239875513	,],
[	0.243883389	,	0.085387457	,	0.237395291	,	0.273098039	,],
[	0.333847721	,	0.317064379	,	0.079052709	,	0.260103675	,],
[	0.287629565	,	0.394192355	,	0.30669722	,	0.079195998	,]],
[[	0.083828346	,	0.326955742	,	0.337170879	,	0.243982939	,],
[	0.24364248	,	0.081166207	,	0.236108203	,	0.272696974	,],
[	0.329246845	,	0.311246612	,	0.069770525	,	0.255309619	,],
[	0.287388172	,	0.394257302	,	0.306671273	,	0.080663214	,]],
[[	0.07996475	,	0.322445467	,	0.332740644	,	0.239217136	,],
[	0.241888984	,	0.081530254	,	0.235029899	,	0.271227015	,],
[	0.330671573	,	0.313350048	,	0.073653877	,	0.256666589	,],
[	0.290795119	,	0.397107292	,	0.309772579	,	0.08206923	,]],
[[	0.084382028	,	0.324717565	,	0.335084139	,	0.241549724	,],
[	0.246374826	,	0.089413085	,	0.240214625	,	0.275730039	,],
[	0.331471506	,	0.313253832	,	0.071260909	,	0.257417168	,],
[	0.28672892	,	0.393450926	,	0.305840627	,	0.078843856	,]],
[[	0.080251676	,	0.326603123	,	0.336198711	,	0.2385596	,],
[	0.236742338	,	0.083766909	,	0.236031653	,	0.266946137	,],
[	0.327525176	,	0.31676416	,	0.077391521	,	0.254006627	,],
[	0.286487171	,	0.39835319	,	0.31003624	,	0.07905322	,]],
[[	0.084622797	,	0.332189285	,	0.341728052	,	0.244515431	,],
[	0.239114524	,	0.082378044	,	0.237348846	,	0.268824749	,],
[	0.324878679	,	0.313042523	,	0.07034826	,	0.251106217	,],
[	0.285186044	,	0.397373575	,	0.308920349	,	0.079210618	,]],
[[	0.079397956	,	0.326661688	,	0.336358714	,	0.23883055	,],
[	0.235701268	,	0.080769223	,	0.234419602	,	0.265617289	,],
[	0.324999379	,	0.313654475	,	0.072730041	,	0.25124707	,],
[	0.290192426	,	0.402303039	,	0.313996496	,	0.086937868	,]],
[[	0.082678179	,	0.327985037	,	0.337693479	,	0.239966139	,],
[	0.238636128	,	0.087007114	,	0.238185352	,	0.268808632	,],
[	0.327830023	,	0.315657201	,	0.072831589	,	0.254021093	,],
[	0.285209947	,	0.397121875	,	0.30866221	,	0.078100519	,]],
[[	0.081562552	,	0.329203495	,	0.33874293	,	0.241335607	,],
[	0.236864955	,	0.080748995	,	0.235223524	,	0.266695932	,],
[	0.324209827	,	0.312519554	,	0.070317842	,	0.250293727	,],
[	0.286743093	,	0.398865671	,	0.310415869	,	0.081650874	,]],
[[	0.079798697	,	0.326381317	,	0.336063209	,	0.238416938	,],
[	0.236293454	,	0.082656908	,	0.235328908	,	0.266362803	,],
[	0.326737811	,	0.31569568	,	0.075881	,	0.253071924	,],
[	0.287439413	,	0.399094007	,	0.310855767	,	0.079732875	,]],
[[	0.086206766	,	0.333680694	,	0.343186842	,	0.246059276	,],
[	0.240312243	,	0.08345555	,	0.238354238	,	0.269913018	,],
[	0.325644368	,	0.313601098	,	0.070900704	,	0.251826366	,],
[	0.285041227	,	0.397160885	,	0.308697843	,	0.078584242	,]],
[[	0.079833676	,	0.327255733	,	0.336796978	,	0.239299178	,],
[	0.235957383	,	0.080510125	,	0.234421251	,	0.265741794	,],
[	0.324604084	,	0.313159897	,	0.071695833	,	0.250722746	,],
[	0.289192198	,	0.401240904	,	0.312868657	,	0.085281632	,]],
[[	0.081662952	,	0.327285479	,	0.337017473	,	0.239292831	,],
[	0.237991917	,	0.085702959	,	0.237311764	,	0.267968227	,],
[	0.329080921	,	0.316872744	,	0.073983634	,	0.255335153	,],
[	0.285572703	,	0.397566912	,	0.309140173	,	0.078255233	,]],
[[	0.082811723	,	0.330438281	,	0.339924881	,	0.242600502	,],
[	0.237792986	,	0.081283633	,	0.235948745	,	0.267427547	,],
[	0.324435303	,	0.312709419	,	0.070263288	,	0.250501071	,],
[	0.286011357	,	0.398203294	,	0.309708363	,	0.080492455	,]],
[[	0.079518796	,	0.326368802	,	0.335977538	,	0.238373956	,],
[	0.235973779	,	0.081671598	,	0.234801556	,	0.265827954	,],
[	0.326065626	,	0.314815633	,	0.074492599	,	0.252182368	,],
[	0.288432239	,	0.400158588	,	0.311832884	,	0.080648868	,]],
[[	0.084676844	,	0.329172596	,	0.338850827	,	0.24122639	,],
[	0.240405038	,	0.089708237	,	0.240084496	,	0.27049011	,],
[	0.326524918	,	0.314390293	,	0.071500996	,	0.252654052	,],
[	0.285099253	,	0.397152602	,	0.308539807	,	0.0781306	,]],
[[	0.092513941	,	0.215190841	,	0.249372386	,	0.26088211	,],
[	0.358884919	,	0.075555984	,	0.241012911	,	0.380003007	,],
[	0.427671496	,	0.28260851	,	0.069302191	,	0.349074516	,],
[	0.325111263	,	0.305076683	,	0.230626144	,	0.089713129	,]],
[[	0.089560369	,	0.215779637	,	0.24963516	,	0.260575517	,],
[	0.36268971	,	0.082149816	,	0.245267887	,	0.383850369	,],
[	0.430610637	,	0.285948213	,	0.070832737	,	0.352314123	,],
[	0.321607032	,	0.301404687	,	0.226674415	,	0.086987633	,]],
[[	0.08908683	,	0.213459007	,	0.247545643	,	0.258689653	,],
[	0.359436142	,	0.075290253	,	0.241663897	,	0.380450095	,],
[	0.427064758	,	0.281969119	,	0.067544494	,	0.348473305	,],
[	0.324617344	,	0.304001555	,	0.229512834	,	0.092217166	,]],
[[	0.093649808	,	0.220279095	,	0.253895638	,	0.264736502	,],
[	0.359831408	,	0.077536488	,	0.242002982	,	0.380966799	,],
[	0.429338138	,	0.284376181	,	0.072055279	,	0.350838782	,],
[	0.322916319	,	0.302738422	,	0.228140366	,	0.087600157	,]],
[[	0.088253594	,	0.213824828	,	0.247885603	,	0.258841477	,],
[	0.362020828	,	0.07756956	,	0.244336669	,	0.382975839	,],
[	0.428441293	,	0.283419736	,	0.068523859	,	0.349913329	,],
[	0.322154749	,	0.301614444	,	0.227138699	,	0.0882796	,]],
[[	0.0912916	,	0.214481271	,	0.248572947	,	0.259893672	,],
[	0.35879758	,	0.075204632	,	0.240898352	,	0.379940975	,],
[	0.427388725	,	0.282287982	,	0.068561247	,	0.348713381	,],
[	0.326223238	,	0.306157775	,	0.231793077	,	0.090899446	,]],
[[	0.090516733	,	0.216893207	,	0.250624652	,	0.261501045	,],
[	0.361904524	,	0.080798627	,	0.244222128	,	0.382916194	,],
[	0.43186326	,	0.287061372	,	0.075851636	,	0.353368411	,],
[	0.321716506	,	0.301481042	,	0.226938923	,	0.086860137	,]],
[[	0.088564967	,	0.213380464	,	0.247378776	,	0.258531054	,],
[	0.360063329	,	0.075724758	,	0.242144986	,	0.380941532	,],
[	0.427297654	,	0.282308126	,	0.067537878	,	0.348659819	,],
[	0.323856191	,	0.303215717	,	0.228615154	,	0.09087328	,]],
[[	0.094757587	,	0.216553264	,	0.25067892	,	0.262224838	,],
[	0.35939149	,	0.076657454	,	0.241555672	,	0.380393011	,],
[	0.428751922	,	0.283767569	,	0.070875008	,	0.350032113	,],
[	0.323669061	,	0.303549487	,	0.229086363	,	0.088303719	,]],
[[	0.08871608	,	0.214473852	,	0.248447094	,	0.259385422	,],
[	0.363278992	,	0.078761903	,	0.245461439	,	0.384067791	,],
[	0.429324843	,	0.284518352	,	0.069423874	,	0.350825588	,],
[	0.32195836	,	0.301404479	,	0.226755861	,	0.087525731	,]],
[[	0.09029835	,	0.213922484	,	0.248031672	,	0.259334364	,],
[	0.359014549	,	0.075154775	,	0.241050931	,	0.379963666	,],
[	0.427272305	,	0.282126044	,	0.068034866	,	0.348554902	,],
[	0.32643791	,	0.305565597	,	0.231039701	,	0.094518614	,]],
[[	0.091612729	,	0.21810285	,	0.25183628	,	0.262653664	,],
[	0.361062129	,	0.079364897	,	0.243340359	,	0.382116457	,],
[	0.430930689	,	0.286038697	,	0.074332503	,	0.352294468	,],
[	0.322195543	,	0.302055435	,	0.227368562	,	0.087041658	,]],
[[	0.089234568	,	0.214965459	,	0.247733236	,	0.254101881	,],
[	0.347188699	,	0.073600171	,	0.236114448	,	0.368617887	,],
[	0.413949131	,	0.277141409	,	0.065571943	,	0.337327784	,],
[	0.317684968	,	0.30356286	,	0.229858226	,	0.092137245	,]],
[[	0.087359599	,	0.216509208	,	0.248844066	,	0.254742948	,],
[	0.348514242	,	0.077169348	,	0.237719404	,	0.370162676	,],
[	0.416668033	,	0.280067798	,	0.070574593	,	0.340056124	,],
[	0.313654666	,	0.300041724	,	0.226263484	,	0.084776658	,]],
[[	0.086120704	,	0.213561629	,	0.246115087	,	0.252286477	,],
[	0.349356805	,	0.075278149	,	0.238380095	,	0.370699697	,],
[	0.41537561	,	0.278663523	,	0.066504715	,	0.338819769	,],
[	0.314568945	,	0.30061575	,	0.226882555	,	0.087357928	,]],
[[	0.091760039	,	0.221192366	,	0.253277964	,	0.259221161	,],
[	0.347009719	,	0.074129988	,	0.236008373	,	0.368506491	,],
[	0.414478559	,	0.27759277	,	0.066847763	,	0.337762841	,],
[	0.316306849	,	0.302858658	,	0.229302558	,	0.087089539	,]],
[[	0.085857545	,	0.214405324	,	0.246889036	,	0.252860277	,],
[	0.350835993	,	0.080612835	,	0.240199464	,	0.372319082	,],
[	0.418682567	,	0.282242859	,	0.069868553	,	0.342283546	,],
[	0.313210872	,	0.299477619	,	0.225586716	,	0.084859126	,]],
[[	0.088189012	,	0.214418252	,	0.247062074	,	0.253337288	,],
[	0.347732158	,	0.07385403	,	0.236599288	,	0.369062049	,],
[	0.414229293	,	0.277448537	,	0.065618027	,	0.337562233	,],
[	0.316741091	,	0.302612378	,	0.228953382	,	0.09061004	,]],
[[	0.088485695	,	0.217704153	,	0.249976774	,	0.255894965	,],
[	0.347892443	,	0.076043754	,	0.236958517	,	0.369425697	,],
[	0.415970757	,	0.279159331	,	0.069325767	,	0.339260687	,],
[	0.314235725	,	0.300680968	,	0.226841837	,	0.085136577	,]],
[[	0.085784782	,	0.21352538	,	0.246081773	,	0.25212983	,],
[	0.350331544	,	0.07619382	,	0.239339956	,	0.371582671	,],
[	0.416202727	,	0.279519011	,	0.067221701	,	0.339596143	,],
[	0.314002124	,	0.300132933	,	0.226243834	,	0.086366786	,]],
[[	0.091342885	,	0.216189231	,	0.248841622	,	0.25530273	,],
[	0.347016147	,	0.07375505	,	0.235910522	,	0.368334203	,],
[	0.414231097	,	0.277353066	,	0.066138126	,	0.337516036	,],
[	0.317458271	,	0.304000917	,	0.230321355	,	0.088155292	,]],
[[	0.086399444	,	0.215213052	,	0.247555896	,	0.253448147	,],
[	0.349910602	,	0.079159939	,	0.239110702	,	0.371362161	,],
[	0.41816424	,	0.281617043	,	0.072898193	,	0.34165758	,],
[	0.313382222	,	0.299696061	,	0.225934876	,	0.084681108	,]],
[[	0.087313602	,	0.21395951	,	0.24656251	,	0.252798642	,],
[	0.34813424	,	0.074237165	,	0.237089209	,	0.369451261	,],
[	0.414561507	,	0.277731753	,	0.065800233	,	0.337855526	,],
[	0.315882393	,	0.301855726	,	0.228089707	,	0.089177997	,]],
[[	0.089735472	,	0.219071087	,	0.251249388	,	0.257175133	,],
[	0.347484566	,	0.075183493	,	0.236528385	,	0.368954602	,],
[	0.415285137	,	0.278497781	,	0.068161167	,	0.33855553	,],
[	0.315006597	,	0.301473391	,	0.227775618	,	0.085766471	,]],
[[	0.086951465	,	0.215782435	,	0.247503381	,	0.249296765	,],
[	0.33708135	,	0.072520867	,	0.232701376	,	0.358996753	,],
[	0.404164195	,	0.274138432	,	0.064689924	,	0.329297462	,],
[	0.309234603	,	0.301396078	,	0.228483418	,	0.089166346	,]],
[[	0.085814897	,	0.217720944	,	0.249216016	,	0.250673919	,],
[	0.337465446	,	0.074857829	,	0.233240288	,	0.35954176	,],
[	0.4057123	,	0.275750193	,	0.06817022	,	0.33083478	,],
[	0.306378001	,	0.298982508	,	0.225943392	,	0.083069887	,]],
[[	0.0840869	,	0.214469156	,	0.246128089	,	0.247729012	,],
[	0.340002371	,	0.075135011	,	0.235607822	,	0.361753181	,],
[	0.406194609	,	0.276383877	,	0.066388763	,	0.331463585	,],
[	0.306328923	,	0.298683067	,	0.225628968	,	0.084731416	,]],
[[	0.090282736	,	0.222613357	,	0.253807606	,	0.255251416	,],
[	0.336475965	,	0.072544124	,	0.232006187	,	0.35836397	,],
[	0.404023686	,	0.274000226	,	0.065182661	,	0.329171874	,],
[	0.309334355	,	0.302015816	,	0.229079507	,	0.085854131	,]],
[[	0.084172969	,	0.215589334	,	0.247057206	,	0.248585227	,],
[	0.339345704	,	0.07798825	,	0.235203877	,	0.361344951	,],
[	0.407927705	,	0.278197119	,	0.071708479	,	0.333111358	,],
[	0.305621709	,	0.298046875	,	0.224983853	,	0.082828404	,]],
[[	0.086003235	,	0.215248719	,	0.246958635	,	0.24868174	,],
[	0.337810938	,	0.073034999	,	0.233245169	,	0.359492564	,],
[	0.404569142	,	0.274545428	,	0.064919888	,	0.32972534	,],
[	0.308460401	,	0.300581422	,	0.227518368	,	0.087703964	,]],
[[	0.086963698	,	0.219062003	,	0.250459431	,	0.251819019	,],
[	0.337037575	,	0.073973068	,	0.232647311	,	0.358915791	,],
[	0.405025624	,	0.275204389	,	0.067025453	,	0.330169681	,],
[	0.307095569	,	0.299677243	,	0.226676103	,	0.0836594	,]],
[[	0.08393155	,	0.214687897	,	0.246291024	,	0.247713098	,],
[	0.341212295	,	0.076325681	,	0.236864589	,	0.362939076	,],
[	0.407237708	,	0.277409609	,	0.06736029	,	0.332482026	,],
[	0.305933194	,	0.298333669	,	0.225239386	,	0.083828638	,]],
[[	0.089172899	,	0.216998375	,	0.248659371	,	0.250607897	,],
[	0.33674859	,	0.072458542	,	0.232173742	,	0.358546783	,],
[	0.404119014	,	0.274043243	,	0.064867248	,	0.329124016	,],
[	0.310814007	,	0.303349141	,	0.230500713	,	0.087061101	,]],
[[	0.084699329	,	0.216362356	,	0.247864056	,	0.249175606	,],
[	0.338463656	,	0.076575768	,	0.234315278	,	0.360405623	,],
[	0.40689394	,	0.277090152	,	0.070129346	,	0.332054574	,],
[	0.305856958	,	0.298276054	,	0.225201961	,	0.082745989	,]],
[[	0.085198224	,	0.214851179	,	0.246487835	,	0.248168896	,],
[	0.338645604	,	0.073775424	,	0.234118524	,	0.360317859	,],
[	0.405194169	,	0.275269249	,	0.065370237	,	0.330369812	,],
[	0.307494443	,	0.299624593	,	0.226655722	,	0.086412455	,]],
[[	0.08824647	,	0.220381901	,	0.251687406	,	0.253068114	,],
[	0.336719914	,	0.07315211	,	0.232285488	,	0.358570457	,],
[	0.404532113	,	0.274588714	,	0.066218258	,	0.329483164	,],
[	0.307979345	,	0.300514066	,	0.22759382	,	0.084351217	,]],
[[	0.085897694	,	0.219823792	,	0.249362113	,	0.246785485	,],
[	0.332409969	,	0.073187083	,	0.232185513	,	0.354383209	,],
[	0.399857328	,	0.275423374	,	0.065191781	,	0.325387565	,],
[	0.306317924	,	0.303349381	,	0.230041238	,	0.087147048	,]],
[[	0.085905457	,	0.222869807	,	0.252087338	,	0.249192029	,],
[	0.331032759	,	0.073326911	,	0.230841802	,	0.353131084	,],
[	0.399534038	,	0.275152251	,	0.066404899	,	0.325043492	,],
[	0.305741875	,	0.303289464	,	0.229996202	,	0.083820372	,]],
[[	0.083476411	,	0.218818757	,	0.248253254	,	0.245552885	,],
[	0.33559793	,	0.080864572	,	0.235957289	,	0.357762979	,],
[	0.402891401	,	0.278627584	,	0.068060786	,	0.328527862	,],
[	0.304321336	,	0.30164359	,	0.22818381	,	0.0836374	,]],
[[	0.089302744	,	0.22180482	,	0.25137482	,	0.2489829	,],
[	0.331046119	,	0.072206454	,	0.23077971	,	0.352977047	,],
[	0.39901549	,	0.274523562	,	0.064725163	,	0.324392962	,],
[	0.308959587	,	0.305786707	,	0.232587015	,	0.090801457	,]],
[[	0.083907481	,	0.220414901	,	0.24965523	,	0.246847164	,],
[	0.33240858	,	0.075715535	,	0.232293761	,	0.354496477	,],
[	0.401250814	,	0.276942255	,	0.069122463	,	0.326595514	,],
[	0.304346705	,	0.301811112	,	0.22841448	,	0.082731952	,]],
[[	0.08501698	,	0.219394879	,	0.248856798	,	0.246231322	,],
[	0.333408864	,	0.07397828	,	0.233150469	,	0.355399596	,],
[	0.40062555	,	0.27617927	,	0.065771433	,	0.326120273	,],
[	0.305589807	,	0.302701691	,	0.229400748	,	0.085865459	,]],
[[	0.0871359	,	0.224124162	,	0.253226536	,	0.250340353	,],
[	0.330934447	,	0.072782404	,	0.230607303	,	0.352955204	,],
[	0.399293239	,	0.274829029	,	0.06574999	,	0.32468668	,],
[	0.306788222	,	0.304149566	,	0.230850148	,	0.084634798	,]],
[[	0.083350489	,	0.219023658	,	0.248439958	,	0.24561712	,],
[	0.334668762	,	0.079250544	,	0.234833883	,	0.356686298	,],
[	0.403607158	,	0.279472191	,	0.072931975	,	0.329137479	,],
[	0.304031007	,	0.301373431	,	0.227950061	,	0.083041599	,]],
[[	0.087914729	,	0.220988811	,	0.250406795	,	0.248069555	,],
[	0.331629607	,	0.072582929	,	0.231289087	,	0.353538272	,],
[	0.399358086	,	0.274762092	,	0.064787624	,	0.324714737	,],
[	0.307854549	,	0.304863975	,	0.231539282	,	0.089214068	,]],
[[	0.084628885	,	0.221228738	,	0.250498401	,	0.247604124	,],
[	0.331760691	,	0.074687762	,	0.231650885	,	0.353799984	,],
[	0.400623283	,	0.276175887	,	0.067961254	,	0.32587976	,],
[	0.304831559	,	0.30224791	,	0.22882201	,	0.082986079	,]],
[[	0.084340798	,	0.21897337	,	0.248573613	,	0.245785998	,],
[	0.334517288	,	0.074992815	,	0.234270539	,	0.356339943	,],
[	0.401464072	,	0.277057753	,	0.066536607	,	0.326845895	,],
[	0.304943015	,	0.302142781	,	0.228715032	,	0.084749272	,]],
[[	0.088563089	,	0.225513691	,	0.25453002	,	0.251762383	,],
[	0.330980833	,	0.072460831	,	0.23071222	,	0.352884935	,],
[	0.399052381	,	0.274545483	,	0.065107985	,	0.324394269	,],
[	0.307817959	,	0.305246193	,	0.232067757	,	0.085578751	,]],
[[	0.084639352	,	0.222963401	,	0.252370251	,	0.245175094	,],
[	0.326319393	,	0.074501143	,	0.233292354	,	0.3500251	,],
[	0.391812375	,	0.273720952	,	0.064945521	,	0.319482824	,],
[	0.300934629	,	0.304183043	,	0.232700285	,	0.085715635	,]],
[[	0.084785874	,	0.226106317	,	0.255224369	,	0.247650939	,],
[	0.323188073	,	0.072633448	,	0.230231188	,	0.347020368	,],
[	0.390847007	,	0.272815456	,	0.065520337	,	0.318576324	,],
[	0.300769549	,	0.304391385	,	0.232870454	,	0.082798599	,]],
[[	0.082241112	,	0.221995549	,	0.251399804	,	0.243954714	,],
[	0.326294291	,	0.07845981	,	0.233683451	,	0.350114902	,],
[	0.395211858	,	0.277296441	,	0.0682953	,	0.323060563	,],
[	0.299052882	,	0.302381558	,	0.230915892	,	0.082326264	,]],
[[	0.087893106	,	0.225011995	,	0.254396132	,	0.247295113	,],
[	0.324298459	,	0.072779079	,	0.231154726	,	0.347962933	,],
[	0.390690257	,	0.272539573	,	0.064143591	,	0.318337363	,],
[	0.303486485	,	0.306456971	,	0.235127038	,	0.089320324	,]],
[[	0.082658166	,	0.223705532	,	0.252810496	,	0.245275122	,],
[	0.323833109	,	0.074180857	,	0.230860236	,	0.347546479	,],
[	0.392242349	,	0.274414703	,	0.068020482	,	0.319934433	,],
[	0.299302011	,	0.302806654	,	0.231250446	,	0.081667094	,]],
[[	0.083787569	,	0.222489402	,	0.25183817	,	0.244528682	,],
[	0.32753449	,	0.07562883	,	0.234425129	,	0.350996274	,],
[	0.392595388	,	0.274573723	,	0.065719677	,	0.320315429	,],
[	0.300329975	,	0.303430405	,	0.232013281	,	0.084458819	,]],
[[	0.085949842	,	0.227391162	,	0.256397639	,	0.248875761	,],
[	0.323310662	,	0.072361157	,	0.230175598	,	0.347026754	,],
[	0.390690254	,	0.272610178	,	0.064902333	,	0.318263383	,],
[	0.301800481	,	0.305341573	,	0.23392557	,	0.083707776	,]],
[[	0.082103229	,	0.222374894	,	0.251654493	,	0.244189266	,],
[	0.325419913	,	0.077046601	,	0.23267234	,	0.349201753	,],
[	0.39455691	,	0.276924232	,	0.071706156	,	0.322363701	,],
[	0.298969457	,	0.302196635	,	0.230700881	,	0.081897635	,]],
[[	0.086559155	,	0.224079941	,	0.253487357	,	0.246309338	,],
[	0.324992762	,	0.073306003	,	0.23182127	,	0.348514206	,],
[	0.391197443	,	0.272987918	,	0.064267844	,	0.318705948	,],
[	0.302407065	,	0.305359539	,	0.234021	,	0.087853355	,]],
[[	0.08348038	,	0.2245436	,	0.253608147	,	0.246048395	,],
[	0.323486112	,	0.073453855	,	0.230515168	,	0.347231579	,],
[	0.391703594	,	0.273726997	,	0.066932495	,	0.319354314	,],
[	0.29988938	,	0.303301204	,	0.231780775	,	0.082006691	,]],
[[	0.083030771	,	0.222316503	,	0.251535132	,	0.244207038	,],
[	0.327875691	,	0.080877296	,	0.235390548	,	0.351664658	,],
[	0.393586801	,	0.275611141	,	0.066601944	,	0.321340583	,],
[	0.299733851	,	0.302976266	,	0.231450682	,	0.083513926	,]],
[[	0.08735351	,	0.228888049	,	0.257810043	,	0.250176983	,],
[	0.323555572	,	0.072411525	,	0.230428409	,	0.347164193	,],
[	0.390649877	,	0.272477918	,	0.064458663	,	0.318099986	,],
[	0.302907587	,	0.3065348	,	0.235024384	,	0.084836924	,]],
[[	0.083831213	,	0.225804549	,	0.254900379	,	0.243373221	,],
[	0.320423912	,	0.073832432	,	0.231934094	,	0.344289565	,],
[	0.386731077	,	0.273731276	,	0.065404257	,	0.314909849	,],
[	0.297614731	,	0.305505878	,	0.234003252	,	0.084846703	,]],
[[	0.083475122	,	0.228526854	,	0.257406279	,	0.245578676	,],
[	0.3166694	,	0.071207332	,	0.228232456	,	0.340630734	,],
[	0.384697546	,	0.271651648	,	0.064548314	,	0.312759953	,],
[	0.297539878	,	0.305799486	,	0.234310668	,	0.082023781	,]],
[[	0.081274096	,	0.224735686	,	0.253760402	,	0.242093451	,],
[	0.319435076	,	0.076789932	,	0.231475234	,	0.343596401	,],
[	0.388532122	,	0.275871244	,	0.071396481	,	0.316803639	,],
[	0.295695061	,	0.303831381	,	0.232272846	,	0.081473849	,]],
[[	0.087205486	,	0.22784603	,	0.25691037	,	0.245616951	,],
[	0.3180645	,	0.071676028	,	0.229602839	,	0.341917445	,],
[	0.3851994	,	0.272008582	,	0.064060934	,	0.31328389	,],
[	0.300044695	,	0.307781334	,	0.236395104	,	0.08840834	,]],
[[	0.081601875	,	0.226203606	,	0.255087411	,	0.243298568	,],
[	0.317170999	,	0.072797236	,	0.228894368	,	0.34113012	,],
[	0.385700235	,	0.2727907	,	0.066645756	,	0.313880087	,],
[	0.295939233	,	0.304148699	,	0.232533188	,	0.080748578	,]],
[[	0.082874365	,	0.225296804	,	0.254370903	,	0.24278141	,],
[	0.321629308	,	0.07495217	,	0.233174442	,	0.345353125	,],
[	0.387738678	,	0.274684382	,	0.066288543	,	0.31586466	,],
[	0.296989184	,	0.304944542	,	0.233303137	,	0.08359099	,]],
[[	0.084713955	,	0.230034843	,	0.25862002	,	0.246827252	,],
[	0.316914053	,	0.071066978	,	0.228334835	,	0.340718744	,],
[	0.384737306	,	0.271632816	,	0.064179448	,	0.312743445	,],
[	0.298616115	,	0.306852178	,	0.235315883	,	0.082974986	,]],
[[	0.08106636	,	0.224997766	,	0.25393247	,	0.242185103	,],
[	0.318591622	,	0.075400391	,	0.230463858	,	0.34259042	,],
[	0.38777125	,	0.274924668	,	0.069829346	,	0.315803333	,],
[	0.295637876	,	0.303706545	,	0.232148665	,	0.080991078	,]],
[[	0.085782736	,	0.226946515	,	0.256063229	,	0.244643584	,],
[	0.318869743	,	0.072382672	,	0.23032053	,	0.342680094	,],
[	0.385817532	,	0.272625189	,	0.064451193	,	0.313819159	,],
[	0.299233871	,	0.30700147	,	0.235493382	,	0.086962997	,]],
[[	0.082325436	,	0.227080131	,	0.255906208	,	0.244150235	,],
[	0.316931365	,	0.071994654	,	0.228452657	,	0.340876502	,],
[	0.385261429	,	0.272149395	,	0.065707206	,	0.313273387	,],
[	0.296542093	,	0.304716273	,	0.233193484	,	0.081079271	,]],
[[	0.08213857	,	0.225012958	,	0.254077059	,	0.242355362	,],
[	0.321064523	,	0.079198773	,	0.233150645	,	0.345080372	,],
[	0.388884541	,	0.275903654	,	0.067442603	,	0.316988444	,],
[	0.296445137	,	0.304337753	,	0.23276369	,	0.082600621	,]],
[[	0.08616161	,	0.231302099	,	0.259975461	,	0.248129199	,],
[	0.3173255	,	0.071197774	,	0.228762152	,	0.341108411	,],
[	0.384806445	,	0.271560499	,	0.064045671	,	0.312738296	,],
[	0.299729478	,	0.30793731	,	0.236518925	,	0.083879461	,]],
[[	0.083864877	,	0.232166821	,	0.260629679	,	0.244231766	,],
[	0.318039516	,	0.079509206	,	0.234606947	,	0.343024438	,],
[	0.387106013	,	0.27854614	,	0.067854999	,	0.315396639	,],
[	0.296651147	,	0.310434389	,	0.237710719	,	0.083148649	,]],
[[	0.083728752	,	0.235061801	,	0.263260284	,	0.246619071	,],
[	0.314634697	,	0.0719362	,	0.230582301	,	0.339475366	,],
[	0.382644588	,	0.273737221	,	0.063829215	,	0.310827013	,],
[	0.298995637	,	0.313070553	,	0.240472131	,	0.083378276	,]],
[[	0.081301338	,	0.231184989	,	0.259600463	,	0.243020392	,],
[	0.31517331	,	0.074709253	,	0.231367632	,	0.339985705	,],
[	0.384500384	,	0.275865434	,	0.067891573	,	0.312553621	,],
[	0.295555313	,	0.309468659	,	0.236705235	,	0.080854873	,]],
[[	0.087159265	,	0.234159228	,	0.262593994	,	0.246472266	,],
[	0.317469646	,	0.074217324	,	0.233299816	,	0.342096706	,],
[	0.384633023	,	0.27584117	,	0.065227344	,	0.312747893	,],
[	0.298737992	,	0.312368734	,	0.239716292	,	0.086268795	,]],
[[	0.081715801	,	0.232570858	,	0.260806572	,	0.244266933	,],
[	0.314180026	,	0.072236926	,	0.23017071	,	0.338844846	,],
[	0.382701921	,	0.273862608	,	0.064568418	,	0.31068646	,],
[	0.296872826	,	0.310882634	,	0.238095203	,	0.081237189	,]],
[[	0.082826817	,	0.231660953	,	0.260183028	,	0.243687389	,],
[	0.317108974	,	0.078023655	,	0.233498136	,	0.342022496	,],
[	0.386803869	,	0.278485439	,	0.071673245	,	0.31496708	,],
[	0.296236818	,	0.31000983	,	0.237245106	,	0.082179199	,]],
[[	0.084953951	,	0.236397173	,	0.264523215	,	0.247858964	,],
[	0.315356518	,	0.072393872	,	0.231209796	,	0.340007959	,],
[	0.383077869	,	0.274239164	,	0.063999913	,	0.311071309	,],
[	0.300332744	,	0.314196207	,	0.241605694	,	0.084336064	,]],
[[	0.081154033	,	0.231366379	,	0.259695349	,	0.24300831	,],
[	0.314684955	,	0.07373794	,	0.230765998	,	0.339447097	,],
[	0.383783175	,	0.275040317	,	0.066682052	,	0.311714759	,],
[	0.295824531	,	0.309692534	,	0.237018502	,	0.080570823	,]],
[[	0.085818789	,	0.233293982	,	0.261746886	,	0.245454495	,],
[	0.318768461	,	0.075418687	,	0.234610807	,	0.343190563	,],
[	0.385706551	,	0.276930724	,	0.066156165	,	0.313721141	,],
[	0.297907641	,	0.311518457	,	0.23898743	,	0.085029543	,]],
[[	0.082461037	,	0.233440024	,	0.26174859	,	0.245031095	,],
[	0.314247582	,	0.072003594	,	0.230137439	,	0.338888455	,],
[	0.382604057	,	0.273849554	,	0.064095802	,	0.310653552	,],
[	0.297642116	,	0.311644384	,	0.238864889	,	0.081930166	,]],
[[	0.082072134	,	0.231359898	,	0.259759546	,	0.24320277	,],
[	0.316356473	,	0.076528295	,	0.232580782	,	0.341175568	,],
[	0.385809086	,	0.277374533	,	0.070058706	,	0.313926677	,],
[	0.295926249	,	0.309749519	,	0.236930545	,	0.0815273	,]],
[[	0.086410265	,	0.237868865	,	0.265933723	,	0.249286645	,],
[	0.31613777	,	0.073019574	,	0.231892703	,	0.340669459	,],
[	0.383760817	,	0.274892745	,	0.064384132	,	0.311610691	,],
[	0.300301932	,	0.313947631	,	0.241228304	,	0.088623832	,]],
[[	0.083340464	,	0.235398241	,	0.263310747	,	0.242642843	,],
[	0.311398879	,	0.078835987	,	0.232744357	,	0.336666539	,],
[	0.38138516	,	0.278862481	,	0.073015658	,	0.310543153	,],
[	0.293417763	,	0.311792438	,	0.239623988	,	0.081569504	,]],
[[	0.083589916	,	0.238647058	,	0.266269807	,	0.24537509	,],
[	0.308881608	,	0.072215476	,	0.229477205	,	0.333893601	,],
[	0.376873509	,	0.273631075	,	0.064179196	,	0.305795439	,],
[	0.296633717	,	0.315278737	,	0.243247423	,	0.08276329	,]],
[[	0.081058875	,	0.234659769	,	0.262436627	,	0.241570235	,],
[	0.308823932	,	0.0743666	,	0.229767194	,	0.333993445	,],
[	0.378366207	,	0.275339483	,	0.067864302	,	0.307251595	,],
[	0.292732713	,	0.311204646	,	0.23901945	,	0.079565193	,]],
[[	0.086704999	,	0.237448444	,	0.265284983	,	0.244848758	,],
[	0.311946925	,	0.074853907	,	0.232630626	,	0.336894794	,],
[	0.379012294	,	0.275741622	,	0.065800465	,	0.307891352	,],
[	0.295200717	,	0.313515927	,	0.241367863	,	0.084365204	,]],
[[	0.081536093	,	0.236172921	,	0.26378369	,	0.242900681	,],
[	0.308093047	,	0.072119674	,	0.228866251	,	0.333103884	,],
[	0.376671925	,	0.273472434	,	0.064729786	,	0.305583626	,],
[	0.294250678	,	0.31289493	,	0.240699639	,	0.080318937	,]],
[[	0.082479322	,	0.235033554	,	0.262813028	,	0.242131569	,],
[	0.310580645	,	0.077368603	,	0.231733049	,	0.335707757	,],
[	0.380449229	,	0.277760728	,	0.07139809	,	0.309429824	,],
[	0.292895817	,	0.311367107	,	0.239278478	,	0.080647746	,]],
[[	0.084821406	,	0.239992141	,	0.267496271	,	0.24653546	,],
[	0.309557611	,	0.072839068	,	0.230145267	,	0.334462659	,],
[	0.377404777	,	0.274085141	,	0.064432263	,	0.306182896	,],
[	0.297928459	,	0.316600745	,	0.244399247	,	0.083908423	,]],
[[	0.080892413	,	0.234901774	,	0.262609742	,	0.241689396	,],
[	0.30839983	,	0.073458332	,	0.229260262	,	0.333459398	,],
[	0.377727434	,	0.274518188	,	0.066779797	,	0.306520215	,],
[	0.293030205	,	0.311523356	,	0.239268618	,	0.079495454	,]],
[[	0.085279699	,	0.236709998	,	0.264453303	,	0.243911849	,],
[	0.313159702	,	0.076142059	,	0.233821637	,	0.338007301	,],
[	0.380026943	,	0.276786428	,	0.066659259	,	0.308975196	,],
[	0.294452603	,	0.31278107	,	0.240576774	,	0.082962867	,]],
[[	0.082281092	,	0.237127494	,	0.264703581	,	0.243717772	,],
[	0.308338268	,	0.072086936	,	0.228919222	,	0.333228759	,],
[	0.376684918	,	0.273441311	,	0.064290163	,	0.305430996	,],
[	0.295093365	,	0.313682237	,	0.241570621	,	0.081188972	,]],
[[	0.08169109	,	0.234657512	,	0.262472877	,	0.241664865	,],
[	0.309868418	,	0.076181761	,	0.230986703	,	0.334982126	,],
[	0.379526999	,	0.276654585	,	0.069818075	,	0.308400454	,],
[	0.292716453	,	0.311235708	,	0.238950096	,	0.079986035	,]],
[[	0.086301272	,	0.241411377	,	0.26882359	,	0.247941254	,],
[	0.310503377	,	0.07344396	,	0.231087761	,	0.335358807	,],
[	0.377966351	,	0.274675076	,	0.064774397	,	0.306757407	,],
[	0.296881291	,	0.315044553	,	0.2430054	,	0.086613431	,]],
[[	0.082738757	,	0.239791544	,	0.267264542	,	0.242199153	,],
[	0.304872083	,	0.075835044	,	0.230916433	,	0.330976889	,],
[	0.374920874	,	0.277319924	,	0.069842289	,	0.304741622	,],
[	0.290580198	,	0.31472402	,	0.242699828	,	0.081387861	,]],
[[	0.083967244	,	0.243779404	,	0.270787209	,	0.245610777	,],
[	0.305862263	,	0.073748671	,	0.231497751	,	0.331811795	,],
[	0.373043933	,	0.27486868	,	0.064190294	,	0.302690822	,],
[	0.294696376	,	0.31886455	,	0.246924468	,	0.08361965	,]],
[[	0.080703415	,	0.239232212	,	0.266310137	,	0.241358929	,],
[	0.303372482	,	0.072784534	,	0.229176025	,	0.329380737	,],
[	0.372492417	,	0.274464739	,	0.065436638	,	0.302175482	,],
[	0.290133642	,	0.314295262	,	0.242152764	,	0.079773671	,]],
[[	0.085768368	,	0.241582257	,	0.268861539	,	0.244071072	,],
[	0.307131531	,	0.079601547	,	0.23351076	,	0.333358964	,],
[	0.376092807	,	0.27806404	,	0.067210126	,	0.305951119	,],
[	0.292382357	,	0.316352058	,	0.244282493	,	0.084175149	,]],
[[	0.08164133	,	0.241020971	,	0.268186306	,	0.243047074	,],
[	0.30399521	,	0.072405505	,	0.229686399	,	0.330023885	,],
[	0.372114723	,	0.27396436	,	0.063807473	,	0.301709956	,],
[	0.292121423	,	0.316291368	,	0.244156727	,	0.081037668	,]],
[[	0.081910641	,	0.239409228	,	0.266690977	,	0.241708119	,],
[	0.304249554	,	0.074804177	,	0.230209454	,	0.330302055	,],
[	0.37418137	,	0.276411458	,	0.068342031	,	0.303871561	,],
[	0.290384386	,	0.314508308	,	0.242367538	,	0.080698311	,]],
[[	0.085177285	,	0.245002809	,	0.272013052	,	0.246897463	,],
[	0.30684968	,	0.074877623	,	0.232488405	,	0.332770912	,],
[	0.373680145	,	0.275589834	,	0.064835069	,	0.303409517	,],
[	0.295000152	,	0.318789331	,	0.24671645	,	0.087881532	,]],
[[	0.080710676	,	0.239473016	,	0.266772261	,	0.241584353	,],
[	0.303441881	,	0.072386061	,	0.229003086	,	0.329400478	,],
[	0.372251295	,	0.274197275	,	0.064856407	,	0.301859988	,],
[	0.29052669	,	0.314627919	,	0.24254975	,	0.079789463	,]],
[[	0.084476582	,	0.240818862	,	0.268096286	,	0.24321919	,],
[	0.306108748	,	0.078035986	,	0.232281576	,	0.332193843	,],
[	0.376535857	,	0.27902142	,	0.07220946	,	0.306348781	,],
[	0.291664727	,	0.315562292	,	0.243472436	,	0.082906792	,]],
[[	0.08247762	,	0.241895256	,	0.269009499	,	0.243894968	,],
[	0.30479496	,	0.072896297	,	0.230316191	,	0.330646381	,],
[	0.37232686	,	0.274205676	,	0.063813671	,	0.301881487	,],
[	0.293077002	,	0.317289759	,	0.245242182	,	0.0819537	,]],
[[	0.081356475	,	0.239272994	,	0.266510191	,	0.241430077	,],
[	0.303899822	,	0.073939326	,	0.229686121	,	0.329866627	,],
[	0.373521572	,	0.275667976	,	0.067211411	,	0.303128094	,],
[	0.29017387	,	0.314298517	,	0.242159965	,	0.080179721	,]],
[[	0.086746972	,	0.246538933	,	0.27348998	,	0.248295945	,],
[	0.308148061	,	0.075942098	,	0.233631081	,	0.333886332	,],
[	0.374690914	,	0.276560845	,	0.065658886	,	0.304381272	,],
[	0.29395763	,	0.317771848	,	0.245736145	,	0.08626131	,]],
[[	0.08116057	,	0.242184297	,	0.268366313	,	0.239661499	,],
[	0.296668036	,	0.073096283	,	0.227218579	,	0.322993811	,],
[	0.365982752	,	0.27375273	,	0.065444177	,	0.296506831	,],
[	0.287003915	,	0.315872098	,	0.243624461	,	0.078944064	,]],
[[	0.082579772	,	0.246229201	,	0.27205288	,	0.243279958	,],
[	0.30119185	,	0.075278704	,	0.231474529	,	0.327221223	,],
[	0.368699677	,	0.276231537	,	0.065885307	,	0.299325556	,],
[	0.290550637	,	0.319072532	,	0.246986223	,	0.084810963	,]],
[[	0.079217979	,	0.241643749	,	0.267650181	,	0.238921507	,],
[	0.296278281	,	0.071389496	,	0.226614642	,	0.322542829	,],
[	0.365192887	,	0.272735303	,	0.06336006	,	0.295671893	,],
[	0.288061479	,	0.316840504	,	0.24474332	,	0.079096473	,]],
[[	0.084079293	,	0.243905713	,	0.269976097	,	0.241488072	,],
[	0.29805219	,	0.075673942	,	0.228878922	,	0.324456816	,],
[	0.367916982	,	0.275857278	,	0.068606836	,	0.298519683	,],
[	0.287833287	,	0.31660067	,	0.244362739	,	0.08053454	,]],
[[	0.08027415	,	0.243498095	,	0.269434131	,	0.240645711	,],
[	0.29843859	,	0.072826702	,	0.228701887	,	0.324551113	,],
[	0.366493945	,	0.273996332	,	0.063798107	,	0.296990224	,],
[	0.291309317	,	0.320150887	,	0.248014436	,	0.082074834	,]],
[[	0.080404194	,	0.241756998	,	0.267905204	,	0.239144511	,],
[	0.29633422	,	0.072316779	,	0.226891839	,	0.322713461	,],
[	0.365692562	,	0.273207772	,	0.064530547	,	0.296043599	,],
[	0.287195758	,	0.316002283	,	0.243782668	,	0.078741998	,]],
[[	0.083982313	,	0.247465229	,	0.273292746	,	0.244461954	,],
[	0.300426755	,	0.079520207	,	0.231556735	,	0.326814085	,],
[	0.369982725	,	0.277552819	,	0.067148721	,	0.300605372	,],
[	0.28973628	,	0.318347331	,	0.246154073	,	0.08343984	,]],
[[	0.079291195	,	0.241990652	,	0.267937112	,	0.239108194	,],
[	0.296801517	,	0.071611191	,	0.22706029	,	0.322901854	,],
[	0.365420858	,	0.272874828	,	0.063159212	,	0.295812119	,],
[	0.288853308	,	0.31760986	,	0.245499783	,	0.079656771	,]],
[[	0.082942081	,	0.243090015	,	0.269244461	,	0.240572568	,],
[	0.297463596	,	0.074585833	,	0.228125681	,	0.323675486	,],
[	0.367102386	,	0.274921231	,	0.067195272	,	0.297583232	,],
[	0.287377171	,	0.316158033	,	0.243977944	,	0.079687682	,]],
[[	0.081160401	,	0.244552657	,	0.270373106	,	0.241547452	,],
[	0.299489747	,	0.07372552	,	0.229791696	,	0.32549486	,],
[	0.367324586	,	0.274847166	,	0.064509665	,	0.297810207	,],
[	0.292383094	,	0.320741522	,	0.248789265	,	0.087230585	,]],
[[	0.079769373	,	0.241581419	,	0.267591948	,	0.238767517	,],
[	0.296286902	,	0.071909746	,	0.226716177	,	0.322406513	,],
[	0.365525177	,	0.272953974	,	0.063845671	,	0.295878429	,],
[	0.287419042	,	0.316247588	,	0.243982565	,	0.07863726	,]],
[[	0.085506228	,	0.249044706	,	0.274844495	,	0.246064956	,],
[	0.299507194	,	0.078039789	,	0.23041635	,	0.325787461	,],
[	0.369633497	,	0.277663275	,	0.071084302	,	0.300069253	,],
[	0.288898855	,	0.317556426	,	0.245367226	,	0.082202283	,]],
[[	0.079389846	,	0.243848065	,	0.268691931	,	0.235945344	,],
[	0.288822505	,	0.071393943	,	0.224130412	,	0.31522364	,],
[	0.358502934	,	0.27192467	,	0.063945113	,	0.290036224	,],
[	0.283395578	,	0.317445051	,	0.245212992	,	0.078381808	,]],
[[	0.08231878	,	0.249086105	,	0.273666741	,	0.240918162	,],
[	0.29285766	,	0.078542195	,	0.228825425	,	0.31931942	,],
[	0.362818852	,	0.276812214	,	0.071411504	,	0.294552389	,],
[	0.285658705	,	0.319620025	,	0.247490359	,	0.082979301	,]],
[[	0.078087633	,	0.243824469	,	0.268539112	,	0.235733448	,],
[	0.289442279	,	0.070746478	,	0.224386917	,	0.315677714	,],
[	0.358478398	,	0.271687893	,	0.062796175	,	0.289876901	,],
[	0.284992043	,	0.31913436	,	0.246906121	,	0.079297004	,]],
[[	0.08217079	,	0.245417061	,	0.270203533	,	0.237584947	,],
[	0.289862326	,	0.073555639	,	0.225280138	,	0.316201851	,],
[	0.359787359	,	0.273334773	,	0.066379657	,	0.291262211	,],
[	0.283501611	,	0.317722845	,	0.245305229	,	0.079364233	,]],
[[	0.079624535	,	0.246197811	,	0.270832832	,	0.238023616	,],
[	0.292181536	,	0.072955763	,	0.22726385	,	0.318366508	,],
[	0.360817278	,	0.273991021	,	0.064490082	,	0.292332904	,],
[	0.288314542	,	0.322093636	,	0.249914746	,	0.086709054	,]],
[[	0.078785566	,	0.243623409	,	0.268499634	,	0.235681255	,],
[	0.288810718	,	0.070924769	,	0.223926235	,	0.314945529	,],
[	0.358271136	,	0.271574695	,	0.06333238	,	0.289761826	,],
[	0.283538385	,	0.317651869	,	0.245352439	,	0.078285331	,]],
[[	0.083777803	,	0.250579183	,	0.275014831	,	0.24222114	,],
[	0.291857975	,	0.076912018	,	0.22758697	,	0.318239981	,],
[	0.361967967	,	0.275755131	,	0.069895825	,	0.293504587	,],
[	0.284952296	,	0.318998468	,	0.246628002	,	0.081716307	,]],
[[	0.078198621	,	0.244282224	,	0.268905269	,	0.236163141	,],
[	0.290023565	,	0.071186538	,	0.224985498	,	0.316099369	,],
[	0.358930655	,	0.272152703	,	0.063047729	,	0.290343912	,],
[	0.285972327	,	0.31998657	,	0.247710719	,	0.080061423	,]],
[[	0.080989593	,	0.244691373	,	0.269435322	,	0.236866192	,],
[	0.289442582	,	0.072567476	,	0.22475835	,	0.315764724	,],
[	0.359204389	,	0.272565124	,	0.065284748	,	0.290691938	,],
[	0.283395054	,	0.317491454	,	0.245120616	,	0.078752748	,]],
[[	0.080671859	,	0.247361019	,	0.271899228	,	0.23911515	,],
[	0.29354015	,	0.074148711	,	0.228491932	,	0.319479998	,],
[	0.361696883	,	0.274943932	,	0.065474981	,	0.29321923	,],
[	0.28727083	,	0.321129239	,	0.24886466	,	0.085088562	,]],
[[	0.078447029	,	0.243631366	,	0.268320812	,	0.235564199	,],
[	0.289048385	,	0.070749102	,	0.224071001	,	0.315172674	,],
[	0.358355107	,	0.271645313	,	0.062992974	,	0.289708106	,],
[	0.284030031	,	0.318229394	,	0.245832173	,	0.078530753	,]],
[[	0.084414699	,	0.246754101	,	0.271580054	,	0.239074973	,],
[	0.291178467	,	0.075591136	,	0.226635054	,	0.317401051	,],
[	0.361088359	,	0.274893195	,	0.068378821	,	0.292668648	,],
[	0.284408212	,	0.3183368	,	0.246082387	,	0.080536678	,]],
[[	0.079029407	,	0.248239437	,	0.272093959	,	0.235202373	,],
[	0.283544756	,	0.071586165	,	0.22392131	,	0.310389663	,],
[	0.353274071	,	0.272055751	,	0.063289014	,	0.285177188	,],
[	0.280892415	,	0.320096123	,	0.247908886	,	0.077509695	,]],
[[	0.082576327	,	0.253911517	,	0.277528191	,	0.240537884	,],
[	0.286672941	,	0.077563677	,	0.227411072	,	0.313589679	,],
[	0.357245125	,	0.276459327	,	0.070260884	,	0.289195236	,],
[	0.283345103	,	0.322422901	,	0.250344718	,	0.082226499	,]],
[[	0.077953591	,	0.248385311	,	0.27221048	,	0.235121205	,],
[	0.284705524	,	0.071704983	,	0.224917589	,	0.311453908	,],
[	0.353820836	,	0.272358626	,	0.062746582	,	0.285618625	,],
[	0.282609428	,	0.321711586	,	0.249727373	,	0.078431039	,]],
[[	0.081553068	,	0.249658177	,	0.27357822	,	0.236662377	,],
[	0.284240721	,	0.073151225	,	0.224648186	,	0.311121672	,],
[	0.354400213	,	0.273249789	,	0.065376852	,	0.28624535	,],
[	0.281321574	,	0.320445696	,	0.248254243	,	0.078519094	,]],
[[	0.079755957	,	0.250947988	,	0.274588775	,	0.237578542	,],
[	0.288138262	,	0.07466839	,	0.228217063	,	0.314720227	,],
[	0.356475404	,	0.275070144	,	0.064958415	,	0.288365406	,],
[	0.286099227	,	0.324988425	,	0.252988653	,	0.086119322	,]],
[[	0.078465749	,	0.248130952	,	0.271970185	,	0.234886945	,],
[	0.283746578	,	0.071310197	,	0.223982973	,	0.31054795	,],
[	0.353336517	,	0.271848226	,	0.062874678	,	0.285009056	,],
[	0.281197186	,	0.320312162	,	0.248203248	,	0.077456422	,]],
[[	0.084178164	,	0.255459034	,	0.279099229	,	0.242046307	,],
[	0.285771423	,	0.076212786	,	0.226435511	,	0.312600841	,],
[	0.356287425	,	0.275544107	,	0.068599454	,	0.28821952	,],
[	0.282701716	,	0.321724356	,	0.249552279	,	0.080950199	,]],
[[	0.078258274	,	0.248908468	,	0.272657229	,	0.235554677	,],
[	0.285608206	,	0.072324737	,	0.22556573	,	0.312120038	,],
[	0.354485308	,	0.272917029	,	0.063079622	,	0.286163505	,],
[	0.283567153	,	0.322672688	,	0.250542065	,	0.07925741	,]],
[[	0.080458568	,	0.249010475	,	0.272808362	,	0.23591855	,],
[	0.283865658	,	0.072439032	,	0.224199371	,	0.310602401	,],
[	0.353924267	,	0.272749957	,	0.06437138	,	0.285635537	,],
[	0.281057172	,	0.320193532	,	0.247970295	,	0.077901749	,]],
[[	0.080725557	,	0.251996861	,	0.27561637	,	0.238564633	,],
[	0.288421927	,	0.080158431	,	0.229281497	,	0.315205511	,],
[	0.357889686	,	0.276348649	,	0.066164749	,	0.289610959	,],
[	0.285059578	,	0.323844831	,	0.251837706	,	0.084363268	,]],
[[	0.078097743	,	0.248122344	,	0.271829465	,	0.234816247	,],
[	0.284165618	,	0.071375573	,	0.22419967	,	0.310813778	,],
[	0.353542478	,	0.271946943	,	0.062677492	,	0.285182876	,],
[	0.281676106	,	0.320769853	,	0.248647782	,	0.077694859	,]],
[[	0.083726624	,	0.250983516	,	0.274818098	,	0.238047554	,],
[	0.285225267	,	0.074850576	,	0.225734198	,	0.312041042	,],
[	0.35555606	,	0.274492763	,	0.067321059	,	0.287280659	,],
[	0.28202693	,	0.321041022	,	0.248951958	,	0.079895817	,]],
[[	0.079033682	,	0.252267353	,	0.27553659	,	0.234394096	,],
[	0.279351002	,	0.072345665	,	0.223819582	,	0.306293234	,],
[	0.34995446	,	0.273684026	,	0.064197094	,	0.281670936	,],
[	0.279953954	,	0.324676324	,	0.250823854	,	0.077472911	,]],
[[	0.079890566	,	0.25571917	,	0.278858749	,	0.237674828	,],
[	0.283707859	,	0.080099291	,	0.228921422	,	0.310805543	,],
[	0.35336121	,	0.27680941	,	0.065295487	,	0.285087619	,],
[	0.28444291	,	0.32902833	,	0.255272675	,	0.084606626	,]],
[[	0.076907413	,	0.251577827	,	0.27482044	,	0.233526407	,],
[	0.279571253	,	0.071096148	,	0.223762097	,	0.306217517	,],
[	0.349277327	,	0.272847616	,	0.062194752	,	0.280934798	,],
[	0.280257659	,	0.324958253	,	0.251061161	,	0.076664835	,]],
[[	0.082265257	,	0.254156148	,	0.277375047	,	0.23630266	,],
[	0.280611748	,	0.074917729	,	0.225385947	,	0.307434089	,],
[	0.351458252	,	0.275633091	,	0.067191279	,	0.283367568	,],
[	0.281066386	,	0.325740547	,	0.251853978	,	0.079600886	,]],
[[	0.077660589	,	0.253161692	,	0.276312541	,	0.235010507	,],
[	0.282061735	,	0.072820229	,	0.22597163	,	0.308599627	,],
[	0.350863447	,	0.274365618	,	0.063058503	,	0.282564071	,],
[	0.282861154	,	0.327565331	,	0.253725852	,	0.078724726	,]],
[[	0.078281483	,	0.251801728	,	0.27511058	,	0.233935225	,],
[	0.279253476	,	0.071844533	,	0.223612814	,	0.306087865	,],
[	0.349571039	,	0.273223627	,	0.063340328	,	0.281226946	,],
[	0.279673027	,	0.324375341	,	0.250517512	,	0.076861725	,]],
[[	0.081281272	,	0.257060631	,	0.28017812	,	0.238943947	,],
[	0.282843901	,	0.078537546	,	0.227832513	,	0.309755263	,],
[	0.354011903	,	0.27837374	,	0.0712594	,	0.285897065	,],
[	0.283417739	,	0.32790366	,	0.254089131	,	0.083047223	,]],
[[	0.076864224	,	0.251756997	,	0.275022771	,	0.233641389	,],
[	0.280128151	,	0.071368658	,	0.224101244	,	0.306700812	,],
[	0.349553612	,	0.273085039	,	0.062189808	,	0.281166133	,],
[	0.280778326	,	0.325524196	,	0.251688238	,	0.076983767	,]],
[[	0.080904148	,	0.253297151	,	0.276532751	,	0.235419486	,],
[	0.280182615	,	0.073829392	,	0.224619784	,	0.306901689	,],
[	0.350851885	,	0.274811774	,	0.065982433	,	0.282493351	,],
[	0.280541967	,	0.325125494	,	0.251316618	,	0.078595715	,]],
[[	0.078431335	,	0.254131854	,	0.277164137	,	0.235860785	,],
[	0.283104901	,	0.073742801	,	0.227065289	,	0.309584206	,],
[	0.351849684	,	0.275288659	,	0.063827671	,	0.283443735	,],
[	0.283983069	,	0.328723792	,	0.254721585	,	0.079744848	,]],
[[	0.077648146	,	0.251611609	,	0.27484487	,	0.233506829	,],
[	0.279427946	,	0.071487397	,	0.223669047	,	0.306062085	,],
[	0.349351653	,	0.272926922	,	0.062727903	,	0.280911052	,],
[	0.279791217	,	0.324514431	,	0.250614707	,	0.076586831	,]],
[[	0.082676067	,	0.258534852	,	0.281589711	,	0.240290922	,],
[	0.281971315	,	0.077016296	,	0.226759505	,	0.308824989	,],
[	0.353015158	,	0.277191552	,	0.069655252	,	0.284800211	,],
[	0.282457718	,	0.327029941	,	0.253137952	,	0.08153689	,]],
[[	0.077696501	,	0.255445769	,	0.277625391	,	0.232048274	,],
[	0.273224246	,	0.071279452	,	0.22271829	,	0.300181823	,],
[	0.344294306	,	0.2733747	,	0.063169155	,	0.276409377	,],
[	0.278263201	,	0.327145962	,	0.253997485	,	0.077075051	,]],
[[	0.080206189	,	0.260353355	,	0.28224357	,	0.236599813	,],
[	0.275354892	,	0.076457059	,	0.225497791	,	0.302375813	,],
[	0.348045036	,	0.277683968	,	0.070091232	,	0.280418995	,],
[	0.281125026	,	0.329805566	,	0.256908365	,	0.082446644	,]],
[[	0.075987969	,	0.25511561	,	0.277172231	,	0.231606533	,],
[	0.274926265	,	0.071947533	,	0.224271614	,	0.301767235	,],
[	0.34474805	,	0.273825244	,	0.062499831	,	0.276905427	,],
[	0.27939805	,	0.32839797	,	0.255192359	,	0.077423931	,]],
[[	0.08045524	,	0.256948819	,	0.279115375	,	0.233653404	,],
[	0.273451549	,	0.0725671	,	0.223160691	,	0.30030832	,],
[	0.345247712	,	0.274556953	,	0.065222798	,	0.277414175	,],
[	0.27874046	,	0.327523254	,	0.254434641	,	0.078349069	,]],
[[	0.077513125	,	0.257339425	,	0.279376845	,	0.233720033	,],
[	0.277966518	,	0.08039822	,	0.228320201	,	0.30492829	,],
[	0.347522039	,	0.276474236	,	0.064741264	,	0.279700806	,],
[	0.282967074	,	0.331770423	,	0.258640513	,	0.080590995	,]],
[[	0.07701453	,	0.255150915	,	0.277352012	,	0.231688709	,],
[	0.273562559	,	0.071195666	,	0.22294612	,	0.300398846	,],
[	0.34425858	,	0.273294935	,	0.06264055	,	0.276259465	,],
[	0.278471903	,	0.327329197	,	0.254223269	,	0.076864046	,]],
[[	0.081577545	,	0.261698974	,	0.283645691	,	0.238082492	,],
[	0.274863625	,	0.075198101	,	0.224798592	,	0.301747139	,],
[	0.347083857	,	0.276736049	,	0.068495061	,	0.279318471	,],
[	0.280498718	,	0.329283618	,	0.256170787	,	0.081050986	,]],
[[	0.07624882	,	0.255678881	,	0.277689254	,	0.232090055	,],
[	0.275894311	,	0.072834473	,	0.225156853	,	0.302556511	,],
[	0.345403183	,	0.274335098	,	0.06295256	,	0.277462107	,],
[	0.280300045	,	0.329144589	,	0.256086021	,	0.0780027	,]],
[[	0.079387154	,	0.256311056	,	0.278464805	,	0.232928385	,],
[	0.273316894	,	0.071928911	,	0.22285477	,	0.300034264	,],
[	0.344790788	,	0.273991083	,	0.064208244	,	0.276810687	,],
[	0.278498807	,	0.327379263	,	0.254111588	,	0.077702701	,]],
[[	0.078531989	,	0.258475834	,	0.280534316	,	0.234793896	,],
[	0.27702498	,	0.078759	,	0.227212062	,	0.30399178	,],
[	0.348778189	,	0.277626662	,	0.065913909	,	0.28078221	,],
[	0.282800419	,	0.331424052	,	0.258519792	,	0.084839034	,]],
[[	0.076522056	,	0.255051174	,	0.277113509	,	0.231484764	,],
[	0.274017662	,	0.07137945	,	0.223347575	,	0.300861463	,],
[	0.344510117	,	0.273405455	,	0.062555103	,	0.276462961	,],
[	0.278760804	,	0.327606448	,	0.254437638	,	0.07689082	,]],
[[	0.0827556	,	0.258419474	,	0.280586078	,	0.23526933	,],
[	0.274211868	,	0.07400609	,	0.224012492	,	0.301108022	,],
[	0.346357166	,	0.275823504	,	0.067078316	,	0.278430881	,],
[	0.279641621	,	0.32841151	,	0.255288726	,	0.079878783	,]],
[[	0.077124898	,	0.259935452	,	0.282341746	,	0.231861705	,],
[	0.268230108	,	0.070805637	,	0.223216128	,	0.296513717	,],
[	0.339478145	,	0.273717054	,	0.062791223	,	0.272297245	,],
[	0.276641275	,	0.330479044	,	0.25817037	,	0.076204715	,]],
[[	0.080405774	,	0.265347166	,	0.287417744	,	0.237148064	,],
[	0.268916077	,	0.074022193	,	0.224345464	,	0.297282281	,],
[	0.341002523	,	0.275733399	,	0.066961226	,	0.273914318	,],
[	0.278337213	,	0.332063855	,	0.259843804	,	0.080003808	,]],
[[	0.075851365	,	0.260039714	,	0.282265879	,	0.231808777	,],
[	0.270995233	,	0.072847572	,	0.225825746	,	0.299151444	,],
[	0.341494104	,	0.275635264	,	0.064145007	,	0.274269221	,],
[	0.278761683	,	0.33255047	,	0.260332934	,	0.077693473	,]],
[[	0.079759798	,	0.261403819	,	0.283722821	,	0.233478636	,],
[	0.267704657	,	0.071126359	,	0.222788859	,	0.296025045	,],
[	0.339461759	,	0.273713862	,	0.063632896	,	0.27228928	,],
[	0.276595249	,	0.330324463	,	0.258076372	,	0.076884569	,]],
[[	0.077672587	,	0.262503306	,	0.284619746	,	0.234270831	,],
[	0.270819903	,	0.07739046	,	0.226610831	,	0.29928994	,],
[	0.34327216	,	0.278275289	,	0.070661937	,	0.276225026	,],
[	0.280670872	,	0.334266009	,	0.262072474	,	0.08366217	,]],
[[	0.076571758	,	0.259744814	,	0.282015508	,	0.231645771	,],
[	0.268874412	,	0.0711079	,	0.223724724	,	0.296967796	,],
[	0.339876848	,	0.274056288	,	0.0629337	,	0.272627015	,],
[	0.27703126	,	0.330949787	,	0.25849327	,	0.076369244	,]],
[[	0.081984114	,	0.266834977	,	0.288849649	,	0.238577109	,],
[	0.268502784	,	0.072998236	,	0.223775944	,	0.296724545	,],
[	0.340442894	,	0.274913217	,	0.06570725	,	0.273164298	,],
[	0.277592002	,	0.331354898	,	0.259003251	,	0.078810114	,]],
[[	0.076198658	,	0.26048704	,	0.282692063	,	0.23222006	,],
[	0.272159804	,	0.073863411	,	0.226965661	,	0.300289432	,],
[	0.342527584	,	0.27666815	,	0.065013024	,	0.27530371	,],
[	0.279810092	,	0.333556043	,	0.261273423	,	0.078620157	,]],
[[	0.078649736	,	0.260744349	,	0.283005584	,	0.232728132	,],
[	0.2679304	,	0.070920631	,	0.222846194	,	0.296192175	,],
[	0.339356978	,	0.273544493	,	0.063137019	,	0.272026975	,],
[	0.276443706	,	0.330328261	,	0.257971136	,	0.076454373	,]],
[[	0.078728093	,	0.263521348	,	0.285669633	,	0.235231493	,],
[	0.27009901	,	0.076028383	,	0.225697427	,	0.298344254	,],
[	0.342328283	,	0.277158565	,	0.069089566	,	0.275183429	,],
[	0.279714472	,	0.333382644	,	0.261146978	,	0.082059695	,]],
[[	0.076150845	,	0.259755857	,	0.282003302	,	0.23156638	,],
[	0.269662083	,	0.071729641	,	0.224411462	,	0.297689937	,],
[	0.340500285	,	0.274637423	,	0.063228743	,	0.273116928	,],
[	0.277677087	,	0.331521814	,	0.259161047	,	0.07676368	,]],
[[	0.081968818	,	0.262797252	,	0.285054532	,	0.234798624	,],
[	0.268054861	,	0.07216457	,	0.223272652	,	0.296319031	,],
[	0.339980823	,	0.274459961	,	0.064822768	,	0.272784636	,],
[	0.277138345	,	0.330920733	,	0.258609725	,	0.077955422	,]],
[[	0.078339431	,	0.267551885	,	0.289897115	,	0.234380265	,],
[	0.266753502	,	0.072486845	,	0.225946119	,	0.295054039	,],
[	0.337839157	,	0.277033898	,	0.063152061	,	0.27015338	,],
[	0.276902798	,	0.336699543	,	0.26270827	,	0.075865697	,]],
[[	0.081324038	,	0.272590253	,	0.294614493	,	0.239296236	,],
[	0.266402649	,	0.074580783	,	0.226140148	,	0.294898422	,],
[	0.33970078	,	0.27944401	,	0.067677318	,	0.272145316	,],
[	0.278946657	,	0.338674511	,	0.26474616	,	0.080142957	,]],
[[	0.077059934	,	0.267565338	,	0.289791128	,	0.234311218	,],
[	0.269788132	,	0.07494492	,	0.228848482	,	0.298003832	,],
[	0.33977179	,	0.278825504	,	0.064346119	,	0.272111888	,],
[	0.278809415	,	0.338627859	,	0.264633186	,	0.077167506	,]],
[[	0.081040163	,	0.26903316	,	0.291310976	,	0.236000924	,],
[	0.265816287	,	0.072341688	,	0.225153883	,	0.294118204	,],
[	0.338010229	,	0.277315029	,	0.064145611	,	0.270365337	,],
[	0.276979588	,	0.336783653	,	0.262797495	,	0.076749992	,]],
[[	0.078560555	,	0.269734501	,	0.291833984	,	0.236353678	,],
[	0.268274683	,	0.07786138	,	0.228260829	,	0.296707948	,],
[	0.342002598	,	0.282057234	,	0.07148775	,	0.274627037	,],
[	0.281485578	,	0.3411671	,	0.267235559	,	0.083974048	,]],
[[	0.077812876	,	0.267329578	,	0.289580409	,	0.234141979	,],
[	0.267304822	,	0.072923499	,	0.226560991	,	0.295597456	,],
[	0.338246667	,	0.277346367	,	0.063202767	,	0.270468412	,],
[	0.277282877	,	0.33702369	,	0.263009916	,	0.07587436	,]],
[[	0.082817988	,	0.274088789	,	0.296104833	,	0.240800104	,],
[	0.265980227	,	0.073776884	,	0.225585927	,	0.294416058	,],
[	0.339098503	,	0.278671258	,	0.066432933	,	0.271389899	,],
[	0.278187067	,	0.337958806	,	0.264001639	,	0.078956491	,]],
[[	0.077318643	,	0.2679767	,	0.290139091	,	0.23457491	,],
[	0.270764305	,	0.081889132	,	0.231034301	,	0.299120455	,],
[	0.340696056	,	0.279735096	,	0.065105758	,	0.272998049	,],
[	0.279794894	,	0.339507203	,	0.265573861	,	0.077890709	,]],
[[	0.080032712	,	0.268474436	,	0.290748158	,	0.235272306	,],
[	0.266013507	,	0.072295889	,	0.225276062	,	0.294333134	,],
[	0.337831229	,	0.277068664	,	0.063563706	,	0.270075929	,],
[	0.276754949	,	0.336630622	,	0.262517813	,	0.076215797	,]],
[[	0.079706288	,	0.270847106	,	0.29289736	,	0.237432356	,],
[	0.267434387	,	0.076542662	,	0.227350584	,	0.295824537	,],
[	0.341145389	,	0.280997662	,	0.06994583	,	0.27366935	,],
[	0.280448498	,	0.340162939	,	0.266215725	,	0.08237914	,]],
[[	0.077405959	,	0.267147954	,	0.289409853	,	0.233949919	,],
[	0.268329554	,	0.073628667	,	0.227439952	,	0.296556384	,],
[	0.338829881	,	0.277851788	,	0.063487859	,	0.271076315	,],
[	0.277889655	,	0.337551455	,	0.263598493	,	0.076286791	,]],
[[	0.083358025	,	0.27052911	,	0.292744241	,	0.237471952	,],
[	0.265964863	,	0.073042402	,	0.225382701	,	0.294268284	,],
[	0.338583766	,	0.278073017	,	0.065397755	,	0.270838648	,],
[	0.277579168	,	0.337356977	,	0.263355839	,	0.077905653	,]],
[[	0.077773396	,	0.271435479	,	0.293232594	,	0.233643692	,],
[	0.262126126	,	0.073127204	,	0.226556315	,	0.291175712	,],
[	0.333519603	,	0.278348709	,	0.063724865	,	0.267135251	,],
[	0.275204357	,	0.340066365	,	0.266464901	,	0.076015156	,]],
[[	0.080174078	,	0.276038809	,	0.297673758	,	0.238144457	,],
[	0.260466034	,	0.073432752	,	0.225218932	,	0.289691092	,],
[	0.333652282	,	0.27893379	,	0.06613016	,	0.267210117	,],
[	0.27650362	,	0.341358635	,	0.267784226	,	0.079274813	,]],
[[	0.076095747	,	0.271166558	,	0.292843166	,	0.233251377	,],
[	0.26474356	,	0.08122101	,	0.230178309	,	0.293952164	,],
[	0.336481916	,	0.281346179	,	0.066126402	,	0.270141257	,],
[	0.277727726	,	0.342441964	,	0.268943974	,	0.077860441	,]],
[[	0.080611791	,	0.273043322	,	0.294872103	,	0.23536879	,],
[	0.260651604	,	0.072323367	,	0.225047615	,	0.289797685	,],
[	0.332817637	,	0.277915612	,	0.063832381	,	0.266293246	,],
[	0.274927426	,	0.339851107	,	0.266168471	,	0.076432775	,]],
[[	0.077629098	,	0.27331833	,	0.294920801	,	0.235299055	,],
[	0.261681837	,	0.076023378	,	0.226736644	,	0.290857107	,],
[	0.335404977	,	0.281036175	,	0.069326257	,	0.269031585	,],
[	0.278863206	,	0.343577707	,	0.270039799	,	0.082859627	,]],
[[	0.077106291	,	0.271182599	,	0.293009777	,	0.233268583	,],
[	0.263047998	,	0.073941503	,	0.227329916	,	0.291985159	,],
[	0.334240572	,	0.279057117	,	0.064262585	,	0.26770628	,],
[	0.275729901	,	0.340672519	,	0.266949404	,	0.076321281	,]],
[[	0.081589912	,	0.277420287	,	0.298994871	,	0.239530996	,],
[	0.260397339	,	0.072851651	,	0.224928765	,	0.289461061	,],
[	0.333302122	,	0.27851576	,	0.065206839	,	0.266772426	,],
[	0.275855247	,	0.340657938	,	0.267061965	,	0.078254368	,]],
[[	0.076340121	,	0.271572073	,	0.293300275	,	0.233680242	,],
[	0.263796274	,	0.079538256	,	0.229066759	,	0.292984973	,],
[	0.33772068	,	0.282463539	,	0.067325475	,	0.271259112	,],
[	0.278791365	,	0.343470479	,	0.269983492	,	0.078962419	,]],
[[	0.079443533	,	0.27231258	,	0.294141864	,	0.234512735	,],
[	0.261134876	,	0.072478264	,	0.225425952	,	0.290156538	,],
[	0.332995473	,	0.277920002	,	0.063680781	,	0.266412696	,],
[	0.275006574	,	0.339895147	,	0.266176404	,	0.076077821	,]],
[[	0.078558159	,	0.274285218	,	0.295868885	,	0.236336383	,],
[	0.261074139	,	0.074874596	,	0.225983493	,	0.290280954	,],
[	0.334725999	,	0.280159766	,	0.068025014	,	0.268290937	,],
[	0.277868642	,	0.342599234	,	0.268980146	,	0.081333438	,]],
[[	0.076605966	,	0.271066089	,	0.292727841	,	0.233139281	,],
[	0.264125188	,	0.074796295	,	0.228301733	,	0.29297396	,],
[	0.335166048	,	0.279861419	,	0.064917981	,	0.268570198	,],
[	0.276501329	,	0.341283084	,	0.267612031	,	0.076754427	,]],
[[	0.082960167	,	0.274598841	,	0.296405604	,	0.236856324	,],
[	0.260408033	,	0.072425287	,	0.224892688	,	0.289377946	,],
[	0.333094161	,	0.278204481	,	0.064538568	,	0.26653664	,],
[	0.275472977	,	0.340288991	,	0.266620967	,	0.077461597	,]],
[[	0.076771338	,	0.274735823	,	0.295906343	,	0.232142097	,],
[	0.256452773	,	0.073135953	,	0.226179192	,	0.286845374	,],
[	0.32812542	,	0.278255353	,	0.063811877	,	0.262785682	,],
[	0.272744266	,	0.342581249	,	0.268805297	,	0.075466109	,]],
[[	0.080082228	,	0.280126631	,	0.300998056	,	0.237422078	,],
[	0.254352333	,	0.073146517	,	0.224502116	,	0.284935446	,],
[	0.327825024	,	0.278231799	,	0.065409399	,	0.262380356	,],
[	0.273007988	,	0.342930755	,	0.269118291	,	0.077957875	,]],
[[	0.075674319	,	0.274842035	,	0.295849845	,	0.232110559	,],
[	0.2584525	,	0.080611955	,	0.229429949	,	0.289208173	,],
[	0.331532442	,	0.281548296	,	0.066715863	,	0.266132693	,],
[	0.275685034	,	0.345306056	,	0.271605911	,	0.077897446	,]],
[[	0.079445223	,	0.276284478	,	0.297315187	,	0.23378461	,],
[	0.254909605	,	0.072222159	,	0.224660931	,	0.285208729	,],
[	0.32734296	,	0.27752776	,	0.063531052	,	0.261870862	,],
[	0.272040676	,	0.34180827	,	0.268004507	,	0.075488123	,]],
[[	0.077287078	,	0.277129693	,	0.298069214	,	0.234398329	,],
[	0.255472593	,	0.075544946	,	0.225893667	,	0.286070881	,],
[	0.329285574	,	0.280060202	,	0.068336627	,	0.26395251	,],
[	0.275161493	,	0.34489037	,	0.27120313	,	0.081095225	,]],
[[	0.076264522	,	0.274729603	,	0.295746635	,	0.231972241	,],
[	0.257374097	,	0.074062163	,	0.227123382	,	0.287710687	,],
[	0.329124146	,	0.2790894	,	0.064403055	,	0.263556346	,],
[	0.273455129	,	0.343314126	,	0.269443082	,	0.076055315	,]],
[[	0.081701204	,	0.281613221	,	0.302464768	,	0.238907836	,],
[	0.254359454	,	0.072683412	,	0.224404354	,	0.284832466	,],
[	0.327422087	,	0.27781985	,	0.064572954	,	0.261835753	,],
[	0.272687108	,	0.342487047	,	0.268680571	,	0.076988195	,]],
[[	0.075970156	,	0.275406466	,	0.296324666	,	0.232563423	,],
[	0.25761688	,	0.079112169	,	0.228287185	,	0.288144142	,],
[	0.33172041	,	0.282836495	,	0.072327714	,	0.266544167	,],
[	0.276938783	,	0.3465426	,	0.272881661	,	0.079103779	,]],
[[	0.078317341	,	0.275625962	,	0.296673527	,	0.232923052	,],
[	0.255485898	,	0.072441766	,	0.225168806	,	0.285749623	,],
[	0.327570318	,	0.277723706	,	0.063539398	,	0.262092647	,],
[	0.272299296	,	0.342111313	,	0.26828055	,	0.075349446	,]],
[[	0.078437498	,	0.278337051	,	0.299167661	,	0.235519115	,],
[	0.255043562	,	0.074514785	,	0.22539513	,	0.285598097	,],
[	0.328637043	,	0.27936461	,	0.067139236	,	0.263252905	,],
[	0.274335713	,	0.344000772	,	0.270287788	,	0.079767931	,]],
[[	0.075841546	,	0.274558602	,	0.295539563	,	0.231762637	,],
[	0.258599896	,	0.07500795	,	0.228201326	,	0.288868599	,],
[	0.3300705	,	0.280010258	,	0.065157876	,	0.264471958	,],
[	0.274295687	,	0.34402764	,	0.270231136	,	0.076647033	,]],
[[	0.081633238	,	0.277583193	,	0.29873936	,	0.235001445	,],
[	0.254622121	,	0.072367913	,	0.224321257	,	0.284925969	,],
[	0.327281311	,	0.277509569	,	0.064019084	,	0.261707531	,],
[	0.272370186	,	0.342092379	,	0.268277242	,	0.07615061	,]],
[[	0.076181254	,	0.278118535	,	0.297816602	,	0.230715872	,],
[	0.25291054	,	0.074502222	,	0.226791666	,	0.283054481	,],
[	0.325071495	,	0.279979717	,	0.064932941	,	0.259277071	,],
[	0.270748473	,	0.344853734	,	0.27083836	,	0.075042146	,]],
[[	0.079964879	,	0.283787068	,	0.303196927	,	0.236389929	,],
[	0.248566016	,	0.071399604	,	0.222663888	,	0.278733531	,],
[	0.32203767	,	0.277283007	,	0.063213987	,	0.256206789	,],
[	0.270116454	,	0.344351066	,	0.270228066	,	0.076118647	,]],
[[	0.075166559	,	0.278283339	,	0.29785015	,	0.230776755	,],
[	0.250576723	,	0.076517378	,	0.225362142	,	0.280931971	,],
[	0.325125828	,	0.281064371	,	0.069470771	,	0.25960289	,],
[	0.274064273	,	0.34807287	,	0.274196839	,	0.078044184	,]],
[[	0.07880433	,	0.279571649	,	0.299167531	,	0.232256696	,],
[	0.250275915	,	0.072304019	,	0.224161552	,	0.280345001	,],
[	0.323111991	,	0.277995907	,	0.063083628	,	0.257211561	,],
[	0.269698645	,	0.34386489	,	0.269880084	,	0.074563223	,]],
[[	0.077162647	,	0.280823174	,	0.300357393	,	0.23346873	,],
[	0.248567148	,	0.07266861	,	0.223005442	,	0.278874879	,],
[	0.322721087	,	0.278204128	,	0.064947042	,	0.256928516	,],
[	0.271800067	,	0.346027566	,	0.272046975	,	0.078942766	,]],
[[	0.075643442	,	0.277970964	,	0.297545391	,	0.23052578	,],
[	0.253049467	,	0.080617446	,	0.228183271	,	0.283349554	,],
[	0.326227675	,	0.281081426	,	0.065884878	,	0.260372071	,],
[	0.271472275	,	0.345639685	,	0.271693193	,	0.075711679	,]],
[[	0.08154062	,	0.285316379	,	0.304636673	,	0.23798539	,],
[	0.248872081	,	0.071366441	,	0.222881743	,	0.278942053	,],
[	0.322166743	,	0.277320005	,	0.062944365	,	0.2562205	,],
[	0.269852301	,	0.343948359	,	0.269930744	,	0.075407352	,]],
[[	0.075512114	,	0.278814574	,	0.298254158	,	0.231335745	,],
[	0.249894886	,	0.075228237	,	0.224468093	,	0.280129275	,],
[	0.324300541	,	0.280029822	,	0.068041819	,	0.258595338	,],
[	0.274534301	,	0.348568989	,	0.274706773	,	0.082855099	,]],
[[	0.077613671	,	0.278939999	,	0.298497924	,	0.231507409	,],
[	0.251332835	,	0.072989366	,	0.225139829	,	0.281347789	,],
[	0.323851329	,	0.27874999	,	0.063597849	,	0.257857062	,],
[	0.270056147	,	0.344230385	,	0.270074024	,	0.074508963	,]],
[[	0.078095882	,	0.281843661	,	0.301244925	,	0.234339165	,],
[	0.248487596	,	0.071997341	,	0.222678339	,	0.27863015	,],
[	0.32237192	,	0.2776812	,	0.064090872	,	0.256469196	,],
[	0.271126086	,	0.345329173	,	0.271278468	,	0.077746035	,]],
[[	0.075395351	,	0.278048208	,	0.297597365	,	0.230577793	,],
[	0.252079763	,	0.078888561	,	0.227040142	,	0.282335554	,],
[	0.326795695	,	0.282998168	,	0.072092944	,	0.261192262	,],
[	0.272541691	,	0.346600301	,	0.272571015	,	0.076485671	,]],
[[	0.080903199	,	0.281056032	,	0.300528713	,	0.233627838	,],
[	0.249436109	,	0.071581415	,	0.223364956	,	0.279458839	,],
[	0.322410193	,	0.277586209	,	0.062881533	,	0.256474458	,],
[	0.269630928	,	0.343826618	,	0.26969587	,	0.07482768	,]],
[[	0.075599059	,	0.28205097	,	0.301696011	,	0.230232261	,],
[	0.24821387	,	0.08195422	,	0.22950959	,	0.279797346	,],
[	0.321281445	,	0.281746377	,	0.065843019	,	0.256212203	,],
[	0.269336412	,	0.349018916	,	0.274226255	,	0.074615277	,]],
[[	0.079874504	,	0.288240537	,	0.307610084	,	0.23629736	,],
[	0.243534186	,	0.072118697	,	0.223621913	,	0.274927983	,],
[	0.317720645	,	0.278508043	,	0.063629519	,	0.252695708	,],
[	0.268991977	,	0.348744232	,	0.273915479	,	0.076107704	,]],
[[	0.074785281	,	0.282475946	,	0.301905858	,	0.230486979	,],
[	0.245047269	,	0.076549232	,	0.225844009	,	0.276559742	,],
[	0.320270337	,	0.281805855	,	0.069285597	,	0.255441449	,],
[	0.272682873	,	0.352284733	,	0.277434132	,	0.077543651	,]],
[[	0.077923008	,	0.283450331	,	0.302992177	,	0.231539966	,],
[	0.245740952	,	0.073317545	,	0.225572708	,	0.276970812	,],
[	0.319031178	,	0.279625868	,	0.063870856	,	0.253883473	,],
[	0.268377676	,	0.348127041	,	0.273291276	,	0.074191606	,]],
[[	0.076882524	,	0.285131807	,	0.304576879	,	0.233154738	,],
[	0.243387882	,	0.073021943	,	0.223769458	,	0.274781636	,],
[	0.318105851	,	0.279121221	,	0.065062316	,	0.253013776	,],
[	0.270796596	,	0.350490443	,	0.275658592	,	0.07898079	,]],
[[	0.07511756	,	0.282075355	,	0.301598819	,	0.230049121	,],
[	0.247256957	,	0.08041752	,	0.228322155	,	0.278765087	,],
[	0.322547865	,	0.283017978	,	0.066938414	,	0.257443231	,],
[	0.270210242	,	0.349755254	,	0.274987948	,	0.07516533	,]],
[[	0.08147932	,	0.289761852	,	0.309127855	,	0.237856468	,],
[	0.244062584	,	0.072219511	,	0.223933376	,	0.275311397	,],
[	0.317834708	,	0.278625926	,	0.063426896	,	0.252752087	,],
[	0.268703986	,	0.348400753	,	0.273541991	,	0.075271552	,]],
[[	0.075195997	,	0.282976913	,	0.302495462	,	0.23092145	,],
[	0.24441653	,	0.07536182	,	0.225066291	,	0.275870863	,],
[	0.31954616	,	0.280879785	,	0.067875539	,	0.254561913	,],
[	0.273469329	,	0.353059556	,	0.278354484	,	0.082901159	,]],
[[	0.076938861	,	0.282729791	,	0.302357098	,	0.230812583	,],
[	0.24675548	,	0.074160966	,	0.226565608	,	0.2779678	,],
[	0.320007626	,	0.280473343	,	0.064585436	,	0.254785092	,],
[	0.26870584	,	0.348414468	,	0.273571033	,	0.07413818	,]],
[[	0.078038157	,	0.286229799	,	0.305623505	,	0.234271639	,],
[	0.24342151	,	0.072546975	,	0.223579751	,	0.274634742	,],
[	0.317921164	,	0.278796816	,	0.064393343	,	0.252810262	,],
[	0.269954151	,	0.349754037	,	0.274875948	,	0.077732224	,]],
[[	0.074865581	,	0.282096103	,	0.301656204	,	0.230105041	,],
[	0.246408337	,	0.078851209	,	0.227318083	,	0.277974431	,],
[	0.32178377	,	0.283435786	,	0.071698837	,	0.256927599	,],
[	0.271069677	,	0.3506208	,	0.275796593	,	0.075816541	,]],
[[	0.079967346	,	0.284585383	,	0.304172758	,	0.232771217	,],
[	0.244578998	,	0.072504012	,	0.224507637	,	0.275842716	,],
[	0.31836244	,	0.278879994	,	0.063484475	,	0.253091424	,],
[	0.26847843	,	0.348176712	,	0.273343987	,	0.074638476	,]],
[[	0.075551911	,	0.286297738	,	0.305777535	,	0.230225792	,],
[	0.242883918	,	0.074793836	,	0.227445998	,	0.274858173	,],
[	0.31571041	,	0.281571899	,	0.064542385	,	0.251151419	,],
[	0.267232518	,	0.351927756	,	0.277198142	,	0.074244288	,]],
[[	0.078976925	,	0.291692152	,	0.310985852	,	0.235601126	,],
[	0.238636038	,	0.072090061	,	0.22355131	,	0.270837699	,],
[	0.313592972	,	0.27976189	,	0.064189666	,	0.24897881	,],
[	0.268380116	,	0.352948691	,	0.278404003	,	0.077624431	,]],
[[	0.074328693	,	0.286387242	,	0.305654067	,	0.230111695	,],
[	0.240745866	,	0.077424241	,	0.226442852	,	0.273124973	,],
[	0.317335935	,	0.284240353	,	0.071388796	,	0.25298857	,],
[	0.269687084	,	0.354114139	,	0.279657015	,	0.076137489	,]],
[[	0.07827645	,	0.287889731	,	0.30722457	,	0.231694689	,],
[	0.240344843	,	0.072758813	,	0.225081304	,	0.272510996	,],
[	0.314113964	,	0.280038212	,	0.063438661	,	0.249380937	,],
[	0.266876732	,	0.351509984	,	0.277001467	,	0.074757477	,]],
[[	0.076192503	,	0.288822063	,	0.308130822	,	0.232633966	,],
[	0.23865895	,	0.073426308	,	0.223931147	,	0.271015327	,],
[	0.31451956	,	0.281062613	,	0.066335032	,	0.249959996	,],
[	0.270553225	,	0.355260636	,	0.280728736	,	0.081014455	,]],
[[	0.074996165	,	0.286167938	,	0.305467098	,	0.229906909	,],
[	0.243280733	,	0.081479115	,	0.22923268	,	0.275594466	,],
[	0.316883078	,	0.282588452	,	0.065508306	,	0.252168925	,],
[	0.267781913	,	0.352344648	,	0.277659986	,	0.074555646	,]],
[[	0.080503467	,	0.293147238	,	0.312287206	,	0.236984754	,],
[	0.238881955	,	0.071998099	,	0.22375474	,	0.27098259	,],
[	0.313548103	,	0.279578486	,	0.063688555	,	0.24882497	,],
[	0.267845806	,	0.35241367	,	0.277792784	,	0.076448924	,]],
[[	0.074625738	,	0.286867913	,	0.306192749	,	0.230605753	,],
[	0.240052469	,	0.076066236	,	0.225506917	,	0.272245593	,],
[	0.316367112	,	0.283248396	,	0.06984395	,	0.251987728	,],
[	0.270841588	,	0.355207287	,	0.280759526	,	0.077175748	,]],
[[	0.077025156	,	0.287249865	,	0.306549341	,	0.23099713	,],
[	0.241300764	,	0.07341994	,	0.225885598	,	0.273289105	,],
[	0.31474429	,	0.280518358	,	0.06363499	,	0.249942047	,],
[	0.267071015	,	0.351608146	,	0.276940171	,	0.074328409	,]],
[[	0.077254724	,	0.28995462	,	0.309122292	,	0.233610853	,],
[	0.238652588	,	0.072838549	,	0.223734271	,	0.270795521	,],
[	0.314087758	,	0.280374812	,	0.065363111	,	0.249344584	,],
[	0.269660431	,	0.354315214	,	0.279704988	,	0.07944876	,]],
[[	0.074634046	,	0.286192255	,	0.305496164	,	0.229857444	,],
[	0.242198482	,	0.079936854	,	0.228140682	,	0.274555752	,],
[	0.318108961	,	0.283776171	,	0.06653018	,	0.2534314	,],
[	0.268377551	,	0.352897021	,	0.278267601	,	0.074950884	,]],
[[	0.080321957	,	0.289106315	,	0.308479087	,	0.233021476	,],
[	0.239394377	,	0.072180553	,	0.224132853	,	0.271500041	,],
[	0.313674414	,	0.279628918	,	0.063429253	,	0.24901234	,],
[	0.26738098	,	0.352039233	,	0.277368015	,	0.075595898	,]],
[[	0.075145097	,	0.29107941	,	0.310087816	,	0.230421432	,],
[	0.238624728	,	0.081487079	,	0.229236884	,	0.271610183	,],
[	0.312866382	,	0.283944893	,	0.066081874	,	0.249000447	,],
[	0.267064046	,	0.356220465	,	0.281481998	,	0.074325325	,]],
[[	0.079021837	,	0.296738246	,	0.315626165	,	0.23614356	,],
[	0.23474073	,	0.072343051	,	0.224192812	,	0.267495551	,],
[	0.309290928	,	0.280762886	,	0.063766226	,	0.245268871	,],
[	0.266627467	,	0.356051515	,	0.281225309	,	0.075838857	,]],
[[	0.074085583	,	0.291074833	,	0.31008879	,	0.230401622	,],
[	0.235605316	,	0.076229585	,	0.225696722	,	0.268626797	,],
[	0.311870649	,	0.283956597	,	0.069527352	,	0.248067215	,],
[	0.270207693	,	0.359235731	,	0.284516984	,	0.077098447	,]],
[[	0.077634707	,	0.292376125	,	0.311386159	,	0.231643415	,],
[	0.237354419	,	0.074025112	,	0.226502823	,	0.269942504	,],
[	0.310634705	,	0.281856379	,	0.064038487	,	0.246668523	,],
[	0.266077631	,	0.355351115	,	0.280608114	,	0.074041626	,]],
[[	0.076188011	,	0.293727778	,	0.312641351	,	0.233049164	,],
[	0.234287998	,	0.072948884	,	0.223975284	,	0.26711413	,],
[	0.309579299	,	0.281264134	,	0.065280827	,	0.245703524	,],
[	0.268465123	,	0.357645941	,	0.282976264	,	0.078713209	,]],
[[	0.074596276	,	0.290797284	,	0.309815973	,	0.230118886	,],
[	0.237754168	,	0.07991575	,	0.228190369	,	0.270726928	,],
[	0.314186524	,	0.285181628	,	0.067117504	,	0.250254028	,],
[	0.267840736	,	0.356914792	,	0.282221755	,	0.074825992	,]],
[[	0.0805588	,	0.29827941	,	0.317142047	,	0.237682124	,],
[	0.235269812	,	0.072590064	,	0.224592808	,	0.267940261	,],
[	0.309513887	,	0.280864411	,	0.063584332	,	0.245398554	,],
[	0.266364247	,	0.355592438	,	0.280740107	,	0.074941975	,]],
[[	0.074508767	,	0.291673115	,	0.310571516	,	0.230929113	,],
[	0.235103582	,	0.075204724	,	0.225030337	,	0.267977559	,],
[	0.311121614	,	0.283162285	,	0.068096168	,	0.247201187	,],
[	0.271054489	,	0.360224657	,	0.285501271	,	0.082617304	,]],
[[	0.076552092	,	0.291698234	,	0.310717941	,	0.23100745	,],
[	0.238449508	,	0.075036958	,	0.22750368	,	0.27095067	,],
[	0.311457457	,	0.282625038	,	0.064728926	,	0.247376415	,],
[	0.266332902	,	0.355604021	,	0.280743794	,	0.073949008	,]],
[[	0.077179473	,	0.294822909	,	0.31366299	,	0.234116034	,],
[	0.234488362	,	0.072727485	,	0.223930243	,	0.26715826	,],
[	0.309428041	,	0.280994936	,	0.064539263	,	0.245400629	,],
[	0.26771	,	0.356925191	,	0.282177402	,	0.077398585	,]],
[[	0.07438469	,	0.290933334	,	0.309886341	,	0.230176799	,],
[	0.23678724	,	0.078301536	,	0.227138679	,	0.269736883	,],
[	0.313283401	,	0.285745807	,	0.072016324	,	0.249460003	,],
[	0.268735144	,	0.357708971	,	0.28304008	,	0.075576166	,]],
[[	0.079694528	,	0.293696081	,	0.312701279	,	0.233073167	,],
[	0.236072544	,	0.073032989	,	0.225126886	,	0.268572059	,],
[	0.309972253	,	0.281140955	,	0.06365908	,	0.245742641	,],
[	0.266031484	,	0.355373457	,	0.280579347	,	0.07435956	,]],
[[	0.075628288	,	0.297579849	,	0.316383453	,	0.231822639	,],
[	0.234881698	,	0.081320421	,	0.230872279	,	0.268603464	,],
[	0.311459704	,	0.287227647	,	0.067352865	,	0.247534046	,],
[	0.266816975	,	0.360633502	,	0.285416263	,	0.074465764	,]],
[[	0.078296685	,	0.302273341	,	0.320992233	,	0.236639997	,],
[	0.232106835	,	0.073661666	,	0.226894568	,	0.265508703	,],
[	0.307293914	,	0.283384472	,	0.064376054	,	0.243321305	,],
[	0.267862437	,	0.36185071	,	0.28657306	,	0.077743092	,]],
[[	0.074180485	,	0.297440592	,	0.316067065	,	0.23157789	,],
[	0.232219837	,	0.076494233	,	0.227704577	,	0.265810233	,],
[	0.309408789	,	0.286162749	,	0.069542272	,	0.245569307	,],
[	0.269293518	,	0.362920573	,	0.287710947	,	0.076367007	,]],
[[	0.078399317	,	0.299214943	,	0.317957085	,	0.233423833	,],
[	0.235067714	,	0.075800132	,	0.229559307	,	0.268301147	,],
[	0.309072901	,	0.28483511	,	0.06515039	,	0.245083986	,],
[	0.266540059	,	0.360464077	,	0.285131956	,	0.074794542	,]],
[[	0.075656211	,	0.299520245	,	0.318213929	,	0.233816124	,],
[	0.231340919	,	0.073902727	,	0.22641479	,	0.264817466	,],
[	0.307564687	,	0.283831372	,	0.065600447	,	0.243479682	,],
[	0.26997429	,	0.363833558	,	0.28871226	,	0.081122995	,]],
[[	0.075021863	,	0.297423058	,	0.316036996	,	0.231511754	,],
[	0.234005038	,	0.079792484	,	0.229763241	,	0.267548136	,],
[	0.31178403	,	0.288830688	,	0.073530569	,	0.248117608	,],
[	0.267279105	,	0.361140746	,	0.285782084	,	0.07463119	,]],
[[	0.079707145	,	0.303703189	,	0.322259491	,	0.238031629	,],
[	0.232830379	,	0.07401864	,	0.227509847	,	0.266106435	,],
[	0.307679204	,	0.283522833	,	0.064346358	,	0.243510705	,],
[	0.267241483	,	0.36123424	,	0.285912648	,	0.076603492	,]],
[[	0.074370811	,	0.297803457	,	0.316430035	,	0.231892644	,],
[	0.231857222	,	0.07556124	,	0.227007186	,	0.265245476	,],
[	0.308787196	,	0.285394566	,	0.068205913	,	0.244808997	,],
[	0.270505267	,	0.364032704	,	0.288828208	,	0.077409154	,]],
[[	0.077243499	,	0.298563018	,	0.317322155	,	0.2326669	,],
[	0.236344515	,	0.076956596	,	0.23082615	,	0.269516706	,],
[	0.310016919	,	0.28573049	,	0.065898423	,	0.245885003	,],
[	0.266438975	,	0.36036214	,	0.285087323	,	0.074413198	,]],
[[	0.07659896	,	0.300478006	,	0.319029559	,	0.234651956	,],
[	0.231585757	,	0.073640933	,	0.226440939	,	0.264929598	,],
[	0.307347278	,	0.283579358	,	0.065001925	,	0.243384989	,],
[	0.26913268	,	0.363064711	,	0.287851827	,	0.079573986	,]],
[[	0.074591217	,	0.297370304	,	0.316028753	,	0.231360859	,],
[	0.233200416	,	0.078363729	,	0.228833091	,	0.266735804	,],
[	0.310755731	,	0.287725879	,	0.071880824	,	0.247046385	,],
[	0.268038296	,	0.361813153	,	0.286481798	,	0.075207004	,]],
[[	0.080640941	,	0.30058442	,	0.319316542	,	0.234944113	,],
[	0.233668052	,	0.074681505	,	0.228245454	,	0.266899845	,],
[	0.308224781	,	0.284033168	,	0.064479649	,	0.24403389	,],
[	0.266893526	,	0.360851116	,	0.285452364	,	0.075695341	,]],
[[	0.075285197	,	0.300471046	,	0.318239611	,	0.230678673	,],
[	0.227338828	,	0.079010923	,	0.227530209	,	0.261687535	,],
[	0.304354641	,	0.287199939	,	0.072872402	,	0.242015059	,],
[	0.262488994	,	0.36080894	,	0.285586422	,	0.073865955	,]],
[[	0.078094651	,	0.30522935	,	0.322998084	,	0.235688644	,],
[	0.226196386	,	0.073210419	,	0.225222867	,	0.260203693	,],
[	0.300114966	,	0.281613555	,	0.063206023	,	0.237305698	,],
[	0.262834508	,	0.36143918	,	0.286046623	,	0.076263801	,]],
[[	0.073876838	,	0.300258907	,	0.318038602	,	0.230457944	,],
[	0.225110959	,	0.074673535	,	0.224810204	,	0.259387143	,],
[	0.301428756	,	0.283675454	,	0.067581405	,	0.238726061	,],
[	0.265398588	,	0.363604798	,	0.28839128	,	0.076124528	,]],
[[	0.077870505	,	0.301943847	,	0.319855882	,	0.232273494	,],
[	0.229873705	,	0.0761359	,	0.228682894	,	0.263812582	,],
[	0.302341837	,	0.283596373	,	0.064545815	,	0.239474797	,],
[	0.261843052	,	0.360304779	,	0.285071536	,	0.073808837	,]],
[[	0.075455967	,	0.302507252	,	0.320252419	,	0.232792886	,],
[	0.224916593	,	0.072813476	,	0.224166552	,	0.259057262	,],
[	0.30003668	,	0.281737161	,	0.064141308	,	0.237083475	,],
[	0.264856601	,	0.363372108	,	0.288162993	,	0.0795145	,]],
[[	0.074612475	,	0.300207799	,	0.318067241	,	0.230278446	,],
[	0.226614676	,	0.077651197	,	0.226564243	,	0.260909497	,],
[	0.30366526	,	0.286141189	,	0.071337246	,	0.241041346	,],
[	0.26314544	,	0.361540556	,	0.286253368	,	0.074221986	,]],
[[	0.079549213	,	0.306726934	,	0.324393305	,	0.237099129	,],
[	0.227072868	,	0.073877495	,	0.225982933	,	0.261043017	,],
[	0.300488241	,	0.282000646	,	0.063279719	,	0.237625355	,],
[	0.262406766	,	0.360894372	,	0.285654104	,	0.075310724	,]],
[[	0.074094722	,	0.30073096	,	0.31850309	,	0.230949265	,],
[	0.224874782	,	0.073856728	,	0.224466019	,	0.259064759	,],
[	0.300876657	,	0.282963753	,	0.066225949	,	0.238012489	,],
[	0.266692335	,	0.3647066	,	0.289595086	,	0.077224232	,]],
[[	0.07674871	,	0.301193855	,	0.319075088	,	0.231441914	,],
[	0.228928408	,	0.081571944	,	0.2292505	,	0.263131531	,],
[	0.303449289	,	0.284541834	,	0.065329212	,	0.240406544	,],
[	0.261964372	,	0.360409892	,	0.285174334	,	0.073611336	,]],
[[	0.076400581	,	0.303555092	,	0.321282969	,	0.233771566	,],
[	0.225362867	,	0.072892764	,	0.224507649	,	0.259417494	,],
[	0.299988482	,	0.281540988	,	0.06358471	,	0.237001778	,],
[	0.263954621	,	0.362480619	,	0.28715418	,	0.07813234	,]],
[[	0.074229257	,	0.300187974	,	0.317953441	,	0.230281079	,],
[	0.22603697	,	0.076422251	,	0.225842667	,	0.260173397	,],
[	0.302607649	,	0.285114889	,	0.069749442	,	0.239898688	,],
[	0.263915068	,	0.362231202	,	0.286941985	,	0.074806935	,]],
[[	0.080062743	,	0.303338202	,	0.321199609	,	0.233685267	,],
[	0.228127509	,	0.074601988	,	0.22689405	,	0.26192725	,],
[	0.301143595	,	0.28260916	,	0.063660972	,	0.238222776	,],
[	0.262052891	,	0.360516496	,	0.285267192	,	0.074589937	,]],
[[	0.075721439	,	0.307769686	,	0.325136422	,	0.232364615	,],
[	0.224768653	,	0.079083	,	0.229894232	,	0.259381523	,],
[	0.305082971	,	0.292483192	,	0.073241806	,	0.241138659	,],
[	0.266277255	,	0.371070076	,	0.294103608	,	0.073903179	,]],
[[	0.078885343	,	0.312800492	,	0.329921057	,	0.237629218	,],
[	0.22488821	,	0.074818275	,	0.228853281	,	0.259143931	,],
[	0.301786003	,	0.287920034	,	0.064854355	,	0.237411334	,],
[	0.266457149	,	0.371351638	,	0.294286447	,	0.076110312	,]],
[[	0.074395724	,	0.307672043	,	0.324917182	,	0.232300891	,],
[	0.222917593	,	0.07522303	,	0.227492843	,	0.257482505	,],
[	0.302274126	,	0.289143847	,	0.068189796	,	0.23809667	,],
[	0.269178224	,	0.373770323	,	0.296829686	,	0.0763955	,]],
[[	0.078250844	,	0.309283436	,	0.326481867	,	0.233827845	,],
[	0.227207852	,	0.083125075	,	0.232555861	,	0.261815429	,],
[	0.304226847	,	0.290217	,	0.066671661	,	0.239947923	,],
[	0.26541607	,	0.370375838	,	0.293209918	,	0.073812239	,]],
[[	0.076071041	,	0.309994557	,	0.327172961	,	0.234625227	,],
[	0.223316379	,	0.073955852	,	0.227491091	,	0.257687427	,],
[	0.301218738	,	0.287646061	,	0.065192245	,	0.236930232	,],
[	0.26847028	,	0.373340836	,	0.296401572	,	0.079256219	,]],
[[	0.075063991	,	0.307619858	,	0.32479293	,	0.232157384	,],
[	0.224067818	,	0.077686979	,	0.228963307	,	0.258549614	,],
[	0.304249254	,	0.291493917	,	0.071612749	,	0.240156925	,],
[	0.266908033	,	0.371531461	,	0.294556499	,	0.074350275	,]],
[[	0.080287005	,	0.314271631	,	0.331370362	,	0.238974997	,],
[	0.225977091	,	0.075625405	,	0.229842205	,	0.260158083	,],
[	0.3023472	,	0.288432935	,	0.06511443	,	0.237997137	,],
[	0.266090677	,	0.370933597	,	0.293926768	,	0.07523164	,]],
[[	0.074670961	,	0.308106701	,	0.325279636	,	0.232730648	,],
[	0.222793598	,	0.074592934	,	0.227226029	,	0.257242506	,],
[	0.301829223	,	0.288609694	,	0.0670146	,	0.237451115	,],
[	0.270490841	,	0.374943318	,	0.298065047	,	0.077541797	,]],
[[	0.077177377	,	0.308600861	,	0.325891139	,	0.233104908	,],
[	0.226110158	,	0.081364766	,	0.231413689	,	0.260727306	,],
[	0.305439541	,	0.291446684	,	0.067617119	,	0.241117476	,],
[	0.265651903	,	0.370416734	,	0.293382988	,	0.073671481	,]],
[[	0.077038604	,	0.311052195	,	0.328118423	,	0.235636278	,],
[	0.223864361	,	0.074215764	,	0.227818219	,	0.258105608	,],
[	0.301388624	,	0.287617743	,	0.064925489	,	0.236840635	,],
[	0.267487526	,	0.372375077	,	0.295420873	,	0.077904045	,]],
[[	0.074751985	,	0.307499769	,	0.324732681	,	0.23198992	,],
[	0.223502014	,	0.076597204	,	0.228265396	,	0.258003608	,],
[	0.30335557	,	0.290517658	,	0.070124914	,	0.239155678	,],
[	0.267766958	,	0.372386675	,	0.295378982	,	0.074965786	,]],
[[	0.080551185	,	0.310606676	,	0.327956866	,	0.235260564	,],
[	0.227135761	,	0.076637233	,	0.230953641	,	0.261271997	,],
[	0.302969906	,	0.289031999	,	0.065587176	,	0.238537308	,],
[	0.265707145	,	0.370624372	,	0.293528138	,	0.074507296	]]],dtype=np.float32)
								
x_train=np.array([[[	0.109434342	,	0.238589958	,	0.25673569	,	0.258792364	,],
[	0.407905154	,	0.093874632	,	0.254000982	,	0.395377646	,],
[	0.485885885	,	0.323996463	,	0.081595254	,	0.370534408	,],
[	0.382072501	,	0.349828124	,	0.25471866	,	0.099717967	,]],
[[	0.104949193	,	0.232680587	,	0.251049251	,	0.25328152	,],
[	0.407339243	,	0.090059773	,	0.253270834	,	0.394904272	,],
[	0.484895058	,	0.322832457	,	0.082046195	,	0.369377255	,],
[	0.38404278	,	0.351398866	,	0.256389893	,	0.103802613	,]],
[[	0.10782474	,	0.232846618	,	0.251265073	,	0.253957981	,],
[	0.406125877	,	0.090450689	,	0.252042036	,	0.39357787	,],
[	0.488934827	,	0.327310058	,	0.084730739	,	0.373856654	,],
[	0.38378137	,	0.351821789	,	0.256802022	,	0.100781039	,]],
[[	0.106633896	,	0.235484878	,	0.25374128	,	0.255828723	,],
[	0.410157157	,	0.097535099	,	0.256565564	,	0.39773928	,],
[	0.48482842	,	0.322867218	,	0.081031814	,	0.369332907	,],
[	0.382293612	,	0.34996916	,	0.254910605	,	0.100663145	,]],
[[	0.105067121	,	0.232136264	,	0.25057601	,	0.25277285	,],
[	0.406027182	,	0.089102752	,	0.251766802	,	0.393434627	,],
[	0.486316499	,	0.324307948	,	0.08448279	,	0.370797033	,],
[	0.386475575	,	0.353673314	,	0.258782689	,	0.107374858	,]],
[[	0.110968122	,	0.234832693	,	0.253352571	,	0.256015464	,],
[	0.407433438	,	0.092748054	,	0.253255989	,	0.394784844	,],
[	0.486728339	,	0.324901958	,	0.082411458	,	0.371413508	,],
[	0.382743573	,	0.35060686	,	0.255472142	,	0.100032643	,]],
[[	0.105144607	,	0.233302763	,	0.251462638	,	0.253735684	,],
[	0.408169778	,	0.090594896	,	0.253970224	,	0.395457615	,],
[	0.48486011	,	0.322752521	,	0.081488878	,	0.369231541	,],
[	0.383684706	,	0.350942665	,	0.255928281	,	0.102738673	,]],
[[	0.106892874	,	0.232537826	,	0.250907944	,	0.25344066	,],
[	0.405950438	,	0.089707989	,	0.251639058	,	0.393274676	,],
[	0.4881456	,	0.326308258	,	0.087695986	,	0.372702409	,],
[	0.384751744	,	0.352879795	,	0.257886579	,	0.101669266	,]],
[[	0.107635468	,	0.23669354	,	0.254901031	,	0.256876961	,],
[	0.409006874	,	0.095858652	,	0.255259924	,	0.396572813	,],
[	0.485324068	,	0.323281926	,	0.081022164	,	0.369788481	,],
[	0.382287027	,	0.34993716	,	0.254849441	,	0.100120858	,]],
[[	0.105187023	,	0.232438499	,	0.25081358	,	0.253020346	,],
[	0.406578832	,	0.089412024	,	0.252282058	,	0.393857786	,],
[	0.485843041	,	0.323768034	,	0.083568285	,	0.370238774	,],
[	0.385341099	,	0.352563626	,	0.257589058	,	0.105658411	,]],
[[	0.108872489	,	0.242517981	,	0.258750393	,	0.256881686	,],
[	0.39704893	,	0.090355109	,	0.249570656	,	0.386126503	,],
[	0.478229421	,	0.323904302	,	0.081874919	,	0.365345321	,],
[	0.375767014	,	0.35114613	,	0.256172984	,	0.098720978	,]],
[[	0.103824279	,	0.23620994	,	0.252598859	,	0.250912481	,],
[	0.399902478	,	0.09084506	,	0.252422344	,	0.388886813	,],
[	0.47562322	,	0.32091675	,	0.079949696	,	0.362450473	,],
[	0.376062934	,	0.351026077	,	0.256037371	,	0.100792177	,]],
[[	0.106287862	,	0.236021522	,	0.252583268	,	0.25118942	,],
[	0.39645998	,	0.088485004	,	0.248866423	,	0.385416683	,],
[	0.477926261	,	0.323482277	,	0.084928097	,	0.36486709	,],
[	0.378374733	,	0.353974131	,	0.259085627	,	0.100833478	,]],
[[	0.105912639	,	0.23929789	,	0.255684381	,	0.253699741	,],
[	0.398473762	,	0.093049109	,	0.251045767	,	0.38737433	,],
[	0.476507893	,	0.322015276	,	0.080177559	,	0.363500485	,],
[	0.375101723	,	0.350353376	,	0.255413127	,	0.098624652	,]],
[[	0.104112318	,	0.235679837	,	0.252179807	,	0.250443938	,],
[	0.397591451	,	0.088810684	,	0.249924867	,	0.38654374	,],
[	0.47596119	,	0.321315914	,	0.081243998	,	0.362675093	,],
[	0.378066427	,	0.352897224	,	0.257998963	,	0.103800339	,]],
[[	0.109359501	,	0.237874824	,	0.25448369	,	0.253160976	,],
[	0.396833107	,	0.089692575	,	0.249122097	,	0.385605861	,],
[	0.479539804	,	0.325187176	,	0.083100408	,	0.366596421	,],
[	0.376303065	,	0.351748454	,	0.256632055	,	0.099029372	,]],
[[	0.104424417	,	0.237174277	,	0.253517883	,	0.251731354	,],
[	0.400620373	,	0.096663087	,	0.253496148	,	0.389589051	,],
[	0.475771245	,	0.321076708	,	0.079800429	,	0.362474802	,],
[	0.375824523	,	0.350790048	,	0.255678355	,	0.100004569	,]],
[[	0.105445893	,	0.235671056	,	0.252293014	,	0.250700056	,],
[	0.396737081	,	0.088295859	,	0.249066249	,	0.385656767	,],
[	0.477209506	,	0.322639782	,	0.083502809	,	0.363942933	,],
[	0.379549758	,	0.355194058	,	0.260217634	,	0.101884241	,]],
[[	0.107106631	,	0.240621781	,	0.256887771	,	0.254889876	,],
[	0.398046519	,	0.09210335	,	0.250497677	,	0.386869824	,],
[	0.477400392	,	0.322846971	,	0.080872476	,	0.364295201	,],
[	0.375284124	,	0.350593007	,	0.255530382	,	0.098443239	,]],
[[	0.103908238	,	0.235745871	,	0.252270349	,	0.25045144	,],
[	0.398757326	,	0.089737916	,	0.251051765	,	0.387552783	,],
[	0.476066315	,	0.321267109	,	0.080745783	,	0.362681464	,],
[	0.377214966	,	0.352079202	,	0.257162892	,	0.102424594	,]],
[[	0.107666511	,	0.24699705	,	0.26266124	,	0.257228048	,],
[	0.387930015	,	0.088555694	,	0.247018824	,	0.379481236	,],
[	0.471076321	,	0.323661157	,	0.085320772	,	0.360577451	,],
[	0.372474497	,	0.354656901	,	0.259792097	,	0.099731247	,]],
[[	0.102063772	,	0.24034251	,	0.256104266	,	0.250857492	,],
[	0.390767276	,	0.094156784	,	0.250244976	,	0.382369171	,],
[	0.46898532	,	0.321336973	,	0.079740732	,	0.358392888	,],
[	0.369947459	,	0.351813415	,	0.256925283	,	0.098461424	,]],
[[	0.104366713	,	0.240154092	,	0.256075378	,	0.25117602	,],
[	0.389067002	,	0.088642978	,	0.248090737	,	0.380524461	,],
[	0.468794822	,	0.321036058	,	0.081262012	,	0.358106239	,],
[	0.373590586	,	0.355016619	,	0.260161441	,	0.104423112	,]],
[[	0.104588162	,	0.24383275	,	0.259523627	,	0.25408938	,],
[	0.388349883	,	0.089942843	,	0.247461695	,	0.379952079	,],
[	0.471570457	,	0.324120697	,	0.082206104	,	0.361249986	,],
[	0.370706452	,	0.352828243	,	0.257975081	,	0.098325342	,]],
[[	0.101760065	,	0.239245267	,	0.255165956	,	0.249879625	,],
[	0.39194248	,	0.091175762	,	0.251001422	,	0.383408655	,],
[	0.468154707	,	0.320416684	,	0.079375934	,	0.357488558	,],
[	0.371258925	,	0.35300391	,	0.258137516	,	0.100705612	,]],
[[	0.10728727	,	0.241847036	,	0.257756397	,	0.252960529	,],
[	0.38825098	,	0.088456814	,	0.247219867	,	0.379706412	,],
[	0.470303499	,	0.322681942	,	0.083944291	,	0.359599199	,],
[	0.373522743	,	0.355665333	,	0.260901337	,	0.100669665	,]],
[[	0.102530686	,	0.241200307	,	0.256850439	,	0.251560357	,],
[	0.3897455	,	0.092619521	,	0.249001826	,	0.381308139	,],
[	0.469528476	,	0.321958726	,	0.080267573	,	0.358916635	,],
[	0.370074686	,	0.351941655	,	0.256975237	,	0.098230618	,]],
[[	0.103358542	,	0.239748678	,	0.255562393	,	0.250492635	,],
[	0.389743305	,	0.089201484	,	0.248799046	,	0.3812243	,],
[	0.468629401	,	0.320813284	,	0.080675673	,	0.357838653	,],
[	0.372863186	,	0.354293342	,	0.259450545	,	0.103208717	,]],
[[	0.105873091	,	0.245055114	,	0.260815815	,	0.255264354	,],
[	0.388392168	,	0.089448237	,	0.247338954	,	0.379784805	,],
[	0.472342069	,	0.324967295	,	0.087437789	,	0.361742915	,],
[	0.371541684	,	0.353595281	,	0.258760001	,	0.098718173	,]],
[[	0.102036153	,	0.239905821	,	0.255770782	,	0.250370499	,],
[	0.392021605	,	0.096316357	,	0.251523843	,	0.383490523	,],
[	0.468438219	,	0.320659148	,	0.079503028	,	0.357689189	,],
[	0.370695767	,	0.352416761	,	0.257511351	,	0.099548785	,]],
[[	0.106837111	,	0.245880113	,	0.260003165	,	0.250407641	,],
[	0.381027808	,	0.087776196	,	0.245221056	,	0.373136901	,],
[	0.462266336	,	0.3209679	,	0.082095271	,	0.353297758	,],
[	0.368235441	,	0.356038992	,	0.260720977	,	0.099228967	,]],
[[	0.101405064	,	0.244614495	,	0.258508669	,	0.248552952	,],
[	0.381748961	,	0.090755345	,	0.246130659	,	0.373837802	,],
[	0.462779335	,	0.321558374	,	0.080056043	,	0.354026781	,],
[	0.365024413	,	0.352598581	,	0.257121136	,	0.096978314	,]],
[[	0.102996483	,	0.243823351	,	0.257794555	,	0.248103255	,],
[	0.383458007	,	0.089554543	,	0.247636754	,	0.3754519	,],
[	0.460674561	,	0.319284947	,	0.079175126	,	0.35173458	,],
[	0.368177206	,	0.355286969	,	0.259928916	,	0.102452519	,]],
[[	0.103889502	,	0.247919663	,	0.261844951	,	0.251645652	,],
[	0.38067827	,	0.088271374	,	0.244889376	,	0.3727971	,],
[	0.463977076	,	0.322876517	,	0.085214516	,	0.355135833	,],
[	0.366275493	,	0.354098523	,	0.258595309	,	0.097418764	,]],
[[	0.100922316	,	0.243352956	,	0.257400491	,	0.247444286	,],
[	0.38381572	,	0.094001094	,	0.24832644	,	0.37588769	,],
[	0.461302932	,	0.319917352	,	0.078864433	,	0.352419643	,],
[	0.365812054	,	0.353220248	,	0.257787732	,	0.098524198	,]],
[[	0.105639167	,	0.245219015	,	0.259170213	,	0.249676898	,],
[	0.381530277	,	0.088013711	,	0.245610207	,	0.373463977	,],
[	0.461495447	,	0.32019396	,	0.080884861	,	0.352493134	,],
[	0.369582278	,	0.357392307	,	0.262005955	,	0.100307913	,]],
[[	0.101845019	,	0.24527837	,	0.25916106	,	0.249125464	,],
[	0.381242253	,	0.0896782	,	0.245409118	,	0.373135758	,],
[	0.463841077	,	0.322580241	,	0.081015845	,	0.355006846	,],
[	0.365330523	,	0.352868991	,	0.25738101	,	0.09689679	,]],
[[	0.102213528	,	0.243543201	,	0.257403233	,	0.247729198	,],
[	0.38466106	,	0.090775912	,	0.248911543	,	0.376736531	,],
[	0.460749545	,	0.319256307	,	0.078777601	,	0.351620751	,],
[	0.367363354	,	0.354495277	,	0.2590182	,	0.101010684	,]],
[[	0.105361814	,	0.249543462	,	0.263337341	,	0.253156771	,],
[	0.380660897	,	0.087821194	,	0.244902468	,	0.372775971	,],
[	0.463170898	,	0.322005692	,	0.083778531	,	0.354143485	,],
[	0.366771854	,	0.354577422	,	0.259127181	,	0.097830131	,]],
[[	0.10115374	,	0.243939263	,	0.257805801	,	0.247805374	,],
[	0.382828898	,	0.092590247	,	0.247233078	,	0.374825038	,],
[	0.461947574	,	0.320640931	,	0.07933993	,	0.3529424	,],
[	0.36570951	,	0.353200824	,	0.257618867	,	0.097831067	,]],
[[	0.106305661	,	0.250575368	,	0.262880635	,	0.250040805	,],
[	0.373570016	,	0.087657539	,	0.243215639	,	0.367193998	,],
[	0.454520194	,	0.319633843	,	0.080322125	,	0.347141989	,],
[	0.362904756	,	0.357710228	,	0.262023936	,	0.098405135	,]],
[[	0.100411671	,	0.249179449	,	0.261293058	,	0.247918154	,],
[	0.373375116	,	0.089821471	,	0.243258	,	0.36704425	,],
[	0.456261944	,	0.321402862	,	0.079476793	,	0.348984146	,],
[	0.359922561	,	0.354476473	,	0.258574626	,	0.096389789	,]],
[[	0.102372323	,	0.248627853	,	0.26078454	,	0.247872236	,],
[	0.376314865	,	0.089982586	,	0.24611846	,	0.369976097	,],
[	0.453594691	,	0.318556746	,	0.077918549	,	0.346038435	,],
[	0.362526937	,	0.35666897	,	0.260852937	,	0.101323673	,]],
[[	0.103100772	,	0.25256666	,	0.264581267	,	0.25107873	,],
[	0.372800348	,	0.087733343	,	0.242600775	,	0.366511315	,],
[	0.456776042	,	0.321951943	,	0.083692058	,	0.34929391	,],
[	0.361082727	,	0.355874311	,	0.260075388	,	0.096817515	,]],
[[	0.100088045	,	0.248022897	,	0.260254946	,	0.246901102	,],
[	0.375117317	,	0.092842193	,	0.24528386	,	0.368773973	,],
[	0.45447731	,	0.31947932	,	0.077869631	,	0.347020623	,],
[	0.36031632	,	0.354804025	,	0.258951416	,	0.097570002	,]],
[[	0.10485825	,	0.249823829	,	0.262073225	,	0.249129538	,],
[	0.374114771	,	0.088040824	,	0.243661812	,	0.367661728	,],
[	0.454220164	,	0.319240809	,	0.079408803	,	0.346711213	,],
[	0.364576665	,	0.359365162	,	0.263558583	,	0.099899029	,]],
[[	0.101217742	,	0.250125748	,	0.262186023	,	0.248763206	,],
[	0.373043488	,	0.088914356	,	0.242806227	,	0.366656096	,],
[	0.457370505	,	0.322551711	,	0.080565291	,	0.350144055	,],
[	0.36006331	,	0.354680869	,	0.258763707	,	0.096110025	,]],
[[	0.101295144	,	0.248116966	,	0.260295081	,	0.2471607	,],
[	0.377220174	,	0.090896856	,	0.247066287	,	0.37077144	,],
[	0.453614149	,	0.318661019	,	0.077657395	,	0.346137736	,],
[	0.361697745	,	0.355959289	,	0.260145667	,	0.100000287	,]],
[[	0.104693233	,	0.254018354	,	0.26607914	,	0.252579432	,],
[	0.372920055	,	0.087509281	,	0.24267324	,	0.366631723	,],
[	0.455995181	,	0.321130144	,	0.082399442	,	0.348454596	,],
[	0.361663951	,	0.356477749	,	0.260722056	,	0.097154944	,]],
[[	0.100070938	,	0.248375224	,	0.260525015	,	0.247150466	,],
[	0.3744741	,	0.091595048	,	0.244501478	,	0.368074486	,],
[	0.455225041	,	0.320258455	,	0.078571436	,	0.347788997	,],
[	0.359983922	,	0.35440852	,	0.258498958	,	0.096726467	,]],
[[	0.104357644	,	0.254561108	,	0.265933705	,	0.248586712	,],
[	0.367855519	,	0.088948392	,	0.243237596	,	0.36247122	,],
[	0.446471477	,	0.318730947	,	0.079356064	,	0.340713239	,],
[	0.360016163	,	0.360447519	,	0.26489325	,	0.103877144	,]],
[[	0.099156007	,	0.253443658	,	0.264550979	,	0.246867219	,],
[	0.366178608	,	0.088869921	,	0.241522949	,	0.36081764	,],
[	0.450566249	,	0.323057001	,	0.081755555	,	0.345196668	,],
[	0.355697924	,	0.356553537	,	0.260731516	,	0.095638941	,]],
[[	0.100444876	,	0.252392896	,	0.263581324	,	0.246173732	,],
[	0.369636171	,	0.09546404	,	0.245534257	,	0.36424282	,],
[	0.446489116	,	0.318711059	,	0.078176803	,	0.340755374	,],
[	0.357035485	,	0.357562509	,	0.261832563	,	0.099025903	,]],
[[	0.101970423	,	0.257068015	,	0.268083354	,	0.250189286	,],
[	0.366406098	,	0.087987904	,	0.241758875	,	0.36106769	,],
[	0.447934174	,	0.320275984	,	0.081806617	,	0.342131823	,],
[	0.357461692	,	0.35859079	,	0.262835783	,	0.096843191	,]],
[[	0.098595344	,	0.252261256	,	0.263434932	,	0.245705623	,],
[	0.36708685	,	0.091022303	,	0.242782298	,	0.361805343	,],
[	0.448405474	,	0.320666032	,	0.079374128	,	0.342850447	,],
[	0.355510954	,	0.356385789	,	0.260646067	,	0.096167339	,]],
[[	0.103278948	,	0.253807083	,	0.265142333	,	0.247702465	,],
[	0.368985152	,	0.089875909	,	0.244316179	,	0.363580428	,],
[	0.446238131	,	0.318549439	,	0.078547314	,	0.340471647	,],
[	0.359065605	,	0.359580288	,	0.263938237	,	0.1022823	,]],
[[	0.099809801	,	0.254268227	,	0.265267431	,	0.24755527	,],
[	0.36610203	,	0.088434958	,	0.24146134	,	0.360616799	,],
[	0.449835788	,	0.322502572	,	0.08530417	,	0.344301948	,],
[	0.355891486	,	0.356859602	,	0.261056967	,	0.095629667	,]],
[[	0.099714106	,	0.252154269	,	0.263352297	,	0.245831271	,],
[	0.368744179	,	0.093917522	,	0.244428744	,	0.363265785	,],
[	0.446954691	,	0.319182843	,	0.078332771	,	0.341158686	,],
[	0.356470444	,	0.357123591	,	0.26127375	,	0.098046094	,]],
[[	0.103314148	,	0.258306267	,	0.269398982	,	0.251422863	,],
[	0.367104841	,	0.088480867	,	0.242436445	,	0.361712018	,],
[	0.447215845	,	0.319567436	,	0.080575843	,	0.341425427	,],
[	0.358382706	,	0.359455729	,	0.263740706	,	0.097597451	,]],
[[	0.098829202	,	0.252753627	,	0.263913195	,	0.246100374	,],
[	0.366594221	,	0.089978964	,	0.242096742	,	0.361192465	,],
[	0.449237629	,	0.321621378	,	0.080225256	,	0.343670362	,],
[	0.355541933	,	0.356436802	,	0.26056282	,	0.095725081	,]],
[[	0.103161013	,	0.259499379	,	0.270046088	,	0.248298419	,],
[	0.362254035	,	0.095189662	,	0.244775166	,	0.359019922	,],
[	0.438705408	,	0.317308876	,	0.077558422	,	0.335134652	,],
[	0.352640824	,	0.358971181	,	0.264416854	,	0.100053519	,]],
[[	0.099281105	,	0.259349162	,	0.269628278	,	0.247467313	,],
[	0.358181193	,	0.086651507	,	0.240095615	,	0.354907253	,],
[	0.44032867	,	0.319215856	,	0.081717825	,	0.336834928	,],
[	0.34992272	,	0.356709249	,	0.261954401	,	0.093820826	,]],
[[	0.099657234	,	0.257674682	,	0.267994898	,	0.246215407	,],
[	0.359315553	,	0.090238915	,	0.241462752	,	0.356038412	,],
[	0.440212139	,	0.318940936	,	0.078519409	,	0.336679849	,],
[	0.349952451	,	0.356272116	,	0.261705365	,	0.095585252	,]],
[[	0.10209376	,	0.262842091	,	0.27310059	,	0.250787813	,],
[	0.360342919	,	0.088138707	,	0.242256686	,	0.357012392	,],
[	0.438592942	,	0.317316101	,	0.078334967	,	0.334993965	,],
[	0.352828276	,	0.359800115	,	0.265216205	,	0.096342489	,]],
[[	0.098134519	,	0.257550394	,	0.267966098	,	0.245725388	,],
[	0.358139891	,	0.087559418	,	0.240169031	,	0.354865659	,],
[	0.442103156	,	0.321231312	,	0.084919345	,	0.338633883	,],
[	0.349220666	,	0.355880351	,	0.261255388	,	0.093570808	,]],
[[	0.102203759	,	0.258880581	,	0.269388101	,	0.247587377	,],
[	0.361172174	,	0.09350341	,	0.243672677	,	0.357895786	,],
[	0.438955602	,	0.317561905	,	0.077469265	,	0.335340909	,],
[	0.35150566	,	0.357812816	,	0.263203299	,	0.098203099	,]],
[[	0.099613773	,	0.259801213	,	0.269991572	,	0.247986979	,],
[	0.358762082	,	0.086847222	,	0.240523655	,	0.355276865	,],
[	0.439737341	,	0.318521156	,	0.080419471	,	0.335995577	,],
[	0.350557624	,	0.357311825	,	0.262570809	,	0.094290372	,]],
[[	0.099123422	,	0.257618049	,	0.267890886	,	0.245957247	,],
[	0.35874359	,	0.089247563	,	0.240833848	,	0.35544485	,],
[	0.440768807	,	0.319479794	,	0.078882457	,	0.337363122	,],
[	0.34956184	,	0.355963405	,	0.261180087	,	0.094636456	,]],
[[	0.10352429	,	0.264180755	,	0.274450403	,	0.252174058	,],
[	0.361470775	,	0.089138865	,	0.24335736	,	0.358097823	,],
[	0.438491539	,	0.317000386	,	0.077598993	,	0.334642378	,],
[	0.353815936	,	0.360804557	,	0.266270373	,	0.097316412	,]],
[[	0.098746627	,	0.258415674	,	0.268718795	,	0.246646219	,],
[	0.357987079	,	0.086793033	,	0.239838586	,	0.354577854	,],
[	0.441400653	,	0.32036323	,	0.083541959	,	0.337749414	,],
[	0.349454443	,	0.356170927	,	0.261498603	,	0.093554116	,]],
[[	0.10145671	,	0.260531486	,	0.269969989	,	0.24427133	,],
[	0.35185918	,	0.093413403	,	0.240759881	,	0.350353849	,],
[	0.43061611	,	0.315624484	,	0.076607695	,	0.328831447	,],
[	0.346274825	,	0.358410398	,	0.264215807	,	0.097240786	,]],
[[	0.096960269	,	0.25994066	,	0.269183941	,	0.2430785	,],
[	0.348427644	,	0.085530791	,	0.23658601	,	0.346841195	,],
[	0.431792717	,	0.316983866	,	0.080092698	,	0.330046758	,],
[	0.345730899	,	0.358146036	,	0.263948667	,	0.093627114	,]],
[[	0.097669163	,	0.258454469	,	0.267852291	,	0.241933892	,],
[	0.349311488	,	0.088748035	,	0.237797899	,	0.347790383	,],
[	0.432524607	,	0.317624636	,	0.077895637	,	0.330866476	,],
[	0.344397173	,	0.356622986	,	0.262354594	,	0.093740524	,]],
[[	0.099868867	,	0.263647764	,	0.272925874	,	0.246635967	,],
[	0.350993748	,	0.087387355	,	0.239159892	,	0.349332324	,],
[	0.430542125	,	0.315532974	,	0.077189764	,	0.32870891	,],
[	0.34881681	,	0.360562662	,	0.2664332	,	0.100927529	,]],
[[	0.095931208	,	0.258382877	,	0.267796556	,	0.241623311	,],
[	0.348247161	,	0.086086093	,	0.236449094	,	0.346732512	,],
[	0.43341112	,	0.318897032	,	0.083176114	,	0.331747258	,],
[	0.344505006	,	0.356973228	,	0.262747268	,	0.092842516	,]],
[[	0.099934484	,	0.259628333	,	0.269159389	,	0.243236114	,],
[	0.350933644	,	0.091983554	,	0.239713146	,	0.349394223	,],
[	0.430900732	,	0.315843819	,	0.076539996	,	0.329078624	,],
[	0.345701205	,	0.357846036	,	0.263551829	,	0.096028411	,]],
[[	0.09750943	,	0.260617736	,	0.269856737	,	0.243773772	,],
[	0.349054999	,	0.085783435	,	0.237146669	,	0.347276511	,],
[	0.431489912	,	0.316598441	,	0.079243701	,	0.329674446	,],
[	0.346724892	,	0.359116526	,	0.264844568	,	0.094266026	,]],
[[	0.09701183	,	0.258283098	,	0.267598496	,	0.241734562	,],
[	0.348779866	,	0.087776672	,	0.237110323	,	0.347151156	,],
[	0.433506342	,	0.318603827	,	0.078840345	,	0.331841528	,],
[	0.34444434	,	0.356725508	,	0.262278807	,	0.093379201	,]],
[[	0.101374484	,	0.265055872	,	0.274404643	,	0.248098137	,],
[	0.351842937	,	0.088073121	,	0.239956221	,	0.350077058	,],
[	0.430689523	,	0.315637013	,	0.07686986	,	0.328651291	,],
[	0.348129672	,	0.359932163	,	0.265742804	,	0.099536166	,]],
[[	0.096308302	,	0.259130814	,	0.26855407	,	0.242242109	,],
[	0.348067961	,	0.085573025	,	0.236192552	,	0.346355624	,],
[	0.432651216	,	0.317957754	,	0.081875154	,	0.330878388	,],
[	0.345058846	,	0.357470452	,	0.263211582	,	0.09298036	,]],
[[	0.10045447	,	0.266662218	,	0.275145891	,	0.244059481	,],
[	0.345812211	,	0.090046856	,	0.238451339	,	0.343769291	,],
[	0.42914956	,	0.319757852	,	0.077807155	,	0.327343424	,],
[	0.345177848	,	0.364329851	,	0.267903041	,	0.094937002	,]],
[[	0.097081649	,	0.266915774	,	0.275223179	,	0.24387703	,],
[	0.347146209	,	0.088111247	,	0.23928281	,	0.345003622	,],
[	0.427938749	,	0.318585731	,	0.078154037	,	0.326039057	,],
[	0.346899199	,	0.366426812	,	0.270032307	,	0.094347319	,]],
[[	0.097197358	,	0.265039856	,	0.273483827	,	0.242295191	,],
[	0.344539376	,	0.087207638	,	0.236882029	,	0.34242702	,],
[	0.432042579	,	0.323225047	,	0.085492366	,	0.330258138	,],
[	0.344079745	,	0.363389936	,	0.266846187	,	0.0924525	,]],
[[	0.100794975	,	0.27118269	,	0.279492764	,	0.247911685	,],
[	0.347610323	,	0.093272546	,	0.240360984	,	0.345463554	,],
[	0.428151244	,	0.318711968	,	0.077135469	,	0.326099106	,],
[	0.347144207	,	0.366014592	,	0.269568584	,	0.098057372	,]],
[[	0.095993079	,	0.265288636	,	0.273732122	,	0.242232549	,],
[	0.345306665	,	0.086788105	,	0.237490964	,	0.343159355	,],
[	0.429158939	,	0.319850187	,	0.080319411	,	0.327052321	,],
[	0.345139727	,	0.364667924	,	0.268096249	,	0.092678127	,]],
[[	0.09951594	,	0.266200275	,	0.274736301	,	0.243574571	,],
[	0.345335737	,	0.089091356	,	0.237765225	,	0.343206567	,],
[	0.430166061	,	0.320682213	,	0.078671683	,	0.328224647	,],
[	0.344496197	,	0.363693081	,	0.267229412	,	0.093820123	,]],
[[	0.098039163	,	0.267965191	,	0.276171628	,	0.244801708	,],
[	0.34835493	,	0.089195179	,	0.240382659	,	0.346016118	,],
[	0.427911779	,	0.318463298	,	0.077653308	,	0.32579967	,],
[	0.347989396	,	0.367471308	,	0.270970278	,	0.095239657	,]],
[[	0.096666526	,	0.264920823	,	0.273299933	,	0.242087925	,],
[	0.344705482	,	0.086931359	,	0.236775812	,	0.34239059	,],
[	0.43103593	,	0.321987195	,	0.083674471	,	0.329116002	,],
[	0.34417836	,	0.363430929	,	0.266843994	,	0.092206198	,]],
[[	0.101949534	,	0.272235351	,	0.280495164	,	0.248927056	,],
[	0.346963255	,	0.092054418	,	0.23958789	,	0.344741526	,],
[	0.428609479	,	0.319097596	,	0.077390872	,	0.326465423	,],
[	0.346280022	,	0.36524177	,	0.268731685	,	0.096557227	,]],
[[	0.096590295	,	0.266060292	,	0.27442522	,	0.243004198	,],
[	0.345795581	,	0.087006962	,	0.237952026	,	0.343570579	,],
[	0.428527173	,	0.319213965	,	0.079290156	,	0.326393847	,],
[	0.345779974	,	0.365283808	,	0.268746397	,	0.093158162	,]],
[[	0.100229774	,	0.273758409	,	0.280933151	,	0.245885683	,],
[	0.342244094	,	0.088499818	,	0.23865363	,	0.341635464	,],
[	0.432162954	,	0.327553501	,	0.081790362	,	0.330382276	,],
[	0.345901979	,	0.371426212	,	0.2732319	,	0.093980392	,]],
[[	0.098013926	,	0.274883892	,	0.281931634	,	0.246563348	,],
[	0.346259099	,	0.095771755	,	0.243113673	,	0.345728877	,],
[	0.427854008	,	0.323083144	,	0.078013617	,	0.325696004	,],
[	0.349857104	,	0.375088418	,	0.27700359	,	0.100483972	,]],
[[	0.097215683	,	0.272331819	,	0.27937549	,	0.244287801	,],
[	0.342313303	,	0.087277001	,	0.238429868	,	0.341689518	,],
[	0.429498256	,	0.324980117	,	0.082208624	,	0.327292505	,],
[	0.346251946	,	0.371776208	,	0.273552016	,	0.093255438	,]],
[[	0.101791047	,	0.279140399	,	0.28610408	,	0.250799111	,],
[	0.343527168	,	0.091013895	,	0.23995548	,	0.342809402	,],
[	0.429623268	,	0.324882677	,	0.079273025	,	0.327522166	,],
[	0.347093784	,	0.372412018	,	0.27426326	,	0.095986616	,]],
[[	0.096652207	,	0.273161227	,	0.280194152	,	0.244767164	,],
[	0.344817579	,	0.089108333	,	0.240935884	,	0.344044757	,],
[	0.427605251	,	0.322927732	,	0.078618873	,	0.325365413	,],
[	0.348356519	,	0.374064127	,	0.275979546	,	0.094803627	,]],
[[	0.099563108	,	0.273504174	,	0.280639265	,	0.245557568	,],
[	0.34217155	,	0.088078863	,	0.238442911	,	0.341518322	,],
[	0.431648271	,	0.327281824	,	0.085647897	,	0.329423369	,],
[	0.345760578	,	0.371410341	,	0.273102686	,	0.093320102	,]],
[[	0.098841889	,	0.275829413	,	0.282768836	,	0.247426857	,],
[	0.345478009	,	0.094270533	,	0.24210845	,	0.344783139	,],
[	0.428254687	,	0.323432322	,	0.078221049	,	0.325988331	,],
[	0.349159875	,	0.374405266	,	0.276142638	,	0.09907784	,]],
[[	0.096978551	,	0.2724426	,	0.279549183	,	0.244265434	,],
[	0.342954264	,	0.087743085	,	0.239058922	,	0.342215188	,],
[	0.428941522	,	0.324276948	,	0.08099321	,	0.326577277	,],
[	0.346505899	,	0.372135579	,	0.273810913	,	0.093216453	,]],
[[	0.102703321	,	0.275051903	,	0.28216422	,	0.247369248	,],
[	0.343145776	,	0.08990806	,	0.239328309	,	0.342290584	,],
[	0.430544302	,	0.325749404	,	0.080157348	,	0.328405654	,],
[	0.34659772	,	0.371971381	,	0.27364861	,	0.09493673	,]],
[[	0.097259559	,	0.273853618	,	0.280896774	,	0.245451556	,],
[	0.345578813	,	0.089785116	,	0.241751078	,	0.344747769	,],
[	0.427593717	,	0.322700967	,	0.078233792	,	0.325195395	,],
[	0.349031432	,	0.374735727	,	0.276672545	,	0.095527268	,]],
[[	0.09962501	,	0.281546881	,	0.286960949	,	0.247269192	,],
[	0.337584199	,	0.088852428	,	0.239591785	,	0.338853314	,],
[	0.425908376	,	0.328073222	,	0.082345391	,	0.324789816	,],
[	0.344121039	,	0.376966687	,	0.277533804	,	0.093454775	,]],
[[	0.098044698	,	0.283150627	,	0.288285908	,	0.24841775	,],
[	0.339025458	,	0.092805576	,	0.241262652	,	0.340302929	,],
[	0.426256206	,	0.328331968	,	0.079962196	,	0.32535179	,],
[	0.346500281	,	0.379023147	,	0.279642192	,	0.097864928	,]],
[[	0.096875269	,	0.280290273	,	0.285561887	,	0.245792683	,],
[	0.339552591	,	0.089995862	,	0.241383858	,	0.340648497	,],
[	0.424467409	,	0.326461575	,	0.079350724	,	0.323296352	,],
[	0.345929232	,	0.378610556	,	0.279249461	,	0.094630947	,]],
[[	0.101942302	,	0.287412239	,	0.29263411	,	0.252628142	,],
[	0.337725386	,	0.089735033	,	0.239555542	,	0.338816054	,],
[	0.427931186	,	0.330210554	,	0.085644911	,	0.32686155	,],
[	0.344604331	,	0.377141166	,	0.277693939	,	0.094594284	,]],
[[	0.096507251	,	0.281159666	,	0.286499696	,	0.246484196	,],
[	0.341277033	,	0.096167093	,	0.24354158	,	0.342347318	,],
[	0.424670044	,	0.326568111	,	0.078644003	,	0.323552468	,],
[	0.349211077	,	0.381410369	,	0.282108615	,	0.101719408	,]],
[[	0.098523021	,	0.280994313	,	0.286370879	,	0.246574854	,],
[	0.337942757	,	0.088871691	,	0.239738485	,	0.339025474	,],
[	0.425382549	,	0.327404995	,	0.081201049	,	0.324046212	,],
[	0.344583462	,	0.377313192	,	0.277916946	,	0.093554545	,]],
[[	0.099038897	,	0.284238751	,	0.289380388	,	0.249455859	,],
[	0.33861594	,	0.091760158	,	0.240692117	,	0.339766022	,],
[	0.427270377	,	0.329333531	,	0.080797962	,	0.326296548	,],
[	0.34592952	,	0.378417544	,	0.27895339	,	0.096923837	,]],
[[	0.096413816	,	0.280293883	,	0.285527939	,	0.245663033	,],
[	0.340429801	,	0.09073073	,	0.242194972	,	0.341375429	,],
[	0.424370802	,	0.326258054	,	0.078952373	,	0.32314396	,],
[	0.346786116	,	0.379427916	,	0.280272808	,	0.095362627	,]],
[[	0.101728954	,	0.282776707	,	0.288051089	,	0.24849817	,],
[	0.337626582	,	0.089233736	,	0.239449305	,	0.338687861	,],
[	0.427016601	,	0.329160392	,	0.084127562	,	0.325815397	,],
[	0.344406195	,	0.376847859	,	0.277415004	,	0.094042982	,]],
[[	0.09700784	,	0.281918238	,	0.287105428	,	0.247116488	,],
[	0.340187082	,	0.094672769	,	0.242423841	,	0.341181648	,],
[	0.425250261	,	0.327115669	,	0.079011417	,	0.324112545	,],
[	0.348151125	,	0.380333705	,	0.280955378	,	0.100112492	,]],
[[	0.099228328	,	0.28498358	,	0.289498657	,	0.246337255	,],
[	0.330266169	,	0.088488221	,	0.237827412	,	0.332593179	,],
[	0.420627662	,	0.328917802	,	0.084190556	,	0.320927297	,],
[	0.340627823	,	0.378915146	,	0.278872287	,	0.093134604	,]],
[[	0.096406159	,	0.285571384	,	0.289875724	,	0.246442888	,],
[	0.332089345	,	0.093124453	,	0.240034768	,	0.334443822	,],
[	0.418350887	,	0.326249009	,	0.078154092	,	0.318569198	,],
[	0.344524271	,	0.382461251	,	0.282638199	,	0.099469743	,]],
[[	0.096048198	,	0.283340867	,	0.287803004	,	0.244429933	,],
[	0.331455308	,	0.088723019	,	0.23879557	,	0.33369532	,],
[	0.418460859	,	0.326383377	,	0.080096032	,	0.318565558	,],
[	0.341310067	,	0.379564116	,	0.279539479	,	0.092653401	,]],
[[	0.100159923	,	0.289701078	,	0.29391165	,	0.250419966	,],
[	0.330381073	,	0.089597579	,	0.23797312	,	0.332611511	,],
[	0.421478853	,	0.32933413	,	0.081039213	,	0.321761683	,],
[	0.341659031	,	0.379709105	,	0.279713193	,	0.094916653	,]],
[[	0.095185791	,	0.283792815	,	0.288238971	,	0.244697779	,],
[	0.334577328	,	0.096999663	,	0.242604301	,	0.33672011	,],
[	0.41756013	,	0.32545881	,	0.077886084	,	0.317673099	,],
[	0.343867266	,	0.382162846	,	0.282400855	,	0.094849078	,]],
[[	0.098245283	,	0.284380571	,	0.288893489	,	0.245542851	,],
[	0.330265343	,	0.088144709	,	0.237743821	,	0.332485499	,],
[	0.420031792	,	0.328198769	,	0.082844398	,	0.320106659	,],
[	0.34055008	,	0.378800235	,	0.278856179	,	0.092525089	,]],
[[	0.097400429	,	0.286614375	,	0.290941197	,	0.247446599	,],
[	0.331383993	,	0.091795093	,	0.23920036	,	0.333668082	,],
[	0.419157571	,	0.32700453	,	0.078879863	,	0.319291446	,],
[	0.343741194	,	0.38174798	,	0.281838569	,	0.0981167	,]],
[[	0.095420056	,	0.283187399	,	0.287551871	,	0.244152453	,],
[	0.332381716	,	0.089416448	,	0.239686421	,	0.334435154	,],
[	0.418003461	,	0.32589541	,	0.079109191	,	0.318066963	,],
[	0.341917208	,	0.380049895	,	0.280175321	,	0.093132505	,]],
[[	0.101517004	,	0.291050612	,	0.295411854	,	0.251832712	,],
[	0.330057376	,	0.088879688	,	0.237550863	,	0.332234865	,],
[	0.42227316	,	0.330626809	,	0.086586562	,	0.322405255	,],
[	0.341148073	,	0.379218313	,	0.279193463	,	0.093971498	,]],
[[	0.095701294	,	0.284695846	,	0.289068459	,	0.245380445	,],
[	0.333877753	,	0.095663981	,	0.241873174	,	0.335972067	,],
[	0.417871709	,	0.325725458	,	0.07805551	,	0.317886034	,],
[	0.344972912	,	0.383303919	,	0.283575286	,	0.095900289	,]],
[[	0.098735852	,	0.29003677	,	0.293301052	,	0.246069173	,],
[	0.322684182	,	0.088159376	,	0.23593483	,	0.326550645	,],
[	0.415142895	,	0.330204259	,	0.085095848	,	0.316953259	,],
[	0.337546712	,	0.382030003	,	0.281813327	,	0.09284539	,]],
[[	0.09540315	,	0.290076634	,	0.29321276	,	0.245752617	,],
[	0.325122088	,	0.093462142	,	0.238791188	,	0.328952173	,],
[	0.411811831	,	0.326332883	,	0.077558764	,	0.313463012	,],
[	0.341699193	,	0.385805047	,	0.285876665	,	0.09965529	,]],
[[	0.095326441	,	0.28828824	,	0.291352908	,	0.244050416	,],
[	0.324002712	,	0.088147929	,	0.236821457	,	0.327494598	,],
[	0.412706653	,	0.327400601	,	0.080454601	,	0.314312984	,],
[	0.337542312	,	0.382048108	,	0.281794486	,	0.091904656	,]],
[[	0.098864746	,	0.294094842	,	0.297192056	,	0.249667642	,],
[	0.323147553	,	0.089520882	,	0.236319736	,	0.326802991	,],
[	0.414493292	,	0.328910103	,	0.079546204	,	0.316192968	,],
[	0.33868445	,	0.382940704	,	0.28272154	,	0.094966857	,]],
[[	0.09431946	,	0.288643911	,	0.291721114	,	0.244169829	,],
[	0.327337884	,	0.091038969	,	0.24036056	,	0.330882123	,],
[	0.411504491	,	0.325960031	,	0.077789034	,	0.313029125	,],
[	0.340074692	,	0.384632962	,	0.284578325	,	0.093832033	,]],
[[	0.097635323	,	0.289416884	,	0.292681114	,	0.245314454	,],
[	0.3230027	,	0.087829508	,	0.235985284	,	0.326598808	,],
[	0.41456982	,	0.329373871	,	0.083674071	,	0.316081487	,],
[	0.337231667	,	0.381739247	,	0.28153197	,	0.092220836	,]],
[[	0.096190258	,	0.291110816	,	0.294142909	,	0.246667219	,],
[	0.324613353	,	0.09230965	,	0.238235787	,	0.32837067	,],
[	0.412654924	,	0.327022618	,	0.077958244	,	0.314255257	,],
[	0.340607658	,	0.384845995	,	0.284761392	,	0.097946783	,]],
[[	0.094840752	,	0.288221718	,	0.291249998	,	0.24397806	,],
[	0.324848796	,	0.0888275	,	0.237799302	,	0.328417771	,],
[	0.412359203	,	0.326878387	,	0.07968582	,	0.31390043	,],
[	0.338335772	,	0.382720597	,	0.282552363	,	0.092248982	,]],
[[	0.10028477	,	0.295581758	,	0.298589556	,	0.251077465	,],
[	0.322889477	,	0.08884405	,	0.236050115	,	0.326545869	,],
[	0.41536127	,	0.329910106	,	0.08053248	,	0.317144004	,],
[	0.338295863	,	0.382505178	,	0.282279049	,	0.093983269	,]],
[[	0.094694902	,	0.28925368	,	0.292332783	,	0.244745513	,],
[	0.326885095	,	0.096094291	,	0.240561412	,	0.330482997	,],
[	0.411936138	,	0.326311025	,	0.077979097	,	0.313396379	,],
[	0.341106748	,	0.385651055	,	0.285719412	,	0.094781411	,]],
[[	0.102161326	,	0.306900185	,	0.308583566	,	0.255075391	,],
[	0.329833861	,	0.091702729	,	0.244980362	,	0.334172827	,],
[	0.422447042	,	0.341325733	,	0.0848566	,	0.321180194	,],
[	0.346491875	,	0.399264957	,	0.294328348	,	0.094475831	,]],
[[	0.099201719	,	0.307275995	,	0.308810065	,	0.255016138	,],
[	0.330035681	,	0.094094233	,	0.245407933	,	0.334341119	,],
[	0.422581195	,	0.341245408	,	0.082068424	,	0.321482334	,],
[	0.348454389	,	0.401000059	,	0.296158804	,	0.098449713	,]],
[[	0.098984872	,	0.305265032	,	0.306853595	,	0.253211041	,],
[	0.332969082	,	0.09408797	,	0.247939281	,	0.33708585	,],
[	0.420989681	,	0.339643222	,	0.08171686	,	0.319650493	,],
[	0.348018196	,	0.400752072	,	0.295989776	,	0.095348654	,]],
[[	0.102776474	,	0.311216398	,	0.312769038	,	0.258886125	,],
[	0.32920014	,	0.091841567	,	0.24432457	,	0.333468998	,],
[	0.42476397	,	0.34382373	,	0.088469603	,	0.323511098	,],
[	0.346804184	,	0.399445422	,	0.294448372	,	0.095412066	,]],
[[	0.097801135	,	0.305511358	,	0.307128773	,	0.253223149	,],
[	0.331805644	,	0.097076388	,	0.247374776	,	0.336100622	,],
[	0.421102199	,	0.339619341	,	0.080748941	,	0.319845354	,],
[	0.351201797	,	0.403448293	,	0.298605949	,	0.102401042	,]],
[[	0.100992442	,	0.30642006	,	0.308101981	,	0.254397753	,],
[	0.330537913	,	0.092179947	,	0.245666253	,	0.334678107	,],
[	0.422065095	,	0.340712105	,	0.083858636	,	0.32057697	,],
[	0.346711981	,	0.399493746	,	0.294556749	,	0.094376044	,]],
[[	0.100141541	,	0.308181911	,	0.309671138	,	0.255890299	,],
[	0.329681551	,	0.093176598	,	0.245036499	,	0.333978824	,],
[	0.423426186	,	0.341972283	,	0.082696356	,	0.322213635	,],
[	0.347759938	,	0.400430039	,	0.29544299	,	0.097251023	,]],
[[	0.098541252	,	0.305252694	,	0.306771595	,	0.253086455	,],
[	0.333953938	,	0.095153082	,	0.249010447	,	0.337949382	,],
[	0.420696704	,	0.339276329	,	0.081052049	,	0.319288863	,],
[	0.349239127	,	0.401938932	,	0.297139718	,	0.096577465	,]],
[[	0.104297908	,	0.312616293	,	0.314175022	,	0.260290011	,],
[	0.329474811	,	0.091669871	,	0.244466227	,	0.333606809	,],
[	0.423922234	,	0.34280367	,	0.08686099	,	0.322534076	,],
[	0.346382186	,	0.399104648	,	0.29407188	,	0.094604882	,]],
[[	0.098435186	,	0.306287063	,	0.307829938	,	0.253961911	,],
[	0.331001281	,	0.095846647	,	0.246530064	,	0.33528957	,],
[	0.421604046	,	0.340074162	,	0.081124824	,	0.320238982	,],
[	0.349935898	,	0.402349016	,	0.297515128	,	0.100703528	,]],
[[	0.101408666	,	0.31290315	,	0.313018145	,	0.254792704	,],
[	0.324492471	,	0.091941129	,	0.24482111	,	0.329978808	,],
[	0.417347905	,	0.341930149	,	0.083484167	,	0.317332269	,],
[	0.343996001	,	0.403307487	,	0.29697506	,	0.094735511	,]],
[[	0.098869463	,	0.313495782	,	0.313412963	,	0.255130327	,],
[	0.322864723	,	0.092152643	,	0.2433656	,	0.328335273	,],
[	0.419467155	,	0.343936646	,	0.083315526	,	0.31985991	,],
[	0.345336427	,	0.404378582	,	0.298073384	,	0.098038278	,]],
[[	0.098106415	,	0.311024575	,	0.311117148	,	0.252757422	,],
[	0.327067128	,	0.099593168	,	0.248192653	,	0.332647273	,],
[	0.416169042	,	0.340603978	,	0.081085346	,	0.316262334	,],
[	0.346095885	,	0.405163846	,	0.299027904	,	0.096312076	,]],
[[	0.102694881	,	0.31763746	,	0.317653344	,	0.259197779	,],
[	0.323118791	,	0.09111403	,	0.243359432	,	0.328439063	,],
[	0.418755355	,	0.343604527	,	0.086238937	,	0.318921011	,],
[	0.344031591	,	0.403113617	,	0.296760852	,	0.095370479	,]],
[[	0.097709481	,	0.312024693	,	0.312021667	,	0.253557661	,],
[	0.324308394	,	0.094814515	,	0.244925605	,	0.329729389	,],
[	0.417259887	,	0.341662157	,	0.081353448	,	0.317427546	,],
[	0.347776763	,	0.406558385	,	0.300250844	,	0.101442787	,]],
[[	0.10025909	,	0.312293842	,	0.31239459	,	0.25402027	,],
[	0.325272263	,	0.09258629	,	0.245582715	,	0.330509658	,],
[	0.416819011	,	0.34136777	,	0.082447475	,	0.316909017	,],
[	0.344241326	,	0.403530153	,	0.297316469	,	0.09475473	,]],
[[	0.100005807	,	0.314647474	,	0.314545411	,	0.25619746	,],
[	0.322876296	,	0.091607922	,	0.243166761	,	0.328220532	,],
[	0.420526593	,	0.345059478	,	0.084432642	,	0.320993565	,],
[	0.344883141	,	0.403932691	,	0.297675221	,	0.097101194	,]],
[[	0.097876315	,	0.311318647	,	0.311212065	,	0.252874243	,],
[	0.326251173	,	0.098115165	,	0.247017675	,	0.331587002	,],
[	0.41643084	,	0.340857933	,	0.080954336	,	0.316519457	,],
[	0.347031414	,	0.406089112	,	0.299970575	,	0.097132488	,]],
[[	0.103549664	,	0.314085639	,	0.314172274	,	0.255973238	,],
[	0.32357851	,	0.09135318	,	0.243729652	,	0.32886206	,],
[	0.418182818	,	0.342863149	,	0.084905189	,	0.318103701	,],
[	0.343849876	,	0.40291427	,	0.296571556	,	0.095018648	,]],
[[	0.098140125	,	0.312484038	,	0.312440989	,	0.254019821	,],
[	0.323750783	,	0.093714692	,	0.244199174	,	0.32899256	,],
[	0.417994719	,	0.342331437	,	0.081961862	,	0.318169421	,],
[	0.346859838	,	0.405630621	,	0.299341762	,	0.099996819	,]],
[[	0.099369329	,	0.316085179	,	0.31518743	,	0.253705314	,],
[	0.31810484	,	0.0930458	,	0.244313474	,	0.325744537	,],
[	0.41077172	,	0.34273548	,	0.08192305	,	0.312738563	,],
[	0.341079821	,	0.407453773	,	0.300408721	,	0.094343306	,]],
[[	0.097574292	,	0.317283172	,	0.316218076	,	0.25458633	,],
[	0.315762648	,	0.092317201	,	0.242269203	,	0.323481222	,],
[	0.414865571	,	0.346789149	,	0.084417824	,	0.317272177	,],
[	0.34274747	,	0.409153379	,	0.301985994	,	0.098097606	,]],
[[	0.096697859	,	0.314773417	,	0.313927035	,	0.252293418	,],
[	0.318945366	,	0.098592189	,	0.245923811	,	0.32671717	,],
[	0.410652265	,	0.342478538	,	0.080733799	,	0.312740314	,],
[	0.342729336	,	0.409145602	,	0.302101478	,	0.095478843	,]],
[[	0.101576174	,	0.321501878	,	0.320462286	,	0.258758514	,],
[	0.31650341	,	0.091795283	,	0.242604054	,	0.324089435	,],
[	0.412109387	,	0.344325445	,	0.084503313	,	0.314148608	,],
[	0.341164299	,	0.40760173	,	0.300323114	,	0.0950891	,]],
[[	0.096132903	,	0.315427482	,	0.314490303	,	0.252738215	,],
[	0.316443318	,	0.094174899	,	0.242902821	,	0.324122815	,],
[	0.412238645	,	0.344020995	,	0.081751137	,	0.314419948	,],
[	0.345161605	,	0.411180843	,	0.304113041	,	0.101487493	,]],
[[	0.098118661	,	0.315283358	,	0.314489875	,	0.252767959	,],
[	0.319013976	,	0.093836502	,	0.245256147	,	0.326648451	,],
[	0.410627208	,	0.34254964	,	0.081402436	,	0.312544925	,],
[	0.341282562	,	0.407877	,	0.300623359	,	0.094347165	,]],
[[	0.098759708	,	0.318491711	,	0.31737686	,	0.255744514	,],
[	0.315857093	,	0.091918638	,	0.242190202	,	0.323490616	,],
[	0.414246557	,	0.346604241	,	0.08797557	,	0.316295912	,],
[	0.342302198	,	0.408656997	,	0.301444547	,	0.097047991	,]],
[[	0.09613343	,	0.314589531	,	0.313628006	,	0.251975884	,],
[	0.318084613	,	0.097157949	,	0.24494956	,	0.325823334	,],
[	0.411231414	,	0.342983197	,	0.081051266	,	0.313199742	,],
[	0.343950323	,	0.41030957	,	0.303150774	,	0.09651909	,]],
[[	0.101548336	,	0.317198349	,	0.316259702	,	0.254878993	,],
[	0.317229427	,	0.092373788	,	0.24335664	,	0.324752613	,],
[	0.411683361	,	0.343676727	,	0.083364678	,	0.313593298	,],
[	0.341320502	,	0.407676002	,	0.300337791	,	0.094776738	,]],
[[	0.096565716	,	0.316091631	,	0.315179996	,	0.253375519	,],
[	0.316374291	,	0.093574044	,	0.242801954	,	0.323912247	,],
[	0.413614466	,	0.3454392	,	0.082991302	,	0.315780385	,],
[	0.344379661	,	0.410495991	,	0.303294242	,	0.100130967	,]],
[[	0.099743528	,	0.323142699	,	0.320650464	,	0.254875013	,],
[	0.316278912	,	0.095573405	,	0.246806735	,	0.324195538	,],
[	0.408711504	,	0.346054656	,	0.082871039	,	0.310267817	,],
[	0.340968986	,	0.412832751	,	0.304706187	,	0.09451774	,]],
[[	0.097320825	,	0.323718416	,	0.320951145	,	0.255133384	,],
[	0.312259638	,	0.092794558	,	0.242964505	,	0.32025253	,],
[	0.41223282	,	0.350299211	,	0.089672906	,	0.31409023	,],
[	0.342122107	,	0.413851311	,	0.305602901	,	0.097501242	,]],
[[	0.096607428	,	0.321484638	,	0.318766394	,	0.25298278	,],
[	0.314316696	,	0.097593233	,	0.245426209	,	0.322378735	,],
[	0.408633354	,	0.345883489	,	0.08169118	,	0.310216482	,],
[	0.343265833	,	0.414920001	,	0.30686574	,	0.096289183	,]],
[[	0.10129585	,	0.328010888	,	0.325308762	,	0.259422676	,],
[	0.314001016	,	0.09355487	,	0.244508777	,	0.32188261	,],
[	0.409568257	,	0.347205561	,	0.08502429	,	0.311219412	,],
[	0.340893223	,	0.41259408	,	0.304389673	,	0.095015969	,]],
[[	0.095995357	,	0.322077035	,	0.319506699	,	0.253450554	,],
[	0.312157615	,	0.093678795	,	0.242915511	,	0.320187581	,],
[	0.410883219	,	0.348169643	,	0.08349253	,	0.312736653	,],
[	0.344472247	,	0.415859519	,	0.307753773	,	0.100895212	,]],
[[	0.098390123	,	0.322225947	,	0.319671435	,	0.253766429	,],
[	0.316403565	,	0.10114605	,	0.247690142	,	0.324401078	,],
[	0.408317657	,	0.345700788	,	0.082183294	,	0.309885516	,],
[	0.341614043	,	0.413425888	,	0.305272433	,	0.094803968	,]],
[[	0.098499161	,	0.325034908	,	0.322202418	,	0.256375765	,],
[	0.312552306	,	0.092639909	,	0.243064024	,	0.320388025	,],
[	0.411731821	,	0.34951249	,	0.08834241	,	0.313357804	,],
[	0.341693751	,	0.41351193	,	0.305267304	,	0.096561326	,]],
[[	0.096386046	,	0.321697489	,	0.318960807	,	0.252988818	,],
[	0.313519004	,	0.096109145	,	0.244450254	,	0.321425161	,],
[	0.409211903	,	0.346466941	,	0.082171266	,	0.310838215	,],
[	0.344235612	,	0.415853499	,	0.307951196	,	0.097186402	,]],
[[	0.101941132	,	0.324410023	,	0.321693872	,	0.256022842	,],
[	0.31489517	,	0.094227426	,	0.245307719	,	0.322699199	,],
[	0.409118361	,	0.346588087	,	0.083899744	,	0.310656028	,],
[	0.341004968	,	0.412697305	,	0.304434269	,	0.094638607	,]],
[[	0.096661726	,	0.322840256	,	0.320044383	,	0.254160251	,],
[	0.312422947	,	0.093409209	,	0.243044922	,	0.320272937	,],
[	0.411776809	,	0.349037515	,	0.084418573	,	0.313616121	,],
[	0.343631247	,	0.415169018	,	0.3070046	,	0.099561133	,]],
[[	0.099467017	,	0.33089344	,	0.326664135	,	0.255879492	,],
[	0.311122471	,	0.100848961	,	0.247815089	,	0.320266528	,],
[	0.406151265	,	0.350068779	,	0.082447363	,	0.307852198	,],
[	0.341492459	,	0.420527215	,	0.310728339	,	0.094945454	,]],
[[	0.09817209	,	0.332250175	,	0.327938366	,	0.257125539	,],
[	0.308141008	,	0.093577213	,	0.244129941	,	0.317302354	,],
[	0.408643099	,	0.353016476	,	0.087557579	,	0.310453482	,],
[	0.340845878	,	0.419820989	,	0.30988937	,	0.095695693	,]],
[[	0.096839286	,	0.329580895	,	0.325298034	,	0.254590726	,],
[	0.308280647	,	0.09611434	,	0.244709263	,	0.317420585	,],
[	0.407491461	,	0.351268477	,	0.082857526	,	0.309262901	,],
[	0.344592525	,	0.423362388	,	0.313773131	,	0.097774905	,]],
[[	0.102679409	,	0.336885883	,	0.332529494	,	0.261754031	,],
[	0.310816057	,	0.095467838	,	0.246638462	,	0.31986604	,],
[	0.406516294	,	0.350476078	,	0.083635215	,	0.30820285	,],
[	0.34047224	,	0.419409807	,	0.30943695	,	0.094224632	,]],
[[	0.09659549	,	0.330552117	,	0.326288171	,	0.255250593	,],
[	0.307500886	,	0.093639028	,	0.243507552	,	0.316597373	,],
[	0.410386804	,	0.354079864	,	0.085683389	,	0.312484608	,],
[	0.342369003	,	0.421159837	,	0.311154037	,	0.098116558	,]],
[[	0.098559699	,	0.330379609	,	0.326113389	,	0.255210151	,],
[	0.309944048	,	0.099223722	,	0.24652338	,	0.319076621	,],
[	0.40642748	,	0.350160825	,	0.082294693	,	0.308056353	,],
[	0.342238472	,	0.42121918	,	0.311441402	,	0.09550489	,]],
[[	0.099482593	,	0.333700036	,	0.329246655	,	0.258459337	,],
[	0.3085888	,	0.093803362	,	0.244577315	,	0.317622685	,],
[	0.407931426	,	0.352106425	,	0.086255838	,	0.309655606	,],
[	0.340605272	,	0.419648604	,	0.309678344	,	0.09509134	,]],
[[	0.096619873	,	0.329876124	,	0.325491333	,	0.254617461	,],
[	0.307882777	,	0.095190724	,	0.244193837	,	0.317019895	,],
[	0.408001626	,	0.351756246	,	0.08334426	,	0.30979075	,],
[	0.344605081	,	0.423331693	,	0.313482791	,	0.101517779	,]],
[[	0.101575203	,	0.331960286	,	0.327648435	,	0.257013943	,],
[	0.312071168	,	0.096696598	,	0.247937714	,	0.321019091	,],
[	0.406372279	,	0.350319979	,	0.082985413	,	0.308023369	,],
[	0.340851747	,	0.419816612	,	0.309815007	,	0.094331976	,]],
[[	0.09721065	,	0.331222247	,	0.326837359	,	0.255985177	,],
[	0.30788334	,	0.093604336	,	0.24376924	,	0.316850482	,],
[	0.409939482	,	0.354329003	,	0.089527136	,	0.31166774	,],
[	0.341768961	,	0.420625284	,	0.310662427	,	0.097063729	,]],
[[	0.098282547	,	0.333511794	,	0.329566821	,	0.254944358	,],
[	0.302358764	,	0.098004404	,	0.245969622	,	0.313587879	,],
[	0.40037038	,	0.350380746	,	0.082822606	,	0.304368208	,],
[	0.33753411	,	0.421846598	,	0.313026987	,	0.094298332	,]],
[[	0.098035901	,	0.335767816	,	0.331731298	,	0.257127728	,],
[	0.302398124	,	0.094320425	,	0.24548731	,	0.313700848	,],
[	0.400445496	,	0.350732766	,	0.084659932	,	0.304302728	,],
[	0.337101237	,	0.421464553	,	0.312358707	,	0.095201048	,]],
[[	0.095878721	,	0.33241424	,	0.32839724	,	0.253778124	,],
[	0.300729104	,	0.094618992	,	0.244081264	,	0.312094319	,],
[	0.403121438	,	0.353046298	,	0.085121919	,	0.307175713	,],
[	0.34087651	,	0.425015711	,	0.316275954	,	0.09740354	,]],
[[	0.101835092	,	0.335530617	,	0.331537705	,	0.257101628	,],
[	0.304628891	,	0.101674058	,	0.248535948	,	0.31596488	,],
[	0.399765309	,	0.349807809	,	0.08264375	,	0.303561587	,],
[	0.336475442	,	0.420745091	,	0.311680394	,	0.093587606	,]],
[[	0.095955784	,	0.333495708	,	0.32945476	,	0.254751854	,],
[	0.300949931	,	0.093382118	,	0.243969134	,	0.312165011	,],
[	0.40210546	,	0.352609561	,	0.08763072	,	0.305952983	,],
[	0.338962165	,	0.423196363	,	0.314017315	,	0.098047623	,]],
[[	0.09744404	,	0.332973995	,	0.329121668	,	0.254351975	,],
[	0.301924263	,	0.096980402	,	0.24542036	,	0.313056705	,],
[	0.401094728	,	0.351109308	,	0.08330574	,	0.304999848	,],
[	0.338313694	,	0.422635006	,	0.313677407	,	0.094786879	,]],
[[	0.09884062	,	0.336670075	,	0.332437453	,	0.257861344	,],
[	0.303528842	,	0.095032599	,	0.246403609	,	0.314527535	,],
[	0.400146241	,	0.350279842	,	0.083839349	,	0.303832765	,],
[	0.33672983	,	0.421179415	,	0.31196492	,	0.094497583	,]],
[[	0.095733824	,	0.332624926	,	0.328448249	,	0.253800978	,],
[	0.300683128	,	0.094132723	,	0.244007462	,	0.311966269	,],
[	0.404248073	,	0.355018855	,	0.091146349	,	0.308132735	,],
[	0.340914852	,	0.425046064	,	0.316078633	,	0.101280166	,]],
[[	0.100490149	,	0.334818432	,	0.330809201	,	0.256277655	,],
[	0.303737111	,	0.100215248	,	0.247588173	,	0.315037787	,],
[	0.400091865	,	0.350099607	,	0.08265723	,	0.303829351	,],
[	0.336979137	,	0.421310941	,	0.3122658	,	0.093754433	,]],
[[	0.096671371	,	0.334330103	,	0.330356625	,	0.255588804	,],
[	0.301550375	,	0.093728251	,	0.244533868	,	0.31268136	,],
[	0.401443075	,	0.351782524	,	0.086287658	,	0.305135404	,],
[	0.338168605	,	0.422444066	,	0.313285722	,	0.096837223	,]],
[[	0.097557256	,	0.338765541	,	0.333758266	,	0.255184339	,],
[	0.29510477	,	0.096311194	,	0.244025185	,	0.308171833	,],
[	0.395072957	,	0.351441463	,	0.082316606	,	0.300184703	,],
[	0.336893302	,	0.427038844	,	0.317197435	,	0.095461654	,]],
[[	0.09723117	,	0.341095009	,	0.335962292	,	0.25734375	,],
[	0.297432731	,	0.095559433	,	0.245803146	,	0.310356718	,],
[	0.394792824	,	0.351352026	,	0.083832975	,	0.299783142	,],
[	0.334869182	,	0.425064874	,	0.315009597	,	0.094442708	,]],
[[	0.09533402	,	0.338004433	,	0.332807458	,	0.254239152	,],
[	0.294334983	,	0.093989972	,	0.242960599	,	0.307326938	,],
[	0.398150694	,	0.354491649	,	0.085028806	,	0.303475751	,],
[	0.338573759	,	0.428652368	,	0.31859321	,	0.100643991	,]],
[[	0.10111482	,	0.341038535	,	0.335961704	,	0.257466627	,],
[	0.296817312	,	0.099450693	,	0.245872864	,	0.309729809	,],
[	0.394746103	,	0.351084739	,	0.082398902	,	0.299682146	,],
[	0.335065203	,	0.425258074	,	0.315229333	,	0.093915714	,]],
[[	0.095332895	,	0.338994152	,	0.333898571	,	0.255091408	,],
[	0.295453654	,	0.093851584	,	0.243728357	,	0.308218437	,],
[	0.396639941	,	0.353356542	,	0.086734676	,	0.30156933	,],
[	0.33571962	,	0.425882671	,	0.315721549	,	0.096238615	,]],
[[	0.096807978	,	0.338650667	,	0.333429947	,	0.254730984	,],
[	0.294500296	,	0.095232219	,	0.243226744	,	0.307397537	,],
[	0.396056843	,	0.352368665	,	0.08302509	,	0.301212163	,],
[	0.338084268	,	0.428121446	,	0.318320474	,	0.09644045	,]],
[[	0.098464224	,	0.34232922	,	0.337099175	,	0.258605803	,],
[	0.298675347	,	0.096661799	,	0.246927284	,	0.311409913	,],
[	0.394699191	,	0.351217372	,	0.08317905	,	0.29967819	,],
[	0.334711908	,	0.4250066	,	0.314975412	,	0.093987179	,]],
[[	0.095010158	,	0.338049358	,	0.332920328	,	0.254123795	,],
[	0.294397284	,	0.093600563	,	0.242913581	,	0.307341299	,],
[	0.398717315	,	0.355658772	,	0.090262116	,	0.303647099	,],
[	0.337667698	,	0.427649078	,	0.317556416	,	0.099111554	,]],
[[	0.099450113	,	0.339993075	,	0.334914083	,	0.256358252	,],
[	0.295860743	,	0.097984332	,	0.244832005	,	0.308821598	,],
[	0.394799432	,	0.351075842	,	0.082059755	,	0.299773145	,],
[	0.336154991	,	0.42622948	,	0.316185652	,	0.094559617	,]],
[[	0.096298516	,	0.339949021	,	0.334854329	,	0.256163316	,],
[	0.296300916	,	0.09465094	,	0.244610584	,	0.309058466	,],
[	0.395690426	,	0.352333784	,	0.085321941	,	0.300563708	,],
[	0.335404275	,	0.425593965	,	0.315386023	,	0.095348866	,]],
[[	0.097779321	,	0.345827574	,	0.339303771	,	0.25585047	,],
[	0.289929661	,	0.096192178	,	0.243839653	,	0.303655749	,],
[	0.394051228	,	0.356377885	,	0.08398764	,	0.299095547	,],
[	0.33633235	,	0.432740917	,	0.321758745	,	0.095428829	,]],
[[	0.097503135	,	0.347904193	,	0.341295979	,	0.257938323	,],
[	0.293582881	,	0.096833673	,	0.246960915	,	0.307078902	,],
[	0.392702222	,	0.355294173	,	0.084120713	,	0.297603313	,],
[	0.334345183	,	0.430899865	,	0.319565088	,	0.094432554	,]],
[[	0.095266009	,	0.344626514	,	0.337971251	,	0.254543826	,],
[	0.289627644	,	0.094317349	,	0.243280435	,	0.303333614	,],
[	0.396393233	,	0.359552841	,	0.090998294	,	0.30135866	,],
[	0.33813768	,	0.434531643	,	0.323380071	,	0.100821925	,]],
[[	0.101093059	,	0.347690051	,	0.341084459	,	0.257852415	,],
[	0.291610567	,	0.099060962	,	0.245773169	,	0.305244524	,],
[	0.392583914	,	0.355000739	,	0.082933486	,	0.297547834	,],
[	0.334554367	,	0.431021466	,	0.3197527	,	0.093971801	,]],
[[	0.095604625	,	0.345936158	,	0.339232324	,	0.255782557	,],
[	0.291446144	,	0.095036331	,	0.244731456	,	0.304915732	,],
[	0.393730376	,	0.356551236	,	0.086364204	,	0.298641141	,],
[	0.335384191	,	0.431791107	,	0.320446756	,	0.096381053	,]],
[[	0.096626292	,	0.345129244	,	0.33874068	,	0.255219268	,],
[	0.289710667	,	0.095439014	,	0.243391019	,	0.303335855	,],
[	0.394874829	,	0.357131326	,	0.084607264	,	0.2998698	,],
[	0.337455611	,	0.433740126	,	0.322725437	,	0.096279442	,]],
[[	0.09856787	,	0.348949775	,	0.342210673	,	0.258941895	,],
[	0.293866688	,	0.102847248	,	0.248232568	,	0.307484406	,],
[	0.392429553	,	0.354948043	,	0.083415739	,	0.297170979	,],
[	0.334183484	,	0.430681472	,	0.319387787	,	0.094017935	,]],
[[	0.095226994	,	0.344823799	,	0.338191904	,	0.254762034	,],
[	0.289825081	,	0.094061221	,	0.243352293	,	0.303418521	,],
[	0.395538298	,	0.358582801	,	0.089438977	,	0.300493357	,],
[	0.337269401	,	0.433556727	,	0.322313647	,	0.099291903	,]],
[[	0.099664794	,	0.346833536	,	0.340253077	,	0.25698849	,],
[	0.29095659	,	0.097849565	,	0.244945741	,	0.304631927	,],
[	0.393081242	,	0.355377294	,	0.083139362	,	0.298022934	,],
[	0.33540586	,	0.431710015	,	0.320480271	,	0.094435866	,]],
[[	0.096524618	,	0.346879227	,	0.340231342	,	0.256712915	,],
[	0.292436195	,	0.095728613	,	0.245672424	,	0.305836477	,],
[	0.393150209	,	0.355880958	,	0.085254838	,	0.298012533	,],
[	0.335022263	,	0.431411027	,	0.320106407	,	0.095596529	,]],
[[	0.097401335	,	0.352841546	,	0.345347945	,	0.256922118	,],
[	0.283948183	,	0.096023406	,	0.244275593	,	0.299070393	,],
[	0.390511821	,	0.360069795	,	0.084416001	,	0.296532958	,],
[	0.335423813	,	0.439328783	,	0.326904513	,	0.095720149	,]],
[[	0.097465997	,	0.355198573	,	0.347520612	,	0.259292782	,],
[	0.28781656	,	0.103036191	,	0.248615698	,	0.302942181	,],
[	0.388670981	,	0.358519901	,	0.084123952	,	0.294580546	,],
[	0.332613188	,	0.436844547	,	0.324104991	,	0.094037172	,]],
[[	0.095402637	,	0.352040275	,	0.344423311	,	0.255999293	,],
[	0.284345958	,	0.095044304	,	0.244413717	,	0.299501618	,],
[	0.392376815	,	0.362634091	,	0.090827507	,	0.298356478	,],
[	0.336021366	,	0.440025056	,	0.327363444	,	0.099722923	,]],
[[	0.100680219	,	0.354665679	,	0.347090339	,	0.258809037	,],
[	0.284988512	,	0.09833421	,	0.245544553	,	0.300192887	,],
[	0.389096593	,	0.358822207	,	0.083393726	,	0.295003381	,],
[	0.333571705	,	0.437582806	,	0.324940113	,	0.094088628	,]],
[[	0.095583684	,	0.353152956	,	0.345598289	,	0.257066851	,],
[	0.286889185	,	0.09660783	,	0.246586702	,	0.301763959	,],
[	0.390069394	,	0.360074954	,	0.086488843	,	0.295849634	,],
[	0.333674041	,	0.437680464	,	0.324796893	,	0.095686157	,]],
[[	0.096662884	,	0.352512773	,	0.344969712	,	0.256510989	,],
[	0.283998301	,	0.095558211	,	0.244096995	,	0.299079317	,],
[	0.391732812	,	0.361222699	,	0.085402671	,	0.297709786	,],
[	0.336513034	,	0.440484478	,	0.328070851	,	0.096794698	,]],
[[	0.098867225	,	0.356520474	,	0.348822286	,	0.260557514	,],
[	0.287071975	,	0.101723005	,	0.247678427	,	0.302119317	,],
[	0.3888057	,	0.358518559	,	0.083693219	,	0.294458917	,],
[	0.332879882	,	0.437020399	,	0.324304549	,	0.093921556	,]],
[[	0.095115432	,	0.352167502	,	0.344480506	,	0.256056875	,],
[	0.284690462	,	0.095119453	,	0.244729266	,	0.299790562	,],
[	0.391576887	,	0.36183055	,	0.089385309	,	0.297442843	,],
[	0.335334391	,	0.439382577	,	0.326724352	,	0.098452406	,]],
[[	0.099368494	,	0.353914069	,	0.346352664	,	0.257934859	,],
[	0.284619831	,	0.097346864	,	0.24497349	,	0.299762901	,],
[	0.389715686	,	0.359283511	,	0.083796644	,	0.295596081	,],
[	0.334287055	,	0.438250569	,	0.325695937	,	0.094606334	,]],
[[	0.096555917	,	0.354169775	,	0.346502347	,	0.258038903	,],
[	0.288022547	,	0.097733076	,	0.247737367	,	0.302917936	,],
[	0.389522744	,	0.359395328	,	0.085445695	,	0.295262512	,],
[	0.333196146	,	0.437336173	,	0.324468069	,	0.094934143	,]],
[[	0.097798292	,	0.358908849	,	0.349569148	,	0.258286877	,],
[	0.278056032	,	0.096157899	,	0.243209265	,	0.294796508	,],
[	0.385879335	,	0.362150576	,	0.083599751	,	0.293164148	,],
[	0.333596082	,	0.443365663	,	0.32958808	,	0.095531654	,]],
[[	0.095931957	,	0.359796095	,	0.350285393	,	0.259113693	,],
[	0.281524757	,	0.102857547	,	0.247288948	,	0.298262685	,],
[	0.384081931	,	0.360713894	,	0.083423711	,	0.291206319	,],
[	0.331476921	,	0.441444491	,	0.327332923	,	0.09439599	,]],
[[	0.094872116	,	0.35736825	,	0.347844816	,	0.2565302	,],
[	0.278660778	,	0.095218543	,	0.243532555	,	0.29526101	,],
[	0.38789143	,	0.364911725	,	0.090190554	,	0.295008066	,],
[	0.335027207	,	0.444808015	,	0.330885783	,	0.10054982	,]],
[[	0.099954097	,	0.363920197	,	0.354408705	,	0.263298703	,],
[	0.279240142	,	0.098415364	,	0.24455743	,	0.295951115	,],
[	0.384474105	,	0.36080012	,	0.082597345	,	0.291580814	,],
[	0.331762235	,	0.441614147	,	0.327671666	,	0.093931965	,]],
[[	0.094275175	,	0.357978592	,	0.348515858	,	0.257088307	,],
[	0.281407342	,	0.097151377	,	0.245953964	,	0.297796196	,],
[	0.385045835	,	0.361775744	,	0.085353263	,	0.29198219	,],
[	0.332326079	,	0.442160251	,	0.328122578	,	0.096229301	,]],
[[	0.096350547	,	0.358027898	,	0.348629799	,	0.257250725	,],
[	0.278172239	,	0.095760936	,	0.243213798	,	0.294822747	,],
[	0.387066436	,	0.363235898	,	0.084696713	,	0.294224609	,],
[	0.334595257	,	0.444337176	,	0.330590122	,	0.096218348	,]],
[[	0.097052322	,	0.360960148	,	0.351436107	,	0.260221009	,],
[	0.281152115	,	0.101782813	,	0.246665626	,	0.297681052	,],
[	0.384240848	,	0.360713671	,	0.082984249	,	0.291215095	,],
[	0.33142002	,	0.441381449	,	0.327363162	,	0.094022765	,]],
[[	0.094603912	,	0.357629147	,	0.348160914	,	0.256810104	,],
[	0.279432042	,	0.095730452	,	0.24414397	,	0.295837574	,],
[	0.38695962	,	0.363938606	,	0.088532482	,	0.294046355	,],
[	0.333988669	,	0.443814013	,	0.329810119	,	0.098797361	,]],
[[	0.099664917	,	0.359893493	,	0.350516498	,	0.259226433	,],
[	0.278690654	,	0.097288366	,	0.243870566	,	0.295301605	,],
[	0.385029006	,	0.36127214	,	0.082873047	,	0.292129648	,],
[	0.33263149	,	0.442346206	,	0.328464829	,	0.094471268	,]],
[[	0.095050837	,	0.358947761	,	0.349461605	,	0.258046459	,],
[	0.282646684	,	0.098308764	,	0.247129633	,	0.298964083	,],
[	0.384481571	,	0.361158441	,	0.084413884	,	0.291478514	,],
[	0.331888937	,	0.441816659	,	0.327721869	,	0.095337941	,]],
[[	0.097031876	,	0.363836216	,	0.35259702	,	0.257461727	,],
[	0.271528716	,	0.095765651	,	0.242501957	,	0.289520827	,],
[	0.383247	,	0.366107535	,	0.08592766	,	0.291393135	,],
[	0.332042316	,	0.447954891	,	0.332991039	,	0.095682258	,]],
[[	0.096744558	,	0.365836302	,	0.354526083	,	0.259482293	,],
[	0.27345665	,	0.100518644	,	0.244966962	,	0.291489441	,],
[	0.379823775	,	0.362875199	,	0.083504689	,	0.287798501	,],
[	0.328822036	,	0.445153209	,	0.32983578	,	0.093548851	,]],
[[	0.094734362	,	0.362741469	,	0.351399047	,	0.256320676	,],
[	0.27305688	,	0.096146213	,	0.243820299	,	0.290983801	,],
[	0.381873133	,	0.365508701	,	0.088290334	,	0.289832378	,],
[	0.332070741	,	0.448262193	,	0.333031351	,	0.099059293	,]],
[[	0.100397294	,	0.365608709	,	0.354397262	,	0.259390011	,],
[	0.271629652	,	0.096823739	,	0.242722612	,	0.289612419	,],
[	0.381083402	,	0.364138084	,	0.084004433	,	0.289252893	,],
[	0.329906279	,	0.445976775	,	0.33075566	,	0.093735078	,]],
[[	0.09467011	,	0.363619755	,	0.352411192	,	0.257194964	,],
[	0.275374613	,	0.104007995	,	0.247142111	,	0.29336001	,],
[	0.380049905	,	0.363335953	,	0.084591438	,	0.287966823	,],
[	0.329615774	,	0.445806582	,	0.330463906	,	0.095051487	,]],
[[	0.09615588	,	0.363338812	,	0.352176335	,	0.256837257	,],
[	0.271864265	,	0.095534784	,	0.242687649	,	0.289749834	,],
[	0.384127965	,	0.36801314	,	0.092040185	,	0.292225524	,],
[	0.332975888	,	0.44881682	,	0.333882591	,	0.096502119	,]],
[[	0.097736809	,	0.366837906	,	0.355480808	,	0.260545225	,],
[	0.272930298	,	0.099445364	,	0.244224172	,	0.2907921	,],
[	0.379883594	,	0.362903201	,	0.083267631	,	0.287818553	,],
[	0.329272518	,	0.445459322	,	0.330175743	,	0.093473838	,]],
[[	0.094509369	,	0.362901997	,	0.35152751	,	0.256371549	,],
[	0.2740619	,	0.096871872	,	0.2446691	,	0.29181854	,],
[	0.381185175	,	0.364703963	,	0.086887599	,	0.289098896	,],
[	0.331103554	,	0.447167374	,	0.332002213	,	0.097457055	,]],
[[	0.098935037	,	0.364809556	,	0.353520511	,	0.258488306	,],
[	0.271500053	,	0.096189506	,	0.242525639	,	0.289420323	,],
[	0.381906242	,	0.364808465	,	0.084597222	,	0.289951774	,],
[	0.330512686	,	0.446535155	,	0.331436157	,	0.094282366	,]],
[[	0.095560491	,	0.364603006	,	0.353294256	,	0.258066132	,],
[	0.27485725	,	0.102752232	,	0.24644738	,	0.292706354	,],
[	0.379644474	,	0.362856556	,	0.083886765	,	0.287583381	,],
[	0.329359961	,	0.445509918	,	0.330142195	,	0.094287795	,]],
[[	0.096059015	,	0.368059689	,	0.356345131	,	0.257871938	,],
[	0.266595291	,	0.095710057	,	0.243413843	,	0.286195379	,],
[	0.378668972	,	0.367737686	,	0.090980674	,	0.287716706	,],
[	0.330095827	,	0.451184188	,	0.336367892	,	0.095540459	,]],
[[	0.096160768	,	0.370458618	,	0.358557838	,	0.260252455	,],
[	0.267364214	,	0.09918185	,	0.244703873	,	0.287049678	,],
[	0.375456756	,	0.363689946	,	0.083739708	,	0.284475959	,],
[	0.326543475	,	0.448053465	,	0.332869035	,	0.09295884	,]],
[[	0.09395448	,	0.36717123	,	0.355412987	,	0.256904164	,],
[	0.268920177	,	0.09701043	,	0.245490645	,	0.288361156	,],
[	0.376155731	,	0.364735827	,	0.086466145	,	0.285050778	,],
[	0.329373777	,	0.450770206	,	0.335630439	,	0.097965601	,]],
[[	0.09951146	,	0.37002233	,	0.358379397	,	0.259997325	,],
[	0.266117911	,	0.096164794	,	0.243128571	,	0.285734028	,],
[	0.37802177	,	0.366017577	,	0.085611704	,	0.287067397	,],
[	0.327974683	,	0.44919788	,	0.334076056	,	0.093679593	,]],
[[	0.094109824	,	0.368297211	,	0.356527629	,	0.257987349	,],
[	0.269096999	,	0.102501444	,	0.246758643	,	0.288769896	,],
[	0.375049124	,	0.363374485	,	0.083896722	,	0.283984215	,],
[	0.327208875	,	0.44853022	,	0.333247347	,	0.094237446	,]],
[[	0.09537011	,	0.367746163	,	0.355954817	,	0.257396217	,],
[	0.267082514	,	0.095863164	,	0.243733047	,	0.286534514	,],
[	0.377724809	,	0.366577843	,	0.089374959	,	0.286628316	,],
[	0.331385405	,	0.452378087	,	0.337642292	,	0.096807637	,]],
[[	0.097386296	,	0.371655119	,	0.359808489	,	0.261506196	,],
[	0.266880349	,	0.098080115	,	0.244056169	,	0.286442329	,],
[	0.376029544	,	0.364129559	,	0.083909664	,	0.284894026	,],
[	0.326739928	,	0.448102844	,	0.33295615	,	0.092815957	,]],
[[	0.093663712	,	0.367251437	,	0.355410493	,	0.256905495	,],
[	0.269946339	,	0.097969506	,	0.246498023	,	0.289344663	,],
[	0.375674084	,	0.364198325	,	0.085551153	,	0.284402024	,],
[	0.328415026	,	0.449850515	,	0.334566097	,	0.096454375	,]],
[[	0.098034224	,	0.369172545	,	0.357373445	,	0.258935583	,],
[	0.266260467	,	0.09580057	,	0.243101785	,	0.285777403	,],
[	0.379157751	,	0.367173187	,	0.086708076	,	0.288163135	,],
[	0.328764059	,	0.449804229	,	0.334893728	,	0.094271841	,]],
[[	0.094962468	,	0.369087976	,	0.357330011	,	0.258781092	,],
[	0.268442465	,	0.10107857	,	0.245905044	,	0.287988278	,],
[	0.375014274	,	0.363290445	,	0.083474171	,	0.2838437	,],
[	0.327059285	,	0.44847979	,	0.333092493	,	0.09376603	,]],
[[	0.095774048	,	0.372862905	,	0.358909054	,	0.25712177	,],
[	0.260510145	,	0.095561688	,	0.24117486	,	0.280462998	,],
[	0.374286554	,	0.370037098	,	0.090409567	,	0.28409723	,],
[	0.329186034	,	0.456013392	,	0.339741907	,	0.095483016	,]],
[[	0.09595873	,	0.375177453	,	0.3611717	,	0.259523462	,],
[	0.260294464	,	0.097742224	,	0.241456492	,	0.28034079	,],
[	0.371530861	,	0.366504178	,	0.083417873	,	0.281220894	,],
[	0.325314009	,	0.452715812	,	0.336008165	,	0.092506179	,]],
[[	0.093617272	,	0.371818655	,	0.357772522	,	0.256017753	,],
[	0.263894123	,	0.098065777	,	0.244355643	,	0.283741612	,],
[	0.37163503	,	0.367076603	,	0.085779956	,	0.281304458	,],
[	0.327732526	,	0.454976002	,	0.338399045	,	0.097104102	,]],
[[	0.099125634	,	0.374767298	,	0.360708422	,	0.25909152	,],
[	0.25980141	,	0.095614234	,	0.240634516	,	0.279796157	,],
[	0.374145734	,	0.368835656	,	0.085433507	,	0.283913086	,],
[	0.327101313	,	0.454051212	,	0.337524808	,	0.093544108	,]],
[[	0.093852105	,	0.372962909	,	0.358919438	,	0.257162188	,],
[	0.261516142	,	0.100475606	,	0.242987669	,	0.281608652	,],
[	0.370981365	,	0.366025439	,	0.083635948	,	0.280585924	,],
[	0.32599522	,	0.453237687	,	0.33649699	,	0.093840812	,]],
[[	0.094965268	,	0.372444919	,	0.358437578	,	0.25658565	,],
[	0.261331111	,	0.096094625	,	0.241855137	,	0.281249232	,],
[	0.37340497	,	0.368974166	,	0.088787679	,	0.283082626	,],
[	0.33035081	,	0.457146859	,	0.340923415	,	0.096568547	,]],
[[	0.097335553	,	0.376669508	,	0.362543224	,	0.26097822	,],
[	0.259965087	,	0.096851683	,	0.240953227	,	0.279899578	,],
[	0.3718686	,	0.366705004	,	0.083481677	,	0.28151332	,],
[	0.325610346	,	0.452867286	,	0.336308179	,	0.092483782	,]],
[[	0.093510642	,	0.372188622	,	0.358017713	,	0.256310673	,],
[	0.26423119	,	0.104709503	,	0.245921599	,	0.284205762	,],
[	0.37119526	,	0.366587082	,	0.08479679	,	0.280831088	,],
[	0.327069939	,	0.454339116	,	0.337645287	,	0.095842653	,]],
[[	0.097717062	,	0.373888941	,	0.359846502	,	0.258115874	,],
[	0.260050398	,	0.095534977	,	0.24079522	,	0.280031753	,],
[	0.37504152	,	0.369783117	,	0.08631459	,	0.284770126	,],
[	0.327768421	,	0.454667266	,	0.338197094	,	0.094066082	,]],
[[	0.094993279	,	0.374197288	,	0.360183434	,	0.258419402	,],
[	0.260919215	,	0.099299233	,	0.242263352	,	0.280993349	,],
[	0.371126293	,	0.366091315	,	0.083247502	,	0.280621232	,],
[	0.325875965	,	0.453014709	,	0.336385748	,	0.093222482	,]],
[[	0.097438524	,	0.225612321	,	0.249412223	,	0.254464124	,],
[	0.388577699	,	0.083362604	,	0.24496056	,	0.386446519	,],
[	0.465497561	,	0.307711715	,	0.073901626	,	0.36178164	,],
[	0.360503654	,	0.332143198	,	0.243589405	,	0.091263159	,]],
[[	0.093952933	,	0.220390026	,	0.244275678	,	0.249633559	,],
[	0.389326149	,	0.080817895	,	0.245673825	,	0.387265646	,],
[	0.463463815	,	0.305535748	,	0.073144445	,	0.359503288	,],
[	0.360745022	,	0.331796575	,	0.243258422	,	0.09335218	,]],
[[	0.098792681	,	0.221999714	,	0.245862269	,	0.25162614	,],
[	0.387096403	,	0.080186455	,	0.243368784	,	0.384990486	,],
[	0.4666025	,	0.309095275	,	0.079251642	,	0.362688172	,],
[	0.363306215	,	0.335137911	,	0.246721119	,	0.093589332	,]],
[[	0.09508092	,	0.22285364	,	0.246638782	,	0.251753711	,],
[	0.390724914	,	0.087106728	,	0.24729667	,	0.388637595	,],
[	0.464101302	,	0.306194325	,	0.072668437	,	0.360237011	,],
[	0.359712934	,	0.331220651	,	0.242636043	,	0.091073588	,]],
[[	0.094892959	,	0.220276187	,	0.244273906	,	0.249671788	,],
[	0.387753808	,	0.07970332	,	0.244038327	,	0.385711689	,],
[	0.464395048	,	0.306519815	,	0.075110079	,	0.360390425	,],
[	0.362583966	,	0.333485347	,	0.245024725	,	0.096321282	,]],
[[	0.098571017	,	0.226823831	,	0.250493612	,	0.255535574	,],
[	0.387983485	,	0.082265728	,	0.244228517	,	0.385879086	,],
[	0.466338437	,	0.308666722	,	0.074655651	,	0.362609291	,],
[	0.361286592	,	0.332769496	,	0.244400747	,	0.091873187	,]],
[[	0.093817708	,	0.220730503	,	0.244607557	,	0.249895333	,],
[	0.390284616	,	0.081507416	,	0.246655888	,	0.388243624	,],
[	0.463620503	,	0.305621839	,	0.072772081	,	0.359630994	,],
[	0.360369324	,	0.331559136	,	0.243042941	,	0.092617774	,]],
[[	0.097368492	,	0.221094518	,	0.245054015	,	0.250707038	,],
[	0.387077712	,	0.07972214	,	0.243238997	,	0.384987827	,],
[	0.466047598	,	0.308321327	,	0.07805316	,	0.361994715	,],
[	0.364232676	,	0.336122907	,	0.247813759	,	0.094571724	,]],
[[	0.095925994	,	0.22381554	,	0.247684008	,	0.252645324	,],
[	0.389787325	,	0.085445114	,	0.246263022	,	0.387536025	,],
[	0.464401209	,	0.306495121	,	0.072961389	,	0.360484469	,],
[	0.360248861	,	0.331697871	,	0.243192873	,	0.091191684	,]],
[[	0.094542934	,	0.220394336	,	0.244327661	,	0.249650257	,],
[	0.388282995	,	0.08002929	,	0.244462551	,	0.386047624	,],
[	0.464067861	,	0.306125792	,	0.07429701	,	0.360040263	,],
[	0.361610264	,	0.332588579	,	0.244054351	,	0.094812681	,]],
[[	0.097169032	,	0.231126165	,	0.253933308	,	0.254159074	,],
[	0.379113677	,	0.080051857	,	0.241815973	,	0.379093478	,],
[	0.458032514	,	0.306945875	,	0.077742101	,	0.356303114	,],
[	0.356970985	,	0.335278483	,	0.247505315	,	0.091643401	,]],
[[	0.093177347	,	0.225619115	,	0.248550775	,	0.248976575	,],
[	0.382113127	,	0.086147738	,	0.245074525	,	0.38187226	,],
[	0.456111409	,	0.304788963	,	0.072080597	,	0.354441328	,],
[	0.354858035	,	0.332749089	,	0.244630928	,	0.090761735	,]],
[[	0.096830271	,	0.226304506	,	0.249195791	,	0.250147925	,],
[	0.379823478	,	0.079695984	,	0.242592964	,	0.379718731	,],
[	0.455814771	,	0.304473009	,	0.073648012	,	0.353988469	,],
[	0.358201642	,	0.335538201	,	0.247627067	,	0.096584249	,]],
[[	0.094548648	,	0.228244819	,	0.251096882	,	0.251358741	,],
[	0.379834129	,	0.081913013	,	0.242635627	,	0.379681569	,],
[	0.45867811	,	0.30756564	,	0.074343239	,	0.357251931	,],
[	0.355490331	,	0.333721444	,	0.245758631	,	0.090455775	,]],
[[	0.093714686	,	0.225256353	,	0.248153342	,	0.24876793	,],
[	0.382677384	,	0.082016716	,	0.245459702	,	0.382506046	,],
[	0.455367179	,	0.304053306	,	0.071858709	,	0.353588727	,],
[	0.355514344	,	0.333231326	,	0.245335668	,	0.0923414	,]],
[[	0.099091042	,	0.232859998	,	0.255598643	,	0.255911851	,],
[	0.378923395	,	0.07952765	,	0.241630759	,	0.378721715	,],
[	0.457182859	,	0.306020665	,	0.076320528	,	0.355456269	,],
[	0.358054784	,	0.336278526	,	0.248470211	,	0.09247392	,]],
[[	0.093345554	,	0.226061596	,	0.2488146	,	0.249381343	,],
[	0.381320719	,	0.084729746	,	0.244199111	,	0.380983904	,],
[	0.456758578	,	0.305433776	,	0.072519168	,	0.355101904	,],
[	0.354769044	,	0.332738174	,	0.244690986	,	0.090250032	,]],
[[	0.095846467	,	0.225822524	,	0.248729724	,	0.249478321	,],
[	0.380779657	,	0.080366079	,	0.243481304	,	0.380593817	,],
[	0.455631338	,	0.304292677	,	0.072956543	,	0.353752303	,],
[	0.357319261	,	0.334610843	,	0.246641438	,	0.095176567	,]],
[[	0.095645115	,	0.229281979	,	0.252135722	,	0.252297583	,],
[	0.379278658	,	0.080799299	,	0.242092212	,	0.379160192	,],
[	0.459338299	,	0.308454869	,	0.079941176	,	0.357638891	,],
[	0.356252347	,	0.334458381	,	0.246521865	,	0.090843605	,]],
[[	0.093379909	,	0.225211449	,	0.248186717	,	0.248675151	,],
[	0.383725601	,	0.088631003	,	0.24675899	,	0.38335975	,],
[	0.455467617	,	0.304138995	,	0.071706892	,	0.353720595	,],
[	0.355060497	,	0.332746754	,	0.24470354	,	0.091225471	,]],
[[	0.097856655	,	0.236729613	,	0.258141361	,	0.254138252	,],
[	0.373262177	,	0.079570582	,	0.241277356	,	0.373774658	,],
[	0.451093807	,	0.305581744	,	0.074933206	,	0.350201224	,],
[	0.354256017	,	0.338827455	,	0.250812656	,	0.092067716	,]],
[[	0.092553624	,	0.23027364	,	0.251811237	,	0.248139455	,],
[	0.374495679	,	0.08352144	,	0.242708185	,	0.374974606	,],
[	0.451985144	,	0.306490059	,	0.073182094	,	0.35139838	,],
[	0.350939922	,	0.335146962	,	0.246922828	,	0.089707554	,]],
[[	0.095286166	,	0.230161937	,	0.251784589	,	0.248444116	,],
[	0.375090353	,	0.080589805	,	0.243280887	,	0.375739867	,],
[	0.449661341	,	0.303954602	,	0.071906277	,	0.348718206	,],
[	0.353333991	,	0.337127588	,	0.24895213	,	0.094680914	,]],
[[	0.09461806	,	0.2334802	,	0.255014716	,	0.250972218	,],
[	0.372625048	,	0.079994148	,	0.240765569	,	0.373152149	,],
[	0.452820708	,	0.307468229	,	0.078060177	,	0.351942299	,],
[	0.352034787	,	0.336624366	,	0.248627774	,	0.09014077	,]],
[[	0.092731265	,	0.229538862	,	0.251227525	,	0.247380458	,],
[	0.376592299	,	0.087006844	,	0.244963053	,	0.376988229	,],
[	0.450650271	,	0.304963429	,	0.071874008	,	0.349834146	,],
[	0.351364584	,	0.335486057	,	0.247284433	,	0.090977108	,]],
[[	0.09829557	,	0.23174648	,	0.253521922	,	0.250168466	,],
[	0.373596234	,	0.079666795	,	0.241566905	,	0.374149824	,],
[	0.450474395	,	0.304950167	,	0.073877086	,	0.349511867	,],
[	0.355467752	,	0.339204618	,	0.251001586	,	0.097996826	,]],
[[	0.092972693	,	0.230861917	,	0.252476263	,	0.248613734	,],
[	0.373770233	,	0.082248484	,	0.24187051	,	0.374214617	,],
[	0.452916173	,	0.307584228	,	0.074045084	,	0.352298296	,],
[	0.351267315	,	0.335528994	,	0.247264479	,	0.089630137	,]],
[[	0.094352657	,	0.229747866	,	0.251383822	,	0.247952882	,],
[	0.376169281	,	0.081465403	,	0.244260655	,	0.376588259	,],
[	0.449959524	,	0.304229537	,	0.071852074	,	0.349003896	,],
[	0.352591814	,	0.336452665	,	0.24817594	,	0.093250538	,]],
[[	0.095911139	,	0.234668108	,	0.256224397	,	0.252102967	,],
[	0.37296186	,	0.079747482	,	0.241008659	,	0.373461216	,],
[	0.451845066	,	0.306419892	,	0.076606821	,	0.350830283	,],
[	0.353145015	,	0.337744738	,	0.249619221	,	0.090890198	,]],
[[	0.092541922	,	0.229796321	,	0.2515682	,	0.247611301	,],
[	0.375545728	,	0.085354293	,	0.243841898	,	0.375960872	,],
[	0.451100426	,	0.305498641	,	0.072221595	,	0.350242826	,],
[	0.350899001	,	0.33507192	,	0.246847841	,	0.090185869	,]],
[[	0.096638822	,	0.242027706	,	0.262721683	,	0.254072198	,],
[	0.367578389	,	0.080452823	,	0.242312851	,	0.370001155	,],
[	0.442748065	,	0.303851107	,	0.072253569	,	0.343635847	,],
[	0.349868569	,	0.34019537	,	0.252574134	,	0.094951223	,]],
[[	0.091230436	,	0.235467997	,	0.256236158	,	0.247820604	,],
[	0.365899328	,	0.080568968	,	0.240520469	,	0.368361384	,],
[	0.446145795	,	0.307625948	,	0.078714859	,	0.347095704	,],
[	0.347714263	,	0.338736716	,	0.251119907	,	0.089408519	,]],
[[	0.0935611	,	0.235240306	,	0.25600416	,	0.247846858	,],
[	0.369104568	,	0.087020019	,	0.24416724	,	0.371412186	,],
[	0.443357831	,	0.304541977	,	0.071984114	,	0.344401573	,],
[	0.347347349	,	0.337994667	,	0.250263576	,	0.090883368	,]],
[[	0.093427483	,	0.238751363	,	0.259486927	,	0.250788462	,],
[	0.365989852	,	0.079428788	,	0.240671625	,	0.368444418	,],
[	0.443552204	,	0.304763369	,	0.074162225	,	0.344342407	,],
[	0.350444756	,	0.341723536	,	0.254271466	,	0.091746018	,]],
[[	0.091087178	,	0.234646058	,	0.255528701	,	0.24702939	,],
[	0.366620383	,	0.082517002	,	0.241504593	,	0.369023794	,],
[	0.446187634	,	0.307476784	,	0.074414494	,	0.347311012	,],
[	0.346787647	,	0.337702908	,	0.250087942	,	0.089061761	,]],
[[	0.096534813	,	0.236883116	,	0.257584561	,	0.249605915	,],
[	0.368573123	,	0.081342706	,	0.243293292	,	0.370909334	,],
[	0.442815985	,	0.30396144	,	0.07194736	,	0.343545351	,],
[	0.349120335	,	0.339534125	,	0.251994049	,	0.093662232	,]],
[[	0.091835901	,	0.236481771	,	0.257146888	,	0.248660145	,],
[	0.365737074	,	0.079933208	,	0.24035229	,	0.368047487	,],
[	0.445260179	,	0.306662795	,	0.077102861	,	0.346033629	,],
[	0.348259152	,	0.339188321	,	0.25170743	,	0.089793813	,]],
[[	0.092796257	,	0.234970641	,	0.255756696	,	0.247521929	,],
[	0.368482059	,	0.085781192	,	0.243380472	,	0.370746842	,],
[	0.44388078	,	0.305072105	,	0.072251172	,	0.344807647	,],
[	0.347039717	,	0.337686575	,	0.249991011	,	0.090063117	,]],
[[	0.094656072	,	0.240015675	,	0.260795651	,	0.251931673	,],
[	0.366575107	,	0.079753008	,	0.241238562	,	0.368861239	,],
[	0.443396875	,	0.304435467	,	0.073431895	,	0.344071015	,],
[	0.35138941	,	0.341418426	,	0.253899848	,	0.097285837	,]],
[[	0.091144881	,	0.235041569	,	0.255880506	,	0.24717669	,],
[	0.366205163	,	0.081515423	,	0.241065722	,	0.368539628	,],
[	0.446964237	,	0.308375177	,	0.07516696	,	0.348166446	,],
[	0.346969435	,	0.33786878	,	0.250326355	,	0.088863832	,]],
[[	0.096163603	,	0.246602001	,	0.266162624	,	0.252281143	,],
[	0.360891209	,	0.080545462	,	0.241027264	,	0.364096972	,],
[	0.435696269	,	0.30338142	,	0.071425288	,	0.338134773	,],
[	0.344651454	,	0.341954918	,	0.253655355	,	0.09309903	,]],
[[	0.09034047	,	0.239962655	,	0.259556398	,	0.245790901	,],
[	0.358112112	,	0.079180191	,	0.23814865	,	0.361260502	,],
[	0.438307239	,	0.306334446	,	0.076856661	,	0.340791143	,],
[	0.34343624	,	0.341344876	,	0.253017623	,	0.088537523	,]],
[[	0.092185918	,	0.239167426	,	0.258863932	,	0.245457873	,],
[	0.36076409	,	0.085054455	,	0.241094028	,	0.363781014	,],
[	0.436846642	,	0.304490815	,	0.071642882	,	0.339349004	,],
[	0.342450921	,	0.339952618	,	0.251553349	,	0.089280953	,]],
[[	0.092679255	,	0.243215164	,	0.262825303	,	0.2488755	,],
[	0.358833493	,	0.078986309	,	0.238822929	,	0.361998237	,],
[	0.436121789	,	0.303889738	,	0.072731723	,	0.338525854	,],
[	0.346696847	,	0.344823744	,	0.256601106	,	0.091497384	,]],
[[	0.089947659	,	0.238749012	,	0.258480824	,	0.244634908	,],
[	0.358576108	,	0.080831414	,	0.238724524	,	0.361678412	,],
[	0.439989382	,	0.307841244	,	0.074479726	,	0.342586793	,],
[	0.342430491	,	0.340242852	,	0.251910577	,	0.087986359	,]],
[[	0.095182839	,	0.240779231	,	0.260355805	,	0.247034172	,],
[	0.362110722	,	0.081574431	,	0.242060295	,	0.365061367	,],
[	0.435928703	,	0.303638219	,	0.071228829	,	0.338318926	,],
[	0.344015074	,	0.341319509	,	0.252956799	,	0.091859418	,]],
[[	0.091025187	,	0.240732935	,	0.260434903	,	0.246676677	,],
[	0.35813181	,	0.078928014	,	0.238181744	,	0.361186715	,],
[	0.437541687	,	0.305390261	,	0.075535533	,	0.339875153	,],
[	0.344345698	,	0.342152087	,	0.253860202	,	0.089178387	,]],
[[	0.091405966	,	0.238956122	,	0.258683392	,	0.245147971	,],
[	0.359871784	,	0.08349418	,	0.240085992	,	0.362812071	,],
[	0.43748364	,	0.305232928	,	0.072191104	,	0.340004628	,],
[	0.342574171	,	0.34002556	,	0.251638079	,	0.088872259	,]],
[[	0.094041051	,	0.244562629	,	0.26422489	,	0.250129213	,],
[	0.359274504	,	0.079298879	,	0.239436544	,	0.362473874	,],
[	0.435910627	,	0.303568124	,	0.072055113	,	0.338197253	,],
[	0.34610815	,	0.343127151	,	0.254792105	,	0.095158983	,]],
[[	0.089933302	,	0.239107572	,	0.258777841	,	0.244968141	,],
[	0.358400038	,	0.080105653	,	0.23841941	,	0.361400431	,],
[	0.439904874	,	0.307967375	,	0.07925592	,	0.342255378	,],
[	0.342774683	,	0.340474086	,	0.252233012	,	0.088006259	,]],
[[	0.095417463	,	0.246212564	,	0.264081137	,	0.247397297	,],
[	0.355349444	,	0.087289354	,	0.241303466	,	0.359546559	,],
[	0.430361518	,	0.30470177	,	0.070994999	,	0.334041209	,],
[	0.339787919	,	0.343232187	,	0.254720546	,	0.090214723	,]],
[[	0.089523505	,	0.24475444	,	0.262570493	,	0.245413736	,],
[	0.351949476	,	0.079185666	,	0.237507758	,	0.356393296	,],
[	0.430016807	,	0.304438936	,	0.072692709	,	0.333660457	,],
[	0.342030352	,	0.345776478	,	0.257397637	,	0.089862395	,]],
[[	0.091128954	,	0.243897716	,	0.261738411	,	0.244852334	,],
[	0.352753611	,	0.082563792	,	0.238466293	,	0.357117355	,],
[	0.433003513	,	0.307543788	,	0.073408445	,	0.336957452	,],
[	0.338797139	,	0.342262429	,	0.253808344	,	0.087861864	,]],
[[	0.092323897	,	0.248505681	,	0.26632774	,	0.249000989	,],
[	0.35410883	,	0.080635389	,	0.239747197	,	0.358483274	,],
[	0.429729659	,	0.303935536	,	0.070894875	,	0.333301564	,],
[	0.341619376	,	0.344789024	,	0.256245545	,	0.093183346	,]],
[[	0.089270928	,	0.243629079	,	0.26162626	,	0.244440328	,],
[	0.351579193	,	0.079755166	,	0.237158575	,	0.355947225	,],
[	0.431416776	,	0.306113481	,	0.075510649	,	0.335049188	,],
[	0.340037371	,	0.343909522	,	0.255453	,	0.088174401	,]],
[[	0.094326893	,	0.245701005	,	0.263570401	,	0.246785228	,],
[	0.354462341	,	0.085737761	,	0.240294464	,	0.358683483	,],
[	0.430882817	,	0.305252177	,	0.071493977	,	0.334662004	,],
[	0.339242266	,	0.342798078	,	0.254273363	,	0.089253986	,]],
[[	0.09014301	,	0.245712009	,	0.263444529	,	0.246238822	,],
[	0.35242321	,	0.079334642	,	0.237767702	,	0.356627001	,],
[	0.430129174	,	0.304459961	,	0.072177324	,	0.333575908	,],
[	0.343042145	,	0.346926212	,	0.258650154	,	0.090958558	,]],
[[	0.090810161	,	0.244093722	,	0.261942028	,	0.245032073	,],
[	0.352302387	,	0.08150551	,	0.237805875	,	0.356481852	,],
[	0.433977385	,	0.308881112	,	0.079429962	,	0.337565999	,],
[	0.338967736	,	0.34253528	,	0.253985979	,	0.087679951	,]],
[[	0.093504846	,	0.249686947	,	0.267458888	,	0.250122981	,],
[	0.35531044	,	0.081731204	,	0.240896731	,	0.359545347	,],
[	0.429769571	,	0.304063669	,	0.070710616	,	0.33328067	,],
[	0.340960597	,	0.344134437	,	0.255573086	,	0.091993765	,]],
[[	0.089283097	,	0.24415131	,	0.262024563	,	0.244899051	,],
[	0.351738636	,	0.079314485	,	0.237229291	,	0.355957653	,],
[	0.431213905	,	0.305695483	,	0.074547507	,	0.334583387	,],
[	0.340496061	,	0.344298506	,	0.255883246	,	0.088440643	,]],
[[	0.09430997	,	0.25135064	,	0.268711432	,	0.247536937	,],
[	0.348402906	,	0.083399652	,	0.23882304	,	0.354342141	,],
[	0.429535806	,	0.309233848	,	0.074109834	,	0.334538294	,],
[	0.337320643	,	0.346782131	,	0.25722799	,	0.088048038	,]],
[[	0.089666456	,	0.250986474	,	0.268179807	,	0.246623564	,],
[	0.349324702	,	0.080764898	,	0.239490368	,	0.355259253	,],
[	0.426416459	,	0.305818408	,	0.071860057	,	0.331050843	,],
[	0.341189927	,	0.350372119	,	0.260827159	,	0.094801985	,]],
[[	0.090507891	,	0.24954207	,	0.266835605	,	0.245530994	,],
[	0.347004259	,	0.080233581	,	0.237169145	,	0.352941373	,],
[	0.428458079	,	0.308232864	,	0.076593144	,	0.333054406	,],
[	0.33731539	,	0.346911982	,	0.257402417	,	0.087107311	,]],
[[	0.092924441	,	0.254928858	,	0.272189328	,	0.250465573	,],
[	0.350241685	,	0.086651756	,	0.240777142	,	0.356019263	,],
[	0.42749295	,	0.30694834	,	0.072198467	,	0.332241469	,],
[	0.338536593	,	0.347818017	,	0.258142794	,	0.090288835	,]],
[[	0.088847687	,	0.249545333	,	0.266866543	,	0.245251145	,],
[	0.347799304	,	0.079656221	,	0.23795025	,	0.353731158	,],
[	0.426352541	,	0.305861501	,	0.072799367	,	0.330887205	,],
[	0.339784367	,	0.349678326	,	0.260190135	,	0.088998184	,]],
[[	0.093038659	,	0.250819164	,	0.268063507	,	0.24680459	,],
[	0.347872904	,	0.082193947	,	0.238115944	,	0.353686413	,],
[	0.430295558	,	0.310315487	,	0.079987178	,	0.334931128	,],
[	0.336833473	,	0.346478577	,	0.256872612	,	0.087338056	,]],
[[	0.090445614	,	0.251993253	,	0.269098423	,	0.247549375	,],
[	0.350445535	,	0.081686714	,	0.240576544	,	0.356307679	,],
[	0.426605393	,	0.305950633	,	0.071738123	,	0.331128387	,],
[	0.340378557	,	0.349573612	,	0.260045129	,	0.093438643	,]],
[[	0.089737307	,	0.249292744	,	0.266497597	,	0.245138476	,],
[	0.347307925	,	0.080034596	,	0.237352091	,	0.35305474	,],
[	0.42772879	,	0.30742147	,	0.075293993	,	0.332230428	,],
[	0.337803362	,	0.34733122	,	0.257850325	,	0.087223345	,]],
[[	0.094346448	,	0.256340834	,	0.27348713	,	0.251767874	,],
[	0.349505641	,	0.08525673	,	0.239923455	,	0.355260377	,],
[	0.428173022	,	0.30772656	,	0.072719304	,	0.332884254	,],
[	0.337783354	,	0.347121335	,	0.257467553	,	0.089026764	,]],
[[	0.089300091	,	0.250262489	,	0.267485113	,	0.245888829	,],
[	0.348175284	,	0.079869647	,	0.238277739	,	0.35400049	,],
[	0.426282216	,	0.305726991	,	0.072100944	,	0.330699319	,],
[	0.340427286	,	0.350280771	,	0.260886515	,	0.0894345	,]],
[[	0.092908178	,	0.255900789	,	0.271245445	,	0.246040942	,],
[	0.340559349	,	0.082546294	,	0.236773881	,	0.347766327	,],
[	0.424106895	,	0.311223527	,	0.079403311	,	0.330387597	,],
[	0.33302138	,	0.349009509	,	0.259159062	,	0.086992701	,]],
[[	0.088966603	,	0.255870423	,	0.271291361	,	0.245647021	,],
[	0.343409848	,	0.082542062	,	0.239459004	,	0.35062522	,],
[	0.420362008	,	0.306852782	,	0.071187719	,	0.326593058	,],
[	0.335857145	,	0.351503619	,	0.261617646	,	0.092289363	,]],
[[	0.089394806	,	0.254180886	,	0.269487216	,	0.244192382	,],
[	0.339941646	,	0.080262836	,	0.235901977	,	0.347152046	,],
[	0.421198189	,	0.307848855	,	0.074407032	,	0.327361958	,],
[	0.334163059	,	0.350237643	,	0.26039383	,	0.08719109	,]],
[[	0.092386162	,	0.259899942	,	0.275452858	,	0.249557045	,],
[	0.341834546	,	0.085334844	,	0.238128744	,	0.348873961	,],
[	0.422106218	,	0.3085913	,	0.072346203	,	0.328563457	,],
[	0.333744324	,	0.349502855	,	0.259477892	,	0.088552482	,]],
[[	0.08781464	,	0.254181873	,	0.269647191	,	0.2440072	,],
[	0.3414193	,	0.080806247	,	0.2373936	,	0.348577777	,],
[	0.419858297	,	0.306333635	,	0.071470267	,	0.325961687	,],
[	0.337469394	,	0.353767984	,	0.264059719	,	0.090268535	,]],
[[	0.091873438	,	0.255327714	,	0.270840309	,	0.245413288	,],
[	0.340246123	,	0.081662929	,	0.236300792	,	0.347367332	,],
[	0.423176218	,	0.310054299	,	0.077836445	,	0.329290218	,],
[	0.333207946	,	0.349275272	,	0.259406941	,	0.086826122	,]],
[[	0.089768803	,	0.256940084	,	0.272316467	,	0.24661373	,],
[	0.344284243	,	0.08917032	,	0.240721455	,	0.351244638	,],
[	0.420541141	,	0.307049518	,	0.071148239	,	0.326790194	,],
[	0.335382577	,	0.351048886	,	0.261065939	,	0.091162062	,]],
[[	0.088598488	,	0.253889153	,	0.269283823	,	0.243751571	,],
[	0.34000912	,	0.079969433	,	0.235840749	,	0.347109296	,],
[	0.420835577	,	0.307469613	,	0.073466335	,	0.326846156	,],
[	0.33515073	,	0.351251277	,	0.261407988	,	0.087882609	,]],
[[	0.093687222	,	0.261226702	,	0.276662763	,	0.250778993	,],
[	0.341393849	,	0.084165124	,	0.237540017	,	0.34838988	,],
[	0.423254423	,	0.309889623	,	0.073502113	,	0.32964385	,],
[	0.333307342	,	0.349144056	,	0.259137354	,	0.087658515	,]],
[[	0.088573873	,	0.255159371	,	0.27058639	,	0.244934986	,],
[	0.342152479	,	0.081309489	,	0.238029004	,	0.349237797	,],
[	0.419932499	,	0.306330958	,	0.071157082	,	0.325918059	,],
[	0.337587889	,	0.353004244	,	0.263092835	,	0.094724833	,]],
[[	0.092412305	,	0.260482111	,	0.275471897	,	0.246058112	,],
[	0.334129572	,	0.081153174	,	0.236016559	,	0.34269187	,],
[	0.416386201	,	0.30899385	,	0.077482614	,	0.324198931	,],
[	0.329412031	,	0.35137735	,	0.26157129	,	0.086990897	,]],
[[	0.088159503	,	0.260222009	,	0.275367225	,	0.245586529	,],
[	0.338129946	,	0.088648732	,	0.240401706	,	0.34651541	,],
[	0.414058621	,	0.306260068	,	0.071190902	,	0.321887813	,],
[	0.332248732	,	0.35379345	,	0.263974232	,	0.092309745	,]],
[[	0.088753618	,	0.258675895	,	0.273653956	,	0.244260601	,],
[	0.334120968	,	0.079717582	,	0.235736226	,	0.342611694	,],
[	0.414065379	,	0.306444469	,	0.073167239	,	0.32165763	,],
[	0.330576886	,	0.352538512	,	0.262793593	,	0.08736555	,]],
[[	0.091198662	,	0.263976951	,	0.279087738	,	0.249264198	,],
[	0.335391806	,	0.083722719	,	0.237314523	,	0.34385943	,],
[	0.416816692	,	0.309113093	,	0.07356187	,	0.324881695	,],
[	0.329851957	,	0.351597441	,	0.261697003	,	0.088377814	,]],
[[	0.087160141	,	0.258694403	,	0.273864601	,	0.244022721	,],
[	0.336299877	,	0.080838589	,	0.237952034	,	0.344827584	,],
[	0.413139158	,	0.305362241	,	0.070928654	,	0.320897658	,],
[	0.333883523	,	0.356091327	,	0.266453906	,	0.090314339	,]],
[[	0.091246551	,	0.259804456	,	0.27484724	,	0.245434614	,],
[	0.333882489	,	0.080309557	,	0.23558727	,	0.342350641	,],
[	0.415591984	,	0.308166615	,	0.07614817	,	0.323251137	,],
[	0.329597338	,	0.35157825	,	0.261724201	,	0.086874648	,]],
[[	0.088825256	,	0.261281594	,	0.276253195	,	0.24653268	,],
[	0.337278858	,	0.087084537	,	0.239376081	,	0.345624072	,],
[	0.414767446	,	0.307044597	,	0.071694087	,	0.322571464	,],
[	0.331055662	,	0.352620749	,	0.262797635	,	0.090535596	,]],
[[	0.08801234	,	0.258361334	,	0.27343111	,	0.243781785	,],
[	0.334538067	,	0.07974013	,	0.236101677	,	0.343006821	,],
[	0.413791319	,	0.306030221	,	0.07244416	,	0.32134549	,],
[	0.331405078	,	0.353261793	,	0.263487347	,	0.087882317	,]],
[[	0.092615151	,	0.265383217	,	0.280423778	,	0.250524702	,],
[	0.334901562	,	0.082649728	,	0.236663842	,	0.343355709	,],
[	0.417592524	,	0.310381001	,	0.079684572	,	0.325336442	,],
[	0.329593393	,	0.351424649	,	0.261477321	,	0.087556399	,]],
[[	0.087461036	,	0.259188927	,	0.274270585	,	0.244550088	,],
[	0.337027091	,	0.081444629	,	0.238642999	,	0.345557793	,],
[	0.41349729	,	0.305795362	,	0.07098478	,	0.321282291	,],
[	0.333623142	,	0.354890797	,	0.265098362	,	0.094365013	,]],
[[	0.091559108	,	0.265289994	,	0.279475478	,	0.245633921	,],
[	0.3284971	,	0.079754011	,	0.235305215	,	0.338197932	,],
[	0.411257381	,	0.309034871	,	0.074353642	,	0.319450444	,],
[	0.32878579	,	0.356718213	,	0.266340531	,	0.086830571	,]],
[[	0.088401187	,	0.265671499	,	0.279871561	,	0.24575036	,],
[	0.330370415	,	0.084576997	,	0.23743293	,	0.339936636	,],
[	0.412340077	,	0.310044419	,	0.072605044	,	0.320737287	,],
[	0.329870959	,	0.357474156	,	0.2670361	,	0.089897491	,]],
[[	0.088200391	,	0.263522831	,	0.277740919	,	0.243793897	,],
[	0.33012621	,	0.080380247	,	0.236866323	,	0.339795649	,],
[	0.410331247	,	0.307944148	,	0.071763893	,	0.318438263	,],
[	0.330937379	,	0.358850945	,	0.268554706	,	0.08845543	,]],
[[	0.091977567	,	0.269921447	,	0.28407751	,	0.249814547	,],
[	0.328692648	,	0.080923433	,	0.235571257	,	0.338297268	,],
[	0.412948284	,	0.311062714	,	0.077505231	,	0.321174047	,],
[	0.328528116	,	0.356270346	,	0.265827629	,	0.087132979	,]],
[[	0.087359664	,	0.264160938	,	0.278419495	,	0.244282144	,],
[	0.332467888	,	0.088263885	,	0.239661989	,	0.341861953	,],
[	0.410646745	,	0.308276071	,	0.071169356	,	0.318866208	,],
[	0.331962904	,	0.35913349	,	0.268760395	,	0.093078993	,]],
[[	0.090388082	,	0.264564582	,	0.278773458	,	0.244830055	,],
[	0.328945037	,	0.079824463	,	0.235684179	,	0.338566552	,],
[	0.410550586	,	0.308362331	,	0.073122515	,	0.31858611	,],
[	0.329368551	,	0.357346734	,	0.266973326	,	0.087169986	,]],
[[	0.089336598	,	0.26686833	,	0.280965367	,	0.246817207	,],
[	0.329806545	,	0.083403267	,	0.236840534	,	0.339383396	,],
[	0.413385903	,	0.311069889	,	0.073525981	,	0.321793404	,],
[	0.32904712	,	0.356757649	,	0.266311939	,	0.088589332	,]],
[[	0.08776684	,	0.263717073	,	0.277805408	,	0.243741262	,],
[	0.331252939	,	0.081308153	,	0.237917168	,	0.340786856	,],
[	0.410117203	,	0.307712992	,	0.071203497	,	0.318239043	,],
[	0.332225789	,	0.360180878	,	0.269909534	,	0.089604055	,]],
[[	0.093430413	,	0.271275922	,	0.285369935	,	0.251130723	,],
[	0.328462314	,	0.080172368	,	0.235151011	,	0.337975395	,],
[	0.412207112	,	0.310119172	,	0.076024757	,	0.320222496	,],
[	0.328572765	,	0.356287289	,	0.265807956	,	0.086864846	,]],
[[	0.087838195	,	0.264884769	,	0.279047914	,	0.244891952	,],
[	0.331775374	,	0.086936504	,	0.23880677	,	0.341123451	,],
[	0.411383785	,	0.308973748	,	0.07156535	,	0.319549182	,],
[	0.330822249	,	0.358256665	,	0.267753277	,	0.091582893	,]],
[[	0.090598936	,	0.270105629	,	0.282549873	,	0.245235394	,],
[	0.324827489	,	0.080178605	,	0.235483632	,	0.33500119	,],
[	0.406496956	,	0.308828321	,	0.071501407	,	0.314653065	,],
[	0.328065688	,	0.361154881	,	0.269895372	,	0.086721777	,]],
[[	0.088440425	,	0.271483653	,	0.28395083	,	0.246290451	,],
[	0.323857623	,	0.081472443	,	0.234563929	,	0.333913894	,],
[	0.409495276	,	0.312216614	,	0.077489702	,	0.317622112	,],
[	0.326893858	,	0.359854752	,	0.26833093	,	0.086988098	,]],
[[	0.087629731	,	0.268892666	,	0.281298128	,	0.243764123	,],
[	0.328105255	,	0.082960296	,	0.238629102	,	0.338125781	,],
[	0.407174333	,	0.309466147	,	0.071197536	,	0.315376499	,],
[	0.331304721	,	0.363804698	,	0.272345648	,	0.094072712	,]],
[[	0.092257921	,	0.275709494	,	0.288202141	,	0.250397126	,],
[	0.323696495	,	0.079772559	,	0.234259597	,	0.333810101	,],
[	0.407234121	,	0.309687766	,	0.07327656	,	0.315264255	,],
[	0.327179402	,	0.360051959	,	0.268668005	,	0.086067453	,]],
[[	0.086921426	,	0.26946007	,	0.281988211	,	0.244286505	,],
[	0.325339413	,	0.084163772	,	0.236193194	,	0.335330891	,],
[	0.409255367	,	0.311532151	,	0.072852269	,	0.317691599	,],
[	0.328635673	,	0.361215301	,	0.269737545	,	0.089633942	,]],
[[	0.089605305	,	0.269676639	,	0.282079296	,	0.244613268	,],
[	0.325690503	,	0.080759452	,	0.236156849	,	0.335760089	,],
[	0.406491314	,	0.308727078	,	0.071072911	,	0.314643839	,],
[	0.329206829	,	0.362258481	,	0.271054097	,	0.087625071	,]],
[[	0.089326065	,	0.272476711	,	0.28492907	,	0.247265704	,],
[	0.323730996	,	0.080761285	,	0.234361643	,	0.333812737	,],
[	0.409021723	,	0.311573307	,	0.076365595	,	0.317089342	,],
[	0.327066203	,	0.359927699	,	0.268502498	,	0.086633562	,]],
[[	0.08692008	,	0.268546546	,	0.281002952	,	0.243333768	,],
[	0.327498913	,	0.087893242	,	0.23850816	,	0.337311849	,],
[	0.407662174	,	0.310025475	,	0.071453048	,	0.315965266	,],
[	0.33048038	,	0.362995712	,	0.271529843	,	0.092620655	,]],
[[	0.092892903	,	0.271214015	,	0.283588391	,	0.246328611	,],
[	0.323974085	,	0.079573328	,	0.234418415	,	0.333996286	,],
[	0.407250789	,	0.309606323	,	0.072706067	,	0.315198723	,],
[	0.327463744	,	0.360393861	,	0.268965653	,	0.086154847	,]],
[[	0.087620591	,	0.270289283	,	0.282909113	,	0.244975808	,],
[	0.324778295	,	0.083046956	,	0.235442875	,	0.334693588	,],
[	0.41085774	,	0.313180614	,	0.074434578	,	0.319260858	,],
[	0.327731406	,	0.360448368	,	0.26890242	,	0.088234785	,]],
[[	0.089765531	,	0.274026811	,	0.286198169	,	0.243915464	,],
[	0.318820617	,	0.080599433	,	0.235388494	,	0.330086573	,],
[	0.399303295	,	0.307946562	,	0.070558392	,	0.309226029	,],
[	0.324903134	,	0.36411757	,	0.27317471	,	0.08654014	,]],
[[	0.088248481	,	0.275749618	,	0.287974164	,	0.245584541	,],
[	0.316744626	,	0.080571516	,	0.233423715	,	0.327979057	,],
[	0.402865372	,	0.311874955	,	0.077257611	,	0.312740227	,],
[	0.323423679	,	0.362361656	,	0.271334921	,	0.086525871	,]],
[[	0.087408173	,	0.273045202	,	0.285251355	,	0.242913819	,],
[	0.320326092	,	0.087543658	,	0.237442137	,	0.331473708	,],
[	0.400307484	,	0.308799651	,	0.070410017	,	0.310193505	,],
[	0.327355415	,	0.366010048	,	0.274879763	,	0.093215979	,]],
[[	0.092471943	,	0.280463759	,	0.29267314	,	0.250033391	,],
[	0.317418992	,	0.079728194	,	0.233856866	,	0.328550173	,],
[	0.40045176	,	0.309191536	,	0.072707493	,	0.310260014	,],
[	0.323554778	,	0.362547587	,	0.271484634	,	0.085525314	,]],
[[	0.086654751	,	0.273829885	,	0.286128778	,	0.243578349	,],
[	0.317879595	,	0.082918427	,	0.23455447	,	0.328960891	,],
[	0.402699159	,	0.311269301	,	0.072584241	,	0.312911379	,],
[	0.324432335	,	0.363244473	,	0.272002227	,	0.088514228	,]],
[[	0.088920369	,	0.273576666	,	0.285899353	,	0.243494373	,],
[	0.319720769	,	0.081303928	,	0.236275793	,	0.330845044	,],
[	0.399783502	,	0.308300001	,	0.070627392	,	0.309568332	,],
[	0.325446479	,	0.36454615	,	0.273716117	,	0.086905449	,]],
[[	0.089437738	,	0.277072766	,	0.289221315	,	0.246748509	,],
[	0.316830961	,	0.080167454	,	0.233360317	,	0.328035444	,],
[	0.402097318	,	0.310999163	,	0.075803977	,	0.311866956	,],
[	0.323232058	,	0.362267378	,	0.27115675	,	0.085855346	,]],
[[	0.087061623	,	0.273220952	,	0.285425026	,	0.242987365	,],
[	0.319750583	,	0.086263407	,	0.23663948	,	0.330828134	,],
[	0.40128657	,	0.309805709	,	0.071249131	,	0.311170662	,],
[	0.32659232	,	0.365238879	,	0.274008013	,	0.091750162	,]],
[[	0.091936998	,	0.275292777	,	0.28724413	,	0.245166947	,],
[	0.317750308	,	0.079770719	,	0.234108434	,	0.32880227	,],
[	0.400359415	,	0.30893902	,	0.072231665	,	0.310066768	,],
[	0.323815504	,	0.362756211	,	0.271671539	,	0.085617873	,]],
[[	0.087422351	,	0.274671555	,	0.287000288	,	0.2443274	,],
[	0.317383158	,	0.081817497	,	0.233929916	,	0.328377376	,],
[	0.404126104	,	0.312646744	,	0.073892527	,	0.314193158	,],
[	0.323916105	,	0.362645602	,	0.271441383	,	0.08753118	,]],
[[	0.088963383	,	0.278038536	,	0.289399775	,	0.24354492	,],
[	0.313538437	,	0.081170337	,	0.235624237	,	0.326212052	,],
[	0.395336071	,	0.309933632	,	0.071042845	,	0.305866893	,],
[	0.322757219	,	0.367884358	,	0.275914076	,	0.08595444	,]],
[[	0.087691667	,	0.279958158	,	0.291380949	,	0.245362228	,],
[	0.310914031	,	0.080451288	,	0.23319831	,	0.323595893	,],
[	0.39838434	,	0.313456766	,	0.077256499	,	0.309006105	,],
[	0.321238103	,	0.366185799	,	0.274029469	,	0.086032357	,]],
[[	0.086299747	,	0.277018072	,	0.288285976	,	0.242453476	,],
[	0.314000316	,	0.086881316	,	0.236681667	,	0.326576582	,],
[	0.39606005	,	0.310641675	,	0.070855137	,	0.306824372	,],
[	0.325566119	,	0.370176837	,	0.278047448	,	0.093016271	,]],
[[	0.091597094	,	0.284352993	,	0.295804938	,	0.249596526	,],
[	0.31153357	,	0.079710027	,	0.233545756	,	0.324056015	,],
[	0.39604339	,	0.310721342	,	0.072894442	,	0.306520618	,],
[	0.321484284	,	0.366466931	,	0.274306716	,	0.085185007	,]],
[[	0.086066907	,	0.278026536	,	0.289491698	,	0.243289154	,],
[	0.311763004	,	0.082397641	,	0.233931193	,	0.324234694	,],
[	0.398937955	,	0.313639505	,	0.073279179	,	0.309828391	,],
[	0.322603968	,	0.367360112	,	0.2751247	,	0.0884528	,]],
[[	0.088094861	,	0.277666471	,	0.288922251	,	0.243011557	,],
[	0.314356515	,	0.081795782	,	0.236497639	,	0.326958809	,],
[	0.395770988	,	0.310290519	,	0.07106464	,	0.306263722	,],
[	0.323661916	,	0.368800426	,	0.27687485	,	0.086643381	,]],
[[	0.088409366	,	0.28099316	,	0.292350882	,	0.246218005	,],
[	0.310990387	,	0.080014846	,	0.233101408	,	0.323517416	,],
[	0.397669779	,	0.31256324	,	0.075947348	,	0.308165916	,],
[	0.321127983	,	0.366081541	,	0.273960849	,	0.08538805	,]],
[[	0.085995825	,	0.277089053	,	0.288363198	,	0.242469452	,],
[	0.313252737	,	0.085377565	,	0.235648112	,	0.325631083	,],
[	0.396982742	,	0.311548704	,	0.07144348	,	0.307642611	,],
[	0.324355226	,	0.368997014	,	0.276805475	,	0.091201421	,]],
[[	0.091186584	,	0.279309131	,	0.290456668	,	0.24487704	,],
[	0.312272899	,	0.08016927	,	0.234236921	,	0.324747372	,],
[	0.395963651	,	0.310601138	,	0.072277165	,	0.306416664	,],
[	0.321780557	,	0.366720038	,	0.274613416	,	0.085129427	,]],
[[	0.086432154	,	0.278632275	,	0.28999293	,	0.243849066	,],
[	0.31129771	,	0.081595674	,	0.233477212	,	0.323764788	,],
[	0.399549677	,	0.314774923	,	0.079350056	,	0.309998821	,],
[	0.32211152	,	0.366859093	,	0.274579922	,	0.087267152	,]],
[[	0.088721701	,	0.283257141	,	0.293379721	,	0.243132225	,],
[	0.308669577	,	0.081647417	,	0.235246464	,	0.321793732	,],
[	0.391656228	,	0.312008567	,	0.071223755	,	0.302875368	,],
[	0.321380204	,	0.372333684	,	0.27919345	,	0.085911307	,]],
[[	0.087118561	,	0.284896712	,	0.295100921	,	0.244696599	,],
[	0.306135606	,	0.080947701	,	0.232806376	,	0.319242077	,],
[	0.394087933	,	0.314834002	,	0.076695592	,	0.305226084	,],
[	0.319912836	,	0.370635613	,	0.277319217	,	0.085778593	,]],
[[	0.085799282	,	0.281910575	,	0.291995018	,	0.241757144	,],
[	0.30901061	,	0.087138359	,	0.236016975	,	0.322005524	,],
[	0.392606769	,	0.312845974	,	0.071077574	,	0.303863017	,],
[	0.324066133	,	0.374414692	,	0.281187539	,	0.092586687	,]],
[[	0.091104996	,	0.28927757	,	0.299595653	,	0.24896936	,],
[	0.306939164	,	0.080415545	,	0.233412315	,	0.319988657	,],
[	0.392054429	,	0.312533769	,	0.072760003	,	0.30316465	,],
[	0.320185377	,	0.370906235	,	0.27772404	,	0.084902185	,]],
[[	0.085449533	,	0.282815036	,	0.293092587	,	0.242476304	,],
[	0.306786241	,	0.08273776	,	0.233410609	,	0.31964529	,],
[	0.395681435	,	0.315934148	,	0.073906073	,	0.307129481	,],
[	0.321241885	,	0.371618185	,	0.278295553	,	0.087972768	,]],
[[	0.087674141	,	0.282571384	,	0.292660072	,	0.24244959	,],
[	0.309866123	,	0.082515798	,	0.236336493	,	0.322918892	,],
[	0.391713572	,	0.312011281	,	0.071049821	,	0.302935028	,],
[	0.32247094	,	0.373257043	,	0.280184035	,	0.086636005	,]],
[[	0.087858104	,	0.285827983	,	0.29607307	,	0.245593424	,],
[	0.305840521	,	0.080155849	,	0.232334436	,	0.31889483	,],
[	0.393661437	,	0.31428795	,	0.075567111	,	0.304649972	,],
[	0.319700951	,	0.370480204	,	0.277161549	,	0.085207928	,]],
[[	0.085670483	,	0.282194396	,	0.292260262	,	0.241924122	,],
[	0.30832357	,	0.085642251	,	0.235121641	,	0.321139223	,],
[	0.393349686	,	0.313551867	,	0.071729548	,	0.304583414	,],
[	0.323027654	,	0.373432236	,	0.280113922	,	0.091072808	,]],
[[	0.090855577	,	0.284295613	,	0.294217242	,	0.244202181	,],
[	0.307418651	,	0.08061823	,	0.233802313	,	0.320391853	,],
[	0.391848893	,	0.312266978	,	0.071995344	,	0.302820383	,],
[	0.320666752	,	0.371433766	,	0.278169719	,	0.085152565	,]],
[[	0.086000608	,	0.283650204	,	0.293844806	,	0.243332767	,],
[	0.30651426	,	0.081801699	,	0.233076265	,	0.319421811	,],
[	0.395429355	,	0.316357926	,	0.078965772	,	0.30653576	,],
[	0.320475422	,	0.371002656	,	0.277496642	,	0.086803677	,]],
[[	0.088451555	,	0.289184647	,	0.297267253	,	0.24300621	,],
[	0.303942854	,	0.089505625	,	0.236153688	,	0.317696041	,],
[	0.386594087	,	0.313202533	,	0.070830465	,	0.298697274	,],
[	0.319285541	,	0.376861876	,	0.283270192	,	0.086354961	,]],
[[	0.087517512	,	0.291493431	,	0.299758831	,	0.245253506	,],
[	0.300227789	,	0.080788382	,	0.232045375	,	0.314171089	,],
[	0.387335777	,	0.314324773	,	0.074025691	,	0.299435872	,],
[	0.316544786	,	0.374135046	,	0.280272657	,	0.084921799	,]],
[[	0.085850912	,	0.288143165	,	0.296283991	,	0.241999329	,],
[	0.301129912	,	0.084530356	,	0.233180173	,	0.315075079	,],
[	0.388802748	,	0.315504578	,	0.072373109	,	0.301109529	,],
[	0.319809142	,	0.377073012	,	0.283217866	,	0.090690241	,]],
[[	0.09186148	,	0.290899637	,	0.29891364	,	0.244948818	,],
[	0.302338036	,	0.08193922	,	0.234039964	,	0.316268075	,],
[	0.386156189	,	0.31299184	,	0.071099636	,	0.298164046	,],
[	0.317543586	,	0.37503166	,	0.281286599	,	0.08494658	,]],
[[	0.085772855	,	0.289313291	,	0.297512564	,	0.242997363	,],
[	0.300044937	,	0.081499445	,	0.231815185	,	0.313777132	,],
[	0.389219064	,	0.316478707	,	0.077262854	,	0.301191614	,],
[	0.31732581	,	0.374613069	,	0.280633916	,	0.086386434	,]],
[[	0.087622989	,	0.28899065	,	0.297075044	,	0.242649227	,],
[	0.303232728	,	0.088058563	,	0.235305579	,	0.316819139	,],
[	0.387047582	,	0.313692417	,	0.070958936	,	0.299201363	,],
[	0.320229928	,	0.377794386	,	0.284245444	,	0.087164538	,]],
[[	0.088744684	,	0.292715406	,	0.300822668	,	0.246371434	,],
[	0.300887289	,	0.080942628	,	0.232501855	,	0.314733997	,],
[	0.386953248	,	0.313824878	,	0.073033618	,	0.298821734	,],
[	0.316656118	,	0.374106717	,	0.280362916	,	0.084549678	,]],
[[	0.085656868	,	0.288435407	,	0.296424206	,	0.242179689	,],
[	0.300443891	,	0.083235064	,	0.232327843	,	0.314327024	,],
[	0.389951773	,	0.316514987	,	0.07342639	,	0.302218727	,],
[	0.318785429	,	0.37604224	,	0.28214426	,	0.089130961	,]],
[[	0.090598923	,	0.2903862	,	0.298377244	,	0.244245328	,],
[	0.303390047	,	0.082556472	,	0.234862553	,	0.317108328	,],
[	0.386444403	,	0.313110642	,	0.071074812	,	0.298444306	,],
[	0.317995691	,	0.375425724	,	0.281688768	,	0.085093108	,]],
[[	0.086196647	,	0.290122292	,	0.298255438	,	0.243725057	,],
[	0.299897278	,	0.080870391	,	0.231665436	,	0.313645136	,],
[	0.388173627	,	0.315287945	,	0.075709359	,	0.300072222	,],
[	0.316811001	,	0.374160586	,	0.280232258	,	0.085612943	,]],
[[	0.086275238	,	0.293792676	,	0.301348546	,	0.242819203	,],
[	0.295421926	,	0.085852292	,	0.234087845	,	0.31147736	,],
[	0.382421293	,	0.31578864	,	0.071522832	,	0.296294042	,],
[	0.318547638	,	0.382467823	,	0.288475637	,	0.087874873	,]],
[[	0.087686866	,	0.297754785	,	0.3054658	,	0.246676965	,],
[	0.295252302	,	0.081526003	,	0.233484332	,	0.311362787	,],
[	0.381342303	,	0.314951552	,	0.072490484	,	0.295038384	,],
[	0.313981055	,	0.377880165	,	0.283583104	,	0.084105226	,]],
[[	0.084475494	,	0.293287111	,	0.300896744	,	0.242387808	,],
[	0.293701243	,	0.082112167	,	0.232116512	,	0.309870364	,],
[	0.385272757	,	0.319456887	,	0.079700762	,	0.298929164	,],
[	0.315716628	,	0.379303034	,	0.284787719	,	0.087705636	,]],
[[	0.089236703	,	0.295085863	,	0.302556489	,	0.244283531	,],
[	0.297535418	,	0.08941271	,	0.236271482	,	0.313400283	,],
[	0.38150185	,	0.314947494	,	0.071202022	,	0.295199497	,],
[	0.316163964	,	0.379905353	,	0.285716212	,	0.085556551	,]],
[[	0.085071614	,	0.295085529	,	0.302853026	,	0.243996397	,],
[	0.293943294	,	0.080756707	,	0.232101308	,	0.310051589	,],
[	0.382655199	,	0.316394684	,	0.075015352	,	0.296327306	,],
[	0.314161749	,	0.377863832	,	0.283327177	,	0.084767746	,]],
[[	0.085472801	,	0.293390167	,	0.301000046	,	0.242339455	,],
[	0.294839869	,	0.084620432	,	0.233291155	,	0.310834448	,],
[	0.383372426	,	0.316671173	,	0.072438418	,	0.297202351	,],
[	0.317929663	,	0.381180035	,	0.286705036	,	0.091134335	,]],
[[	0.088446241	,	0.298595168	,	0.306276191	,	0.247691822	,],
[	0.295815411	,	0.081826958	,	0.23409696	,	0.311947441	,],
[	0.381390299	,	0.31489062	,	0.071997653	,	0.294993554	,],
[	0.314580962	,	0.378420894	,	0.284183567	,	0.084258831	,]],
[[	0.084355948	,	0.293594245	,	0.301102132	,	0.242570837	,],
[	0.293489979	,	0.081392291	,	0.231758248	,	0.309536406	,],
[	0.384503868	,	0.318461397	,	0.078247662	,	0.29805942	,],
[	0.314964262	,	0.378662214	,	0.284106173	,	0.086450006	,]],
[[	0.088146586	,	0.294597993	,	0.302047352	,	0.243668186	,],
[	0.296732151	,	0.087996685	,	0.235393787	,	0.312493997	,],
[	0.38208799	,	0.315480625	,	0.071511614	,	0.295758868	,],
[	0.316697984	,	0.380491457	,	0.2862786	,	0.085909757	,]],
[[	0.085673285	,	0.295818755	,	0.30346024	,	0.244670792	,],
[	0.294262146	,	0.080898487	,	0.232335121	,	0.310342139	,],
[	0.381993988	,	0.315734501	,	0.073776096	,	0.29559308	,],
[	0.313815279	,	0.377554543	,	0.283077305	,	0.084093447	,]],
[[	0.087305375	,	0.300057367	,	0.306711496	,	0.243681445	,],
[	0.290639953	,	0.087476329	,	0.235044983	,	0.307969654	,],
[	0.377248889	,	0.316392583	,	0.071074413	,	0.291524648	,],
[	0.315709735	,	0.385096544	,	0.290820735	,	0.086026384	,]],
[[	0.085899993	,	0.301864616	,	0.308510491	,	0.245384617	,],
[	0.289210222	,	0.081184614	,	0.233005078	,	0.306693055	,],
[	0.377086559	,	0.31657374	,	0.073089406	,	0.291179685	,],
[	0.312678125	,	0.381937519	,	0.287557442	,	0.08419993	,]],
[[	0.084508499	,	0.298792632	,	0.305346744	,	0.242373467	,],
[	0.288535038	,	0.083174087	,	0.232580031	,	0.306023705	,],
[	0.380567558	,	0.319728726	,	0.074085185	,	0.295036502	,],
[	0.31595792	,	0.384975491	,	0.290582417	,	0.089950135	,]],
[[	0.089939481	,	0.306204183	,	0.312899435	,	0.249553414	,],
[	0.291959082	,	0.083321445	,	0.235583608	,	0.309363999	,],
[	0.37659176	,	0.315863907	,	0.071105171	,	0.290717571	,],
[	0.31380239	,	0.382996136	,	0.288659216	,	0.084331255	,]],
[[	0.084204088	,	0.29986668	,	0.306575974	,	0.243256579	,],
[	0.288302976	,	0.081246115	,	0.23207784	,	0.305746656	,],
[	0.378653734	,	0.318291919	,	0.075816091	,	0.292615046	,],
[	0.313564276	,	0.382621238	,	0.288062987	,	0.085779586	,]],
[[	0.086331666	,	0.299531874	,	0.30617818	,	0.243003577	,],
[	0.289868709	,	0.085967697	,	0.234145264	,	0.307272379	,],
[	0.377966563	,	0.317133422	,	0.071745503	,	0.292273161	,],
[	0.316689124	,	0.38598553	,	0.291767793	,	0.086934085	,]],
[[	0.087027021	,	0.303089708	,	0.309758319	,	0.246578002	,],
[	0.289826708	,	0.081649014	,	0.233589943	,	0.307249211	,],
[	0.376961146	,	0.316289518	,	0.072332374	,	0.290960422	,],
[	0.312989955	,	0.382219872	,	0.28779536	,	0.083976455	,]],
[[	0.084484177	,	0.299164939	,	0.305805047	,	0.242641333	,],
[	0.28818719	,	0.082206924	,	0.232242194	,	0.305690827	,],
[	0.380729935	,	0.320681552	,	0.079653622	,	0.294688366	,],
[	0.315215723	,	0.384194234	,	0.289725031	,	0.088565301	,]],
[[	0.089448383	,	0.30108976	,	0.307597179	,	0.24467189	,],
[	0.292340722	,	0.089942646	,	0.236748809	,	0.309468885	,],
[	0.376875955	,	0.316087518	,	0.071096248	,	0.291048651	,],
[	0.314715504	,	0.383894424	,	0.28946797	,	0.084996485	,]],
[[	0.084887851	,	0.300709933	,	0.30738317	,	0.244084965	,],
[	0.288736711	,	0.08131366	,	0.232491445	,	0.306051787	,],
[	0.378010837	,	0.317553332	,	0.074591251	,	0.291921358	,],
[	0.313343503	,	0.382430611	,	0.287820181	,	0.08509771	,]],
[[	0.086735183	,	0.304742432	,	0.311671393	,	0.244042461	,],
[	0.283223439	,	0.083532271	,	0.232387058	,	0.301812468	,],
[	0.376628375	,	0.321255876	,	0.073804743	,	0.291952643	,],
[	0.315710657	,	0.390526219	,	0.295639229	,	0.087565292	,]],
[[	0.086337748	,	0.307355932	,	0.314239673	,	0.246634556	,],
[	0.285923476	,	0.082782351	,	0.234856662	,	0.304724381	,],
[	0.373313427	,	0.318103031	,	0.071853823	,	0.288439273	,],
[	0.311350536	,	0.386392848	,	0.290986994	,	0.083851147	,]],
[[	0.084397344	,	0.303842976	,	0.310726366	,	0.243112533	,],
[	0.282479266	,	0.08104265	,	0.231551677	,	0.301192991	,],
[	0.375324557	,	0.320597623	,	0.076945939	,	0.290392403	,],
[	0.312246849	,	0.386933771	,	0.291510916	,	0.086824808	,]],
[[	0.090243222	,	0.306661705	,	0.313389855	,	0.24602646	,],
[	0.284950072	,	0.086689567	,	0.234394062	,	0.303427547	,],
[	0.374499087	,	0.31917344	,	0.072077345	,	0.28972152	,],
[	0.313318385	,	0.388146468	,	0.293013595	,	0.085209471	,]],
[[	0.084544109	,	0.305427458	,	0.31237221	,	0.244550304	,],
[	0.283752402	,	0.081042031	,	0.232591478	,	0.302330302	,],
[	0.373683599	,	0.318700633	,	0.073315839	,	0.288740769	,],
[	0.311085909	,	0.385938268	,	0.290445068	,	0.084242521	,]],
[[	0.08581919	,	0.304280447	,	0.311222192	,	0.243528523	,],
[	0.283050941	,	0.082788138	,	0.232105565	,	0.301523535	,],
[	0.377183154	,	0.322904417	,	0.080374119	,	0.292376718	,],
[	0.315075854	,	0.389446732	,	0.294050043	,	0.090643109	,]],
[[	0.087734377	,	0.308817679	,	0.315605301	,	0.248027322	,],
[	0.287075144	,	0.083797837	,	0.235954242	,	0.305663012	,],
[	0.373553941	,	0.31832925	,	0.071918291	,	0.288712112	,],
[	0.311438411	,	0.386313295	,	0.291094705	,	0.083647941	,]],
[[	0.084227948	,	0.30418526	,	0.311012119	,	0.243315457	,],
[	0.282826293	,	0.080958033	,	0.231739175	,	0.301439126	,],
[	0.374734363	,	0.319982851	,	0.075720039	,	0.289850154	,],
[	0.312132775	,	0.386918249	,	0.29144998	,	0.086105519	,]],
[[	0.088961266	,	0.305911517	,	0.312715888	,	0.245287221	,],
[	0.284231196	,	0.08534995	,	0.233531075	,	0.302681511	,],
[	0.375503342	,	0.320147229	,	0.072978057	,	0.290700051	,],
[	0.314231757	,	0.389051294	,	0.293868661	,	0.085873822	,]],
[[	0.085247226	,	0.306154861	,	0.3131005	,	0.245252107	,],
[	0.284639845	,	0.081640151	,	0.233423928	,	0.303164524	,],
[	0.37360325	,	0.318403741	,	0.072742763	,	0.28857251	,],
[	0.310926696	,	0.385680301	,	0.290291391	,	0.083632974	,]],
[[	0.089066218	,	0.319404932	,	0.323267573	,	0.249344253	,],
[	0.286776038	,	0.085719315	,	0.238661894	,	0.304703081	,],
[	0.382742998	,	0.333583868	,	0.081312959	,	0.29477278	,],
[	0.321449762	,	0.404920878	,	0.30498295	,	0.09170614	,]],
[[	0.088493136	,	0.321676823	,	0.325560433	,	0.251695906	,],
[	0.290810374	,	0.093409341	,	0.243213279	,	0.308513413	,],
[	0.37997098	,	0.329877401	,	0.074145523	,	0.292090431	,],
[	0.318602137	,	0.402267191	,	0.302510378	,	0.0852452	,]],
[[	0.086657643	,	0.318377938	,	0.322327697	,	0.248465243	,],
[	0.286967898	,	0.084295296	,	0.238630536	,	0.30490015	,],
[	0.380441143	,	0.330824087	,	0.076898428	,	0.292400044	,],
[	0.318912579	,	0.402710554	,	0.302691787	,	0.087477472	,]],
[[	0.092315037	,	0.321047953	,	0.324801536	,	0.251108746	,],
[	0.288169873	,	0.088466824	,	0.240232328	,	0.306068695	,],
[	0.382405555	,	0.332258487	,	0.076092851	,	0.294685381	,],
[	0.321699308	,	0.404974494	,	0.305503098	,	0.087805925	,]],
[[	0.086467659	,	0.319436759	,	0.323368435	,	0.249297364	,],
[	0.289537858	,	0.085664783	,	0.240919416	,	0.307260395	,],
[	0.37961943	,	0.329636684	,	0.074479667	,	0.291505997	,],
[	0.318152661	,	0.401808992	,	0.301773137	,	0.085288183	,]],
[[	0.088119869	,	0.318899509	,	0.32281133	,	0.248781218	,],
[	0.286778416	,	0.085248073	,	0.238535745	,	0.304572883	,],
[	0.382161664	,	0.33290459	,	0.080079488	,	0.294032838	,],
[	0.320653089	,	0.404230748	,	0.304152006	,	0.090317679	,]],
[[	0.089552062	,	0.322789179	,	0.32665536	,	0.252841864	,],
[	0.289852674	,	0.091937673	,	0.242161158	,	0.307590682	,],
[	0.380789068	,	0.330610073	,	0.074676557	,	0.292771616	,],
[	0.319412336	,	0.403010376	,	0.303290733	,	0.085796919	,]],
[[	0.086268182	,	0.318343488	,	0.322246061	,	0.248271543	,],
[	0.28736009	,	0.084426106	,	0.238951141	,	0.305206476	,],
[	0.380175898	,	0.330374307	,	0.075973804	,	0.292026909	,],
[	0.318452375	,	0.402217849	,	0.302105557	,	0.086487799	,]],
[[	0.091072152	,	0.32053991	,	0.324344681	,	0.250412019	,],
[	0.287561979	,	0.087290576	,	0.2395158	,	0.305349967	,],
[	0.383850185	,	0.333679518	,	0.077439168	,	0.296042935	,],
[	0.322933653	,	0.406271968	,	0.306712927	,	0.089087263	,]],
[[	0.087546742	,	0.320498785	,	0.324493091	,	0.250385672	,],
[	0.290595205	,	0.086722324	,	0.242005514	,	0.308266751	,],
[	0.379683126	,	0.329672889	,	0.074226059	,	0.291564163	,],
[	0.318455718	,	0.40205807	,	0.302079584	,	0.085371902	,]],
[[	0.087666703	,	0.325872792	,	0.328737854	,	0.249987174	,],
[	0.281336006	,	0.085080925	,	0.239238492	,	0.30156115	,],
[	0.377984974	,	0.334417982	,	0.078064173	,	0.290324858	,],
[	0.318902266	,	0.408923423	,	0.307989792	,	0.088902836	,]],
[[	0.089191994	,	0.329659293	,	0.332573375	,	0.253978498	,],
[	0.283385166	,	0.090367754	,	0.241580014	,	0.303438328	,],
[	0.378791048	,	0.33453118	,	0.075614995	,	0.29137246	,],
[	0.319605241	,	0.409551385	,	0.308995483	,	0.087023579	,]],
[[	0.085616865	,	0.325201263	,	0.32810516	,	0.249279348	,],
[	0.283076969	,	0.085549177	,	0.24071009	,	0.303200766	,],
[	0.376623561	,	0.332737537	,	0.074886843	,	0.288933884	,],
[	0.317758956	,	0.40786693	,	0.306933227	,	0.086247075	,]],
[[	0.090862435	,	0.327623816	,	0.330434498	,	0.251745241	,],
[	0.281358811	,	0.086239262	,	0.239438797	,	0.301558181	,],
[	0.379894254	,	0.33659324	,	0.081377948	,	0.292177361	,],
[	0.321122063	,	0.41107194	,	0.310101022	,	0.092300532	,]],
[[	0.086771398	,	0.327189705	,	0.330123584	,	0.25130066	,],
[	0.285144323	,	0.093879532	,	0.243761934	,	0.305203298	,],
[	0.377426789	,	0.333253559	,	0.074501347	,	0.289789145	,],
[	0.318131647	,	0.408003139	,	0.307220543	,	0.085747003	,]],
[[	0.086993807	,	0.32565228	,	0.328563406	,	0.249699376	,],
[	0.281610094	,	0.084925704	,	0.239376685	,	0.301635898	,],
[	0.37731919	,	0.33372126	,	0.076792943	,	0.289595086	,],
[	0.318538669	,	0.408479281	,	0.307388342	,	0.087985376	,]],
[[	0.090373911	,	0.330914258	,	0.333693468	,	0.255098486	,],
[	0.282764338	,	0.089012549	,	0.240895416	,	0.302748262	,],
[	0.379980102	,	0.335634615	,	0.076654725	,	0.29246388	,],
[	0.320717887	,	0.410537841	,	0.309993996	,	0.087923766	,]],
[[	0.086007177	,	0.325750472	,	0.32857598	,	0.249819148	,],
[	0.284161369	,	0.08645563	,	0.241694083	,	0.304144586	,],
[	0.376604354	,	0.332573424	,	0.074558597	,	0.288865756	,],
[	0.317675099	,	0.407714034	,	0.306774006	,	0.085802217	,]],
[[	0.089525057	,	0.326923589	,	0.329612122	,	0.250855352	,],
[	0.281552613	,	0.085707158	,	0.239346667	,	0.301518822	,],
[	0.379241089	,	0.335787227	,	0.080066842	,	0.291413122	,],
[	0.320334881	,	0.410226059	,	0.309222803	,	0.090840783	,]],
[[	0.087891638	,	0.328338393	,	0.331272002	,	0.252478224	,],
[	0.284481314	,	0.092355215	,	0.242807618	,	0.304400768	,],
[	0.377599792	,	0.333310947	,	0.074412005	,	0.289965567	,],
[	0.319124902	,	0.409047575	,	0.308340969	,	0.086458699	,]],
[[	0.087855588	,	0.331368143	,	0.334443252	,	0.251373712	,],
[	0.276203585	,	0.085196844	,	0.240004424	,	0.297700716	,],
[	0.373765479	,	0.33695851	,	0.079293359	,	0.287750151	,],
[	0.318783146	,	0.414440449	,	0.313298377	,	0.090985457	,]],
[[	0.087904034	,	0.333857458	,	0.336980203	,	0.25407218	,],
[	0.277874127	,	0.090188029	,	0.242142618	,	0.299353081	,],
[	0.374552442	,	0.336769472	,	0.076579861	,	0.28864481	,],
[	0.317000693	,	0.412705687	,	0.31186101	,	0.085899906	,]],
[[	0.085504688	,	0.330306244	,	0.333355468	,	0.250344371	,],
[	0.277506541	,	0.085383706	,	0.241187674	,	0.298990229	,],
[	0.372789525	,	0.335360936	,	0.076448206	,	0.2866396	,],
[	0.316791228	,	0.412661287	,	0.311380098	,	0.0872398	,]],
[[	0.090925281	,	0.332934786	,	0.33581024	,	0.252913829	,],
[	0.276248535	,	0.086464013	,	0.240211415	,	0.297746243	,],
[	0.376030442	,	0.339350815	,	0.082971341	,	0.289923251	,],
[	0.320102383	,	0.415594233	,	0.314948607	,	0.088568346	,]],
[[	0.085750741	,	0.331681182	,	0.334838255	,	0.251724584	,],
[	0.279912425	,	0.093957581	,	0.244633031	,	0.301391945	,],
[	0.373247413	,	0.335525344	,	0.07557366	,	0.287149698	,],
[	0.315983673	,	0.411742799	,	0.310691512	,	0.085345638	,]],
[[	0.086726521	,	0.330700447	,	0.333732582	,	0.250653499	,],
[	0.276365442	,	0.084960069	,	0.23999628	,	0.297787711	,],
[	0.373592664	,	0.336515043	,	0.078332923	,	0.287370719	,],
[	0.31821193	,	0.413974547	,	0.31278635	,	0.089913981	,]],
[[	0.088956641	,	0.334996503	,	0.338099418	,	0.255240955	,],
[	0.27726776	,	0.08892701	,	0.241450622	,	0.298626542	,],
[	0.375258378	,	0.337605644	,	0.077274908	,	0.289432588	,],
[	0.317661516	,	0.413342573	,	0.3125321	,	0.086256114	,]],
[[	0.085490532	,	0.330649478	,	0.33354757	,	0.250571909	,],
[	0.27858672	,	0.086059112	,	0.242083382	,	0.299976734	,],
[	0.372668151	,	0.335135608	,	0.075798157	,	0.28641712	,],
[	0.316083339	,	0.411890726	,	0.310710257	,	0.086174944	,]],
[[	0.089722093	,	0.332277549	,	0.335196628	,	0.252171445	,],
[	0.276070266	,	0.085759282	,	0.239925581	,	0.297455997	,],
[	0.375285386	,	0.338466427	,	0.081529388	,	0.289039206	,],
[	0.320205776	,	0.415970723	,	0.314781802	,	0.093150598	,]],
[[	0.086809136	,	0.33271918	,	0.335875005	,	0.252784363	,],
[	0.27905321	,	0.092370267	,	0.243417053	,	0.300432989	,],
[	0.373408849	,	0.335754202	,	0.075753999	,	0.28740046	,],
[	0.31663472	,	0.41225063	,	0.311307555	,	0.085662856	,]],
[[	0.087718052	,	0.338301829	,	0.339341056	,	0.252372614	,],
[	0.272674409	,	0.087016348	,	0.241436053	,	0.294976215	,],
[	0.370819407	,	0.340041261	,	0.078983946	,	0.285171697	,],
[	0.317549474	,	0.420208467	,	0.316747302	,	0.089842332	,]],
[[	0.089034412	,	0.341939836	,	0.343107029	,	0.256242022	,],
[	0.273267178	,	0.090610549	,	0.242562795	,	0.295558094	,],
[	0.371828338	,	0.340270866	,	0.076482192	,	0.286253158	,],
[	0.317574082	,	0.420174352	,	0.317037019	,	0.086930233	,]],
[[	0.085912142	,	0.337692827	,	0.33865018	,	0.25181384	,],
[	0.274895312	,	0.088220612	,	0.243567063	,	0.297237168	,],
[	0.369715626	,	0.338450312	,	0.075869984	,	0.283942988	,],
[	0.315867611	,	0.418502973	,	0.315249292	,	0.08666433	,]],
[[	0.090841702	,	0.340050554	,	0.340972383	,	0.254108028	,],
[	0.272187702	,	0.087566826	,	0.241236175	,	0.294564547	,],
[	0.372643391	,	0.342205164	,	0.082217861	,	0.286989126	,],
[	0.319785062	,	0.422438451	,	0.31902285	,	0.093222976	,]],
[[	0.086878884	,	0.339643125	,	0.340799779	,	0.253850121	,],
[	0.275043187	,	0.094115895	,	0.244709876	,	0.297291894	,],
[	0.370280507	,	0.338875483	,	0.075398677	,	0.284683306	,],
[	0.316278866	,	0.418738389	,	0.315569545	,	0.085996672	,]],
[[	0.087117165	,	0.338101192	,	0.339214998	,	0.252169262	,],
[	0.273105687	,	0.086912344	,	0.241705435	,	0.295410732	,],
[	0.370560055	,	0.339476603	,	0.077789414	,	0.284653632	,],
[	0.317068342	,	0.419698018	,	0.316216856	,	0.088752462	,]],
[[	0.089959214	,	0.342818592	,	0.34384185	,	0.257126797	,],
[	0.273009843	,	0.089674625	,	0.242113363	,	0.295201772	,],
[	0.372908465	,	0.34121457	,	0.077306824	,	0.28734146	,],
[	0.318622544	,	0.421073422	,	0.318099587	,	0.087858001	,]],
[[	0.085912835	,	0.338065558	,	0.339093159	,	0.252186196	,],
[	0.275802662	,	0.088868216	,	0.244380687	,	0.298039548	,],
[	0.369584362	,	0.338290357	,	0.075331337	,	0.283793034	,],
[	0.315612901	,	0.418368849	,	0.315011231	,	0.08608938	,]],
[[	0.089773039	,	0.339474218	,	0.340371847	,	0.253433103	,],
[	0.272472227	,	0.087260765	,	0.241240002	,	0.294773575	,],
[	0.371965554	,	0.341287219	,	0.080864946	,	0.28618347	,],
[	0.318925623	,	0.42162925	,	0.318102135	,	0.091927711	,]],
[[	0.087585515	,	0.340497343	,	0.341591894	,	0.254681312	,],
[	0.274366803	,	0.092631255	,	0.243719196	,	0.296499264	,],
[	0.370730747	,	0.339256838	,	0.075514221	,	0.285090748	,],
[	0.316910758	,	0.419378204	,	0.316119791	,	0.086294326	,]],
[[	0.087541918	,	0.342927398	,	0.342154424	,	0.252502435	,],
[	0.267663937	,	0.087133659	,	0.24043497	,	0.291512639	,],
[	0.36530835	,	0.340188704	,	0.07668464	,	0.28039688	,],
[	0.315906675	,	0.423639642	,	0.318943727	,	0.089152335	,]],
[[	0.088319449	,	0.34604946	,	0.345255693	,	0.255909903	,],
[	0.267183341	,	0.089344142	,	0.240400567	,	0.291032348	,],
[	0.36838112	,	0.342572449	,	0.077058699	,	0.283760086	,],
[	0.316230693	,	0.423765571	,	0.319705068	,	0.086708491	,]],
[[	0.085662134	,	0.342370434	,	0.341573893	,	0.251966994	,],
[	0.270635909	,	0.089186936	,	0.243188684	,	0.294408782	,],
[	0.364531126	,	0.339014147	,	0.074259547	,	0.279615583	,],
[	0.314478863	,	0.422198654	,	0.317710174	,	0.086100067	,]],
[[	0.090869383	,	0.344787643	,	0.343794276	,	0.254295734	,],
[	0.266701704	,	0.08715422	,	0.239749561	,	0.290601937	,],
[	0.367262731	,	0.342420868	,	0.080010612	,	0.282260005	,],
[	0.317837093	,	0.42549254	,	0.320971394	,	0.092296374	,]],
[[	0.086137788	,	0.343696686	,	0.343069681	,	0.253400437	,],
[	0.268395053	,	0.092287173	,	0.242001072	,	0.29234105	,],
[	0.366212312	,	0.340466401	,	0.075082553	,	0.281375736	,],
[	0.315017632	,	0.422452299	,	0.318109921	,	0.085644901	,]],
[[	0.086805854	,	0.342555772	,	0.341833905	,	0.252009735	,],
[	0.268489025	,	0.087623199	,	0.241223911	,	0.292261345	,],
[	0.364908065	,	0.33967617	,	0.075706495	,	0.279973407	,],
[	0.31516429	,	0.422788504	,	0.318257816	,	0.087939812	,]],
[[	0.089308816	,	0.34709587	,	0.346338649	,	0.2568295	,],
[	0.26674275	,	0.088338597	,	0.239849737	,	0.290516725	,],
[	0.369195138	,	0.344799285	,	0.083615379	,	0.284249579	,],
[	0.317448076	,	0.424810242	,	0.320776453	,	0.087776738	,]],
[[	0.085732943	,	0.342667521	,	0.341823946	,	0.252262748	,],
[	0.270959945	,	0.096370878	,	0.244644475	,	0.294481429	,],
[	0.364894163	,	0.339281207	,	0.0742702	,	0.279899189	,],
[	0.314055531	,	0.421773088	,	0.31727102	,	0.085432181	,]],
[[	0.089285854	,	0.34385353	,	0.342924046	,	0.253289164	,],
[	0.267068272	,	0.087086631	,	0.239889069	,	0.290804537	,],
[	0.366079812	,	0.341121347	,	0.07828053	,	0.281022146	,],
[	0.317417778	,	0.425007886	,	0.320487316	,	0.091341842	,]],
[[	0.087086146	,	0.344767591	,	0.343976641	,	0.254301086	,],
[	0.26787802	,	0.091140166	,	0.241334185	,	0.291716745	,],
[	0.366916525	,	0.341122587	,	0.07548121	,	0.282090764	,],
[	0.31542071	,	0.422893052	,	0.318565319	,	0.085880189	,]],
[[	0.08773378	,	0.350066741	,	0.348353958	,	0.253481336	,],
[	0.263152536	,	0.088301111	,	0.242044756	,	0.287912758	,],
[	0.361873516	,	0.343329388	,	0.077213171	,	0.277277346	,],
[	0.314748441	,	0.429031346	,	0.324098771	,	0.089018987	,]],
[[	0.08903573	,	0.353321057	,	0.351767649	,	0.257112915	,],
[	0.261642776	,	0.089281071	,	0.240796428	,	0.286373767	,],
[	0.365944414	,	0.348031851	,	0.084587676	,	0.281422479	,],
[	0.315361261	,	0.429463877	,	0.324910658	,	0.086889473	,]],
[[	0.085653597	,	0.349006912	,	0.347473945	,	0.252613611	,],
[	0.265481871	,	0.097077188	,	0.245352381	,	0.290069211	,],
[	0.361783971	,	0.342850884	,	0.075744926	,	0.277193891	,],
[	0.313199895	,	0.427521099	,	0.322655506	,	0.085998425	,]],
[[	0.090911085	,	0.351545171	,	0.349844997	,	0.255015019	,],
[	0.26170308	,	0.087747844	,	0.240753797	,	0.286551145	,],
[	0.363075134	,	0.34484566	,	0.079622264	,	0.278377128	,],
[	0.316782678	,	0.431089536	,	0.326152198	,	0.092284879	,]],
[[	0.086603881	,	0.350968506	,	0.349435025	,	0.254523326	,],
[	0.262710019	,	0.092041449	,	0.242217578	,	0.287492764	,],
[	0.363998299	,	0.344946448	,	0.077106894	,	0.279613548	,],
[	0.31381163	,	0.427884525	,	0.323135712	,	0.085489999	,]],
[[	0.086888297	,	0.349520199	,	0.347972071	,	0.252961734	,],
[	0.263963899	,	0.088842132	,	0.242582785	,	0.288635614	,],
[	0.361778729	,	0.342998657	,	0.076615626	,	0.277172273	,],
[	0.314252677	,	0.428438314	,	0.323447794	,	0.087916445	,]],
[[	0.089968558	,	0.354342334	,	0.352651313	,	0.258125942	,],
[	0.261654537	,	0.08877188	,	0.240706941	,	0.286337653	,],
[	0.365200158	,	0.347297107	,	0.083289354	,	0.280584164	,],
[	0.316340881	,	0.430311343	,	0.325806956	,	0.087801213	,]],
[[	0.085956648	,	0.349631903	,	0.347936487	,	0.253179212	,],
[	0.264777666	,	0.095592103	,	0.244438433	,	0.289328371	,],
[	0.362174617	,	0.343196652	,	0.075768757	,	0.277649105	,],
[	0.313035628	,	0.427296201	,	0.32246042	,	0.085472135	,]],
[[	0.089425119	,	0.350600803	,	0.348963151	,	0.254042919	,],
[	0.262026428	,	0.087612448	,	0.240968856	,	0.286797311	,],
[	0.362713945	,	0.344270731	,	0.07868524	,	0.277982489	,],
[	0.31601457	,	0.430206838	,	0.325063612	,	0.090808466	,]],
[[	0.087314146	,	0.351704207	,	0.350230742	,	0.255433611	,],
[	0.262234429	,	0.090669457	,	0.241552893	,	0.286923937	,],
[	0.364993839	,	0.345810497	,	0.07800863	,	0.280502114	,],
[	0.314527907	,	0.428505575	,	0.323909963	,	0.086120508	,]],
[[	0.087320437	,	0.354821132	,	0.352641493	,	0.253083442	,],
[	0.257892511	,	0.088661641	,	0.242560698	,	0.283448142	,],
[	0.357682878	,	0.345091723	,	0.07748097	,	0.273625288	,],
[	0.313110588	,	0.432603331	,	0.326991384	,	0.088307842	,]],
[[	0.088529706	,	0.358120148	,	0.356003769	,	0.256781937	,],
[	0.255707486	,	0.08880093	,	0.240743822	,	0.281201913	,],
[	0.361840651	,	0.348667905	,	0.079290474	,	0.278087144	,],
[	0.314149379	,	0.433515735	,	0.32819649	,	0.086676046	,]],
[[	0.085503573	,	0.354135196	,	0.35198366	,	0.252604828	,],
[	0.259094658	,	0.096185718	,	0.244748858	,	0.284512877	,],
[	0.357501505	,	0.3445578	,	0.075770497	,	0.273373816	,],
[	0.311307048	,	0.430973889	,	0.325387854	,	0.085177491	,]],
[[	0.090413615	,	0.356522968	,	0.354138136	,	0.254773971	,],
[	0.256068623	,	0.087718536	,	0.240954857	,	0.281698486	,],
[	0.358889693	,	0.346573891	,	0.079988433	,	0.274747852	,],
[	0.314968379	,	0.434513408	,	0.328802223	,	0.091347038	,]],
[[	0.085938653	,	0.355541078	,	0.353473516	,	0.254017602	,],
[	0.256083251	,	0.090886477	,	0.241532303	,	0.28169013	,],
[	0.359328742	,	0.346257506	,	0.076884231	,	0.275470611	,],
[	0.312263632	,	0.431530027	,	0.326101468	,	0.084985422	,]],
[[	0.086450076	,	0.354456057	,	0.352298241	,	0.252682624	,],
[	0.258630026	,	0.089318106	,	0.243281755	,	0.284105059	,],
[	0.357632188	,	0.344951748	,	0.076915842	,	0.273502697	,],
[	0.312367918	,	0.431943619	,	0.326123288	,	0.087026697	,]],
[[	0.089833216	,	0.359439453	,	0.357181299	,	0.258050879	,],
[	0.255215187	,	0.087975233	,	0.240225415	,	0.280811432	,],
[	0.360954333	,	0.348953319	,	0.083537295	,	0.276772378	,],
[	0.315114053	,	0.434344759	,	0.329160382	,	0.087447879	,]],
[[	0.085515791	,	0.354601344	,	0.352300575	,	0.252943363	,],
[	0.25823664	,	0.094509389	,	0.243744301	,	0.283596902	,],
[	0.357837096	,	0.344817913	,	0.075827895	,	0.273763172	,],
[	0.311733755	,	0.431348159	,	0.325742133	,	0.085096946	,]],
[[	0.088939275	,	0.355515444	,	0.353267994	,	0.253872165	,],
[	0.256552142	,	0.087715085	,	0.241211461	,	0.281974012	,],
[	0.358160382	,	0.345757016	,	0.078770045	,	0.273941203	,],
[	0.314464935	,	0.434022321	,	0.328338762	,	0.090179155	,]],
[[	0.087253966	,	0.356788104	,	0.354703562	,	0.255262672	,],
[	0.256002864	,	0.090052675	,	0.241189859	,	0.281508766	,],
[	0.360332302	,	0.347095118	,	0.077749088	,	0.276392546	,],
[	0.313087019	,	0.432353577	,	0.326990094	,	0.085610519	,]],
[[	0.087509979	,	0.36090962	,	0.357843679	,	0.253854677	,],
[	0.254295322	,	0.090237385	,	0.244734665	,	0.280981842	,],
[	0.355016322	,	0.348894231	,	0.078262427	,	0.271099356	,],
[	0.313329735	,	0.439453839	,	0.33256291	,	0.089189812	,]],
[[	0.087945904	,	0.363675399	,	0.360459564	,	0.256860299	,],
[	0.251378214	,	0.089472949	,	0.242279121	,	0.278087285	,],
[	0.358855012	,	0.353467511	,	0.085813813	,	0.274974123	,],
[	0.313439806	,	0.439299522	,	0.332876288	,	0.08652691	,]],
[[	0.085496946	,	0.360190554	,	0.35703459	,	0.253128048	,],
[	0.254036134	,	0.095906961	,	0.245571974	,	0.280633061	,],
[	0.354792631	,	0.348329402	,	0.076706159	,	0.270952956	,],
[	0.31134807	,	0.437611268	,	0.33079967	,	0.085888575	,]],
[[	0.090942021	,	0.362751585	,	0.359405451	,	0.255540598	,],
[	0.252072096	,	0.088638314	,	0.242695989	,	0.278706559	,],
[	0.356285374	,	0.350412733	,	0.080838756	,	0.272286775	,],
[	0.315616286	,	0.44184697	,	0.33499878	,	0.092796101	,]],
[[	0.085615833	,	0.361421162	,	0.358361679	,	0.254391815	,],
[	0.251763522	,	0.091366964	,	0.242944865	,	0.278443424	,],
[	0.356989027	,	0.350311925	,	0.078074933	,	0.273281558	,],
[	0.311891611	,	0.437879442	,	0.331252915	,	0.085345517	,]],
[[	0.086746031	,	0.36057041	,	0.35753119	,	0.253363141	,],
[	0.255262338	,	0.09102736	,	0.245561323	,	0.281705555	,],
[	0.35484646	,	0.348549873	,	0.077656495	,	0.270840308	,],
[	0.312693482	,	0.438859521	,	0.331920561	,	0.088082727	,]],
[[	0.089136106	,	0.364882082	,	0.361649465	,	0.258085777	,],
[	0.251478846	,	0.089136426	,	0.242173776	,	0.278011286	,],
[	0.358087298	,	0.352583769	,	0.084318998	,	0.274050394	,],
[	0.314318382	,	0.440176976	,	0.333739307	,	0.087258525	,]],
[[	0.085300355	,	0.360408874	,	0.357141718	,	0.253294137	,],
[	0.253458607	,	0.094627896	,	0.244731145	,	0.279988675	,],
[	0.355599247	,	0.348919845	,	0.077020729	,	0.271619111	,],
[	0.31134243	,	0.437427439	,	0.330764902	,	0.085447355	,]],
[[	0.089450567	,	0.362035944	,	0.358683303	,	0.254829009	,],
[	0.253035104	,	0.089318475	,	0.243496701	,	0.27956486	,],
[	0.355699073	,	0.349706023	,	0.079747302	,	0.271659393	,],
[	0.31462388	,	0.440785686	,	0.333865543	,	0.091182964	,]],
[[	0.086711453	,	0.36247929	,	0.359286856	,	0.255409647	,],
[	0.251553436	,	0.090454421	,	0.242629472	,	0.278143213	,],
[	0.357676487	,	0.350867618	,	0.078697119	,	0.273959483	,],
[	0.312531479	,	0.438487664	,	0.331867517	,	0.085753493	,]],
[[	0.101317534	,	0.232928651	,	0.261651529	,	0.27091758	,],
[	0.387225058	,	0.080490699	,	0.250104544	,	0.396301036	,],
[	0.460961012	,	0.30406942	,	0.074777424	,	0.366371884	,],
[	0.357936786	,	0.33133991	,	0.246769581	,	0.099804193	,]],
[[	0.096112555	,	0.226494707	,	0.255307509	,	0.264963366	,],
[	0.388657084	,	0.084423089	,	0.251705349	,	0.397776876	,],
[	0.463664511	,	0.307196131	,	0.075834449	,	0.369599169	,],
[	0.354855735	,	0.328768151	,	0.244330273	,	0.093270632	,]],
[[	0.098763702	,	0.226451794	,	0.255368614	,	0.265399566	,],
[	0.38939082	,	0.082032501	,	0.252524319	,	0.398490845	,],
[	0.460091958	,	0.303247475	,	0.072787042	,	0.36555675	,],
[	0.355394766	,	0.328858517	,	0.244240365	,	0.095517164	,]],
[[	0.098046022	,	0.229715914	,	0.2584688	,	0.267819143	,],
[	0.387432161	,	0.081642277	,	0.250395587	,	0.396536736	,],
[	0.462207046	,	0.305386839	,	0.077130823	,	0.367486627	,],
[	0.357541094	,	0.331691333	,	0.247354446	,	0.095470022	,]],
[[	0.095905331	,	0.225711165	,	0.254723677	,	0.264216964	,],
[	0.390534243	,	0.087531046	,	0.253625706	,	0.399509163	,],
[	0.461578699	,	0.304850475	,	0.07362517	,	0.367262749	,],
[	0.35456207	,	0.328382638	,	0.243865155	,	0.093462132	,]],
[[	0.101873578	,	0.228353269	,	0.257228947	,	0.267300789	,],
[	0.387801068	,	0.08072465	,	0.250649714	,	0.39687408	,],
[	0.460368778	,	0.303463345	,	0.073687479	,	0.365667976	,],
[	0.357020516	,	0.330464646	,	0.245949471	,	0.098351564	,]],
[[	0.096471463	,	0.227294065	,	0.255909483	,	0.265578505	,],
[	0.388140835	,	0.08325552	,	0.251044677	,	0.397091515	,],
[	0.464572669	,	0.308091419	,	0.081015092	,	0.369953801	,],
[	0.355643119	,	0.329494478	,	0.244986293	,	0.0937524	,]],
[[	0.097820154	,	0.22625162	,	0.255004712	,	0.264949655	,],
[	0.390450501	,	0.083028331	,	0.253400791	,	0.399377571	,],
[	0.460401957	,	0.303502926	,	0.072759561	,	0.365752817	,],
[	0.355078249	,	0.328437022	,	0.243941357	,	0.094719366	,]],
[[	0.099240911	,	0.230774935	,	0.259487045	,	0.26880331	,],
[	0.387046343	,	0.080815436	,	0.249979301	,	0.396110778	,],
[	0.46166523	,	0.304844304	,	0.076076583	,	0.366883952	,],
[	0.358610685	,	0.332816016	,	0.248477153	,	0.096559838	,]],
[[	0.096292669	,	0.2263404	,	0.255211824	,	0.264791775	,],
[	0.389816891	,	0.086288855	,	0.252816272	,	0.398794109	,],
[	0.462266079	,	0.305594909	,	0.07429542	,	0.367889409	,],
[	0.354459651	,	0.328300271	,	0.243759688	,	0.092963328	,]],
[[	0.100426109	,	0.231849	,	0.25998181	,	0.265054064	,],
[	0.380889042	,	0.081933252	,	0.250639559	,	0.390696273	,],
[	0.451618499	,	0.301327559	,	0.072237635	,	0.359247932	,],
[	0.34974329	,	0.329429749	,	0.24603653	,	0.094642671	,]],
[[	0.094647948	,	0.230353004	,	0.258374113	,	0.262970829	,],
[	0.37865475	,	0.081075998	,	0.248226779	,	0.388512613	,],
[	0.453598177	,	0.303498316	,	0.07645993	,	0.361252306	,],
[	0.351944466	,	0.331977566	,	0.248854282	,	0.094585301	,]],
[[	0.096232696	,	0.229428801	,	0.257500089	,	0.262398083	,],
[	0.381941087	,	0.087330153	,	0.251800063	,	0.391684912	,],
[	0.453345648	,	0.30312317	,	0.073482368	,	0.361104249	,],
[	0.348930703	,	0.328551907	,	0.245192121	,	0.092454966	,]],
[[	0.097320352	,	0.234005356	,	0.261888551	,	0.266221504	,],
[	0.379532902	,	0.08092921	,	0.24917568	,	0.389291404	,],
[	0.451746079	,	0.301393451	,	0.072952442	,	0.359268686	,],
[	0.351633583	,	0.330975441	,	0.247558637	,	0.097623298	,]],
[[	0.094218704	,	0.229172428	,	0.257259124	,	0.261826066	,],
[	0.379818779	,	0.08331883	,	0.249467921	,	0.389560427	,],
[	0.455495184	,	0.3054712	,	0.07973731	,	0.363016552	,],
[	0.349820267	,	0.329968804	,	0.246608622	,	0.092710447	,]],
[[	0.098533586	,	0.230725186	,	0.258806739	,	0.263775925	,],
[	0.382019456	,	0.08286912	,	0.251683343	,	0.391616073	,],
[	0.452194404	,	0.301877291	,	0.072415373	,	0.359767938	,],
[	0.349247789	,	0.328940719	,	0.245548542	,	0.093712328	,]],
[[	0.09565332	,	0.231438925	,	0.259359693	,	0.263870502	,],
[	0.378861182	,	0.080950686	,	0.248479769	,	0.388632747	,],
[	0.452796724	,	0.302616524	,	0.075159235	,	0.36026959	,],
[	0.353041572	,	0.333229154	,	0.249964224	,	0.095522463	,]],
[[	0.0952263	,	0.229032566	,	0.256996805	,	0.261797134	,],
[	0.380893263	,	0.085878932	,	0.250677129	,	0.390581034	,],
[	0.454297711	,	0.304094188	,	0.074406611	,	0.362025461	,],
[	0.348661359	,	0.328512634	,	0.245114577	,	0.092053281	,]],
[[	0.098564558	,	0.235088896	,	0.263022593	,	0.267210249	,],
[	0.379975273	,	0.081316519	,	0.249671823	,	0.389768916	,],
[	0.451814464	,	0.301291198	,	0.072686006	,	0.3591846	,],
[	0.350718963	,	0.330032893	,	0.246594572	,	0.096019014	,]],
[[	0.094236744	,	0.229591868	,	0.25766352	,	0.262066158	,],
[	0.378791328	,	0.081811278	,	0.248327299	,	0.388526609	,],
[	0.454815084	,	0.304685142	,	0.078384674	,	0.362259257	,],
[	0.350764371	,	0.330704489	,	0.247480304	,	0.093385937	,]],
[[	0.097660229	,	0.234059549	,	0.26097231	,	0.26097495	,],
[	0.374085514	,	0.087260151	,	0.249518683	,	0.384392309	,],
[	0.445910716	,	0.302949555	,	0.072921492	,	0.354388621	,],
[	0.344542919	,	0.330966427	,	0.246766656	,	0.092091069	,]],
[[	0.093232315	,	0.233612116	,	0.260362776	,	0.259906253	,],
[	0.371383271	,	0.08022248	,	0.246531162	,	0.381720965	,],
[	0.444262871	,	0.301116592	,	0.072315047	,	0.352315961	,],
[	0.348232736	,	0.334225527	,	0.250047961	,	0.09820674	,]],
[[	0.094302719	,	0.232338471	,	0.259298969	,	0.259114631	,],
[	0.371704171	,	0.082843416	,	0.246822192	,	0.381971251	,],
[	0.44755028	,	0.304888243	,	0.078607209	,	0.35581671	,],
[	0.345094841	,	0.331540329	,	0.247352533	,	0.091528527	,]],
[[	0.096488572	,	0.237501896	,	0.264354771	,	0.263562792	,],
[	0.374013457	,	0.082486115	,	0.249224634	,	0.384225046	,],
[	0.444280314	,	0.301189496	,	0.071549399	,	0.352546514	,],
[	0.345651038	,	0.331711345	,	0.247468343	,	0.093878416	,]],
[[	0.092340436	,	0.232238777	,	0.259210727	,	0.258586319	,],
[	0.371122386	,	0.080863197	,	0.246206511	,	0.381348741	,],
[	0.445224777	,	0.302052547	,	0.074239295	,	0.353248279	,],
[	0.347134696	,	0.3338599	,	0.249955861	,	0.093323031	,]],
[[	0.09669078	,	0.23353496	,	0.260539264	,	0.260325586	,],
[	0.373373396	,	0.085848431	,	0.248590968	,	0.383611909	,],
[	0.446523088	,	0.303650949	,	0.073457445	,	0.354935325	,],
[	0.34441714	,	0.330873325	,	0.246741777	,	0.091511059	,]],
[[	0.094129109	,	0.234664131	,	0.261424833	,	0.260887426	,],
[	0.372136162	,	0.080871371	,	0.247098307	,	0.382180133	,],
[	0.444054002	,	0.300869145	,	0.071759774	,	0.352141406	,],
[	0.347409752	,	0.333452447	,	0.249238052	,	0.096823044	,]],
[[	0.093502711	,	0.232245413	,	0.259015485	,	0.258754435	,],
[	0.371369016	,	0.082055648	,	0.246447093	,	0.381595026	,],
[	0.447104917	,	0.304198367	,	0.077477274	,	0.35513672	,],
[	0.345401235	,	0.331919603	,	0.24786348	,	0.091790134	,]],
[[	0.097696778	,	0.238977247	,	0.265628797	,	0.264859137	,],
[	0.375101425	,	0.083497943	,	0.250187424	,	0.385248784	,],
[	0.44498448	,	0.301851881	,	0.071949783	,	0.353082691	,],
[	0.345299594	,	0.331438639	,	0.247191493	,	0.093130388	,]],
[[	0.092665302	,	0.232728214	,	0.259639201	,	0.25901643	,],
[	0.371134043	,	0.08034147	,	0.246094388	,	0.381245357	,],
[	0.444688092	,	0.301563402	,	0.07335128	,	0.352756158	,],
[	0.348513936	,	0.335231442	,	0.251376848	,	0.094452377	,]],
[[	0.096326638	,	0.238316434	,	0.263122062	,	0.258475327	,],
[	0.364542163	,	0.084052067	,	0.245825975	,	0.375553596	,],
[	0.441353312	,	0.304483773	,	0.075312847	,	0.35125511	,],
[	0.340426471	,	0.332923607	,	0.249213923	,	0.090865569	,]],
[[	0.092872217	,	0.238281467	,	0.263166923	,	0.258067055	,],
[	0.36478202	,	0.081055824	,	0.245917692	,	0.37584154	,],
[	0.43767009	,	0.300554342	,	0.071919946	,	0.347193188	,],
[	0.342887854	,	0.33504824	,	0.251243839	,	0.095619699	,]],
[[	0.092816987	,	0.236327247	,	0.26131672	,	0.256498367	,],
[	0.363012789	,	0.080736472	,	0.244130982	,	0.373889812	,],
[	0.439288347	,	0.302363096	,	0.075776937	,	0.348684126	,],
[	0.34145348	,	0.334036352	,	0.250473174	,	0.09108325	,]],
[[	0.096159249	,	0.242408853	,	0.2671779	,	0.261882907	,],
[	0.366562351	,	0.087487229	,	0.247971015	,	0.377521421	,],
[	0.439147542	,	0.302141859	,	0.073049138	,	0.348807197	,],
[	0.340876753	,	0.333234612	,	0.249348223	,	0.092021972	,]],
[[	0.091558186	,	0.236775371	,	0.261673651	,	0.256524174	,],
[	0.363485754	,	0.080125325	,	0.24454001	,	0.374444156	,],
[	0.437399917	,	0.300254862	,	0.072367721	,	0.346767098	,],
[	0.345027375	,	0.337790001	,	0.254309387	,	0.094269822	,]],
[[	0.095012279	,	0.237462184	,	0.262369712	,	0.257652992	,],
[	0.363956482	,	0.082787271	,	0.245185417	,	0.374959119	,],
[	0.441049918	,	0.304203069	,	0.079030494	,	0.350489717	,],
[	0.340521324	,	0.33308341	,	0.249341275	,	0.090604756	,]],
[[	0.093688981	,	0.239481903	,	0.264297738	,	0.259098247	,],
[	0.365727507	,	0.08173906	,	0.246849687	,	0.376622215	,],
[	0.43775299	,	0.300530511	,	0.071744058	,	0.347146846	,],
[	0.342123786	,	0.334367272	,	0.250537859	,	0.094279577	,]],
[[	0.092376092	,	0.236406441	,	0.261185186	,	0.256390295	,],
[	0.362773512	,	0.080165308	,	0.243749931	,	0.37369887	,],
[	0.438486799	,	0.301381378	,	0.07436669	,	0.347757805	,],
[	0.342591141	,	0.335218361	,	0.251537873	,	0.09195809	,]],
[[	0.097330635	,	0.243658002	,	0.268514467	,	0.263143672	,],
[	0.365491678	,	0.085887326	,	0.246844666	,	0.376444013	,],
[	0.440056093	,	0.303008433	,	0.073705859	,	0.349604393	,],
[	0.340669953	,	0.33299956	,	0.249171763	,	0.0913621	,]],
[[	0.092035366	,	0.237387736	,	0.262212546	,	0.257085812	,],
[	0.363838591	,	0.080126953	,	0.244762043	,	0.374629277	,],
[	0.437646254	,	0.300451976	,	0.072279134	,	0.346895444	,],
[	0.344573481	,	0.336641038	,	0.252690888	,	0.098032738	,]],
[[	0.095393348	,	0.242696252	,	0.267252453	,	0.257626719	,],
[	0.35696052	,	0.082953007	,	0.244599635	,	0.37005027	,],
[	0.434275309	,	0.303989121	,	0.078587408	,	0.345856649	,],
[	0.336127378	,	0.335436302	,	0.251927191	,	0.090462986	,]],
[[	0.091847253	,	0.242656517	,	0.267274562	,	0.25727935	,],
[	0.358992467	,	0.082170494	,	0.246370319	,	0.371877241	,],
[	0.431636268	,	0.300768449	,	0.072099556	,	0.343093995	,],
[	0.337575988	,	0.336422973	,	0.25298526	,	0.093951395	,]],
[[	0.092054019	,	0.240871967	,	0.265483911	,	0.255737354	,],
[	0.355881565	,	0.080357337	,	0.243362038	,	0.368911268	,],
[	0.432321034	,	0.301668605	,	0.07470187	,	0.343706935	,],
[	0.338131556	,	0.337428629	,	0.25397546	,	0.091785131	,]],
[[	0.095329903	,	0.246812471	,	0.271371808	,	0.261202923	,],
[	0.358317499	,	0.085757926	,	0.246067872	,	0.371286717	,],
[	0.433779615	,	0.30309178	,	0.073940277	,	0.345485422	,],
[	0.336044495	,	0.335094912	,	0.251579796	,	0.091045503	,]],
[[	0.090705081	,	0.241047188	,	0.265622073	,	0.255626061	,],
[	0.357164659	,	0.08065931	,	0.244542237	,	0.370092798	,],
[	0.430877061	,	0.300003959	,	0.071929658	,	0.342254705	,],
[	0.340046272	,	0.338620021	,	0.255108626	,	0.097512183	,]],
[[	0.094117598	,	0.241915725	,	0.266555601	,	0.256867357	,],
[	0.356500293	,	0.081960826	,	0.24399798	,	0.369493621	,],
[	0.433756602	,	0.303304606	,	0.077596644	,	0.345208973	,],
[	0.336464839	,	0.335694043	,	0.252349331	,	0.090389555	,]],
[[	0.092887572	,	0.244020764	,	0.268468631	,	0.258393055	,],
[	0.360104521	,	0.083219904	,	0.247534762	,	0.372993726	,],
[	0.432045427	,	0.30129345	,	0.072314213	,	0.343545394	,],
[	0.337084799	,	0.335992338	,	0.252421823	,	0.092813137	,]],
[[	0.091438055	,	0.240784348	,	0.265328018	,	0.255579826	,],
[	0.356170479	,	0.080281828	,	0.243520907	,	0.369070562	,],
[	0.431786129	,	0.301061086	,	0.073705527	,	0.343051387	,],
[	0.339005656	,	0.338209916	,	0.254992758	,	0.092519667	,]],
[[	0.096757483	,	0.248396957	,	0.272868294	,	0.262603106	,],
[	0.358048564	,	0.084845245	,	0.245514289	,	0.370883033	,],
[	0.434869132	,	0.304249362	,	0.074941946	,	0.346526629	,],
[	0.335988676	,	0.334854516	,	0.251432674	,	0.090480001	,]],
[[	0.091333244	,	0.24192659	,	0.266656444	,	0.256451995	,],
[	0.357680272	,	0.080890276	,	0.245070739	,	0.370622526	,],
[	0.431215001	,	0.30044034	,	0.07194825	,	0.342509209	,],
[	0.339235717	,	0.337947512	,	0.254399606	,	0.096168455	,]],
[[	0.094403875	,	0.245617205	,	0.269271791	,	0.255415599	,],
[	0.349138382	,	0.080732805	,	0.241803936	,	0.362237367	,],
[	0.426386918	,	0.301345972	,	0.076268029	,	0.338211477	,],
[	0.331917497	,	0.336517458	,	0.253263235	,	0.088562577	,]],
[[	0.091105729	,	0.245975299	,	0.269575675	,	0.2552606	,],
[	0.352929243	,	0.081919392	,	0.245486795	,	0.365921581	,],
[	0.425093359	,	0.299751997	,	0.071458787	,	0.337066996	,],
[	0.332947599	,	0.337221259	,	0.253905925	,	0.091594931	,]],
[[	0.090897869	,	0.243611433	,	0.267181773	,	0.253232531	,],
[	0.348665256	,	0.078637149	,	0.241083052	,	0.361687157	,],
[	0.424337687	,	0.299128869	,	0.072408821	,	0.336186151	,],
[	0.3342332	,	0.338866736	,	0.255662768	,	0.090255118	,]],
[[	0.094517678	,	0.250022835	,	0.273570999	,	0.259097059	,],
[	0.350298936	,	0.08307452	,	0.242944657	,	0.363306596	,],
[	0.428152918	,	0.30284177	,	0.074342918	,	0.340135765	,],
[	0.331670768	,	0.33600314	,	0.252663007	,	0.088951784	,]],
[[	0.089829999	,	0.244183379	,	0.267937646	,	0.25355673	,],
[	0.350520416	,	0.079764482	,	0.242915591	,	0.363453743	,],
[	0.423888903	,	0.29842041	,	0.070789507	,	0.335665601	,],
[	0.335110702	,	0.339174541	,	0.255719143	,	0.094850622	,]],
[[	0.09312472	,	0.245008784	,	0.268684059	,	0.254648757	,],
[	0.348851464	,	0.079833854	,	0.241307262	,	0.361785986	,],
[	0.426035103	,	0.300750914	,	0.075159112	,	0.337691601	,],
[	0.332240713	,	0.336896764	,	0.253556775	,	0.088629134	,]],
[[	0.092104705	,	0.247092676	,	0.27063008	,	0.256295959	,],
[	0.352384182	,	0.086657465	,	0.245332771	,	0.365303931	,],
[	0.425853005	,	0.300554385	,	0.072080134	,	0.337861878	,],
[	0.332532174	,	0.336863386	,	0.25343486	,	0.090610968	,]],
[[	0.090451265	,	0.243731873	,	0.267243741	,	0.253203811	,],
[	0.349253965	,	0.079015626	,	0.24159815	,	0.362152687	,],
[	0.424414871	,	0.299017256	,	0.072012809	,	0.336148998	,],
[	0.335054235	,	0.339680947	,	0.256572034	,	0.091041653	,]],
[[	0.096077102	,	0.251557142	,	0.27498146	,	0.260519163	,],
[	0.349780935	,	0.082043742	,	0.242338241	,	0.362686439	,],
[	0.42785467	,	0.302942454	,	0.078470608	,	0.339733501	,],
[	0.331884587	,	0.336290689	,	0.252906859	,	0.088687194	,]],
[[	0.090405859	,	0.244892794	,	0.268476137	,	0.254269328	,],
[	0.351433827	,	0.080534842	,	0.243771181	,	0.364260067	,],
[	0.424578297	,	0.299071326	,	0.071143884	,	0.336281963	,],
[	0.334471838	,	0.338506562	,	0.255037504	,	0.093640092	,]],
[[	0.093063249	,	0.250268943	,	0.274174778	,	0.255516354	,],
[	0.343992047	,	0.079563973	,	0.242067405	,	0.358899752	,],
[	0.421110588	,	0.30148657	,	0.07432829	,	0.33424371	,],
[	0.330015772	,	0.341098765	,	0.257191631	,	0.088568665	,]],
[[	0.090927322	,	0.251410909	,	0.275242152	,	0.256278921	,],
[	0.346788327	,	0.085244854	,	0.245167524	,	0.361649167	,],
[	0.423011661	,	0.30331777	,	0.073899469	,	0.336355974	,],
[	0.329906722	,	0.340806762	,	0.256817125	,	0.090059782	,]],
[[	0.089994318	,	0.248739686	,	0.272698496	,	0.253795376	,],
[	0.345326988	,	0.079898754	,	0.243322444	,	0.360086168	,],
[	0.419883486	,	0.300102387	,	0.071735196	,	0.332998567	,],
[	0.333594167	,	0.344669276	,	0.260937461	,	0.091913398	,]],
[[	0.094605034	,	0.255750098	,	0.279551998	,	0.260436233	,],
[	0.344840799	,	0.081428927	,	0.242904886	,	0.359651237	,],
[	0.422732402	,	0.303409571	,	0.077286156	,	0.335850118	,],
[	0.329573076	,	0.340441247	,	0.256517267	,	0.088559591	,]],
[[	0.089344779	,	0.249598579	,	0.273424183	,	0.254412519	,],
[	0.348745504	,	0.082958518	,	0.246793057	,	0.363386025	,],
[	0.421179444	,	0.30131633	,	0.072125051	,	0.334241943	,],
[	0.331582138	,	0.342198357	,	0.258083678	,	0.092680635	,]],
[[	0.091729526	,	0.249542851	,	0.273353077	,	0.254594962	,],
[	0.344224262	,	0.079402658	,	0.242212062	,	0.359084644	,],
[	0.42082625	,	0.300979777	,	0.073403676	,	0.333694698	,],
[	0.331250485	,	0.342359782	,	0.258538075	,	0.089606308	,]],
[[	0.091759677	,	0.252568895	,	0.276283826	,	0.257300321	,],
[	0.346359466	,	0.084238295	,	0.244669058	,	0.361113792	,],
[	0.424040692	,	0.304442011	,	0.075003469	,	0.337496924	,],
[	0.329700364	,	0.340603617	,	0.25647574	,	0.089380481	,]],
[[	0.089509032	,	0.248757159	,	0.272507791	,	0.253748393	,],
[	0.346366142	,	0.080765293	,	0.244242028	,	0.360959084	,],
[	0.420211426	,	0.300419348	,	0.071678408	,	0.333234213	,],
[	0.333695455	,	0.344264092	,	0.260300685	,	0.096114027	,]],
[[	0.09488666	,	0.251109638	,	0.274771378	,	0.256328764	,],
[	0.344458256	,	0.080464392	,	0.242331367	,	0.359046813	,],
[	0.422204539	,	0.302665283	,	0.076025951	,	0.335131023	,],
[	0.329776421	,	0.34071299	,	0.256719311	,	0.08848383	,]],
[[	0.089847237	,	0.250240521	,	0.274055452	,	0.255010088	,],
[	0.348321121	,	0.08763133	,	0.246757145	,	0.362988263	,],
[	0.42191369	,	0.302146417	,	0.072913687	,	0.335072713	,],
[	0.33094298	,	0.341589259	,	0.257521943	,	0.091536619	,]],
[[	0.092049501	,	0.254974748	,	0.277327673	,	0.253521332	,],
[	0.337425448	,	0.079182131	,	0.240768444	,	0.352645056	,],
[	0.415320866	,	0.301807878	,	0.073735291	,	0.328393753	,],
[	0.32812744	,	0.344580321	,	0.260718428	,	0.08808093	,]],
[[	0.090117474	,	0.256264803	,	0.278658422	,	0.254465541	,],
[	0.340046403	,	0.084674079	,	0.24373515	,	0.355226385	,],
[	0.41749844	,	0.303935966	,	0.073658917	,	0.330903276	,],
[	0.32785601	,	0.344194892	,	0.260250556	,	0.089566677	,]],
[[	0.089149173	,	0.253487002	,	0.276009023	,	0.252007641	,],
[	0.338936516	,	0.079684725	,	0.242133487	,	0.353938576	,],
[	0.414608461	,	0.300952258	,	0.071623399	,	0.327708702	,],
[	0.331158877	,	0.347521687	,	0.263889004	,	0.09068044	,]],
[[	0.09421933	,	0.260907051	,	0.283280197	,	0.258994598	,],
[	0.338106473	,	0.080795176	,	0.241404575	,	0.353253604	,],
[	0.417040008	,	0.303796198	,	0.076922861	,	0.330134249	,],
[	0.32750158	,	0.343668534	,	0.259797591	,	0.087816892	,]],
[[	0.088852682	,	0.254784719	,	0.277150559	,	0.252860688	,],
[	0.342229804	,	0.082421838	,	0.245552902	,	0.357258655	,],
[	0.415367808	,	0.301743979	,	0.071853487	,	0.328513779	,],
[	0.329687237	,	0.345672003	,	0.261666636	,	0.092258079	,]],
[[	0.091302366	,	0.254708562	,	0.277130208	,	0.253116835	,],
[	0.337603455	,	0.078966602	,	0.240827551	,	0.352708459	,],
[	0.414969394	,	0.301189842	,	0.072796967	,	0.327811158	,],
[	0.328994297	,	0.345439409	,	0.261685315	,	0.088632505	,]],
[[	0.091331317	,	0.257602971	,	0.279902799	,	0.255723206	,],
[	0.339394299	,	0.083444941	,	0.242941097	,	0.35455905	,],
[	0.41883339	,	0.305355317	,	0.075139147	,	0.332244426	,],
[	0.327629441	,	0.343747999	,	0.259857242	,	0.088647603	,]],
[[	0.088782914	,	0.25363494	,	0.275998186	,	0.251940975	,],
[	0.339450291	,	0.080035123	,	0.242678744	,	0.354545642	,],
[	0.414840963	,	0.30113405	,	0.071522168	,	0.327882865	,],
[	0.33202607	,	0.347897952	,	0.26402725	,	0.0957333	,]],
[[	0.094381449	,	0.256249195	,	0.278543283	,	0.254827448	,],
[	0.337921292	,	0.080282101	,	0.241111782	,	0.352984261	,],
[	0.416500447	,	0.303072309	,	0.075549854	,	0.329432613	,],
[	0.327576995	,	0.343900686	,	0.259968471	,	0.087606754	,]],
[[	0.089179012	,	0.255095195	,	0.277428831	,	0.253173116	,],
[	0.341668961	,	0.087277868	,	0.245311671	,	0.356688354	,],
[	0.416593881	,	0.302903329	,	0.072778967	,	0.329754682	,],
[	0.3291988	,	0.345193346	,	0.261201595	,	0.091211886	,]],
[[	0.091861167	,	0.261699386	,	0.283286178	,	0.254816135	,],
[	0.334680732	,	0.080403622	,	0.242767827	,	0.350959789	,],
[	0.412088342	,	0.303444686	,	0.072801019	,	0.325515753	,],
[	0.327977373	,	0.350948293	,	0.265966655	,	0.088222178	,]],
[[	0.090341634	,	0.263439879	,	0.285080365	,	0.256135522	,],
[	0.335244539	,	0.08345579	,	0.243634727	,	0.351608491	,],
[	0.415970457	,	0.307763602	,	0.07971145	,	0.329440728	,],
[	0.326294758	,	0.349110002	,	0.263952825	,	0.087933865	,]],
[[	0.089223142	,	0.260508809	,	0.282147665	,	0.253485033	,],
[	0.336777898	,	0.081783951	,	0.24481059	,	0.35301458	,],
[	0.412099259	,	0.303430787	,	0.071790936	,	0.325681113	,],
[	0.330589646	,	0.353046118	,	0.267946994	,	0.094920286	,]],
[[	0.094259464	,	0.267664392	,	0.289287874	,	0.260350779	,],
[	0.334312717	,	0.080865131	,	0.242393163	,	0.350596169	,],
[	0.413178019	,	0.3047556	,	0.075041933	,	0.326609273	,],
[	0.32663697	,	0.349435859	,	0.264328396	,	0.087227528	,]],
[[	0.088708751	,	0.261409276	,	0.283098365	,	0.254043643	,],
[	0.337070625	,	0.086646803	,	0.245345993	,	0.353181401	,],
[	0.414164246	,	0.305499621	,	0.073315173	,	0.327796601	,],
[	0.327643104	,	0.350114479	,	0.264928112	,	0.090186888	,]],
[[	0.090958923	,	0.261267939	,	0.28295359	,	0.254241378	,],
[	0.335121955	,	0.080501121	,	0.243208203	,	0.351435449	,],
[	0.411976541	,	0.303348281	,	0.072268812	,	0.325320571	,],
[	0.328945471	,	0.351868165	,	0.266967678	,	0.088999625	,]],
[[	0.091266133	,	0.264477835	,	0.286020711	,	0.257147013	,],
[	0.334961053	,	0.082531341	,	0.24319152	,	0.351099167	,],
[	0.415017092	,	0.306831235	,	0.078069457	,	0.328473775	,],
[	0.326690878	,	0.349450696	,	0.264350356	,	0.087829213	,]],
[[	0.088741068	,	0.26051901	,	0.282140384	,	0.253383869	,],
[	0.337986529	,	0.082742878	,	0.245878946	,	0.354090996	,],
[	0.412736001	,	0.304003108	,	0.072176284	,	0.326195949	,],
[	0.329720432	,	0.352169062	,	0.267089977	,	0.093448848	,]],
[[	0.094439917	,	0.263153855	,	0.284561644	,	0.25616982	,],
[	0.334468937	,	0.080559358	,	0.242405336	,	0.350528882	,],
[	0.412614269	,	0.304054889	,	0.073877833	,	0.325949648	,],
[	0.327058764	,	0.349719837	,	0.264828438	,	0.087477352	,]],
[[	0.089295748	,	0.262318577	,	0.283955922	,	0.254942227	,],
[	0.336582823	,	0.08548612	,	0.244761119	,	0.352655114	,],
[	0.415368881	,	0.306792267	,	0.074656287	,	0.329063563	,],
[	0.32730774	,	0.349827298	,	0.264680165	,	0.089266357	,]],
[[	0.091395354	,	0.266107221	,	0.286823106	,	0.25445321	,],
[	0.330207699	,	0.080765665	,	0.242614162	,	0.347115685	,],
[	0.406648789	,	0.303528629	,	0.071103465	,	0.321486949	,],
[	0.327467866	,	0.356273092	,	0.271026171	,	0.090043522	,]],
[[	0.090853196	,	0.268413992	,	0.288992197	,	0.256459766	,],
[	0.329005682	,	0.081533845	,	0.24151961	,	0.345919922	,],
[	0.408961872	,	0.306250271	,	0.075985088	,	0.323810446	,],
[	0.324438346	,	0.353190043	,	0.267585179	,	0.087919458	,]],
[[	0.088903799	,	0.264871533	,	0.285531092	,	0.253129594	,],
[	0.333018809	,	0.08868993	,	0.245866701	,	0.349739981	,],
[	0.407923459	,	0.304898344	,	0.071652979	,	0.322915274	,],
[	0.327051076	,	0.355583294	,	0.269879084	,	0.093031946	,]],
[[	0.094650969	,	0.267711174	,	0.288278154	,	0.256222947	,],
[	0.329233794	,	0.080380169	,	0.241537449	,	0.34614251	,],
[	0.407006536	,	0.304011987	,	0.072336169	,	0.321742576	,],
[	0.325516803	,	0.354223361	,	0.268609746	,	0.08813589	,]],
[[	0.088922012	,	0.266240739	,	0.286969307	,	0.254311875	,],
[	0.33024073	,	0.083853975	,	0.242716867	,	0.347033901	,],
[	0.411144671	,	0.308762056	,	0.079700224	,	0.325966312	,],
[	0.324874303	,	0.353401966	,	0.267624787	,	0.08910651	,]],
[[	0.090356736	,	0.265402555	,	0.286205698	,	0.253629153	,],
[	0.331395437	,	0.081665365	,	0.243714063	,	0.348185377	,],
[	0.407052083	,	0.304002311	,	0.071264277	,	0.321831924	,],
[	0.328363928	,	0.357263441	,	0.271919304	,	0.090894795	,]],
[[	0.091935433	,	0.269581733	,	0.290153075	,	0.257692794	,],
[	0.329051628	,	0.080942545	,	0.241421042	,	0.345909548	,],
[	0.408327423	,	0.305497316	,	0.074789588	,	0.322991784	,],
[	0.324270677	,	0.353092926	,	0.267463658	,	0.087398652	,]],
[[	0.0888	,	0.265241189	,	0.285768904	,	0.253375221	,],
[	0.332234405	,	0.087370737	,	0.244962747	,	0.348880737	,],
[	0.408910161	,	0.305818491	,	0.072576144	,	0.323867101	,],
[	0.32628602	,	0.354827665	,	0.26918669	,	0.091579424	,]],
[[	0.093270142	,	0.266945933	,	0.287390284	,	0.255319253	,],
[	0.329698913	,	0.08046019	,	0.241868259	,	0.346393545	,],
[	0.40687249	,	0.303763299	,	0.071667498	,	0.321518031	,],
[	0.326453927	,	0.355128208	,	0.269657732	,	0.088960133	,]],
[[	0.08976669	,	0.267209482	,	0.287875004	,	0.255215963	,],
[	0.329874853	,	0.082969645	,	0.242338318	,	0.346548124	,],
[	0.4103533	,	0.307768755	,	0.078331432	,	0.325086409	,],
[	0.324685872	,	0.35327842	,	0.267550663	,	0.088507658	,]],
[[	0.09072736	,	0.270991592	,	0.290749066	,	0.254165072	,],
[	0.327135979	,	0.082736541	,	0.245177099	,	0.345361749	,],
[	0.403115497	,	0.305745294	,	0.071910413	,	0.319319989	,],
[	0.326433633	,	0.360020296	,	0.274200695	,	0.095143414	,]],
[[	0.090131671	,	0.273328496	,	0.292968674	,	0.256225375	,],
[	0.323887945	,	0.080876851	,	0.241998652	,	0.342170974	,],
[	0.403357197	,	0.306171301	,	0.074059945	,	0.319329592	,],
[	0.322706498	,	0.356979948	,	0.271135594	,	0.087821072	,]],
[[	0.088384818	,	0.270172699	,	0.289839475	,	0.25324107	,],
[	0.325876398	,	0.085939318	,	0.244313584	,	0.344204713	,],
[	0.405594709	,	0.308358827	,	0.073973015	,	0.321993256	,],
[	0.323534148	,	0.357618179	,	0.271739914	,	0.090480403	,]],
[[	0.094017808	,	0.272752512	,	0.292340226	,	0.255997733	,],
[	0.325106821	,	0.081040477	,	0.242950084	,	0.343229261	,],
[	0.402622739	,	0.305289242	,	0.071919885	,	0.318766202	,],
[	0.325139253	,	0.359202716	,	0.273634494	,	0.08974371	,]],
[[	0.088418782	,	0.271480858	,	0.291152142	,	0.254238663	,],
[	0.32422467	,	0.082186304	,	0.242359849	,	0.34247752	,],
[	0.405015793	,	0.308102371	,	0.076911149	,	0.321119773	,],
[	0.322437944	,	0.356360995	,	0.270488714	,	0.088046657	,]],
[[	0.089954606	,	0.270775408	,	0.29048518	,	0.253831111	,],
[	0.328288786	,	0.083821629	,	0.246305774	,	0.346397529	,],
[	0.403477369	,	0.305999476	,	0.072121515	,	0.3197126	,],
[	0.325670043	,	0.359323637	,	0.273506526	,	0.093729637	,]],
[[	0.091412837	,	0.27465059	,	0.294280829	,	0.257461095	,],
[	0.324011948	,	0.080685016	,	0.242021593	,	0.342147073	,],
[	0.403030531	,	0.305669097	,	0.073137455	,	0.318980436	,],
[	0.323152923	,	0.357282399	,	0.271624147	,	0.088050793	,]],
[[	0.088088513	,	0.27027336	,	0.289919332	,	0.253228846	,],
[	0.325439807	,	0.084851191	,	0.24389887	,	0.343707496	,],
[	0.406515316	,	0.30917514	,	0.074870279	,	0.322890131	,],
[	0.32293242	,	0.356944382	,	0.271135256	,	0.089524682	,]],
[[	0.092366836	,	0.271827047	,	0.291290269	,	0.254925026	,],
[	0.325775015	,	0.081502271	,	0.243643094	,	0.343863174	,],
[	0.402822185	,	0.305480668	,	0.071861908	,	0.318903746	,],
[	0.326221474	,	0.360189933	,	0.274620926	,	0.090627506	,]],
[[	0.089351403	,	0.272428623	,	0.292002322	,	0.255277145	,],
[	0.323951474	,	0.081468598	,	0.241863144	,	0.342018575	,],
[	0.404300671	,	0.307183565	,	0.075579697	,	0.32025163	,],
[	0.32254977	,	0.356475213	,	0.270676073	,	0.087859284	,]],
[[	0.089900496	,	0.275515904	,	0.294196382	,	0.253375516	,],
[	0.321570042	,	0.082943719	,	0.244884487	,	0.340678867	,],
[	0.397860774	,	0.306246071	,	0.071627459	,	0.31444736	,],
[	0.324846002	,	0.364842212	,	0.278690013	,	0.091372533	,]],
[[	0.089700461	,	0.278068789	,	0.296637214	,	0.255620377	,],
[	0.317316615	,	0.079933176	,	0.240758096	,	0.336580041	,],
[	0.397648689	,	0.306145004	,	0.073158828	,	0.313968618	,],
[	0.320291762	,	0.360220816	,	0.273809591	,	0.087526853	,]],
[[	0.087510037	,	0.274406539	,	0.293006528	,	0.252166741	,],
[	0.319258395	,	0.084738677	,	0.243026376	,	0.338338321	,],
[	0.400753011	,	0.309263142	,	0.074275912	,	0.317530415	,],
[	0.32175196	,	0.361510892	,	0.275003361	,	0.090963122	,]],
[[	0.093163916	,	0.277126801	,	0.295646104	,	0.25517783	,],
[	0.319262827	,	0.080782659	,	0.242516221	,	0.338191512	,],
[	0.397046386	,	0.305435908	,	0.071248272	,	0.313391301	,],
[	0.322150242	,	0.361994854	,	0.27567401	,	0.088663227	,]],
[[	0.087622493	,	0.275783862	,	0.294438967	,	0.253321766	,],
[	0.317726501	,	0.081195648	,	0.241122733	,	0.336876153	,],
[	0.399346719	,	0.308039403	,	0.076063822	,	0.315578231	,],
[	0.320161797	,	0.359969151	,	0.273374426	,	0.088004674	,]],
[[	0.089038278	,	0.275050885	,	0.293689729	,	0.252760225	,],
[	0.321451744	,	0.088348348	,	0.245274257	,	0.340374773	,],
[	0.398596311	,	0.306966559	,	0.072087397	,	0.315103309	,],
[	0.324303526	,	0.363770145	,	0.277215531	,	0.094540224	,]],
[[	0.090685284	,	0.279157091	,	0.297716308	,	0.256718627	,],
[	0.317742045	,	0.079992961	,	0.241049514	,	0.336869878	,],
[	0.397485027	,	0.305971311	,	0.072466012	,	0.313728104	,],
[	0.320612439	,	0.360511547	,	0.274137483	,	0.087502735	,]],
[[	0.087220268	,	0.27455981	,	0.293040622	,	0.252188165	,],
[	0.318651694	,	0.083536206	,	0.242261272	,	0.337746093	,],
[	0.401385742	,	0.310389312	,	0.079518417	,	0.317678854	,],
[	0.321159288	,	0.360918148	,	0.274469303	,	0.089865561	,]],
[[	0.091909779	,	0.276512981	,	0.29496715	,	0.254334083	,],
[	0.32019227	,	0.081609029	,	0.243325383	,	0.339092422	,],
[	0.397244357	,	0.305592509	,	0.071186973	,	0.313542127	,],
[	0.323204535	,	0.363003633	,	0.276741731	,	0.089638907	,]],
[[	0.088521606	,	0.276740243	,	0.295326272	,	0.254234246	,],
[	0.317608218	,	0.080669194	,	0.24088566	,	0.336601217	,],
[	0.398588823	,	0.307249496	,	0.07475097	,	0.314765937	,],
[	0.320297555	,	0.360092326	,	0.273618908	,	0.087901242	,]],
[[	0.08943695	,	0.281110516	,	0.29861982	,	0.252720737	,],
[	0.315933471	,	0.088471434	,	0.244902712	,	0.336565509	,],
[	0.394365314	,	0.30908797	,	0.072506885	,	0.311634477	,],
[	0.322753129	,	0.368841599	,	0.28183246	,	0.09483797	,]],
[[	0.089475014	,	0.283849843	,	0.3012647	,	0.255220947	,],
[	0.313097543	,	0.081008371	,	0.241545482	,	0.333835577	,],
[	0.393153595	,	0.308065385	,	0.072842513	,	0.310181472	,],
[	0.318304745	,	0.365020307	,	0.278041737	,	0.086658976	,]],
[[	0.087378165	,	0.280305163	,	0.297757252	,	0.251775502	,],
[	0.313361709	,	0.083855636	,	0.242075254	,	0.334017946	,],
[	0.397345528	,	0.312719442	,	0.080142155	,	0.314300337	,],
[	0.319584652	,	0.365913961	,	0.279021086	,	0.089982837	,]],
[[	0.092420093	,	0.282534319	,	0.299885407	,	0.254238531	,],
[	0.315504272	,	0.082731653	,	0.243903456	,	0.336175402	,],
[	0.393290246	,	0.307973494	,	0.071729444	,	0.310294937	,],
[	0.320412869	,	0.366915732	,	0.280148974	,	0.088235284	,]],
[[	0.087484505	,	0.281849935	,	0.299355614	,	0.2531111	,],
[	0.312756952	,	0.081451332	,	0.241112477	,	0.333379439	,],
[	0.394598152	,	0.309653015	,	0.075483357	,	0.311679391	,],
[	0.318278071	,	0.364585403	,	0.277630557	,	0.087203492	,]],
[[	0.088653973	,	0.280825139	,	0.298282468	,	0.252230389	,],
[	0.315142724	,	0.087093297	,	0.243992991	,	0.335724025	,],
[	0.395235656	,	0.309847844	,	0.073084103	,	0.312395737	,],
[	0.321915694	,	0.368084456	,	0.281047317	,	0.093359921	,]],
[[	0.090525475	,	0.285102509	,	0.302413897	,	0.256337755	,],
[	0.313420947	,	0.080988534	,	0.241777533	,	0.334026537	,],
[	0.393121338	,	0.307931797	,	0.072360186	,	0.310008657	,],
[	0.318928856	,	0.365436165	,	0.278648923	,	0.086980622	,]],
[[	0.087096968	,	0.2804599	,	0.29790995	,	0.251882055	,],
[	0.31313645	,	0.083002	,	0.241699274	,	0.33376843	,],
[	0.396562523	,	0.311863689	,	0.078790523	,	0.313502779	,],
[	0.318947414	,	0.365391229	,	0.278313784	,	0.08875869	,]],
[[	0.091279014	,	0.281999522	,	0.299324116	,	0.253498965	,],
[	0.316451029	,	0.083563216	,	0.24471999	,	0.33689563	,],
[	0.39387438	,	0.30858652	,	0.07211007	,	0.310819363	,],
[	0.321775066	,	0.36821258	,	0.281415508	,	0.089356913	,]],
[[	0.088250129	,	0.282511384	,	0.300006308	,	0.25385122	,],
[	0.31279125	,	0.081219631	,	0.241125563	,	0.333390433	,],
[	0.393918551	,	0.308837763	,	0.074280602	,	0.310781111	,],
[	0.318465812	,	0.364837419	,	0.277819658	,	0.087044096	,]],
[[	0.08852267	,	0.285629347	,	0.302333252	,	0.252215001	,],
[	0.310399234	,	0.087702487	,	0.244701802	,	0.331041228	,],
[	0.391408436	,	0.311040364	,	0.073888137	,	0.308928611	,],
[	0.320326157	,	0.371981451	,	0.284440031	,	0.09376288	,]],
[[	0.089112896	,	0.288891135	,	0.305519468	,	0.255304787	,],
[	0.308229487	,	0.08118832	,	0.242022297	,	0.328968142	,],
[	0.388975459	,	0.30872164	,	0.072541323	,	0.306237201	,],
[	0.316469967	,	0.36846774	,	0.281023448	,	0.08622075	,]],
[[	0.086804589	,	0.285227825	,	0.301948905	,	0.251736192	,],
[	0.307831198	,	0.083280238	,	0.242054745	,	0.328692158	,],
[	0.392346416	,	0.312592133	,	0.078929035	,	0.309650117	,],
[	0.317233298	,	0.36913963	,	0.28148672	,	0.088903902	,]],
[[	0.091954441	,	0.287508313	,	0.304081739	,	0.25418541	,],
[	0.311036199	,	0.083380382	,	0.24466862	,	0.331518038	,],
[	0.3897869	,	0.309452242	,	0.072468665	,	0.307179285	,],
[	0.319021337	,	0.370757824	,	0.28351058	,	0.088346106	,]],
[[	0.087014703	,	0.286546384	,	0.303302472	,	0.25296219	,],
[	0.307514751	,	0.081216143	,	0.24131373	,	0.32810931	,],
[	0.389893082	,	0.309873069	,	0.074555368	,	0.307158177	,],
[	0.316196062	,	0.368041357	,	0.280411057	,	0.086513705	,]],
[[	0.087974484	,	0.28546868	,	0.302107303	,	0.251989722	,],
[	0.309644775	,	0.086244342	,	0.243763121	,	0.330234142	,],
[	0.392593608	,	0.312342849	,	0.074886077	,	0.310066776	,],
[	0.319459334	,	0.371065557	,	0.283484471	,	0.092198174	,]],
[[	0.090180362	,	0.289929472	,	0.30659055	,	0.256402802	,],
[	0.308869248	,	0.081429238	,	0.242595604	,	0.32949984	,],
[	0.389195792	,	0.308905002	,	0.07227768	,	0.306417441	,],
[	0.316767908	,	0.36872564	,	0.281369474	,	0.086400917	,]],
[[	0.086572617	,	0.285157855	,	0.301821531	,	0.251653536	,],
[	0.307928077	,	0.082577966	,	0.241916005	,	0.328495572	,],
[	0.391464646	,	0.311617458	,	0.077434869	,	0.308621695	,],
[	0.317003323	,	0.368801715	,	0.281226513	,	0.088224479	,]],
[[	0.09038821	,	0.286608005	,	0.303193481	,	0.253168211	,],
[	0.31188463	,	0.09026063	,	0.246490206	,	0.332464568	,],
[	0.39058439	,	0.310275723	,	0.073086159	,	0.308050226	,],
[	0.320269678	,	0.372035169	,	0.284746821	,	0.089448906	,]],
[[	0.087904989	,	0.287515147	,	0.304207032	,	0.253870522	,],
[	0.307923166	,	0.081201592	,	0.241540029	,	0.328417611	,],
[	0.389533185	,	0.309358422	,	0.073732231	,	0.306682573	,],
[	0.316421135	,	0.368248555	,	0.280586135	,	0.086513527	,]],
[[	0.087969888	,	0.290031958	,	0.306591534	,	0.25271837	,],
[	0.303720669	,	0.084411374	,	0.242526347	,	0.325552523	,],
[	0.388387821	,	0.314806561	,	0.080540769	,	0.306945502	,],
[	0.316403017	,	0.373859067	,	0.28612515	,	0.090199575	,]],
[[	0.089280817	,	0.293670158	,	0.310195776	,	0.256281478	,],
[	0.304573152	,	0.081789943	,	0.242960323	,	0.326516349	,],
[	0.384602723	,	0.310208419	,	0.072446174	,	0.303086644	,],
[	0.316006595	,	0.373759256	,	0.286221694	,	0.087021918	,]],
[[	0.08606366	,	0.289263528	,	0.305741658	,	0.251875722	,],
[	0.302188179	,	0.081171486	,	0.240874649	,	0.324143885	,],
[	0.385760561	,	0.311739727	,	0.075773679	,	0.304124584	,],
[	0.314778239	,	0.372333183	,	0.284665809	,	0.08714136	,]],
[[	0.090787123	,	0.291362354	,	0.307768619	,	0.254078125	,],
[	0.305377402	,	0.087690766	,	0.24446461	,	0.327348208	,],
[	0.386499675	,	0.312124693	,	0.073849165	,	0.305066243	,],
[	0.318636535	,	0.376126835	,	0.288433382	,	0.093737033	,]],
[[	0.086988088	,	0.29123049	,	0.307846304	,	0.253801663	,],
[	0.30301147	,	0.080681263	,	0.241252357	,	0.324776059	,],
[	0.38450794	,	0.310283249	,	0.073114419	,	0.302892997	,],
[	0.314970486	,	0.372521986	,	0.284849869	,	0.08626598	,]],
[[	0.086951426	,	0.289533433	,	0.306112064	,	0.252075546	,],
[	0.302889985	,	0.083093995	,	0.241612786	,	0.324764316	,],
[	0.387737468	,	0.313859759	,	0.079179964	,	0.306108641	,],
[	0.315772702	,	0.373180975	,	0.28544958	,	0.089124112	,]],
[[	0.090589026	,	0.295016971	,	0.311478845	,	0.257578669	,],
[	0.305668663	,	0.082702127	,	0.244133165	,	0.327368675	,],
[	0.385120171	,	0.310675281	,	0.072684641	,	0.303457561	,],
[	0.31709239	,	0.374750908	,	0.287208893	,	0.087878932	,]],
[[	0.086186515	,	0.289845159	,	0.30630711	,	0.252329539	,],
[	0.302371499	,	0.080937619	,	0.240862851	,	0.324221455	,],
[	0.385078853	,	0.310953986	,	0.074716649	,	0.303301005	,],
[	0.314513763	,	0.372197491	,	0.284546741	,	0.086624922	,]],
[[	0.089412018	,	0.290609311	,	0.306972465	,	0.253306014	,],
[	0.304798864	,	0.086297312	,	0.243597254	,	0.326503352	,],
[	0.387684759	,	0.313314218	,	0.074938348	,	0.306305196	,],
[	0.317834081	,	0.3753589	,	0.287509915	,	0.092381291	,]],
[[	0.087642204	,	0.292000356	,	0.308523112	,	0.254463819	,],
[	0.303839168	,	0.081268919	,	0.242067911	,	0.325505308	,],
[	0.384444027	,	0.310062243	,	0.072776808	,	0.302802762	,],
[	0.315210164	,	0.372825501	,	0.285120807	,	0.086247601	,]],
[[	0.087111377	,	0.295253653	,	0.310483681	,	0.251355798	,],
[	0.297375928	,	0.084053252	,	0.242132592	,	0.319997869	,],
[	0.383587785	,	0.316005074	,	0.07983719	,	0.302510364	,],
[	0.314829667	,	0.378412405	,	0.289922793	,	0.090833623	,]],
[[	0.089196746	,	0.299485759	,	0.314772376	,	0.255620298	,],
[	0.300220279	,	0.083851965	,	0.244692548	,	0.32285775	,],
[	0.379909333	,	0.311442875	,	0.071840781	,	0.298765549	,],
[	0.313987076	,	0.377850528	,	0.289647281	,	0.08715944	,]],
[[	0.085724196	,	0.294843601	,	0.310120846	,	0.250965094	,],
[	0.296654824	,	0.081830278	,	0.241373956	,	0.319370943	,],
[	0.380602226	,	0.312376291	,	0.07478812	,	0.299363029	,],
[	0.312703702	,	0.376610407	,	0.288088945	,	0.087479217	,]],
[[	0.090400625	,	0.296971769	,	0.312136971	,	0.253211	,],
[	0.298981283	,	0.086930685	,	0.243950401	,	0.321591744	,],
[	0.382444792	,	0.313873448	,	0.073734604	,	0.301393596	,],
[	0.317184423	,	0.380810427	,	0.292344859	,	0.094586391	,]],
[[	0.086571763	,	0.296817606	,	0.312175452	,	0.252799743	,],
[	0.298350746	,	0.082238416	,	0.242709864	,	0.320856388	,],
[	0.379720245	,	0.311261664	,	0.072284023	,	0.298418706	,],
[	0.313040037	,	0.376638521	,	0.288262666	,	0.086492111	,]],
[[	0.086718066	,	0.295307964	,	0.310542858	,	0.251374054	,],
[	0.297074982	,	0.083235609	,	0.241727265	,	0.319670023	,],
[	0.38249756	,	0.314638663	,	0.078105496	,	0.301355298	,],
[	0.31436457	,	0.377999746	,	0.289374561	,	0.08996037	,]],
[[	0.090465833	,	0.300732268	,	0.315906045	,	0.256783694	,],
[	0.301312876	,	0.090886101	,	0.246567914	,	0.323862531	,],
[	0.380559382	,	0.312062521	,	0.072368613	,	0.299415789	,],
[	0.314936602	,	0.378776147	,	0.290539288	,	0.087950536	,]],
[[	0.085921408	,	0.295573361	,	0.310796526	,	0.251537004	,],
[	0.297131101	,	0.081829368	,	0.241640281	,	0.319812742	,],
[	0.380742639	,	0.312413959	,	0.074167484	,	0.299407563	,],
[	0.312732644	,	0.376537221	,	0.288095259	,	0.086891965	,]],
[[	0.08915032	,	0.296314578	,	0.311446993	,	0.252491369	,],
[	0.298405588	,	0.085813328	,	0.243208609	,	0.320981996	,],
[	0.383396255	,	0.314957877	,	0.07485638	,	0.30252948	,],
[	0.316382879	,	0.380023089	,	0.291592746	,	0.093117719	,]],
[[	0.087663177	,	0.298012838	,	0.313354716	,	0.253970545	,],
[	0.298991235	,	0.082745393	,	0.243349278	,	0.321513027	,],
[	0.37954876	,	0.311024145	,	0.071819468	,	0.298238634	,],
[	0.313461435	,	0.377224166	,	0.28870567	,	0.086754431	,]],
[[	0.08733768	,	0.30066178	,	0.314205354	,	0.250622656	,],
[	0.291319913	,	0.083954435	,	0.24072492	,	0.314417608	,],
[	0.378657517	,	0.316715365	,	0.078675101	,	0.297732801	,],
[	0.313296774	,	0.382730002	,	0.292819488	,	0.090403454	,]],
[[	0.08838797	,	0.303924656	,	0.317415913	,	0.253878702	,],
[	0.295498056	,	0.08544389	,	0.244620102	,	0.318434914	,],
[	0.376592938	,	0.314104895	,	0.07282014	,	0.295737615	,],
[	0.312402676	,	0.382077198	,	0.292427585	,	0.086591555	,]],
[[	0.085694314	,	0.300133951	,	0.313631364	,	0.250017073	,],
[	0.291071922	,	0.08220137	,	0.24028922	,	0.314142676	,],
[	0.376504989	,	0.314148606	,	0.07454472	,	0.295442774	,],
[	0.311357414	,	0.380989346	,	0.291120518	,	0.08704163	,]],
[[	0.090489062	,	0.30220632	,	0.315667377	,	0.252308186	,],
[	0.292650402	,	0.08672681	,	0.242228612	,	0.315696179	,],
[	0.379251299	,	0.316779848	,	0.075012263	,	0.298571446	,],
[	0.315559722	,	0.385058503	,	0.295179566	,	0.093879922	,]],
[[	0.086509295	,	0.302113079	,	0.315679559	,	0.251898719	,],
[	0.293103242	,	0.083278351	,	0.242086022	,	0.315997595	,],
[	0.375877158	,	0.313421702	,	0.07271206	,	0.294941189	,],
[	0.311443273	,	0.38091139	,	0.29117561	,	0.085926898	,]],
[[	0.086565227	,	0.300146264	,	0.313844222	,	0.250050154	,],
[	0.291386222	,	0.083548203	,	0.240598712	,	0.314314438	,],
[	0.377692117	,	0.315751992	,	0.077246365	,	0.296776504	,],
[	0.312591541	,	0.382037687	,	0.292027636	,	0.089187312	,]],
[[	0.090056556	,	0.305671608	,	0.31904665	,	0.255578528	,],
[	0.294575912	,	0.090129504	,	0.244261333	,	0.317379472	,],
[	0.377338131	,	0.314694115	,	0.073223033	,	0.296393499	,],
[	0.313359744	,	0.382955236	,	0.293431594	,	0.087452136	,]],
[[	0.085718288	,	0.300514005	,	0.313995225	,	0.250368214	,],
[	0.291512548	,	0.082216131	,	0.240581508	,	0.31444927	,],
[	0.376209727	,	0.313819763	,	0.073803917	,	0.295140116	,],
[	0.311009441	,	0.380635279	,	0.290747295	,	0.086189874	,]],
[[	0.089130338	,	0.301416953	,	0.314807361	,	0.251362215	,],
[	0.292049477	,	0.085489908	,	0.241538308	,	0.31504236	,],
[	0.380218969	,	0.318441381	,	0.081111207	,	0.299185426	,],
[	0.314522125	,	0.383981028	,	0.294093705	,	0.0921961	,]],
[[	0.087297382	,	0.302762031	,	0.316283763	,	0.252573776	,],
[	0.293957058	,	0.084045957	,	0.243021562	,	0.316895682	,],
[	0.375803962	,	0.313313725	,	0.072325833	,	0.294851717	,],
[	0.311753371	,	0.381278311	,	0.291555118	,	0.086024601	,]],
[[	0.087559427	,	0.306162411	,	0.319761912	,	0.25192512	,],
[	0.285688334	,	0.083656467	,	0.241161484	,	0.310643393	,],
[	0.37256312	,	0.316920678	,	0.0766761	,	0.292692122	,],
[	0.310417967	,	0.385999391	,	0.295996952	,	0.088871547	,]],
[[	0.088590163	,	0.309351738	,	0.322871028	,	0.255136294	,],
[	0.289745533	,	0.091106996	,	0.245717317	,	0.314488097	,],
[	0.372155076	,	0.315990727	,	0.072904552	,	0.292362174	,],
[	0.310667246	,	0.386317404	,	0.296690198	,	0.086538433	,]],
[[	0.085777998	,	0.305499954	,	0.319065623	,	0.25121782	,],
[	0.285957699	,	0.082440842	,	0.241250945	,	0.310740689	,],
[	0.370818132	,	0.31487787	,	0.07319756	,	0.290877589	,],
[	0.308751581	,	0.384476387	,	0.294562577	,	0.085827157	,]],
[[	0.090499308	,	0.30742053	,	0.320901025	,	0.253279106	,],
[	0.286819283	,	0.086094022	,	0.242455448	,	0.311657234	,],
[	0.37459054	,	0.319373807	,	0.08020562	,	0.294629734	,],
[	0.312365882	,	0.387853244	,	0.297956311	,	0.091848214	,]],
[[	0.085926159	,	0.306930622	,	0.320557097	,	0.252512716	,],
[	0.288361639	,	0.083932354	,	0.243445474	,	0.313012717	,],
[	0.371170368	,	0.315117608	,	0.072380584	,	0.291264987	,],
[	0.309323375	,	0.384885896	,	0.295127362	,	0.085389642	,]],
[[	0.086760881	,	0.305668562	,	0.319288972	,	0.251349206	,],
[	0.285430508	,	0.082893658	,	0.24082711	,	0.310257905	,],
[	0.372068576	,	0.31642775	,	0.075510258	,	0.292064609	,],
[	0.309955225	,	0.385420628	,	0.295486453	,	0.087899671	,]],
[[	0.089803964	,	0.310685697	,	0.324168501	,	0.256430218	,],
[	0.289066904	,	0.089708636	,	0.244816255	,	0.313684979	,],
[	0.373340197	,	0.317168898	,	0.073936205	,	0.293392737	,],
[	0.311610728	,	0.387242878	,	0.297684249	,	0.087269587	,]],
[[	0.085727923	,	0.305787566	,	0.319289116	,	0.251479486	,],
[	0.286514163	,	0.082465562	,	0.241619567	,	0.311134019	,],
[	0.37087935	,	0.314955874	,	0.072828913	,	0.290870045	,],
[	0.308882257	,	0.384583473	,	0.294621924	,	0.08539491	,]],
[[	0.089415275	,	0.307007958	,	0.320450856	,	0.25273595	,],
[	0.286360723	,	0.085011169	,	0.241919379	,	0.311102163	,],
[	0.373766853	,	0.318370335	,	0.078779405	,	0.293854263	,],
[	0.311351134	,	0.386874906	,	0.296883652	,	0.090309104	,]],
[[	0.086914856	,	0.307846433	,	0.321381015	,	0.253442968	,],
[	0.2891786	,	0.084526949	,	0.244203975	,	0.313770004	,],
[	0.371392514	,	0.315306922	,	0.072375294	,	0.291498911	,],
[	0.309804566	,	0.385410436	,	0.295640187	,	0.085691546	,]],
[[	0.085100205	,	0.308870798	,	0.321449671	,	0.250660577	,],
[	0.280133284	,	0.082772635	,	0.240818847	,	0.306383957	,],
[	0.365159972	,	0.315122593	,	0.073022847	,	0.286818229	,],
[	0.306606938	,	0.387523279	,	0.297312798	,	0.086218737	,]],
[[	0.088323812	,	0.313657769	,	0.326337792	,	0.255530492	,],
[	0.280510423	,	0.085620433	,	0.24154766	,	0.306707427	,],
[	0.368156969	,	0.318660924	,	0.0789915	,	0.289790357	,],
[	0.310989111	,	0.39166837	,	0.301453887	,	0.092914383	,]],
[[	0.084111393	,	0.308869604	,	0.321488608	,	0.250598293	,],
[	0.282847709	,	0.084680232	,	0.243543448	,	0.30907529	,],
[	0.365690343	,	0.315448051	,	0.072490459	,	0.287360355	,],
[	0.306817822	,	0.387815053	,	0.297686289	,	0.085313486	,]],
[[	0.088204264	,	0.310481135	,	0.323069891	,	0.252265861	,],
[	0.279619499	,	0.083016033	,	0.240426988	,	0.305819375	,],
[	0.365922333	,	0.316142508	,	0.074863905	,	0.287539149	,],
[	0.307993775	,	0.38896634	,	0.298645006	,	0.088266973	,]],
[[	0.085470717	,	0.310955601	,	0.323605544	,	0.252656097	,],
[	0.281983178	,	0.088528214	,	0.243168321	,	0.308097256	,],
[	0.368744219	,	0.31843585	,	0.075173307	,	0.290573478	,],
[	0.309527614	,	0.390242889	,	0.300215789	,	0.087276694	,]],
[[	0.08477771	,	0.308873141	,	0.321586819	,	0.250533416	,],
[	0.28074532	,	0.083060134	,	0.241367244	,	0.306925681	,],
[	0.365052189	,	0.314989955	,	0.072595164	,	0.286634505	,],
[	0.306759765	,	0.38764739	,	0.297431731	,	0.085926559	,]],
[[	0.089640605	,	0.315062653	,	0.327644634	,	0.256874909	,],
[	0.280095611	,	0.084602642	,	0.241019517	,	0.306216827	,],
[	0.367205082	,	0.317612271	,	0.077403657	,	0.2887608	,],
[	0.310151142	,	0.390884717	,	0.300547607	,	0.0915149	,]],
[[	0.08440584	,	0.309303236	,	0.321881246	,	0.250981958	,],
[	0.283838384	,	0.085581405	,	0.244458024	,	0.309988721	,],
[	0.366601521	,	0.31630628	,	0.073252516	,	0.288271768	,],
[	0.307427277	,	0.38837145	,	0.298358347	,	0.085631872	,]],
[[	0.086941713	,	0.309734088	,	0.322354439	,	0.251491067	,],
[	0.279736395	,	0.082709642	,	0.240361309	,	0.305864878	,],
[	0.365483659	,	0.315489776	,	0.073896348	,	0.286947682	,],
[	0.30730769	,	0.388376941	,	0.298061178	,	0.087218642	,]],
[[	0.086690775	,	0.312099969	,	0.324757096	,	0.253716558	,],
[	0.281727609	,	0.087579589	,	0.242807595	,	0.307777577	,],
[	0.369773385	,	0.31938984	,	0.076136126	,	0.291605585	,],
[	0.310517898	,	0.391247718	,	0.301275613	,	0.088317146	,]],
[[	0.085935442	,	0.314980426	,	0.327241161	,	0.251545602	,],
[	0.276163727	,	0.083380579	,	0.242343802	,	0.303049183	,],
[	0.360498764	,	0.316039148	,	0.072683827	,	0.282759163	,],
[	0.304342793	,	0.390489866	,	0.300219372	,	0.085284506	,]],
[[	0.088739156	,	0.319407911	,	0.331655241	,	0.256097819	,],
[	0.274876467	,	0.084386848	,	0.241400623	,	0.301826169	,],
[	0.363094775	,	0.319068305	,	0.077901792	,	0.28529775	,],
[	0.308066823	,	0.394158352	,	0.303707362	,	0.09147522	,]],
[[	0.085033333	,	0.314879726	,	0.327190871	,	0.251565353	,],
[	0.279267213	,	0.092191701	,	0.246223649	,	0.305974222	,],
[	0.361495886	,	0.316747482	,	0.072700411	,	0.28381496	,],
[	0.304834917	,	0.391259457	,	0.301031733	,	0.085069985	,]],
[[	0.088788031	,	0.31630156	,	0.328582965	,	0.25301128	,],
[	0.274888522	,	0.082914452	,	0.241183328	,	0.301806499	,],
[	0.361107576	,	0.316606792	,	0.073893901	,	0.283172879	,],
[	0.305083271	,	0.391447415	,	0.30114826	,	0.087007619	,]],
[[	0.086128401	,	0.316883615	,	0.329232639	,	0.253388891	,],
[	0.275858584	,	0.08666898	,	0.242557563	,	0.302807656	,],
[	0.364638688	,	0.319876718	,	0.075463874	,	0.28697835	,],
[	0.307740252	,	0.393689938	,	0.303587725	,	0.087414192	,]],
[[	0.085296404	,	0.314578427	,	0.326932934	,	0.251072823	,],
[	0.277091309	,	0.084095405	,	0.243164769	,	0.303842484	,],
[	0.360710238	,	0.31607203	,	0.072456071	,	0.282874212	,],
[	0.304532253	,	0.390734925	,	0.300301355	,	0.085139267	,]],
[[	0.090084669	,	0.320725848	,	0.332898037	,	0.257453495	,],
[	0.274671521	,	0.083538279	,	0.240879356	,	0.301489386	,],
[	0.362511625	,	0.318339156	,	0.076614047	,	0.284461504	,],
[	0.307109448	,	0.393114706	,	0.302757184	,	0.089961716	,]],
[[	0.085071355	,	0.315239361	,	0.327471692	,	0.251761338	,],
[	0.278406	,	0.090624516	,	0.245170879	,	0.305082359	,],
[	0.362284293	,	0.317538461	,	0.073315617	,	0.284558569	,],
[	0.305352236	,	0.391692654	,	0.301528496	,	0.085364786	,]],
[[	0.087782786	,	0.315890026	,	0.328027863	,	0.252464951	,],
[	0.275302629	,	0.082841184	,	0.241376317	,	0.302088239	,],
[	0.360796555	,	0.31635758	,	0.0733191	,	0.282846158	,],
[	0.304719827	,	0.391067347	,	0.300677592	,	0.086139917	,]],
[[	0.087048415	,	0.317774517	,	0.329991493	,	0.254245117	,],
[	0.275327659	,	0.085497822	,	0.241924061	,	0.302136312	,],
[	0.364667057	,	0.32088062	,	0.080387629	,	0.286748698	,],
[	0.308808528	,	0.394810772	,	0.304837045	,	0.088421517	,]],
[[	0.085315536	,	0.319889573	,	0.330922312	,	0.25111399	,],
[	0.270242465	,	0.08333979	,	0.240803828	,	0.297898079	,],
[	0.358573057	,	0.320497807	,	0.076048514	,	0.280737136	,],
[	0.304733673	,	0.397009671	,	0.305622398	,	0.086963327	,]],
[[	0.087854576	,	0.324443284	,	0.335232015	,	0.255653201	,],
[	0.270998911	,	0.08686374	,	0.241921637	,	0.298563074	,],
[	0.360779255	,	0.322108879	,	0.075391424	,	0.283116899	,],
[	0.306369959	,	0.398697379	,	0.307608539	,	0.086069488	,]],
[[	0.084227763	,	0.319943998	,	0.330834757	,	0.251033884	,],
[	0.2722844	,	0.084353303	,	0.242661922	,	0.299859481	,],
[	0.357482987	,	0.319010979	,	0.073424422	,	0.279540678	,],
[	0.303589861	,	0.396038792	,	0.304637628	,	0.084302841	,]],
[[	0.088187667	,	0.321382635	,	0.332137484	,	0.252595813	,],
[	0.269770531	,	0.083843236	,	0.240379583	,	0.297446663	,],
[	0.36007892	,	0.322203671	,	0.078821735	,	0.282137199	,],
[	0.30658659	,	0.398886948	,	0.307507704	,	0.089911256	,]],
[[	0.085474786	,	0.322115917	,	0.333063352	,	0.253256773	,],
[	0.272828716	,	0.09027729	,	0.243907551	,	0.300280309	,],
[	0.358854935	,	0.320240808	,	0.073795141	,	0.281009809	,],
[	0.304643815	,	0.396862266	,	0.305567847	,	0.084430261	,]],
[[	0.084873593	,	0.319990121	,	0.330865128	,	0.251021543	,],
[	0.270688672	,	0.083349076	,	0.241048432	,	0.298264645	,],
[	0.357993873	,	0.319683785	,	0.075035805	,	0.280113101	,],
[	0.30422284	,	0.396548723	,	0.304948277	,	0.085824157	,]],
[[	0.089197362	,	0.325883778	,	0.336673583	,	0.257124262	,],
[	0.270938538	,	0.086092774	,	0.241606723	,	0.298442024	,],
[	0.361829657	,	0.323012386	,	0.07642443	,	0.28411374	,],
[	0.307327056	,	0.399608825	,	0.308552867	,	0.087021733	,]],
[[	0.084197672	,	0.320234957	,	0.331036255	,	0.251386506	,],
[	0.273358065	,	0.085097216	,	0.243543247	,	0.300936427	,],
[	0.357685485	,	0.319134675	,	0.07316621	,	0.279722295	,],
[	0.303452917	,	0.395956543	,	0.304628559	,	0.083981128	,]],
[[	0.087188472	,	0.320836516	,	0.331615651	,	0.251925883	,],
[	0.269829081	,	0.083502927	,	0.240357348	,	0.297475703	,],
[	0.359313228	,	0.321348115	,	0.077589849	,	0.281358921	,],
[	0.306055454	,	0.398480771	,	0.306947754	,	0.088694122	,]],
[[	0.086246563	,	0.322889832	,	0.333755586	,	0.254046623	,],
[	0.272291013	,	0.089035015	,	0.243288933	,	0.299746571	,],
[	0.359784648	,	0.321074327	,	0.074461277	,	0.281906008	,],
[	0.305332475	,	0.397546891	,	0.30625147	,	0.085088298	,]],
[[	0.085771041	,	0.327879062	,	0.337950663	,	0.252745674	,],
[	0.265150587	,	0.083984897	,	0.242500243	,	0.294391925	,],
[	0.354796559	,	0.322729955	,	0.074958481	,	0.277519136	,],
[	0.304330228	,	0.402785377	,	0.310504808	,	0.087140455	,]],
[[	0.087945771	,	0.332079011	,	0.342063512	,	0.256972129	,],
[	0.265424953	,	0.086985173	,	0.243094936	,	0.294513712	,],
[	0.358667082	,	0.327298351	,	0.082400089	,	0.281532771	,],
[	0.306974564	,	0.405330587	,	0.313481541	,	0.087605043	,]],
[[	0.084377797	,	0.327594319	,	0.337613171	,	0.252344456	,],
[	0.268059018	,	0.085882864	,	0.245207153	,	0.297056037	,],
[	0.354593117	,	0.322302226	,	0.073574278	,	0.277440506	,],
[	0.303541047	,	0.402095196	,	0.309879919	,	0.084982862	,]],
[[	0.088797695	,	0.329348366	,	0.339355485	,	0.254237166	,],
[	0.26453383	,	0.084371003	,	0.242082029	,	0.293699318	,],
[	0.35584287	,	0.324008979	,	0.077385348	,	0.278508004	,],
[	0.306018054	,	0.40449843	,	0.31230971	,	0.089929879	,]],
[[	0.085368266	,	0.329362516	,	0.339513339	,	0.254222394	,],
[	0.267028673	,	0.090017544	,	0.24491401	,	0.295974733	,],
[	0.356702956	,	0.324195317	,	0.074774422	,	0.279520501	,],
[	0.30501142	,	0.403336872	,	0.311330912	,	0.085767928	,]],
[[	0.085269735	,	0.327763221	,	0.337831923	,	0.252515001	,],
[	0.265831934	,	0.08426284	,	0.243054632	,	0.294851747	,],
[	0.354460725	,	0.322278271	,	0.074176776	,	0.277221116	,],
[	0.30397029	,	0.402436541	,	0.310150399	,	0.086441737	,]],
[[	0.08901369	,	0.333180525	,	0.34311365	,	0.25813847	,],
[	0.264908477	,	0.085882567	,	0.242484832	,	0.293979688	,],
[	0.357546741	,	0.326095213	,	0.080675751	,	0.280405442	,],
[	0.308422595	,	0.406578808	,	0.314413468	,	0.093659261	,]],
[[	0.084569452	,	0.328101945	,	0.338132014	,	0.252906918	,],
[	0.269075959	,	0.086944212	,	0.246236261	,	0.298048871	,],
[	0.355084847	,	0.322636568	,	0.073530867	,	0.277835249	,],
[	0.303573185	,	0.402096512	,	0.309935119	,	0.084821235	,]],
[[	0.087585315	,	0.328738915	,	0.338696491	,	0.253432573	,],
[	0.264973346	,	0.084272644	,	0.242291328	,	0.29400675	,],
[	0.355167378	,	0.323283154	,	0.076108581	,	0.277849509	,],
[	0.305436395	,	0.403871682	,	0.311687269	,	0.08890906	,]],
[[	0.086212821	,	0.330418777	,	0.340447217	,	0.255156892	,],
[	0.266431309	,	0.088819882	,	0.244234645	,	0.295342328	,],
[	0.357538349	,	0.324958721	,	0.075458735	,	0.280369949	,],
[	0.305953512	,	0.404221407	,	0.312152943	,	0.086493798	,]],
[[	0.085444929	,	0.332172251	,	0.3417709	,	0.251969437	,],
[	0.261243954	,	0.084615492	,	0.243827708	,	0.291173368	,],
[	0.350539896	,	0.324397577	,	0.075052122	,	0.274084284	,],
[	0.302407106	,	0.406029169	,	0.313533475	,	0.08695203	,]],
[[	0.087293696	,	0.335819428	,	0.345408441	,	0.255907846	,],
[	0.25956224	,	0.085333198	,	0.242572335	,	0.289560594	,],
[	0.353625524	,	0.328176866	,	0.081364506	,	0.277271201	,],
[	0.305427424	,	0.408673143	,	0.316718639	,	0.087685217	,]],
[[	0.083866282	,	0.331599104	,	0.341172291	,	0.251565326	,],
[	0.264131499	,	0.093316453	,	0.247623709	,	0.293857109	,],
[	0.351285284	,	0.324793539	,	0.074473327	,	0.274809999	,],
[	0.301853772	,	0.405449918	,	0.313022961	,	0.08496511	,]],
[[	0.088378918	,	0.333570286	,	0.343036396	,	0.253447285	,],
[	0.259964024	,	0.084048773	,	0.242700449	,	0.289885782	,],
[	0.351674015	,	0.325658099	,	0.077195101	,	0.274982252	,],
[	0.304163628	,	0.407661573	,	0.315266158	,	0.089760201	,]],
[[	0.084858965	,	0.333458715	,	0.343122097	,	0.253274399	,],
[	0.260743781	,	0.087896794	,	0.244074918	,	0.290692841	,],
[	0.353983147	,	0.327351679	,	0.076725507	,	0.277588948	,],
[	0.30327786	,	0.406588138	,	0.314373969	,	0.08551961	,]],
[[	0.084730579	,	0.331681245	,	0.341409734	,	0.251515252	,],
[	0.262350393	,	0.085342921	,	0.244704321	,	0.292052371	,],
[	0.350569289	,	0.324313242	,	0.074589239	,	0.274094753	,],
[	0.302378211	,	0.405880782	,	0.313436535	,	0.086262104	,]],
[[	0.088606186	,	0.337232159	,	0.346682067	,	0.25721162	,],
[	0.25984108	,	0.084981146	,	0.242677419	,	0.289745938	,],
[	0.352716743	,	0.327147616	,	0.079772718	,	0.276189537	,],
[	0.306519937	,	0.409770862	,	0.317735517	,	0.088718067	,]],
[[	0.084064066	,	0.332049469	,	0.341568676	,	0.251882449	,],
[	0.263105208	,	0.091795287	,	0.246350173	,	0.292808239	,],
[	0.352094309	,	0.32550665	,	0.075104965	,	0.27546475	,],
[	0.301993074	,	0.405502721	,	0.313191981	,	0.084979867	,]],
[[	0.087359197	,	0.333055898	,	0.342528766	,	0.252817783	,],
[	0.260486562	,	0.084154606	,	0.243108797	,	0.290359868	,],
[	0.350860781	,	0.324861197	,	0.076022956	,	0.274236757	,],
[	0.303600632	,	0.40707135	,	0.314642682	,	0.088651863	,]],
[[	0.085926633	,	0.33460996	,	0.344210556	,	0.254516419	,],
[	0.260438962	,	0.087010388	,	0.243505757	,	0.290258646	,],
[	0.354995551	,	0.328259737	,	0.077603405	,	0.278535623	,],
[	0.304353574	,	0.407598709	,	0.31542818	,	0.086575795	,]],
[[	0.084982247	,	0.336424592	,	0.345288176	,	0.252080395	,],
[	0.257317177	,	0.085109111	,	0.244206825	,	0.287817104	,],
[	0.346463904	,	0.325052802	,	0.074805377	,	0.270419922	,],
[	0.301333904	,	0.409909556	,	0.31689628	,	0.086579156	,]],
[[	0.087061568	,	0.340428908	,	0.349282642	,	0.25635528	,],
[	0.255203627	,	0.085110495	,	0.242495226	,	0.285691207	,],
[	0.348855046	,	0.328250221	,	0.08041396	,	0.272889569	,],
[	0.304912438	,	0.41323966	,	0.320734718	,	0.088212345	,]],
[[	0.083384821	,	0.335715746	,	0.344655251	,	0.251517061	,],
[	0.25820472	,	0.091654078	,	0.246108471	,	0.28865807	,],
[	0.347632049	,	0.3258907	,	0.074892656	,	0.271541588	,],
[	0.300686926	,	0.409417596	,	0.316421316	,	0.084959462	,]],
[[	0.088078487	,	0.338111491	,	0.346838691	,	0.253722121	,],
[	0.255517914	,	0.083992045	,	0.242605383	,	0.286090618	,],
[	0.34703667	,	0.325854335	,	0.076467688	,	0.270838135	,],
[	0.302536889	,	0.411275325	,	0.318295774	,	0.088870487	,]],
[[	0.084422417	,	0.337923869	,	0.346839652	,	0.253631055	,],
[	0.255758005	,	0.087014228	,	0.243370317	,	0.286235577	,],
[	0.351000477	,	0.32915756	,	0.077804905	,	0.274918902	,],
[	0.302625099	,	0.411006536	,	0.318202585	,	0.086009208	,]],
[[	0.084400136	,	0.336253509	,	0.345230591	,	0.251787082	,],
[	0.258406027	,	0.085914652	,	0.245191813	,	0.288772904	,],
[	0.346402776	,	0.324924767	,	0.074347981	,	0.270378803	,],
[	0.301040576	,	0.409602727	,	0.316487318	,	0.085809543	,]],
[[	0.088349181	,	0.341731336	,	0.350525001	,	0.257546436	,],
[	0.255226553	,	0.084575754	,	0.242290291	,	0.285671902	,],
[	0.348375537	,	0.327456929	,	0.079132736	,	0.272183356	,],
[	0.305188121	,	0.413645489	,	0.320505609	,	0.092552282	,]],
[[	0.083541895	,	0.336416266	,	0.345251157	,	0.252052958	,],
[	0.257687778	,	0.090354736	,	0.245403834	,	0.287999112	,],
[	0.348338549	,	0.326536806	,	0.075423688	,	0.272261304	,],
[	0.301393403	,	0.409957074	,	0.317097774	,	0.085185711	,]],
[[	0.086711127	,	0.337255296	,	0.346054608	,	0.252840263	,],
[	0.256214337	,	0.084291442	,	0.243117351	,	0.286624316	,],
[	0.346637714	,	0.32533723	,	0.075465467	,	0.270384276	,],
[	0.302221509	,	0.410943966	,	0.317870208	,	0.087984506	,]],
[[	0.08551186	,	0.338957308	,	0.347939481	,	0.254656162	,],
[	0.255463642	,	0.085966847	,	0.242809386	,	0.285909534	,],
[	0.350400621	,	0.329817811	,	0.082672162	,	0.274176862	,],
[	0.303732171	,	0.412049467	,	0.319400466	,	0.086981928	,]],
[[	0.084594599	,	0.340454417	,	0.349228519	,	0.251680444	,],
[	0.251490805	,	0.08591883	,	0.245156188	,	0.283123113	,],
[	0.340380072	,	0.325532825	,	0.07444508	,	0.26523242	,],
[	0.298675181	,	0.413217823	,	0.320107094	,	0.085017513	,]],
[[	0.086826629	,	0.344463057	,	0.353112061	,	0.255877816	,],
[	0.248519377	,	0.085034397	,	0.242530984	,	0.280297179	,],
[	0.342899929	,	0.328772676	,	0.080027897	,	0.267796451	,],
[	0.302203695	,	0.416470846	,	0.323806616	,	0.086313127	,]],
[[	0.083298007	,	0.340086368	,	0.34884191	,	0.25144893	,],
[	0.251455318	,	0.09149292	,	0.246048881	,	0.283053763	,],
[	0.341634136	,	0.326570938	,	0.074544147	,	0.266478477	,],
[	0.298106405	,	0.41268589	,	0.319636823	,	0.083183565	,]],
[[	0.087782507	,	0.34198511	,	0.350514418	,	0.253235041	,],
[	0.249418928	,	0.084469421	,	0.243258178	,	0.281134803	,],
[	0.34088849	,	0.326380373	,	0.075923925	,	0.265661732	,],
[	0.300108547	,	0.414582255	,	0.321582731	,	0.087443657	,]],
[[	0.084219665	,	0.341916715	,	0.350656624	,	0.253228757	,],
[	0.248972731	,	0.08677669	,	0.243338936	,	0.280633835	,],
[	0.345251427	,	0.329992632	,	0.077541756	,	0.270210126	,],
[	0.300059277	,	0.414200105	,	0.321397224	,	0.084348912	,]],
[[	0.083929489	,	0.34002815	,	0.348798717	,	0.251277149	,],
[	0.252348	,	0.086746295	,	0.245960045	,	0.283941782	,],
[	0.34071388	,	0.325750888	,	0.074142297	,	0.265509827	,],
[	0.298191638	,	0.412678993	,	0.319616703	,	0.084066653	,]],
[[	0.088025878	,	0.345736135	,	0.354284448	,	0.25720069	,],
[	0.248728399	,	0.084604834	,	0.242568773	,	0.280272251	,],
[	0.342326975	,	0.327899385	,	0.078595194	,	0.26696141	,],
[	0.30242156	,	0.416720772	,	0.323579475	,	0.090975152	,]],
[[	0.083512945	,	0.340594477	,	0.34924364	,	0.251879858	,],
[	0.250493119	,	0.089915834	,	0.245029528	,	0.282119107	,],
[	0.342600232	,	0.327412111	,	0.07514153	,	0.26735689	,],
[	0.298412816	,	0.412883564	,	0.319945124	,	0.083144869	,]],
[[	0.086493483	,	0.341360676	,	0.350007697	,	0.252545186	,],
[	0.250312669	,	0.084960989	,	0.243908053	,	0.281858603	,],
[	0.340797051	,	0.326083575	,	0.075340678	,	0.265496897	,],
[	0.299279601	,	0.41383389	,	0.320684731	,	0.086153551	,]],
[[	0.085108261	,	0.342758606	,	0.351575878	,	0.254126491	,],
[	0.248657763	,	0.085946029	,	0.24273228	,	0.28034717	,],
[	0.344392484	,	0.330373375	,	0.082235791	,	0.269139291	,],
[	0.30099427	,	0.415219715	,	0.322328201	,	0.085159353	,]],
[[	0.084609405	,	0.347958512	,	0.355173009	,	0.252893153	,],
[	0.247475961	,	0.087362832	,	0.246132019	,	0.280084556	,],
[	0.33823859	,	0.329446781	,	0.075057127	,	0.262908413	,],
[	0.298284647	,	0.418698381	,	0.324457774	,	0.085064454	,]],
[[	0.085884545	,	0.351260025	,	0.358497729	,	0.256472832	,],
[	0.243990717	,	0.086054771	,	0.243198382	,	0.276727318	,],
[	0.340391957	,	0.332367666	,	0.080478775	,	0.265098275	,],
[	0.302081822	,	0.422090239	,	0.328236712	,	0.08680461	,]],
[[	0.082629895	,	0.347158949	,	0.35436821	,	0.252141132	,],
[	0.246651831	,	0.092025448	,	0.246317799	,	0.279192557	,],
[	0.339187473	,	0.3301649	,	0.075048637	,	0.263980984	,],
[	0.297789197	,	0.418158941	,	0.324040016	,	0.083359864	,]],
[[	0.08754899	,	0.349333812	,	0.356415863	,	0.254324877	,],
[	0.245199863	,	0.085767864	,	0.24405165	,	0.277829038	,],
[	0.338277483	,	0.329724377	,	0.076305516	,	0.262897283	,],
[	0.299920363	,	0.420361919	,	0.326095268	,	0.087740891	,]],
[[	0.083305599	,	0.348702872	,	0.35597344	,	0.253683807	,],
[	0.244268144	,	0.087633332	,	0.243759631	,	0.277000433	,],
[	0.34267615	,	0.333482828	,	0.077978471	,	0.267530914	,],
[	0.299618656	,	0.419764678	,	0.325677645	,	0.08436326	,]],
[[	0.083843572	,	0.347433369	,	0.354745551	,	0.252410603	,],
[	0.248438341	,	0.088179151	,	0.247024091	,	0.280975563	,],
[	0.338204187	,	0.32940295	,	0.07459682	,	0.26293423	,],
[	0.29816876	,	0.418477566	,	0.324187674	,	0.084373355	,]],
[[	0.087034132	,	0.352362345	,	0.359435306	,	0.257478672	,],
[	0.244313392	,	0.085792042	,	0.243239437	,	0.276863852	,],
[	0.33960437	,	0.331552575	,	0.078981399	,	0.264444633	,],
[	0.302005148	,	0.422095275	,	0.327825048	,	0.09095527	,]],
[[	0.08281276	,	0.347519553	,	0.354693288	,	0.252460488	,],
[	0.246008998	,	0.090824612	,	0.245612358	,	0.278605417	,],
[	0.340165877	,	0.331110829	,	0.075784821	,	0.264904961	,],
[	0.298132817	,	0.418377147	,	0.324212182	,	0.083347549	,]],
[[	0.086427196	,	0.348755058	,	0.355893836	,	0.253621198	,],
[	0.246003527	,	0.086318967	,	0.244834682	,	0.278654566	,],
[	0.33822907	,	0.329580631	,	0.075612059	,	0.262820492	,],
[	0.298849144	,	0.419295823	,	0.324929107	,	0.086125125	,]],
[[	0.084419132	,	0.349730336	,	0.35696833	,	0.254774604	,],
[	0.244268882	,	0.086957874	,	0.24345282	,	0.276885732	,],
[	0.341679465	,	0.333842969	,	0.082529213	,	0.266321383	,],
[	0.300831162	,	0.420793673	,	0.326898471	,	0.08549106	,]],
[[	0.097655891	,	0.227039508	,	0.259637536	,	0.271111113	,],
[	0.371813111	,	0.07816366	,	0.24669501	,	0.389514229	,],
[	0.440951731	,	0.290664072	,	0.07095722	,	0.358024373	,],
[	0.335371425	,	0.311939498	,	0.23449411	,	0.092411307	,]],
[[	0.091842822	,	0.220288879	,	0.253067569	,	0.264687108	,],
[	0.372024652	,	0.080446358	,	0.24698577	,	0.389696552	,],
[	0.443486955	,	0.293644634	,	0.076466931	,	0.360613265	,],
[	0.338529309	,	0.315626425	,	0.238404665	,	0.09325873	,]],
[[	0.093650735	,	0.219427096	,	0.252228898	,	0.264163013	,],
[	0.374684537	,	0.080431569	,	0.249653108	,	0.392313688	,],
[	0.441860845	,	0.291606237	,	0.071170012	,	0.359037005	,],
[	0.334715528	,	0.311476815	,	0.23418435	,	0.090470353	,]],
[[	0.094226504	,	0.223667359	,	0.256265005	,	0.267758423	,],
[	0.371124667	,	0.078063158	,	0.246008293	,	0.388810781	,],
[	0.441275703	,	0.29115042	,	0.072413797	,	0.358287678	,],
[	0.337145339	,	0.313381889	,	0.236048521	,	0.095044151	,]],
[[	0.091667247	,	0.219365461	,	0.252110122	,	0.26372664	,],
[	0.373337641	,	0.083163181	,	0.248452109	,	0.390844969	,],
[	0.44475287	,	0.294749106	,	0.073798572	,	0.362123559	,],
[	0.336364758	,	0.313327085	,	0.235982913	,	0.09116601	,]],
[[	0.096615678	,	0.221169899	,	0.253967213	,	0.266038704	,],
[	0.372609104	,	0.078703948	,	0.247422202	,	0.390100262	,],
[	0.440914558	,	0.29059269	,	0.070646105	,	0.357933401	,],
[	0.33509678	,	0.311803085	,	0.234313473	,	0.091562215	,]],
[[	0.092341195	,	0.221109883	,	0.253652683	,	0.265244009	,],
[	0.371289501	,	0.079305848	,	0.246129803	,	0.388849274	,],
[	0.442728091	,	0.29276113	,	0.075139987	,	0.359735058	,],
[	0.339050381	,	0.315234762	,	0.238041794	,	0.098218066	,]],
[[	0.092979778	,	0.219408674	,	0.252144579	,	0.264018503	,],
[	0.375530398	,	0.086975227	,	0.250927731	,	0.392923461	,],
[	0.442584444	,	0.292357793	,	0.071722665	,	0.359710966	,],
[	0.334956903	,	0.311731368	,	0.234190464	,	0.0902953	,]],
[[	0.095528529	,	0.224952984	,	0.257564709	,	0.268896807	,],
[	0.371445317	,	0.078077078	,	0.246236774	,	0.38906346	,],
[	0.441167123	,	0.290995266	,	0.071749425	,	0.35812253	,],
[	0.336405124	,	0.312764721	,	0.23530723	,	0.093776933	,]],
[[	0.091701667	,	0.219700355	,	0.252432055	,	0.264124589	,],
[	0.372432475	,	0.081621776	,	0.247389652	,	0.389936372	,],
[	0.444575822	,	0.294909691	,	0.07851229	,	0.361668637	,],
[	0.337182045	,	0.314202888	,	0.236852852	,	0.091853725	,]],
[[	0.096127056	,	0.225597295	,	0.258185111	,	0.2656841	,],
[	0.372054597	,	0.085178446	,	0.250736789	,	0.389585595	,],
[	0.440844673	,	0.293488233	,	0.07263109	,	0.357600082	,],
[	0.333920532	,	0.315020227	,	0.237534926	,	0.089734908	,]],
[[	0.092516229	,	0.225895887	,	0.25826531	,	0.265458613	,],
[	0.36871309	,	0.07753724	,	0.247003609	,	0.386346929	,],
[	0.43889822	,	0.291376336	,	0.071811003	,	0.355391589	,],
[	0.336311688	,	0.317087058	,	0.239624224	,	0.094413213	,]],
[[	0.092836863	,	0.224032848	,	0.256508248	,	0.263936826	,],
[	0.36928169	,	0.080464273	,	0.247746871	,	0.386808572	,],
[	0.442071625	,	0.294909659	,	0.077825197	,	0.358605011	,],
[	0.335482395	,	0.316677241	,	0.239219015	,	0.090503191	,]],
[[	0.095672807	,	0.229835326	,	0.262171297	,	0.269052499	,],
[	0.371154896	,	0.079429467	,	0.249533603	,	0.388807747	,],
[	0.439431159	,	0.291982241	,	0.071402454	,	0.356018818	,],
[	0.334452188	,	0.315286533	,	0.237637782	,	0.090924704	,]],
[[	0.091429824	,	0.22425374	,	0.256805441	,	0.263859528	,],
[	0.368464077	,	0.078146438	,	0.246812804	,	0.386073662	,],
[	0.439473758	,	0.29202377	,	0.073368876	,	0.355972682	,],
[	0.33892392	,	0.320243595	,	0.242962162	,	0.09368041	,]],
[[	0.095093729	,	0.225108512	,	0.257729932	,	0.26511256	,],
[	0.370895943	,	0.08338857	,	0.249492655	,	0.388466243	,],
[	0.441629041	,	0.29425555	,	0.073287862	,	0.358352051	,],
[	0.33437543	,	0.315456834	,	0.237947701	,	0.089778422	,]],
[[	0.092976352	,	0.226705571	,	0.259033722	,	0.266110665	,],
[	0.369369152	,	0.077946296	,	0.247552115	,	0.386892268	,],
[	0.438926425	,	0.291294134	,	0.07138866	,	0.355309706	,],
[	0.335848764	,	0.316604752	,	0.239029053	,	0.093342178	,]],
[[	0.091978575	,	0.223754882	,	0.256159281	,	0.263549594	,],
[	0.368873151	,	0.079487246	,	0.247210327	,	0.386453407	,],
[	0.441119682	,	0.293925656	,	0.076345414	,	0.357634741	,],
[	0.336433789	,	0.317625289	,	0.240129793	,	0.091097752	,]],
[[	0.097280971	,	0.231492755	,	0.263811692	,	0.270545009	,],
[	0.372321103	,	0.080616991	,	0.250758825	,	0.389959856	,],
[	0.439909185	,	0.29238104	,	0.071687188	,	0.356458518	,],
[	0.334254492	,	0.315144813	,	0.23748027	,	0.090329112	,]],
[[	0.091857111	,	0.224949755	,	0.257408135	,	0.264409991	,],
[	0.368387909	,	0.077520189	,	0.246653313	,	0.385970514	,],
[	0.439186058	,	0.291631623	,	0.072670008	,	0.355452821	,],
[	0.338036526	,	0.318501974	,	0.240978877	,	0.096698419	,]],
[[	0.094978012	,	0.228620503	,	0.260603431	,	0.263672119	,],
[	0.36136549	,	0.081564622	,	0.246761771	,	0.380976244	,],
[	0.433973339	,	0.294246793	,	0.079259523	,	0.353044461	,],
[	0.328719059	,	0.316722241	,	0.239996595	,	0.090006794	,]],
[[	0.090242728	,	0.227983415	,	0.259760396	,	0.262536015	,],
[	0.361144446	,	0.077777897	,	0.246247693	,	0.380747534	,],
[	0.429911134	,	0.289769097	,	0.070914123	,	0.349057489	,],
[	0.329946109	,	0.317556761	,	0.240961861	,	0.09349475	,]],
[[	0.091216808	,	0.226523958	,	0.258473518	,	0.261518456	,],
[	0.35948949	,	0.077970125	,	0.244685387	,	0.379248912	,],
[	0.431092255	,	0.291010822	,	0.074304239	,	0.350117989	,],
[	0.330515617	,	0.318660576	,	0.242050356	,	0.091190972	,]],
[[	0.093470642	,	0.231969496	,	0.263686907	,	0.266241577	,],
[	0.363287448	,	0.084964952	,	0.248862583	,	0.382820943	,],
[	0.431709161	,	0.291601765	,	0.072305094	,	0.350955029	,],
[	0.328564201	,	0.316245461	,	0.239573127	,	0.090530866	,]],
[[	0.089307208	,	0.226432764	,	0.258342186	,	0.261077769	,],
[	0.359959682	,	0.077090263	,	0.24510548	,	0.37961628	,],
[	0.429453368	,	0.289241776	,	0.071225333	,	0.34852298	,],
[	0.332437828	,	0.319800265	,	0.243120972	,	0.096948236	,]],
[[	0.093657949	,	0.227814575	,	0.259712457	,	0.26274707	,],
[	0.360575339	,	0.080212718	,	0.245962874	,	0.380229269	,],
[	0.433154703	,	0.293295721	,	0.07785788	,	0.352079347	,],
[	0.328782482	,	0.316827005	,	0.240210069	,	0.089888403	,]],
[[	0.09095721	,	0.228970086	,	0.260620616	,	0.263348774	,],
[	0.361775171	,	0.07833322	,	0.246889511	,	0.38134005	,],
[	0.430534967	,	0.290253808	,	0.071256306	,	0.349590288	,],
[	0.329648539	,	0.317336539	,	0.240695721	,	0.092489855	,]],
[[	0.090437102	,	0.226320072	,	0.258232509	,	0.261137809	,],
[	0.359340375	,	0.077364984	,	0.244456885	,	0.378969465	,],
[	0.430904289	,	0.290696078	,	0.073604334	,	0.349698889	,],
[	0.331742569	,	0.319783556	,	0.243277001	,	0.09218846	,]],
[[	0.094844571	,	0.233384581	,	0.264940802	,	0.267440781	,],
[	0.362534264	,	0.083646621	,	0.248047189	,	0.382100353	,],
[	0.432704545	,	0.292637012	,	0.073127573	,	0.351974581	,],
[	0.328465315	,	0.316289043	,	0.239559466	,	0.09013069	,]],
[[	0.089797733	,	0.227208035	,	0.258983532	,	0.261640029	,],
[	0.360375891	,	0.077173861	,	0.245371218	,	0.379859468	,],
[	0.430058315	,	0.289759016	,	0.071500688	,	0.349018354	,],
[	0.331551885	,	0.319034552	,	0.242339224	,	0.095653001	,]],
[[	0.092699496	,	0.231580603	,	0.262804077	,	0.260906758	,],
[	0.352988838	,	0.080853872	,	0.244595571	,	0.373842408	,],
[	0.425684024	,	0.292796098	,	0.076677844	,	0.34592185	,],
[	0.325195691	,	0.319049598	,	0.242736856	,	0.088930088	,]],
[[	0.089329272	,	0.231866312	,	0.262890219	,	0.260729415	,],
[	0.353941752	,	0.078505231	,	0.245271251	,	0.374764609	,],
[	0.422799114	,	0.289608636	,	0.069931092	,	0.34325885	,],
[	0.324828769	,	0.318323108	,	0.24191136	,	0.090272165	,]],
[[	0.089327281	,	0.229772646	,	0.260925181	,	0.258928201	,],
[	0.351732166	,	0.077886617	,	0.243029905	,	0.372530259	,],
[	0.423147317	,	0.289968331	,	0.072222348	,	0.343368976	,],
[	0.328226468	,	0.321893068	,	0.245695148	,	0.091423204	,]],
[[	0.093022896	,	0.236178172	,	0.267129085	,	0.26461164	,],
[	0.354961244	,	0.084231701	,	0.246712949	,	0.375710758	,],
[	0.425064356	,	0.291892937	,	0.071895069	,	0.345697958	,],
[	0.324470569	,	0.317922957	,	0.241579157	,	0.088492338	,]],
[[	0.088389289	,	0.230287749	,	0.261483604	,	0.259245158	,],
[	0.352316979	,	0.077424207	,	0.243616369	,	0.373149305	,],
[	0.422194497	,	0.288879915	,	0.069976077	,	0.342430586	,],
[	0.32687253	,	0.320114122	,	0.243646992	,	0.093243207	,]],
[[	0.091865334	,	0.231064356	,	0.262259881	,	0.260264909	,],
[	0.352701505	,	0.080103924	,	0.244231018	,	0.373660329	,],
[	0.424896522	,	0.291912809	,	0.07521664	,	0.345070282	,],
[	0.32595279	,	0.319828621	,	0.24356499	,	0.089509411	,]],
[[	0.09020979	,	0.232979183	,	0.26389472	,	0.261695386	,],
[	0.355195165	,	0.079577997	,	0.246449204	,	0.375918692	,],
[	0.423255066	,	0.28997554	,	0.070176919	,	0.343618996	,],
[	0.324603998	,	0.318165591	,	0.241759895	,	0.089604955	,]],
[[	0.08895515	,	0.229898089	,	0.26098395	,	0.258890464	,],
[	0.35169564	,	0.077577418	,	0.243014591	,	0.372464626	,],
[	0.422717437	,	0.289502293	,	0.071272278	,	0.342836347	,],
[	0.329168111	,	0.323014419	,	0.24677124	,	0.092421511	,]],
[[	0.09423746	,	0.237440723	,	0.268431935	,	0.265922533	,],
[	0.354098135	,	0.082839104	,	0.245813882	,	0.374920257	,],
[	0.426143572	,	0.293071825	,	0.072935624	,	0.346722973	,],
[	0.324722623	,	0.318307824	,	0.241912924	,	0.08853244	,]],
[[	0.088727306	,	0.230878142	,	0.261960784	,	0.259671295	,],
[	0.352754578	,	0.077535871	,	0.24408969	,	0.373552365	,],
[	0.422216394	,	0.288941299	,	0.069697987	,	0.34248723	,],
[	0.326282754	,	0.3195678	,	0.243091686	,	0.092163397	,]],
[[	0.091701031	,	0.234882611	,	0.26487135	,	0.257946025	,],
[	0.344737654	,	0.078738477	,	0.241857651	,	0.365963274	,],
[	0.416238675	,	0.289446329	,	0.073489306	,	0.337931565	,],
[	0.320452287	,	0.320683358	,	0.244822155	,	0.088115171	,]],
[[	0.089155738	,	0.235783605	,	0.26562082	,	0.258356889	,],
[	0.348640841	,	0.080098067	,	0.245555837	,	0.369788249	,],
[	0.416563606	,	0.289630762	,	0.070738535	,	0.338617725	,],
[	0.319570191	,	0.319456371	,	0.243452648	,	0.088728104	,]],
[[	0.088664948	,	0.233464684	,	0.263240729	,	0.256162501	,],
[	0.344592897	,	0.07718113	,	0.241538059	,	0.365730895	,],
[	0.41439347	,	0.287499278	,	0.069946469	,	0.336102788	,],
[	0.324095092	,	0.324255903	,	0.248386357	,	0.091245934	,]],
[[	0.093001605	,	0.240237453	,	0.269991487	,	0.262455215	,],
[	0.345923407	,	0.08124709	,	0.243227902	,	0.367065366	,],
[	0.418181881	,	0.291734849	,	0.07688588	,	0.339951349	,],
[	0.319436516	,	0.319484807	,	0.243432348	,	0.087434913	,]],
[[	0.087996712	,	0.234114175	,	0.264088382	,	0.256743118	,],
[	0.346340429	,	0.07810094	,	0.243301915	,	0.367477546	,],
[	0.415010942	,	0.287989231	,	0.069541036	,	0.336752441	,],
[	0.321286258	,	0.32096992	,	0.244914042	,	0.091405147	,]],
[[	0.090507759	,	0.234230061	,	0.264246214	,	0.257110935	,],
[	0.344309691	,	0.07781202	,	0.24129887	,	0.365424471	,],
[	0.415458784	,	0.288556774	,	0.072170325	,	0.337060123	,],
[	0.321230602	,	0.321459419	,	0.24554948	,	0.088641854	,]],
[[	0.090198871	,	0.23701976	,	0.266638487	,	0.259414169	,],
[	0.348009134	,	0.084902882	,	0.245629287	,	0.369080622	,],
[	0.41732776	,	0.290395207	,	0.071461735	,	0.339279639	,],
[	0.31964707	,	0.319588906	,	0.243584247	,	0.088418567	,]],
[[	0.088068988	,	0.233278964	,	0.263166138	,	0.255998705	,],
[	0.34493234	,	0.07726248	,	0.241868379	,	0.366064936	,],
[	0.414793389	,	0.287708765	,	0.07001559	,	0.336375967	,],
[	0.323474517	,	0.323118886	,	0.247177241	,	0.094851684	,]],
[[	0.094001424	,	0.236065905	,	0.26586447	,	0.259040519	,],
[	0.345280482	,	0.0800369	,	0.242517553	,	0.366350924	,],
[	0.417257823	,	0.290528748	,	0.075245502	,	0.338857694	,],
[	0.319834354	,	0.319878546	,	0.243827345	,	0.087673437	,]],
[[	0.088245594	,	0.234805135	,	0.264654945	,	0.25728321	,],
[	0.34716753	,	0.078798211	,	0.244085216	,	0.368183773	,],
[	0.415663328	,	0.288579063	,	0.069981453	,	0.337355268	,],
[	0.320430586	,	0.320169136	,	0.244189951	,	0.090224241	,]],
[[	0.090212461	,	0.238209446	,	0.267597332	,	0.25708283	,],
[	0.338780798	,	0.077585306	,	0.240981423	,	0.36152566	,],
[	0.41071123	,	0.288861802	,	0.07202131	,	0.333128859	,],
[	0.318959585	,	0.324672672	,	0.248533869	,	0.08856781	,]],
[[	0.088520255	,	0.239871169	,	0.269046618	,	0.25820029	,],
[	0.342042865	,	0.084060662	,	0.244710001	,	0.364619433	,],
[	0.413188233	,	0.2913183	,	0.072092939	,	0.336020078	,],
[	0.316759755	,	0.322292178	,	0.246152379	,	0.087815277	,]],
[[	0.087585892	,	0.237099639	,	0.26634375	,	0.255735934	,],
[	0.339436228	,	0.077038203	,	0.241438769	,	0.362049532	,],
[	0.409965883	,	0.287945877	,	0.069827294	,	0.332500714	,],
[	0.320207574	,	0.325436624	,	0.249337694	,	0.093887013	,]],
[[	0.092435618	,	0.244366267	,	0.273508348	,	0.262509113	,],
[	0.339515402	,	0.079470869	,	0.241857887	,	0.362202614	,],
[	0.412413073	,	0.290699106	,	0.07496079	,	0.334830132	,],
[	0.317490998	,	0.322802457	,	0.246704955	,	0.087431997	,]],
[[	0.086947101	,	0.237929056	,	0.26714186	,	0.256316551	,],
[	0.341958207	,	0.078990063	,	0.244045767	,	0.364502581	,],
[	0.411159986	,	0.289174642	,	0.070248698	,	0.333675975	,],
[	0.317879362	,	0.323006096	,	0.246825199	,	0.089765944	,]],
[[	0.089202796	,	0.237630054	,	0.26705049	,	0.256386632	,],
[	0.338698854	,	0.077175866	,	0.240881239	,	0.36139924	,],
[	0.410490582	,	0.288489109	,	0.071243474	,	0.33279124	,],
[	0.319848384	,	0.325555363	,	0.249590566	,	0.089404488	,]],
[[	0.089261857	,	0.240953647	,	0.269959341	,	0.259066039	,],
[	0.341355405	,	0.082671127	,	0.243915308	,	0.363900744	,],
[	0.414344123	,	0.292500925	,	0.07324485	,	0.337111401	,],
[	0.316818357	,	0.322261674	,	0.246151544	,	0.087490699	,]],
[[	0.087018158	,	0.237002781	,	0.266198302	,	0.255565121	,],
[	0.340042521	,	0.07729414	,	0.241909118	,	0.362491564	,],
[	0.410320814	,	0.288229137	,	0.069647603	,	0.332739363	,],
[	0.319512493	,	0.324721641	,	0.248628714	,	0.092701146	,]],
[[	0.092513487	,	0.239436413	,	0.268647969	,	0.258261583	,],
[	0.339026495	,	0.078653898	,	0.241369674	,	0.361686736	,],
[	0.41186834	,	0.290020109	,	0.073789395	,	0.334087519	,],
[	0.318043291	,	0.323520057	,	0.247427909	,	0.087809415	,]],
[[	0.087692749	,	0.238957907	,	0.268181107	,	0.257258409	,],
[	0.343256584	,	0.080150175	,	0.245242867	,	0.365714905	,],
[	0.412130616	,	0.290205812	,	0.071126713	,	0.3346976	,],
[	0.317527256	,	0.32278418	,	0.246584655	,	0.089085287	,]],
[[	0.088973516	,	0.243309455	,	0.271666547	,	0.256007738	,],
[	0.333120357	,	0.076963409	,	0.241096135	,	0.356627679	,],
[	0.404629139	,	0.288652205	,	0.070363548	,	0.32776517	,],
[	0.317974005	,	0.329543305	,	0.253508636	,	0.089493853	,]],
[[	0.08941239	,	0.246689969	,	0.274855027	,	0.259019277	,],
[	0.334414051	,	0.08106476	,	0.242774606	,	0.357860462	,],
[	0.408629973	,	0.293202535	,	0.077795946	,	0.331931552	,],
[	0.314509417	,	0.32592562	,	0.249729995	,	0.087024187	,]],
[[	0.086926994	,	0.242602779	,	0.270911609	,	0.255145105	,],
[	0.33490957	,	0.077865317	,	0.242790295	,	0.358343131	,],
[	0.404344678	,	0.288341842	,	0.06916672	,	0.327754353	,],
[	0.316856109	,	0.327931812	,	0.251782199	,	0.091554204	,]],
[[	0.092382703	,	0.245061603	,	0.273351035	,	0.257929673	,],
[	0.332894867	,	0.077763835	,	0.240937956	,	0.356347361	,],
[	0.40576881	,	0.290096436	,	0.072887117	,	0.329081561	,],
[	0.315907145	,	0.327330676	,	0.251106251	,	0.087598878	,]],
[[	0.087078369	,	0.244077559	,	0.272368116	,	0.256521767	,],
[	0.336766126	,	0.08470141	,	0.245219562	,	0.359984156	,],
[	0.40626969	,	0.290305641	,	0.070456014	,	0.329827838	,],
[	0.314926692	,	0.325956439	,	0.249789235	,	0.088043754	,]],
[[	0.088367407	,	0.243164953	,	0.271492735	,	0.255717297	,],
[	0.33334697	,	0.077009297	,	0.241322075	,	0.356839129	,],
[	0.404469485	,	0.288436098	,	0.069883664	,	0.327647393	,],
[	0.319085568	,	0.33072562	,	0.254710831	,	0.090428635	,]],
[[	0.090478474	,	0.247923248	,	0.276003778	,	0.260058746	,],
[	0.333689068	,	0.079671068	,	0.241833365	,	0.357185228	,],
[	0.407782602	,	0.292303444	,	0.076281897	,	0.330967829	,],
[	0.314800267	,	0.326174566	,	0.249990741	,	0.086841531	,]],
[[	0.086949003	,	0.243062282	,	0.271351419	,	0.255518166	,],
[	0.33597056	,	0.078709942	,	0.243719759	,	0.35908195	,],
[	0.404725072	,	0.288723877	,	0.069213672	,	0.328055945	,],
[	0.316079054	,	0.327164015	,	0.250943198	,	0.090173278	,]],
[[	0.091110795	,	0.244506806	,	0.272796646	,	0.257149286	,],
[	0.332950301	,	0.077414483	,	0.240820592	,	0.356306058	,],
[	0.405270841	,	0.289480616	,	0.071838691	,	0.328439853	,],
[	0.316544988	,	0.327832759	,	0.251695402	,	0.087930327	,]],
[[	0.087936341	,	0.245112629	,	0.273418669	,	0.257439938	,],
[	0.335648472	,	0.083145289	,	0.244012081	,	0.358922235	,],
[	0.407480921	,	0.291510897	,	0.071494051	,	0.330893134	,],
[	0.314579756	,	0.325724734	,	0.249510031	,	0.087282824	,]],
[[	0.090152019	,	0.249083576	,	0.2779991	,	0.257049337	,],
[	0.329186905	,	0.07704338	,	0.241592175	,	0.353458337	,],
[	0.401231526	,	0.289602467	,	0.070309609	,	0.32484442	,],
[	0.316016968	,	0.33361089	,	0.257374371	,	0.089034003	,]],
[[	0.088048915	,	0.25048442	,	0.27929207	,	0.258058892	,],
[	0.329991186	,	0.08050704	,	0.242898742	,	0.354339708	,],
[	0.404389761	,	0.293141031	,	0.07624613	,	0.327966108	,],
[	0.313199863	,	0.330709554	,	0.254245261	,	0.087170535	,]],
[[	0.087292704	,	0.247807714	,	0.276581996	,	0.255502	,],
[	0.331404638	,	0.078466627	,	0.243754743	,	0.355553003	,],
[	0.402180678	,	0.290361351	,	0.070200738	,	0.325728376	,],
[	0.316070677	,	0.333288639	,	0.256854807	,	0.092357439	,]],
[[	0.092001189	,	0.254884955	,	0.283589387	,	0.262312801	,],
[	0.328796893	,	0.077651167	,	0.241372864	,	0.353137683	,],
[	0.402200728	,	0.290543551	,	0.072015713	,	0.325577047	,],
[	0.314156479	,	0.331517683	,	0.255193529	,	0.087299834	,]],
[[	0.086682577	,	0.248681716	,	0.277466546	,	0.25624034	,],
[	0.332179292	,	0.084129925	,	0.245104792	,	0.356355113	,],
[	0.404993839	,	0.293309195	,	0.072606793	,	0.328793657	,],
[	0.313954812	,	0.331258484	,	0.254734374	,	0.088634511	,]],
[[	0.089105644	,	0.24864922	,	0.277490962	,	0.256511827	,],
[	0.329696378	,	0.077217182	,	0.242137028	,	0.353884688	,],
[	0.401702263	,	0.289806394	,	0.070238887	,	0.325107485	,],
[	0.317511101	,	0.335233182	,	0.25886162	,	0.090296585	,]],
[[	0.089369223	,	0.251823581	,	0.280479433	,	0.259273974	,],
[	0.329743122	,	0.079627811	,	0.242377371	,	0.353968796	,],
[	0.403781647	,	0.292479487	,	0.075060702	,	0.327251002	,],
[	0.313078273	,	0.330612222	,	0.254139792	,	0.086754358	,]],
[[	0.086846631	,	0.247870422	,	0.276618996	,	0.255537916	,],
[	0.33267298	,	0.079542694	,	0.245018165	,	0.356709205	,],
[	0.402873978	,	0.291136533	,	0.070634832	,	0.32645948	,],
[	0.315174179	,	0.33243924	,	0.255880955	,	0.090932708	,]],
[[	0.0921791	,	0.250213639	,	0.278937381	,	0.258168058	,],
[	0.329121056	,	0.077375687	,	0.241524363	,	0.353256362	,],
[	0.40204909	,	0.290161254	,	0.071438375	,	0.325271604	,],
[	0.314917703	,	0.332439381	,	0.255903756	,	0.08779631	,]],
[[	0.087172826	,	0.249376253	,	0.2781214	,	0.256869879	,],
[	0.331419892	,	0.082649885	,	0.244268041	,	0.355429988	,],
[	0.406019141	,	0.294890109	,	0.078824714	,	0.32949385	,],
[	0.313462873	,	0.330817928	,	0.254272664	,	0.087960726	,]],
[[	0.088779309	,	0.252410226	,	0.27987024	,	0.255170044	,],
[	0.324322494	,	0.077231444	,	0.241231179	,	0.348819416	,],
[	0.396270388	,	0.290410172	,	0.070197409	,	0.320312419	,],
[	0.314500601	,	0.336853799	,	0.259958163	,	0.088606786	,]],
[[	0.087679239	,	0.254536058	,	0.281811424	,	0.257029325	,],
[	0.323912566	,	0.079102205	,	0.240964952	,	0.348346046	,],
[	0.399068414	,	0.293620949	,	0.075791026	,	0.323102368	,],
[	0.310824436	,	0.33319886	,	0.256138023	,	0.085946242	,]],
[[	0.086190205	,	0.251332183	,	0.278709059	,	0.253928914	,],
[	0.327452291	,	0.079828997	,	0.244327749	,	0.351837125	,],
[	0.397137129	,	0.291229225	,	0.070027581	,	0.321247058	,],
[	0.313487947	,	0.335529971	,	0.25852481	,	0.090809107	,]],
[[	0.091678106	,	0.259072556	,	0.286375828	,	0.261335481	,],
[	0.323392795	,	0.077045748	,	0.240342631	,	0.347828995	,],
[	0.396821712	,	0.291150048	,	0.071679143	,	0.320868323	,],
[	0.312329964	,	0.334538617	,	0.257551418	,	0.086393318	,]],
[[	0.086060937	,	0.252521536	,	0.279866658	,	0.254951155	,],
[	0.32567799	,	0.08212427	,	0.242946209	,	0.349997391	,],
[	0.399809597	,	0.294009918	,	0.072396546	,	0.324134561	,],
[	0.311603277	,	0.333715572	,	0.25656875	,	0.087304598	,]],
[[	0.087866999	,	0.252077754	,	0.2795589	,	0.254681932	,],
[	0.324933712	,	0.07761521	,	0.241776024	,	0.34938637	,],
[	0.396198921	,	0.29020703	,	0.069648511	,	0.320181817	,],
[	0.315581863	,	0.338015506	,	0.261082034	,	0.089378268	,]],
[[	0.088894845	,	0.255868081	,	0.283050908	,	0.258105792	,],
[	0.323776851	,	0.078373999	,	0.240780531	,	0.348154665	,],
[	0.398504177	,	0.292875894	,	0.074696315	,	0.322444912	,],
[	0.311222917	,	0.333549365	,	0.2565356	,	0.085899691	,]],
[[	0.085949398	,	0.251550675	,	0.27881429	,	0.253995902	,],
[	0.32754564	,	0.08563157	,	0.245014586	,	0.351671679	,],
[	0.397755744	,	0.291782406	,	0.070432857	,	0.321897323	,],
[	0.312858157	,	0.334831081	,	0.257826155	,	0.089556086	,]],
[[	0.090863564	,	0.253526877	,	0.280859919	,	0.25626816	,],
[	0.323751203	,	0.077103978	,	0.24054405	,	0.347971473	,],
[	0.396692002	,	0.290799468	,	0.071119554	,	0.320623568	,],
[	0.313110912	,	0.335299456	,	0.258370455	,	0.087059482	,]],
[[	0.086551734	,	0.253351886	,	0.280596921	,	0.255637415	,],
[	0.324918624	,	0.08081247	,	0.241943591	,	0.349102828	,],
[	0.400613452	,	0.295319845	,	0.078248131	,	0.324558103	,],
[	0.311186976	,	0.333366055	,	0.256293564	,	0.086579176	,]],
[[	0.087963811	,	0.257029125	,	0.283672502	,	0.25470035	,],
[	0.320718263	,	0.078771787	,	0.242949792	,	0.346055528	,],
[	0.391729022	,	0.291211767	,	0.070014701	,	0.316970308	,],
[	0.312193648	,	0.340017505	,	0.262949624	,	0.093055708	,]],
[[	0.086775882	,	0.258985769	,	0.285507127	,	0.256303956	,],
[	0.318150278	,	0.07787557	,	0.240657619	,	0.343598322	,],
[	0.391389957	,	0.290994946	,	0.071722311	,	0.316555873	,],
[	0.308326484	,	0.336704444	,	0.259563533	,	0.085643021	,]],
[[	0.085331803	,	0.255886084	,	0.28242685	,	0.25327995	,],
[	0.321117891	,	0.083946576	,	0.244141221	,	0.346551639	,],
[	0.39462282	,	0.294111984	,	0.072560045	,	0.320020582	,],
[	0.309044467	,	0.3372243	,	0.260063984	,	0.088211244	,]],
[[	0.090842435	,	0.263521053	,	0.290086772	,	0.260832835	,],
[	0.318937223	,	0.077470104	,	0.241244507	,	0.344298682	,],
[	0.390718527	,	0.290110254	,	0.069524231	,	0.315816415	,],
[	0.311041224	,	0.339347341	,	0.262261297	,	0.087716045	,]],
[[	0.085220547	,	0.257240218	,	0.283736795	,	0.25450177	,],
[	0.31896233	,	0.079744792	,	0.241603233	,	0.344381838	,],
[	0.393112834	,	0.292916964	,	0.074641605	,	0.318225786	,],
[	0.308233596	,	0.336288323	,	0.259074925	,	0.085808925	,]],
[[	0.087051353	,	0.256707979	,	0.283346291	,	0.254137976	,],
[	0.322140772	,	0.08013772	,	0.244421729	,	0.34746973	,],
[	0.39208034	,	0.291498196	,	0.070232009	,	0.317396688	,],
[	0.311233534	,	0.339133223	,	0.261951704	,	0.091460159	,]],
[[	0.087753873	,	0.260151734	,	0.286542041	,	0.257337155	,],
[	0.318170405	,	0.077398454	,	0.240569046	,	0.343590769	,],
[	0.390976073	,	0.290642146	,	0.070766298	,	0.31610967	,],
[	0.309058665	,	0.337431777	,	0.260311012	,	0.085985929	,]],
[[	0.084904968	,	0.255977455	,	0.28247386	,	0.253368171	,],
[	0.320206971	,	0.082308883	,	0.242985384	,	0.345497569	,],
[	0.395065336	,	0.29528742	,	0.078114463	,	0.320253879	,],
[	0.308643735	,	0.336858297	,	0.25965111	,	0.087367321	,]],
[[	0.090067114	,	0.258157406	,	0.284613827	,	0.255713701	,],
[	0.319772829	,	0.077957085	,	0.242060441	,	0.345108648	,],
[	0.390964501	,	0.290341882	,	0.069546071	,	0.316124828	,],
[	0.312004669	,	0.340354899	,	0.263283855	,	0.088560636	,]],
[[	0.085535116	,	0.257726523	,	0.284234339	,	0.254946645	,],
[	0.318582574	,	0.078829892	,	0.241113748	,	0.343928592	,],
[	0.39194757	,	0.291751004	,	0.073009482	,	0.316971	,],
[	0.308307553	,	0.336613218	,	0.259321391	,	0.085808975	,]],
[[	0.087282638	,	0.261377787	,	0.287114561	,	0.252466131	,],
[	0.31651794	,	0.086155788	,	0.243863837	,	0.341191725	,],
[	0.386991159	,	0.292650177	,	0.070399529	,	0.311895803	,],
[	0.309423339	,	0.342930272	,	0.265427189	,	0.090889861	,]],
[[	0.086975416	,	0.263998907	,	0.289499163	,	0.25481811	,],
[	0.312152305	,	0.07709762	,	0.238841346	,	0.337025842	,],
[	0.385304396	,	0.291066228	,	0.070218397	,	0.309982277	,],
[	0.307152743	,	0.341114819	,	0.263551149	,	0.085246323	,]],
[[	0.084649431	,	0.260233246	,	0.285809572	,	0.251212961	,],
[	0.313341392	,	0.080881022	,	0.240396819	,	0.338201094	,],
[	0.389201125	,	0.295510735	,	0.077409992	,	0.31389264	,],
[	0.306908363	,	0.340632454	,	0.263080145	,	0.08694643	,]],
[[	0.090354349	,	0.262996197	,	0.288514168	,	0.254143372	,],
[	0.314410124	,	0.078465491	,	0.24098622	,	0.339239263	,],
[	0.385639127	,	0.291281117	,	0.069424143	,	0.310429101	,],
[	0.309824378	,	0.343627226	,	0.266214892	,	0.087617215	,]],
[[	0.085108898	,	0.261810663	,	0.287462329	,	0.25266717	,],
[	0.312188612	,	0.077940552	,	0.238946508	,	0.337051767	,],
[	0.386491919	,	0.292477837	,	0.072586905	,	0.311170504	,],
[	0.306011852	,	0.339833576	,	0.26215272	,	0.084646669	,]],
[[	0.086223539	,	0.260803957	,	0.286388261	,	0.251795766	,],
[	0.315738348	,	0.084734596	,	0.242922729	,	0.340337529	,],
[	0.387984999	,	0.293674282	,	0.071180034	,	0.31290068	,],
[	0.308709066	,	0.342228218	,	0.264636267	,	0.089540445	,]],
[[	0.088068461	,	0.265132513	,	0.290624828	,	0.255885895	,],
[	0.31262596	,	0.077104649	,	0.239258363	,	0.337459911	,],
[	0.38521194	,	0.290893691	,	0.069784421	,	0.309860211	,],
[	0.307787177	,	0.341563802	,	0.264161962	,	0.08570471	,]],
[[	0.08483629	,	0.26081101	,	0.286251904	,	0.251639785	,],
[	0.312990412	,	0.079945129	,	0.239820482	,	0.337762244	,],
[	0.388626239	,	0.294782981	,	0.076100604	,	0.313278581	,],
[	0.306386415	,	0.340104766	,	0.262538694	,	0.08578132	,]],
[[	0.089076263	,	0.262332469	,	0.287853312	,	0.253349822	,],
[	0.315252143	,	0.079157304	,	0.241802576	,	0.33989145	,],
[	0.386132251	,	0.291702892	,	0.069701516	,	0.310850054	,],
[	0.310973898	,	0.344456142	,	0.267017477	,	0.093113749	,]],
[[	0.085786945	,	0.262679948	,	0.288247969	,	0.253385092	,],
[	0.312038119	,	0.077411578	,	0.238697582	,	0.336782284	,],
[	0.385969243	,	0.291838479	,	0.071522006	,	0.310569811	,],
[	0.306655961	,	0.340416574	,	0.262737395	,	0.084922877	,]],
[[	0.086564688	,	0.265603237	,	0.290645182	,	0.252738966	,],
[	0.310370861	,	0.084697478	,	0.243402215	,	0.336760011	,],
[	0.384893161	,	0.295001078	,	0.07224367	,	0.310653314	,],
[	0.307094432	,	0.345704196	,	0.268129936	,	0.08928924	,]],
[[	0.087304841	,	0.268910852	,	0.293795832	,	0.255877475	,],
[	0.307516413	,	0.077420069	,	0.239834629	,	0.334059692	,],
[	0.381555756	,	0.291718464	,	0.069906135	,	0.307102753	,],
[	0.305819084	,	0.344724267	,	0.267290511	,	0.084978718	,]],
[[	0.084533783	,	0.264828696	,	0.289839917	,	0.251951522	,],
[	0.307926939	,	0.080088539	,	0.240525916	,	0.334415663	,],
[	0.383722424	,	0.294388663	,	0.074967746	,	0.309273451	,],
[	0.304787904	,	0.343715519	,	0.26611741	,	0.085572395	,]],
[[	0.089814725	,	0.26740066	,	0.292315134	,	0.254628251	,],
[	0.310107289	,	0.079078652	,	0.242116302	,	0.336341636	,],
[	0.382539868	,	0.292694277	,	0.070162006	,	0.308290696	,],
[	0.309490262	,	0.348015719	,	0.270478761	,	0.09288584	,]],
[[	0.085023121	,	0.266477791	,	0.291400793	,	0.253382084	,],
[	0.307120465	,	0.077648541	,	0.239462889	,	0.333486387	,],
[	0.381835274	,	0.292147425	,	0.071144673	,	0.307312192	,],
[	0.304795692	,	0.343602823	,	0.265952259	,	0.084453053	,]],
[[	0.085780282	,	0.265272409	,	0.290314623	,	0.252331215	,],
[	0.309615829	,	0.083318222	,	0.24243238	,	0.33592966	,],
[	0.385855549	,	0.296074983	,	0.073219612	,	0.311620961	,],
[	0.306281548	,	0.344816346	,	0.267192545	,	0.087973014	,]],
[[	0.088680572	,	0.270330151	,	0.295125546	,	0.25720133	,],
[	0.308234234	,	0.077600991	,	0.240372748	,	0.334597027	,],
[	0.381669598	,	0.291700344	,	0.069816211	,	0.307121228	,],
[	0.306871688	,	0.345770321	,	0.2682491	,	0.085797019	,]],
[[	0.084578531	,	0.265384481	,	0.290298422	,	0.252363483	,],
[	0.307665907	,	0.079278208	,	0.240110402	,	0.334011025	,],
[	0.382861672	,	0.293338141	,	0.073538721	,	0.308253325	,],
[	0.30437748	,	0.343225614	,	0.265545646	,	0.084715369	,]],
[[	0.088399327	,	0.266639705	,	0.291496917	,	0.253675293	,],
[	0.311503096	,	0.080263788	,	0.243607464	,	0.337625629	,],
[	0.383316467	,	0.293428174	,	0.070769876	,	0.308989539	,],
[	0.308470947	,	0.346937358	,	0.269509433	,	0.091315134	,]],
[[	0.085741867	,	0.267342234	,	0.292224956	,	0.25422628	,],
[	0.307190248	,	0.077479984	,	0.239321088	,	0.333533396	,],
[	0.38150428	,	0.29174055	,	0.070342085	,	0.306935524	,],
[	0.305291083	,	0.34409326	,	0.266509135	,	0.084595516	,]],
[[	0.086720165	,	0.271598174	,	0.29607815	,	0.252616439	,],
[	0.305780635	,	0.084728725	,	0.244115042	,	0.332874693	,],
[	0.381013009	,	0.297119924	,	0.073250174	,	0.307385304	,],
[	0.305531773	,	0.349876678	,	0.271750967	,	0.089214827	,]],
[[	0.086542844	,	0.274198316	,	0.298521784	,	0.255175164	,],
[	0.303141864	,	0.077825146	,	0.240757474	,	0.330306068	,],
[	0.377392489	,	0.293567662	,	0.070643493	,	0.303489014	,],
[	0.304414645	,	0.349065885	,	0.271000225	,	0.085228851	,]],
[[	0.084556738	,	0.270764749	,	0.295078165	,	0.251802247	,],
[	0.302941648	,	0.080055512	,	0.240954226	,	0.330128984	,],
[	0.379877757	,	0.296454257	,	0.075922052	,	0.305892519	,],
[	0.303476895	,	0.347940217	,	0.269863757	,	0.085659414	,]],
[[	0.090307615	,	0.273577922	,	0.297998392	,	0.254760728	,],
[	0.306280068	,	0.080389516	,	0.243782876	,	0.333371414	,],
[	0.378760975	,	0.294881517	,	0.07122789	,	0.305008194	,],
[	0.30820011	,	0.352697629	,	0.274726612	,	0.088558713	,]],
[[	0.084518895	,	0.271985888	,	0.296401627	,	0.252930406	,],
[	0.302629246	,	0.078048934	,	0.240266399	,	0.329744235	,],
[	0.377905318	,	0.294153385	,	0.072038996	,	0.303873395	,],
[	0.303371083	,	0.347843772	,	0.269623305	,	0.084323976	,]],
[[	0.085980969	,	0.271298805	,	0.295775866	,	0.252225957	,],
[	0.304734992	,	0.083138633	,	0.242898551	,	0.331784423	,],
[	0.38197109	,	0.298065978	,	0.07415768	,	0.308271404	,],
[	0.30514714	,	0.349499701	,	0.271199543	,	0.088176625	,]],
[[	0.087898418	,	0.275645643	,	0.299921746	,	0.256497302	,],
[	0.304154655	,	0.078548863	,	0.241706986	,	0.331292044	,],
[	0.377455742	,	0.293645558	,	0.070526818	,	0.303602029	,],
[	0.305365148	,	0.349989993	,	0.271875573	,	0.08583382	,]],
[[	0.084256554	,	0.270993057	,	0.295337889	,	0.251926519	,],
[	0.30292773	,	0.079390058	,	0.240784577	,	0.330006877	,],
[	0.379060242	,	0.295523288	,	0.074577001	,	0.305013901	,],
[	0.303169197	,	0.347651993	,	0.269422639	,	0.084920542	,]],
[[	0.088771541	,	0.272710043	,	0.297066299	,	0.25371165	,],
[	0.307192768	,	0.087027573	,	0.245584733	,	0.334071917	,],
[	0.379551023	,	0.295608665	,	0.071718445	,	0.305594221	,],
[	0.307219535	,	0.351439077	,	0.273228074	,	0.091470044	,]],
[[	0.085286527	,	0.272818735	,	0.297096366	,	0.253625686	,],
[	0.302790382	,	0.077915164	,	0.240416327	,	0.329888224	,],
[	0.377664953	,	0.293816617	,	0.071450712	,	0.303647919	,],
[	0.303839064	,	0.348377177	,	0.270207065	,	0.084752665	,]],
[[	0.086066597	,	0.27642804	,	0.300963148	,	0.252964046	,],
[	0.299309216	,	0.082479215	,	0.242946028	,	0.32778496	,],
[	0.377717818	,	0.29985093	,	0.079268306	,	0.304427186	,],
[	0.303298245	,	0.353970731	,	0.276250992	,	0.087556009	,]],
[[	0.086511192	,	0.279478048	,	0.303826159	,	0.255925751	,],
[	0.299965775	,	0.079192646	,	0.243049017	,	0.3284538	,],
[	0.373948025	,	0.295325925	,	0.070831169	,	0.300689291	,],
[	0.304307308	,	0.355269865	,	0.277688954	,	0.086262113	,]],
[[	0.084151079	,	0.275847216	,	0.300341462	,	0.25233308	,],
[	0.298102778	,	0.079343565	,	0.241482074	,	0.326589549	,],
[	0.374792517	,	0.296564583	,	0.074197536	,	0.301469583	,],
[	0.301959451	,	0.352864936	,	0.274948121	,	0.08494582	,]],
[[	0.089675245	,	0.278354671	,	0.302757573	,	0.254896583	,],
[	0.301364559	,	0.085923053	,	0.245291663	,	0.329749509	,],
[	0.376199839	,	0.297473217	,	0.0726259	,	0.303054897	,],
[	0.305220958	,	0.355888202	,	0.278239619	,	0.090730384	,]],
[[	0.084264654	,	0.276917969	,	0.301433683	,	0.253374603	,],
[	0.298487669	,	0.07816034	,	0.241413306	,	0.326864226	,],
[	0.373848465	,	0.295292548	,	0.071543422	,	0.300456809	,],
[	0.302681547	,	0.353497398	,	0.275834894	,	0.084759415	,]],
[[	0.085253646	,	0.276109337	,	0.300593265	,	0.252427042	,],
[	0.298916623	,	0.081482886	,	0.242382309	,	0.327258412	,],
[	0.376799329	,	0.298709059	,	0.077581713	,	0.303314573	,],
[	0.302828744	,	0.353562551	,	0.275721917	,	0.086621569	,]],
[[	0.087732934	,	0.280856364	,	0.305069725	,	0.257184632	,],
[	0.300978159	,	0.080077165	,	0.244005939	,	0.329398749	,],
[	0.374167424	,	0.295468755	,	0.070883207	,	0.300922839	,],
[	0.305461023	,	0.356395401	,	0.278906883	,	0.087119447	,]],
[[	0.08404917	,	0.276073868	,	0.300449672	,	0.252415343	,],
[	0.29801356	,	0.078791874	,	0.241244144	,	0.326458619	,],
[	0.37433068	,	0.295958011	,	0.07318667	,	0.300874992	,],
[	0.301791008	,	0.35266184	,	0.2749038	,	0.084364199	,]],
[[	0.088046987	,	0.277394338	,	0.301839483	,	0.253900689	,],
[	0.30094201	,	0.084903305	,	0.244719006	,	0.329276847	,],
[	0.377173543	,	0.298458054	,	0.07329681	,	0.303965929	,],
[	0.304532215	,	0.355127328	,	0.277511424	,	0.089372692	,]],
[[	0.085144898	,	0.278101167	,	0.302570487	,	0.25445823	,],
[	0.299034449	,	0.078510585	,	0.242010035	,	0.327419681	,],
[	0.373548611	,	0.295013931	,	0.070958097	,	0.300195355	,],
[	0.303315744	,	0.354173112	,	0.276425233	,	0.085059708	,]],
[[	0.084507198	,	0.280113265	,	0.304550497	,	0.250989397	,],
[	0.292086532	,	0.081203712	,	0.241885663	,	0.321462525	,],
[	0.369584581	,	0.297885261	,	0.076511075	,	0.297843128	,],
[	0.30006414	,	0.3563559	,	0.278615288	,	0.087345819	,]],
[[	0.085600784	,	0.283750504	,	0.307978561	,	0.254548125	,],
[	0.294302539	,	0.080172395	,	0.243672787	,	0.323549038	,],
[	0.368364972	,	0.295846929	,	0.071216145	,	0.296654254	,],
[	0.300400969	,	0.356956792	,	0.279427267	,	0.085089854	,]],
[[	0.082459108	,	0.279362911	,	0.303630503	,	0.250124228	,],
[	0.291147304	,	0.078491317	,	0.240629411	,	0.320402792	,],
[	0.367421683	,	0.295421899	,	0.072480617	,	0.295671956	,],
[	0.298323635	,	0.354964529	,	0.277187661	,	0.084229557	,]],
[[	0.087711755	,	0.281778597	,	0.306076516	,	0.252718713	,],
[	0.293772341	,	0.084383559	,	0.243817633	,	0.322959105	,],
[	0.371393972	,	0.298908943	,	0.073790322	,	0.299679048	,],
[	0.302028915	,	0.358432847	,	0.280731878	,	0.090518224	,]],
[[	0.083300978	,	0.281207178	,	0.305588707	,	0.251981497	,],
[	0.292554025	,	0.078773539	,	0.241829563	,	0.32169959	,],
[	0.367291825	,	0.294839233	,	0.07076406	,	0.295458192	,],
[	0.298982612	,	0.355410307	,	0.27776699	,	0.0839394	,]],
[[	0.083559118	,	0.279770383	,	0.304163022	,	0.250517814	,],
[	0.291867597	,	0.080480179	,	0.241519087	,	0.320988492	,],
[	0.368900717	,	0.297061142	,	0.07518876	,	0.297040425	,],
[	0.299347898	,	0.355672555	,	0.277958826	,	0.086178957	,]],
[[	0.086849464	,	0.28497196	,	0.309190991	,	0.255797464	,],
[	0.295722886	,	0.081281185	,	0.244901507	,	0.324842821	,],
[	0.368933775	,	0.296574713	,	0.071707161	,	0.297204886	,],
[	0.301588472	,	0.358060033	,	0.280519215	,	0.0859953	,]],
[[	0.082402523	,	0.279697473	,	0.303944278	,	0.250454485	,],
[	0.291581384	,	0.078581285	,	0.240975338	,	0.320875396	,],
[	0.367710796	,	0.295485961	,	0.072089881	,	0.295832857	,],
[	0.298401135	,	0.35496432	,	0.277255775	,	0.083891742	,]],
[[	0.086371547	,	0.281045664	,	0.305437492	,	0.25183525	,],
[	0.293063998	,	0.083046293	,	0.242966608	,	0.32217744	,],
[	0.371138541	,	0.299499285	,	0.078836032	,	0.299204208	,],
[	0.301175496	,	0.357565613	,	0.27988939	,	0.089186264	,]],
[[	0.084164838	,	0.282232479	,	0.306604752	,	0.253033594	,],
[	0.293384131	,	0.079360998	,	0.242580693	,	0.322496159	,],
[	0.367502156	,	0.295044102	,	0.070621277	,	0.295669311	,],
[	0.299508278	,	0.356057385	,	0.278298031	,	0.084188534	,]],
[[	0.084449567	,	0.285341301	,	0.308617617	,	0.251431874	,],
[	0.286887971	,	0.080158012	,	0.241440116	,	0.317392726	,],
[	0.364174931	,	0.298432421	,	0.074839147	,	0.293244501	,],
[	0.297555105	,	0.359739976	,	0.28158295	,	0.085742567	,]],
[[	0.08636674	,	0.289539629	,	0.312675262	,	0.255471395	,],
[	0.290619848	,	0.08749864	,	0.245812587	,	0.321010796	,],
[	0.36450109	,	0.298292819	,	0.07174157	,	0.29383172	,],
[	0.300431004	,	0.362784466	,	0.284724131	,	0.086275833	,]],
[[	0.082559447	,	0.28473231	,	0.307911214	,	0.25086547	,],
[	0.286771465	,	0.078658855	,	0.241235673	,	0.317372444	,],
[	0.362875665	,	0.296806049	,	0.071672577	,	0.291907361	,],
[	0.296811988	,	0.359113632	,	0.280907847	,	0.083646944	,]],
[[	0.087514651	,	0.286979003	,	0.310002226	,	0.252974614	,],
[	0.287979093	,	0.082723402	,	0.242821804	,	0.318452247	,],
[	0.3660998	,	0.300731018	,	0.078304874	,	0.295210568	,],
[	0.29951816	,	0.361784139	,	0.283532643	,	0.088747588	,]],
[[	0.083688445	,	0.286777764	,	0.310039398	,	0.252737115	,],
[	0.289005728	,	0.079736441	,	0.243104287	,	0.319464121	,],
[	0.363062545	,	0.296849446	,	0.070730114	,	0.292243713	,],
[	0.298192185	,	0.360521003	,	0.282198513	,	0.084180246	,]],
[[	0.083768007	,	0.284999613	,	0.308335541	,	0.251018936	,],
[	0.286663404	,	0.079581529	,	0.241079056	,	0.317139281	,],
[	0.363523521	,	0.29773036	,	0.073695934	,	0.292639469	,],
[	0.297351562	,	0.359654487	,	0.281306813	,	0.084986825	,]],
[[	0.087403348	,	0.290630205	,	0.313726977	,	0.256632193	,],
[	0.289768817	,	0.085999511	,	0.244751356	,	0.320125877	,],
[	0.365555084	,	0.299179798	,	0.072672399	,	0.294809522	,],
[	0.301394437	,	0.363754503	,	0.285720244	,	0.087190766	,]],
[[	0.082796993	,	0.28527888	,	0.308460994	,	0.251231392	,],
[	0.2870767	,	0.078523298	,	0.241332196	,	0.317597206	,],
[	0.362885501	,	0.296654706	,	0.07122933	,	0.29178418	,],
[	0.297134303	,	0.359602943	,	0.281225211	,	0.083688878	,]],
[[	0.086348497	,	0.286375528	,	0.309481281	,	0.252373227	,],
[	0.287369205	,	0.081497716	,	0.242058684	,	0.317820574	,],
[	0.365288192	,	0.299704065	,	0.076837245	,	0.294312767	,],
[	0.298658937	,	0.3609466	,	0.282675656	,	0.087428498	,]],
[[	0.084753576	,	0.287915824	,	0.311058037	,	0.253826408	,],
[	0.290024429	,	0.080742054	,	0.244099435	,	0.320374459	,],
[	0.363582778	,	0.297325302	,	0.070973741	,	0.292685189	,],
[	0.299088749	,	0.361284737	,	0.283091042	,	0.084852661	,]],
[[	0.086263205	,	0.296083911	,	0.318142238	,	0.254807204	,],
[	0.288030812	,	0.082526097	,	0.246560358	,	0.318509683	,],
[	0.369484709	,	0.307993129	,	0.078176941	,	0.29550197	,],
[	0.304864594	,	0.374084663	,	0.29262983	,	0.088437194	,]],
[[	0.088027859	,	0.30010754	,	0.321955784	,	0.258831579	,],
[	0.292219206	,	0.09008488	,	0.251250625	,	0.322537359	,],
[	0.368779689	,	0.30664237	,	0.073760593	,	0.294959295	,],
[	0.305349439	,	0.374644747	,	0.29345413	,	0.086423966	,]],
[[	0.084797144	,	0.295855759	,	0.317848191	,	0.254472258	,],
[	0.28791153	,	0.080920239	,	0.246214315	,	0.318402004	,],
[	0.36795171	,	0.306062408	,	0.074834572	,	0.293857068	,],
[	0.302976916	,	0.372376047	,	0.29094983	,	0.085336967	,]],
[[	0.089258337	,	0.29765703	,	0.319628565	,	0.256461025	,],
[	0.28919221	,	0.084917812	,	0.247958176	,	0.31952975	,],
[	0.371831975	,	0.310526047	,	0.082118467	,	0.297854185	,],
[	0.306891508	,	0.376152241	,	0.294691391	,	0.091752056	,]],
[[	0.085822205	,	0.297786477	,	0.319826851	,	0.256402398	,],
[	0.290470675	,	0.082622284	,	0.248536888	,	0.32080012	,],
[	0.367963756	,	0.305864609	,	0.073409601	,	0.293957813	,],
[	0.30389294	,	0.373069688	,	0.291732349	,	0.085243957	,]],
[[	0.085738461	,	0.296088603	,	0.318042258	,	0.254591715	,],
[	0.287813115	,	0.081859203	,	0.246207397	,	0.318158906	,],
[	0.368916435	,	0.307291027	,	0.077017539	,	0.294904339	,],
[	0.304147196	,	0.373376336	,	0.291935347	,	0.087245797	,]],
[[	0.089373105	,	0.301491751	,	0.323310041	,	0.26014351	,],
[	0.291252221	,	0.088436357	,	0.250133097	,	0.321416508	,],
[	0.369499906	,	0.307294895	,	0.074239352	,	0.29565691	,],
[	0.306571813	,	0.375883507	,	0.29463994	,	0.087432228	,]],
[[	0.08480078	,	0.296244794	,	0.318232485	,	0.254838734	,],
[	0.288475909	,	0.08111485	,	0.246638663	,	0.318884869	,],
[	0.367609843	,	0.30562568	,	0.073993351	,	0.293573118	,],
[	0.303115973	,	0.372629625	,	0.29113188	,	0.085071438	,]],
[[	0.087935226	,	0.296918316	,	0.318837944	,	0.255581283	,],
[	0.288749777	,	0.083877233	,	0.247332088	,	0.319134117	,],
[	0.370999014	,	0.309601177	,	0.080529945	,	0.29696464	,],
[	0.30594341	,	0.375120607	,	0.293671589	,	0.090226461	,]],
[[	0.086591957	,	0.298591558	,	0.320555367	,	0.257109392	,],
[	0.291166254	,	0.083096823	,	0.249224036	,	0.321421575	,],
[	0.368123983	,	0.306003943	,	0.073312183	,	0.29416373	,],
[	0.304562703	,	0.373835285	,	0.292450607	,	0.085743	,]],
[[	0.086977899	,	0.302818993	,	0.324190655	,	0.256253773	,],
[	0.283643366	,	0.082394382	,	0.24685355	,	0.314491469	,],
[	0.366456601	,	0.309879457	,	0.077755646	,	0.292202033	,],
[	0.304045183	,	0.378929351	,	0.29689755	,	0.088584777	,]],
[[	0.087295168	,	0.305498109	,	0.326813009	,	0.258919889	,],
[	0.287858733	,	0.089998986	,	0.251538042	,	0.318482656	,],
[	0.366687851	,	0.309566488	,	0.074806619	,	0.292731781	,],
[	0.304670384	,	0.379782293	,	0.297824714	,	0.086398993	,]],
[[	0.08474554	,	0.301797985	,	0.323136452	,	0.25531717	,],
[	0.283869055	,	0.081232272	,	0.246907514	,	0.314852119	,],
[	0.364700591	,	0.307872733	,	0.074360059	,	0.290526523	,],
[	0.302301084	,	0.377407936	,	0.295398371	,	0.085405381	,]],
[[	0.090214424	,	0.304281787	,	0.325723798	,	0.257829709	,],
[	0.284862192	,	0.084976776	,	0.248386744	,	0.315776864	,],
[	0.368206975	,	0.311925636	,	0.081021089	,	0.294044379	,],
[	0.306146156	,	0.381112809	,	0.299146391	,	0.09184011	,]],
[[	0.085125999	,	0.303262816	,	0.324554358	,	0.256681429	,],
[	0.286281378	,	0.082708251	,	0.249056807	,	0.317008428	,],
[	0.365289176	,	0.308299972	,	0.073786417	,	0.29115333	,],
[	0.303023718	,	0.377889996	,	0.295870319	,	0.084971493	,]],
[[	0.086012858	,	0.302173456	,	0.323607356	,	0.255621948	,],
[	0.283518763	,	0.081862092	,	0.2466984	,	0.314458536	,],
[	0.36569634	,	0.309170121	,	0.076589547	,	0.29146417	,],
[	0.303441318	,	0.378317357	,	0.296247203	,	0.087383917	,]],
[[	0.088345576	,	0.306657863	,	0.327781303	,	0.260086533	,],
[	0.287037726	,	0.088569393	,	0.250638239	,	0.317697608	,],
[	0.367592944	,	0.310424571	,	0.075585315	,	0.293450776	,],
[	0.30538247	,	0.380441905	,	0.29852427	,	0.08707313	,]],
[[	0.084701681	,	0.30209773	,	0.323289636	,	0.255474709	,],
[	0.284271293	,	0.081266123	,	0.247231909	,	0.315095149	,],
[	0.36472162	,	0.307801169	,	0.073998694	,	0.290521824	,],
[	0.302185268	,	0.377320579	,	0.295233898	,	0.084823047	,]],
[[	0.088871207	,	0.303702031	,	0.324963138	,	0.257198637	,],
[	0.28433758	,	0.083853272	,	0.247701385	,	0.31518321	,],
[	0.367639636	,	0.311207725	,	0.079788628	,	0.293335838	,],
[	0.305480979	,	0.380419535	,	0.298353433	,	0.090599759	,]],
[[	0.085891083	,	0.304053487	,	0.325499335	,	0.257511771	,],
[	0.287566178	,	0.083777906	,	0.250087277	,	0.318175726	,],
[	0.365871574	,	0.3086909	,	0.073946559	,	0.2916804	,],
[	0.303523186	,	0.378451428	,	0.296445477	,	0.085297002	,]],
[[	0.085597153	,	0.308120033	,	0.328493064	,	0.256036987	,],
[	0.280018306	,	0.082690836	,	0.248039046	,	0.311510997	,],
[	0.36114662	,	0.309588203	,	0.07386894	,	0.286824523	,],
[	0.302407736	,	0.382352028	,	0.299517598	,	0.085097318	,]],
[[	0.087905782	,	0.312296827	,	0.332440112	,	0.260246398	,],
[	0.280783385	,	0.086096125	,	0.249203849	,	0.312215754	,],
[	0.363657946	,	0.312664861	,	0.079128872	,	0.289382139	,],
[	0.305555327	,	0.385407106	,	0.302474052	,	0.090551167	,]],
[[	0.084356129	,	0.307751188	,	0.328079436	,	0.255679889	,],
[	0.282460059	,	0.084497748	,	0.250372039	,	0.314054588	,],
[	0.362570499	,	0.310910718	,	0.074382001	,	0.288487487	,],
[	0.303362811	,	0.383413179	,	0.300594945	,	0.085227491	,]],
[[	0.088268226	,	0.309444391	,	0.329704347	,	0.257287628	,],
[	0.279678672	,	0.083447858	,	0.247860901	,	0.311261662	,],
[	0.361445773	,	0.31014292	,	0.075139547	,	0.28725769	,],
[	0.302690493	,	0.3828357	,	0.299936828	,	0.086315416	,]],
[[	0.085085772	,	0.309685132	,	0.330005285	,	0.257525981	,],
[	0.282260963	,	0.08911915	,	0.25096765	,	0.313638294	,],
[	0.365760496	,	0.314080878	,	0.077190694	,	0.291815786	,],
[	0.30653617	,	0.386236287	,	0.303557948	,	0.088135046	,]],
[[	0.084945445	,	0.307748324	,	0.328119637	,	0.255613738	,],
[	0.280227521	,	0.082615727	,	0.248048594	,	0.31162941	,],
[	0.361528687	,	0.309957986	,	0.073881491	,	0.287262945	,],
[	0.302376724	,	0.382322177	,	0.299351081	,	0.084642358	,]],
[[	0.089239883	,	0.31365801	,	0.333790469	,	0.261564187	,],
[	0.280195405	,	0.084978593	,	0.248447769	,	0.311583874	,],
[	0.362971625	,	0.311910337	,	0.077881256	,	0.288653276	,],
[	0.304777702	,	0.384556825	,	0.301626091	,	0.089206971	,]],
[[	0.084207425	,	0.308127309	,	0.328321681	,	0.255977653	,],
[	0.28354569	,	0.08540805	,	0.251459382	,	0.314933546	,],
[	0.363005787	,	0.311296064	,	0.074614862	,	0.288845103	,],
[	0.303830185	,	0.383875983	,	0.301120756	,	0.085572663	,]],
[[	0.087191694	,	0.308866138	,	0.329031509	,	0.256657317	,],
[	0.279712821	,	0.082943027	,	0.247772402	,	0.311208935	,],
[	0.361633538	,	0.310201805	,	0.074708638	,	0.287244189	,],
[	0.302366051	,	0.382419264	,	0.29950159	,	0.085390183	,]],
[[	0.086353773	,	0.310825275	,	0.331111212	,	0.258758214	,],
[	0.281985275	,	0.08809744	,	0.250437128	,	0.313361804	,],
[	0.364937426	,	0.314118187	,	0.081191092	,	0.290547454	,],
[	0.306995703	,	0.386872682	,	0.303952077	,	0.092513393	,]],
[[	0.085747248	,	0.313492753	,	0.333185482	,	0.256176094	,],
[	0.275422743	,	0.082701502	,	0.248550673	,	0.307847269	,],
[	0.357181868	,	0.311434173	,	0.074151234	,	0.283479616	,],
[	0.30047125	,	0.386287984	,	0.303119048	,	0.084264083	,]],
[[	0.087938864	,	0.317496739	,	0.337077061	,	0.260369964	,],
[	0.275388479	,	0.085229149	,	0.248981004	,	0.307769273	,],
[	0.359098712	,	0.31382312	,	0.078795743	,	0.285350036	,],
[	0.304230856	,	0.389834546	,	0.306758536	,	0.090441807	,]],
[[	0.084292279	,	0.312973553	,	0.332653023	,	0.255765317	,],
[	0.278231798	,	0.084711841	,	0.25124195	,	0.310612298	,],
[	0.358665562	,	0.312722786	,	0.074858805	,	0.285034032	,],
[	0.301317713	,	0.387026879	,	0.304049534	,	0.08427042	,]],
[[	0.088398642	,	0.314667654	,	0.334320784	,	0.257438167	,],
[	0.274676181	,	0.082771356	,	0.247931401	,	0.30704635	,],
[	0.35721864	,	0.311636381	,	0.075060799	,	0.283401469	,],
[	0.301325336	,	0.387150038	,	0.303986144	,	0.086059985	,]],
[[	0.085170926	,	0.314918882	,	0.334651135	,	0.257633048	,],
[	0.277058527	,	0.08831508	,	0.250871351	,	0.309376509	,],
[	0.361257513	,	0.316340918	,	0.082438589	,	0.287470075	,],
[	0.304285151	,	0.389815444	,	0.306902624	,	0.086889042	,]],
[[	0.084814586	,	0.312841663	,	0.332630025	,	0.255513156	,],
[	0.276102211	,	0.083098252	,	0.249158694	,	0.308357602	,],
[	0.357308201	,	0.311466353	,	0.073940242	,	0.283674379	,],
[	0.30052986	,	0.386277088	,	0.303076915	,	0.083943708	,]],
[[	0.089362571	,	0.318888727	,	0.338453439	,	0.261692272	,],
[	0.275188981	,	0.084399209	,	0.248594143	,	0.307492611	,],
[	0.358290567	,	0.312952029	,	0.077485723	,	0.284501194	,],
[	0.303378082	,	0.388949111	,	0.305869568	,	0.089084897	,]],
[[	0.084164205	,	0.313329492	,	0.332920801	,	0.255961054	,],
[	0.279648442	,	0.092465117	,	0.253625108	,	0.311698293	,],
[	0.35938313	,	0.313351363	,	0.075270988	,	0.285806532	,],
[	0.301787133	,	0.387522617	,	0.304538452	,	0.084527738	,]],
[[	0.087286422	,	0.314124695	,	0.333746384	,	0.256774049	,],
[	0.274981145	,	0.082686397	,	0.24817241	,	0.307305473	,],
[	0.357002547	,	0.311310597	,	0.074551384	,	0.283267711	,],
[	0.301144895	,	0.3869371	,	0.303766876	,	0.085293958	,]],
[[	0.086318256	,	0.315913258	,	0.335632176	,	0.258631614	,],
[	0.27642453	,	0.08693586	,	0.250042186	,	0.308592168	,],
[	0.360362655	,	0.315306853	,	0.080896148	,	0.286530182	,],
[	0.305223342	,	0.390708165	,	0.307910399	,	0.087713932	,]],
[[	0.086308912	,	0.318989168	,	0.338372556	,	0.256434282	,],
[	0.26983885	,	0.082425062	,	0.248564358	,	0.30321607	,],
[	0.353656435	,	0.314006378	,	0.075900446	,	0.280866502	,],
[	0.300632156	,	0.39238305	,	0.308719081	,	0.086580861	,]],
[[	0.086900443	,	0.321811513	,	0.341128326	,	0.259399904	,],
[	0.270996173	,	0.08659684	,	0.250214865	,	0.304331629	,],
[	0.357667309	,	0.31877712	,	0.083393251	,	0.284994748	,],
[	0.304080754	,	0.395683965	,	0.312284494	,	0.087928786	,]],
[[	0.084376406	,	0.318157437	,	0.337537882	,	0.255668263	,],
[	0.271841209	,	0.083455856	,	0.250430376	,	0.305260739	,],
[	0.353522765	,	0.313552572	,	0.074164123	,	0.280723325	,],
[	0.300153951	,	0.392011299	,	0.308407136	,	0.084930848	,]],
[[	0.089441937	,	0.320687178	,	0.339925315	,	0.258127204	,],
[	0.269563139	,	0.083261786	,	0.248550538	,	0.303004434	,],
[	0.354952805	,	0.315419002	,	0.078373327	,	0.282024272	,],
[	0.302256328	,	0.394110935	,	0.310365223	,	0.089403537	,]],
[[	0.084703102	,	0.319686897	,	0.339086447	,	0.257154053	,],
[	0.272790056	,	0.090078619	,	0.252467572	,	0.306099426	,],
[	0.355498367	,	0.315471565	,	0.075391639	,	0.282864903	,],
[	0.301673183	,	0.393278933	,	0.309739697	,	0.085601622	,]],
[[	0.085249151	,	0.318300874	,	0.337816401	,	0.25580749	,],
[	0.270139521	,	0.082508879	,	0.248778189	,	0.303471961	,],
[	0.353192813	,	0.31350228	,	0.074764902	,	0.280413951	,],
[	0.300309638	,	0.392048709	,	0.30819372	,	0.085683291	,]],
[[	0.088559961	,	0.323526154	,	0.342819503	,	0.261082722	,],
[	0.270521878	,	0.085533633	,	0.249666334	,	0.303824673	,],
[	0.356758089	,	0.317709983	,	0.081786281	,	0.283966006	,],
[	0.304758218	,	0.396214743	,	0.312583413	,	0.092835629	,]],
[[	0.084315244	,	0.318553785	,	0.337930399	,	0.256099552	,],
[	0.272892603	,	0.084300507	,	0.251402553	,	0.306212357	,],
[	0.353968445	,	0.313896264	,	0.074190896	,	0.281096265	,],
[	0.300360974	,	0.392178245	,	0.308528594	,	0.084798103	,]],
[[	0.088174107	,	0.320019193	,	0.339372091	,	0.257519164	,],
[	0.269788268	,	0.083104576	,	0.248658563	,	0.303217506	,],
[	0.354359579	,	0.314808607	,	0.077340878	,	0.281437536	,],
[	0.30156913	,	0.393256818	,	0.309561342	,	0.087985022	,]],
[[	0.08559723	,	0.320617059	,	0.340008052	,	0.258025855	,],
[	0.272179521	,	0.088760579	,	0.251634741	,	0.305401275	,],
[	0.356185773	,	0.316034162	,	0.075881644	,	0.283537013	,],
[	0.302681187	,	0.394207085	,	0.310690884	,	0.086464878	,]],
[[	0.086522044	,	0.324993802	,	0.343870849	,	0.257150759	,],
[	0.265787698	,	0.083367617	,	0.250068602	,	0.299909744	,],
[	0.350595452	,	0.31644547	,	0.076265655	,	0.277863204	,],
[	0.300060269	,	0.397529445	,	0.313078357	,	0.086765171	,]],
[[	0.087199566	,	0.327953334	,	0.346715008	,	0.260208245	,],
[	0.266822301	,	0.087310732	,	0.251663624	,	0.300830349	,],
[	0.354451115	,	0.321118542	,	0.083710708	,	0.281893295	,],
[	0.302653913	,	0.399869827	,	0.315933702	,	0.08706428	,]],
[[	0.084708341	,	0.324279092	,	0.343110645	,	0.256385321	,],
[	0.268384974	,	0.085074719	,	0.252461168	,	0.30235943	,],
[	0.350719079	,	0.316217083	,	0.074913764	,	0.278020445	,],
[	0.299210735	,	0.396661065	,	0.312459835	,	0.084682128	,]],
[[	0.089711737	,	0.326580721	,	0.345428408	,	0.258730732	,],
[	0.265695375	,	0.084165046	,	0.250095783	,	0.299590434	,],
[	0.351727637	,	0.317877228	,	0.078765569	,	0.278957933	,],
[	0.302206635	,	0.399681367	,	0.315397234	,	0.089859626	,]],
[[	0.085145998	,	0.325874515	,	0.344819122	,	0.257885464	,],
[	0.268668919	,	0.090585818	,	0.253699053	,	0.302516356	,],
[	0.352776086	,	0.318131689	,	0.076097722	,	0.280055115	,],
[	0.300609868	,	0.39777858	,	0.313574721	,	0.085191178	,]],
[[	0.085649626	,	0.324474596	,	0.343376223	,	0.256516893	,],
[	0.266201811	,	0.08351532	,	0.250415565	,	0.300285233	,],
[	0.35024971	,	0.315982152	,	0.075368957	,	0.277401729	,],
[	0.299713421	,	0.397125203	,	0.312817767	,	0.085937121	,]],
[[	0.088298512	,	0.329145363	,	0.347788779	,	0.261295434	,],
[	0.26650446	,	0.086327848	,	0.251161085	,	0.300298718	,],
[	0.353834366	,	0.320275699	,	0.082277237	,	0.281044674	,],
[	0.303962005	,	0.401200206	,	0.317210727	,	0.088324761	,]],
[[	0.084489083	,	0.32455941	,	0.343362248	,	0.256628735	,],
[	0.269433914	,	0.085848264	,	0.253374806	,	0.303210401	,],
[	0.351004661	,	0.316533123	,	0.07498693	,	0.278324014	,],
[	0.299455481	,	0.396876503	,	0.312615662	,	0.084549845	,]],
[[	0.088484513	,	0.326081859	,	0.34481684	,	0.25809758	,],
[	0.265701261	,	0.083829849	,	0.250069552	,	0.299679689	,],
[	0.351497009	,	0.317543454	,	0.077792031	,	0.278650014	,],
[	0.30112419	,	0.398650391	,	0.314290142	,	0.088435736	,]],
[[	0.085797667	,	0.326606736	,	0.345479324	,	0.258669601	,],
[	0.268157639	,	0.089443055	,	0.253104357	,	0.301921131	,],
[	0.353391665	,	0.318753967	,	0.076645433	,	0.280764886	,],
[	0.301341885	,	0.398561581	,	0.314455188	,	0.085796049	,]],
[[	0.085863016	,	0.329529317	,	0.346691537	,	0.25696556	,],
[	0.26192316	,	0.083618405	,	0.250220907	,	0.2966652	,],
[	0.346872	,	0.317893713	,	0.075334339	,	0.274420331	,],
[	0.298640315	,	0.400936795	,	0.315464017	,	0.086188459	,]],
[[	0.087733862	,	0.333268992	,	0.350292502	,	0.261004747	,],
[	0.261159382	,	0.085357446	,	0.250006277	,	0.296018453	,],
[	0.349948644	,	0.321562051	,	0.081599579	,	0.277518146	,],
[	0.302411753	,	0.40427091	,	0.319319946	,	0.087833759	,]],
[[	0.08415657	,	0.328809635	,	0.345999683	,	0.256416508	,],
[	0.26537426	,	0.086364924	,	0.253541369	,	0.300019444	,],
[	0.34746295	,	0.318097477	,	0.074666744	,	0.27507027	,],
[	0.298011548	,	0.40022311	,	0.314999961	,	0.084423513	,]],
[[	0.0888247	,	0.330905473	,	0.347977172	,	0.258429576	,],
[	0.260998988	,	0.083519808	,	0.249492067	,	0.295769644	,],
[	0.347489442	,	0.318647006	,	0.077053571	,	0.275006608	,],
[	0.299963353	,	0.402211358	,	0.316864836	,	0.088528615	,]],
[[	0.085164681	,	0.330819018	,	0.348002658	,	0.258322973	,],
[	0.26257121	,	0.08812702	,	0.251620129	,	0.297251726	,],
[	0.35026917	,	0.320754308	,	0.076830429	,	0.27796729	,],
[	0.299939249	,	0.401832479	,	0.316646995	,	0.085545587	,]],
[[	0.085165697	,	0.329187361	,	0.346362453	,	0.256564723	,],
[	0.262832635	,	0.084220355	,	0.251010294	,	0.297599471	,],
[	0.346764599	,	0.317671512	,	0.074733363	,	0.274315837	,],
[	0.298307471	,	0.40056877	,	0.315093973	,	0.085469426	,]],
[[	0.088991651	,	0.334636091	,	0.351604377	,	0.262255562	,],
[	0.260935443	,	0.084601947	,	0.249584829	,	0.295682362	,],
[	0.348783389	,	0.320343126	,	0.079806392	,	0.276312542	,],
[	0.302849828	,	0.404829761	,	0.319516038	,	0.092570664	,]],
[[	0.084425237	,	0.329527884	,	0.346471324	,	0.256909768	,],
[	0.26493535	,	0.092035868	,	0.2541262	,	0.299424771	,],
[	0.348373861	,	0.318974641	,	0.075272947	,	0.275903433	,],
[	0.298275694	,	0.400640697	,	0.315199454	,	0.08442234	,]],
[[	0.087679198	,	0.330395755	,	0.347560823	,	0.257862972	,],
[	0.261568025	,	0.083637931	,	0.249886047	,	0.296236215	,],
[	0.347015531	,	0.318109046	,	0.076131728	,	0.274363358	,],
[	0.299504989	,	0.401752842	,	0.31633987	,	0.087609935	,]],
[[	0.086204411	,	0.331765671	,	0.349003606	,	0.259399989	,],
[	0.261894172	,	0.086882204	,	0.250786616	,	0.296564841	,],
[	0.35110999	,	0.322964973	,	0.083747607	,	0.27877661	,],
[	0.300726675	,	0.402731735	,	0.317568394	,	0.086246738	,]],
[[	0.084843658	,	0.334242145	,	0.352645351	,	0.257230501	,],
[	0.258354741	,	0.08516507	,	0.254876234	,	0.295165947	,],
[	0.342172849	,	0.319239958	,	0.075887844	,	0.270990196	,],
[	0.296893618	,	0.405305942	,	0.321022177	,	0.085320687	,]],
[[	0.086952253	,	0.338202612	,	0.35649201	,	0.261448281	,],
[	0.255886084	,	0.084789262	,	0.252811759	,	0.292767661	,],
[	0.34380864	,	0.321482882	,	0.080417363	,	0.272542261	,],
[	0.301093652	,	0.409354292	,	0.325125488	,	0.092235732	,]],
[[	0.083453854	,	0.333864157	,	0.352136908	,	0.256847972	,],
[	0.259151681	,	0.091551932	,	0.256672323	,	0.295857283	,],
[	0.343485869	,	0.320251839	,	0.076282387	,	0.272289342	,],
[	0.297090937	,	0.405549009	,	0.321256685	,	0.084490757	,]],
[[	0.087812806	,	0.335764383	,	0.354006877	,	0.258700077	,],
[	0.256492527	,	0.084064738	,	0.253247257	,	0.293407558	,],
[	0.342307487	,	0.319467608	,	0.076942558	,	0.270820632	,],
[	0.298090812	,	0.406600635	,	0.322295818	,	0.087561728	,]],
[[	0.084458573	,	0.335752018	,	0.354146487	,	0.258750616	,],
[	0.25657399	,	0.086845303	,	0.253851735	,	0.293483806	,],
[	0.345922511	,	0.323838099	,	0.083996608	,	0.27444597	,],
[	0.299374567	,	0.407507722	,	0.323336597	,	0.086105139	,]],
[[	0.084029408	,	0.333719133	,	0.35221392	,	0.256651336	,],
[	0.259213885	,	0.085799063	,	0.255612102	,	0.295901139	,],
[	0.342552335	,	0.319510201	,	0.075861177	,	0.271235139	,],
[	0.296807394	,	0.405110745	,	0.320713304	,	0.084818838	,]],
[[	0.088324774	,	0.339677508	,	0.35780979	,	0.262803789	,],
[	0.255999163	,	0.084490029	,	0.252824973	,	0.292756304	,],
[	0.343260997	,	0.320799322	,	0.07923819	,	0.27187705	,],
[	0.300231264	,	0.408552745	,	0.324155894	,	0.090779725	,]],
[[	0.083641732	,	0.334208173	,	0.352536566	,	0.257243603	,],
[	0.258587848	,	0.090368394	,	0.256002139	,	0.295233786	,],
[	0.344878588	,	0.321607866	,	0.077479228	,	0.27357744	,],
[	0.297430712	,	0.405813163	,	0.321596471	,	0.084671155	,]],
[[	0.086591409	,	0.335034245	,	0.353338956	,	0.257941715	,],
[	0.257285905	,	0.084352393	,	0.253837984	,	0.294112846	,],
[	0.341955192	,	0.31905113	,	0.076187788	,	0.270534521	,],
[	0.297512693	,	0.405988985	,	0.321579827	,	0.086368198	,]],
[[	0.085399649	,	0.336657764	,	0.355042988	,	0.259722583	,],
[	0.256306584	,	0.085882287	,	0.253317485	,	0.293003928	,],
[	0.345157133	,	0.322991233	,	0.082469148	,	0.273747767	,],
[	0.300474684	,	0.408519932	,	0.324446702	,	0.087197064	,]],
[[	0.085559949	,	0.34111938	,	0.358076433	,	0.258714017	,],
[	0.255755797	,	0.086974326	,	0.256023048	,	0.292911864	,],
[	0.340766578	,	0.323277127	,	0.076151351	,	0.269116988	,],
[	0.298158267	,	0.412342391	,	0.326111014	,	0.085573817	,]],
[[	0.087552944	,	0.345054613	,	0.361909871	,	0.262843506	,],
[	0.252478889	,	0.085765098	,	0.253234573	,	0.289663668	,],
[	0.341917968	,	0.32504969	,	0.080113975	,	0.270239682	,],
[	0.302151914	,	0.416200901	,	0.329986242	,	0.092112319	,]],
[[	0.083915715	,	0.340520033	,	0.357366588	,	0.258203677	,],
[	0.255440371	,	0.092018432	,	0.256790827	,	0.292320416	,],
[	0.342802116	,	0.325134123	,	0.077240245	,	0.271267854	,],
[	0.298644591	,	0.412744053	,	0.326533898	,	0.084990272	,]],
[[	0.088414171	,	0.342567673	,	0.359346222	,	0.260077613	,],
[	0.253736416	,	0.085608274	,	0.254245029	,	0.290862731	,],
[	0.340581857	,	0.323295702	,	0.076899514	,	0.268856485	,],
[	0.299238561	,	0.413615232	,	0.327292239	,	0.087618977	,]],
[[	0.084858693	,	0.342448065	,	0.359417844	,	0.260188281	,],
[	0.252978403	,	0.087535702	,	0.254102921	,	0.290274239	,],
[	0.343911999	,	0.327336686	,	0.083579064	,	0.272172112	,],
[	0.301142626	,	0.414933215	,	0.32893023	,	0.086912783	,]],
[[	0.084777108	,	0.340729367	,	0.357680015	,	0.258314891	,],
[	0.256654538	,	0.087635804	,	0.256872898	,	0.293591666	,],
[	0.34130925	,	0.323719969	,	0.076353538	,	0.269649898	,],
[	0.298267136	,	0.412417101	,	0.3261168	,	0.08516673	,]],
[[	0.088982117	,	0.346499968	,	0.363220818	,	0.264334796	,],
[	0.252717911	,	0.085429115	,	0.253337515	,	0.289734861	,],
[	0.341446512	,	0.324416926	,	0.078991105	,	0.269583684	,],
[	0.301264903	,	0.415300831	,	0.32907441	,	0.090682873	,]],
[[	0.084020918	,	0.341051837	,	0.357882389	,	0.25860272	,],
[	0.254592113	,	0.09062617	,	0.255876795	,	0.291523429	,],
[	0.343705003	,	0.325928323	,	0.078107118	,	0.272093546	,],
[	0.298794483	,	0.41295984	,	0.326791843	,	0.084998754	,]],
[[	0.087031144	,	0.341771294	,	0.358562582	,	0.259253996	,],
[	0.254550063	,	0.08593866	,	0.254828654	,	0.291454199	,],
[	0.340816075	,	0.32335003	,	0.076486674	,	0.268984242	,],
[	0.298688475	,	0.41290879	,	0.326613381	,	0.086569879	,]],
[[	0.085942312	,	0.343452842	,	0.360404544	,	0.261182136	,],
[	0.252704501	,	0.086668721	,	0.253610325	,	0.28977475	,],
[	0.343122248	,	0.326508234	,	0.082271437	,	0.271311105	,],
[	0.302165183	,	0.415901196	,	0.329954202	,	0.087891473	,]],
[[	0.085419773	,	0.346290948	,	0.363044619	,	0.259025554	,],
[	0.251645155	,	0.087507401	,	0.257661544	,	0.289871175	,],
[	0.336812137	,	0.325184287	,	0.07638189	,	0.265659737	,],
[	0.297264755	,	0.416790266	,	0.330644547	,	0.085925611	,]],
[[	0.087172185	,	0.350037474	,	0.366590711	,	0.263025304	,],
[	0.248426997	,	0.086301836	,	0.254927996	,	0.286753508	,],
[	0.337813582	,	0.326944976	,	0.080397188	,	0.266567738	,],
[	0.301668689	,	0.421029096	,	0.334981489	,	0.092939726	,]],
[[	0.083723736	,	0.345656548	,	0.362446822	,	0.258484477	,],
[	0.251208754	,	0.092508013	,	0.258500261	,	0.289374779	,],
[	0.338819569	,	0.326943651	,	0.077458993	,	0.267624937	,],
[	0.297367431	,	0.416911774	,	0.330795547	,	0.084817523	,]],
[[	0.088427731	,	0.347758851	,	0.364383146	,	0.260460305	,],
[	0.249386067	,	0.085907447	,	0.255646684	,	0.28766548	,],
[	0.336539167	,	0.325073843	,	0.077073562	,	0.265298407	,],
[	0.298575982	,	0.418106483	,	0.331993795	,	0.088153545	,]],
[[	0.084529421	,	0.347386542	,	0.364170137	,	0.26032865	,],
[	0.248677211	,	0.087937438	,	0.255527415	,	0.286966677	,],
[	0.339805167	,	0.329186029	,	0.083901109	,	0.268608372	,],
[	0.299705057	,	0.41888198	,	0.332908956	,	0.086525683	,]],
[[	0.084507308	,	0.345731149	,	0.362537968	,	0.258498496	,],
[	0.252906914	,	0.088588791	,	0.258854855	,	0.291005099	,],
[	0.336979797	,	0.325321971	,	0.07618341	,	0.265761354	,],
[	0.297337416	,	0.416733446	,	0.330598657	,	0.085313897	,]],
[[	0.088405399	,	0.351208015	,	0.367722801	,	0.264227286	,],
[	0.248784442	,	0.086168462	,	0.25517275	,	0.286990458	,],
[	0.337467028	,	0.326357995	,	0.079398964	,	0.266211098	,],
[	0.30055173	,	0.419901058	,	0.333754811	,	0.091312211	,]],
[[	0.083792191	,	0.345982009	,	0.362599547	,	0.258859354	,],
[	0.250192581	,	0.090960706	,	0.25717754	,	0.288264039	,],
[	0.339517029	,	0.32753534	,	0.078070404	,	0.268368948	,],
[	0.297886826	,	0.417355671	,	0.331252653	,	0.085165888	,]],
[[	0.08713894	,	0.347039163	,	0.36373658	,	0.259797831	,],
[	0.250223386	,	0.086435312	,	0.256379927	,	0.288517701	,],
[	0.336452739	,	0.324954839	,	0.076527878	,	0.265164612	,],
[	0.297985973	,	0.41743444	,	0.331369262	,	0.087047311	,]],
[[	0.08585115	,	0.348495161	,	0.365182918	,	0.261367892	,],
[	0.248474041	,	0.087074127	,	0.255125307	,	0.28663058	,],
[	0.339217185	,	0.328455588	,	0.082570478	,	0.267941627	,],
[	0.300486754	,	0.419621823	,	0.333735423	,	0.087206224	,]],
[[	0.098644095	,	0.225274434	,	0.264359487	,	0.279878537	,],
[	0.375837302	,	0.079176697	,	0.25538391	,	0.40329385	,],
[	0.442562708	,	0.290603621	,	0.071567271	,	0.366926863	,],
[	0.332330953	,	0.309787493	,	0.236145426	,	0.092545743	,]],
[[	0.092905439	,	0.223561796	,	0.262472542	,	0.277521588	,],
[	0.373839336	,	0.078864745	,	0.253336852	,	0.401343097	,],
[	0.44355776	,	0.291596214	,	0.07453783	,	0.367858247	,],
[	0.336431529	,	0.313584149	,	0.240129652	,	0.099354969	,]],
[[	0.094496238	,	0.222819693	,	0.261865473	,	0.277169363	,],
[	0.377573748	,	0.085669882	,	0.257602713	,	0.405200182	,],
[	0.444735583	,	0.292937345	,	0.07355412	,	0.369213877	,],
[	0.332787772	,	0.31022531	,	0.236681073	,	0.091911396	,]],
[[	0.095321521	,	0.226897335	,	0.265659427	,	0.280466068	,],
[	0.374393238	,	0.078237802	,	0.253862997	,	0.401899069	,],
[	0.442105328	,	0.290004894	,	0.07169747	,	0.366343333	,],
[	0.333547627	,	0.310635982	,	0.237133559	,	0.094631909	,]],
[[	0.092389087	,	0.222430047	,	0.261473211	,	0.276447239	,],
[	0.375109409	,	0.081273215	,	0.254784955	,	0.402740663	,],
[	0.445332949	,	0.293573632	,	0.077666075	,	0.369625352	,],
[	0.334864283	,	0.31258035	,	0.239151745	,	0.093442278	,]],
[[	0.097186113	,	0.224295345	,	0.263466114	,	0.278795107	,],
[	0.376820582	,	0.080110196	,	0.256171884	,	0.404107668	,],
[	0.443081731	,	0.291021398	,	0.071875785	,	0.367476651	,],
[	0.332406368	,	0.309891186	,	0.236369465	,	0.092120195	,]],
[[	0.093322042	,	0.22425958	,	0.263019546	,	0.27806938	,],
[	0.374089356	,	0.078712197	,	0.25351862	,	0.401515154	,],
[	0.442991895	,	0.29091757	,	0.073454965	,	0.367235272	,],
[	0.335694935	,	0.312790102	,	0.239271922	,	0.097797255	,]],
[[	0.093646449	,	0.22262901	,	0.261557824	,	0.276833448	,],
[	0.376607725	,	0.084213702	,	0.25649126	,	0.404282611	,],
[	0.445935185	,	0.294048983	,	0.074553525	,	0.370350574	,],
[	0.333398806	,	0.310838461	,	0.237246992	,	0.09222203	,]],
[[	0.096633312	,	0.228239942	,	0.266989701	,	0.281715169	,],
[	0.37518594	,	0.078703297	,	0.254488797	,	0.402555416	,],
[	0.442745462	,	0.290494896	,	0.071821567	,	0.366906561	,],
[	0.333045723	,	0.310253682	,	0.23662759	,	0.093586521	,]],
[[	0.092248892	,	0.222563036	,	0.261567511	,	0.276488267	,],
[	0.374624489	,	0.080264686	,	0.254218644	,	0.402127277	,],
[	0.444862693	,	0.293016992	,	0.076474113	,	0.369044745	,],
[	0.336200514	,	0.314012159	,	0.240520636	,	0.094646908	,]],
[[	0.095776069	,	0.228291622	,	0.266974605	,	0.277505441	,],
[	0.369038449	,	0.084439277	,	0.256304884	,	0.398360031	,],
[	0.437413591	,	0.292153081	,	0.074424832	,	0.3640937	,],
[	0.32716005	,	0.311207771	,	0.239049043	,	0.091265826	,]],
[[	0.091549804	,	0.227819615	,	0.266278842	,	0.276428858	,],
[	0.366400719	,	0.077905908	,	0.252969085	,	0.395514101	,],
[	0.433680915	,	0.288162323	,	0.071228534	,	0.360138312	,],
[	0.328083522	,	0.311921029	,	0.239591372	,	0.093928221	,]],
[[	0.092017269	,	0.226179007	,	0.264728915	,	0.275134811	,],
[	0.366309984	,	0.079872299	,	0.253111471	,	0.395516807	,],
[	0.435921324	,	0.290618305	,	0.075643816	,	0.362350428	,],
[	0.329509527	,	0.313620672	,	0.241348076	,	0.092888526	,]],
[[	0.094629288	,	0.2315556	,	0.270006672	,	0.279800358	,],
[	0.369084357	,	0.080235348	,	0.255851077	,	0.398073728	,],
[	0.435044627	,	0.289658783	,	0.071863475	,	0.361615664	,],
[	0.326655831	,	0.310551628	,	0.23812243	,	0.091245709	,]],
[[	0.090663466	,	0.226408572	,	0.264968827	,	0.275072803	,],
[	0.365567463	,	0.077844696	,	0.252289508	,	0.394733033	,],
[	0.433788972	,	0.288329175	,	0.072054293	,	0.360127732	,],
[	0.330154425	,	0.31357697	,	0.241259896	,	0.09689594	,]],
[[	0.094375338	,	0.227442918	,	0.266155848	,	0.276499853	,],
[	0.367787729	,	0.082743735	,	0.254860253	,	0.397018165	,],
[	0.438191961	,	0.293219133	,	0.079581014	,	0.364735822	,],
[	0.327523162	,	0.31160725	,	0.239386609	,	0.091090158	,]],
[[	0.092031272	,	0.228506719	,	0.266892123	,	0.276950888	,],
[	0.367112206	,	0.07843424	,	0.253720251	,	0.39606689	,],
[	0.434124808	,	0.288543145	,	0.071304661	,	0.360447971	,],
[	0.32739773	,	0.31123725	,	0.238813263	,	0.092794318	,]],
[[	0.091146268	,	0.225770708	,	0.264341289	,	0.274594672	,],
[	0.365872736	,	0.078937416	,	0.252564441	,	0.394969436	,],
[	0.435336088	,	0.290002572	,	0.074670639	,	0.361704954	,],
[	0.330905564	,	0.315057613	,	0.242774711	,	0.094128279	,]],
[[	0.096010002	,	0.233061295	,	0.271282686	,	0.281066323	,],
[	0.370116616	,	0.086447676	,	0.257340867	,	0.399269379	,],
[	0.436073024	,	0.290589778	,	0.072816675	,	0.362463681	,],
[	0.326943682	,	0.310844072	,	0.238456741	,	0.091067498	,]],
[[	0.090547025	,	0.226477601	,	0.265027762	,	0.275014579	,],
[	0.365764058	,	0.077674186	,	0.252420458	,	0.394893944	,],
[	0.43402605	,	0.288560737	,	0.071948631	,	0.360373676	,],
[	0.329433385	,	0.313013352	,	0.240652363	,	0.095685658	,]],
[[	0.09383034	,	0.230690472	,	0.26854757	,	0.273830732	,],
[	0.359678161	,	0.08088428	,	0.251636373	,	0.388976486	,],
[	0.428223801	,	0.289340158	,	0.075088737	,	0.355408947	,],
[	0.323895015	,	0.314325879	,	0.241779334	,	0.091472129	,]],
[[	0.090776133	,	0.231232948	,	0.268805654	,	0.273724689	,],
[	0.361293751	,	0.079473351	,	0.252930974	,	0.390301415	,],
[	0.427540787	,	0.288424667	,	0.071119645	,	0.354722542	,],
[	0.322659897	,	0.312889583	,	0.240193785	,	0.091569589	,]],
[[	0.090743334	,	0.229019443	,	0.266827904	,	0.272032813	,],
[	0.358371431	,	0.078062357	,	0.250148356	,	0.38767734	,],
[	0.426441592	,	0.287223414	,	0.071467652	,	0.35347902	,],
[	0.326837243	,	0.316640214	,	0.244118935	,	0.09838171	,]],
[[	0.094358968	,	0.235232448	,	0.272883093	,	0.277471963	,],
[	0.361593673	,	0.084083664	,	0.253584414	,	0.390825935	,],
[	0.430790954	,	0.291889946	,	0.074462026	,	0.358104868	,],
[	0.32260827	,	0.312819822	,	0.240196295	,	0.090553374	,]],
[[	0.089463359	,	0.229316809	,	0.26714414	,	0.272035794	,],
[	0.359322668	,	0.077836528	,	0.250925557	,	0.388383913	,],
[	0.426399321	,	0.287049485	,	0.070280397	,	0.353352245	,],
[	0.324264301	,	0.314099682	,	0.241337197	,	0.093933947	,]],
[[	0.092940009	,	0.230158963	,	0.268159042	,	0.273260283	,],
[	0.35924072	,	0.079808493	,	0.251029503	,	0.388470993	,],
[	0.428139592	,	0.289037792	,	0.074320526	,	0.355097848	,],
[	0.324831553	,	0.315265741	,	0.2427225	,	0.092239912	,]],
[[	0.091409471	,	0.231978498	,	0.26949163	,	0.274465049	,],
[	0.362373148	,	0.080529484	,	0.253938695	,	0.391284056	,],
[	0.428229212	,	0.289107857	,	0.071824281	,	0.355410135	,],
[	0.322503929	,	0.312655687	,	0.239969031	,	0.09099794	,]],
[[	0.089913036	,	0.228797449	,	0.266504894	,	0.271577745	,],
[	0.358641635	,	0.077930188	,	0.250280072	,	0.387832711	,],
[	0.426552904	,	0.287359008	,	0.071314664	,	0.3535668	,],
[	0.326095937	,	0.315970423	,	0.243323665	,	0.096880544	,]],
[[	0.095814186	,	0.236700507	,	0.274205303	,	0.278864428	,],
[	0.360442411	,	0.082402031	,	0.252428506	,	0.389663935	,],
[	0.430349891	,	0.291657391	,	0.078030502	,	0.357544573	,],
[	0.322920961	,	0.313227696	,	0.240500229	,	0.090562429	,]],
[[	0.089677781	,	0.229853699	,	0.26741826	,	0.272412876	,],
[	0.360160259	,	0.078457322	,	0.251787196	,	0.389258711	,],
[	0.426874106	,	0.287653517	,	0.070607495	,	0.353922852	,],
[	0.323296906	,	0.313240417	,	0.240475631	,	0.092563523	,]],
[[	0.094531058	,	0.238487171	,	0.276273421	,	0.276412395	,],
[	0.351322758	,	0.079029958	,	0.249698502	,	0.381710463	,],
[	0.420446584	,	0.288195789	,	0.073907922	,	0.349881068	,],
[	0.320666924	,	0.31750007	,	0.245390123	,	0.091204532	,]],
[[	0.090427489	,	0.238176346	,	0.275815567	,	0.275554394	,],
[	0.35591389	,	0.086662958	,	0.254821754	,	0.386198411	,],
[	0.421181199	,	0.288883293	,	0.071947422	,	0.350744013	,],
[	0.31805547	,	0.314709268	,	0.242488011	,	0.089735477	,]],
[[	0.091024671	,	0.236677048	,	0.274400575	,	0.274431261	,],
[	0.351390253	,	0.077784371	,	0.249557105	,	0.381614587	,],
[	0.419174626	,	0.286886873	,	0.071072101	,	0.348584492	,],
[	0.321675227	,	0.317990698	,	0.245824057	,	0.095670372	,]],
[[	0.093578672	,	0.242078801	,	0.279604063	,	0.279089025	,],
[	0.35271201	,	0.08152788	,	0.251164013	,	0.383039565	,],
[	0.422490942	,	0.290519995	,	0.077201976	,	0.352016147	,],
[	0.31896056	,	0.315663894	,	0.243375026	,	0.089700034	,]],
[[	0.089522985	,	0.236835656	,	0.27449227	,	0.274202352	,],
[	0.353447836	,	0.079061546	,	0.251681741	,	0.383452086	,],
[	0.419722457	,	0.287354378	,	0.070760373	,	0.349161032	,],
[	0.31891951	,	0.315494446	,	0.243102408	,	0.091391436	,]],
[[	0.093197037	,	0.237701057	,	0.275497292	,	0.275455098	,],
[	0.351405519	,	0.078622762	,	0.249715587	,	0.381661027	,],
[	0.420031751	,	0.287670231	,	0.072778068	,	0.349276243	,],
[	0.321912298	,	0.318742129	,	0.246667247	,	0.09211047	,]],
[[	0.09126957	,	0.23927934	,	0.276697286	,	0.276426338	,],
[	0.354872705	,	0.084843582	,	0.253492758	,	0.385000148	,],
[	0.422589528	,	0.290359168	,	0.073288016	,	0.352146505	,],
[	0.318091146	,	0.314799126	,	0.242479964	,	0.089293206	,]],
[[	0.090522029	,	0.236793299	,	0.274385281	,	0.274356198	,],
[	0.351919462	,	0.077960781	,	0.250012989	,	0.382010915	,],
[	0.419019783	,	0.286591026	,	0.070629593	,	0.348390693	,],
[	0.320531418	,	0.316812651	,	0.244638717	,	0.093875327	,]],
[[	0.09498827	,	0.243359151	,	0.280871515	,	0.280300834	,],
[	0.352446531	,	0.080679955	,	0.250843121	,	0.382635345	,],
[	0.421566147	,	0.28946851	,	0.075721649	,	0.350996776	,],
[	0.319510634	,	0.316252991	,	0.244000606	,	0.089921947	,]],
[[	0.089751428	,	0.237274335	,	0.274916599	,	0.274541339	,],
[	0.354540016	,	0.079960285	,	0.252587113	,	0.384531001	,],
[	0.420438675	,	0.288146015	,	0.07124897	,	0.349824228	,],
[	0.318351538	,	0.314843485	,	0.242502305	,	0.090268926	,]],
[[	0.091453674	,	0.237200013	,	0.273201919	,	0.268723577	,],
[	0.344088258	,	0.077292651	,	0.247422503	,	0.374411011	,],
[	0.413517669	,	0.285793472	,	0.07204244	,	0.342416534	,],
[	0.317290232	,	0.318744674	,	0.24783071	,	0.090042511	,]],
[[	0.089261622	,	0.238375035	,	0.274121452	,	0.269353484	,],
[	0.348004642	,	0.084343003	,	0.251835558	,	0.378438253	,],
[	0.415610071	,	0.287852495	,	0.071870782	,	0.344809481	,],
[	0.31479663	,	0.316141246	,	0.24511338	,	0.088648879	,]],
[[	0.088365756	,	0.235827569	,	0.271732345	,	0.267156016	,],
[	0.344785074	,	0.076742867	,	0.247837356	,	0.37493653	,],
[	0.412726385	,	0.284812008	,	0.069862982	,	0.341653652	,],
[	0.317470604	,	0.318517403	,	0.247700242	,	0.093861119	,]],
[[	0.09304264	,	0.242816683	,	0.278388064	,	0.27344485	,],
[	0.34544193	,	0.0797206	,	0.248791838	,	0.375819034	,],
[	0.415539916	,	0.288121935	,	0.075433343	,	0.344608219	,],
[	0.315448217	,	0.316885069	,	0.245787657	,	0.088389585	,]],
[[	0.08762723	,	0.236457902	,	0.2722966	,	0.267526583	,],
[	0.347381816	,	0.078625675	,	0.250377961	,	0.377463704	,],
[	0.413854262	,	0.285912447	,	0.0701279	,	0.342747703	,],
[	0.315216405	,	0.316479156	,	0.245255701	,	0.089824181	,]],
[[	0.090470649	,	0.236781819	,	0.272725499	,	0.26811882	,],
[	0.344130061	,	0.076977567	,	0.247382332	,	0.374520299	,],
[	0.413167632	,	0.285387231	,	0.071279859	,	0.342084095	,],
[	0.318466484	,	0.320065424	,	0.249149332	,	0.091058828	,]],
[[	0.090139268	,	0.23942431	,	0.275110578	,	0.270231258	,],
[	0.347170576	,	0.082818578	,	0.250784844	,	0.377469817	,],
[	0.416998713	,	0.289193698	,	0.073032244	,	0.34610454	,],
[	0.314497348	,	0.315889428	,	0.244757483	,	0.087990144	,]],
[[	0.087992699	,	0.235686376	,	0.271550458	,	0.2669363	,],
[	0.345455491	,	0.077188366	,	0.248471514	,	0.375563304	,],
[	0.413331503	,	0.285302214	,	0.069984151	,	0.342086193	,],
[	0.316483034	,	0.317541988	,	0.246539104	,	0.092272438	,]],
[[	0.093592232	,	0.238456571	,	0.274313781	,	0.269978998	,],
[	0.34482794	,	0.078668043	,	0.248042114	,	0.375119349	,],
[	0.414598882	,	0.286968273	,	0.073980196	,	0.343537017	,],
[	0.316077861	,	0.317447379	,	0.246441449	,	0.08892606	,]],
[[	0.088017721	,	0.237013367	,	0.27275292	,	0.267908859	,],
[	0.348330866	,	0.079634244	,	0.251444886	,	0.378391767	,],
[	0.414274052	,	0.286367796	,	0.070503098	,	0.343145263	,],
[	0.314635759	,	0.31585922	,	0.24483943	,	0.088859158	,]],
[[	0.090376528	,	0.241447398	,	0.277691033	,	0.267940394	,],
[	0.338380934	,	0.076539669	,	0.247915472	,	0.37013807	,],
[	0.40741992	,	0.285337224	,	0.071698877	,	0.337632358	,],
[	0.315075947	,	0.322959204	,	0.252275026	,	0.090685858	,]],
[[	0.088181352	,	0.242401879	,	0.27837784	,	0.268422742	,],
[	0.340502612	,	0.081387971	,	0.250390964	,	0.37219832	,],
[	0.411038704	,	0.288982981	,	0.073712976	,	0.341585668	,],
[	0.311392979	,	0.319301714	,	0.248387698	,	0.088045181	,]],
[[	0.087143587	,	0.239633915	,	0.275782094	,	0.266026576	,],
[	0.339531927	,	0.076732814	,	0.248839077	,	0.37105326	,],
[	0.407066277	,	0.284847212	,	0.070228426	,	0.337309928	,],
[	0.313844764	,	0.321425545	,	0.25060284	,	0.092844317	,]],
[[	0.092400274	,	0.247175734	,	0.283162863	,	0.273018594	,],
[	0.338534262	,	0.07778439	,	0.248163552	,	0.370249304	,],
[	0.408653021	,	0.286693278	,	0.074117975	,	0.338868206	,],
[	0.312845049	,	0.320707429	,	0.249791537	,	0.088752108	,]],
[[	0.08673739	,	0.240514207	,	0.27667148	,	0.266573041	,],
[	0.343005644	,	0.079783538	,	0.252367977	,	0.374464364	,],
[	0.40897559	,	0.286718202	,	0.071439395	,	0.33915836	,],
[	0.312165016	,	0.319811652	,	0.248854852	,	0.08954555	,]],
[[	0.08896919	,	0.240383828	,	0.276663009	,	0.266912763	,],
[	0.338532737	,	0.076421882	,	0.248033571	,	0.370167591	,],
[	0.407055323	,	0.28486994	,	0.070896777	,	0.337188822	,],
[	0.316275091	,	0.324343021	,	0.253606512	,	0.091801955	,]],
[[	0.089147537	,	0.243556471	,	0.27938002	,	0.269465762	,],
[	0.339919973	,	0.080245964	,	0.249598599	,	0.371576159	,],
[	0.410573971	,	0.289000084	,	0.077768696	,	0.341055885	,],
[	0.311480314	,	0.319393676	,	0.248476944	,	0.08789815	,]],
[[	0.086893663	,	0.239820474	,	0.275789888	,	0.265971424	,],
[	0.340520069	,	0.077653874	,	0.249832912	,	0.371984568	,],
[	0.407562213	,	0.28539698	,	0.070501816	,	0.337779517	,],
[	0.31322883	,	0.32078955	,	0.249909757	,	0.091606237	,]],
[[	0.092379889	,	0.24246847	,	0.278478464	,	0.268995761	,],
[	0.33835772	,	0.077007751	,	0.247735247	,	0.369917629	,],
[	0.408141009	,	0.286184809	,	0.073216883	,	0.338433453	,],
[	0.313488084	,	0.321404426	,	0.25047621	,	0.089280945	,]],
[[	0.087268677	,	0.241364058	,	0.27737947	,	0.267313573	,],
[	0.341914175	,	0.083624222	,	0.251873292	,	0.373480825	,],
[	0.410074862	,	0.287918484	,	0.07254057	,	0.340360391	,],
[	0.311565611	,	0.319230034	,	0.248269501	,	0.088529482	,]],
[[	0.089366194	,	0.245999276	,	0.280992294	,	0.266630566	,],
[	0.333083944	,	0.076012977	,	0.247092128	,	0.365338011	,],
[	0.402698176	,	0.285494372	,	0.069682413	,	0.333019604	,],
[	0.314260521	,	0.327395801	,	0.256282036	,	0.090750264	,]],
[[	0.087127568	,	0.246797849	,	0.281732473	,	0.267050296	,],
[	0.334005527	,	0.079288	,	0.248240367	,	0.366291704	,],
[	0.405163722	,	0.288376143	,	0.074939931	,	0.335596506	,],
[	0.309596193	,	0.322606954	,	0.251380731	,	0.0869403	,]],
[[	0.086079846	,	0.244124525	,	0.279080123	,	0.264690115	,],
[	0.335788571	,	0.077882077	,	0.249452185	,	0.367741064	,],
[	0.403813755	,	0.286661616	,	0.070117577	,	0.334214591	,],
[	0.311606524	,	0.324325539	,	0.253171594	,	0.091059707	,]],
[[	0.091405632	,	0.251695401	,	0.286421378	,	0.271542275	,],
[	0.332738003	,	0.076391024	,	0.246632074	,	0.36494503	,],
[	0.402990006	,	0.285947195	,	0.07104706	,	0.333396437	,],
[	0.311545396	,	0.324492245	,	0.253220896	,	0.088166503	,]],
[[	0.085751971	,	0.245217083	,	0.280129186	,	0.265465132	,],
[	0.335844033	,	0.082431882	,	0.250181122	,	0.368048388	,],
[	0.406665733	,	0.289558575	,	0.072602742	,	0.337154622	,],
[	0.309861396	,	0.322676589	,	0.251229909	,	0.087743628	,]],
[[	0.088274502	,	0.245364509	,	0.280419681	,	0.26592572	,],
[	0.333837187	,	0.076314971	,	0.247619889	,	0.365903634	,],
[	0.402959962	,	0.285665057	,	0.069677236	,	0.333199862	,],
[	0.314000886	,	0.326428517	,	0.255157362	,	0.094454061	,]],
[[	0.088263406	,	0.248089929	,	0.282948697	,	0.268200434	,],
[	0.33362536	,	0.078267049	,	0.247738836	,	0.36580043	,],
[	0.404542778	,	0.287630565	,	0.073677062	,	0.334759089	,],
[	0.310028357	,	0.323005263	,	0.251738837	,	0.08694266	,]],
[[	0.085685326	,	0.244170225	,	0.279057155	,	0.264535738	,],
[	0.336855534	,	0.078905108	,	0.250632065	,	0.368754956	,],
[	0.404774273	,	0.287523815	,	0.070815735	,	0.335000438	,],
[	0.310649748	,	0.323461981	,	0.252197266	,	0.089603824	,]],
[[	0.091613084	,	0.247198631	,	0.282080946	,	0.267808006	,],
[	0.332964607	,	0.076269522	,	0.246800516	,	0.365022584	,],
[	0.403056588	,	0.285863243	,	0.070500019	,	0.333242807	,],
[	0.312565195	,	0.325558022	,	0.254385881	,	0.089026464	,]],
[[	0.086076507	,	0.245646709	,	0.280642611	,	0.265860514	,],
[	0.33501939	,	0.080945635	,	0.249307726	,	0.367126268	,],
[	0.4066871	,	0.290083288	,	0.077330719	,	0.337149472	,],
[	0.309817434	,	0.322687742	,	0.251213255	,	0.087345036	,]],
[[	0.088983103	,	0.250945074	,	0.28526576	,	0.266115519	,],
[	0.329434193	,	0.076715885	,	0.247612088	,	0.361659134	,],
[	0.398183564	,	0.286657325	,	0.069293888	,	0.329309842	,],
[	0.313557242	,	0.332392106	,	0.260500171	,	0.092125863	,]],
[[	0.087593431	,	0.252556056	,	0.286574266	,	0.267335521	,],
[	0.328994007	,	0.078491248	,	0.24749656	,	0.361363228	,],
[	0.399705551	,	0.288420653	,	0.073147076	,	0.330822785	,],
[	0.308671124	,	0.327454699	,	0.255378909	,	0.08775037	,]],
[[	0.086322209	,	0.249514713	,	0.283699139	,	0.264540595	,],
[	0.332603002	,	0.079322862	,	0.250667377	,	0.36453903	,],
[	0.400244966	,	0.288668447	,	0.070650144	,	0.331362615	,],
[	0.310325427	,	0.328897607	,	0.256826255	,	0.09134844	,]],
[[	0.0915795	,	0.256939077	,	0.291026681	,	0.271577282	,],
[	0.328831906	,	0.076797576	,	0.247005337	,	0.361068081	,],
[	0.398275316	,	0.286779745	,	0.070111839	,	0.329428185	,],
[	0.311106958	,	0.329796396	,	0.257679712	,	0.089454284	,]],
[[	0.085577466	,	0.250274912	,	0.284471659	,	0.265087959	,],
[	0.330716341	,	0.081420934	,	0.249328959	,	0.362951568	,],
[	0.401974019	,	0.291092906	,	0.07690836	,	0.333356538	,],
[	0.309018118	,	0.32753802	,	0.255419537	,	0.088530429	,]],
[[	0.087802403	,	0.250120651	,	0.284466284	,	0.26528682	,],
[	0.330160815	,	0.07712405	,	0.248278765	,	0.362218753	,],
[	0.398607748	,	0.286968488	,	0.069621261	,	0.329659413	,],
[	0.312408236	,	0.330719277	,	0.258657157	,	0.094465427	,]],
[[	0.088348021	,	0.253444789	,	0.287353568	,	0.268158642	,],
[	0.328694083	,	0.077562461	,	0.247090421	,	0.360937703	,],
[	0.39907313	,	0.287713752	,	0.071912866	,	0.330067983	,],
[	0.309150166	,	0.327932482	,	0.255926086	,	0.087929207	,]],
[[	0.085869985	,	0.249604266	,	0.283737916	,	0.264540001	,],
[	0.33299993	,	0.085147493	,	0.25182903	,	0.364986151	,],
[	0.401178874	,	0.289674147	,	0.07170743	,	0.332352537	,],
[	0.309799036	,	0.328356158	,	0.25620771	,	0.090198561	,]],
[[	0.091212523	,	0.252180507	,	0.286305454	,	0.267376221	,],
[	0.328948267	,	0.07658413	,	0.247136031	,	0.361023527	,],
[	0.398483749	,	0.286875775	,	0.070034289	,	0.329492902	,],
[	0.311939714	,	0.330709977	,	0.25864249	,	0.090209529	,]],
[[	0.086242425	,	0.251136682	,	0.285277629	,	0.265818169	,],
[	0.330283253	,	0.080254206	,	0.248652835	,	0.362268537	,],
[	0.400823386	,	0.289680325	,	0.075152028	,	0.331846918	,],
[	0.308755125	,	0.327315265	,	0.25520179	,	0.088084784	,]],
[[	0.087819419	,	0.254356478	,	0.288002524	,	0.264451036	,],
[	0.324003869	,	0.077574768	,	0.248232993	,	0.357562057	,],
[	0.39319416	,	0.286895614	,	0.069724722	,	0.324563802	,],
[	0.30969814	,	0.333583571	,	0.261910192	,	0.093754692	,]],
[[	0.086442205	,	0.255852593	,	0.289341017	,	0.26566657	,],
[	0.322374992	,	0.077924479	,	0.246958795	,	0.356099427	,],
[	0.393626165	,	0.287613727	,	0.072144159	,	0.325008117	,],
[	0.305687356	,	0.330043435	,	0.25829941	,	0.086363005	,]],
[[	0.085123575	,	0.252995724	,	0.286640833	,	0.26310183	,],
[	0.326585005	,	0.085342492	,	0.251644396	,	0.360163057	,],
[	0.3957637	,	0.289552607	,	0.071781815	,	0.327208052	,],
[	0.306537537	,	0.33073175	,	0.259046552	,	0.089050861	,]],
[[	0.090892443	,	0.260626707	,	0.294094129	,	0.27035437	,],
[	0.322587299	,	0.07671203	,	0.246837326	,	0.356214627	,],
[	0.392591316	,	0.286379908	,	0.069748218	,	0.323892017	,],
[	0.308300779	,	0.332573172	,	0.260873212	,	0.088346476	,]],
[[	0.085009967	,	0.254121489	,	0.287722648	,	0.263969661	,],
[	0.323678355	,	0.080368807	,	0.248300511	,	0.357295308	,],
[	0.395422706	,	0.289701429	,	0.075453826	,	0.326836956	,],
[	0.305505331	,	0.329683847	,	0.257929962	,	0.086722909	,]],
[[	0.086791338	,	0.253722941	,	0.287346113	,	0.263718576	,],
[	0.324679883	,	0.077961054	,	0.248842476	,	0.358192731	,],
[	0.393843311	,	0.287446016	,	0.070072618	,	0.325195865	,],
[	0.308604259	,	0.332467944	,	0.260753105	,	0.092000241	,]],
[[	0.08762746	,	0.257222331	,	0.290514489	,	0.266868195	,],
[	0.322052751	,	0.076942823	,	0.246419335	,	0.355719401	,],
[	0.393331662	,	0.287262118	,	0.071506569	,	0.324628086	,],
[	0.306418461	,	0.330907862	,	0.259093655	,	0.086653601	,]],
[[	0.084771982	,	0.253050088	,	0.286574524	,	0.262980013	,],
[	0.325683316	,	0.083538473	,	0.25047743	,	0.359167125	,],
[	0.396921598	,	0.290659649	,	0.072939156	,	0.328334321	,],
[	0.306125802	,	0.33023496	,	0.258488258	,	0.088108002	,]],
[[	0.09019047	,	0.255682515	,	0.289263222	,	0.265952904	,],
[	0.3229305	,	0.07672812	,	0.247024753	,	0.356396656	,],
[	0.392679891	,	0.286380084	,	0.069538688	,	0.323976708	,],
[	0.309330206	,	0.333573	,	0.261876831	,	0.089232473	,]],
[[	0.085578047	,	0.254830457	,	0.288342893	,	0.264568735	,],
[	0.323332903	,	0.079343522	,	0.247804047	,	0.356810206	,],
[	0.394758763	,	0.288849689	,	0.07419337	,	0.326132914	,],
[	0.305472316	,	0.329648626	,	0.257849753	,	0.086211491	,]],
[[	0.086494737	,	0.2578017	,	0.290980185	,	0.262559626	,],
[	0.320965264	,	0.080027777	,	0.250451199	,	0.354879505	,],
[	0.39005673	,	0.289517254	,	0.07178549	,	0.322317824	,],
[	0.305839783	,	0.335296311	,	0.263705135	,	0.091082503	,]],
[[	0.086603783	,	0.260549676	,	0.293536926	,	0.265028022	,],
[	0.316652108	,	0.076920333	,	0.246309238	,	0.350795214	,],
[	0.387437759	,	0.287165503	,	0.07048281	,	0.319773554	,],
[	0.305130943	,	0.334963807	,	0.263361311	,	0.087697572	,]],
[[	0.084220987	,	0.25692142	,	0.289916435	,	0.261501393	,],
[	0.318294198	,	0.081290969	,	0.248353739	,	0.352388759	,],
[	0.390614496	,	0.290821072	,	0.076646836	,	0.322939491	,],
[	0.303697688	,	0.333480895	,	0.261789881	,	0.087671246	,]],
[[	0.089942483	,	0.25977611	,	0.292877414	,	0.264644073	,],
[	0.318462953	,	0.07781907	,	0.247889007	,	0.352428108	,],
[	0.388106695	,	0.287573273	,	0.070176817	,	0.320400751	,],
[	0.308054441	,	0.337564816	,	0.265937021	,	0.094394468	,]],
[[	0.084710238	,	0.258508768	,	0.291509741	,	0.262914924	,],
[	0.316763736	,	0.077911615	,	0.246389936	,	0.350837422	,],
[	0.388334359	,	0.288215065	,	0.072397721	,	0.32068893	,],
[	0.304150434	,	0.333930891	,	0.262133219	,	0.086861864	,]],
[[	0.085476	,	0.25713615	,	0.290334111	,	0.261892983	,],
[	0.320821937	,	0.085044371	,	0.250943865	,	0.354746669	,],
[	0.390890421	,	0.290496439	,	0.072759864	,	0.323282333	,],
[	0.305083581	,	0.334583838	,	0.262898274	,	0.089879523	,]],
[[	0.087654301	,	0.261667751	,	0.294517776	,	0.266052318	,],
[	0.317019153	,	0.076933805	,	0.246564878	,	0.351082556	,],
[	0.387225685	,	0.28686241	,	0.06999778	,	0.319532808	,],
[	0.306165741	,	0.335997278	,	0.264383525	,	0.088557292	,]],
[[	0.083783013	,	0.256787817	,	0.289867543	,	0.261325418	,],
[	0.317923761	,	0.080232102	,	0.247772301	,	0.351989487	,],
[	0.389643306	,	0.289743951	,	0.075126292	,	0.322038748	,],
[	0.303641893	,	0.333297119	,	0.261600366	,	0.086903642	,]],
[[	0.088639081	,	0.258964009	,	0.29203308	,	0.263718577	,],
[	0.319407124	,	0.078464225	,	0.248668828	,	0.353073477	,],
[	0.389059348	,	0.288553291	,	0.070902076	,	0.321283174	,],
[	0.307121961	,	0.336508519	,	0.264888873	,	0.092830352	,]],
[[	0.08519397	,	0.259025402	,	0.29201287	,	0.263368517	,],
[	0.316714452	,	0.077415972	,	0.246232945	,	0.350654573	,],
[	0.387724107	,	0.28751961	,	0.071408487	,	0.31992577	,],
[	0.304422281	,	0.33408886	,	0.262435886	,	0.086910759	,]],
[[	0.085538073	,	0.261577161	,	0.293679712	,	0.260709578	,],
[	0.313655887	,	0.083948452	,	0.249292711	,	0.348261057	,],
[	0.385596465	,	0.292880947	,	0.078832271	,	0.318983996	,],
[	0.301587401	,	0.336917996	,	0.265267434	,	0.088108361	,]],
[[	0.08564162	,	0.264220191	,	0.296123398	,	0.263041875	,],
[	0.311295008	,	0.077336037	,	0.24610697	,	0.345818753	,],
[	0.381290756	,	0.287696234	,	0.070158191	,	0.314582133	,],
[	0.303327576	,	0.338943976	,	0.26746236	,	0.087574187	,]],
[[	0.083637847	,	0.260948481	,	0.2928821	,	0.259870256	,],
[	0.310765902	,	0.079121287	,	0.245989314	,	0.345365882	,],
[	0.382653908	,	0.289413957	,	0.073897912	,	0.315848233	,],
[	0.300181575	,	0.335799131	,	0.264142046	,	0.08553386	,]],
[[	0.08886189	,	0.26330905	,	0.29529598	,	0.262560954	,],
[	0.314184139	,	0.079768431	,	0.249005666	,	0.348629129	,],
[	0.383116765	,	0.289536085	,	0.071198841	,	0.316343098	,],
[	0.303609503	,	0.338887146	,	0.267307398	,	0.091269757	,]],
[[	0.083688415	,	0.262044114	,	0.294084331	,	0.260912099	,],
[	0.310574945	,	0.077346049	,	0.245496177	,	0.345026213	,],
[	0.381399657	,	0.287954607	,	0.070997042	,	0.314623658	,],
[	0.301448673	,	0.336924121	,	0.265269497	,	0.085729756	,]],
[[	0.084663619	,	0.261099944	,	0.29323847	,	0.26013536	,],
[	0.312584262	,	0.082196473	,	0.247995392	,	0.347099315	,],
[	0.384550785	,	0.291682172	,	0.077163022	,	0.317880685	,],
[	0.300681339	,	0.33608408	,	0.264343325	,	0.086840529	,]],
[[	0.086919994	,	0.265635232	,	0.297392767	,	0.264336505	,],
[	0.311711241	,	0.077626772	,	0.246529856	,	0.346120832	,],
[	0.3818692	,	0.288236673	,	0.07042038	,	0.315052227	,],
[	0.30429375	,	0.339871012	,	0.268345273	,	0.088464224	,]],
[[	0.083410542	,	0.261175592	,	0.293110834	,	0.260001535	,],
[	0.310788637	,	0.078477309	,	0.24581115	,	0.345255971	,],
[	0.382175298	,	0.288842277	,	0.072810147	,	0.315274409	,],
[	0.300179566	,	0.335704035	,	0.26409197	,	0.085167653	,]],
[[	0.087641063	,	0.262711694	,	0.294666294	,	0.261742432	,],
[	0.315176758	,	0.08069138	,	0.249968519	,	0.349475971	,],
[	0.38384108	,	0.290265996	,	0.0719264	,	0.317200376	,],
[	0.302502859	,	0.337805392	,	0.266169603	,	0.089550805	,]],
[[	0.084543412	,	0.263097955	,	0.295067908	,	0.261803788	,],
[	0.310718635	,	0.077150221	,	0.245562206	,	0.34513504	,],
[	0.381191455	,	0.287784377	,	0.070616011	,	0.314412351	,],
[	0.301940643	,	0.337471795	,	0.265826988	,	0.086120679	,]],
[[	0.084871072	,	0.26486851	,	0.296611984	,	0.259445846	,],
[	0.306084316	,	0.080530106	,	0.246639704	,	0.341080961	,],
[	0.377626057	,	0.289544995	,	0.075066942	,	0.311551328	,],
[	0.297358901	,	0.338183708	,	0.266145889	,	0.085833307	,]],
[[	0.086534707	,	0.268889378	,	0.300336095	,	0.263118852	,],
[	0.306649696	,	0.077622829	,	0.246622069	,	0.341399848	,],
[	0.377094537	,	0.288423413	,	0.070967136	,	0.310940498	,],
[	0.301351487	,	0.342402071	,	0.270547888	,	0.088140649	,]],
[[	0.083158222	,	0.264446417	,	0.296063007	,	0.258831564	,],
[	0.304813805	,	0.077489011	,	0.24500449	,	0.339727086	,],
[	0.375773606	,	0.287331481	,	0.071521407	,	0.309689261	,],
[	0.297074245	,	0.338072882	,	0.266115415	,	0.084561542	,]],
[[	0.088151604	,	0.266755706	,	0.298344486	,	0.261385583	,],
[	0.308477416	,	0.084042964	,	0.249054049	,	0.343124221	,],
[	0.379566662	,	0.291640396	,	0.078379746	,	0.313540541	,],
[	0.298928969	,	0.339782568	,	0.267703618	,	0.088432589	,]],
[[	0.084039207	,	0.266238028	,	0.297898489	,	0.260508274	,],
[	0.305300973	,	0.076721777	,	0.245163395	,	0.339947456	,],
[	0.375597624	,	0.287003403	,	0.07013126	,	0.309473871	,],
[	0.298885735	,	0.339725475	,	0.26780877	,	0.085720012	,]],
[[	0.084001789	,	0.264457332	,	0.296220981	,	0.258912332	,],
[	0.305511589	,	0.079327106	,	0.245844801	,	0.340210124	,],
[	0.377106137	,	0.288762508	,	0.073824889	,	0.310876305	,],
[	0.297141522	,	0.337971695	,	0.265987865	,	0.085329994	,]],
[[	0.087524892	,	0.269758709	,	0.301175951	,	0.263923515	,],
[	0.307795179	,	0.078572516	,	0.247587788	,	0.342441309	,],
[	0.37769767	,	0.289029696	,	0.071512615	,	0.311540326	,],
[	0.301782476	,	0.342298341	,	0.270323814	,	0.092255791	,]],
[[	0.083235227	,	0.264824699	,	0.296305447	,	0.259084031	,],
[	0.304655022	,	0.076818587	,	0.244724043	,	0.339391163	,],
[	0.375468779	,	0.286839724	,	0.070721767	,	0.309308657	,],
[	0.297270624	,	0.33822889	,	0.26625652	,	0.084318527	,]],
[[	0.086816733	,	0.266074139	,	0.297622691	,	0.260519877	,],
[	0.307488379	,	0.082541206	,	0.247970419	,	0.342123808	,],
[	0.378882526	,	0.290923951	,	0.077120481	,	0.312754862	,],
[	0.298484334	,	0.339380777	,	0.267207026	,	0.087511214	,]],
[[	0.084799565	,	0.267064154	,	0.298533912	,	0.261191055	,],
[	0.305813962	,	0.076932	,	0.245640064	,	0.340440453	,],
[	0.376262291	,	0.287537184	,	0.070421568	,	0.310072832	,],
[	0.299709677	,	0.340748967	,	0.268775483	,	0.086445729	,]],
[[	0.084132249	,	0.270300182	,	0.301612715	,	0.259265865	,],
[	0.301674339	,	0.080919054	,	0.247521859	,	0.337565065	,],
[	0.373663067	,	0.291335135	,	0.074850723	,	0.308216101	,],
[	0.29673852	,	0.343829289	,	0.271224971	,	0.086490063	,]],
[[	0.085438666	,	0.273962715	,	0.305060519	,	0.262674769	,],
[	0.303012946	,	0.078952222	,	0.248384052	,	0.338813439	,],
[	0.373512632	,	0.290588002	,	0.071179119	,	0.308032834	,],
[	0.300262202	,	0.347540631	,	0.275063887	,	0.088176505	,]],
[[	0.082384292	,	0.26970185	,	0.300934691	,	0.258582973	,],
[	0.300474206	,	0.078135301	,	0.24617882	,	0.336489366	,],
[	0.371790922	,	0.289188995	,	0.071207427	,	0.306237576	,],
[	0.296034854	,	0.343408482	,	0.270773521	,	0.084694985	,]],
[[	0.08767216	,	0.27229091	,	0.303504449	,	0.261278006	,],
[	0.303548911	,	0.084154916	,	0.249783189	,	0.3394701	,],
[	0.37545698	,	0.293471121	,	0.078082855	,	0.310055355	,],
[	0.298406615	,	0.345463743	,	0.272880959	,	0.089095542	,]],
[[	0.082996002	,	0.271356905	,	0.302749303	,	0.260095618	,],
[	0.301233853	,	0.077525296	,	0.246627455	,	0.336952325	,],
[	0.372400557	,	0.289581314	,	0.070550253	,	0.306803265	,],
[	0.298240281	,	0.345434897	,	0.272751621	,	0.086179271	,]],
[[	0.083505233	,	0.269930529	,	0.301270426	,	0.258784464	,],
[	0.30138607	,	0.080012779	,	0.247082856	,	0.337216759	,],
[	0.373119204	,	0.290866413	,	0.073701717	,	0.307577802	,],
[	0.296566498	,	0.343748354	,	0.271014823	,	0.085856292	,]],
[[	0.086714575	,	0.275197495	,	0.306265985	,	0.263995139	,],
[	0.303982471	,	0.079710749	,	0.249242946	,	0.339606377	,],
[	0.37457325	,	0.291666877	,	0.071988535	,	0.308978694	,],
[	0.301053558	,	0.347859276	,	0.275353532	,	0.092803838	,]],
[[	0.082182085	,	0.269902831	,	0.301111396	,	0.258765719	,],
[	0.300312601	,	0.077426585	,	0.245769768	,	0.336116023	,],
[	0.371864277	,	0.289273275	,	0.070794494	,	0.306277158	,],
[	0.296615278	,	0.343825403	,	0.271221558	,	0.08494024	,]],
[[	0.086264937	,	0.271461347	,	0.30270156	,	0.260444408	,],
[	0.302640554	,	0.082636122	,	0.248621882	,	0.338325179	,],
[	0.375061098	,	0.29286283	,	0.076913648	,	0.309467647	,],
[	0.297772807	,	0.344821276	,	0.27213745	,	0.087896895	,]],
[[	0.083829242	,	0.272276127	,	0.303380214	,	0.260920971	,],
[	0.30136476	,	0.077495053	,	0.246713942	,	0.337219156	,],
[	0.372974718	,	0.290102244	,	0.070785093	,	0.307347304	,],
[	0.29905741	,	0.346185636	,	0.273542502	,	0.086897736	,]],
[[	0.084343227	,	0.274773368	,	0.304730071	,	0.257473709	,],
[	0.295371077	,	0.079709856	,	0.245705666	,	0.330618562	,],
[	0.368337751	,	0.291250531	,	0.074882925	,	0.302379041	,],
[	0.295210464	,	0.346405506	,	0.274486064	,	0.086025486	,]],
[[	0.084656907	,	0.277645725	,	0.307285545	,	0.260075454	,],
[	0.297706051	,	0.078823954	,	0.247443093	,	0.33276565	,],
[	0.368450312	,	0.290584714	,	0.071316896	,	0.302319748	,],
[	0.297748255	,	0.349198105	,	0.277464367	,	0.086460021	,]],
[[	0.082080484	,	0.273880753	,	0.303641163	,	0.25644413	,],
[	0.294551266	,	0.077256126	,	0.244582372	,	0.329746334	,],
[	0.366732756	,	0.289230504	,	0.071455846	,	0.300595787	,],
[	0.29419256	,	0.345591008	,	0.273650488	,	0.083812086	,]],
[[	0.087505742	,	0.276600528	,	0.306268218	,	0.259246788	,],
[	0.297163455	,	0.082463742	,	0.247568668	,	0.332188616	,],
[	0.370627461	,	0.293707243	,	0.078564605	,	0.304620257	,],
[	0.296827632	,	0.348001427	,	0.276209911	,	0.088799858	,]],
[[	0.082406907	,	0.275265503	,	0.305053889	,	0.257725996	,],
[	0.295862429	,	0.077294667	,	0.245517198	,	0.330848487	,],
[	0.367145817	,	0.289426948	,	0.070562104	,	0.301058498	,],
[	0.295735	,	0.346951329	,	0.275050479	,	0.084438721	,]],
[[	0.083467281	,	0.274274715	,	0.304191825	,	0.256868226	,],
[	0.294647783	,	0.078272717	,	0.24471102	,	0.329767885	,],
[	0.367871087	,	0.290607925	,	0.073796623	,	0.301766854	,],
[	0.294876718	,	0.346107627	,	0.27405148	,	0.085185697	,]],
[[	0.086092595	,	0.279086174	,	0.308653369	,	0.261497005	,],
[	0.299100778	,	0.080090974	,	0.248732226	,	0.334026503	,],
[	0.369048473	,	0.291272953	,	0.071903033	,	0.302998945	,],
[	0.298823259	,	0.350072876	,	0.278340148	,	0.087309194	,]],
[[	0.082113746	,	0.274178213	,	0.303885361	,	0.256717565	,],
[	0.294541135	,	0.076801412	,	0.244375506	,	0.32964464	,],
[	0.366700797	,	0.289111189	,	0.070839662	,	0.300460631	,],
[	0.294260899	,	0.345602143	,	0.273666742	,	0.083570785	,]],
[[	0.085914494	,	0.27555326	,	0.30527707	,	0.258130939	,],
[	0.296294406	,	0.081169518	,	0.246545868	,	0.331385531	,],
[	0.36966467	,	0.29265022	,	0.076894423	,	0.303598779	,],
[	0.296539044	,	0.347792463	,	0.275778702	,	0.087834237	,]],
[[	0.083309455	,	0.276101122	,	0.305835237	,	0.258487783	,],
[	0.29632644	,	0.077653267	,	0.24597656	,	0.331338345	,],
[	0.367398472	,	0.289701024	,	0.070681881	,	0.301366745	,],
[	0.296374717	,	0.347614944	,	0.27569378	,	0.084930605	,]],
[[	0.083849977	,	0.280009594	,	0.309425001	,	0.256858391	,],
[	0.290368605	,	0.07866724	,	0.246258606	,	0.326052325	,],
[	0.36414173	,	0.292302531	,	0.073179249	,	0.298032806	,],
[	0.293918747	,	0.351861594	,	0.278433626	,	0.084937916	,]],
[[	0.085227037	,	0.283651998	,	0.312986142	,	0.260372945	,],
[	0.294729566	,	0.080466168	,	0.250280248	,	0.330272445	,],
[	0.366433379	,	0.294116106	,	0.07288779	,	0.30055323	,],
[	0.298562985	,	0.356532526	,	0.283367634	,	0.08769434	,]],
[[	0.082127065	,	0.279510347	,	0.308870066	,	0.256327264	,],
[	0.290198578	,	0.07707329	,	0.245861626	,	0.325814181	,],
[	0.363114666	,	0.291003814	,	0.070736663	,	0.297101148	,],
[	0.293584887	,	0.351644836	,	0.278269884	,	0.083395333	,]],
[[	0.087272252	,	0.281981472	,	0.311318622	,	0.258930149	,],
[	0.29147649	,	0.081050128	,	0.247600363	,	0.327160652	,],
[	0.365908072	,	0.294313845	,	0.076373601	,	0.299816171	,],
[	0.295145005	,	0.353115733	,	0.279710557	,	0.087021334	,]],
[[	0.082932875	,	0.28118184	,	0.310658587	,	0.257939025	,],
[	0.292437654	,	0.078419298	,	0.247821777	,	0.327864831	,],
[	0.364396872	,	0.292212536	,	0.071177022	,	0.29844256	,],
[	0.296102854	,	0.353922493	,	0.280538188	,	0.085181808	,]],
[[	0.083172946	,	0.279728628	,	0.309180825	,	0.256545706	,],
[	0.290117614	,	0.077816573	,	0.245835922	,	0.325643353	,],
[	0.363904457	,	0.292009067	,	0.072535124	,	0.29781704	,],
[	0.293674046	,	0.351459499	,	0.278041988	,	0.084101289	,]],
[[	0.086505609	,	0.285117385	,	0.314253827	,	0.261663361	,],
[	0.293858619	,	0.084781347	,	0.250138869	,	0.329239085	,],
[	0.367621962	,	0.295370673	,	0.074072796	,	0.301687972	,],
[	0.297936107	,	0.355543687	,	0.282033133	,	0.090602408	,]],
[[	0.081956503	,	0.279612341	,	0.308984882	,	0.256348768	,],
[	0.290566274	,	0.076958568	,	0.246152559	,	0.32606469	,],
[	0.36351063	,	0.291304843	,	0.070734176	,	0.297295805	,],
[	0.294156859	,	0.352153982	,	0.278682	,	0.083567226	,]],
[[	0.086075063	,	0.281348165	,	0.31070011	,	0.258199487	,],
[	0.290939933	,	0.079915949	,	0.246881966	,	0.326479296	,],
[	0.365212556	,	0.293530163	,	0.075090627	,	0.299103891	,],
[	0.294742428	,	0.352628416	,	0.279196036	,	0.085973514	,]],
[[	0.083591939	,	0.281984918	,	0.311360028	,	0.258707989	,],
[	0.293544007	,	0.079344425	,	0.248836418	,	0.328876541	,],
[	0.365053628	,	0.292778277	,	0.07160851	,	0.299030749	,],
[	0.296830684	,	0.354669619	,	0.281318527	,	0.085781804	,]],
[[	0.083680261	,	0.285994844	,	0.314848334	,	0.257755775	,],
[	0.285320919	,	0.078956965	,	0.246339607	,	0.322487708	,],
[	0.359033418	,	0.293132106	,	0.071789472	,	0.29367114	,],
[	0.292425466	,	0.355683144	,	0.281969402	,	0.084642914	,]],
[[	0.085821939	,	0.29018165	,	0.318752409	,	0.261827263	,],
[	0.28889414	,	0.0856246	,	0.250382481	,	0.325861723	,],
[	0.362836983	,	0.296400458	,	0.073111096	,	0.297403162	,],
[	0.297162682	,	0.360067124	,	0.286487136	,	0.091676092	,]],
[[	0.082085594	,	0.285510914	,	0.314176459	,	0.257195391	,],
[	0.286016307	,	0.078423006	,	0.246722474	,	0.323124602	,],
[	0.358843529	,	0.29265103	,	0.070113402	,	0.29337359	,],
[	0.293017647	,	0.356353274	,	0.282706903	,	0.08418887	,]],
[[	0.086505428	,	0.287431189	,	0.316107881	,	0.259158534	,],
[	0.286415756	,	0.081035276	,	0.247514938	,	0.323466951	,],
[	0.360310772	,	0.294750209	,	0.074371953	,	0.29499604	,],
[	0.293903457	,	0.357017729	,	0.283432476	,	0.086889853	,]],
[[	0.083252248	,	0.287523591	,	0.316298382	,	0.259144238	,],
[	0.288932711	,	0.080545521	,	0.249513323	,	0.325811186	,],
[	0.360282081	,	0.294001847	,	0.070798849	,	0.29496192	,],
[	0.295695303	,	0.358718643	,	0.285164512	,	0.086155026	,]],
[[	0.082867791	,	0.285570399	,	0.3143951	,	0.257229819	,],
[	0.285501753	,	0.078621325	,	0.246272046	,	0.322481613	,],
[	0.358659503	,	0.292733577	,	0.071012395	,	0.293292958	,],
[	0.292727606	,	0.355844731	,	0.282147779	,	0.084315982	,]],
[[	0.087181551	,	0.291524649	,	0.320103202	,	0.263020368	,],
[	0.288053662	,	0.084165139	,	0.249397724	,	0.324975692	,],
[	0.362263072	,	0.296978248	,	0.077805374	,	0.297009072	,],
[	0.296084874	,	0.359007122	,	0.285383479	,	0.090050954	,]],
[[	0.08229798	,	0.285868249	,	0.314394625	,	0.257444273	,],
[	0.286535661	,	0.078508372	,	0.247212705	,	0.323495538	,],
[	0.358881188	,	0.292753515	,	0.069998957	,	0.293404111	,],
[	0.293696964	,	0.35682731	,	0.283263421	,	0.084418091	,]],
[[	0.08537226	,	0.286826226	,	0.315516063	,	0.258476941	,],
[	0.285640758	,	0.079872055	,	0.246657365	,	0.322574278	,],
[	0.359853008	,	0.293981091	,	0.073164662	,	0.294324621	,],
[	0.293547711	,	0.356611412	,	0.282911615	,	0.085801988	,]],
[[	0.084117408	,	0.288476431	,	0.317049435	,	0.259935649	,],
[	0.28997627	,	0.08148305	,	0.250491499	,	0.326813885	,],
[	0.361214583	,	0.294836698	,	0.071708767	,	0.29582549	,],
[	0.296515411	,	0.35961564	,	0.286051062	,	0.086991731	,]],
[[	0.08346699	,	0.290791537	,	0.319999515	,	0.25732854	,],
[	0.282019685	,	0.079339914	,	0.248220356	,	0.319331591	,],
[	0.356519324	,	0.294961483	,	0.072595842	,	0.290595089	,],
[	0.292094661	,	0.360787165	,	0.287532675	,	0.083738686	,]],
[[	0.086012528	,	0.295362846	,	0.324348504	,	0.261864776	,],
[	0.284671988	,	0.085014868	,	0.251401737	,	0.321849283	,],
[	0.360408453	,	0.299517946	,	0.07989864	,	0.294568961	,],
[	0.29621978	,	0.364691612	,	0.291514744	,	0.090238006	,]],
[[	0.082217975	,	0.290517657	,	0.31963409	,	0.257132809	,],
[	0.283479591	,	0.07967774	,	0.249478694	,	0.320693399	,],
[	0.356346862	,	0.294535845	,	0.071032683	,	0.290317082	,],
[	0.292334822	,	0.361101191	,	0.287915737	,	0.082930405	,]],
[[	0.086323546	,	0.292274926	,	0.321386649	,	0.258927305	,],
[	0.282573981	,	0.081042926	,	0.24910306	,	0.319915188	,],
[	0.357879218	,	0.296429371	,	0.075198695	,	0.291877938	,],
[	0.293223993	,	0.361933661	,	0.288683276	,	0.085635131	,]],
[[	0.083245921	,	0.292553463	,	0.321575305	,	0.25897653	,],
[	0.28663112	,	0.082132582	,	0.252452926	,	0.323653744	,],
[	0.358206921	,	0.296181556	,	0.072191591	,	0.292261172	,],
[	0.295179305	,	0.363671092	,	0.290625672	,	0.085113845	,]],
[[	0.082869013	,	0.29057653	,	0.319735165	,	0.257152929	,],
[	0.282161107	,	0.079071057	,	0.248265964	,	0.319435396	,],
[	0.356223505	,	0.294487583	,	0.07191061	,	0.290220517	,],
[	0.291919051	,	0.36051021	,	0.287181925	,	0.083009154	,]],
[[	0.087077584	,	0.296519326	,	0.325366654	,	0.262984391	,],
[	0.28423421	,	0.083990979	,	0.250893341	,	0.321383312	,],
[	0.359661291	,	0.298601658	,	0.078343895	,	0.293626062	,],
[	0.295571789	,	0.364045446	,	0.290796468	,	0.089056187	,]],
[[	0.082092045	,	0.290838239	,	0.319803888	,	0.257331777	,],
[	0.283950683	,	0.079703532	,	0.249848411	,	0.320964165	,],
[	0.356930403	,	0.294932103	,	0.071237034	,	0.290813844	,],
[	0.292801802	,	0.361497272	,	0.288416882	,	0.083164334	,]],
[[	0.085384784	,	0.291889768	,	0.32086188	,	0.258411256	,],
[	0.28248339	,	0.080293401	,	0.2486335	,	0.319490895	,],
[	0.35708257	,	0.29565709	,	0.073888584	,	0.290970086	,],
[	0.292660733	,	0.361332084	,	0.288007784	,	0.084666227	,]],
[[	0.084137407	,	0.293409902	,	0.322465474	,	0.259820476	,],
[	0.286430567	,	0.087769948	,	0.253347317	,	0.323617541	,],
[	0.359327261	,	0.297307053	,	0.073147154	,	0.293343768	,],
[	0.296146065	,	0.364678227	,	0.291543295	,	0.085931499	,]],
[[	0.083287943	,	0.296314081	,	0.325231296	,	0.258423335	,],
[	0.277394556	,	0.078863773	,	0.249237503	,	0.31611952	,],
[	0.352396635	,	0.296759565	,	0.072236242	,	0.287221401	,],
[	0.291862716	,	0.366113768	,	0.29233277	,	0.08438126	,]],
[[	0.084967048	,	0.300122774	,	0.328860254	,	0.262257394	,],
[	0.279856971	,	0.084286843	,	0.252169193	,	0.318424817	,],
[	0.355892227	,	0.300960137	,	0.079113113	,	0.290730461	,],
[	0.296000048	,	0.370110188	,	0.296510515	,	0.091173472	,]],
[[	0.08153162	,	0.295623109	,	0.324624124	,	0.257764278	,],
[	0.278885745	,	0.079132844	,	0.250388471	,	0.317559122	,],
[	0.35259835	,	0.296673556	,	0.071156373	,	0.287410574	,],
[	0.292167183	,	0.366478026	,	0.292758634	,	0.083793061	,]],
[[	0.086124678	,	0.297555777	,	0.326409107	,	0.25966292	,],
[	0.277719703	,	0.08012082	,	0.249584423	,	0.316150376	,],
[	0.353139356	,	0.297774834	,	0.074274726	,	0.287930551	,],
[	0.293019855	,	0.36738762	,	0.293668643	,	0.086554783	,]],
[[	0.082650491	,	0.297690849	,	0.326540056	,	0.259672118	,],
[	0.282318397	,	0.088331674	,	0.254874829	,	0.320703594	,],
[	0.355171368	,	0.299057175	,	0.073010231	,	0.28996617	,],
[	0.294810107	,	0.368862788	,	0.295269888	,	0.08591749	,]],
[[	0.082297959	,	0.295716774	,	0.324711355	,	0.257685828	,],
[	0.277703496	,	0.078565003	,	0.249275643	,	0.316189069	,],
[	0.35199705	,	0.296291868	,	0.071450325	,	0.286771244	,],
[	0.291854457	,	0.366094934	,	0.292258332	,	0.083972359	,]],
[[	0.086273414	,	0.301471713	,	0.330091916	,	0.263501201	,],
[	0.279190916	,	0.083084199	,	0.251289445	,	0.317573928	,],
[	0.354993848	,	0.299868679	,	0.077400211	,	0.289692692	,],
[	0.295054702	,	0.369158769	,	0.295413944	,	0.089606393	,]],
[[	0.081471944	,	0.295837415	,	0.324610321	,	0.257774571	,],
[	0.279520885	,	0.07959894	,	0.250996009	,	0.318003301	,],
[	0.353296641	,	0.297291744	,	0.071564497	,	0.288034574	,],
[	0.292578819	,	0.366894976	,	0.293186418	,	0.084037589	,]],
[[	0.084937937	,	0.297045442	,	0.325984576	,	0.259091082	,],
[	0.277649492	,	0.079450015	,	0.249462322	,	0.316187783	,],
[	0.35272389	,	0.297187701	,	0.073187077	,	0.287371302	,],
[	0.29248426	,	0.36677001	,	0.293029146	,	0.085485964	,]],
[[	0.08298977	,	0.298076839	,	0.326937295	,	0.259990559	,],
[	0.281534165	,	0.08671548	,	0.253922839	,	0.319834535	,],
[	0.355967089	,	0.29986884	,	0.073901006	,	0.290781406	,],
[	0.295901221	,	0.36994819	,	0.296412389	,	0.086765679	,]],
[[	0.082903647	,	0.300903209	,	0.329637853	,	0.257659225	,],
[	0.273478086	,	0.079188889	,	0.250249015	,	0.312661138	,],
[	0.349342588	,	0.298275086	,	0.072142571	,	0.284334954	,],
[	0.290888985	,	0.370997542	,	0.297022664	,	0.084121132	,]],
[[	0.085523262	,	0.305530265	,	0.333919456	,	0.262178334	,],
[	0.274253439	,	0.082618979	,	0.251609221	,	0.313429826	,],
[	0.350982497	,	0.3004679	,	0.076704325	,	0.285940518	,],
[	0.293779294	,	0.373923092	,	0.299806096	,	0.089298662	,]],
[[	0.081725271	,	0.300856009	,	0.32938869	,	0.25749391	,],
[	0.275720793	,	0.080683812	,	0.252348424	,	0.314874352	,],
[	0.351036305	,	0.299776019	,	0.073009442	,	0.285980225	,],
[	0.292055596	,	0.37225572	,	0.29826356	,	0.084547779	,]],
[[	0.086105963	,	0.302684579	,	0.331227747	,	0.259337065	,],
[	0.273004759	,	0.079686084	,	0.249996504	,	0.312138844	,],
[	0.349216021	,	0.298377138	,	0.073129851	,	0.284153303	,],
[	0.29117381	,	0.371485173	,	0.297406859	,	0.085203222	,]],
[[	0.082849098	,	0.302880075	,	0.331404123	,	0.259421477	,],
[	0.276138485	,	0.086007118	,	0.253639758	,	0.315188007	,],
[	0.353546591	,	0.303189655	,	0.080441098	,	0.288276138	,],
[	0.295669029	,	0.375670335	,	0.301673431	,	0.087875098	,]],
[[	0.082204187	,	0.300572764	,	0.329264777	,	0.257211067	,],
[	0.274066316	,	0.079449122	,	0.250645418	,	0.313000065	,],
[	0.349445228	,	0.298327267	,	0.071943005	,	0.284253355	,],
[	0.290836218	,	0.371045599	,	0.296901391	,	0.083650015	,]],
[[	0.086680989	,	0.306790343	,	0.335124694	,	0.263407901	,],
[	0.273796435	,	0.081678495	,	0.250960952	,	0.312915337	,],
[	0.350418298	,	0.299762952	,	0.075574577	,	0.285227666	,],
[	0.292875236	,	0.372940584	,	0.298794362	,	0.087842382	,]],
[[	0.081864872	,	0.301317949	,	0.329693466	,	0.257820334	,],
[	0.27674796	,	0.081508185	,	0.253273556	,	0.315707562	,],
[	0.351874255	,	0.300457762	,	0.073551448	,	0.286760218	,],
[	0.292865798	,	0.373032434	,	0.299065388	,	0.085125679	,]],
[[	0.084765802	,	0.301986149	,	0.330522921	,	0.258595107	,],
[	0.273346583	,	0.079436103	,	0.250165947	,	0.312316367	,],
[	0.349171521	,	0.29816391	,	0.072528594	,	0.283883701	,],
[	0.291064497	,	0.371265158	,	0.297104239	,	0.084527468	,]],
[[	0.083700047	,	0.30379682	,	0.332320102	,	0.260283767	,],
[	0.275317665	,	0.084704438	,	0.252849638	,	0.314424286	,],
[	0.352371323	,	0.301965643	,	0.078723941	,	0.287130678	,],
[	0.295501567	,	0.37550491	,	0.30143756	,	0.091619857	,]],
[[	0.082189304	,	0.307434123	,	0.335303864	,	0.259159642	,],
[	0.270875644	,	0.08099523	,	0.253447573	,	0.311651147	,],
[	0.346371282	,	0.301572433	,	0.073010386	,	0.282474597	,],
[	0.290150704	,	0.375895827	,	0.301469563	,	0.083553219	,]],
[[	0.0862285	,	0.31298791	,	0.340560644	,	0.264789016	,],
[	0.269425798	,	0.081506811	,	0.25224822	,	0.310147872	,],
[	0.346027284	,	0.301802335	,	0.075027534	,	0.282140518	,],
[	0.292278432	,	0.377764879	,	0.303291815	,	0.087418967	,]],
[[	0.081248333	,	0.307542507	,	0.335294454	,	0.259316921	,],
[	0.273889767	,	0.089437787	,	0.257353814	,	0.314546082	,],
[	0.349050764	,	0.30415157	,	0.075187057	,	0.285121569	,],
[	0.292171637	,	0.377712294	,	0.303417815	,	0.084835461	,]],
[[	0.084464226	,	0.308488428	,	0.336257992	,	0.260232845	,],
[	0.269667905	,	0.080345306	,	0.252279906	,	0.31038996	,],
[	0.345688596	,	0.301006052	,	0.073004655	,	0.281649882	,],
[	0.290211395	,	0.376038454	,	0.301457531	,	0.084186058	,]],
[[	0.083407715	,	0.310420106	,	0.338076351	,	0.262053671	,],
[	0.270640685	,	0.084093702	,	0.253754986	,	0.311345276	,],
[	0.347934964	,	0.303748397	,	0.078140981	,	0.283910262	,],
[	0.294451459	,	0.379990788	,	0.305592638	,	0.090773567	,]],
[[	0.081397814	,	0.306996754	,	0.334941312	,	0.258640516	,],
[	0.271733096	,	0.081399343	,	0.253883106	,	0.31233849	,],
[	0.347083863	,	0.302220041	,	0.073534118	,	0.283077114	,],
[	0.290615733	,	0.376277342	,	0.301686241	,	0.083584282	,]],
[[	0.087602357	,	0.314364963	,	0.341912645	,	0.266128057	,],
[	0.26924983	,	0.080888709	,	0.251855393	,	0.309913026	,],
[	0.345533862	,	0.301158896	,	0.074065084	,	0.281592729	,],
[	0.291501116	,	0.377133963	,	0.302541748	,	0.086244635	,]],
[[	0.081680524	,	0.308168144	,	0.335829265	,	0.259765959	,],
[	0.273052124	,	0.088006903	,	0.256360872	,	0.313580133	,],
[	0.350026694	,	0.305007572	,	0.076137221	,	0.286145229	,],
[	0.293302095	,	0.378861857	,	0.304587692	,	0.085761519	,]],
[[	0.083787402	,	0.308127349	,	0.335909484	,	0.259809141	,],
[	0.269997828	,	0.080332433	,	0.252543883	,	0.310675774	,],
[	0.345975246	,	0.301127053	,	0.072816552	,	0.281796511	,],
[	0.290243725	,	0.375909481	,	0.301319544	,	0.083708368	,]],
[[	0.084165923	,	0.310963897	,	0.338826103	,	0.262691971	,],
[	0.270269787	,	0.083097951	,	0.253303251	,	0.310898809	,],
[	0.346911611	,	0.302663756	,	0.076522802	,	0.282800605	,],
[	0.293684581	,	0.379258961	,	0.30474419	,	0.089447566	,]],
[[	0.082099298	,	0.311835373	,	0.340126731	,	0.259224298	,],
[	0.266743956	,	0.081616926	,	0.255531591	,	0.308783732	,],
[	0.342276491	,	0.302370886	,	0.073451221	,	0.278859937	,],
[	0.289327473	,	0.380538423	,	0.306368124	,	0.084090579	,]],
[[	0.085487596	,	0.316846248	,	0.344910863	,	0.264311994	,],
[	0.265030249	,	0.08189648	,	0.254271532	,	0.307180762	,],
[	0.342078668	,	0.30274304	,	0.075373596	,	0.278554542	,],
[	0.291880169	,	0.383006646	,	0.308858013	,	0.088718329	,]],
[[	0.081157412	,	0.311948066	,	0.340095894	,	0.259314046	,],
[	0.268963172	,	0.089279148	,	0.258713219	,	0.310944567	,],
[	0.34535343	,	0.305200684	,	0.075803993	,	0.281846731	,],
[	0.291139606	,	0.382322843	,	0.308222624	,	0.085124678	,]],
[[	0.084715204	,	0.313265622	,	0.341356953	,	0.260612993	,],
[	0.265203367	,	0.080841806	,	0.254030409	,	0.307249474	,],
[	0.341371165	,	0.30158359	,	0.073164922	,	0.277786698	,],
[	0.289756785	,	0.381043532	,	0.306794532	,	0.085024939	,]],
[[	0.082563808	,	0.31403217	,	0.342215023	,	0.261351646	,],
[	0.265987423	,	0.084199227	,	0.255417116	,	0.30805012	,],
[	0.343748341	,	0.304577499	,	0.078288228	,	0.280115029	,],
[	0.294108729	,	0.385212598	,	0.311115055	,	0.092114065	,]],
[[	0.081521838	,	0.311592751	,	0.339912253	,	0.258908115	,],
[	0.267677079	,	0.082211216	,	0.256177539	,	0.309496333	,],
[	0.342870717	,	0.30291981	,	0.073801252	,	0.27940185	,],
[	0.289813201	,	0.380835635	,	0.306696011	,	0.084144382	,]],
[[	0.08675698	,	0.318134633	,	0.3461681	,	0.265592941	,],
[	0.264971231	,	0.081357292	,	0.253910581	,	0.306865179	,],
[	0.341633873	,	0.302078285	,	0.074320996	,	0.278023058	,],
[	0.291098554	,	0.382273057	,	0.308044418	,	0.087406303	,]],
[[	0.081144451	,	0.312249568	,	0.340401651	,	0.259619394	,],
[	0.268257911	,	0.087859135	,	0.257807754	,	0.310143651	,],
[	0.345610759	,	0.306621477	,	0.081571631	,	0.282004508	,],
[	0.291742932	,	0.382768906	,	0.308716681	,	0.085559539	,]],
[[	0.083670127	,	0.312630759	,	0.340834471	,	0.259899772	,],
[	0.266134677	,	0.081157892	,	0.254771564	,	0.308003443	,],
[	0.341660705	,	0.301824817	,	0.073087213	,	0.278047284	,],
[	0.289766422	,	0.380924633	,	0.306738677	,	0.084608433	,]],
[[	0.08339974	,	0.314979218	,	0.343125744	,	0.262310177	,],
[	0.265792077	,	0.083403586	,	0.255027399	,	0.307718594	,],
[	0.342801529	,	0.303469585	,	0.076696481	,	0.279111173	,],
[	0.293489916	,	0.384581909	,	0.310475557	,	0.090917813	,]],
[[	0.082227827	,	0.317783503	,	0.344743016	,	0.25952421	,],
[	0.262805036	,	0.082117052	,	0.255965732	,	0.304833204	,],
[	0.339407582	,	0.30518618	,	0.073552079	,	0.275850257	,],
[	0.28915496	,	0.386006685	,	0.310571415	,	0.083896759	,]],
[[	0.084658231	,	0.322023359	,	0.348778408	,	0.263812836	,],
[	0.260844031	,	0.082252383	,	0.25455488	,	0.30309913	,],
[	0.339192248	,	0.305548353	,	0.075693721	,	0.275569218	,],
[	0.291864994	,	0.388576041	,	0.313319296	,	0.088808411	,]],
[[	0.08097093	,	0.31768643	,	0.344534192	,	0.259313766	,],
[	0.264766194	,	0.089827931	,	0.259200051	,	0.306899043	,],
[	0.342280836	,	0.308017839	,	0.075888031	,	0.278723059	,],
[	0.290677748	,	0.387447508	,	0.312174243	,	0.084702583	,]],
[[	0.085148387	,	0.319308463	,	0.346153678	,	0.261050089	,],
[	0.261052272	,	0.081142639	,	0.254516079	,	0.303256854	,],
[	0.338459194	,	0.304568481	,	0.073382765	,	0.274881056	,],
[	0.289717329	,	0.386506511	,	0.31119879	,	0.085116103	,]],
[[	0.081802454	,	0.319342017	,	0.346265888	,	0.26108997	,],
[	0.261668966	,	0.08459994	,	0.255802472	,	0.303977143	,],
[	0.340838698	,	0.307435363	,	0.07851887	,	0.27707583	,],
[	0.294254889	,	0.390658369	,	0.315556919	,	0.087930744	,]],
[[	0.081638678	,	0.31762772	,	0.344553415	,	0.259236089	,],
[	0.263819159	,	0.082881534	,	0.256894638	,	0.305758039	,],
[	0.340004459	,	0.305863986	,	0.074042771	,	0.276436205	,],
[	0.289557185	,	0.386251019	,	0.310852078	,	0.083825588	,]],
[[	0.086010067	,	0.323398637	,	0.349987386	,	0.265204536	,],
[	0.260917773	,	0.081826141	,	0.254334525	,	0.302860422	,],
[	0.338966082	,	0.305176068	,	0.074736333	,	0.275163365	,],
[	0.291469977	,	0.388216751	,	0.312842407	,	0.087842439	,]],
[[	0.080698468	,	0.317521073	,	0.344267485	,	0.259173261	,],
[	0.26417915	,	0.088417062	,	0.258282827	,	0.306179637	,],
[	0.342680895	,	0.309663964	,	0.082040193	,	0.278983425	,],
[	0.291678241	,	0.388244054	,	0.313085557	,	0.085305845	,]],
[[	0.084068017	,	0.318784263	,	0.345652353	,	0.260408212	,],
[	0.261900524	,	0.081524696	,	0.255201262	,	0.30387065	,],
[	0.338756979	,	0.304706765	,	0.073214349	,	0.275024836	,],
[	0.289419563	,	0.386173192	,	0.310847212	,	0.084361331	,]],
[[	0.083075759	,	0.320461496	,	0.347347982	,	0.262184002	,],
[	0.261651703	,	0.083838662	,	0.255494738	,	0.303732577	,],
[	0.339987447	,	0.306421773	,	0.077082479	,	0.276162873	,],
[	0.293277561	,	0.390046154	,	0.314686832	,	0.090889168	,]],
[[	0.082595288	,	0.322736267	,	0.35043656	,	0.260024045	,],
[	0.258875366	,	0.082503804	,	0.257479516	,	0.30142634	,],
[	0.336222766	,	0.306691163	,	0.074545594	,	0.272002724	,],
[	0.287988347	,	0.389291552	,	0.314711136	,	0.082938706	,]],
[[	0.085072243	,	0.327105076	,	0.354543617	,	0.264401407	,],
[	0.256023872	,	0.081463426	,	0.255078762	,	0.298675955	,],
[	0.33500442	,	0.306044145	,	0.075491721	,	0.270630496	,],
[	0.290352077	,	0.391549568	,	0.316996047	,	0.087247954	,]],
[[	0.081352458	,	0.322553611	,	0.350113713	,	0.259780269	,],
[	0.258962211	,	0.087535987	,	0.258555804	,	0.301479702	,],
[	0.338843802	,	0.310528357	,	0.082724045	,	0.274619189	,],
[	0.289565217	,	0.39079432	,	0.316335696	,	0.083808347	,]],
[[	0.085321449	,	0.324298257	,	0.351846003	,	0.261508189	,],
[	0.256749456	,	0.080843125	,	0.255575491	,	0.299370674	,],
[	0.334941968	,	0.305611804	,	0.073977727	,	0.27052252	,],
[	0.28825017	,	0.389707823	,	0.315106983	,	0.083787469	,]],
[[	0.08239471	,	0.324389417	,	0.351954083	,	0.261717354	,],
[	0.256030215	,	0.082704541	,	0.255330735	,	0.298770157	,],
[	0.336286503	,	0.30743709	,	0.077775359	,	0.271790981	,],
[	0.292335401	,	0.3936255	,	0.319080819	,	0.090591291	,]],
[[	0.081777391	,	0.322374971	,	0.350008285	,	0.259462351	,],
[	0.260058578	,	0.083431404	,	0.258601764	,	0.302529424	,],
[	0.336901576	,	0.307401206	,	0.075177079	,	0.272631524	,],
[	0.288068722	,	0.389304196	,	0.314607718	,	0.082651912	,]],
[[	0.086414167	,	0.328391188	,	0.355723399	,	0.265691868	,],
[	0.256012057	,	0.08091335	,	0.25493709	,	0.298547091	,],
[	0.334660168	,	0.305520696	,	0.074547147	,	0.270219109	,],
[	0.289498733	,	0.390831569	,	0.316119696	,	0.086002831	,]],
[[	0.081187461	,	0.322719893	,	0.350220263	,	0.259973087	,],
[	0.258349619	,	0.08628532	,	0.257850705	,	0.300818988	,],
[	0.337909299	,	0.309409551	,	0.08094562	,	0.273443638	,],
[	0.290577116	,	0.391796518	,	0.317335167	,	0.084593084	,]],
[[	0.083950946	,	0.323369167	,	0.351012691	,	0.260548396	,],
[	0.257580593	,	0.081243719	,	0.256171131	,	0.300028116	,],
[	0.33520706	,	0.305747591	,	0.073861955	,	0.270707647	,],
[	0.288038808	,	0.389421213	,	0.314774179	,	0.083211729	,]],
[[	0.083469962	,	0.32546723	,	0.353036203	,	0.262695349	,],
[	0.256190173	,	0.082275018	,	0.255360147	,	0.298746065	,],
[	0.335569567	,	0.306641993	,	0.076596941	,	0.2710211	,],
[	0.291787076	,	0.393097626	,	0.318527922	,	0.089363342	,]],
[[	0.082071891	,	0.327818832	,	0.354491729	,	0.259276125	,],
[	0.255412204	,	0.084142803	,	0.259808739	,	0.298423755	,],
[	0.332795923	,	0.309412791	,	0.074590948	,	0.268826196	,],
[	0.288414244	,	0.395679686	,	0.319585835	,	0.083338166	,]],
[[	0.085317635	,	0.332750424	,	0.359061189	,	0.264222372	,],
[	0.252342325	,	0.08294325	,	0.257252803	,	0.29550106	,],
[	0.331647536	,	0.308752299	,	0.075612268	,	0.267677016	,],
[	0.291490096	,	0.398632034	,	0.322525614	,	0.088564752	,]],
[[	0.081095298	,	0.327789928	,	0.354415145	,	0.259280892	,],
[	0.254897677	,	0.088864209	,	0.260434696	,	0.297947066	,],
[	0.335897566	,	0.312300052	,	0.077212675	,	0.272074951	,],
[	0.289732109	,	0.396859766	,	0.321016787	,	0.083822946	,]],
[[	0.085042363	,	0.329356549	,	0.355945413	,	0.260780811	,],
[	0.253412637	,	0.082710494	,	0.257996161	,	0.296519503	,],
[	0.33165939	,	0.308335769	,	0.073999591	,	0.267531512	,],
[	0.288834895	,	0.39627532	,	0.32014847	,	0.084466049	,]],
[[	0.082325121	,	0.329865749	,	0.356477465	,	0.261278145	,],
[	0.252524229	,	0.084306059	,	0.257750166	,	0.29579232	,],
[	0.333204828	,	0.310572927	,	0.078521285	,	0.269143095	,],
[	0.293367816	,	0.400073937	,	0.324246434	,	0.087080168	,]],
[[	0.081792829	,	0.327747951	,	0.354464552	,	0.25913768	,],
[	0.256990024	,	0.085639053	,	0.261260614	,	0.299894838	,],
[	0.333510379	,	0.310004037	,	0.074939815	,	0.269430563	,],
[	0.288858043	,	0.395969893	,	0.319928918	,	0.083222578	,]],
[[	0.086539522	,	0.333826717	,	0.36009384	,	0.265390283	,],
[	0.252521556	,	0.08256524	,	0.257179142	,	0.295558093	,],
[	0.331652604	,	0.308580222	,	0.074946209	,	0.267406686	,],
[	0.29034101	,	0.397542866	,	0.321439886	,	0.086979079	,]],
[[	0.081218084	,	0.328258872	,	0.35472406	,	0.259617812	,],
[	0.254258801	,	0.087452341	,	0.259624877	,	0.297250023	,],
[	0.335286256	,	0.312890923	,	0.082001374	,	0.27114215	,],
[	0.290861989	,	0.397823928	,	0.321993728	,	0.084675674	,]],
[[	0.083851921	,	0.328776107	,	0.355288397	,	0.2600577	,],
[	0.254343193	,	0.083416185	,	0.25878911	,	0.29738088	,],
[	0.33197867	,	0.308698211	,	0.074069513	,	0.267870017	,],
[	0.288490795	,	0.395737503	,	0.319605104	,	0.083714536	,]],
[[	0.083404999	,	0.330842139	,	0.357360832	,	0.262257305	,],
[	0.252304562	,	0.083617755	,	0.257305326	,	0.295471021	,],
[	0.332810214	,	0.309952013	,	0.077310137	,	0.268580447	,],
[	0.292557265	,	0.399718048	,	0.323778728	,	0.090366742	,]],
[[	0.082796077	,	0.332877272	,	0.359489596	,	0.25916135	,],
[	0.251521837	,	0.084157823	,	0.260557485	,	0.294544389	,],
[	0.329263272	,	0.311136015	,	0.074708984	,	0.265019712	,],
[	0.287244278	,	0.399731714	,	0.323359449	,	0.082919934	,]],
[[	0.085117365	,	0.337023914	,	0.3633145	,	0.263527223	,],
[	0.247787203	,	0.082203772	,	0.257287461	,	0.290906717	,],
[	0.327785268	,	0.31007735	,	0.075212595	,	0.263419146	,],
[	0.290206784	,	0.402694667	,	0.326294432	,	0.088331856	,]],
[[	0.081634501	,	0.33279342	,	0.359278053	,	0.259120651	,],
[	0.250089702	,	0.087976953	,	0.260361298	,	0.293151702	,],
[	0.331667112	,	0.31478623	,	0.08275051	,	0.267409198	,],
[	0.288549834	,	0.400897941	,	0.324691444	,	0.083417068	,]],
[[	0.085609256	,	0.334416242	,	0.360844477	,	0.260637569	,],
[	0.249090609	,	0.082401338	,	0.258314641	,	0.292218099	,],
[	0.327969379	,	0.309871734	,	0.073895309	,	0.263585614	,],
[	0.28781786	,	0.400376014	,	0.324054623	,	0.084297591	,]],
[[	0.082486782	,	0.33455386	,	0.360983309	,	0.260896711	,],
[	0.247793065	,	0.083489839	,	0.257668415	,	0.290979236	,],
[	0.328968883	,	0.311503896	,	0.077609622	,	0.264523111	,],
[	0.291879611	,	0.403858348	,	0.327807433	,	0.086451442	,]],
[[	0.082104487	,	0.332549861	,	0.359152062	,	0.258838755	,],
[	0.252188541	,	0.091505538	,	0.262773711	,	0.295247555	,],
[	0.330193188	,	0.311967902	,	0.075451483	,	0.265860484	,],
[	0.287395396	,	0.3997659	,	0.323368622	,	0.082755671	,]],
[[	0.086504519	,	0.338406407	,	0.364634208	,	0.264971626	,],
[	0.247955423	,	0.081971602	,	0.257265437	,	0.290947101	,],
[	0.327553464	,	0.309758194	,	0.074574746	,	0.263048294	,],
[	0.289307115	,	0.401774182	,	0.325348567	,	0.086882181	,]],
[[	0.08136266	,	0.332894176	,	0.359163368	,	0.259185747	,],
[	0.249198759	,	0.086312121	,	0.259268169	,	0.292233718	,],
[	0.330915035	,	0.313754932	,	0.080953758	,	0.266423524	,],
[	0.289426194	,	0.401843962	,	0.325564911	,	0.084087886	,]],
[[	0.084376117	,	0.333775031	,	0.360193977	,	0.26000262	,],
[	0.250174903	,	0.083127553	,	0.25930656	,	0.293156086	,],
[	0.328510633	,	0.310337086	,	0.074105843	,	0.264049058	,],
[	0.287431144	,	0.399994465	,	0.323661462	,	0.083513287	,]],
[[	0.083401568	,	0.335347783	,	0.361691409	,	0.261625765	,],
[	0.24754896	,	0.082760337	,	0.257323578	,	0.290723924	,],
[	0.328679601	,	0.311060763	,	0.076664985	,	0.264072838	,],
[	0.29189571	,	0.404376471	,	0.328027876	,	0.090623964	,]],
[[	0.082749803	,	0.338052987	,	0.363608106	,	0.259485343	,],
[	0.247776702	,	0.092688092	,	0.262945896	,	0.291389685	,],
[	0.325966426	,	0.312998031	,	0.075205076	,	0.261990092	,],
[	0.287156819	,	0.403560432	,	0.327270535	,	0.083215928	,]],
[[	0.085129416	,	0.342150798	,	0.367363372	,	0.263785365	,],
[	0.243891661	,	0.083324845	,	0.257894408	,	0.287546482	,],
[	0.323975688	,	0.311456776	,	0.07508081	,	0.259824669	,],
[	0.289514867	,	0.405967696	,	0.329685666	,	0.088042355	,]],
[[	0.081470738	,	0.337759092	,	0.363229814	,	0.259301321	,],
[	0.244901529	,	0.087562988	,	0.259653189	,	0.288594614	,],
[	0.327142454	,	0.315553167	,	0.081880749	,	0.263200899	,],
[	0.288468639	,	0.404819451	,	0.328600384	,	0.083694293	,]],
[[	0.085591313	,	0.339516498	,	0.36494996	,	0.260890954	,],
[	0.245844475	,	0.084203982	,	0.259584405	,	0.289487083	,],
[	0.324416737	,	0.311652762	,	0.074200177	,	0.260375303	,],
[	0.287586352	,	0.404205209	,	0.327774954	,	0.084480033	,]],
[[	0.082506249	,	0.33951319	,	0.365057008	,	0.261120661	,],
[	0.243237201	,	0.083945157	,	0.257604965	,	0.287038055	,],
[	0.325020201	,	0.312818592	,	0.077384719	,	0.260804211	,],
[	0.291968589	,	0.407818802	,	0.331760192	,	0.086729063	,]],
[[	0.081932659	,	0.3377048	,	0.363219762	,	0.259119835	,],
[	0.247117832	,	0.091249758	,	0.262107212	,	0.290716227	,],
[	0.326740395	,	0.313681421	,	0.075853577	,	0.26273253	,],
[	0.287184943	,	0.403484914	,	0.32707435	,	0.082865856	,]],
[[	0.086528973	,	0.343578903	,	0.368785592	,	0.265253992	,],
[	0.244336782	,	0.083381082	,	0.258102244	,	0.287920912	,],
[	0.323860523	,	0.311240548	,	0.074530503	,	0.259687623	,],
[	0.288778419	,	0.405203238	,	0.328896582	,	0.086643872	,]],
[[	0.081306859	,	0.338063819	,	0.363312081	,	0.259428133	,],
[	0.244637266	,	0.086657178	,	0.259154653	,	0.28831396	,],
[	0.326764258	,	0.314842635	,	0.080718264	,	0.262620057	,],
[	0.289359372	,	0.405444914	,	0.329295703	,	0.084293016	,]],
[[	0.084278457	,	0.338693795	,	0.364129865	,	0.260185974	,],
[	0.246970869	,	0.08511865	,	0.260514386	,	0.290487046	,],
[	0.324884766	,	0.311995738	,	0.074313862	,	0.260789048	,],
[	0.287143781	,	0.403654412	,	0.327248292	,	0.083670369	,]],
[[	0.08340197	,	0.340482683	,	0.365814659	,	0.261932385	,],
[	0.243551162	,	0.083636645	,	0.257586784	,	0.287215402	,],
[	0.324557193	,	0.312137758	,	0.07631655	,	0.260306294	,],
[	0.291237065	,	0.407601035	,	0.331408903	,	0.090239316	,]]],dtype=np.float32)
								
y_train_x=np.array([[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	]],dtype=np.float32)						
								
y_train_y=np.array([[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	]],dtype=np.float32)						
								
y_test_x=np.array([[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.045454545	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.075757576	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.106060606	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.136363636	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.196969697	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.227272727	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.257575758	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.287878788	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.318181818	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.348484848	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.378787879	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.409090909	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.439393939	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.46969697	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.5	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.53030303	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.560606061	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.590909091	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.621212121	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.651515152	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.681818182	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.712121212	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.742424242	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	],						
[	0.772727273	]],dtype=np.float32)						
								
y_test_y=np.array([[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.083333333	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.138888889	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.194444444	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	],						
[	0.25	]],dtype=np.float32)						


In [4]:
x_test=np.expand_dims(x_test,axis=-1)
x_train=np.expand_dims(x_train,axis=-1)
x_train[:,:,0,0]+=-0.00238
x_train[:,:,1,0]+=0.001496742
x_train[:,:,2,0]+=0.009334
x_train[:,:,3,0]+=-0.00588
x_test[:,:,0,0]+=-0.00238
x_test[:,:,1,0]+=0.001496742
x_test[:,:,2,0]+=0.009334
x_test[:,:,3,0]+=-0.00588
x_train[:,0,0,0]=0
x_train[:,1,1,0]=0
x_train[:,2,2,0]=0
x_train[:,3,3,0]=0
x_test[:,0,0,0]=0
x_test[:,1,1,0]=0
x_test[:,2,2,0]=0
x_test[:,3,3,0]=0
for i in range(len(x_train)):
    for j in range(4):
        x_train[i,j,0:4,:]=x_train[i,j,0:4,:]/max(x_train[i,j,0:4,:])
for i in range(len(x_test)):
    for j in range(4):
        x_test[i,j,0:4,:]=x_test[i,j,0:4,:]/max(x_test[i,j,0:4,:])

In [5]:
len(x_test)

1200

In [6]:
x_test.shape

(1200, 4, 4, 1)

In [7]:
x_train.shape

(1300, 4, 4, 1)

In [8]:
y_train_x.shape

(1300, 1)

In [9]:
y_train_y.shape

(1300, 1)

In [10]:
y_test_x.shape

(1200, 1)

In [11]:
y_test_y.shape

(1200, 1)